In [1]:
#!pip3 install julia
#import julia
#julia.install()

#Temporal fix because this python installation does not have libpython
# from julia.api import Julia
# jl = Julia(compiled_modules=False)
#!pip3 install numpy
# !pip3 install matplotlib

import matplotlib.pyplot as plt
from npodjlEnv import NpodJLEnv
import numpy as np
env = NpodJLEnv()
env.ver()

0.8

In [2]:
def argmaxQ(Q, s, n_actions):
    if s in Q.keys():
        print("Policy: Greedy")
        print("Q[s]: ", Q[s]) #if Q[s]== 0 then the policy is not really greedy
        return(np.argmax(Q[s]))
    else:
        print("Policy: Random")
        action = np.random.randint(0, n_actions)
        if action in [6,7]:
            action = np.random.randint(0, n_actions)
        return(action)

In [3]:
def policy_greedy(Q, s, epsilon, n_actions):
    if np.random.rand() < epsilon:
        return(argmaxQ(Q,s,n_actions))
    else:
        print("Policy: Random")
        return(np.random.randint(0, n_actions))

In [4]:
def SARSA(env, alpha, gamma, epsilon, Q, max_cycles):
    n_actions = len(env.actions())
    r = [0.0] * (max_cycles+1)
    s = [None] * (2*(max_cycles+1))
    a = [None] * (2*max_cycles)
    
    strikes = 0
    t=0
    s[t] = env.encoded_state()
    a[t] = policy_greedy(Q,s[t], epsilon, n_actions)
    obs = env.run(a[t])
    r[t] = obs['reward']
    s[t+1] = env.encoded_state() 
    if not s[t] in Q.keys():
        Q[s[t]] = [0]*n_actions
    try:
        while t < max_cycles:  
            print("State: ", s[t+1])
            a[t+1] = policy_greedy(Q,s[t+1], epsilon, n_actions) 
            print("Action: ", a[t+1])
            obs = env.run(a[t+1])
            r[t+1] = obs['reward']
            if(r[t+1]==0.0):
                r[t+1] = -20000 # Basically a crash
                break
            print("Reward: ", r[t+1])
            print("-----")
            s[t+2] = env.encoded_state()
            if not s[t+1] in Q.keys():
                Q[s[t+1]] = [0]*n_actions
            Q[s[t]][a[t]] = Q[s[t]][a[t]] + alpha * ( r[t] + (gamma * Q[s[t+1]][a[t+1]] ) - Q[s[t]][a[t]] )
            t += 1
        print(r)
        print("Total Reward: ", sum(r))
        return(sum(r))
            
    except KeyboardInterrupt:
        print("Press Ctrl-C to terminate while statement")
        pass
    return(s,a,r,Q)

In [5]:
Q = dict()
total_reward=[]

In [6]:
# total_reward = [-11881.798252915374, -19999.99999999, -12324.07014371438, -8985.467698612041, -19999.99999999, -19999.99999999, -13061.699700363095, -11980.228744939763, -19999.99999999, -11650.60397328222, -19999.99999999, -6038.238914661793, -2116.637837296439, -19999.99999999, -12987.261518559497, -11269.50417070563, -10126.29786725363, -12356.518370336498, -10569.256445136634, -11174.738877945681, -11464.37761768033, -19999.99999999, -7844.418904845646, -11494.254515492705, -10402.71470682464, -7061.236447238953, -9881.41790924608, -9080.44100832374, -11297.233829809735, -19999.99999999, -19999.99999999, -8681.525493415933, -19999.99999999, -8455.283184557635, -19999.99999999, -12911.262024739553, -8447.825401296484, -9085.534918302923, -9841.965799520805, -7378.040497195972, -19999.99999999, -7742.4322901274545, -6811.424447477465, -7239.259921400643, 1.0000803740695119e-08, -13692.256411172786, -11882.260600407431, -10385.635526231074, -12346.05277592601, -19999.99999999, -7580.071964985403, -19999.99999999, -12630.065544581965, -6491.086446288386, -19999.99999999, -9000.614457898486, -19999.99999999, -8853.504330654267, -9199.617650752632, -19999.99999999, -19999.99999999, -7716.9043451201405, -19999.99999999, -6813.5006617583, -19999.99999999, -6060.120121852626, -8044.919953060386, -19999.99999999, -9631.277294010513, -8066.306330507563, -7734.094953711783, -8358.68480548826, -11141.506771231927, -13120.9173352478, -7923.382897758896, -10537.469938015734, -11715.87244223059, -11800.66903984477, -7163.593383747877, -12652.449934581467, -19999.99999999, -10131.790927534024, -5722.846577963775, -7012.367759713788, -11596.22862652589, -9008.24632527325, -8889.150587202153, -19999.99999999, -19999.99999999, -9677.110380965782, -9485.04928135289, -19999.99999999, -19999.99999999, -11629.554076647724, -19999.99999999, -10163.69627545239, -1935.4929290371467, -9497.266961184609, -19999.99999999, -8132.07210808527, -10421.037036586915, -12339.153106598755, -8866.032813777601, -11881.798252915376, -11873.033893622072, -10967.53489346255, -5735.949079441301, -19999.99999999, -19999.99999999, -7238.186832128848, -9229.790588596392, -12875.513002580781, -7944.125520289745, -10549.304293311729, 1.0000803740695119e-08, -12183.258048780674, -11026.094744641185, -7639.947991665314, -10202.793354296944, -5712.118213529569, -8749.377314992267, -8373.053508229128, -10573.036525035657, -9773.589351326913, -7572.226193667225, -8455.36068347689, -11272.389905545373, -8830.72035630459, -8044.927763369245, -1948.0291707627373, -10476.321820231798, -8207.944056908835, -4204.671278734793, -19999.99999999, -9508.68859612947, -6846.79194876542, -8810.769580392387, -5768.800969218992, -10151.03414016289, -9507.131913160114, -9798.337113636682, -10655.338930180531, -10027.334988564755, -9100.526832514774, -4840.908774332658, -7249.947312169631, -11660.788762667247, -8341.498555659544, -9709.63667260334, -8097.322543792194, -9360.14253709627, -13471.251601399314, -11328.899059419859, -10459.894211166356, -9635.107755789617, -9158.352400683298, -7796.924200728054, -8646.659650184809, -19999.99999999, -10325.26924291026, -9193.697960946876, -9324.189514026433, -8802.53212703409, -9326.882811944219, -10365.448007948877, -11343.495876484223, -19999.99999999, -1935.3792252729054, -11552.604621851651, -9716.844388861608, -10066.656380907116, -5811.844814794391, -8628.740390413604, -9435.193869827503, -12531.437182456473, -8977.724359822598, -8465.843005671, -19999.99999999, -4622.869655884213, -9408.769551383657, -5753.440408159646, -6712.009556446308, -19999.99999999, -10702.807682844918, -7406.591877244917, -5125.245653412498, -8875.222827859718, -8562.377085625481, -7048.146126949223, -19999.99999999, -1935.4065154703048, -7134.986183878389, -2016.6172249336887, -19999.99999999, -11272.389905545373, -19999.99999999, -19999.99999999, -9531.682108200404, -8738.856792546954, -8769.590717360388, -8997.554986616718, -12309.393741184265, -1934.6451990898436, -10060.278164208547, -1935.5423977675591, -1965.3671239887628, -7446.07510824228, -19999.99999999, -11198.046734353544, -11893.54965208131, -10446.291799642566, -19999.99999999, -7726.47838623964, -2075.5839858464183, -1935.4837836885863, -7859.739327503759, -19999.99999999, -3710.9441585684117, -19999.99999999, -9531.682108200404, -2070.511014817097, -9661.206094529372, -9688.10138132941, -8615.85102733785, -10224.986063079928, -8378.204748916221, -9457.956323440796, -8143.284484247005, -7770.1803295036, -6834.607569259892, -6611.4070932271525, -5344.176741516326, -6221.073376275403, -8036.600548197265, -12338.797203440183, -5772.551656019954, -19999.99999999, -12474.40585917131, -6866.155909761841, -1935.507928695357, -9333.419085992384, -10133.002657242077, -8702.386578589028, -1935.4703766946384, 1.0000803740695119e-08, -10810.12438125299, -6611.3125621258105, -8621.987734597658, -9229.742077600153, -4206.426498689662, -8173.8476570204875, -3628.912137666644, -4977.147351146896, -5118.099129197351, -8057.722927129422, -8143.284484247005, -5647.36833656481, -19999.99999999, -7385.016058353598, 1.0000803740695119e-08, -7456.067015804738, -10057.177486576542, -9869.64689747812, -8631.249525146794, -7378.040497195972, -9830.976164392068, -1935.5314631038323, -4540.039864640523, -8990.013990699084, -6531.704323810038, -12539.714067752171, -6171.750605007664, -6928.998468666632, -6023.711899147616, -8347.247336691416, -9912.623394263135, -19999.99999999, -9091.800512570508, -9458.387945031089, -19999.99999999, -9441.437043118001, -19999.99999999, -5475.698836999031, -6388.068438893527, -8657.751090497964, -5615.1582045308605, -1935.4851565823656, -19999.99999999, -1934.6612959636986, -1934.6696078555287, -7885.744957580153, -5318.172862632872, -8902.419425570166, -7075.597786352964, -8095.76516415873, -6719.984516499455, -19999.99999999, -7242.900552805421, -9393.613104677455, -4997.421680027929, -9222.83734999509, -1934.6301028232501, -7969.51776406774, -8391.655378067306, -12149.855188910184, -8060.1785342751145, -4377.506893914157, -8343.628395491149, -4267.551879666556, -6755.681011031968, -10512.160321788559, -7356.501905088403, -8853.504330654267, -5472.557561217555, -1935.475539194017, -11583.087738546647, -11236.638115409789, -7844.418904845646, -9761.558969444424, -5503.711518129292, -6393.123158219525, -9256.194332534242, -5367.004788926411, -7169.956734930366, -5345.035531006912, -6114.943570828583, -7999.3827182078985, -6000.976045021315, -3873.5887688880575, -7651.137595288976, -6181.210739291172, -4566.601389716535, -9205.072573265925, -9542.304976832858, -7406.591877244917, -5249.049348900964, -3653.8154364961147, -5274.730718936212, -9785.377870842658, -8798.121302484895, -2029.299080611467, -1934.6362131104327, -9575.347316623483, -2037.705714606488, -7462.216893703455, -7923.382897758896, -7012.96685195289, -12214.416349267756, -5274.730718936212, -8064.8647194858, -11924.170873652205, -9514.58603432129, -9238.939492816842, -9507.131913160114, -7018.385525648298, -9833.635188071165, -1935.4027703352424, -6759.063350740884, -4836.129920671619, -11909.671068922462, -4311.21944138804, -9796.717918316706, -1935.5226854069476, -6544.47337360024, -8107.530135522111, -7933.44208850152, -5580.757301626401, -1966.7459565737927, -7282.175488384484, -5458.844393884635, -9691.41553342457, -19999.99999999, -5535.3725385550715, -6416.437495259781, -4491.708312553726, -11697.977518032714, -1935.4973496366224, -4766.407362466671, -3873.5887688880575, -1934.706380431185, -6834.607569259892, -7120.025257108575, -4364.346267590235, -5290.110006171666, -4977.147351146896, -1935.4508155916249, -7350.208682900169, -1935.3281886041561, -2028.5734617668525, -7205.933344570422, -3924.0841038275375, -1935.4770058415559, -5264.407205965695, -1935.4486957540405, -1935.4050548542857, -3833.2784889603317, -8729.328236033429, -5761.493952047206, -9816.035278928279, -12493.979911030463, -5739.839812146455, -7476.023028024581, -1934.6106838809158, -6104.0550094295395, -6838.545823747202, -1935.5829172218455, -3849.591076685017, -5600.334886609038, -5573.70511799634, -6943.585957395522, -6966.921847932284, -4521.726986114137, -4588.06118622005, -4630.48005297586, -8450.182046961661, -10373.154907051518, -10212.894082349867, -8023.884124971473, -1934.6488261687014, -8659.583549281475, -6862.21612596623, -5703.320906463429, -4708.946096570214, -6469.59117338591, -1934.6513010878737, -7120.321887058034, -6418.37991298226, -7128.286661629947, -8437.76121510371, -6834.607569259892, -4540.039864640523, -7999.361695534239, -7404.979150849009, -6114.943570828583, -6531.585846081111, -1935.4828269798754, -1935.4517214983289, -5350.545161510619, -6171.750605007664, -7410.9348416947305, -5410.552620315018, -1935.4090241220217, -1935.475539194017, -9744.777190817815, -8264.349718903943, -8260.470802319574, -10124.217600998272, -7410.9348416947305, -7708.137628366795, -5137.74152230783, -1935.369702158123, -5621.409862486094, -1940.6838464129983, -1935.551894848021, -1940.9563177525392, -7835.103435378187, -5839.062493145225, -7363.115187071566, -4748.666543364853, -2122.395243542015, -8040.755095794488, -9222.83734999509, -8854.516997959394, -5350.45738839723, -6302.88218750367, -3668.880178022149, -4928.0000846300245, -10598.915381257419, -10216.878506621732, -6424.61824778506, -4986.476073161874, -5142.195565105091, -1935.6229241677497, -9524.452720414858, -3857.4644353979875, -7601.962826032033, -6724.131860448253, -1935.4548519412158, -8468.184589137723, -6797.193938283547, -6531.585846081111, -6792.879776302641, -8649.490355857348, -19999.99999999, -10445.704684120727, -9211.709001231306, -9719.025416871205, -6266.250730362087, -4864.827860931329, -8222.126598898658, -5344.727119765591, -3862.442151050095, -5578.908144318034, -7757.441493641932, -4824.101184352488, -7120.025257108575, -5681.359432658823, -1934.6425884664638, -7025.728615615618, -8466.58725221984, -1934.6699920547155, -1934.6106612845394, -19999.99999999, -7750.952156552681, -6575.944517704549, -1934.6663645444805, -11921.306328937811, -5125.245653412498, -8254.250155129934, -7120.321887058034, -6429.905407184319, -2099.1210405754837, -6594.780040207994, -1935.5414085337227, -1934.639970415237, -3876.6126024363475, -6565.4713780337415, -19999.99999999, -6714.101492354764, -1935.4027703352435, -6794.479089248938, -3966.45276449454, -3535.947477586178, -5224.377253326744, -1935.4027703352426, -6424.61824778506, -6120.791741238111, -8738.856792546954, -7659.503065193135, -1934.6956138389498, -1935.529602825965, -5164.167824905199, -5760.742239191024, -5667.945536238178, -5080.349612146136, -1934.6443433800796, -8653.18426595573, -5916.552780844154, -6168.941680842594, -4010.5123272479473, -10347.825006293106, -9609.778422421565, -11163.62689572469, -1935.5109702709087, -3974.463692281249, -6741.841947386355, -1935.3968947159387, -5116.650052393329, -7338.052141983604, -1935.4678803275644, -8017.411420047686, -8384.459123393754, -6859.110326834366, -5913.447381107957, -6086.686017187631, -4491.708312553726, -8657.751090497964, -1934.6374603005897, -4063.8399105631693, -4479.16702659322, -5317.379204457973, -6719.984516499455, -7410.9348416947305, -6990.515043994574, -1934.690104883026, -4620.839543014561, -9313.440579990749, -6068.628562387862, -10604.909687634456, -5323.865829128895, -4233.820554098794, -1934.6867175648035, -1934.6742412827311, -10454.385094029425, -2750.7208021453002, -8866.032813777601, -6631.558451739408, -1941.276547922289, -1935.3778042317056, -7527.256421593585, -4983.350604655804, -6794.479089248934, -1935.4785709783725, -4721.171855219641, -1934.701332386169, -6583.740924634166, -4358.660183503527, -4241.565639605523, -6206.992503179362, -10394.68634421009, -5952.863716176385, -1934.6033894562227, -1934.7039545047514, -8653.18426595573, -1934.6336745402607, -7232.516733975, -1934.5799845475108, -1935.4732338234232, -1935.5061279726708, -1934.6195969116789, -1934.655901889132, -2101.5471384923026, -2745.4476902933416, -4330.377016433254, -5874.541433040151, -7012.367759713788, -3833.2784889603317, -1935.4629508355445, -8818.028110118481, -4620.035894673158, -4456.557836394494, -6210.659575346401, -5514.965274955872, -4054.445020958452, -4171.723714745573, -3487.6183208624225, -1934.6533502296788, -1935.5329033450635, -5301.392013904402, -4315.832135572868, -4255.741458465673, -5009.991848192823, -6107.40642666812, -6741.841947386355, -3918.150691047095, -2275.876551162286, -9507.798902070503, -5716.7272043792245, -1934.6186389272957, -1935.483662525385, -1934.6398564845458, -1935.4698201348683, -4553.17415666383, -2047.7328352901338, -3790.369088575665, -5890.749539377109, -2750.7208021453002, -1935.4090241220215, -7969.51776406774, -7507.493167297892, -7395.496064217996, -1934.6368858228984, -1969.5640373360638, -1934.6513636214245, -19999.99999999, -7845.17616084312, -5312.504347659753, -2036.55741185447, -6681.772945158907, -5901.177646827578, -1934.645296830992, -8870.15824202703, -4238.0871533659865, -1935.4387180936803, -7378.040497195972, -1935.3835417899334, -1934.6252190136054, -1934.6249418740574, -1934.6217949783822, -5264.852700363655, -3362.1948445724574, -4860.361486418249, -4959.9198283030355, -1935.4261749059276, -1935.4633335347025, -5080.349612146136, -6616.771674077738, -6837.022645420149, -3465.090306273592, -2043.009709769449, -5598.097262272724, -4444.36274533262, -1935.4397709230634, -2070.511014817097, -1934.6608667692567, -1971.2537966512864, -1935.465956167658, -8977.7243598226, -5475.408052603049, -2072.5805694256896, -2030.2835219044584, -7176.265191483539, -5008.08442858361, -1935.488087427484, -4412.0377901246375, -7336.817428514577, -1935.3610508388715, -6922.602540087389, -2745.4476902933416, -1934.6525514546654, -1935.4951093995592, -1934.6870673073877, -1934.6437547547985, -1935.5165776036863, -2064.888573587541, -1935.4508155916246, -1954.1807674973645, -7639.947991665314, -1935.4963580373123, -1934.6706771136976, -1934.65631120054, -6228.360656027409, -1935.4322304843004, -6290.42118254796, -2953.2081463935683, -5703.320906463429, -7974.0210723300215, -8653.957636585614, -8749.65199812363, -1935.418130131378, -2488.0428049479806, -5345.035531006912, -6616.328847783184, -5258.06620674468, -6862.21612596623, -7035.67944684594, -1935.47770979156, -7771.818818645403, -1934.655901889132, -1934.668391928265, -1935.4729722558247, -1934.6858232431882, -3709.0966667888597, -1935.5414085337227, -1935.5823882789227, -9229.742077600149, -1935.4845378494438, -1934.5870696849972, -6785.785884456451, -5660.278884457572, -7979.596371378899, -3365.3894144547858, -2331.7810605806776, -4684.7833168657, -1935.3417101354064, -1935.4821018956038, -6278.95950953945, -6503.959208771692, -1934.623428402651, -1935.3666517244424, -3836.3568040957366, -7039.128123701407, -5756.664467574318, -1935.46371449676, -2062.6445743659583, -1935.3870972846128, -1934.6045797949212, -1934.5710228289493, -7340.404095914084, -1935.4277876919036, -1934.6067460380368, -1934.6579200636738, -3232.9259958679886, -7535.879348264651, -1935.2842684378982, -1935.4777097915603, -2704.5503211203186, -1934.5861724777963, -8137.890226737027, -1935.5188300243888, -2488.0428049479806, -1934.6182672151222, -7173.573316607784, -6859.110326834366, -1935.4655711091375, -10049.744920575939, -4647.951586677323, -1934.6125640077355, -1934.626412618289, -1971.2537966512864, -1935.3872760746053, -3745.8754457505966, -3644.96257330097, -1934.599498910979, -1934.6222808787236, -2488.0428049479806, -1939.4415875573652, -1934.5980319168746, -3646.663377681559, -1935.4260699315598, -3971.202784550922, -7140.631415594772, -1934.5980319168746, -1934.6065182252212, -1935.443892626217, -4353.584926411928, -3910.573786075027, -1934.5946200892065, -3487.6183208624234, -2032.9199426030061, -1954.1807674973645, -2750.7208021453002, -1935.5420743698176, -4244.246262810711, -1934.653318233919, -4983.350604655804, -1934.6241373703006, -1934.6464289054586, -1934.6386313542612, -4141.72678812493, -1934.5943014161278, -1934.6138062636144, -1934.5987465892413, -1935.4224722592019, -5227.116828019227, -1935.400177855515, -9388.045957612014, -1934.650623718007, -4001.40032916837, -1935.4367567830686, -1935.4948366489343, -1935.3802182879526, -6615.154208886361, -1935.4489519585345, -1935.5811774507677, -4456.557836394493, -1934.6106838809158, -1934.5877623805864, -1935.5114806619863, -2080.3719956668306, -5184.124296687729, -1934.597482993112, -9229.742077600153, -2488.0428049479806, -1934.6241373703006, -1934.5892471883162, -1934.6511795322576, -1934.5799845475108, -2488.0428049479806, -4861.100728253251, -1935.4655711091375, -1935.3564001202803, -1934.6033599023453, -4863.687652685031, -1935.4678247388279, -1935.505114855953, -1934.5935556397658, -1934.5976841172062, -3531.48111133611, -7545.90517896145, -4738.4149440914125, -1934.5821488768963, -9497.26696118461, -1934.5889474034345, -1934.5710228289493, -8738.856792546956, -1935.4716116669424, -3918.150691047097, -8738.856792546956, -7727.135129016009, -4607.880110031254, -2494.440708311634, -1934.6073147308905, -1934.6612789706594, -1935.3946911139074, -1935.4996939397774, -1935.4508155916246, -1935.4489519585345, -4540.039864640523, -1935.4974715712062, -5587.225321957599, -1934.6117905094673, -1934.6181331836697, -6015.219390530136, -1934.6012351909808, -1934.58930650422, -1935.46278254076, -1934.6088340156687, -1934.5908268131443, -1934.601040152917, -3921.4964175386613, -1934.6052656839945, -3698.4788668014753, -1935.508636851517, -1935.4278340506446, -1934.6241462439339, -4661.927906619787, -2335.2240363206897, -3237.726646374531, -1934.5955619714682, -1934.6793045778065, -2488.0428049479806, -4772.126882712968, -5768.800969218992, -1934.6590729477484, -5085.26947523331, -1934.6242489643191, -1947.032788874807, -1935.4551968398926, -1935.4359078313505, -1935.334422870427, -4222.046795149332, -2742.511988243083, -1934.5917239308833, -1978.4841432015578, -1934.5872955123323, -1934.5722876702464, -1934.5891640777359, -1935.4835785716416, -1935.4098942657517, -1934.584163842987, -1935.4285291023034, -1935.4098942657513, -1934.5980319168746, -2488.0428049479806, -1934.5870342549924, -1935.4287572556502, -1934.6045451445666, -8096.875295382375, -1934.5833657477294, -1935.4459091606354, -1934.6470271758908, -1935.306340635319, -1934.58930650422, -1935.394189329489, -1935.3942390508996, -1934.6243290347438, -1934.6145082463127, -1935.2907589758101, -1934.7013957603315]
# Q={'0.001000_125.000000_2.000000_625.000000_0.000000_0.100000': [-6679.841537566954, -9522.256092921307, -19625.743640339264, -17215.607199667604, -9569.129004414302, -9527.978495274323, -11093.944340301225, -11880.691215385259], '0.001100_125.000000_2.000000_625.000000_0.000000_0.100000': [-164.48892417992965, -3844.9565028125703, 923.8346223493692, -553.4820248300116, -9310.894723928359, 1106.403933652714, -763.9874463229676, 1678.9308495799135], '0.001100_125.000000_2.000000_568.181818_0.000000_0.100000': [2594.8745211670102, -377.1247841117656, -7.560820898324855, -1834.7290305288996, 4411.14503931666, 593.1500457257073, 1836.6084894194992, -8348.66906706439], '0.001100_125.000000_2.000000_516.528926_0.000000_0.100000': [1453.4033326957433, 1863.8195682897042, -6.843277553012195, 207.47662780010785, 340.88765002666753, 255.4737545294263, -507.9562379266222, -908.4676535293602], '0.001210_125.000000_2.000000_516.528926_0.000000_0.100000': [359.995283371678, 443.28835227713205, -7.534420104456331, 0, 0, 1709.9742749062589, -457.6985663293908, 0], '0.001210_125.000000_1.818182_516.528926_0.000000_0.100000': [321.4925981551152, 820.6607544582029, 0, -1206.2305233815382, 320.66342462513546, 329.478102549359, 265.42141000108364, -224.19731460717176], '0.001210_137.500000_1.818182_516.528926_0.000000_0.100000': [0, 0, -4.585180320260024, -825.6386462629446, 0, 1111.4496818876194, 0, -271.8332319651373], '0.001210_137.500000_1.818182_568.181818_0.000000_0.100000': [389.11653519164423, 436.1160929768185, -4.64316049858935, -884.7100902293802, 0, 0, -427.40677926303545, -437.1322346679233], '0.001210_151.250000_1.818182_568.181818_0.000000_0.100000': [0, 469.1771869729777, -5.196466651740411, 0, 388.2824627856516, 0, 0, -448.38691876501724], '0.001210_151.250000_1.818182_625.000000_0.000000_0.100000': [376.780691017302, 0, 0, 0, 0, 314.3872585295454, 0, 0], '0.001210_151.250000_1.652893_625.000000_0.000000_0.100000': [0, 449.08050830347275, 0, 0, 0, 0, 0, -231.8269371362666], '0.001210_151.250000_1.652893_687.500000_0.000000_0.100000': [822.2501370958402, 0, 0, 0, 0, 0, -346.396859019721, 0], '0.001210_137.500000_1.652893_687.500000_0.000000_0.100000': [60.60801812137606, 1316.776400673584, 0, 0, 0, 45.30035727450268, -317.0792890528805, 0], '0.001210_125.000000_1.652893_687.500000_0.000000_0.100000': [316.59209877832967, 1188.2309024629137, 0, 0, 0, 0, -317.47057111197, 0], '0.001210_113.636364_1.652893_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -494.932738472703], '0.001210_113.636364_1.652893_756.250000_0.000000_0.100000': [0, 0, 0, 0, 4.178661094275049, 0, 0, 0], '0.001100_113.636364_1.652893_756.250000_0.000000_0.100000': [1827.4091621831421, 0, 0, 0, 0, -243.5236727351371, 0, 0], '0.001100_103.305785_1.652893_756.250000_0.000000_0.100000': [0, 2088.6398681270175, 0, -274.74783450586403, 1666.3696314572487, 332.53601672821344, 0, -526.7307087042147], '0.001000_103.305785_1.652893_756.250000_0.000000_0.100000': [0, 1455.3828687359733, -3.458967633710563, -274.4394872464025, 0, 0, 0, 0], '0.001000_103.305785_1.818182_756.250000_0.000000_0.100000': [246.66840329465685, 2503.9647875892874, 218.19254521513992, 68.65975651858025, 250.5277848401797, 0, -271.76155290633307, -556.0862215746229], '0.001000_113.636364_1.818182_756.250000_0.000000_0.100000': [1144.9627987665192, 0, -4.109390924712716, -1464.649561059737, 7.778273524329324, 259.231071340692, 0, 0], '0.000909_113.636364_1.818182_756.250000_0.000000_0.100000': [0, 0, 2.696472145027292, 0, 3.7461942697075754, 0, 0, 0], '0.000826_113.636364_1.818182_756.250000_0.000000_0.100000': [0, 733.0627433266229, 0, 0, -20.90820365918057, 0, -806.3256255839028, -546.0244058252662], '0.000751_113.636364_1.818182_756.250000_0.000000_0.100000': [270.901759844496, 523.6820914023731, -245.78248090755187, -922.1308139288236, 2.774752985590749, 0, 0, 0], '0.000751_113.636364_1.818182_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -523.6820914023731], '0.000751_103.305785_1.818182_756.250000_0.000000_0.100000': [0, 0, 0, 0, 2.5363262114558895, 275.753078542556, 0, 0], '0.000683_103.305785_1.818182_756.250000_0.000000_0.100000': [0, 0, 0, -867.3700507203188, 0, 0, 0, -554.761550596797], '0.000683_103.305785_2.000000_756.250000_0.000000_0.100000': [0, 0, -4.204832471367263, 0, 0, 0, -323.99603238137246, 0], '0.000751_103.305785_2.000000_756.250000_0.000000_0.100000': [0, 2239.6661890611713, -4.624381300340429, 0, 0, 0, -323.99401679308943, 0], '0.000826_103.305785_2.000000_756.250000_0.000000_0.100000': [0, 0, -5.087868143169908, 0, 0, 0, 0, 0], '0.001000_125.000000_2.000000_687.500000_0.000000_0.100000': [1852.8434503583044, -8376.917847006174, -1058.5419847680046, 910.8547608949507, -6307.4341577287905, 3231.1612191761806, 365.06080129602026, 945.7968949227785], '0.001000_125.000000_2.200000_687.500000_0.000000_0.100000': [248.71611230872304, 27.86199127839984, 1126.4052643018904, -382.7247216709404, 0, 3483.6442292673255, -103.21377370601294, -494.8697027322223], '0.001000_137.500000_2.200000_687.500000_0.000000_0.100000': [2545.040251827975, 2321.1196821794097, -3.316628146307085, -379.6381741036521, 1797.0299850805404, 1329.2590047612455, 0, -462.46217563965547], '0.001000_137.500000_2.420000_687.500000_0.000000_0.100000': [244.10885006280097, 0, 0, 0, 2.2241929860967504, 0, 0, 0], '0.000909_137.500000_2.420000_687.500000_0.000000_0.100000': [0, 0, -2.224192986095113, 0, 0, 0, 0, 0], '0.001000_113.636364_2.000000_625.000000_0.000000_0.100000': [4890.428179195377, -302.55875347326173, -1058.0155451641067, 1522.3430995574126, 556.9766892022731, 1987.0191281701227, -9626.92353413958, -9460.662868480735], '0.001000_113.636364_2.200000_625.000000_0.000000_0.100000': [1201.8908714637446, 4471.02209088595, -6.0153845441076115, -612.6099512681003, -642.7236455918551, 1394.1935252438966, -2082.8133368774397, -677.212304178855], '0.001000_125.000000_2.200000_625.000000_0.000000_0.100000': [3725.141742805661, 732.5694766086126, 1875.730482052389, 329.4096811581864, 81.47886165979355, -6907.092365206218, 1114.9692932454927, 1522.9019761104032], '0.000909_125.000000_2.200000_625.000000_0.000000_0.100000': [366.74481273642414, 66.96239737881399, -7208.585895035744, -332.22542824163565, 3062.2189182968796, 364.72760948913447, 501.8637129696408, -32.45504571855963], '0.000909_125.000000_2.420000_625.000000_0.000000_0.100000': [0, 0, -2.6637082353156076, -457.1998590026895, 798.2185487901334, 427.897893721278, 0, -486.5499051130402], '0.000826_125.000000_2.420000_625.000000_0.000000_0.100000': [945.4863685664275, 0, -2.4497842001041135, -457.50336120792747, 0, 0, -256.43978994516914, -486.49374531791494], '0.000826_125.000000_2.420000_687.500000_0.000000_0.100000': [0, 486.49374531791494, 0, 0, 2.4960379734587876, 0, 0, 0], '0.000826_137.500000_2.420000_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -226.10942337243452, 0], '0.000826_151.250000_2.420000_625.000000_0.000000_0.100000': [0, 458.4486733726648, 0, 0, 1.619712042356514, 0, 0, 0], '0.000751_151.250000_2.420000_625.000000_0.000000_0.100000': [0, 0, -1.619712042356514, 0, 0, 238.16304228971603, 0, 0], '0.000826_151.250000_2.420000_568.181818_0.000000_0.100000': [245.19286812175307, 0, 0, -402.9446614389861, 0, 0, 0, 0], '0.000826_137.500000_2.420000_568.181818_0.000000_0.100000': [0, 0, 0, 0, 1.8529148041760708, 379.3511467597802, 0, 0], '0.000826_137.500000_2.200000_568.181818_0.000000_0.100000': [0, 0, 0, -417.2862614357582, 4.371802368262154, 0, 0, 0], '0.000751_137.500000_2.200000_568.181818_0.000000_0.100000': [399.96006007805516, 509.62630752172976, 0, -602.4364978768459, 3.574958670739761, 0, 0, -484.2738507515416], '0.000683_137.500000_2.200000_568.181818_0.000000_0.100000': [400.004259455375, 509.49310923539906, 0, 0, 0, 1206.9567597526495, -364.954955624962, -484.61922921577974], '0.000683_137.500000_2.200000_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -509.49310923539906], '0.000683_137.500000_2.000000_568.181818_0.000000_0.100000': [0, 0, 0, -1206.9567597526495, 0, 0, 0, 0], '0.000683_125.000000_2.200000_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -400.004259455375, 0], '0.000683_151.250000_2.200000_568.181818_0.000000_0.100000': [364.954955624962, 0, 0, 0, 3.2131197308011905, 0, 0, 0], '0.000683_137.500000_2.200000_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -469.1124292131794], '0.000683_137.500000_2.200000_687.500000_0.000000_0.100000': [327.8945908599417, 0, 0, 0, 0, 0, 0, 0], '0.000683_125.000000_2.200000_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -547.0561500844292], '0.000683_125.000000_2.200000_756.250000_0.000000_0.100000': [0, 0, -2.5063620694607383, 0, 0, 1114.7384738339658, 0, 0], '0.000683_125.000000_2.000000_756.250000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001000_125.000000_1.818182_625.000000_0.000000_0.100000': [2168.124079128231, 4027.055945215345, -361.77732092902556, -11144.21840094361, 379.9267190803889, 317.9999321463447, 916.5099072614555, -1187.8869690710603], '0.001000_125.000000_1.652893_625.000000_0.000000_0.100000': [178.78482249773538, 325.395819438995, -217.1893459395293, 325.2278128076652, 543.5974004448288, 1653.6188368804858, -101.31635122124646, -7095.526733936418], '0.001000_113.636364_1.652893_625.000000_0.000000_0.100000': [-1132.831806841377, 1616.3043442919704, -733.9965172401387, -873.6263042449131, -1326.4657432008967, -702.2839664855184, -1452.7686029510073, -950.254588883019], '0.001000_113.636364_1.818182_625.000000_0.000000_0.100000': [532.3852557850082, 700.1973015648996, 390.0095106096196, 418.6611469968693, 406.8447795828058, 3121.0468798328106, -76.96191094171081, -738.0014223324065], '0.001000_103.305785_1.652893_625.000000_0.000000_0.100000': [303.11971955376606, 505.68494398997143, -2.4047815387378795, -617.6646388780648, 0, 1136.5373271427177, -273.19580562428655, 432.70535191778436], '0.001000_103.305785_1.818182_625.000000_0.000000_0.100000': [223.3901066372175, 4682.6367448903, 841.3833096226405, 69.22380621206432, 9.368246064695413, 943.3652852772944, 2250.5268390937626, -331.3502765262831], '0.001000_103.305785_1.652893_687.500000_0.000000_0.100000': [-1022.9004991595334, -720.2895907032122, -1031.3216985943839, -1840.363943526067, -1058.7531027934313, -616.9883322630071, -1472.3379751766065, -198.38260195234102], '0.001000_113.636364_1.652893_687.500000_0.000000_0.100000': [-25.364386957977842, 772.5624165625902, 257.6553624574937, 1761.848095735927, 32.74160993935487, 0, 573.4761501789208, -493.9903442012641], '0.001000_103.305785_1.818182_687.500000_0.000000_0.100000': [274.08237702275915, 2781.2919060054382, 0, 553.7037900167879, 750.6177588498971, 297.2722603546408, -252.91621725201207, -1318.4457616903321], '0.001100_103.305785_1.652893_687.500000_0.000000_0.100000': [-178.268663508279, -343.9142045317831, -3.900383862182753, -1134.8725255626882, 49.977443791652284, -1047.5719580274604, -1472.149538826177, -1679.2835693078912], '0.001210_103.305785_1.652893_687.500000_0.000000_0.100000': [-1468.519208431284, -715.5063912791937, -258.5081370718059, -874.823214761158, 3.900383862182753, 0, 0, 0], '0.001210_93.914350_1.652893_687.500000_0.000000_0.100000': [249.2675573452527, 0, 0, 0, 0, 0, 1021.1358729911645, -552.9063545525848], '0.001210_103.305785_1.652893_625.000000_0.000000_0.100000': [0, 538.0404401787687, 0, 0, 401.5511180642384, 16.6608923136082, 0, 0], '0.001210_103.305785_1.502630_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -325.94550211316636, 0], '0.001210_113.636364_1.502630_625.000000_0.000000_0.100000': [0, 0, 0, 641.2673951845914, 1406.0159998571169, 966.4840101888519, -315.93813991631686, 0], '0.001210_113.636364_1.366027_625.000000_0.000000_0.100000': [448.85713669805426, 0, 0, 47.87307301144426, 0, 0, -482.9966414710495, 0], '0.001210_125.000000_1.502630_625.000000_0.000000_0.100000': [315.93813991631686, 0, -2.071569899363567, 0, 0, 755.4721114779555, 0, -392.0969890295102], '0.001331_125.000000_1.502630_625.000000_0.000000_0.100000': [314.8864601195452, 0, 0, 0, 0, 0, -304.9458168128893, 0], '0.001331_137.500000_1.502630_625.000000_0.000000_0.100000': [0, 0, -2.014829071316126, 44.95623878892766, 0, 0, 0, 0], '0.001464_137.500000_1.502630_625.000000_0.000000_0.100000': [0, 0, 0, 42.745334199960965, 0, 0, 0, 0], '0.001464_137.500000_1.652893_625.000000_0.000000_0.100000': [0, 0, -4.662399495186583, 0, 0, 0, 0, 0], '0.001611_137.500000_1.652893_625.000000_0.000000_0.100000': [0, 474.10530745501654, 0, 0, 0, 0, 0, 0], '0.001611_137.500000_1.652893_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, -98.63619261649511, 0, 0], '0.001611_137.500000_1.502630_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000909_125.000000_2.000000_625.000000_0.000000_0.100000': [770.2940093159514, 265.1151691256423, -8671.624798941164, -10.973522330274136, 223.8421050406605, 3553.557332330998, 610.3131366903403, -252.430765760172], '0.000909_125.000000_2.000000_687.500000_0.000000_0.100000': [0, 1019.4717106313838, -459.523092302593, -1401.4750182263108, 5.688574731899122, 2156.941520535056, -381.6630243878377, 809.966249488331], '0.000909_137.500000_2.000000_687.500000_0.000000_0.100000': [-340.1820671689651, 578.6982325608898, 0, -1116.2295199299451, 5.619239874085779, 1590.5073268849324, -444.7875709221575, 317.772593482347], '0.000909_137.500000_1.818182_687.500000_0.000000_0.100000': [-892.4459879749942, 167.28561594668315, -4.030518846795348, -1445.2989412277898, -821.2045010803263, 1171.8294561505354, -118.49975829115311, -460.80174951335414], '0.000909_151.250000_1.818182_687.500000_0.000000_0.100000': [36.16593652050421, 0, -4.737804443362984, 0, 0, 625.5252523959125, -426.1235679343286, -870.6739667845745], '0.001000_151.250000_1.818182_687.500000_0.000000_0.100000': [0, 467.8815183701936, 0, -1018.1847355245626, 0, 0, -387.61576496786677, -485.707868151635], '0.001000_151.250000_1.818182_756.250000_0.000000_0.100000': [0, 0, 446.92001114445964, 0, 0, 306.9276687479449, -368.7567931467553, 0], '0.001000_166.375000_1.818182_756.250000_0.000000_0.100000': [0, 466.84889633052353, 0, 0, 0, 0, 0, -480.8793663645005], '0.001000_166.375000_1.818182_687.500000_0.000000_0.100000': [0, 450.6601392433566, 0, -1051.5252173829106, 4.991577973687308, 0, 0, 0], '0.001000_166.375000_2.000000_687.500000_0.000000_0.100000': [420.95624682621474, 471.61662748560576, 0, -642.9165231599279, 5.525150514981033, 0, -431.7468548169436, 0], '0.001000_166.375000_2.000000_625.000000_0.000000_0.100000': [0, 100.19403611387605, 0, -166.37590743027368, 0, 0, 0, -518.7782902341663], '0.001000_166.375000_2.000000_568.181818_0.000000_0.100000': [0, 500.73414637478277, 0, -578.8639139817021, 0, 0, -506.2363514251935, 0], '0.001000_183.012500_2.000000_568.181818_0.000000_0.100000': [460.2148649319941, 0, 0, -930.821008410332, 0, 0, -419.5185584349172, 0], '0.001000_183.012500_2.200000_568.181818_0.000000_0.100000': [0, 0, 0, 0, 4.039684960832855, 0, -323.0770177216251, -431.54690235488414], '0.001000_183.012500_2.200000_625.000000_0.000000_0.100000': [0, 0, 0, 0, 2.6074294851339803, 0, 0, 0], '0.000909_183.012500_2.200000_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001100_113.636364_2.000000_625.000000_0.000000_0.100000': [563.3725538903284, 716.5988188581123, -530.0037319421626, -884.1266701992114, 4443.011001843979, 1001.9601879623721, 841.4232081068749, -972.7201860121183], '0.001100_113.636364_2.200000_625.000000_0.000000_0.100000': [349.197672017084, 533.9769646434514, 0, -640.9298947795519, 5.419265355051903, 3438.845487107059, -73.69415159401689, 2073.726162668567], '0.001100_125.000000_2.200000_625.000000_0.000000_0.100000': [3638.3234956157467, 1241.459888712649, 452.6413237700331, -496.364375201455, 80.70717602026349, -7011.129479512257, -41.97228957949968, -371.15729692857576], '0.001210_125.000000_2.200000_625.000000_0.000000_0.100000': [-564.885291329042, 912.6028996073177, -7.182025421091967, 0, -31.292299178836167, 884.160181679798, 0, -56.335335649817054], '0.001331_125.000000_2.200000_625.000000_0.000000_0.100000': [366.47692355164236, 0, 0, -525.2537588078123, 6.470293172154925, 0, 0, -509.62039144625436], '0.001331_125.000000_2.200000_687.500000_0.000000_0.100000': [327.73987685153605, 0, -5.526782469416685, -455.63121228804374, 0, 0, -320.9734816046017, 0], '0.001331_113.636364_2.200000_687.500000_0.000000_0.100000': [0, 54.835743814636025, -6.6267382967464075, 0, 500.6957977423278, 0, 0, 0], '0.001464_113.636364_2.200000_687.500000_0.000000_0.100000': [0, 0, -7.281143009672997, 0, 0, 0, 0, 0], '0.001611_113.636364_2.200000_687.500000_0.000000_0.100000': [0, 0, -8.804019153446243, 0, 0, 0, 0, 0], '0.001772_113.636364_2.200000_687.500000_0.000000_0.100000': [0, 546.3626483909962, 0, 0, 0.8003653775860224, 0, 0, 0], '0.001772_113.636364_2.200000_625.000000_0.000000_0.100000': [0, 0, 0, -561.7190415928701, 0, 0, 0, 0], '0.001772_113.636364_2.420000_625.000000_0.000000_0.100000': [261.23007036433194, 0, 0, 0, 0, 0, 0, 0], '0.001772_103.305785_2.420000_625.000000_0.000000_0.100000': [0, 0, 0, -578.0493675389185, 0, 0, 0, 0], '0.001772_103.305785_2.662000_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -539.0213626561874], '0.001772_103.305785_2.662000_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001000_125.000000_2.000000_568.181818_0.000000_0.100000': [-266.7642276609581, 31.05070823033111, -1029.316602868747, -949.6296598075915, 4247.572428244225, -612.0693168187067, 255.17419271982362, -6772.759695862576], '0.001000_137.500000_2.000000_568.181818_0.000000_0.100000': [-8169.404315947609, 432.87363422986346, 759.042747133482, -287.27898721700876, 495.3739465249678, 3244.1264453695385, -421.97515541264806, -49.25454749337155], '0.001000_151.250000_2.000000_568.181818_0.000000_0.100000': [131.16302202856502, 470.89624172158346, 849.2833752305734, 0, 5.704265620244951, 944.9672675192901, -191.83738050401257, 1194.527899383866], '0.000909_151.250000_2.000000_568.181818_0.000000_0.100000': [0, 1329.3228522072604, 0, 0, 0, 1037.6199835250375, 0, -898.055259014677], '0.000909_151.250000_1.818182_568.181818_0.000000_0.100000': [533.7829536046207, 0, -858.0402435501161, -946.7680143983264, 0, 0, -419.75412638461825, -492.3587984761127], '0.000909_137.500000_1.818182_568.181818_0.000000_0.100000': [1802.2419202440153, 83.41230695055123, 256.8618920892211, 674.3567932808074, 27.199930483172196, 2331.5199454881827, -74.68977046824817, 0], '0.001000_137.500000_1.818182_568.181818_0.000000_0.100000': [1246.778717183217, 2355.150600329794, -14.555487681359367, 29.334028677634024, 3023.9053138557983, 527.6859946984337, -622.2531051228195, -158.74164952136778], '0.001000_137.500000_1.818182_625.000000_0.000000_0.100000': [-8855.313885727835, 3127.4573568614787, -859.2793802563538, -568.9761516146516, 34.23790374828627, -746.8080615917079, -253.21000571717138, -535.3957559558294], '0.001000_151.250000_1.818182_625.000000_0.000000_0.100000': [241.55039347725824, -530.1290741929561, 277.5473046991892, 44.5076802196412, 1295.665041077811, -46.31820747497204, 0, 0], '0.000909_151.250000_1.818182_625.000000_0.000000_0.100000': [0, 1910.3720013212924, -4.103999969805045, -985.0769499126612, 3.7186686900868153, 342.875183941268, -404.54644006216824, -948.3670811088251], '0.000909_151.250000_1.652893_625.000000_0.000000_0.100000': [324.7268512896903, 450.1622121635748, 0, 0, 0, 0, -356.8141584318435, 0], '0.000909_166.375000_1.652893_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -388.3006771792114, 0], '0.000909_183.012500_1.652893_625.000000_0.000000_0.100000': [0, 0, 0, -402.2234238766367, 0, 0, 0, 0], '0.000909_183.012500_1.818182_625.000000_0.000000_0.100000': [0, 414.0680932752343, 0, 0, 0, 0, 0, 0], '0.000909_183.012500_1.818182_568.181818_0.000000_0.100000': [902.8673588726015, 0, -4.515363567544774, -1005.488605043638, 0, 0, 0, 0], '0.001000_183.012500_1.818182_568.181818_0.000000_0.100000': [449.70385511429157, 0, 0, -1006.0189987104308, 0, 0, 0, 0], '0.001000_201.313750_2.000000_568.181818_0.000000_0.100000': [-342.0622666280804, 0, 0, 0, 0, 0, 0, 0], '0.000909_183.012500_2.200000_568.181818_0.000000_0.100000': [330.9352890992177, 475.8570159263425, 0, 0, 0, 847.5742330975172, 0, 0], '0.000909_183.012500_2.000000_568.181818_0.000000_0.100000': [0, 0, 0, 0, 4.597918127186495, 1005.488605043638, 0, 0], '0.000826_183.012500_2.000000_568.181818_0.000000_0.100000': [461.28026479513863, 0, 0, -848.7655178399376, 0, 0, 0, 0], '0.000826_183.012500_2.200000_568.181818_0.000000_0.100000': [0, 0, 0, -116.22058725489224, 3.1335408724255105, 0, -324.13883872835214, -433.65848260405585], '0.000826_201.313750_2.200000_568.181818_0.000000_0.100000': [324.13883872835214, 0, 0, 0, 0, 0, 0, 0], '0.000751_183.012500_2.200000_568.181818_0.000000_0.100000': [331.3349734401878, 0, 0, 0, 0, 0, 0, 0], '0.000751_166.375000_2.200000_568.181818_0.000000_0.100000': [332.9818223477327, 0, 0, -579.7505887947921, 0, 980.212884452337, 0, 0], '0.001210_125.000000_2.000000_625.000000_0.000000_0.100000': [2251.706349219867, 195.7258595759929, 412.8458770218056, 181.34559669027158, -124.39094336956356, 871.0853002713757, -368.03620781868204, -606.083982617145], '0.001210_113.636364_2.000000_625.000000_0.000000_0.100000': [0, 654.0556954882842, 494.1362318081447, 0, 1319.7461426475793, 0, 0, -574.2281833692114], '0.000909_113.636364_2.000000_625.000000_0.000000_0.100000': [488.9720500332106, 285.4114045366292, 63.755727012937086, -1794.2461616497355, -63.279134439013774, 2510.232687359173, 2122.994146665812, -1253.7314571006164], '0.001000_113.636364_2.000000_687.500000_0.000000_0.100000': [1410.491000524686, 3219.5878573097634, -558.9120703665162, 941.5267802414594, 1618.0973483207847, 4622.28369539768, -926.0182557011499, 667.3973320170276], '0.001000_113.636364_2.200000_687.500000_0.000000_0.100000': [32.46412641633073, 160.0729704663084, 1234.5203609712494, -543.4544066228694, -269.5381491051721, 2581.9337199411434, -395.56797752727414, -161.56956446348147], '0.000909_113.636364_2.000000_687.500000_0.000000_0.100000': [1214.473942492203, 1228.686244979917, 82.1473151794412, -1213.1233727874635, -324.3808230916344, 862.1507425739146, -1386.7607254612708, -431.04233337552614], '0.000909_103.305785_2.000000_625.000000_0.000000_0.100000': [632.0189818816458, 640.5640486724991, 3.563428622989318, -1781.5850150030578, 5.599949682198476, 0, -909.9852484739347, 350.32182953399115], '0.000826_103.305785_2.000000_625.000000_0.000000_0.100000': [343.89015074228666, 0, 565.1487890162568, 0, 4.626627035558158, 797.2633195405497, -1359.0293371871303, 0], '0.000909_103.305785_2.000000_568.181818_0.000000_0.100000': [396.5053378154498, 0, -809.2918537431575, -1274.1773362925417, -338.5088352391816, 1437.4927782613547, 0, -1964.0530920232359], '0.000909_103.305785_2.200000_568.181818_0.000000_0.100000': [0, 551.6114831713911, -598.5588126227715, 0, 5.3210402068085205, 0, 0, -557.8621549295345], '0.000826_103.305785_2.200000_568.181818_0.000000_0.100000': [402.0688883925815, 0, -4.837309278916837, -712.7325050602509, 0, 0, 0, 0], '0.000909_103.305785_2.200000_625.000000_0.000000_0.100000': [330.9965434439575, 1011.7882667933028, 0, -718.569544258165, 4.508181436203995, -500.37313903075625, 735.1543339348243, 0], '0.000909_93.914350_2.200000_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -596.3102402019402], '0.000909_93.914350_2.200000_687.500000_0.000000_0.100000': [0, 0, 0, -711.1999108381423, 0, 0, 0, 0], '0.000909_93.914350_2.420000_687.500000_0.000000_0.100000': [217.68702625653532, 558.1005499076891, 0, 0, 0, 0, 0, 0], '0.000909_93.914350_2.420000_625.000000_0.000000_0.100000': [0, 53.79218966053249, 0, 0, 2.852048023659518, 0, 0, -558.1005499076891], '0.000826_93.914350_2.420000_625.000000_0.000000_0.100000': [0, 0, 0, -606.8508030103445, 0, 0, -252.68128386759128, 0], '0.000826_93.914350_2.662000_625.000000_0.000000_0.100000': [0, 0, 0, 0, 3.0871496014211517, 0, 0, 0], '0.000751_93.914350_2.662000_625.000000_0.000000_0.100000': [0, 603.6408106230632, 0, 0, 0, 0, 0, 0], '0.000751_93.914350_2.662000_568.181818_0.000000_0.100000': [235.81420851670683, 0, 0, 0, 0, 0, 0, 0], '0.001000_125.000000_2.000000_756.250000_0.000000_0.100000': [342.5607764722163, 563.4158800793283, 0, -222.61246943070608, 0, 1373.382167225297, 0, -635.3935965221062], '0.001000_113.636364_2.000000_756.250000_0.000000_0.100000': [2204.512492795977, 0, -6.151993347577991, 0, 5.593099802420148, 0, 0, -656.7934725267819], '0.001100_113.636364_2.000000_756.250000_0.000000_0.100000': [356.3796353927542, 0, 0, -1164.742394806437, 0, 0, 0, 0], '0.001100_113.636364_2.200000_756.250000_0.000000_0.100000': [283.532766560577, 1631.1367700226724, 0, -524.5987224943713, 0, 0, 861.8702405600296, 0], '0.001100_103.305785_2.200000_756.250000_0.000000_0.100000': [280.75493791693174, 3132.318481569263, -5.551589338143095, -595.7609269345639, 0, 0, 0, 0], '0.001100_103.305785_2.420000_756.250000_0.000000_0.100000': [0, 0, -3.8507190915452156, 0, 0, 0, 0, 0], '0.001210_103.305785_2.420000_756.250000_0.000000_0.100000': [205.30053701519736, 0, 0, 0, 0, 0, 0, 0], '0.001210_93.914350_2.420000_756.250000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -578.9469051020968], '0.001210_93.914350_2.420000_831.875000_0.000000_0.100000': [0, 0, 0, 0, 0, 640.8247247210446, 0, 0], '0.001210_93.914350_2.200000_831.875000_0.000000_0.100000': [0, 0, 0, -640.8247247210446, 0, 1313.8366730392017, 0, 11627.817102830255], '0.001210_93.914350_2.000000_831.875000_0.000000_0.100000': [0, 1874.3339322143804, 0, -1194.3969754901834, 0, 900.402535880484, 0, 0], '0.001000_137.500000_2.200000_625.000000_0.000000_0.100000': [615.8654220261927, 482.96989662458674, 3402.034603833269, -368.7718613741241, -317.0684992942297, -5911.890543185625, -395.15926345112257, 0], '0.001000_137.500000_2.200000_568.181818_0.000000_0.100000': [1065.0678898762721, 168.81392164170842, 0, -374.1817105716844, 317.7460978100891, 1098.0992404178546, -69.08940989434029, 0], '0.001000_151.250000_2.200000_568.181818_0.000000_0.100000': [401.6453773709344, 0, 0, -254.23931716531806, -426.7999016075862, 1111.75422956295, -330.99310880804586, 0], '0.001000_166.375000_2.200000_568.181818_0.000000_0.100000': [0, 941.4554116037378, 0, -177.5627014464628, 0, 0, -327.37482508307414, -462.80810169196286], '0.001000_166.375000_2.420000_568.181818_0.000000_0.100000': [0, 424.7943817893429, 0, -405.8058903244253, 2.0188121999008217, 0, 0, -485.3238278131722], '0.001000_166.375000_2.420000_516.528926_0.000000_0.100000': [273.6974290026108, 0, -2.1802170324497454, 0, 0, 0, 0, 0], '0.001100_166.375000_2.420000_516.528926_0.000000_0.100000': [0, 411.94317631689665, 0, 0, 0, 0, -277.8164410950503, 0], '0.001100_183.012500_2.420000_516.528926_0.000000_0.100000': [0, 0, 0, 0, 1.6137869633828814, 0, 0, 0], '0.001000_183.012500_2.420000_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -229.311239628988, 0], '0.001000_201.313750_2.420000_516.528926_0.000000_0.100000': [0, 479.91103549496313, 0, 0, 0, 0, 0, 0], '0.001000_201.313750_2.420000_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 137.07706005545734, 0, 0], '0.001000_201.313750_2.200000_469.571751_0.000000_0.100000': [0, 0, -3.801268769660783, 0, 0, 0, 0, 0], '0.001100_201.313750_2.200000_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 595.7310026385861, 0, 0], '0.001100_201.313750_2.000000_469.571751_0.000000_0.100000': [423.98504016308453, 0, 0, 0, 0, 827.9427027973176, 0, -438.46993292320514], '0.001100_201.313750_1.818182_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -359.36279229360406], '0.001100_201.313750_1.818182_516.528926_0.000000_0.100000': [0, 0, 0, 0, 5.132377763504519, 0, 0, 0], '0.001000_201.313750_1.818182_516.528926_0.000000_0.100000': [0, 0, 0, -907.3085962215912, 0, 0, 0, -376.7485187652492], '0.001000_201.313750_1.818182_568.181818_0.000000_0.100000': [0, 376.7485187652492, 0, 0, 0, 0, 0, 0], '0.001000_201.313750_2.000000_516.528926_0.000000_0.100000': [0, 0, -4.873624968831973, -692.0504445551608, 0, 0, 0, 0], '0.001000_201.313750_2.200000_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 692.0504445551608, 0, 0], '0.001100_201.313750_2.000000_516.528926_0.000000_0.100000': [439.3350800399374, 438.46993292320514, 0, 0, 0, 0, 0, 0], '0.001100_183.012500_2.000000_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -423.98504016308453, 0], '0.001100_183.012500_2.000000_516.528926_0.000000_0.100000': [475.26400941253286, 0, 0, 0, 0, 0, 0, 0], '0.001100_166.375000_2.000000_516.528926_0.000000_0.100000': [482.60706171707244, 0, -7.46056420440993, -933.2947702081552, 0, 0, 0, 0], '0.001000_103.305785_2.200000_625.000000_0.000000_0.100000': [330.9665586115569, 2465.7864615671433, -395.6598988161813, -189.32797740937207, 0, 600.3287682068333, -349.2374358326213, -51.16626846237159], '0.000909_113.636364_2.200000_625.000000_0.000000_0.100000': [0, 587.2184866032633, 3088.4668494081916, -69.2338294462362, 2117.744159967155, 1235.5849362228007, 625.3873505112119, 1003.278083870617], '0.000909_113.636364_2.420000_625.000000_0.000000_0.100000': [1421.6157029805624, 491.53299184574445, 0, 0, 580.8782783138339, 607.0398085954648, -300.228150355332, 0], '0.000826_113.636364_2.420000_625.000000_0.000000_0.100000': [260.54880017328577, 0, 0, -527.0722150510919, 2.24103318743164, 3038.017042622465, -273.1613454155673, 0], '0.000751_113.636364_2.420000_625.000000_0.000000_0.100000': [0, 491.52125413923824, -2.24103318743164, 0, 2.02916912331566, 0, -273.1656747775922, 0], '0.000751_125.000000_2.420000_625.000000_0.000000_0.100000': [273.1656747775922, 0, 0, 0, 0, 0, -256.9678539972334, 0], '0.000751_137.500000_2.420000_625.000000_0.000000_0.100000': [256.9678539972334, 0, 0, 0, 0, 0, -248.81818621376252, 0], '0.000751_151.250000_2.200000_625.000000_0.000000_0.100000': [0, 0, -0.25769165691350276, 0, 0, 0, -317.47309423039405, -428.6439754198038], '0.000751_166.375000_2.200000_625.000000_0.000000_0.100000': [0, 466.72142379330893, 0, 0, 1.917324080832259, 0, 0, 0], '0.000683_166.375000_2.200000_625.000000_0.000000_0.100000': [0, 0, 0, -157.46732303678655, 3.208056452096717, 0, 0, 0], '0.000683_166.375000_2.420000_625.000000_0.000000_0.100000': [0, 0, 0, 0, 1.4161872412414596, 0, 0, 0], '0.000621_166.375000_2.420000_625.000000_0.000000_0.100000': [236.22439067159357, 0, 0, 0, 0, 157.80702189449022, 0, 0], '0.000621_166.375000_2.200000_625.000000_0.000000_0.100000': [0, 0, -1.7558860989451206, 0, 1.7748748760741728, 0, 0, 0], '0.000564_166.375000_2.200000_625.000000_0.000000_0.100000': [0, 0, -1.6135226146128845, -158.14050967169098, 0, 0, 0, 0], '0.000564_166.375000_2.420000_625.000000_0.000000_0.100000': [0, 0, -1.2800348374121313, 0, 0, 0, 0, 0], '0.000621_151.250000_2.420000_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001000_113.636364_2.000000_568.181818_0.000000_0.100000': [520.5330602259326, 6530.031622816367, 901.1613484441485, 1161.6585989100652, 25.79972530326495, 2018.9601251803338, 2044.0224267635463, 25.888268937284465], '0.001000_113.636364_2.000000_516.528926_0.000000_0.100000': [872.4215613677559, 513.6376638895957, -6.797658486417641, -1024.4750189205497, -501.8302990689027, 6115.165625089198, 179.0583649909678, 599.2203174859187], '0.001000_103.305785_2.000000_516.528926_0.000000_0.100000': [635.6637304272253, 1215.293646961818, -5.991046888542406, 293.22446047289634, 832.6036401605284, 0, 3161.927796636386, -1803.140383506487], '0.001000_93.914350_2.000000_516.528926_0.000000_0.100000': [0, 1903.634951036005, 0, 0, 5.288489626908268, 0, 321.6751938408104, 0], '0.001000_93.914350_2.000000_469.571751_0.000000_0.100000': [424.97690398221675, 2001.5791863580762, 113.37456056781275, 0, 5.085788861200036, 394.4591767835881, -418.20941484772817, -537.8282872821886], '0.000909_93.914350_2.000000_469.571751_0.000000_0.100000': [0, 1034.2325646235915, 0, -1240.4947467435786, 797.6199012318202, 392.66446403175485, 0, -537.6255865164804], '0.000909_93.914350_2.000000_516.528926_0.000000_0.100000': [0, 537.6255865164804, 476.93231531153907, -1266.9511582817574, 0, 419.3608126030178, 0, -56.75213429220434], '0.000909_93.914350_2.200000_516.528926_0.000000_0.100000': [0, 0, -5.6670887871203375, 0, 0, 1266.9511582817574, 0, -573.0538865717666], '0.000909_93.914350_2.200000_568.181818_0.000000_0.100000': [0, 573.0538865717666, 0, 0, 4.908500680910402, 0, 0, 0], '0.000909_93.914350_2.000000_426.883410_0.000000_0.100000': [0, -525.2020009861423, -4.826394301233449, -927.52003735377, 4.380539775215584, 618.1271688527678, 46.04924592312949, 0], '0.000909_93.914350_2.000000_388.075827_0.000000_0.100000': [0, 0, 0, -904.4753361883025, 4.545922577123567, 0, -391.7161842199954, 0], '0.000826_93.914350_2.000000_388.075827_0.000000_0.100000': [2290.1653658643427, 0, 0, -1158.609063852022, 0, 0, -391.3942082952299, 0], '0.000826_103.305785_2.000000_388.075827_0.000000_0.100000': [0, 0, 0, 0, 329.79947597264976, 0, 0, 0], '0.000751_103.305785_2.000000_388.075827_0.000000_0.100000': [0, 0, 0, 0, 3.635016563841782, 399.6601461917172, 0, -464.6313804364783], '0.000751_103.305785_1.818182_388.075827_0.000000_0.100000': [0, 0, -0.024085535964286465, -361.5453742658342, 108.53555177435895, 0, 0, -431.02252708058785], '0.000683_103.305785_1.818182_388.075827_0.000000_0.100000': [0, 0, 0, 0, 1.9703615493840516, 109.31310938545644, 0, -430.6267307639916], '0.000683_103.305785_1.818182_426.883410_0.000000_0.100000': [0, 86.12534615279588, 655.0733751345678, 0, 0, 0, 0, 0], '0.000621_103.305785_1.818182_388.075827_0.000000_0.100000': [312.3423781329766, 0, 0, 0, 0, 0, 0, 0], '0.000621_93.914350_1.818182_388.075827_0.000000_0.100000': [0, 0, -2.081398894632457, 0, 0, 0, 0, 0], '0.000683_93.914350_1.818182_388.075827_0.000000_0.100000': [0, 0, 266.341454549627, 0, 0, 0, 0, 0], '0.000751_93.914350_1.818182_388.075827_0.000000_0.100000': [3137.229718539503, 0, -2.542201190533706, 0, 0, 0, -312.10955525425027, 0], '0.000826_93.914350_1.818182_388.075827_0.000000_0.100000': [269.4454121055328, 709.2724865464043, 0, 653.4892253197354, 0, 0, 0, 0], '0.000826_85.376682_1.818182_388.075827_0.000000_0.100000': [0, 0, -2.2120325774059895, 727.7445391361538, 0, 0, 0, 0], '0.000826_85.376682_2.000000_388.075827_0.000000_0.100000': [329.9835112091439, 0, 0, 0, 3.0196858125579276, 0, 175.0197918009288, -621.6776404495466], '0.000751_85.376682_2.000000_388.075827_0.000000_0.100000': [2810.0899899432065, 0, 0, 0, 0, 957.7757624532021, 0, 0], '0.000909_113.636364_2.200000_687.500000_0.000000_0.100000': [0, 549.9689973762476, 0, -545.3859128250964, 4.270256969447291, 79.89393068946947, 0, -346.0620437317097], '0.000909_103.305785_2.000000_687.500000_0.000000_0.100000': [-1200.6811890172762, -75.19651817709541, -223.72893290510194, -2877.9598764977018, 4.980315628840648, 474.5291740786698, -1753.2244922272703, -3305.057256854174], '0.000909_93.914350_2.000000_687.500000_0.000000_0.100000': [334.2386254109281, 614.0148296051435, 0, 0, 5.074188595638953, 0, 0, 2359.573160565542], '0.000909_85.376682_2.000000_687.500000_0.000000_0.100000': [0, 0, 0, -1262.8763431689692, 0, 1922.2023534691286, 0, 0], '0.000909_85.376682_2.200000_687.500000_0.000000_0.100000': [0, 622.0537131267329, 0, 0, 0, 0, 0, 0], '0.000909_85.376682_2.200000_625.000000_0.000000_0.100000': [0, 0, 0, -824.8982322934091, 0, 1271.0586957977491, 0, 0], '0.000909_85.376682_2.420000_625.000000_0.000000_0.100000': [227.50751495352762, 0, 0, 0, 0, 0, 0, 0], '0.000909_77.615165_2.420000_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 2649.098822170774, 0, -600.6058891581464], '0.000909_77.615165_2.420000_687.500000_0.000000_0.100000': [0, 0, 0, -703.4488011321085, 0, 0, 0, 0], '0.000909_77.615165_2.662000_687.500000_0.000000_0.100000': [0, 0, 0, 0, 3.4571627333139987, 0, 0, 0], '0.000826_77.615165_2.662000_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -608.7001300226888], '0.000826_77.615165_2.662000_756.250000_0.000000_0.100000': [0, 0, 0, 0, 0, 696.5485773288983, 0, 0], '0.000826_77.615165_2.420000_756.250000_0.000000_0.100000': [177.52176113074492, 0, 0, 0, 0, 0, 0, -624.0369837401598], '0.000826_77.615165_2.420000_831.875000_0.000000_0.100000': [0, 624.0369837401598, 0, 0, 0, 0, 0, 0], '0.000826_70.559241_2.420000_756.250000_0.000000_0.100000': [0, 636.8889625487775, 0, 0, 0, 0, 0, 0], '0.000826_70.559241_2.420000_687.500000_0.000000_0.100000': [0, 0, -3.1975721245145907, 0, 0, 0, 0, 0], '0.000909_70.559241_2.420000_687.500000_0.000000_0.100000': [0, 614.7291631814824, 0, 0, 0, 0, 0, 0], '0.000909_70.559241_2.420000_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 4528.119776156657, -213.1932344857587, 0], '0.000909_70.559241_2.200000_625.000000_0.000000_0.100000': [0, 0, 0, -4528.119776156657, 3453.535653966604, 0, 0, -4227.901081347693], '0.000826_70.559241_2.200000_625.000000_0.000000_0.100000': [140.8816227242499, 5.841622099803344, 0, 0, 0, 0, -5267.736453133771, 0], '0.000826_70.559241_2.200000_568.181818_0.000000_0.100000': [0, 0, -31.496690680737856, 0, 0, 0, 0, -5.363671200728525], '0.000909_137.500000_2.200000_625.000000_0.000000_0.100000': [-5994.283023294857, 483.4593664762959, -17.933050334523784, 0, 0, 2490.871131465323, -609.4590409248979, -928.9887706667753], '0.000909_137.500000_2.200000_687.500000_0.000000_0.100000': [682.263065876532, 544.564935655305, -3.006309955040888, -380.4202910725946, 0, 2203.8190725108025, 0, -509.0727244974742], '0.000909_137.500000_2.000000_625.000000_0.000000_0.100000': [-14760.455024901723, -263.35005206361217, 464.38960711164486, -1310.2069263529177, 140.034182915246, 2583.987866782235, -58.73051973709653, -1027.9381140721969], '0.000909_151.250000_2.200000_625.000000_0.000000_0.100000': [1395.3962932104112, 479.43768675517475, -3.303550068043169, -259.83101607821754, 3.0749631311833974, 1480.4310201698424, 0, 0], '0.001000_151.250000_2.200000_625.000000_0.000000_0.100000': [3202.1973936928134, 530.2990563556341, 319.04596915560796, 0, 3.3335823413890155, 0, -316.0543128823203, -428.55016456253963], '0.000826_151.250000_2.200000_625.000000_0.000000_0.100000': [0, 0, -5.5237154662445125, 0, 2.834608226048531, 0, 0, 0], '0.001000_151.250000_2.420000_568.181818_0.000000_0.100000': [0, 0, 0, 0, 1.8590127299232335, 0, -676.8312742452506, 0], '0.001000_166.375000_2.420000_625.000000_0.000000_0.100000': [236.81043400754825, 485.3238278131722, 0, 12053.828504873874, 0, 0, 12053.828504873874, 0], '0.001000_183.012500_2.420000_625.000000_0.000000_0.100000': [-12053.828504873874, 0, 0, 0, 0, 0, 0, 0], '0.001000_166.375000_2.662000_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, -12053.828504873874, 0, 0], '0.001000_151.250000_2.420000_625.000000_0.000000_0.100000': [0, 481.6920084249516, 0, 0, 1.9951711774518117, 0, 10611.63522037894, 0], '0.001000_137.500000_2.000000_625.000000_0.000000_0.100000': [-8799.248649810856, -411.8341163472023, -1313.567328416871, 2317.294052743665, -491.5952914549057, -597.0237652442759, -321.05789895298653, -1424.4267976692731], '0.000909_137.500000_1.818182_625.000000_0.000000_0.100000': [1113.5073885082243, 950.4212602475666, -1287.2653543136855, 82.16813285840016, 264.02306397531385, 431.4957547256863, 756.0704238470614, -525.977994569952], '0.000826_137.500000_1.818182_687.500000_0.000000_0.100000': [528.9456877780905, 0, -65.92664316853359, 0, 0, 0, -368.6353791831263, 0], '0.000826_151.250000_1.818182_687.500000_0.000000_0.100000': [397.5479579425872, 0, 0, 0, 0, 0, -387.17342089826946, 0], '0.000909_137.500000_1.652893_687.500000_0.000000_0.100000': [287.1399914586224, 1595.703246787844, -3.2195338172974517, 0, -41.50045383788665, 37.02934496897779, -626.5923240718546, -497.0049278910046], '0.000909_151.250000_1.652893_687.500000_0.000000_0.100000': [347.26678063747613, 0, 0, -778.3875752537381, 0, 0, 0, 0], '0.000909_166.375000_1.818182_687.500000_0.000000_0.100000': [383.5086763048201, 450.5170450878026, -4.537798157894577, 0, 0, 0, -411.2282945121013, -466.7368338600263], '0.000909_166.375000_1.818182_756.250000_0.000000_0.100000': [0, 0, -4.649860628395072, -1066.3182818250186, 0, 0, -429.10228445595766, -528.8780067328023], '0.000909_166.375000_1.818182_831.875000_0.000000_0.100000': [0, 480.7981879389112, 0, 0, 0, 0, 0, 0], '0.000909_183.012500_1.818182_756.250000_0.000000_0.100000': [39.009298586905246, 0, 0, 0, 0, 399.01055440582786, 0, 0], '0.000909_183.012500_1.652893_756.250000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -346.8697916142617, 0], '0.000909_201.313750_1.652893_756.250000_0.000000_0.100000': [0, 0, 0, 0, 0, 278.856415343468, 0, 0], '0.000909_201.313750_1.502630_756.250000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -324.2439715100749, 0], '0.000909_221.445125_1.502630_756.250000_0.000000_0.100000': [0, 0, 0, 0, 0, -105.79509492523448, 0, 0], '0.000909_221.445125_1.366027_756.250000_0.000000_0.100000': [0, 0, 0, 0, 2.823727156647692, 0, 0, 0], '0.000826_221.445125_1.366027_756.250000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001100_125.000000_1.818182_625.000000_0.000000_0.100000': [3456.783254082824, 1750.1011467640617, 172.01061124807467, -274.5852945381773, 1211.136000428452, 160.1207072643436, -456.14098628968395, 108.40794083537025], '0.001100_137.500000_1.818182_625.000000_0.000000_0.100000': [-106.48434011145304, 1083.7102971207128, -221.4861898819904, -20.182005436609188, -834.3908108863767, 214.3352470958435, -414.0053287870165, -896.3500116930161], '0.001100_151.250000_1.818182_625.000000_0.000000_0.100000': [0, 0, -4.991844922657584, -988.3188150229682, 0, 749.250857370851, 0, 0], '0.001100_151.250000_1.652893_625.000000_0.000000_0.100000': [357.58532463247974, 1096.515002038395, 0, -313.4139600649729, 3.996742554997436, 0, 0, 0], '0.001000_151.250000_1.652893_625.000000_0.000000_0.100000': [734.8086109816813, 449.8097700097311, -0.7266804645449882, 0, 0, -6.16442876907844, 0, -457.585836952422], '0.001100_137.500000_1.652893_625.000000_0.000000_0.100000': [778.4621174172153, 126.23299832573821, 73.4301998824485, 269.58879436265397, 0, 248.7660390812287, -328.0626125442941, -477.9648153791619], '0.001100_151.250000_2.000000_625.000000_0.000000_0.100000': [0, 485.8001286322595, 380.9339181872327, 0, 0, 0, 527.971313228553, 0], '0.001100_166.375000_2.000000_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 1534.8806812118473, 0, 0], '0.001100_166.375000_1.818182_625.000000_0.000000_0.100000': [405.15540023192716, 805.1261818068575, 0, 0, 0, 0, 0, 0], '0.001100_166.375000_1.818182_568.181818_0.000000_0.100000': [0, 413.0597567348406, -5.134137934508362, -997.0237735731598, 4.655140426405706, 402.2313398275283, 0, 0], '0.001000_166.375000_1.818182_568.181818_0.000000_0.100000': [0, 0, -4.655140426405706, 0, -399.8252937295204, 0, 0, -432.5641820726816], '0.001100_166.375000_2.000000_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 997.0237735731598, 0, 0], '0.001210_166.375000_1.818182_568.181818_0.000000_0.100000': [420.9320548659998, 0, -5.663739443878876, 0, 5.647551727959199, 0, 0, 0], '0.001100_166.375000_1.652893_568.181818_0.000000_0.100000': [0, 0, 0, 0, 3.7864237015679465, 0, 0, -437.606119873171], '0.001100_166.375000_1.652893_625.000000_0.000000_0.100000': [357.7580686386247, 0, 0, 0, 4.126421863478754, 0, 0, 0], '0.001210_137.500000_1.818182_625.000000_0.000000_0.100000': [672.0888347338647, 110.46138949889047, 68.46287574305131, -94.63848742247312, 949.2597720280929, 339.88798229790837, 0, -493.09684817621303], '0.001210_137.500000_1.818182_687.500000_0.000000_0.100000': [0, 1451.3247881478194, -5.952395395496133, -1077.377932248245, 4.903943814771447, 750.1661066446208, 0, -5.02371820737543], '0.001100_137.500000_1.818182_687.500000_0.000000_0.100000': [-434.13551464972807, 0, -4.903943814773084, -1397.3448441589676, 0, 295.3677646459714, -835.7593262031928, 0], '0.001100_151.250000_1.818182_687.500000_0.000000_0.100000': [0, 0, -5.234172122291056, 0, 0, 0, 0, -534.4770669581012], '0.001100_151.250000_1.818182_756.250000_0.000000_0.100000': [388.451606486309, 0, 0, 0, 0, 0, 0, 0], '0.001100_125.000000_1.818182_568.181818_0.000000_0.100000': [267.0460025583643, -692.6953402504981, -396.8006729783129, -319.2767261605818, -742.8575640332679, 360.47566162723274, 1135.1309429166618, 0], '0.001100_137.500000_1.818182_568.181818_0.000000_0.100000': [307.3492629034374, 1710.926318737732, -4.1995895553300215, -273.63310124980126, 296.1836096955638, 308.3891569586695, 0, -481.5102103872596], '0.001100_137.500000_1.818182_516.528926_0.000000_0.100000': [994.3718732006844, 1895.4430927121944, 314.80660578429615, -822.4796136217982, 3.741833323277242, 292.19379464874186, 0, -435.07746867399186], '0.001210_137.500000_1.652893_516.528926_0.000000_0.100000': [314.9962891352361, 909.5372649829548, 0, 0, 3.2844691879789027, -143.37217819842027, 0, 0], '0.001210_125.000000_1.652893_516.528926_0.000000_0.100000': [320.5655019040262, 439.30477381280906, 257.10366468950855, 0, 265.3414566320327, 0, 0, -450.10401858293847], '0.001210_113.636364_1.652893_516.528926_0.000000_0.100000': [791.9327750123213, 0, -2.8533942139268222, -213.62150983636676, 234.97117657348312, -18.148374773826347, -321.4760049268964, -465.43352930302234], '0.001331_113.636364_1.652893_516.528926_0.000000_0.100000': [258.04403528284786, 445.7855942761531, -3.5359692145276957, 0, 2.8533942139268222, 0, 0, 0], '0.001210_103.305785_1.652893_516.528926_0.000000_0.100000': [267.8387963759427, 1359.085750997858, 0, -165.8668083763745, 0, 19.16698397856217, -197.7379294710057, -744.0147454739542], '0.001210_103.305785_1.652893_568.181818_0.000000_0.100000': [0, 393.150873491133, 0, -179.3679167428456, 626.3335335100601, 0, -246.26543136735557, -443.3392791940265], '0.001210_113.636364_1.652893_568.181818_0.000000_0.100000': [447.3150416453851, 346.271706957828, -2.9633860857682883, -198.9207199872679, 2.413583677158749, 1335.9663738895022, 0, 0], '0.001100_113.636364_1.652893_568.181818_0.000000_0.100000': [0, 438.0482191717308, 999.381130505041, 330.47464376553205, 0, -85.34695423249377, -275.896001388237, -473.3280440966082], '0.001100_113.636364_1.652893_516.528926_0.000000_0.100000': [286.64216394634514, 582.2381579232913, 717.6725055149557, -39.069013130790154, 753.649908792753, 0, 68.7085727863277, 0], '0.001100_125.000000_1.652893_516.528926_0.000000_0.100000': [364.07108427946474, 0, 0, -269.74216963340587, 1380.2993255873664, 0, 0, 255.50782585589297], '0.001100_125.000000_1.818182_516.528926_0.000000_0.100000': [353.66281450538486, -132.68720004052193, -45.91700411983368, 305.9315352875201, 12.300825068184395, 249.30167418390283, -163.42334120648226, 953.9341174033368], '0.001210_125.000000_1.652893_568.181818_0.000000_0.100000': [0, 0, -3.5245696925196173, 1307.7660657299919, 54.386430013148754, -124.79932708593006, -634.5309408745075, -481.1675706599348], '0.001210_125.000000_1.818182_568.181818_0.000000_0.100000': [1297.4368946295644, 216.65926242824764, -4.403725284254542, -803.2862043688049, 962.9531688966225, -125.6604634420168, -322.01014420736817, -460.00131878960315], '0.001210_125.000000_2.000000_568.181818_0.000000_0.100000': [0, 613.4678182184324, -7.5627354139847744, -1237.0010594232133, 1587.070035836767, 0, 0, -200.9073570051242], '0.001000_103.305785_2.000000_625.000000_0.000000_0.100000': [464.81888828724675, 1100.8754353254599, 3766.2777948815556, 206.9334786442762, 748.8606786640296, 826.9685539866714, 721.2348959078854, 672.6953691375099], '0.001100_103.305785_2.000000_625.000000_0.000000_0.100000': [406.7898253808292, 628.3103935184113, 577.0262324167646, -2140.0795837099963, 734.054172057956, 3347.3554147192, 470.5687560893929, -1482.8225519356783], '0.001210_103.305785_2.000000_625.000000_0.000000_0.100000': [632.4963930061289, 1892.4357089698663, -7.46437110470597, -1253.3520382278496, 6.7831019293205825, 1004.2782569442865, 521.1246226469887, 0], '0.001210_103.305785_2.200000_625.000000_0.000000_0.100000': [896.1998656603694, 0, 0, -1147.4961926269036, 6.007081771879894, 250.67040764556987, 0, -574.3525470157559], '0.001210_93.914350_2.200000_625.000000_0.000000_0.100000': [347.6142398222567, 1161.742596377524, 0, -743.3443516222242, 0, 0, 0, -596.4514091524649], '0.001210_93.914350_2.200000_687.500000_0.000000_0.100000': [437.19118847510856, 0, -6.702860211778716, 0, 0, 0, -308.8022232901216, 0], '0.001331_93.914350_2.200000_687.500000_0.000000_0.100000': [0, 596.506991107998, 0, 0, 0, 0, 0, 0], '0.001331_93.914350_2.200000_625.000000_0.000000_0.100000': [0, 0, 0, 0, 6.647278256245591, 0, 0, 0], '0.001210_93.914350_2.200000_568.181818_0.000000_0.100000': [0, 2266.305062436309, 0, -791.1859211336222, 0, 0, 0, 0], '0.001210_93.914350_2.420000_568.181818_0.000000_0.100000': [0, 499.2565225160959, 0, 0, 0, 791.1859211336222, 0, 0], '0.001210_93.914350_2.200000_516.528926_0.000000_0.100000': [2118.6590027384004, 0, 0, -445.84679573890537, 0, 0, -382.94839803988236, 0], '0.001210_103.305785_2.200000_516.528926_0.000000_0.100000': [756.9743805458262, 0, 0, 0, 12.95297380568482, 0, 0, -606.2971960644211], '0.001210_103.305785_2.200000_568.181818_0.000000_0.100000': [0, 0, -6.731134115910391, 0, 6.234490838642341, 0, 0, -557.0060164750721], '0.001331_103.305785_2.000000_625.000000_0.000000_0.100000': [344.2572748448994, 634.5081136832723, 0, 0, 7.46437110470597, 0, 0, 0], '0.001331_103.305785_2.000000_568.181818_0.000000_0.100000': [0, 0, 0, 0, -325.5163038051214, 605.8018377323327, -386.96165289539675, 0], '0.001331_113.636364_2.000000_568.181818_0.000000_0.100000': [386.96165289539675, 597.5870785819974, 0, -1257.3238502179427, 0, 784.0906642445743, -411.8314730730468, -611.942206137908], '0.001331_125.000000_2.000000_568.181818_0.000000_0.100000': [-139.99319871185958, 0, -8.322174121081844, -1248.2236542282158, 0, 0, 0, 0], '0.001464_125.000000_2.000000_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 810.729998193794, 0, 0], '0.001464_125.000000_1.818182_568.181818_0.000000_0.100000': [318.5874633876016, 0, 0, 0, 0, 240.3354603997199, -355.0798958121557, 0], '0.001464_137.500000_1.818182_568.181818_0.000000_0.100000': [0, 0, -5.6838080270496, -890.8826761176814, 0, 0, 0, 0], '0.001464_137.500000_2.000000_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -456.6477129054154, 0], '0.001464_151.250000_2.000000_568.181818_0.000000_0.100000': [0, 0, 0, -1105.868469604837, 0, 0, 0, 0], '0.001464_151.250000_2.200000_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001000_125.000000_2.000000_516.528926_0.000000_0.100000': [-2.2072869798789725, 448.538326303644, 441.34632939295335, -98.72731975170768, 1837.2898370260664, 3753.4643707305718, -470.76606726883756, 416.3728440586928], '0.001000_103.305785_2.000000_469.571751_0.000000_0.100000': [0, -270.98196163123356, 2317.350290108684, -290.7222418256975, 78.05996757788989, 0, -1507.645359261082, -105.51755692645162], '0.000909_103.305785_2.000000_469.571751_0.000000_0.100000': [434.8323226946907, 56.642021757890234, 442.80303508619113, 0, 0, 853.4508064075828, -505.5676565268553, -580.1850890641199], '0.000909_93.914350_1.818182_469.571751_0.000000_0.100000': [509.30697062576826, 1286.4437446105912, 311.6673049361337, -356.9676945743226, 0, 0, -297.60992696107144, 0], '0.000909_93.914350_2.200000_469.571751_0.000000_0.100000': [0, 539.4775258602122, 0, 0, 0, 0, 0, 0], '0.000909_93.914350_2.200000_426.883410_0.000000_0.100000': [915.9054832897822, 0, -5.879361133996826, 0, 5.398316863352649, 0, 0, 0], '0.001000_93.914350_2.200000_426.883410_0.000000_0.100000': [401.7797939716203, 0, -6.4672808621000515, 0, 5.879361133996826, 0, -425.7959582213799, -539.4720160569341], '0.001000_93.914350_2.200000_469.571751_0.000000_0.100000': [0, 0, 0, -971.0537028245795, 0, 0, -413.4077386220624, 0], '0.001000_93.914350_2.420000_469.571751_0.000000_0.100000': [0, 0, -2.3468187951841175, 0, 0, 0, 0, 0], '0.001100_93.914350_2.420000_469.571751_0.000000_0.100000': [0, 544.7310971212231, 0, -332.1258821251984, 0, 0, 0, 0], '0.001100_93.914350_2.662000_469.571751_0.000000_0.100000': [0, 577.3602791561009, 0, 0, 4.040038559141613, 0, 0, 0], '0.001100_93.914350_2.662000_426.883410_0.000000_0.100000': [240.40188848197013, 0, 0, 0, 0, 0, 0, 0], '0.001100_85.376682_2.662000_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -581.1418467415218], '0.001100_85.376682_2.662000_469.571751_0.000000_0.100000': [0, 0, -4.266327247217851, 0, 0, 0, 0, 0], '0.001210_85.376682_2.662000_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 397.3723686172543, -236.81218295804373, 0], '0.001210_85.376682_2.420000_469.571751_0.000000_0.100000': [290.75220950081604, 0, 0, -397.3723686172543, 0, 0, 0, 0], '0.001210_93.914350_2.662000_469.571751_0.000000_0.100000': [0, 0, 0, 0, 4.458189308712463, 0, 0, 0], '0.001000_93.914350_2.662000_469.571751_0.000000_0.100000': [0, 576.6754249194337, 0, 0, 0, 0, 0, 0], '0.001000_93.914350_2.662000_426.883410_0.000000_0.100000': [0, 0, 0, -468.09164865999463, 0, 0, -256.6991745581061, 0], '0.001000_93.914350_2.928200_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -243.79405693566386, 0], '0.001000_103.305785_2.928200_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001210_125.000000_2.000000_687.500000_0.000000_0.100000': [0, 0, -7.501883038241977, -1171.6284858630613, -1154.4520541163533, 1869.7191690869222, -386.08693850026356, 0], '0.001100_125.000000_2.000000_687.500000_0.000000_0.100000': [1459.8091915542955, 1010.9806527916077, 76.02807555094671, 1609.669366670417, 1.0800533942992283, 873.9469251374337, -429.575341256147, 357.2429902473805], '0.001100_125.000000_2.200000_687.500000_0.000000_0.100000': [329.39434478058666, 512.0799147682463, -5.233235181729942, -504.28569666057945, 1830.9661620985307, 2229.710532463258, 106.45911490211411, 1239.9358284585721], '0.001000_125.000000_1.818182_687.500000_0.000000_0.100000': [-538.0158032365412, 501.04188624402104, -132.58274868659328, 3.711549721921017, -20.288004356521014, 3109.3595111178265, 1223.657914225395, -113.73038477044315], '0.000909_125.000000_1.818182_687.500000_0.000000_0.100000': [837.1868392409025, 1636.7677191811274, 972.2365357786041, -2383.9136072434403, 3.73361098752291, 916.5030271468627, -114.05468810253132, -855.4308781628449], '0.000826_125.000000_1.818182_687.500000_0.000000_0.100000': [303.2309579794277, 2093.564486197124, 0, -920.7857442246385, 5.537231269451095, 238.03953736099817, 0, 346.8556720157257], '0.000826_125.000000_1.652893_687.500000_0.000000_0.100000': [0, 1015.4372923050854, 0, 0, 0, -313.78363277933516, -57.38582364538246, -470.5459661078125], '0.000826_137.500000_1.652893_687.500000_0.000000_0.100000': [360.688652121046, 478.27037716882444, 0, 0, 0, 0, -315.31274331180265, -546.38233310308], '0.000826_137.500000_1.652893_756.250000_0.000000_0.100000': [0, 0, 0, 0, 2.9054658990660758, 145.38055301717486, -307.8344818282456, 0], '0.000751_137.500000_1.652893_756.250000_0.000000_0.100000': [0, 496.44245885563595, 0, 0, 0, 0, 0, 0], '0.000751_137.500000_1.652893_687.500000_0.000000_0.100000': [0, 0, 0, 0, 2.6265734740373046, 0, -314.96270248937236, 0], '0.000751_151.250000_1.652893_687.500000_0.000000_0.100000': [0, 0, -2.9867536652414857, 0, 0, 0, 0, 0], '0.000826_151.250000_1.652893_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 60.89766590295658, 0, -492.44705226249147], '0.000826_151.250000_1.502630_687.500000_0.000000_0.100000': [289.1588541705123, 0, 0, 0, 0, 0, 0, 0], '0.000826_137.500000_1.502630_687.500000_0.000000_0.100000': [290.4412741764265, 0, 0, 0, 0, 0, 0, -386.07443565638226], '0.000826_125.000000_1.502630_687.500000_0.000000_0.100000': [0, 460.22222110820974, 0, 395.0436885599537, 0, 0, -302.259365989071, -450.8350730345519], '0.000826_125.000000_1.502630_756.250000_0.000000_0.100000': [0, 0, -1.2713302247917455, -110.76992423659013, 0, 0, -278.4837352504061, 0], '0.000826_137.500000_1.502630_756.250000_0.000000_0.100000': [0, 658.1078650465462, -1.2325117645299544, 0, 1.10440187457516, 0, 0, 0], '0.000909_137.500000_1.502630_756.250000_0.000000_0.100000': [0, 0, -1.3774631124666485, 0, 0, 479.325277964429, 0, 0], '0.001000_137.500000_1.502630_756.250000_0.000000_0.100000': [0, 0, -1.541696334208791, 0, 0, 0, -307.5803198793581, 0], '0.001100_137.500000_1.502630_756.250000_0.000000_0.100000': [0, 0, 0, 0, 1.6958659676296701, 0, -308.2050676548006, 0], '0.001100_151.250000_1.502630_756.250000_0.000000_0.100000': [0, 0, -2.4113694467123423, 0, 0, 342.40743831902074, 0, 0], '0.001100_151.250000_1.366027_756.250000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -439.1264726570889, 0], '0.001100_166.375000_1.366027_756.250000_0.000000_0.100000': [0, 0, 0, 0, 0, 882.2231464027187, 0, 0], '0.001100_166.375000_1.241843_756.250000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -418.4263796553696], '0.001100_166.375000_1.241843_831.875000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001000_103.305785_2.000000_756.250000_0.000000_0.100000': [0, 33.26914406718015, 0, 0, 5.596741209233733, 1073.3999163564963, 153.09332398949553, -124.65184871363863], '0.001000_103.305785_1.818182_568.181818_0.000000_0.100000': [1008.1757345997703, 4198.787991607309, -253.74951643309404, -1608.0243260137395, 639.5340943815818, 159.0224264802987, -1189.3346423008513, 722.8240858620338], '0.000909_103.305785_1.818182_568.181818_0.000000_0.100000': [309.60876246970486, 529.1522554237247, 0, 464.47399729927474, 2.363490631754394, 2361.63018541796, 289.31418117648, -468.43191740056756], '0.000909_103.305785_2.200000_687.500000_0.000000_0.100000': [0, 33.825115070229685, -5.492297852858155, 0, 4.53853249719923, 1640.7003376984667, 0, 0], '0.000909_103.305785_2.420000_625.000000_0.000000_0.100000': [0, 0, -2.6708678018298087, 0, 469.04575332028065, 1866.0001875872135, -232.0619724675101, -536.5490739212939], '0.000826_103.305785_2.420000_625.000000_0.000000_0.100000': [0, 575.2122704754305, 0, 0, 2.1916922546681237, 0, 0, -536.0765722363697], '0.000826_103.305785_2.420000_568.181818_0.000000_0.100000': [271.86179853096127, 0, -2.3959786112109214, -506.6907480158238, 0, 1038.7343064596425, -28.723180869974932, -5.182092526805684], '0.000826_103.305785_2.662000_568.181818_0.000000_0.100000': [261.6939851717642, 596.2851782719281, 0, 0, 0, 0, 0, 0], '0.000826_93.914350_2.662000_568.181818_0.000000_0.100000': [235.83905484366642, 0, 0, 0, 3.1363135335648624, 0, 0, 0], '0.000751_85.376682_2.662000_568.181818_0.000000_0.100000': [0, 0, 0, -339.5356509193611, 0, 0, 0, 0], '0.000751_85.376682_2.928200_568.181818_0.000000_0.100000': [0, 525.5474213440987, 0, 0, 0, 0, 0, 0], '0.000751_85.376682_2.928200_516.528926_0.000000_0.100000': [0, 0, 0, 0, 1.8799222648847718, 0, 0, 0], '0.000683_85.376682_2.928200_516.528926_0.000000_0.100000': [0, 2561.352612637302, 0, 0, 0, 0, -197.23771632523494, -527.52081103346], '0.000683_93.914350_2.928200_516.528926_0.000000_0.100000': [197.23771632523494, 0, 0, 0, 0, 0, 0, 0], '0.000683_85.376682_2.928200_568.181818_0.000000_0.100000': [0, 527.52081103346, 0, 0, 0, 0, 0, 0], '0.000683_85.376682_2.928200_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -2218.9865466948113, 0], '0.000683_93.914350_2.928200_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000909_125.000000_2.000000_568.181818_0.000000_0.100000': [5055.921384588502, 711.9771529674715, -9658.456866330165, -714.3678958697208, 1861.2726611926132, 1299.0881584745766, 1767.2863073330234, 297.301950026129], '0.000909_137.500000_2.000000_568.181818_0.000000_0.100000': [0, 2614.7441566333623, 1607.811746238233, -244.8242607737539, 5.7040309327175605, 0, 0, 339.78152206297693], '0.000909_137.500000_2.000000_516.528926_0.000000_0.100000': [0, 2098.2888577457766, 0, -1352.2982488593489, 0, 0, -480.3986136672731, 0], '0.000909_151.250000_2.000000_516.528926_0.000000_0.100000': [908.1481116835457, 459.7863554454003, 0, -1066.388060666554, 5.169445457560687, 1762.0972898072973, -482.7755546611803, -470.8778470649273], '0.000909_151.250000_2.200000_516.528926_0.000000_0.100000': [0, 503.8788722804529, 0, -333.08710263913963, 0, 1066.388060666554, 0, -568.9796916204414], '0.000909_151.250000_2.200000_568.181818_0.000000_0.100000': [760.2841587397766, 0, 0, -25.70743209074657, 4.2743916662418995, 0, -331.75941717662647, -524.8427256520858], '0.000909_151.250000_2.420000_625.000000_0.000000_0.100000': [226.02486137162305, 0, 0, 0, 0, 236.21001461656144, -236.67171482679896, 11815.022899688875], '0.000909_137.500000_2.420000_625.000000_0.000000_0.100000': [255.87162664417193, 0, -2.0705774179212315, 0, 0, 0, 0, 0], '0.001000_137.500000_2.420000_625.000000_0.000000_0.100000': [689.7882806666245, 0, 0, -417.8477690265396, 2.0705774179212315, 0, -225.94945513115363, 0], '0.001100_103.305785_2.200000_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 1897.9477191240749, -912.2262118590238, -574.3164582988778], '0.001100_103.305785_2.000000_687.500000_0.000000_0.100000': [-597.2864926739628, -249.60765886091048, -491.5575594662254, -476.9072743478445, -104.11065324621563, -40.458108242641174, -239.8003163963727, -2019.0644890294432], '0.001100_113.636364_2.000000_687.500000_0.000000_0.100000': [368.66618995334443, 175.97114520688842, 184.29016108675592, -1698.1098915997813, 842.1958828345178, 3943.0939808475946, -882.335703861094, -354.65862123519634], '0.001210_103.305785_2.000000_687.500000_0.000000_0.100000': [-1132.4364418189746, -832.8800714117392, -6.760157786958786, -3207.1587343185106, -969.3341702051779, -608.35578151642, -2338.4549854223583, -2020.353364064611], '0.001210_93.914350_2.000000_687.500000_0.000000_0.100000': [583.8518756505752, 614.1711159009977, 0, 0, 983.0228597662345, 799.6920310130164, 0, 3604.765483909961], '0.001100_93.914350_2.000000_687.500000_0.000000_0.100000': [0, 0, -6.759513636393058, -1249.56953224327, 6.143094906231454, 0, 1678.0885872607737, 0], '0.001210_93.914350_1.818182_687.500000_0.000000_0.100000': [0, 0, 0, -726.9927554663785, 0, 1188.2976446400942, 0, 0], '0.001210_85.376682_1.652893_687.500000_0.000000_0.100000': [0, 484.46203736362713, 0, 465.8570088690614, 0, 0, 668.9275255693949, 0], '0.001210_85.376682_1.652893_625.000000_0.000000_0.100000': [0, 0, 0, 0, 2.9525321258470285, 0, -255.5811778414016, -484.46203736362713], '0.001100_85.376682_1.652893_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 1526.5772877653394, 0, 0], '0.001100_85.376682_1.502630_625.000000_0.000000_0.100000': [0, 0, 0, 0, 4.8834308848734835, 2072.2101639290704, 0.19161981905597258, 0], '0.001000_85.376682_1.502630_625.000000_0.000000_0.100000': [5043.928010511165, 0, 0, 0, 3.9906863434704065, 0, -337.13217919000954, 0], '0.001000_93.914350_1.502630_625.000000_0.000000_0.100000': [0, 482.4818794075592, 0, 0, 4.1287098804376, 0, 0, 0], '0.000909_93.914350_1.502630_625.000000_0.000000_0.100000': [0, 0, 0, 0, 3.7819442211339265, 0, 0, 0], '0.000826_93.914350_1.502630_625.000000_0.000000_0.100000': [779.4749958807375, 0, 0, 0, 3.447495366906878, 0, -341.5776085998072, 0], '0.000826_103.305785_1.502630_625.000000_0.000000_0.100000': [0, 0, 0, 0, 2.5762267220168726, 0, 0, 0], '0.000751_103.305785_1.502630_625.000000_0.000000_0.100000': [0, 488.1349326379869, 0, 0, 0, 0, 0, 0], '0.000751_103.305785_1.502630_568.181818_0.000000_0.100000': [0, 472.8273050997942, 0, 0, 2.840095014418512, 0, 0, -44.37590296708969], '0.000751_103.305785_1.502630_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000909_125.000000_2.662000_625.000000_0.000000_0.100000': [0, 532.8349387629216, 0, 0, 0, 0, 0, 0], '0.000909_125.000000_2.662000_568.181818_0.000000_0.100000': [239.3185427591994, 0, -4.087600168309836, -48.941680430848464, 3.71027629935852, 0, 0, 0], '0.000909_125.000000_2.928200_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 48.941680430848464, 0, 0], '0.001000_125.000000_2.662000_568.181818_0.000000_0.100000': [263.28823274996495, 0, -4.504954042345708, 0, 0, 0, 0, 0], '0.001100_125.000000_2.662000_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -277.12627932085377, 0], '0.001100_137.500000_2.662000_568.181818_0.000000_0.100000': [25.193298120077607, 549.6885423657938, -4.094692099790336, 0, 0, 0, 0, 0], '0.001210_137.500000_2.662000_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 403.8039299880482, 0, 0], '0.001210_137.500000_2.420000_568.181818_0.000000_0.100000': [0, 0, 0, 0, 2.5197233829912875, 0, 0, -477.5408583796734], '0.001100_137.500000_2.420000_568.181818_0.000000_0.100000': [260.38613776409983, 0, -2.5197233829912875, -402.22896127124915, 0, 371.21807389021177, 0, -477.5580306924232], '0.001100_137.500000_2.662000_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 316.970158656855, 0, 0], '0.001100_137.500000_2.420000_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 417.05952003079784, 0, -464.4297397513996], '0.001100_137.500000_2.200000_516.528926_0.000000_0.100000': [2313.8800842135274, 184.26408009136117, 0, -417.05952003079784, 0, 0, 0, -510.2711858919856], '0.001210_137.500000_2.420000_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 363.8195364703608, 0, 0], '0.001210_137.500000_2.200000_625.000000_0.000000_0.100000': [401.8665118957402, 486.8821768862637, 0, 0, 0, 0, 0, -461.86334454671635], '0.001100_137.500000_2.000000_516.528926_0.000000_0.100000': [457.61823236632625, 522.8512166880672, 0, -477.8950292151428, 0, 0, 0, -494.60344100079584], '0.001100_125.000000_2.000000_469.571751_0.000000_0.100000': [981.3922910672518, 606.1469763628764, -373.7475373994339, -1383.677160456759, 479.0415049064339, 1203.9070186642457, -1501.091386605684, 0], '0.001100_125.000000_2.200000_469.571751_0.000000_0.100000': [407.22137352853497, 2730.1611782816567, 0, -599.6386439831547, 0, 144.2524996742261, -614.8696479133762, 120.1445002886412], '0.001100_113.636364_2.200000_469.571751_0.000000_0.100000': [0, 890.9717040009937, 0, 0, 0, 2086.321591516452, 300.18114449388673, 0], '0.001100_113.636364_2.200000_426.883410_0.000000_0.100000': [1857.7658354855093, 0, -6.70956186655385, -860.7940362849843, 6.097817304614, 0, 0, 0], '0.001100_113.636364_2.420000_426.883410_0.000000_0.100000': [0, 0, 0, 0, 2.602685018479679, 2281.090738729764, 0, 0], '0.001100_103.305785_2.200000_426.883410_0.000000_0.100000': [474.2025698799383, 0, 2331.618433195709, 0, 0, 0, 0, -193.39928761748752], '0.001210_103.305785_2.200000_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, 3052.863540125903, 0, 0], '0.001210_103.305785_2.000000_426.883410_0.000000_0.100000': [0, 0, -6.825301362884966, 0, 692.4255272699531, 0, 0, 0], '0.001331_103.305785_2.000000_426.883410_0.000000_0.100000': [0, 0, 0, -1249.0399580237768, 0, 446.0630798377784, -429.3362817687574, 0], '0.001331_103.305785_2.200000_426.883410_0.000000_0.100000': [0, 0, -8.608080536787929, 0, 0, 0, -429.791769578948, -521.6820986516647], '0.001464_103.305785_2.200000_426.883410_0.000000_0.100000': [0, 0, 0, 0, 8.608080536787929, 0, 0, 0], '0.001331_113.636364_2.200000_426.883410_0.000000_0.100000': [429.791769578948, 0, -8.12438926665327, -774.5373069685177, 0, 0, 0, 0], '0.001464_113.636364_2.200000_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -519.9632140870584], '0.001464_113.636364_2.200000_469.571751_0.000000_0.100000': [0, 0, 0, -700.1464598566718, 0, 0, -406.02881626946186, 0], '0.001464_113.636364_2.420000_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 700.1464598566718, 0, 0], '0.001464_125.000000_2.200000_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001000_137.500000_2.000000_516.528926_0.000000_0.100000': [2269.3699626462185, 2019.9388467099402, 420.8866369154078, -1306.997893527764, 0, 2126.7044045082785, -528.401930139477, 1630.9412355737727], '0.001000_137.500000_1.818182_516.528926_0.000000_0.100000': [3086.927963510506, -196.02069315032873, 718.9188359054546, -349.05414609621204, 2437.2936922396684, 1583.4874263736106, 599.8301193039094, -478.1471756359795], '0.001000_137.500000_1.652893_516.528926_0.000000_0.100000': [1942.6958515333072, 139.92222753732972, 0, 149.2735910905305, 0, -144.96970475001527, -355.8155660283925, -448.92130684743444], '0.001000_125.000000_1.652893_516.528926_0.000000_0.100000': [548.3245589805526, 455.21703749621895, -43.03739859297929, 458.0948319426918, 0, -131.76091659498124, -313.5996075440742, 1387.346104617799], '0.001000_125.000000_1.652893_469.571751_0.000000_0.100000': [852.8908469537463, 381.54321448977055, 0, -270.187484295517, 325.0716334364187, -43.94255099106476, 427.3958404775629, 0], '0.001000_137.500000_1.652893_469.571751_0.000000_0.100000': [1036.3046441669812, 0, 0, 0, 2.8066160777905296, 914.6188554048714, -393.6806123136223, 0], '0.000909_137.500000_1.652893_469.571751_0.000000_0.100000': [0, 409.6412022433098, -2.5284829529644415, 0, 0, 0, -357.56412275535695, 0], '0.000909_137.500000_1.652893_426.883410_0.000000_0.100000': [417.813800095474, 0, 0, 0, 0, 0, 0, 0], '0.000909_125.000000_1.652893_426.883410_0.000000_0.100000': [476.3501667306439, 396.1599393742143, 0, 683.8676039919579, 0, -37.831472028501956, 0, 0], '0.000909_125.000000_1.502630_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, 856.2110253866351, 0, -483.64504525782087], '0.000909_125.000000_1.502630_469.571751_0.000000_0.100000': [0, 0, 0, 0, 3.4336291311357856, 0, 0, 0], '0.000826_125.000000_1.502630_469.571751_0.000000_0.100000': [388.74855414608646, 532.7367227101097, 0, 0, 0, 0, 0, 0], '0.000826_125.000000_1.502630_426.883410_0.000000_0.100000': [405.85515457041595, 0, 0, 0, 0, 0, -425.77072953990773, -48.43061115546448], '0.000826_113.636364_1.502630_426.883410_0.000000_0.100000': [466.40056137595525, 0, 0, 0, 0, 494.26704058082663, -405.85515457041595, 0], '0.000826_137.500000_1.502630_426.883410_0.000000_0.100000': [0, 0, 0, 132.69155465234476, 0, 0, 0, 0], '0.000826_137.500000_1.652893_426.883410_0.000000_0.100000': [0, 386.9671296687303, 748.3366965039963, 0, 0, 0, 0, 0], '0.000826_137.500000_1.652893_388.075827_0.000000_0.100000': [0, 0, 0, 0, 1.979601714443288, 0, 0, 0], '0.000751_137.500000_1.652893_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -387.0677735076558], '0.000751_137.500000_1.652893_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, -119.55669724048239, 0, 0], '0.000751_137.500000_1.502630_426.883410_0.000000_0.100000': [426.5350905234084, 0, 0, 0, 0, 0, 0, 0], '0.000751_125.000000_1.502630_426.883410_0.000000_0.100000': [0, 0, 0, 38.67836680068929, 0, 0, 0, 0], '0.000751_125.000000_1.652893_426.883410_0.000000_0.100000': [0, 396.18356878714377, 354.5689441254207, 0, 1.7897133683515585, -3.3830959875406434, 0, 0], '0.000683_125.000000_1.652893_426.883410_0.000000_0.100000': [338.0563390635691, 0, 0, 0, 0, -32.08692156872285, -340.74197807546943, -417.9738717409909], '0.000683_125.000000_1.652893_469.571751_0.000000_0.100000': [298.5146237593637, 740.9605216471609, 0, 0, 0, -99.55312972610227, 0, -438.5807044748448], '0.001210_125.000000_1.818182_625.000000_0.000000_0.100000': [646.0341541802035, 469.9980377866136, 142.6355053660593, 0, 180.61999956668436, 608.1926902923358, -1270.1522737998926, -164.75512762783515], '0.001210_113.636364_1.818182_625.000000_0.000000_0.100000': [733.8846752719842, 953.4278362578812, -4.343333588490569, 0, 0, 215.11755192613907, -1061.966937443451, -499.894226914617], '0.001331_113.636364_1.818182_625.000000_0.000000_0.100000': [0, 0, 0, -798.9054049925044, 0, 462.48913706721663, 0, 0], '0.001331_113.636364_2.000000_625.000000_0.000000_0.100000': [367.53475337010514, 1260.241236080153, 0, 0, 8.274816795947153, 0, -389.956443399262, 211.17009138932335], '0.001210_113.636364_2.000000_568.181818_0.000000_0.100000': [177.07225885827756, 1049.256842744713, -8.243545515262158, -1261.8563778831312, 0, 0, 0, -1015.7384668293807], '0.001331_113.636364_2.000000_516.528926_0.000000_0.100000': [0, 0, -8.984904914874388, 0, 646.5147051099525, 0, 0, -157.60397953990002], '0.001210_113.636364_2.000000_516.528926_0.000000_0.100000': [0, 154.50306948664536, -943.1999812083307, -25.54297167740563, 1545.4561450582526, 0, 1464.6356538718774, -306.82266804514336], '0.001210_113.636364_2.000000_469.571751_0.000000_0.100000': [0, 487.44407861853614, 0, 0, 0, 553.93526458762, -989.9924507598985, -520.3611675808139], '0.001210_113.636364_2.000000_426.883410_0.000000_0.100000': [429.0483147328314, 458.82200166518163, 0, -1249.2256621254808, 0, 0, -464.3830430390044, 0], '0.001210_125.000000_2.000000_426.883410_0.000000_0.100000': [464.3830430390044, 447.01603526738364, -7.353427096403903, 0, 0, 0, 0, -460.4176514049154], '0.001210_125.000000_2.000000_469.571751_0.000000_0.100000': [0, 472.5083212784113, -1.5675794860853145, -1327.2465699153522, 486.5086514782419, 1087.2962467181878, 486.17224627601126, 1323.2257044694338], '0.001331_125.000000_2.000000_469.571751_0.000000_0.100000': [0, 0, 0, -1239.05429727988, -778.1239291219342, 0, 0, -496.0685252346646], '0.001210_125.000000_1.818182_469.571751_0.000000_0.100000': [30.653993930621027, 61.43082248000275, -3.80388156100762, -647.1372243850643, 276.13173929943207, 1299.7523363387552, 0, -1112.2956062129126], '0.001100_125.000000_2.200000_426.883410_0.000000_0.100000': [465.80856821586553, 0, 0, -704.5429885791442, 5.803458966329527, 2040.5946357878718, 0, 203.522968067593], '0.001100_125.000000_2.420000_426.883410_0.000000_0.100000': [0, 0, -2.5152510546289704, 0, 0, 0, -300.3281567565602, 0], '0.001210_125.000000_2.420000_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, 700.6676562516177, 0, -428.10961737756935], '0.001210_125.000000_2.200000_426.883410_0.000000_0.100000': [426.6998875688813, 0, 0, 0, 0, 0, 0, 0], '0.001210_113.636364_2.200000_426.883410_0.000000_0.100000': [1553.4701277133934, 0, -7.3830521072974085, -778.717364198178, 0, 1249.2256621254808, 0, 0], '0.001331_113.636364_2.420000_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001000_103.305785_2.000000_568.181818_0.000000_0.100000': [591.6687674444308, 1188.4693095203545, 311.1297914830092, -2063.167177145131, 1184.7851836630812, 481.63785320205136, 827.3412743935169, -576.6233319645055], '0.000826_103.305785_2.000000_568.181818_0.000000_0.100000': [0, 553.7585706937547, 0, 0, 0, 2098.9818558913084, -424.74700088355723, 336.9983219446324], '0.000826_103.305785_1.818182_568.181818_0.000000_0.100000': [0, 3709.2291069681132, 1069.6698454799916, -117.57434666226078, 256.1492860031648, 177.77136142366143, -309.24867582770395, -497.4878798159351], '0.000826_113.636364_1.818182_568.181818_0.000000_0.100000': [1956.394598687063, 1233.7908505198689, 117.69584349702814, -695.0188704540509, 0, 854.1136153003154, 126.24780521439695, 938.2825825950241], '0.000909_113.636364_1.818182_568.181818_0.000000_0.100000': [-140.56700521306948, 2803.170625608439, -2.3964702547045817, 1298.500512510525, 46.87163886802375, 3331.0726441740067, 554.5981750094483, -148.67399869706117], '0.000909_113.636364_1.652893_568.181818_0.000000_0.100000': [128.43982100168665, 2570.9776985248513, 633.4170269962511, 387.31714402533856, 2.0972234056240637, 349.30316617847035, -264.47298087940777, 637.7807041379089], '0.000909_103.305785_1.652893_568.181818_0.000000_0.100000': [0, 2293.390450856633, -354.1639828199458, -34.759765875507355, 1.8612909789493643, 18.923268228895722, 424.65458968759435, 0], '0.000826_103.305785_1.652893_568.181818_0.000000_0.100000': [2065.289026109577, 0, -1.8612909789493643, 0, 1.676710873906268, 95.17423220810497, -258.69436131579124, 0], '0.000751_103.305785_1.818182_568.181818_0.000000_0.100000': [4019.3535786952903, 0, -424.1189942009195, 0, 0, 0, -562.4143437277118, 0], '0.000826_103.305785_2.000000_516.528926_0.000000_0.100000': [373.9881530878892, 0, 91.78441433174555, 0, 1431.3775157381444, 0, 0, 0], '0.000909_103.305785_2.000000_516.528926_0.000000_0.100000': [0, -151.10933538523727, -9.074986543402376, -372.9202815872206, 1054.8620745077878, 0, 0, -553.8519619364836], '0.000909_93.914350_1.818182_568.181818_0.000000_0.100000': [4898.411039678978, 3515.4103798933643, -2.924442248098785, 0, 0, 0, 0, -500.0731918269301], '0.001000_93.914350_1.818182_568.181818_0.000000_0.100000': [0, 1237.8590837356523, 0, 0, 6169.933831826875, 907.0562093851655, 719.3699662451552, 0], '0.001000_93.914350_1.652893_568.181818_0.000000_0.100000': [1688.6291704264702, 490.58211836500163, 0, 447.30177142150274, 0, 0, 138.3332596271475, 0], '0.000909_93.914350_1.818182_625.000000_0.000000_0.100000': [1086.665284930785, 0, 884.8307266318606, 881.3284775540344, 0, 0, -249.8212369097344, 0], '0.000909_103.305785_1.818182_625.000000_0.000000_0.100000': [363.7123384586812, 1824.7932825589644, 180.71256804035613, -2444.7970923836247, 252.19126271992906, 916.0361312114641, 0, 476.1726562116011], '0.001100_137.500000_2.000000_625.000000_0.000000_0.100000': [587.1834328302807, 3211.838335735553, -310.89669553851695, -1413.9998069134851, 98.3787208370727, 125.68950649679482, -877.8305701196496, 615.1191227551413], '0.000909_125.000000_2.200000_687.500000_0.000000_0.100000': [-121.82970751910614, 86.97183570936727, 246.3110715775634, 0, 4.037049956334631, 670.0195929226044, -331.7672927183641, 0], '0.000826_125.000000_2.200000_687.500000_0.000000_0.100000': [0, 73.37125166321694, -3.670045414849665, -462.1593335198433, 0, 0, 0, 0], '0.000751_125.000000_2.420000_687.500000_0.000000_0.100000': [0, 0, 0, 0, 2.055366144956861, 463.2538181247197, -245.6001347163885, 0], '0.000683_125.000000_2.420000_687.500000_0.000000_0.100000': [0, 0, -2.055366144956861, 0, 0, 0, 0, 0], '0.000751_125.000000_2.200000_687.500000_0.000000_0.100000': [335.12132216212177, 0, 0, 0, -444.5042120895297, 0, -327.05745541544366, 0], '0.000751_125.000000_2.200000_756.250000_0.000000_0.100000': [283.66312191345224, 0, 0, 0, 0, 0, 0, 0], '0.000751_113.636364_2.200000_756.250000_0.000000_0.100000': [287.0034499352716, 599.4536683650662, 0, 0, 0, 0, 0, -530.1267376806089], '0.000751_103.305785_2.200000_756.250000_0.000000_0.100000': [0, 0, 0, 0, 3.463803439654839, 0, -287.0034499352716, 0], '0.000751_113.636364_2.200000_831.875000_0.000000_0.100000': [0, 530.1267376806089, 0, 0, 0, 0, 0, 0], '0.000751_113.636364_2.200000_687.500000_0.000000_0.100000': [0, 1593.9058689956387, 0, 0, 3.963998324225195, 0, 0, -54.4957880331878], '0.000751_113.636364_2.200000_625.000000_0.000000_0.100000': [349.33330699134586, 0, 0, -585.822050027126, 0, 2033.8833657882308, 0, -55.0317179586965], '0.000751_103.305785_2.200000_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -574.1912543104733], '0.000751_103.305785_2.200000_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -325.45923226783816, 0], '0.000683_113.636364_2.200000_687.500000_0.000000_0.100000': [325.60531357692946, 0, 0, 0, 0, 1215.342420527337, 0, 0], '0.000683_113.636364_2.000000_687.500000_0.000000_0.100000': [1750.456203522214, 0, 0, 0, 0, 0, 0, 0], '0.000909_113.636364_2.000000_756.250000_0.000000_0.100000': [652.9115332354304, 2533.2035870894924, -5.593099802420148, 0, 0, 0, 531.6515227527037, -597.1595373548053], '0.001000_113.636364_2.000000_831.875000_0.000000_0.100000': [356.72674060506677, 0, -6.676174857822861, -1209.619602776087, 0, 0, 0, 0], '0.001000_113.636364_2.200000_831.875000_0.000000_0.100000': [0, 0, 0, 0, 3.616047245009213, 1094.1918594582269, 0, 0], '0.001100_125.000000_1.818182_687.500000_0.000000_0.100000': [206.76893511663025, 737.5030642772715, 67.9594564693856, -926.57644433925, 841.8987211362173, 1726.1194003109213, -335.14527538561146, -32.72724336259532], '0.001100_113.636364_1.818182_687.500000_0.000000_0.100000': [218.05070538056344, 496.1590820145758, -280.8199498335698, -1012.8774234629309, 384.8077618562664, 1183.3745171462695, 1561.8624659265754, -573.9244550414622], '0.001100_113.636364_1.818182_756.250000_0.000000_0.100000': [0, 1208.1793844949386, -4.536533367338052, 0, 4.520330017183987, 1101.0169474227946, 0, -546.9390596792103], '0.001100_113.636364_1.818182_831.875000_0.000000_0.100000': [0, 0, 0, 0, 0, 689.7540383595424, 0, -564.6180221481945], '0.001100_113.636364_1.818182_915.062500_0.000000_0.100000': [0, 564.6180221481945, 0, 0, 4.702009782540154, 0, 0, 0], '0.001000_113.636364_1.818182_915.062500_0.000000_0.100000': [0, 0, -4.702009782540154, 0, 0, 342.12090499408697, 0, 0], '0.001000_113.636364_1.652893_915.062500_0.000000_0.100000': [0, 524.0039869318965, 0, 0, 0, 0, 0, 0], '0.001000_113.636364_1.652893_831.875000_0.000000_0.100000': [0, 0, 0, 0, 3.507083293662745, 0, -28.377289052727253, 0], '0.000909_113.636364_1.652893_831.875000_0.000000_0.100000': [0, 0, 0, 0, 0, 176.6309218069144, -283.46085732107366, 0], '0.000909_113.636364_1.502630_831.875000_0.000000_0.100000': [0, 441.91407197829705, -1.7490546826994977, 0, 0, 0, 0, 0], '0.000909_113.636364_1.502630_756.250000_0.000000_0.100000': [274.2494417813827, 0, 0, 0, 0, 740.7472519321349, 0, 0], '0.000909_103.305785_1.502630_756.250000_0.000000_0.100000': [298.5717275198993, 0, 0, 0, 0, 0, 0, 0], '0.000909_93.914350_1.502630_756.250000_0.000000_0.100000': [0, 482.7317537815687, 0, 0, 0, 0, 0, -480.3350747680683], '0.000909_93.914350_1.502630_831.875000_0.000000_0.100000': [0, 0, 0, -146.97472677009372, 0, 0, 0, 0], '0.000909_93.914350_1.652893_831.875000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -256.9825124966936, -557.5208913327133], '0.000909_93.914350_1.652893_915.062500_0.000000_0.100000': [0, 557.5208913327133, 0, 0, 0, 0, 0, 0], '0.000909_103.305785_1.652893_831.875000_0.000000_0.100000': [0, 0, 0, -304.06019589685314, 0, 0, 0, 0], '0.000909_103.305785_1.818182_831.875000_0.000000_0.100000': [0, 0, 0, -937.368955578454, 0, 0, 0, 0], '0.000909_103.305785_2.000000_831.875000_0.000000_0.100000': [0, 0, 0, -1154.6453405976997, 0, 0, 0, 0], '0.000909_103.305785_2.200000_831.875000_0.000000_0.100000': [239.56350794802893, 877.8017350070093, 0, 0, 0, 0, 0, 0], '0.001000_125.000000_1.818182_568.181818_0.000000_0.100000': [4081.7067642715424, 793.059031916174, 225.4565219933321, -8562.882190799291, 754.7764618593903, 814.0755103048984, 402.69521187220107, -128.01549394314452], '0.001210_113.636364_1.818182_516.528926_0.000000_0.100000': [347.30933516160167, 894.8232409295508, -3.6304639600806694, -615.5001181588294, 0, 22.033321300445355, -96.63629311988167, -446.9724629757854], '0.001210_103.305785_1.818182_516.528926_0.000000_0.100000': [279.2913386431795, 771.9607672314627, -4.480859666780604, 0, 3.6372327439589753, 1199.2290598599927, 0, -273.4655808745739], '0.001210_103.305785_1.818182_469.571751_0.000000_0.100000': [353.6733185231461, 0, 261.9728706058999, -451.2137371619689, 2.967872986856219, 166.5389416054176, -315.4437056065678, -0.45495282999949044], '0.001210_103.305785_2.000000_469.571751_0.000000_0.100000': [385.0017044751592, 0, 373.6432323032534, 0, 6.422070512416667, 496.3351108781657, 38.55998179341714, 0], '0.001210_113.636364_1.818182_469.571751_0.000000_0.100000': [346.98807616722456, 1565.017797500615, 0, -999.4927152312866, 0, 270.8193672781833, -356.24532590143536, 725.4822400048472], '0.001210_113.636364_1.818182_426.883410_0.000000_0.100000': [1194.1240430653752, 0, -3.444667423192823, 0, 146.26148096420857, 216.7285690810214, -361.3032302685415, -129.9076508765553], '0.001210_103.305785_1.818182_426.883410_0.000000_0.100000': [0, 1047.2234227607776, 0, 0, 4.24197943352865, 415.6889392228617, 0, 0], '0.001100_103.305785_1.818182_426.883410_0.000000_0.100000': [590.630724233866, 2022.3618879929743, 66.65624361087134, 0, 0, 256.8276876069275, -745.6743373157608, -45.62852681240582], '0.001100_103.305785_1.818182_469.571751_0.000000_0.100000': [0, 2442.6423368625333, 515.0102627009533, -447.7595396364084, 0, -74.90896846445055, -318.0405793302425, 0], '0.001100_113.636364_1.818182_426.883410_0.000000_0.100000': [777.5223144577785, 908.3744179061626, 72.90403841303464, -53.60837213948736, 134.15355423179398, 237.86641679825738, -392.63811658561417, 705.0012648715544], '0.001100_113.636364_1.818182_469.571751_0.000000_0.100000': [0, 470.21721394822305, 454.0458885963207, -568.5051697581189, -531.9366966829841, 3413.2000187398835, 0, 0], '0.001100_125.000000_1.818182_469.571751_0.000000_0.100000': [414.89368199566604, 855.6806542108304, -339.16636766696337, -643.8044217929421, -642.5909242342556, 358.7861580879129, 0, -459.5339489562108], '0.000909_151.250000_2.000000_687.500000_0.000000_0.100000': [1377.5959911733835, 1025.3571439996183, -5.580986632873464, -1035.3100251025905, 0, 781.1413919238312, -421.01208294410884, -510.67484555004705], '0.000909_151.250000_2.000000_756.250000_0.000000_0.100000': [377.2569018210905, 510.67484555004705, -5.48985056720303, 0, 0, 0, 0, 0], '0.000909_151.250000_2.200000_687.500000_0.000000_0.100000': [0, 0, -3.0253717219107785, 0, 0, 0, 0, 0], '0.001000_151.250000_2.200000_687.500000_0.000000_0.100000': [355.7970945556244, 0, -3.3126771451381503, 0, 3.0253717219107785, 0, 0, 0], '0.001100_151.250000_2.200000_687.500000_0.000000_0.100000': [0, 0, 0, 0, 3.3126771451381503, 0, -308.74612493176977, 0], '0.000826_125.000000_2.000000_625.000000_0.000000_0.100000': [0, 2393.642779361424, 2201.9740287520876, -1015.3903774848229, 108.45392803415685, 0, 86.26708885413609, -55.11352174979254], '0.000826_125.000000_2.200000_625.000000_0.000000_0.100000': [3412.552480925766, 896.6537926062788, 0, -293.5600276425885, 4.030360436721094, 0, 40.3146805390806, -514.7028104495254], '0.000826_137.500000_2.200000_625.000000_0.000000_0.100000': [0, 537.988196214726, 0, 0, 3.550466242924267, 0, 0, 0], '0.000751_137.500000_2.200000_625.000000_0.000000_0.100000': [741.8666337848902, 0, 0, -578.4143059829571, 0, 0, 0, 0], '0.000751_125.000000_2.200000_625.000000_0.000000_0.100000': [2171.453925890515, 0, 0, 0, 0, 0, -37.42941192701228, -515.369560850953], '0.000751_137.500000_2.200000_687.500000_0.000000_0.100000': [327.05745541544366, 0, 0, 0, 0, 0, 0, -463.1693281616168], '0.000751_137.500000_2.200000_756.250000_0.000000_0.100000': [0, 463.1693281616168, 0, 12141.874212812814, 2.166500169229221, 0, 0, 0], '0.000683_137.500000_2.200000_756.250000_0.000000_0.100000': [0, 0, -2.166500169229221, 0, 0, 0, 0, 0], '0.000683_103.305785_2.200000_687.500000_0.000000_0.100000': [339.9391938695346, 0, 0, 0, 0, 0, 0, 0], '0.000683_93.914350_2.200000_687.500000_0.000000_0.100000': [0, 1798.6243104382565, 0, 0, 0, 0, -30.90356307904861, 0], '0.000683_93.914350_2.200000_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 1930.2807417705976, 0, 0], '0.000683_93.914350_2.000000_625.000000_0.000000_0.100000': [0, 655.4424951543838, 0, 0, 0, 0, -343.7815391735225, -613.8893522005816], '0.000683_103.305785_2.000000_625.000000_0.000000_0.100000': [343.7815391735225, 0, 0, 0, 0, 0, -367.2084963085652, 0], '0.000683_113.636364_2.000000_625.000000_0.000000_0.100000': [0, 623.7416860838931, -4.241009591685361, 0, 0, 0, 0, 0], '0.000751_113.636364_2.000000_625.000000_0.000000_0.100000': [0, 622.7384399150801, 0, 0, 0, 1368.9483400085473, 0, -264.1418520815299], '0.000751_113.636364_2.000000_568.181818_0.000000_0.100000': [386.44586414682016, 536.8051267998177, 0, 0, -496.8311917029571, 1138.1221669574247, 53.79237514922806, -612.9867101554262], '0.000751_113.636364_1.818182_568.181818_0.000000_0.100000': [586.6852074160769, 0, -346.452553936388, -692.3251191515186, -98.64464275435968, 0, -345.7096694627288, 0], '0.000751_125.000000_1.818182_568.181818_0.000000_0.100000': [0, 437.2496904270174, 0, -837.7455370733487, 474.2691414624728, 237.46586742526617, -351.2765941082087, 0], '0.000683_125.000000_1.818182_568.181818_0.000000_0.100000': [0, 863.4841420943719, 0, 0, 1.9767017960486555, 0, 0, 0], '0.000683_125.000000_1.818182_516.528926_0.000000_0.100000': [322.6824301464676, 702.5209175689038, 0, 0, 0, 248.5334887401812, 0, 0], '0.000683_125.000000_1.652893_516.528926_0.000000_0.100000': [555.5822671179286, 0, 0, 0, 0, 0, 0, 0], '0.001000_151.250000_2.000000_625.000000_0.000000_0.100000': [3363.3086990925567, 611.3898903717297, 0, -741.0391849301373, 1211.860112804571, 1231.2770054806465, -1039.543299689933, -928.244565523245], '0.001100_137.500000_2.000000_687.500000_0.000000_0.100000': [-317.471116008845, 1755.7595077028643, -6.797187390331238, -1221.8285849723536, 305.3169399895179, 203.30527262911096, -404.2731549685497, -537.5849343803204], '0.001100_151.250000_2.000000_687.500000_0.000000_0.100000': [0, 0, 0, -1029.9280572375342, 0, 0, 0, 0], '0.001100_166.375000_2.200000_687.500000_0.000000_0.100000': [0, 421.7467064755354, 0, 0, 0, 0, 0, 0], '0.001100_166.375000_2.200000_625.000000_0.000000_0.100000': [0, 0, 0, -155.51786982201574, 0, 1896.2626064223682, 0, 0], '0.001100_166.375000_2.420000_625.000000_0.000000_0.100000': [236.96145267623123, 0, 0, 0, 0, 0, 0, 12056.196242392825], '0.001100_166.375000_2.420000_687.500000_0.000000_0.100000': [0, -12056.196242392825, 0, 0, 0, 0, 0, 0], '0.001100_151.250000_2.420000_625.000000_0.000000_0.100000': [0, 0, 0, 0, 209.2806163536695, 0, 0, 0], '0.000909_151.250000_2.420000_687.500000_0.000000_0.100000': [0, -11815.022899688875, 0, 0, 0, 0, 0, 0], '0.000909_166.375000_2.420000_625.000000_0.000000_0.100000': [0, 0, -2.1338903582010973, 0, 0, 0, 0, 0], '0.001000_166.375000_2.662000_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001000_125.000000_2.200000_568.181818_0.000000_0.100000': [696.8592543110254, 1278.3834946226298, 868.1385666760756, -88.867653889562, -372.56414916505605, -770.2659279420823, -782.7207698192968, -1058.870957762154], '0.001000_125.000000_1.818182_516.528926_0.000000_0.100000': [3990.1365797095636, 367.7114923908516, 411.1876380778328, -287.0295963177057, 75.79299892451354, 50.15518359984367, -493.13795452517115, 426.7571101250146], '0.001000_125.000000_2.000000_469.571751_0.000000_0.100000': [1796.4895625157508, -575.3962554046617, 0, -1958.3867152275445, 0, 2340.5421346936264, -478.1358310624235, -193.085777350731], '0.001000_125.000000_1.818182_469.571751_0.000000_0.100000': [967.9531928098272, 1960.6822041511004, 1702.1845405823356, 213.22972534452492, 2.954957449158428, 270.187484295517, -476.0011154773524, -487.8812361128336], '0.001000_137.500000_1.818182_469.571751_0.000000_0.100000': [369.25398934582654, 1504.3177676890575, 0, -157.9984538640051, 355.03161742970775, 305.9989591512205, -402.0493155910937, 274.2079142896294], '0.001000_151.250000_1.818182_469.571751_0.000000_0.100000': [429.50754115100574, 415.45812123210055, 336.635551109278, 0, 0.6552864780548409, 347.65361087921957, -14.199195387054601, 288.2751222302047], '0.001100_137.500000_2.000000_469.571751_0.000000_0.100000': [-749.9824833523268, 2137.1259514569488, 126.83273114193565, -227.3012425590597, -439.29389252739134, 0, 0, 0], '0.001100_137.500000_2.200000_469.571751_0.000000_0.100000': [0, 0, -5.961076455922376, 0, 5.4184739973974505, 2325.668819603994, 497.1183479623542, -145.92839406227301], '0.001210_137.500000_2.200000_469.571751_0.000000_0.100000': [0, 0, 0, 0, 6.557184101514613, 0, -379.11099898292304, 0], '0.001210_151.250000_2.200000_469.571751_0.000000_0.100000': [0, 0, 0, -401.12414979300576, 0, 0, -378.54387997652503, 0], '0.001210_151.250000_2.420000_469.571751_0.000000_0.100000': [0, 0, 0, -250.6700737021005, 0, 401.12414979300576, 0, -425.9938189906541], '0.001210_151.250000_2.420000_516.528926_0.000000_0.100000': [0, 425.9938189906541, 0, 0, 0, 0, -271.26195359533233, 0], '0.001210_166.375000_2.420000_516.528926_0.000000_0.100000': [0, 0, -2.2769606042616943, 0, 0, 0, 0, 0], '0.001331_166.375000_2.420000_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000909_125.000000_1.818182_625.000000_0.000000_0.100000': [21.418896963826228, 3540.5257106876006, -960.0376202634177, -2443.9869344935487, 1139.4625860155609, 249.985609888301, 94.75925916187191, 1635.8234297426104], '0.000909_125.000000_1.818182_568.181818_0.000000_0.100000': [-468.271041140609, -70.17477880489014, -342.15158093227893, 112.82091645385472, 64.02192725239411, 2766.1028788580284, -989.1897517768559, 210.76150382276296], '0.000909_113.636364_1.818182_625.000000_0.000000_0.100000': [-137.89720788444907, 878.5268558905846, 929.5996055158844, -1788.9349777932648, 778.5176349932206, 1421.669613972492, 434.43047444204126, 360.07355669119164], '0.000826_113.636364_2.000000_625.000000_0.000000_0.100000': [403.0078354235761, 127.10809887486153, 944.6449561411109, -1068.4146942152984, 1348.8247542339277, -824.8987440333974, 144.10536068420683, 142.35278062264763], '0.000826_113.636364_2.000000_687.500000_0.000000_0.100000': [0, 631.6998743056288, 1512.6436004418056, -12.139801569018983, 0, 1073.8225877930581, -190.7750199096088, -587.6285494434413], '0.000826_125.000000_2.000000_687.500000_0.000000_0.100000': [0, 1339.4814065654793, 44.47452514195976, 0, 4.659069143746456, 0, 430.77523689245817, -563.536784115218], '0.000909_137.500000_1.652893_568.181818_0.000000_0.100000': [1656.8186497540573, 977.8090470026475, 279.09380719436433, 0, 680.1135040597555, 0, -347.1376950073374, 612.8782205501609], '0.000909_125.000000_1.652893_568.181818_0.000000_0.100000': [3407.4360392225017, 633.2966697550389, 0, -212.40760820093226, 42.93260562703381, -100.43464461652161, -239.924135174693, -527.1433143226627], '0.000826_125.000000_1.652893_568.181818_0.000000_0.100000': [273.99041930666993, 332.0957112340242, 0, 327.1676876598358, 0, 0, -312.5125314888557, 273.8409529141777], '0.000826_137.500000_1.652893_568.181818_0.000000_0.100000': [312.5125314888557, 940.0749951566847, 278.88277759664555, 0, 2.575443423483739, 0, -282.4914384341945, -4.752586163687522], '0.000751_137.500000_1.652893_568.181818_0.000000_0.100000': [312.03749660925877, 447.6090706446452, -2.575443423483739, 0, 2.330659035619101, 0, 0, 0], '0.000683_137.500000_1.652893_568.181818_0.000000_0.100000': [0, 447.24500115571135, 0, 0, 2.1101603107104894, 0, 0, 0], '0.000621_137.500000_1.652893_568.181818_0.000000_0.100000': [0, 0, -2.1101603107104894, 0, 0, 0, 0, 0], '0.000683_137.500000_1.652893_516.528926_0.000000_0.100000': [0, 0, 0, 0, 1.7779034649051029, 0, 0, 0], '0.000621_137.500000_1.652893_516.528926_0.000000_0.100000': [0, 0, 0, 0, 1.6079382538575373, 0, 0, 0], '0.000564_137.500000_1.652893_516.528926_0.000000_0.100000': [0, 0, 0, 0, 1.454880863704784, 0, 0, -446.6095335972574], '0.000564_137.500000_1.652893_568.181818_0.000000_0.100000': [0, 446.6095335972574, 0, 0, 0, 0, 0, 0], '0.000513_137.500000_1.652893_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001100_113.636364_1.818182_625.000000_0.000000_0.100000': [338.2297054875811, 2084.263191484866, 1537.538240471319, 147.9825524352118, -1137.8959480447077, 371.9589839484294, -148.30221421208662, -820.8321609987792], '0.001100_113.636364_1.652893_625.000000_0.000000_0.100000': [321.46277184202626, 2822.5720248110474, -3.078674778126151, 847.381103693066, 121.93209989710556, 521.0173250361213, 972.1369375483057, 211.6128510773814], '0.001100_125.000000_1.652893_625.000000_0.000000_0.100000': [-280.0074590233575, 2355.655764724377, -368.0832005651906, 0, -21.57907132761312, 118.80198518539123, 367.4917385126102, -879.5954158358753], '0.001100_113.636364_1.502630_625.000000_0.000000_0.100000': [0, 0, -2.84953547270743, 311.4602722843358, 0, 2029.9745354466202, -669.8226696952397, -410.93979412588345], '0.001100_113.636364_1.502630_687.500000_0.000000_0.100000': [295.231254482025, 1884.1263601076805, 0, -35.36789523206626, 1.779142855459577, 0, 0, -426.4621701071148], '0.001100_113.636364_1.502630_756.250000_0.000000_0.100000': [0, 426.4621701071148, -1.9762641221905142, 0, 0, 814.4640697189317, 0, 353.99389654926205], '0.001100_113.636364_1.366027_756.250000_0.000000_0.100000': [0, 936.8529631960781, 0, 0, 5.423457388400857, 622.9576830222013, 0, -517.5634140513383], '0.001100_113.636364_1.241843_756.250000_0.000000_0.100000': [246.87550186883692, 0, 0, -622.9576830222013, 0, 0, 0, 0], '0.001100_113.636364_1.366027_831.875000_0.000000_0.100000': [0, 0, 0, -658.1073149780383, 0, 0, 0, 0], '0.001100_113.636364_1.502630_831.875000_0.000000_0.100000': [1071.438879187374, 44.19193148681767, -1.9716669049623305, 0, 0, 0, 0, 0], '0.001210_113.636364_1.502630_831.875000_0.000000_0.100000': [256.66825075144027, 0, 0, 0, 0, 0, 0, 0], '0.001210_103.305785_1.502630_831.875000_0.000000_0.100000': [249.19926837762523, 458.9183419299814, 0, 0, 0, 0, 0, 0], '0.001210_93.914350_1.502630_831.875000_0.000000_0.100000': [0, 0, 0, 0, 218.89840013117345, 0, -249.19926837762523, 0], '0.001210_103.305785_1.502630_756.250000_0.000000_0.100000': [0, 0, 0, -9.15609237878498, 192.61090996608544, 0, -271.89323732050156, 0], '0.001100_103.305785_1.502630_756.250000_0.000000_0.100000': [1110.764019693167, 0, -0.22084207312906942, 0, 0, 0, 0, 0], '0.001100_93.914350_1.502630_756.250000_0.000000_0.100000': [0, 0, -3.2485421488894644, -107.7466932789667, 2.932217237418081, 1476.8097669236854, 0, -478.99439110662263], '0.001100_93.914350_1.652893_756.250000_0.000000_0.100000': [0, 0, 0, 0, 0, 107.7466932789667, -284.5826880891892, 0], '0.001100_103.305785_1.818182_756.250000_0.000000_0.100000': [320.2199264362861, 0, 0, 0, 4.663298600896241, 0, 0, 0], '0.001000_93.914350_2.000000_568.181818_0.000000_0.100000': [3026.4255498794873, 1074.522403657827, 1050.884496832891, -1272.4461148955913, 0, 0, 0, -1145.8269865990424], '0.001100_93.914350_2.000000_568.181818_0.000000_0.100000': [4497.832387770763, 567.832408620306, 0, 0, 192.59600292996993, 0, 0, -593.2753961719785], '0.001100_93.914350_2.000000_625.000000_0.000000_0.100000': [182.49834983667216, 0, 0, -1267.2851125890227, 0, 0, 70.80038275906861, 0], '0.001100_93.914350_2.200000_625.000000_0.000000_0.100000': [0, 0, -6.0410001288735655, 0, 0.5490583998321928, 1267.2851125890227, 0, 0], '0.001210_85.376682_2.200000_625.000000_0.000000_0.100000': [0, 2266.3511868613346, 0, 0, 0, 1270.970512946347, 0, -620.8689556691396], '0.001210_85.376682_2.200000_568.181818_0.000000_0.100000': [0, 0, 0, 0, 456.2859906934477, 0, 0, 0], '0.001100_85.376682_2.200000_568.181818_0.000000_0.100000': [0, 0, 0, 0, -6.634812155230793, 493.5466189250506, 0, 0], '0.001100_85.376682_2.000000_568.181818_0.000000_0.100000': [-534.328251408673, -263.0645125817961, -7.051012755135162, -444.63659362616926, -126.42479660274405, 3878.4059485321914, -1175.0317287119865, -1448.0497240747716], '0.001100_85.376682_2.000000_625.000000_0.000000_0.100000': [1185.6694330376727, 0, 1087.5770624221207, 0, 0, 2565.1341476259076, 0, -630.4148553248157], '0.001210_85.376682_2.000000_625.000000_0.000000_0.100000': [-267.4101685801743, 0, 0, 0, 1154.3130995078852, 1403.9431233424195, -320.36112432159626, -968.6399111343467], '0.001210_85.376682_2.000000_687.500000_0.000000_0.100000': [549.0717740241262, 63.680787672643646, 0, -1261.3366203714593, 0, 0, -333.82397494914636, 0], '0.001210_77.615165_2.000000_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -2019.077337839714, 0], '0.001210_85.376682_2.200000_687.500000_0.000000_0.100000': [274.3585734496026, 0, 0, 0, 6.131536071374103, 1954.9228458282296, -291.59539877628595, 0], '0.001100_85.376682_2.200000_687.500000_0.000000_0.100000': [0, 0, 0, -781.1652584554855, 5.5730845595808205, 0, 0, 0], '0.001100_85.376682_2.420000_687.500000_0.000000_0.100000': [0, 0, -3.81414213711887, 0, 3.801052434420726, 0, -217.74439821776087, 0], '0.001100_93.914350_2.420000_687.500000_0.000000_0.100000': [217.74439821776087, 0, 0, 0, 0, 0, 0, 0], '0.001000_85.376682_2.420000_687.500000_0.000000_0.100000': [770.732964679705, 0, -0.691100442624629, 0, 0, 0, -24.570185978349045, -595.7724324438876], '0.001210_85.376682_2.420000_687.500000_0.000000_0.100000': [0, 0, 0, -669.3973121714589, 0, 778.8478645212302, 0, 0], '0.001210_85.376682_2.662000_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 669.3973121714589, 0, 0], '0.000909_125.000000_1.652893_625.000000_0.000000_0.100000': [14.215205339638032, 2372.7225436463978, -412.14882791370803, -795.2433453941068, 105.09709130846295, 426.76149489922346, -67.56461572758747, -500.0367580660137], '0.000909_113.636364_1.652893_625.000000_0.000000_0.100000': [-636.0292465038608, -658.3614117606425, -343.1123875032872, -1257.2695042895339, -527.7652963955242, -1156.5275019777725, -155.26842958732982, -1589.9239304006035], '0.000826_113.636364_1.652893_625.000000_0.000000_0.100000': [-1289.3114957819841, -724.812456912244, -1326.8824405816795, -872.0453604300333, 183.7663893564261, -1272.6238341151045, -400.86753301545434, -1788.0079476074397], '0.000751_113.636364_1.652893_625.000000_0.000000_0.100000': [-849.5598390904158, 33.25306697672576, -1592.3388954137856, -1321.4038234018449, -1186.713709935552, -1484.1756427342366, -1624.9324826920952, -691.0754337510974], '0.000683_113.636364_1.652893_625.000000_0.000000_0.100000': [-841.7577370228362, -658.1917361682944, -86.1017432221687, -1454.37634247434, 2.6416143904318234, 0, -1550.253910905119, 0], '0.000683_125.000000_1.652893_625.000000_0.000000_0.100000': [281.8643474372943, 0, 0, 0, 0, -60.31577463804606, 0, -499.97336498435544], '0.000683_103.305785_1.652893_625.000000_0.000000_0.100000': [0, 443.86852686504466, 0, 0, 0, 22.48268899695213, 925.9335107251197, 0], '0.000683_103.305785_1.502630_625.000000_0.000000_0.100000': [343.2312022376853, 0, 0, 0, 0, 0, 0, 0], '0.000683_93.914350_1.502630_625.000000_0.000000_0.100000': [0, 482.90398602529524, 0, 0, 0, 0, 0, 0], '0.000683_93.914350_1.502630_568.181818_0.000000_0.100000': [345.80441350204376, 0, 0, -123.82222530172967, 0, 0, 0, 0], '0.000683_93.914350_1.652893_568.181818_0.000000_0.100000': [0, 465.73512940156036, 0, 0, 0, 123.82222530172967, 0, 0], '0.000683_93.914350_1.652893_516.528926_0.000000_0.100000': [0, 0, 0, 0, 176.48505111235465, 0, -239.92049759022785, -423.3955721832367], '0.000683_103.305785_1.652893_516.528926_0.000000_0.100000': [0, 0, 0, -161.04680552318504, 1.4092400514468864, 31.672480020026253, -258.5942340320959, -464.4021996861672], '0.000621_103.305785_1.652893_516.528926_0.000000_0.100000': [263.51108332404795, 0, 0, 0, 0, 33.204516564298956, 0, 0], '0.000621_93.914350_1.652893_516.528926_0.000000_0.100000': [1444.248510092739, 456.50627278972297, 0, -124.39866594969608, 0, 194.93419817394062, -239.55553029458906, 0], '0.000621_103.305785_1.502630_516.528926_0.000000_0.100000': [383.56392116114523, 0, 0, 182.3954607008312, 0, 0, 0, 0], '0.000621_93.914350_1.502630_516.528926_0.000000_0.100000': [0, 0, 0, -177.2129074308551, 0, 986.442279058602, -38.3563921161145, 0], '0.000621_93.914350_1.652893_469.571751_0.000000_0.100000': [212.4004561899349, 0, 0, 0, 0, 0, 0, -297.0146561019534], '0.000621_93.914350_1.366027_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, -6.823039737417275, -373.4880959154514, 0], '0.000621_93.914350_1.241843_516.528926_0.000000_0.100000': [0, 0, 0, 6.823039737417275, 0, 0, 0, 0], '0.001000_137.500000_1.818182_687.500000_0.000000_0.100000': [582.5796922357641, 1491.3442729889273, -884.6769591008889, -644.2388614411445, 845.3994078040512, 0, -361.9359413067423, -527.7668695398671], '0.001100_125.000000_2.000000_756.250000_0.000000_0.100000': [0, 0, 0, -1104.5476875668794, 969.3366731842024, 794.0429905683585, -360.34974751208773, 0], '0.001100_137.500000_2.000000_756.250000_0.000000_0.100000': [0, 23.778616640517964, -6.700152776743562, 0, 0, 1256.497825672475, 0, 0], '0.001210_137.500000_2.000000_756.250000_0.000000_0.100000': [0, 0, 0, -1031.568749929787, 0, 1516.420011808038, 0, 0], '0.001210_137.500000_2.200000_756.250000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000826_125.000000_1.818182_625.000000_0.000000_0.100000': [360.34214865660874, 680.3492204770617, -385.55688769909534, 849.7439485214821, 0, 374.02198034473236, -144.2031610035187, -540.9920766683232], '0.000826_125.000000_1.818182_568.181818_0.000000_0.100000': [1638.811533780775, 503.0529761929864, 312.26636611490227, -791.8224071589032, 2.6782429622090205, 1213.6962229726246, -386.8533860973529, -401.96684178578573], '0.000751_125.000000_1.652893_568.181818_0.000000_0.100000': [952.3956972999817, 0, 0, 0, 0, 0, 0, -478.5373616662329], '0.000751_125.000000_1.652893_625.000000_0.000000_0.100000': [1549.577496245503, 0, -2.5364364147706513, -247.0582886148296, 0, -59.12037518676589, 0, 0], '0.000751_125.000000_1.818182_625.000000_0.000000_0.100000': [0, 467.63491420235067, 0, 0, 0, 0, 0, -491.42751285304405], '0.000683_113.636364_1.818182_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -322.6824301464676, -242.97729232923461], '0.000683_113.636364_1.818182_568.181818_0.000000_0.100000': [700.7896999790519, 0, -1.7618088834038645, -753.333225770059, 0, 196.31068144990496, 431.88970569789603, 0], '0.000683_113.636364_1.652893_568.181818_0.000000_0.100000': [261.5296589621843, 0, 0, 0, 1.4158498138922733, 0, 0, 0], '0.000621_113.636364_1.652893_568.181818_0.000000_0.100000': [589.9236349787018, 0, 0, -196.00577820122234, 0, 0, -272.56320204181685, 0], '0.000621_125.000000_1.652893_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, -129.57325534537267, 0, 0], '0.000621_125.000000_1.502630_568.181818_0.000000_0.100000': [319.3901546754616, 0, -1.4369001296283386, 0, 0, 0, -297.83981357405537, 0], '0.000621_137.500000_1.502630_568.181818_0.000000_0.100000': [297.83981357405537, 0, 0, 0, 0, 0, 0, 0], '0.000683_125.000000_1.502630_568.181818_0.000000_0.100000': [0, 0, 0, 0, 1.4369001296283386, 0, 0, 0], '0.000621_113.636364_1.502630_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -407.8096689264154], '0.000621_113.636364_1.502630_625.000000_0.000000_0.100000': [330.3274693569207, 0, 0, 0, 1.4016195053294724, 0, -321.18800285791366, 0], '0.000621_103.305785_1.502630_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 973.8711559220023, 0, 0], '0.000621_103.305785_1.366027_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 292.02350495774766, 0, 0], '0.000621_103.305785_1.241843_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000826_93.914350_2.000000_625.000000_0.000000_0.100000': [0, 592.9833618892432, -5.029110988731008, 0, 0, 867.3508007015573, 0, 0], '0.000826_93.914350_1.818182_625.000000_0.000000_0.100000': [0, 0, 0, -60.192721782182744, -478.6306273698723, 253.86274588919193, 0, 0], '0.000751_93.914350_1.818182_625.000000_0.000000_0.100000': [0, 0, 0, 0, 1.9718269724597122, 227.63077214868756, 0, -557.2099724117221], '0.000751_93.914350_1.818182_687.500000_0.000000_0.100000': [3166.9184829214914, 0, 2782.1438044267425, -711.5523165666708, 0, 276.50391909599807, 0, 0], '0.000751_93.914350_1.652893_687.500000_0.000000_0.100000': [0, 0, 0, 0, 1.7412523243394389, 83.56298778780383, -259.5063296936158, 0], '0.000683_93.914350_1.652893_687.500000_0.000000_0.100000': [0, 0, -1.7412523243394389, 0, 0, 0, 0, 0], '0.000751_93.914350_1.502630_687.500000_0.000000_0.100000': [0, 0, 0, 0, 2.495583848077331, 1075.711052868022, 0, 0], '0.000751_93.914350_1.366027_687.500000_0.000000_0.100000': [0, 0, 0, 0, 191.24116518071932, 0, 0, -527.5036160039443], '0.000683_93.914350_1.366027_687.500000_0.000000_0.100000': [0, 0, 0, 0, 3.4018439758424392, 228.70687946259687, 0, 0], '0.000683_93.914350_1.241843_687.500000_0.000000_0.100000': [902.8027952129398, 0, 0, 0, 0, 0, 0, -412.49676970891454], '0.000683_93.914350_1.241843_756.250000_0.000000_0.100000': [0, 0, 0, -322.9917811867385, 0, 0, 0, 0], '0.000683_93.914350_1.366027_756.250000_0.000000_0.100000': [0, 527.5732059296558, 0, 0, 0, 0, 0, 0], '0.000621_93.914350_1.366027_687.500000_0.000000_0.100000': [0, 511.4433374764849, 0, 0, 0, 0, 0, 0], '0.000621_93.914350_1.366027_625.000000_0.000000_0.100000': [0, 485.41155835073914, 0, 0, 0, 0, 0, 0], '0.000621_93.914350_1.366027_568.181818_0.000000_0.100000': [0, 463.2764303825671, 0, 0, 0, 0, 0, 0], '0.000621_103.305785_1.366027_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -426.1167858941656, 0], '0.000621_113.636364_1.366027_516.528926_0.000000_0.100000': [0, 0, -3.4316843118464933, 0, 0, 0, 0, 0], '0.000683_113.636364_1.366027_516.528926_0.000000_0.100000': [0, 0, 0, -940.1644948861486, 0, 325.4933672821976, 0, 0], '0.000683_113.636364_1.502630_516.528926_0.000000_0.100000': [0, 0, 0, 79.98687014002317, 0, 940.1644948861486, 0, 0], '0.000683_113.636364_1.241843_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, -138.535701308423, 0, 0], '0.000683_113.636364_1.128948_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0.6547202155586093, 0, 0, 0], '0.000621_113.636364_1.128948_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001100_125.000000_2.200000_568.181818_0.000000_0.100000': [1833.3368634616484, -517.217084587935, 0, 274.8699228963359, 450.51199722445614, 0, 0, -5517.336758145385], '0.001100_125.000000_2.200000_516.528926_0.000000_0.100000': [467.75153308224935, 1801.2023673023812, -5.686152975892037, -100.99520238354143, 0, 0, -383.6935262938259, -25.45691653363906], '0.001210_125.000000_2.200000_516.528926_0.000000_0.100000': [770.6303995616053, 515.8622531158347, 0, 0, 0, 0, 0, -494.3778670075821], '0.001210_125.000000_2.200000_568.181818_0.000000_0.100000': [900.2899229830434, 0, 0, 0, 0, 1237.0010594232133, 0, -510.2033548974227], '0.001210_113.636364_2.200000_568.181818_0.000000_0.100000': [372.10289785776916, 1846.01226535586, 0, 0, 5.942088338753716, 0, -0.30429896889938446, 0], '0.001210_113.636364_2.200000_516.528926_0.000000_0.100000': [444.55866343106544, 0, 0, 0, 0, 1424.372287714386, 0, 0], '0.001464_113.636364_2.000000_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 623.4285964146817, -432.0013174812266, 0], '0.001464_113.636364_1.818182_516.528926_0.000000_0.100000': [0, 451.1084426713822, 0, 0, 0, 135.8204946187783, 0, 0], '0.001464_113.636364_1.818182_469.571751_0.000000_0.100000': [0, 0, -5.054896317846942, 0, 0, 0, 0, 0], '0.001611_113.636364_1.818182_469.571751_0.000000_0.100000': [0, 0, -6.173760997672789, 0, 0, 0, 0, 0], '0.001772_113.636364_1.818182_469.571751_0.000000_0.100000': [0, 0, -6.234860591125744, 0, 0.5612509997884351, 0, 0, 0], '0.001949_113.636364_1.818182_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 221.08569635987462, 0, 0], '0.001949_113.636364_1.652893_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, -39.214140187397064, 0, 0], '0.001949_113.636364_1.502630_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -381.3543161945792, 0], '0.001949_125.000000_1.502630_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 804.7256307308826, 0, 0], '0.001949_125.000000_1.366027_469.571751_0.000000_0.100000': [0, 0, 0, 0, 8.759653688799881, 0, 0, 0], '0.001772_125.000000_1.366027_469.571751_0.000000_0.100000': [0, 0, 0, -805.9463152284216, 0, 0, 0, 0], '0.001772_125.000000_1.502630_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -451.52285051654434], '0.001772_125.000000_1.502630_516.528926_0.000000_0.100000': [322.98511553729605, 0, 0, 0, 0, 0, 0, 0], '0.001772_113.636364_1.502630_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001210_113.636364_2.000000_687.500000_0.000000_0.100000': [1943.41976029188, 2580.166162538456, 855.5882366978583, 0, 1004.9158853229139, 0, -360.54607870551604, -652.0553149643808], '0.001331_113.636364_2.000000_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 1206.4917223764583, -1419.0273509616695, -587.3738926774325], '0.001331_113.636364_1.818182_687.500000_0.000000_0.100000': [58.67244225672539, 0, -5.005524269273337, 184.2178155760629, 0, 287.7210521476604, 0, 0], '0.001464_113.636364_1.818182_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -308.4601669133417, 0], '0.001464_125.000000_1.818182_687.500000_0.000000_0.100000': [0, 0, -6.1001644329346165, 0, 0, 0, 0, 0], '0.001611_125.000000_1.818182_687.500000_0.000000_0.100000': [0, 494.70146144066183, 0, 0, 0, 0, 0, 0], '0.001611_125.000000_1.818182_625.000000_0.000000_0.100000': [316.1042046375691, 0, -6.152609109375407, 0, 0, 0, 0, 0], '0.001772_125.000000_1.818182_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 227.39799287759251, 0, 0], '0.001772_125.000000_1.652893_625.000000_0.000000_0.100000': [287.4655624037752, 0, 0, 0, 0, 0, 0, -496.4588414725533], '0.001772_125.000000_1.652893_687.500000_0.000000_0.100000': [0, 0, 0, 0, 5.0126652026056036, 0, 0, -474.24388403423603], '0.001611_125.000000_1.652893_687.500000_0.000000_0.100000': [0, 0, -5.0126652026056036, 0, 0, 0, 0, 0], '0.001772_125.000000_1.652893_756.250000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -496.8667912767891], '0.001772_125.000000_1.652893_831.875000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -516.8884027661812], '0.001772_125.000000_1.652893_915.062500_0.000000_0.100000': [276.0027881407754, 0, 0, 0, 0, 0, 0, 0], '0.001772_113.636364_1.652893_915.062500_0.000000_0.100000': [0, 0, -7.1936543036044895, 0, 0, 0, 0, 0], '0.001949_113.636364_1.652893_915.062500_0.000000_0.100000': [0, 0, 0, -349.0453715771679, 0, 0, 0, 0], '0.001949_113.636364_1.818182_915.062500_0.000000_0.100000': [0, 566.7363365023766, 0, 0, 0, 0, 0, 0], '0.001949_113.636364_1.818182_831.875000_0.000000_0.100000': [0, 0, 0, -992.6955042362309, 0, 0, 0, 0], '0.001949_113.636364_2.000000_831.875000_0.000000_0.100000': [0, 0, -12.980271783526824, 0, 0, 0, 0, 0], '0.002144_113.636364_2.000000_831.875000_0.000000_0.100000': [0, 0, 0, -1072.739768634313, 1.1800247075933479, 0, 0, 0], '0.002144_113.636364_2.200000_831.875000_0.000000_0.100000': [243.5326867876922, 0, 0, 0, 0, 0, 0, 0], '0.002144_103.305785_2.200000_831.875000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001000_125.000000_2.420000_625.000000_0.000000_0.100000': [57.58934749143876, 997.1147239858681, -2.860201273770872, -456.91194388936964, 0, 0, -255.27849582677754, -338.3761286918939], '0.001000_125.000000_2.420000_568.181818_0.000000_0.100000': [313.34482804856617, 533.6860192269608, 0, -408.57746199481426, -363.82453443993734, 0, -260.50709822180323, 0], '0.001000_113.636364_2.420000_568.181818_0.000000_0.100000': [0, 0, 255.37111941468456, 0, 0.5450799278876364, 276.1897417757628, -284.44337175254935, 0], '0.001100_113.636364_2.420000_568.181818_0.000000_0.100000': [1369.6982927888143, 0, -3.3044048143532563, -452.5051343610365, 0, 0, -281.2127823073337, -491.56683888466176], '0.001100_113.636364_2.662000_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 452.5051343610365, 0, 0], '0.001100_125.000000_2.420000_568.181818_0.000000_0.100000': [281.2127823073337, 1938.4928766637245, -0.26652025350520464, 0, 0, 0, -260.38613776409983, -483.24855289323585], '0.001100_125.000000_2.420000_625.000000_0.000000_0.100000': [0, 0, -3.3552271860707843, -456.6830008757861, 0, 444.7327052494602, 0, -486.8502113693163], '0.001100_125.000000_2.420000_687.500000_0.000000_0.100000': [0, 486.8502113693163, -3.3747360432575078, 0, 3.0568356826704988, 0, 0, 0], '0.000909_125.000000_2.200000_568.181818_0.000000_0.100000': [425.22253027276867, 1066.4994641742958, -267.9899867139718, -318.76937749501764, 4.382642585011673, 119.35459637907161, 454.70853445427065, 106.9184343390833], '0.001000_113.636364_2.200000_568.181818_0.000000_0.100000': [410.15424624133095, 549.0806740532942, 0, -666.1941762262583, 4884.364339228432, 0, -193.01019571088477, -1640.6829231397094], '0.001000_113.636364_2.200000_516.528926_0.000000_0.100000': [2619.4166216511144, 4724.340118193828, 94.52231322449904, -668.5998594338286, 0, 0, 7.912936576530342, -571.3182151767955], '0.001000_113.636364_2.420000_516.528926_0.000000_0.100000': [0, 0, 0, -339.72118693222404, 0, 0, 0, -475.64976186325134], '0.001100_113.636364_2.420000_625.000000_0.000000_0.100000': [0, 776.1445238914228, 513.9718196867841, -525.9776515075617, 0, 2570.577394367473, -272.8944963159078, 0], '0.001210_125.000000_2.420000_625.000000_0.000000_0.100000': [299.8274886062257, 0, -3.1731567512451875, -456.5760978569668, 0, 0, 0, 0], '0.001210_125.000000_2.662000_625.000000_0.000000_0.100000': [0, 527.2017955925776, 0, 0, 0, 0, 0, 0], '0.001210_125.000000_2.662000_568.181818_0.000000_0.100000': [0, 568.3456318975898, 0, 0, 0, 412.9803798172314, 0, 0], '0.001210_125.000000_2.420000_568.181818_0.000000_0.100000': [0, 0, 0, 0, 2.931722788557254, 0, 0, 0], '0.001210_113.636364_2.420000_568.181818_0.000000_0.100000': [287.5104571277878, 278.0838067803025, 0, -454.1271189452926, 0, 0, 0, 0], '0.001210_113.636364_2.662000_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -239.4268408999713, 0], '0.001210_125.000000_2.662000_516.528926_0.000000_0.100000': [0, 0, 0, 0, 4.546800696008723, 0, -270.25254691330866, 0], '0.001210_137.500000_2.662000_516.528926_0.000000_0.100000': [270.25254691330866, 0, 0, 0, 0, 0, 0, 0], '0.001100_125.000000_2.662000_516.528926_0.000000_0.100000': [0, 0, 0, 0, 4.1203776719263265, 0, 0, 0], '0.001000_125.000000_2.662000_516.528926_0.000000_0.100000': [0, 0, 0, 0, 3.7349198724759844, 0, 0, 0], '0.000909_125.000000_2.662000_516.528926_0.000000_0.100000': [256.00463563918385, 0, 0, 0, 0, 0, 0, 0], '0.001000_93.914350_1.818182_625.000000_0.000000_0.100000': [2326.3273737573477, 0, 0, 0, 0, 0, -71.26227257876147, -554.1724603933612], '0.001100_125.000000_1.652893_568.181818_0.000000_0.100000': [1711.1470989630454, 225.07678334954278, -31.524608119034813, 1071.8330882183134, 298.587636398958, -125.88502776757706, -345.64699307079604, 0], '0.001210_137.500000_1.652893_568.181818_0.000000_0.100000': [957.6083716210143, 0, 0, 0, 0, 0, -386.8080664538448, 0], '0.001331_125.000000_2.000000_516.528926_0.000000_0.100000': [424.5260742877415, 0, -8.292052731200966, -1258.465910699391, 0, 0, 0, 0], '0.001331_125.000000_2.200000_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001000_137.500000_2.000000_469.571751_0.000000_0.100000': [478.1358310624235, 454.15552808049125, 0, 0, 6.258444561817832, 2093.843844662892, 216.14053604822521, -475.30681855685145], '0.001000_137.500000_2.200000_516.528926_0.000000_0.100000': [0, 0, -5.0653905949868205, 0, 0, 1696.0203519934048, -396.9336285376511, 0], '0.001100_137.500000_2.000000_568.181818_0.000000_0.100000': [22.087867197893388, 0, -115.72995178042243, 328.1986690164282, 3726.5780816273723, 226.48986750689915, -277.61008464930956, -54.26906240052972], '0.001100_137.500000_2.200000_568.181818_0.000000_0.100000': [657.432988910866, 855.595359556429, 0, -371.21807389021177, 667.585136260282, 1423.9681763737644, -364.6429587139932, 720.7670003743057], '0.000909_137.500000_2.200000_568.181818_0.000000_0.100000': [2406.6662521232743, 0, 0, 0, 4.329431723279413, 12.046123215633315, 0, -531.8053031239255], '0.001000_113.636364_2.420000_625.000000_0.000000_0.100000': [0, 0, 0, 0, 3.0377576676699025, -229.3335098687345, -232.47562492315143, 0], '0.000909_125.000000_2.420000_687.500000_0.000000_0.100000': [0, 0, 0, 0, 4.5481568826047996, 507.0947553102454, 0, 0], '0.000909_137.500000_2.200000_756.250000_0.000000_0.100000': [0, 0, 0, 0, 2.4976815333480773, 0, 0, 0], '0.000826_137.500000_2.200000_756.250000_0.000000_0.100000': [0, 0, 0, 0, 2.3295544335524028, 0, 0, 0], '0.000751_137.500000_2.420000_756.250000_0.000000_0.100000': [0, -12060.501382076862, 0, 0, 0, 0, 0, 0], '0.000751_137.500000_2.420000_687.500000_0.000000_0.100000': [0, 475.0936044320597, 0, 0, 0, 0, 0, 0], '0.000683_113.636364_2.420000_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -513.4222575883326], '0.000683_113.636364_2.420000_687.500000_0.000000_0.100000': [0, 0, -2.010965596982533, 0, 0, 0, 0, 0], '0.000751_113.636364_2.420000_687.500000_0.000000_0.100000': [238.2551069259369, 0, 0, 0, 0, 0, 0, 0], '0.000751_103.305785_2.420000_687.500000_0.000000_0.100000': [230.63958765978916, 0, -2.6198581144528363, -585.4336631741843, 0, 0, 0, 0], '0.000751_103.305785_2.662000_687.500000_0.000000_0.100000': [0, 0, -2.0858378550483394, 0, 0, 0, 0, 0], '0.000826_103.305785_2.662000_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000909_113.636364_1.818182_516.528926_0.000000_0.100000': [512.7913599165752, 1693.7594718494029, 285.9559499494894, 215.8381724181387, 529.0750143317109, 0, 1446.5646791982451, -117.33111835813543], '0.000909_113.636364_1.818182_469.571751_0.000000_0.100000': [575.3051507343012, 509.80562424223103, 2669.427201878065, -75.61533030287376, 721.1425884161505, 232.24128425640882, 296.66741779643064, 72.3836419809756], '0.000909_113.636364_1.652893_469.571751_0.000000_0.100000': [0, 144.27299522641, -2.127111784326189, 1085.6144840238853, 3503.053524612643, 0, -298.53211242482604, 441.22047986961815], '0.000909_125.000000_1.652893_469.571751_0.000000_0.100000': [865.6919084565541, 0, -302.0660563279922, -269.3661870197713, 0, 0, 0, 0], '0.000909_125.000000_1.818182_469.571751_0.000000_0.100000': [392.4468451326481, 630.1349125547956, -2.954957449158428, -667.9478956757216, -127.30923777907037, 1269.0304548932372, 318.70742027327464, -418.63709824122964], '0.000909_125.000000_2.000000_469.571751_0.000000_0.100000': [1561.1562184014997, 0, -5.5975708716470765, 0, 5.594531219184119, 63.81616400255185, -47.80439068778706, 626.0682566405295], '0.001000_113.636364_2.000000_469.571751_0.000000_0.100000': [2199.3269009935475, 0, 696.1205303373963, 908.3499524815936, -342.4846480406573, 882.3991680837153, -377.17030269108244, -183.71787832331873], '0.001100_113.636364_2.000000_469.571751_0.000000_0.100000': [2590.4840757332545, -334.00974947789746, -6.625500855685096, -1282.6945778798752, 59.39654937483033, 0, 0, 0], '0.001100_113.636364_2.000000_426.883410_0.000000_0.100000': [0, 706.7533357809805, -6.471452634264097, 264.12531754341876, 1.2326716037951924, 263.8563637434893, -464.17593506699524, 0], '0.001100_113.636364_1.818182_388.075827_0.000000_0.100000': [396.3480235340534, 446.6862516548857, 176.7938471520633, 0, 355.26556805910417, 441.8645348260992, -39.25328906003938, 0.31232991223128126], '0.001000_113.636364_1.818182_388.075827_0.000000_0.100000': [502.8915766731046, 0, -3.9456261672384243, 0, 0, 196.2925922075883, 0, 0], '0.001100_103.305785_1.818182_388.075827_0.000000_0.100000': [910.247021778863, 1782.5022451729478, 0, -368.90009116554864, 0, 116.19387630606097, 0, 0], '0.001100_103.305785_1.652893_388.075827_0.000000_0.100000': [887.3302860103161, 0, -1.4137733460745057, 0, 0, -21.015561865424047, 252.49443860788926, -39.821599133516365], '0.001100_113.636364_1.652893_388.075827_0.000000_0.100000': [320.0186405022026, 0, 1094.7613651182978, 203.57962881865882, 1.5604720822879246, 181.60248816200908, 0, -155.76891322672986], '0.001100_113.636364_1.502630_388.075827_0.000000_0.100000': [460.8972380403113, 3080.1216137264246, -5.075987555213487, 0, 0, 911.7936608341678, -451.61453034344015, -467.20211983440913], '0.001100_125.000000_1.502630_388.075827_0.000000_0.100000': [1971.9550244420966, 0, -5.0272131278002234, 0, 0, 0, -446.2053065964052, 0], '0.001210_125.000000_1.502630_388.075827_0.000000_0.100000': [451.5608784732855, 0, -5.554809601464785, 0, 0, 0, -44.63360174038118, 0], '0.001331_125.000000_1.502630_388.075827_0.000000_0.100000': [0, 0, -6.140259180304201, 0, 0, 0, 0, 0], '0.001464_125.000000_1.502630_388.075827_0.000000_0.100000': [411.22404589741154, 428.640678888319, 0, 0, 0, 0, 0, 0], '0.001464_125.000000_1.502630_352.796206_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -428.640678888319], '0.001464_113.636364_1.502630_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -468.28406506699486], '0.001464_113.636364_1.502630_426.883410_0.000000_0.100000': [380.5813803413128, 0, 0, 0, 0, 797.1472003696824, 0, 0], '0.000909_125.000000_1.818182_756.250000_0.000000_0.100000': [0, 0, 0, -971.3687433798831, 991.2304589969239, -49.290639015069736, -1259.4963871003695, -533.9213813580573], '0.000909_137.500000_1.818182_756.250000_0.000000_0.100000': [353.59794986215593, 289.8164537494578, -4.255866779445387, -108.01118022517016, 7.717249554495089, 0, -345.57075836450804, 0], '0.000909_137.500000_2.000000_756.250000_0.000000_0.100000': [0, 0, -5.521243916158347, 0, 5.038639066157702, 1121.8420929767344, -377.2569018210905, -546.4574738358248], '0.001000_137.500000_2.000000_756.250000_0.000000_0.100000': [-154.7968977071035, 0, 0, 0, 0, 0, 0, -601.0130622480617], '0.001000_137.500000_2.000000_831.875000_0.000000_0.100000': [0, 0, 0, 0, 5.439281214935181, 0, 11657.345578924653, 0], '0.001000_151.250000_2.000000_831.875000_0.000000_0.100000': [0, -11488.195576262187, 0, 0, 0, 0, 0, 0], '0.001000_151.250000_2.000000_756.250000_0.000000_0.100000': [0, 0, 0, -659.6421998703887, 5.48985056720303, 0, 0, 0], '0.001000_151.250000_2.200000_756.250000_0.000000_0.100000': [0, 764.2687951902171, 0, 0, 0, 0, 0, 0], '0.001000_125.000000_1.652893_568.181818_0.000000_0.100000': [327.90180963931334, 2237.976573916754, 0, 27.31826792998413, 424.3534604452142, 64.24029041932704, 994.5840603134232, -658.9832241498432], '0.000909_103.305785_1.818182_516.528926_0.000000_0.100000': [517.1887948613489, 4247.980366685939, 366.67908451105797, -470.08314168753606, 282.9587416933633, 611.8944614420118, -862.4615628026008, -63.359910277861445], '0.001000_103.305785_1.818182_516.528926_0.000000_0.100000': [504.04713943704746, 448.84991325517876, 1934.6709541607813, -863.9076832368933, 1809.4550155513505, 112.51965273243468, -705.5845981010648, 0], '0.001000_103.305785_1.652893_516.528926_0.000000_0.100000': [514.702385340709, 2111.496780899579, 1669.8723826887249, 257.721439856125, 1867.3812703598699, 24.073710592545286, -608.2802857390566, 0], '0.001210_137.500000_2.000000_469.571751_0.000000_0.100000': [1563.2527870263368, 0, 0, -1135.5775646729658, 6.889724578255846, 0, 0, 0], '0.000909_137.500000_2.000000_469.571751_0.000000_0.100000': [1671.2151263452126, 0, 0, 0, 0, 1134.3451814701073, -495.8894782399535, -1545.6484978145259], '0.000826_125.000000_2.000000_469.571751_0.000000_0.100000': [0, 0, -5.085937471985562, -875.1650793820962, 0, 0, 0, 0], '0.000909_113.636364_2.000000_469.571751_0.000000_0.100000': [-7.697136559361638, 541.0639665193512, 1602.4376478266922, -1283.080224773573, 4.965488207379713, 160.43905711007835, 16.39058431711254, -452.98525636208785], '0.000909_113.636364_2.000000_516.528926_0.000000_0.100000': [0, 643.1961521575406, 2428.8718600265684, -1298.5661134495933, -1263.1563123034075, 294.90886222989957, 624.1336046623452, 216.6965806656079], '0.001000_93.914350_1.818182_469.571751_0.000000_0.100000': [828.1093793611678, 0, -3.8185780394219364, -358.5992516214437, 0, 111.05437239273797, 0, -474.1853920371007], '0.001000_93.914350_1.818182_516.528926_0.000000_0.100000': [4613.121026779623, 754.4231107155365, 0, 0, 160.42907842827822, 0, 0, -475.8510090299725], '0.001100_113.636364_2.200000_516.528926_0.000000_0.100000': [537.0960150231657, 567.4189677624893, 1255.0610063082163, -138.228397825662, 1502.0410361488985, 864.2401540793724, 745.8845770909081, -233.2844151117257], '0.001210_113.636364_2.420000_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -309.8592807524228, 0], '0.001210_125.000000_2.420000_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 589.144552416809, 0, 0], '0.001210_125.000000_2.200000_469.571751_0.000000_0.100000': [447.5415825957595, 888.7508110414627, -6.642953779907657, 0, 0, 0, 0, -515.8622531158347], '0.001210_113.636364_2.200000_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 1282.4673092403425, -406.8559841779632, 0], '0.001331_125.000000_2.000000_625.000000_0.000000_0.100000': [984.6763239580873, 0, 0, -1640.3414187722085, 0, 0, 0, 232.21305705465022], '0.001331_125.000000_2.000000_687.500000_0.000000_0.100000': [0, 0, 0, -1286.2006362034642, 0, 1531.563958563333, -386.058677624273, -563.1807234667058], '0.001331_137.500000_2.200000_687.500000_0.000000_0.100000': [0, 460.46573160677474, 0, -414.51269452640736, 0, 0, 0, 0], '0.001331_137.500000_2.200000_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -460.46573160677474], '0.001331_137.500000_2.420000_687.500000_0.000000_0.100000': [242.1719916132919, 0, 0, 0, 0, 0, 0, 0], '0.001100_103.305785_2.000000_469.571751_0.000000_0.100000': [384.7518794362921, 0, -13.430663661369199, -857.4302097928149, 5.831329781855584, 1964.1505244003004, 0, -580.5222519128379], '0.001331_113.636364_1.818182_426.883410_0.000000_0.100000': [0, 420.5361595554576, 0, -494.3856947388091, 0, 0, -358.58406052650747, 0], '0.001331_125.000000_1.818182_426.883410_0.000000_0.100000': [358.58406052650747, 0, 0, 0, 0, 267.69507311524313, 0, 0], '0.001331_113.636364_2.000000_426.883410_0.000000_0.100000': [429.3362817687574, 0, 0, -1249.4954458339673, 0, 0, 0, -487.62601137250624], '0.001331_103.305785_1.818182_426.883410_0.000000_0.100000': [0, 0, 0, -405.5118907616167, 0, 0, 0, -858.5886450245551], '0.001331_113.636364_2.000000_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -442.89855096804393, 0], '0.000826_137.500000_2.000000_625.000000_0.000000_0.100000': [-1051.2478424240635, 907.6381652785029, 0, -1167.2347486645476, 0, 0, -478.351897250282, 0], '0.000826_151.250000_2.000000_625.000000_0.000000_0.100000': [-628.9747283768396, 0, -5.139991033487422, 0, 0, 0, -448.861795466591, 0], '0.000909_151.250000_2.000000_625.000000_0.000000_0.100000': [-16845.547785746032, 1043.4978111961645, -5.653355424943766, 0, 0, 985.0769499126645, -448.82286000577716, -453.36879124180183], '0.000909_166.375000_2.000000_625.000000_0.000000_0.100000': [448.82286000577716, 0, 0, -973.9846877145158, 0, 0, 0, 0], '0.000909_166.375000_2.200000_625.000000_0.000000_0.100000': [0, 0, 0, 0, 2.3087343527859048, 0, 0, 0], '0.000826_166.375000_2.200000_625.000000_0.000000_0.100000': [0, 465.4914718505692, 0, 0, 0, 0, 0, 0], '0.000826_166.375000_2.200000_568.181818_0.000000_0.100000': [0, 0, 0, 0, 3.664262191981288, 0, -331.1373668653658, 0], '0.000826_183.012500_2.200000_625.000000_0.000000_0.100000': [0, 433.65848260405585, 0, 0, 0, 829.5837554351328, 0, 0], '0.000826_183.012500_2.000000_625.000000_0.000000_0.100000': [0, 0, 0, -829.5837554351328, 0, 0, 0, 0], '0.000826_183.012500_2.420000_568.181818_0.000000_0.100000': [266.09624486543964, 0, 0, 0, 0, 0, 11874.883858463107, 0], '0.000826_166.375000_2.420000_568.181818_0.000000_0.100000': [0, 0, 0, 0, 1.6391085856788778, 0, 0, 0], '0.000751_166.375000_2.420000_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -484.9739163847073], '0.000751_166.375000_2.420000_625.000000_0.000000_0.100000': [236.42775479531647, 0, 0, 0, 0, 157.11770285463845, 0, 0], '0.000909_125.000000_1.502630_625.000000_0.000000_0.100000': [405.75011150688704, 1711.7929324957895, 0, 0, 409.36255692391796, 427.8697521426264, -305.09470269543016, 267.32172575399994], '0.000909_125.000000_1.366027_625.000000_0.000000_0.100000': [1076.9531489499839, 0, 0, -399.89843212358295, 770.6092984316972, 633.2025319741825, 230.45176840271023, -516.6671840297358], '0.000909_125.000000_1.366027_687.500000_0.000000_0.100000': [0, 521.8126477751186, 0, 0, 4.430496520207089, 0, -45.88371694629444, 0], '0.000826_125.000000_1.366027_687.500000_0.000000_0.100000': [0, 521.090769834964, 0, 0, 0, 0, -445.2674505445008, -482.8602049356825], '0.000826_125.000000_1.366027_625.000000_0.000000_0.100000': [927.8492707457106, 0, 0, -847.5112113408524, -691.4320093289407, 0, -470.8421595342256, -469.4511439954631], '0.000826_125.000000_1.366027_756.250000_0.000000_0.100000': [0, 905.3662345315993, 0, 0, 0, 724.6300525231496, 0, 0], '0.000751_125.000000_1.366027_625.000000_0.000000_0.100000': [0, 0, -4.231220418640531, -154.68585595320997, 0, 628.7032284563277, 0, -469.61842415987616], '0.000751_125.000000_1.502630_625.000000_0.000000_0.100000': [0, 0, 0, 65.03241270544248, 1.1001017982078338, 850.3951682994101, 0, -392.2982867124674], '0.000826_125.000000_1.652893_625.000000_0.000000_0.100000': [1462.9375372224617, 478.9733895371168, 1326.2386752152174, 591.0227460279535, 2.5364364147706513, 41.333515363126025, -308.7977583204891, 0], '0.000751_125.000000_1.502630_687.500000_0.000000_0.100000': [0, 445.50662438055673, 0, 0, 0, 0, 0, 0], '0.000909_125.000000_2.420000_568.181818_0.000000_0.100000': [778.1937321570908, 0, 0, -447.31860125223784, 1.9513402132404736, 0, 0, 0], '0.000909_113.636364_2.662000_568.181818_0.000000_0.100000': [0, 0, -4.053204063904741, 0, 209.8200782654933, 449.71448835662324, 0, -568.7235756587521], '0.000909_113.636364_2.662000_625.000000_0.000000_0.100000': [0, 0, -2.3414130607236077, 0, 0, 0, 0, 0], '0.001000_113.636364_2.662000_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 693.1205226265151, 0, 0], '0.001100_113.636364_2.200000_687.500000_0.000000_0.100000': [619.494480713804, 0, 0, -541.395389197497, 0, 3226.320924517465, 0, -854.8992801680778], '0.001100_103.305785_1.818182_687.500000_0.000000_0.100000': [1011.7113502493664, 0, 0, 0, 0, 0, 0, -572.7858515515667], '0.000909_103.305785_1.818182_756.250000_0.000000_0.100000': [246.32811416892383, 0, 0, -872.8628253202729, 3.0892122601002483, 1964.2108887475536, 0, 0], '0.000909_93.914350_1.818182_756.250000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -246.32811416892383, 0], '0.000909_103.305785_2.000000_756.250000_0.000000_0.100000': [3883.9837108055044, 0, -949.3900735375527, -1328.01018539966, 5.087868143169908, 872.8628253202729, 0, 0], '0.000909_103.305785_1.652893_756.250000_0.000000_0.100000': [0, 1864.1695409214888, 0, 0, 0, 0, 0, 0], '0.000909_103.305785_1.652893_687.500000_0.000000_0.100000': [-922.5663209292018, -798.7070383383524, -1514.6250284392618, -227.67569531495812, -1296.4724078332026, -1143.1066405187028, -1466.6890143206413, -167.94320188481876], '0.000909_103.305785_1.652893_625.000000_0.000000_0.100000': [0, 0, 294.8879114371133, 0, 84.9887761177688, 19.611131961485444, 0, 798.7789939273919], '0.000909_103.305785_1.502630_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000826_113.636364_2.200000_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 1347.5179741611162, 0, 0], '0.000826_113.636364_1.818182_687.500000_0.000000_0.100000': [0, 1392.8943739660076, -3.338284384423805, 1063.818235378897, 0, 693.3184871396325, 0, 1305.5074050560197], '0.000909_113.636364_1.818182_687.500000_0.000000_0.100000': [307.95843251534444, 2126.237510535321, 0, -943.7514798558016, -422.112858250988, 0, 139.0118074141756, -524.3334808727508], '0.000909_103.305785_2.200000_756.250000_0.000000_0.100000': [313.9001279682343, 0, -4.60402957222741, -658.6484993688814, 0, 0, -285.4927927547702, -567.5788748279274], '0.000909_113.636364_2.200000_756.250000_0.000000_0.100000': [314.0420720302472, 0, -3.6924388063102014, -526.6281697539624, 0, 0, 0, 0], '0.000909_103.305785_2.420000_756.250000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -213.34959944610983, 0], '0.001100_137.500000_2.200000_687.500000_0.000000_0.100000': [323.172010841494, 511.0025194603787, 0, 0, 633.5304555708776, 3312.6399692401237, 0, 0], '0.001100_113.636364_2.420000_687.500000_0.000000_0.100000': [0, 0, 0, 0, 2.9972342913895775, 597.4310844738627, 0, 0], '0.001000_113.636364_2.420000_687.500000_0.000000_0.100000': [0, 0, -2.9972342913895775, 11719.21850598848, 0, 0, 172.30471630351255, -332.88066011913475], '0.001000_113.636364_2.662000_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 9418.42668581857], '0.001000_113.636364_2.662000_756.250000_0.000000_0.100000': [0, 0, 0, 0, 0, -12100.538047531249, 0, 0], '0.001000_113.636364_2.420000_756.250000_0.000000_0.100000': [234.17116207689003, 0, 0, 0, 0, 0, -241.40891371358654, 0], '0.001000_125.000000_2.420000_756.250000_0.000000_0.100000': [219.46264883053172, 497.5569994929683, 0, 0, 0, 0, 0, 0], '0.001000_125.000000_2.420000_687.500000_0.000000_0.100000': [1252.169585518117, 0, -3.0568356826704988, -525.3967426298867, 44.53368897998132, 2929.873977617752, -244.10885006280097, -497.5569994929683], '0.001000_103.305785_2.420000_756.250000_0.000000_0.100000': [205.24597669631748, 0, 0, 0, 0, 0, -212.88287461535455, 0], '0.001100_103.305785_2.000000_568.181818_0.000000_0.100000': [1560.0310915767063, 0, 1299.3591056526543, -1273.5805595922272, 0, 1479.56999627084, 0, 327.58703469026807], '0.001210_103.305785_2.000000_568.181818_0.000000_0.100000': [0, 705.1524668454636, -0.7392146438650343, 0, 0, 1024.787974344839, 18.949826051794275, -634.4286665506111], '0.001331_113.636364_2.200000_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 1257.3238502179427, -390.3726270414565, 0], '0.001210_103.305785_2.000000_516.528926_0.000000_0.100000': [0, 250.83423827688614, 0, -931.4499580414847, 0, 680.5724667058164, -54.924966699949074, 52.11379611683173], '0.001210_93.914350_2.000000_469.571751_0.000000_0.100000': [0, 0, 0, 0, 6.172245473549538, 0, 0, 0], '0.001100_93.914350_2.000000_469.571751_0.000000_0.100000': [963.5793759876511, 637.1830349411115, 0, 0, 38.43655603540299, 2851.859701764769, 0, 0], '0.001100_85.376682_2.000000_469.571751_0.000000_0.100000': [0, 0, 0, -1818.4530158414645, 4.419798178376893, -94.80953237676451, 0, 0], '0.001000_85.376682_2.000000_469.571751_0.000000_0.100000': [4554.8508154038445, 452.3547151997924, -3.9818001606998226, -181.5973976478497, 0, -326.9978007347031, 0, -337.90094288848354], '0.001000_85.376682_1.818182_469.571751_0.000000_0.100000': [0, 461.1970095306022, 0, 0, 3.227278331057678, 3447.669684155636, -2.5259723481853427, 0], '0.000909_85.376682_1.818182_469.571751_0.000000_0.100000': [0, 461.3929396952773, 0, 0, 3593.274692611695, 46.07509111933041, 0, 808.3574850039237], '0.000909_85.376682_1.652893_469.571751_0.000000_0.100000': [0, 0, 0, 0, 612.3726416535076, 0, -212.93730968085157, 0], '0.000826_85.376682_1.652893_469.571751_0.000000_0.100000': [0, 400.9728998928934, 484.14199541733797, 0, 0, 354.85031174543946, 0, 1084.282081133329], '0.000826_85.376682_1.652893_516.528926_0.000000_0.100000': [-784.2140460164625, -29.882662689810992, 97.36919932227201, 2994.123247469924, -5.642730414541418, -738.0241874988774, -1270.6676275012596, -390.45913264071123], '0.000826_85.376682_1.818182_516.528926_0.000000_0.100000': [-531.3262853180672, 4093.833171839704, 0, 0, 0, -594.0384903303553, -1469.922855603029, 0], '0.000826_93.914350_1.652893_516.528926_0.000000_0.100000': [-52.727727407845066, 0, 0, 0, 1282.0718692607031, 0, 2262.801860480691, -424.4419737070957], '0.000826_93.914350_1.652893_568.181818_0.000000_0.100000': [0, 602.4490077845805, 0, 0, 1.7936288915559089, 0, 0, 0], '0.000751_93.914350_1.652893_568.181818_0.000000_0.100000': [0, 423.8840213085968, 0, 0, 0, 0, 0, -447.58368896521324], '0.000751_93.914350_1.652893_516.528926_0.000000_0.100000': [1411.8380181731395, 0, 0, 0, 1.1291566642804356, 0, -238.94762983672888, 0], '0.001000_137.500000_2.000000_687.500000_0.000000_0.100000': [-166.37792011736462, 0, 786.8252684850751, -806.8226091663053, 102.68470682750501, -3595.1287782180025, 9.697020806100852, -640.6135262469396], '0.000909_137.500000_2.000000_831.875000_0.000000_0.100000': [0, 546.4574738358248, 0, 0, 0, 0, 0, 0], '0.001000_137.500000_1.652893_625.000000_0.000000_0.100000': [52.98307616334273, 1514.3506305860699, 0, -123.94703578385838, 3.3517620081896893, -273.074755667607, -343.8588379573929, -140.84892497982565], '0.001100_103.305785_1.818182_625.000000_0.000000_0.100000': [2318.1977733369563, 565.3139733219411, -2.966271875680468, -705.3615853958893, 66.35758200382838, 2156.5447846250636, 193.16899626596563, 0], '0.001100_103.305785_1.652893_625.000000_0.000000_0.100000': [434.3218525989139, 2029.110705949374, -2.4323168262875696, -203.95626465158404, 697.7617705368004, 17.57807248079389, -308.67477824919274, 0], '0.001000_93.914350_1.652893_625.000000_0.000000_0.100000': [284.2332538608872, 0, -2.6708092616340036, 0, 0, 0, 0, 0], '0.001000_85.376682_1.652893_625.000000_0.000000_0.100000': [0, 264.23394422591537, 0, 0, 0, 512.1352452113, -2.560659944692702, 0], '0.001000_85.376682_1.652893_568.181818_0.000000_0.100000': [0, 0, -1.8388416692891951, 0, 5672.7531680977145, 0, -156.3673574915788, -61.25234838650228], '0.001100_85.376682_1.652893_568.181818_0.000000_0.100000': [0, 0, 0, 3122.1925349340386, 0, 199.7931728495624, -260.8428005016126, 0], '0.001100_85.376682_1.818182_568.181818_0.000000_0.100000': [-3264.3897880387935, -3093.761806762862, -2652.1873044928775, -3154.7852968976067, -2879.455072234543, -3671.11318507746, -4041.649669948416, -4032.975330380978], '0.001000_85.376682_1.818182_568.181818_0.000000_0.100000': [-3299.1598387891327, -3453.8376186056985, -3370.8314211240017, -3304.716379134065, -3660.9016933379908, -3705.8141214374377, 472.08014929342244, -3701.8562300806416], '0.000909_85.376682_1.652893_568.181818_0.000000_0.100000': [5815.691725789107, 975.0783317120952, 984.0482237722625, 0, 0, 204.93749658560856, 0, 640.4384298115292], '0.000909_85.376682_1.502630_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 3016.340546568693, 0, 0], '0.000909_85.376682_1.366027_568.181818_0.000000_0.100000': [-1478.506629410404, -553.0264059745682, 1322.1455542288206, -750.5919508348901, -1117.625569540377, 0, -2278.8477775570827, 0], '0.000909_93.914350_1.366027_568.181818_0.000000_0.100000': [435.25992551340255, 2498.291455365151, -4.765259546109974, 0, 8.224361028535819, 0, 0, 0], '0.001000_125.000000_2.000000_831.875000_0.000000_0.100000': [-671.7863009264066, -523.4775867718874, 0, 0, 0, 0, 0, 0], '0.000909_125.000000_1.818182_516.528926_0.000000_0.100000': [862.3861373131226, 495.9466933901288, 678.9673056153172, 2550.7527924582655, -391.63317830439837, 66.85496145621389, 71.88620165053774, -345.70596886236183], '0.000909_113.636364_1.818182_426.883410_0.000000_0.100000': [419.55941573796946, 489.6899127774082, 508.4409443743905, 0, 26.198412960977123, 297.81595168490765, -279.137528348679, -34.29793380569708], '0.000826_113.636364_1.818182_426.883410_0.000000_0.100000': [1334.2089035419551, 0, 0, 0, 0, -37.99508288612358, 0, -182.86848468998127], '0.000826_113.636364_1.818182_469.571751_0.000000_0.100000': [1365.3213681870434, 108.00458939569967, 0, 0, 191.31917696718392, 210.30458905235602, -356.68922688833663, 586.1861736397343], '0.000826_125.000000_1.818182_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 2100.16287034518, -403.98969967214043, 0], '0.000826_125.000000_1.652893_469.571751_0.000000_0.100000': [0, 1257.6375210911815, 0, 36.95987270382555, -393.29719778407264, 0, -335.7144266939676, 0], '0.000826_125.000000_1.652893_426.883410_0.000000_0.100000': [690.4748364004447, 468.07999568655725, 120.80083749845653, -262.49228684631436, 0.36102683636359867, 0, 0, 0], '0.000826_113.636364_1.652893_426.883410_0.000000_0.100000': [3581.136323882045, 0, 512.3136039916052, 96.1278321438139, 38.153308601780225, 62.60857277486057, -61.5010896871689, -161.80640478746744], '0.000751_113.636364_1.652893_426.883410_0.000000_0.100000': [0, 402.8324420587646, -1.8812862145410691, -214.43772842735106, 0, 64.40081466897064, -307.41057444906403, 0], '0.000751_113.636364_1.652893_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -314.0594477206849, -402.8324420587646], '0.000751_113.636364_1.818182_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, 235.88150127008618, 0, 0], '0.000751_113.636364_1.502630_426.883410_0.000000_0.100000': [0, 468.4805000398478, 0, 0, 0, 0, 0, 0], '0.000751_113.636364_1.502630_388.075827_0.000000_0.100000': [498.37947725085775, 0, -3.9030090785870017, 0, 3.571868246351096, 0, 0, 0], '0.000826_113.636364_1.502630_388.075827_0.000000_0.100000': [0, 0, 0, 0, 820.3196793010841, 0, 0, 0], '0.000751_103.305785_1.502630_388.075827_0.000000_0.100000': [0, 504.1098917799037, 0, 0, 0, 0, -411.83254276474383, 0], '0.000751_103.305785_1.502630_352.796206_0.000000_0.100000': [0, 0, 0, 0, 0, 588.0259941447379, 0, 0], '0.000826_103.305785_1.818182_625.000000_0.000000_0.100000': [277.83479480783683, -60.46353891998982, 0, 0, 0, 0, -279.2240609883224, -500.0657626107437], '0.000826_113.636364_1.818182_625.000000_0.000000_0.100000': [0, 2538.129117790369, 268.3399892339214, 752.833731108266, 46.8688789128511, 184.94022585872904, -1079.2637926544621, 0], '0.000826_103.305785_1.652893_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 585.4666745704023, 391.31781867847747], '0.000826_113.636364_1.652893_687.500000_0.000000_0.100000': [0, 449.90077234926827, 26.39838734682151, 126.3394580915024, 2.7235045361780976, 31.065788291108767, 0, -493.15251227361495], '0.000751_113.636364_1.652893_687.500000_0.000000_0.100000': [289.07738592884033, 1564.7121215290413, 0, 0, 2.241232585919352, 0, 0, 0], '0.000751_103.305785_1.652893_687.500000_0.000000_0.100000': [0, 1359.9254452564521, 96.56835409953135, -239.83290223738024, 0, 0, 0, -495.8108117878293], '0.000751_103.305785_1.818182_687.500000_0.000000_0.100000': [272.7692360280977, 0, 0, 0, 0, 0, 0, -529.6419284235608], '0.000826_93.914350_1.818182_687.500000_0.000000_0.100000': [3434.096683960385, 0, 0, 0, 0, 0, -248.3343867154652, 0], '0.000826_103.305785_1.818182_687.500000_0.000000_0.100000': [3336.0108137841103, 0, -2.6702725332896082, 614.958813241481, 225.59074007463167, 0, 207.29843881506818, -53.00247866698279], '0.000909_103.305785_1.818182_687.500000_0.000000_0.100000': [0, 592.7766855551356, 750.7609495050426, 528.8258698066966, 2.9640025119478635, 0, 0, 0], '0.000826_85.376682_1.818182_687.500000_0.000000_0.100000': [-1902.7584611300233, -1604.1740510069444, -158.6909787636989, -4641.054559918939, 0, -2877.0438843082675, 0, 0], '0.000826_85.376682_1.818182_625.000000_0.000000_0.100000': [0, 0, 0, 0, 4.222282116203196, 0, 0, 0], '0.000751_85.376682_1.818182_625.000000_0.000000_0.100000': [199.2501547883462, 0, 0, 0, 0, -505.2016732285612, -989.4797672108024, 0], '0.000683_93.914350_1.818182_625.000000_0.000000_0.100000': [0, 500.7143458168308, -295.2571368361429, 0, 0, 0, 0, 0], '0.000683_93.914350_1.818182_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 556.6435092855708, 0, 0], '0.001464_125.000000_2.200000_687.500000_0.000000_0.100000': [0, 0, 0, -454.18127382824844, 0, 0, 0, -492.33972871773386], '0.001464_125.000000_2.200000_756.250000_0.000000_0.100000': [0, 492.33972871773386, -5.490633321943824, 0, 0, 0, 0, 0], '0.001611_125.000000_2.200000_756.250000_0.000000_0.100000': [0, 0, -6.057508020500973, 0, 0, 0, 0, 0], '0.001772_125.000000_2.200000_756.250000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000826_125.000000_1.818182_756.250000_0.000000_0.100000': [299.3863748832587, 2280.4968248183386, -3.6332723660114428, 0, 0, 0, 0, -331.44133509873035], '0.000826_125.000000_1.818182_831.875000_0.000000_0.100000': [312.4394335192368, 0, 0, 0, 0, 0, 0, 0], '0.000826_113.636364_1.818182_831.875000_0.000000_0.100000': [261.8227249535152, 0, 0, -974.8960878844464, 0, 0, 0, -563.8996940925501], '0.000826_113.636364_2.000000_831.875000_0.000000_0.100000': [331.04775648902955, 0, 0, 0, 0, 0, 0, 0], '0.000826_103.305785_2.000000_831.875000_0.000000_0.100000': [0, 0, 0, -1156.35128742763, 4.578256493870322, 0, 0, 0], '0.000751_103.305785_2.000000_831.875000_0.000000_0.100000': [0, 0, 0, 0, 0, 934.3019521194009, 0, 0], '0.000751_103.305785_1.818182_831.875000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -261.62214002662733, -575.6213651478828], '0.000751_113.636364_1.818182_831.875000_0.000000_0.100000': [0, 543.0159184279688, 0, 0, 0, 0, 0, 0], '0.000826_103.305785_1.818182_831.875000_0.000000_0.100000': [239.749320789764, 0, 0, 0, 0, 0, 0, 0], '0.000826_93.914350_1.818182_831.875000_0.000000_0.100000': [0, 561.6325823257575, 0, 0, 0, 0, 0, 0], '0.000826_93.914350_1.818182_756.250000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001100_151.250000_2.000000_568.181818_0.000000_0.100000': [0, 0, 140.9713894991437, 0, 0, 1070.650296649833, 0, 0], '0.001100_151.250000_1.818182_568.181818_0.000000_0.100000': [2231.8370945925826, 577.7037812817396, -4.706182697342593, 0, 0, 916.883760974854, 0, 0], '0.001100_151.250000_1.818182_516.528926_0.000000_0.100000': [0, -326.01005707866125, 614.1805580235898, 0, 69.47441754647332, 332.6421022841709, 0, 0], '0.001100_151.250000_1.818182_469.571751_0.000000_0.100000': [442.9871108560218, 461.0904623740102, 0, -941.9161178347392, 0, 0, 0, 186.88993850770433], '0.001100_151.250000_1.818182_426.883410_0.000000_0.100000': [776.3793162621276, 0, 0, -786.5168539712807, 323.0004001667583, 75.74685423059572, 0, -1141.7798786302776], '0.001100_151.250000_2.000000_426.883410_0.000000_0.100000': [0, 458.2223248571629, 0, 0, 0, 0, 0, 0], '0.001100_151.250000_2.000000_388.075827_0.000000_0.100000': [0, 0, 0, 0, 5.8628124413635305, 0, -486.99823566187587, 0], '0.001100_166.375000_2.000000_388.075827_0.000000_0.100000': [0, 535.3536625377443, 0, 0, 0, 0, 0, -455.37200916726925], '0.001100_166.375000_2.000000_426.883410_0.000000_0.100000': [0, 455.37200916726925, 0, 0, 0, 0, 0, 0], '0.001100_166.375000_2.000000_352.796206_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -355.6135810809943, 0], '0.001100_183.012500_2.000000_352.796206_0.000000_0.100000': [0, 0, -5.814364743323859, 0, 0, 0, 0, 0], '0.001210_183.012500_2.000000_352.796206_0.000000_0.100000': [355.47760733152285, 0, 0, 0, 0, 0, 0, 0], '0.001210_166.375000_2.000000_352.796206_0.000000_0.100000': [0, 547.8231348567363, 0, 0, 0, 0, 0, 0], '0.001210_166.375000_2.000000_320.723824_0.000000_0.100000': [0, 0, 0, 0, 5.486440201111691, 0, 0, 0], '0.001100_166.375000_2.000000_320.723824_0.000000_0.100000': [0, 0, 0, 0, 0, 417.92953694178885, 0, 0], '0.001100_166.375000_1.818182_320.723824_0.000000_0.100000': [0, 434.644744567572, 0, 0, 0, 0, 0, 0], '0.001100_166.375000_1.818182_291.567113_0.000000_0.100000': [0, 0, 0, -319.6616574441324, 3.6521398981853963, 0, 0, 0], '0.001000_166.375000_1.818182_291.567113_0.000000_0.100000': [0, 0, -3.6521398981853963, 0, 0, 0, 0, 0], '0.001100_166.375000_2.000000_291.567113_0.000000_0.100000': [0, 498.35550483580573, 0, 0, 4.6759801515022446, 0, 0, 0], '0.001100_166.375000_2.000000_265.061011_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -498.35550483580573], '0.001000_166.375000_2.000000_291.567113_0.000000_0.100000': [347.79750973155245, 0, 0, 0, 0, 0, 0, 0], '0.001000_151.250000_2.000000_291.567113_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -529.845072930015], '0.001000_151.250000_2.000000_320.723824_0.000000_0.100000': [0, 0, -5.1477218009996255, 0, 0, 0, 0, 0], '0.001100_151.250000_2.000000_320.723824_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001100_103.305785_1.818182_568.181818_0.000000_0.100000': [1954.8343643575165, 244.22755111293725, 387.3273667103124, 1908.9112252840516, 1801.5616897092464, 1942.24046367127, 161.94767393321786, -382.87938041104746], '0.001100_93.914350_1.818182_568.181818_0.000000_0.100000': [4850.180544682142, 481.1800117058846, 772.420446175182, 1162.682426271429, -19.076825392188738, 198.56438670799972, 188.54068978090046, 0], '0.001000_103.305785_1.652893_568.181818_0.000000_0.100000': [741.2081118686851, 0, 102.43086822372084, 239.90694163154234, 886.132704070302, 93.28386431030401, -190.79561219854588, 66.43774097035225], '0.001100_103.305785_1.652893_568.181818_0.000000_0.100000': [280.28195566088016, 0, -2.55837234109531, -893.6262322567579, 2.2990123178235082, 1170.781523587393, -259.74339137080824, -441.4977861771396], '0.001331_103.305785_1.818182_516.528926_0.000000_0.100000': [0, 0, -4.475860803972774, -527.4081759064587, 4.036810510611668, 0, 0, 0], '0.001464_103.305785_1.818182_516.528926_0.000000_0.100000': [280.83920419095114, 0, 0, 0, 0, 0, 0, 0], '0.001464_93.914350_1.818182_516.528926_0.000000_0.100000': [0, 843.0352967417458, 0, 0, 0, 0, 0, 0], '0.001464_93.914350_1.818182_469.571751_0.000000_0.100000': [525.1253861391876, 223.49456846882146, 0, 0, 0, 0, 0, 0], '0.001464_85.376682_1.818182_469.571751_0.000000_0.100000': [681.4382969696736, 0, 0, 0, 0, 0, -278.98028737134376, 0], '0.001464_93.914350_1.818182_426.883410_0.000000_0.100000': [0, 431.91939317762547, 0, -356.35985763616293, 0, 0, -313.9983697896819, -46.832722028027185], '0.001464_103.305785_1.818182_426.883410_0.000000_0.100000': [0, 0, 0, -407.8358010204136, 0, 0, 0, -447.57267838791904], '0.001464_103.305785_2.000000_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, 407.8358010204136, 0, 0], '0.001464_103.305785_1.818182_469.571751_0.000000_0.100000': [0, 0, 0, 0, 3.6700187991401436, 0, 0, 0], '0.001331_103.305785_1.818182_469.571751_0.000000_0.100000': [841.3514598247386, 0, 0, -153.73810920426132, 0, 0, 0, 0], '0.001331_93.914350_1.818182_469.571751_0.000000_0.100000': [278.23588559383217, 1327.9434499472513, 245.90973455740533, -36.44578611261385, 0, 0, 0, 0], '0.000826_137.500000_1.818182_625.000000_0.000000_0.100000': [0, 0, 1229.8092248176802, 0, 26.85471654049698, 717.1594336444139, 0, -484.70536019638797], '0.000826_137.500000_1.652893_625.000000_0.000000_0.100000': [308.7977583204891, 1547.7762540563785, 0, -260.5112410938642, 5.535189230029518, -392.8106748361279, -324.8842248315121, -526.0974148857068], '0.000751_137.500000_1.502630_756.250000_0.000000_0.100000': [278.52851795041045, 0, 0, 0, 0, 0, 0, 0], '0.000751_125.000000_1.502630_756.250000_0.000000_0.100000': [281.93798809190923, 0, 0, 0, 0, 0, 0, 0], '0.000751_113.636364_1.502630_756.250000_0.000000_0.100000': [0, 426.49164859666837, 0, 0, 0, 0, 0, 0], '0.000751_113.636364_1.502630_687.500000_0.000000_0.100000': [0, 0, 0, 0, 1.26985470080097, 0, 0, 0], '0.000683_113.636364_1.502630_687.500000_0.000000_0.100000': [0, 0, -0.11544133643645178, -28.78739647207567, 0, 0, 0, 0], '0.000683_113.636364_1.652893_687.500000_0.000000_0.100000': [0, 1592.122895828908, -2.241232585919352, 0, 0, 0, 0, 0], '0.000683_113.636364_1.818182_625.000000_0.000000_0.100000': [524.4033023822351, 231.94966494128892, 0, -180.94224963293658, 0, 0, 0, 0], '0.000683_103.305785_1.818182_568.181818_0.000000_0.100000': [0, 528.2489216415642, 0, 0, 0, 178.22074448528937, 0, -515.8715066735277], '0.000683_103.305785_1.818182_625.000000_0.000000_0.100000': [0, 0, -1.7731018997850698, 0, 0, 981.7758431883083, 0, 0], '0.000683_103.305785_1.652893_568.181818_0.000000_0.100000': [280.92712509315834, 0, 0, 0, -173.19739841048087, 0, 0, 0], '0.000621_103.305785_1.652893_568.181818_0.000000_0.100000': [769.0355395627192, 0, 0, -195.88793661261178, 0, 0, 0, 0], '0.000621_103.305785_1.818182_568.181818_0.000000_0.100000': [0, 0, -1.7587607259431024, -581.203526886838, 1.5918666787929396, 803.3616604890183, -279.90903050477937, 0], '0.000909_166.375000_2.000000_687.500000_0.000000_0.100000': [463.11329123851954, 0, -917.9489515474786, 0, 0, 0, 0, 0], '0.000826_151.250000_1.652893_756.250000_0.000000_0.100000': [0, 0, -3.9283466861927447, 0, 0, 0, 0, 0], '0.000909_151.250000_1.652893_756.250000_0.000000_0.100000': [0, 0, 0, 0, 0, 148.5449865183826, 0, -499.07250222904406], '0.000909_151.250000_1.652893_831.875000_0.000000_0.100000': [0, 0, 0, 0, 3.351121928624234, 0, 0, 0], '0.000826_151.250000_1.652893_831.875000_0.000000_0.100000': [320.9854816344156, 0, 0, -307.1210112600094, 0, 0, 0, 0], '0.000826_137.500000_1.652893_831.875000_0.000000_0.100000': [0, 0, -3.4291850286146657, 0, 0, 0, 0, 0], '0.000909_137.500000_1.652893_831.875000_0.000000_0.100000': [306.87429514167616, 0, 0, -301.0893019853485, 0, 0, 0, -500.1771505837367], '0.000909_137.500000_1.652893_915.062500_0.000000_0.100000': [0, 0, 0, 0, 3.185801075820746, 0, 0, 0], '0.000826_137.500000_1.652893_915.062500_0.000000_0.100000': [0, 0, 0, 0, 2.989774053515976, 0, 0, 0], '0.000751_137.500000_1.652893_915.062500_0.000000_0.100000': [0, 500.5407339741365, 0, 0, 0, 0, 0, 0], '0.000751_137.500000_1.652893_831.875000_0.000000_0.100000': [306.3809381809535, 0, 0, 0, 0, 0, 0, 0], '0.000751_125.000000_1.652893_831.875000_0.000000_0.100000': [0, 0, -2.874387477119672, 0, 0, 0, 0, 0], '0.000826_125.000000_1.652893_831.875000_0.000000_0.100000': [0, 492.92409203851696, -3.1714140818941816, 0, 0, 0, 0, 0], '0.000909_125.000000_1.652893_831.875000_0.000000_0.100000': [429.1479208597776, 0, 0, 0, 0, 190.0323124685041, -306.87429514167616, 0], '0.000909_125.000000_1.502630_831.875000_0.000000_0.100000': [0, 0, 0, 0, 1.3099993693753278, 0, 0, 0], '0.000826_125.000000_1.502630_831.875000_0.000000_0.100000': [255.42274550089934, 0, 0, 0, 0, 0, 0, 0], '0.000826_113.636364_1.502630_831.875000_0.000000_0.100000': [0, 0, -1.429684974700649, 0, 0, 0, 0, 0], '0.001000_113.636364_1.502630_831.875000_0.000000_0.100000': [257.13026062511017, 0, 0, -163.47272287115285, 0, 0, 0, 0], '0.000826_166.375000_2.000000_625.000000_0.000000_0.100000': [0, 514.3851665544657, 0, 0, 0, 0, 0, 0], '0.000826_166.375000_2.000000_568.181818_0.000000_0.100000': [0, 0, -5.099426263766327, -978.9084157697122, 0, 992.8501191126257, -461.28026479513863, -46.76228786858777], '0.000826_166.375000_1.818182_568.181818_0.000000_0.100000': [419.426602433279, 0, -3.829462131800028, -992.8501191126257, 0, 0, 0, 0], '0.000909_166.375000_2.000000_568.181818_0.000000_0.100000': [0, 0, 0, -530.7792497378236, 5.099426263766327, 0, 0, 0], '0.000751_151.250000_2.200000_568.181818_0.000000_0.100000': [0, 0, 0, 0, 3.534800690254997, 0, 0, 0], '0.000621_151.250000_2.200000_568.181818_0.000000_0.100000': [0, 0, 0, 0, 2.9204355306235814, 0, 0, 0], '0.000564_151.250000_2.200000_568.181818_0.000000_0.100000': [0, 0, 0, 0, 2.6543071522855826, 0, 0, -485.3918381599282], '0.000513_151.250000_2.200000_568.181818_0.000000_0.100000': [0, 0, -2.6543071522855826, 0, 0, 0, 0, 0], '0.000564_151.250000_2.200000_625.000000_0.000000_0.100000': [0, 0, 0, -264.7676428978544, 0, 0, 0, 0], '0.000564_151.250000_2.420000_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 24.069785717986758, 0, 0], '0.000909_125.000000_2.000000_756.250000_0.000000_0.100000': [492.3603105245588, 0, 0, 0, 0, 1072.1025573602226, -360.4393855422697, 0], '0.000826_125.000000_1.652893_756.250000_0.000000_0.100000': [0, 0, 0, -260.24164957414035, 0, 110.76992423659013, 0, -492.92409203851696], '0.000751_113.636364_2.000000_756.250000_0.000000_0.100000': [323.99401679308943, 738.3844296790572, 0, 0, 0, 0, 0, 0], '0.000751_113.636364_2.000000_687.500000_0.000000_0.100000': [0, 0, 85.14388619409829, 739.6720939933615, 4.234305759807467, 0, 0, 809.6642852211542], '0.001000_166.375000_2.200000_625.000000_0.000000_0.100000': [0, 558.3616790814439, 0, 241.12505226248442, 0, 970.9189391708013, 0, -422.3378913779334], '0.001000_166.375000_2.200000_516.528926_0.000000_0.100000': [0, 0, 0, 0, 47.52558047366057, 0, 0, 0], '0.000909_166.375000_2.200000_516.528926_0.000000_0.100000': [0, 1332.0271849107721, 0, 0, 0, 936.3612428258984, -351.3432186384693, 0], '0.000909_183.012500_2.200000_516.528926_0.000000_0.100000': [0, 0, -4.079971777064202, 0, 0, 0, 0, -475.8570159263425], '0.001000_183.012500_2.200000_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -475.4494850227626], '0.001000_201.313750_2.200000_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 12088.819363188093], '0.001000_201.313750_2.200000_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, -11777.36825667072, 0, 0], '0.001000_201.313750_2.000000_625.000000_0.000000_0.100000': [404.49144781763624, 0, 0, 0, 0, 0, 0, 0], '0.001000_183.012500_2.000000_625.000000_0.000000_0.100000': [0, 0, -5.820016783499341, 0, 0, 0, 0, 0], '0.001100_183.012500_2.000000_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 1045.8870941550313, 0, 0], '0.001100_183.012500_1.818182_625.000000_0.000000_0.100000': [0, 414.31138259083747, 0, 0, 0, 0, 0, 0], '0.001100_183.012500_1.818182_568.181818_0.000000_0.100000': [0, 0, 0, 0, 4.975918871345676, 0, 0, 0], '0.001000_166.375000_1.818182_625.000000_0.000000_0.100000': [0, 0, 0, -609.7062990521125, 0, 0, 0, -446.57612090125144], '0.000909_183.012500_1.818182_687.500000_0.000000_0.100000': [411.2282945121013, 0, 0, 0, 0, 0, 0, 0], '0.000909_166.375000_1.818182_625.000000_0.000000_0.100000': [404.54644006216824, 0, 0, -1029.3533698562735, 0, 0, 0, 0], '0.000826_125.000000_2.200000_568.181818_0.000000_0.100000': [429.56128358274236, 0, 0, 0, 0, 1366.297023948657, 0, 822.9627252954139], '0.000826_113.636364_2.200000_568.181818_0.000000_0.100000': [373.72400702194466, 0, -4.85463851483235, 0, 0, 0, 725.8732035401305, 1099.338781344679], '0.000826_113.636364_2.200000_625.000000_0.000000_0.100000': [387.7281752017751, 533.7719380942443, 2119.0005625076888, -642.3940740777074, 4.068470416641867, 2548.9083287200506, -366.78755956202934, 542.4070433372498], '0.000826_93.914350_2.200000_568.181818_0.000000_0.100000': [2395.172951951257, 573.4388163299669, 0, -803.0949303063895, 0, 0, 0, -589.388804451437], '0.000826_93.914350_2.200000_625.000000_0.000000_0.100000': [349.6233886028867, 0, 0, -751.0958931988906, 0, 0, 0, 0], '0.000751_103.305785_2.420000_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -260.4994592405223, 0], '0.000751_113.636364_2.420000_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 628.0145164502892, 0, 0], '0.000751_113.636364_2.200000_568.181818_0.000000_0.100000': [0, 520.6621029231449, 0, 0, 0, 1266.0212208400444, 0, -533.7137205624015], '0.000751_113.636364_2.200000_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 1406.2015458846934, 0, -572.7283132154593], '0.000751_113.636364_2.000000_516.528926_0.000000_0.100000': [403.3134471919224, 0, -4.593530764830939, -255.67300834267155, 0, 0, -431.313484012415, 94.91622848020404], '0.001100_125.000000_1.818182_756.250000_0.000000_0.100000': [353.55426437815913, 0, 0, 0, 286.91405656037267, 0, -354.17274374502426, 0], '0.001000_103.305785_2.000000_831.875000_0.000000_0.100000': [283.5886458055402, 867.8678487843282, -6.09320051056202, 0, 0, 0, 0, 0], '0.001100_103.305785_2.000000_831.875000_0.000000_0.100000': [0, 0, 0, 0, 6.09320051056202, 0, 0, -626.1065927322245], '0.001100_103.305785_2.000000_915.062500_0.000000_0.100000': [0, 0, -6.591442504810948, 0, 0, 0, 0, 0], '0.001210_103.305785_2.000000_915.062500_0.000000_0.100000': [0, 625.9976447099806, 0, 0, 0, 0, 0, 0], '0.001210_103.305785_2.000000_831.875000_0.000000_0.100000': [3596.3605216781093, 0, 0, 0, 0, 0, 0, 0], '0.001210_93.914350_2.000000_756.250000_0.000000_0.100000': [358.49382644913203, -475.55026485222425, 0, -4236.640168057696, 0, 0, 0, 0], '0.001210_85.376682_2.000000_756.250000_0.000000_0.100000': [0, 0, 0, 0, 7.771211765308453, 1072.6998941600345, 0, 0], '0.001210_85.376682_1.818182_756.250000_0.000000_0.100000': [66.38383015748076, 0, -813.072564639662, 0, 0, 0, 0, 0], '0.001331_85.376682_1.818182_756.250000_0.000000_0.100000': [-19.46015437826345, -19.248356899321198, 0, -1099.1589857906233, 0, -2907.2124858389975, 0, 0], '0.001331_85.376682_2.000000_756.250000_0.000000_0.100000': [-2708.210297154121, -2326.301383555274, -1.8341943826232636, 0, 0, 0, 0, -293.4817024965529], '0.001331_85.376682_2.000000_831.875000_0.000000_0.100000': [88.38730171118269, 0, 0, 0, 0, 0, 0, 0], '0.001331_77.615165_2.000000_831.875000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -3868.6250651504006], '0.001331_77.615165_2.000000_915.062500_0.000000_0.100000': [0, 0, -8.04244437251, 0, 0, 0, 0, 0], '0.001464_77.615165_2.000000_915.062500_0.000000_0.100000': [0, 3872.764101071568, 0, 0, 0, 0, 0, 0], '0.001464_77.615165_2.000000_831.875000_0.000000_0.100000': [0, 0, 0, 0, 0, -691.1114337324525, 0, 0], '0.001464_77.615165_1.818182_831.875000_0.000000_0.100000': [0, 0, 0, 691.1114337324525, 0, 0, 0, 0], '0.001000_113.636364_1.818182_568.181818_0.000000_0.100000': [4094.776638119173, -220.62097799523394, 609.425964997619, 245.0406547665084, -164.11399889134958, -88.28155520857581, -729.5614268455295, -646.9420517095598], '0.001000_113.636364_1.652893_568.181818_0.000000_0.100000': [882.2811346944763, 1941.341815159, 39.49860183170142, -1473.4532209360873, 975.5348344849926, 4446.905917697639, 352.5139572498822, 462.46869569552797], '0.001000_103.305785_1.652893_469.571751_0.000000_0.100000': [0, 4805.01024248516, -2.312687220661974, -165.74719976934975, 4.017017036433913, 9.018858517322906, 534.2499109809551, -455.44003876313303], '0.000909_103.305785_1.652893_469.571751_0.000000_0.100000': [624.8518648046849, 0, 2648.5502396146426, -147.5371389268911, 809.2753228227665, 0, 0, 0], '0.001000_113.636364_1.652893_516.528926_0.000000_0.100000': [499.42184621877414, 5063.591932627845, -0.25258426021385283, 569.489311320941, 375.8944569082621, 143.49915846836348, -522.6884443810845, -467.65824871063955], '0.001000_113.636364_1.818182_516.528926_0.000000_0.100000': [236.8445153191783, 471.8710137596886, 439.0674508479249, 526.0079240284513, -439.31487117241477, 4338.24827553399, -61.66074695232783, 625.4767061382765], '0.001000_113.636364_1.652893_469.571751_0.000000_0.100000': [4803.702119135285, 1357.308295660429, 227.27945793600202, 720.1835166034409, 2.127111784326189, 1516.68774240305, 210.33970940493543, 509.07990953418147], '0.001000_113.636364_1.652893_426.883410_0.000000_0.100000': [927.2859611012399, 786.7896849374, 0, 978.2628533682347, 2.324701440334284, 0, 0, 420.77145222351], '0.001000_113.636364_1.502630_469.571751_0.000000_0.100000': [310.2273092982896, 679.331703792863, 217.6099581910355, 826.3370055800337, 17.91465105473675, 2802.845315130599, -386.1886044203564, 0], '0.000909_113.636364_1.502630_469.571751_0.000000_0.100000': [0, 0, 0, 1489.2592937357329, 0, 0, 0, 0], '0.000826_113.636364_1.652893_469.571751_0.000000_0.100000': [1276.7988818373105, 731.2913484046971, 0, -210.30458905235602, 0, 0, 0, 0], '0.000826_103.305785_1.652893_426.883410_0.000000_0.100000': [3414.456778114042, 0, 169.10970522720217, -553.9801069283626, 209.60558579436008, 1546.9112180467664, 292.5749245633286, -844.149297461838], '0.001100_137.500000_2.200000_625.000000_0.000000_0.100000': [1205.2366163466013, 0, -5.063678082155093, -366.38066348227443, 0, 4071.1052461747618, -230.00120460607695, 849.1613632277148], '0.001331_125.000000_2.200000_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 1248.2236542282158, 0, -510.2469678924537], '0.001331_113.636364_2.200000_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -603.1931819609966], '0.001210_113.636364_2.200000_687.500000_0.000000_0.100000': [0, 976.2308660515337, -0.6040810193182912, 0, 0, 0, 0, -598.3168758024462], '0.001210_113.636364_2.200000_756.250000_0.000000_0.100000': [0, 0, -4.701275438373341, 0, 0, 0, 0, -529.7846474735821], '0.001331_113.636364_2.200000_756.250000_0.000000_0.100000': [0, 0, 0, 0, 0, 1159.5793117035792, 0, 0], '0.001331_113.636364_2.000000_756.250000_0.000000_0.100000': [323.96807758498744, 0, 0, 0, 0, 0, -342.48820667538115, -596.8098085886999], '0.001331_113.636364_2.000000_831.875000_0.000000_0.100000': [0, 0, -8.072597342144219, 0, 0, 0, 0, 0], '0.001464_113.636364_2.000000_831.875000_0.000000_0.100000': [300.800466855127, 0, 0, 0, 0, 0, 0, 0], '0.001464_103.305785_2.000000_831.875000_0.000000_0.100000': [283.5408249638109, 0, -8.914098089951949, 0, 0, 0, 0, 0], '0.001611_103.305785_2.000000_831.875000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -625.5905432152749], '0.001611_103.305785_2.000000_915.062500_0.000000_0.100000': [0, 0, -9.636517503392316, 0, 0, 0, 0, 0], '0.001772_103.305785_2.000000_915.062500_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001000_103.305785_2.200000_756.250000_0.000000_0.100000': [858.3715361001583, 0, -5.058302777789686, 0, -534.2901971841301, 0, -284.581201988853, 0], '0.001100_93.914350_2.200000_756.250000_0.000000_0.100000': [0, 0, 0, -671.2426060477037, 0, 0, -280.75493791693174, 0], '0.001210_103.305785_2.200000_756.250000_0.000000_0.100000': [280.24794742356664, 0, 0, -594.060056687966, 0, 0, 0, 0], '0.001000_113.636364_1.818182_469.571751_0.000000_0.100000': [21.875743511616292, 35.64435180285892, 34.405858189901096, -807.7283328819283, 82.71357279090603, 2540.4389868776007, 1497.260554149072, -452.9939721862412], '0.001000_125.000000_1.818182_426.883410_0.000000_0.100000': [496.45113900211834, 1335.876757649555, 414.9888068556337, 0, 91.24218355378463, 0, 0, 322.1915845929484], '0.001100_125.000000_1.818182_426.883410_0.000000_0.100000': [553.9655834314937, 441.45048475081614, -3.885424715128829, 3.8355083011003672, 0.04968682619823994, 568.9346805647273, -77.13520069122755, -42.31821316146998], '0.001000_151.250000_1.652893_516.528926_0.000000_0.100000': [355.8155660283925, 956.7712855386596, 0, 0, 3.6039899534052893, 0, 0, 0], '0.001000_137.500000_1.652893_568.181818_0.000000_0.100000': [2838.691226186109, 1204.7258449890962, 0, -274.41518108939147, 3.1475733273397055, 251.66683226388528, 9.783546712020211, -52.332266992310906], '0.000909_137.500000_1.652893_625.000000_0.000000_0.100000': [2336.315088573084, 0, 0, -598.8669867629919, 3.4446510865940763, -53.69490726937682, -324.7268512896903, 0], '0.000909_113.636364_1.652893_687.500000_0.000000_0.100000': [0, 0, 0, -1004.1244956030322, 0, 303.0073472074184, 0, -493.55466270147355], '0.000909_113.636364_1.502630_687.500000_0.000000_0.100000': [598.6836066054511, 0, 0, -35.60860402263963, 0, 0, 0, -426.48021153650683], '0.000909_113.636364_1.652893_756.250000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001100_113.636364_1.818182_568.181818_0.000000_0.100000': [794.167951491037, 1230.3898259960686, -7.512767747242599, -388.49677691784905, -47.32782551508171, 1087.996635320871, 330.55531355337166, -91.55585762870996], '0.001210_113.636364_1.818182_568.181818_0.000000_0.100000': [556.0892114542393, 1988.7232627291514, -3.6072992876971512, 0, 3.2690909253089586, 1387.4625185130037, -320.67463900671225, -299.3348945550247], '0.001100_113.636364_1.818182_516.528926_0.000000_0.100000': [532.231811250314, 1944.5814831907358, 518.0384788817055, 424.9008181322067, 556.8202903698314, 324.83246778616524, 16.791900640188466, -558.0053383358611], '0.001210_103.305785_1.818182_568.181818_0.000000_0.100000': [1878.5790335448596, 100.72161282351612, 0, -600.2915025330428, 0, 270.2147783291873, 0, 0], '0.001210_103.305785_1.652893_469.571751_0.000000_0.100000': [245.38828591184648, 916.7840358737279, -2.7174982983931844, 0, 0, 84.93463105306829, -299.44540258979606, 0], '0.001210_103.305785_1.652893_426.883410_0.000000_0.100000': [826.5962857433752, 0, -2.0674299455257277, 0, 0, 0, 0, -437.5269775931551], '0.001331_103.305785_1.652893_426.883410_0.000000_0.100000': [233.8425388905187, 0, 0, 0, 0, 0, 0, -438.3649941228885], '0.001331_93.914350_1.652893_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, 366.37766845481593, 0, -426.06215599712226], '0.001331_93.914350_1.502630_426.883410_0.000000_0.100000': [342.42742258685615, 565.8309266640105, 0, 0, 0, 0, 0, -523.9218427839754], '0.001331_93.914350_1.502630_388.075827_0.000000_0.100000': [0, 0, 0, 0, 6.160541176898378, 0, 0, 0], '0.000751_137.500000_1.818182_625.000000_0.000000_0.100000': [0, 458.5525873896275, 0, 0, 0, 288.93689271194006, 0, 0], '0.000751_137.500000_1.652893_625.000000_0.000000_0.100000': [0, 0, 0, 0, -289.8441515697755, 0, -325.0718268585821, -478.51367875209763], '0.000751_151.250000_1.652893_625.000000_0.000000_0.100000': [0, 450.7884132487949, 0, 0, 0, 0, 0, 0], '0.000751_151.250000_1.652893_568.181818_0.000000_0.100000': [349.2374589759367, 0, 0, 0, 0, 0, 0, 0], '0.000751_137.500000_1.652893_516.528926_0.000000_0.100000': [1091.8612435951318, 0, -2.1765701816095318, 0, 0, 0, 0, 0], '0.000751_125.000000_1.652893_516.528926_0.000000_0.100000': [291.2424543272964, 533.2185158327696, 0, 0, 263.6611503760548, -131.53963370009168, 0, 8.63991867406283], '0.000751_113.636364_1.652893_516.528926_0.000000_0.100000': [258.53539741349385, 0, 0, 0, 1.5040524892126996, 0, -291.2424543272964, 0], '0.000751_103.305785_1.652893_516.528926_0.000000_0.100000': [0, 0, 0, 0, 1.7191780185962489, 0, 0, 0], '0.000683_103.305785_1.502630_516.528926_0.000000_0.100000': [0, 0, 0, -31.672480020026253, 0, 0, 0, 0], '0.000683_113.636364_1.652893_516.528926_0.000000_0.100000': [284.45365743530556, 0, -1.5040524892126996, -679.6344699131441, 1.3556815504181032, 0, -2.9085356440454007, 0], '0.000683_125.000000_1.818182_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 293.17582232194525, 0, 0], '0.000683_103.305785_1.818182_516.528926_0.000000_0.100000': [276.01540815159524, 451.704599249639, 0, -509.8564819859954, 0, 0, -715.3969466117662, 0], '0.000683_103.305785_1.818182_469.571751_0.000000_0.100000': [0, 844.189405471996, 0, 0, 0, 0, 0, 0], '0.001000_151.250000_2.200000_516.528926_0.000000_0.100000': [35.72759932832144, 503.6270118730623, 0, -306.24268025144806, 0, 0, 0, -517.5599658301493], '0.000909_151.250000_1.818182_756.250000_0.000000_0.100000': [0, 0, -4.470864395766512, 0, 4.058371126814018, 0, -838.1539457505595, 0], '0.001000_151.250000_1.652893_756.250000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -498.4893309008523], '0.001000_151.250000_1.652893_831.875000_0.000000_0.100000': [320.1787587535579, 0, 0, 0, 0, 0, 0, 0], '0.001000_137.500000_1.652893_831.875000_0.000000_0.100000': [0, 0, 0, 0, 3.7823285305577885, 0, 0, 0], '0.000909_137.500000_1.818182_831.875000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -327.7625775550923, 0], '0.000909_151.250000_1.818182_831.875000_0.000000_0.100000': [0, 0, 0, 0, 0, 310.9909354865832, 0, 0], '0.000826_151.250000_1.818182_831.875000_0.000000_0.100000': [0, 500.80711825718265, 0, 0, 0, 0, 0, 0], '0.000826_151.250000_1.818182_756.250000_0.000000_0.100000': [0, 0, 0, 0, 0, 305.9111450114786, -368.41387627219046, 0], '0.001210_113.636364_1.652893_625.000000_0.000000_0.100000': [1211.488688571605, 521.3924487173332, 0, -532.6498754176218, 0, 75.26953425216948, 0, 0], '0.001210_113.636364_1.502630_516.528926_0.000000_0.100000': [0, 0, 0, 90.74187386913172, 0, 0, 0, 0], '0.001331_113.636364_1.652893_568.181818_0.000000_0.100000': [259.44287226188175, 465.274122439794, 0, 0, 4.893152533648303, -94.52977130435588, -27.748654385325093, -778.8197397515921], '0.001331_103.305785_1.652893_516.528926_0.000000_0.100000': [0, 455.860081844838, 0, 0, 0, 0, 0, 0], '0.001331_103.305785_1.652893_469.571751_0.000000_0.100000': [456.09235308590905, 0, 0, -167.1202725586166, 0, 0, 158.60532153740633, 0], '0.001331_103.305785_2.000000_469.571751_0.000000_0.100000': [1167.1654497370364, 0, 0, -1272.146435388383, 0, 0, 0, 0], '0.001331_103.305785_2.200000_469.571751_0.000000_0.100000': [0, 0, -8.562823059167204, 0, 0, 0, 0, -531.9311218584535], '0.001464_103.305785_2.200000_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -531.2088043398728], '0.001464_103.305785_2.200000_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 1275.8106520619692, 0, 0], '0.001464_103.305785_2.000000_516.528926_0.000000_0.100000': [0, 528.3158673559586, 0, 0, 7.997093801671326, 0, 0, 0], '0.001331_103.305785_2.000000_516.528926_0.000000_0.100000': [0, 0, -7.997093801671326, 0, 7.262459543201294, 0, 0, 0], '0.001464_103.305785_2.000000_469.571751_0.000000_0.100000': [0, 0, 0, -1272.9177150780552, 0, 0, 0, 0], '0.000909_113.636364_2.000000_568.181818_0.000000_0.100000': [414.0492779755359, 1131.4294763461903, 5846.441540394166, -1321.2561158976798, 1990.4827163892514, 1777.6934369624064, 478.8860076428003, -2177.06744388836], '0.000826_113.636364_2.000000_568.181818_0.000000_0.100000': [1174.5520441481967, 1364.8054303475403, 0, -1483.552881472871, -496.4252704082826, 2918.867328385386, 0, -623.7860198297573], '0.000909_113.636364_2.200000_568.181818_0.000000_0.100000': [377.65358383480964, 0, 462.099483930548, 0, 589.1347239519317, 6237.425248838484, 1396.211057519921, -984.9910230042351], '0.001100_113.636364_2.200000_568.181818_0.000000_0.100000': [524.3217965703144, 249.81886382302946, -6.471581359038701, -619.8244453800074, -350.40602209802273, 0, 0, 3628.7084665956363], '0.001331_103.305785_2.200000_568.181818_0.000000_0.100000': [1670.8780841631435, 550.7842196765041, -0.729185388089772, 0, 0, 0, -371.84716045547356, 0], '0.001331_113.636364_1.818182_568.181818_0.000000_0.100000': [307.9491754459282, 0, 0, 0, 0, 199.50609562538503, -317.7956146801016, 0], '0.001331_103.305785_1.818182_568.181818_0.000000_0.100000': [278.24134351721005, 0, -2.76178738133076, -149.59503409660908, 0, 0, 0, 0], '0.001464_103.305785_1.818182_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001000_151.250000_2.000000_687.500000_0.000000_0.100000': [0, 937.7003242088472, 0, 0, 1021.4749196292267, 1018.1847355245626, -463.0518715088362, -1364.1822710367583], '0.001000_183.012500_2.000000_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001000_103.305785_1.502630_568.181818_0.000000_0.100000': [538.0906861833321, 0, -3.7833101922204153, -19.621276342558012, 0, 1488.3656532092261, 0, -442.5572924240165], '0.001000_103.305785_1.366027_568.181818_0.000000_0.100000': [368.5374195001029, 0, 530.6395894588852, -1004.7877905256518, 0, 394.1920182066208, -456.7199210880828, 203.94817741836016], '0.001210_137.500000_2.200000_687.500000_0.000000_0.100000': [0, 0, -343.19111525592473, 0, 3.6612949177824703, 0, 0, 0], '0.000751_103.305785_1.652893_625.000000_0.000000_0.100000': [0, 0, -1.6115492050685134, 0, 0, 0, 331.2102300141229, -462.05995967878897], '0.000751_113.636364_1.818182_625.000000_0.000000_0.100000': [278.8069397972135, 771.1404028532943, -2.3796066288741713, 0, 420.7842244368211, 0, 0, 0], '0.000751_103.305785_1.818182_625.000000_0.000000_0.100000': [-201.4491187190035, 0, 0, -693.0980850193506, 0, 0, 0, -499.61233745141834], '0.000751_103.305785_2.000000_625.000000_0.000000_0.100000': [343.83351929034365, 0, 0, 0, 0, 693.0980850193506, 0, -1003.6985110638468], '0.001000_93.914350_2.200000_625.000000_0.000000_0.100000': [348.71174182285205, 588.609448612941, -6.039642398154119, 0, 0, 0, 0, -522.9667389321978], '0.001000_93.914350_2.200000_687.500000_0.000000_0.100000': [851.1941395123528, 611.1329826221572, 0, 0, 0, 0, -308.89605407268226, 0], '0.001000_103.305785_2.200000_687.500000_0.000000_0.100000': [308.89605407268226, 0, 0, -631.1472624243742, 0, 2718.8777944467065, -357.1053905796382, -640.594219307221], '0.001210_113.636364_2.000000_756.250000_0.000000_0.100000': [323.9759935646334, 0, 0, -1162.319582312278, 0, 0, -342.51015512611247, 0], '0.001210_113.636364_2.200000_831.875000_0.000000_0.100000': [0, 0, 0, 0, 4.323430183098572, 0, 0, 0], '0.001100_113.636364_2.200000_831.875000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001000_125.000000_2.662000_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 47.76334023908058, 0, 0], '0.001000_125.000000_2.200000_516.528926_0.000000_0.100000': [415.8003495041636, 35.27254363520899, 1052.1435323379205, -528.8565948982153, 568.5304972939094, 523.49063749278, 632.9842475926143, -853.9073393548849], '0.000909_125.000000_2.200000_516.528926_0.000000_0.100000': [458.0751992637701, 2092.038808000506, -0.04692181337388508, 0, 4.689147318974992, 0, -1159.9923983949966, 41.55880038683837], '0.000826_125.000000_2.200000_516.528926_0.000000_0.100000': [417.00859156464867, 517.070139107638, 0, -533.8199148855025, 0, 0, -383.97744213231766, 0], '0.000826_125.000000_2.200000_469.571751_0.000000_0.100000': [448.94878123589285, 0, 0, 0, 0, 0, -374.28085939022293, 0], '0.000826_113.636364_2.200000_469.571751_0.000000_0.100000': [469.97103294187184, 962.4555571517683, 0, 0, 0, 0, -408.1352556689935, 0], '0.000826_137.500000_2.200000_469.571751_0.000000_0.100000': [0, 517.2759089201558, 0, 0, 0, 0, -380.3235925348934, 0], '0.000826_151.250000_2.200000_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 1023.1215751064409, 0, 0], '0.000826_151.250000_2.000000_469.571751_0.000000_0.100000': [0, 0, 0, 0, 5.048528995333181, 0, 0, 0], '0.000751_151.250000_2.000000_469.571751_0.000000_0.100000': [496.11960903094786, 0, 0, 0, 0, 923.7914952795293, 0, -459.5562681453348], '0.000751_137.500000_2.000000_469.571751_0.000000_0.100000': [0, 0, 0, -1139.439989019325, 0, 0, -496.11960903094786, 0], '0.000751_137.500000_2.200000_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 1139.439989019325, 0, 0], '0.000751_151.250000_1.818182_469.571751_0.000000_0.100000': [0, 0, 336.6456396658355, 0, 0, 0, -442.3322550317751, 0], '0.000751_166.375000_1.818182_469.571751_0.000000_0.100000': [0, 0, 0, 0, 2.8232813766297116, 0, 0, 0], '0.000683_166.375000_1.818182_469.571751_0.000000_0.100000': [0, 0, 0, -901.0259741924585, 0, 0, 0, 0], '0.000683_166.375000_2.000000_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -543.4002563864943, 0], '0.000683_183.012500_2.000000_469.571751_0.000000_0.100000': [49.40002330786313, 0, 0, 0, 3.1769989329237434, 0, 0, 0], '0.000621_183.012500_2.000000_469.571751_0.000000_0.100000': [0, 0, 0, -763.7453596598513, 0, 0, 0, 0], '0.000621_183.012500_2.200000_469.571751_0.000000_0.100000': [0, 0, -2.6783489844357975, 0, 0, 0, 0, 0], '0.000683_183.012500_2.200000_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -262.11890822952864, 0], '0.000683_201.313750_2.200000_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 601.1539111303308, 0, 0], '0.000683_201.313750_2.000000_469.571751_0.000000_0.100000': [0, 0, -3.4626452148537283, 0, 0, 0, 0, 0], '0.000751_201.313750_2.000000_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -439.1710188958188], '0.000751_201.313750_2.000000_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001210_125.000000_2.000000_756.250000_0.000000_0.100000': [0, 0, 0, 0, 6.738070930440336, 0, 0, 0], '0.001100_125.000000_2.200000_756.250000_0.000000_0.100000': [0, 0, 0, 0, 0, 2199.7210613249313, 0, 0], '0.001100_137.500000_1.818182_756.250000_0.000000_0.100000': [0, 0, 39.5243829822889, -1013.0223018272991, -462.32927609433784, 297.58119581077045, 0, -519.0957296384744], '0.001100_137.500000_1.652893_756.250000_0.000000_0.100000': [1228.8682067402763, 0, 0, 0, 0, 153.26828609772, -309.04173901052746, 0], '0.001100_125.000000_1.652893_756.250000_0.000000_0.100000': [650.5111614093776, 0, -3.804163572029029, 0, 0, 23.19346397551729, 0, -543.5016876841488], '0.001100_125.000000_1.502630_756.250000_0.000000_0.100000': [281.1989028548287, 677.8514689906502, 0, -572.2472508796487, 0, 0, 0, 0], '0.001100_125.000000_1.652893_831.875000_0.000000_0.100000': [0, 0, 0, -303.2583800463073, 0, 0, -307.46469087083534, 0], '0.001100_125.000000_1.818182_831.875000_0.000000_0.100000': [0, 0, 0, -1012.5577274661754, 0, 0, 0, 0], '0.001100_125.000000_2.000000_831.875000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001000_137.500000_2.420000_568.181818_0.000000_0.100000': [0, 464.3450404765419, 0, 0, 0, 0, 0, -477.56004742702646], '0.001000_137.500000_2.420000_516.528926_0.000000_0.100000': [274.8677472992222, 437.12288063164686, 0, 0, 1.9747122259117533, 419.9302721557586, -540.7752042526729, 0], '0.001000_137.500000_2.420000_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 513.3777165304621, 0, -437.12288063164686], '0.000909_137.500000_2.420000_516.528926_0.000000_0.100000': [0, 0, -1.9747122259117533, 0, 0, 427.4650956770627, 0, 0], '0.001000_125.000000_2.420000_516.528926_0.000000_0.100000': [0, 0, 0, 0, 250.76554144523362, 0, -274.8677472992222, 0], '0.001000_151.250000_2.420000_516.528926_0.000000_0.100000': [270.3563575743877, 425.2572266374835, 0, 0, 0, 0, -300.1411103038333, 0], '0.001100_166.375000_2.420000_469.571751_0.000000_0.100000': [0, 450.15288373487493, 0, -232.63144028821134, 0, 0, 0, 0], '0.001100_166.375000_2.662000_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 232.63144028821134, 0, 0], '0.001100_166.375000_2.420000_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, 383.65064723075426, 0, 0], '0.001100_166.375000_2.200000_426.883410_0.000000_0.100000': [384.2169378789282, 0, 0, -34.877331566432176, 0, 0, 0, 0], '0.001100_151.250000_2.200000_426.883410_0.000000_0.100000': [0, 0, 0, 0, 4.921307971920214, 0, 0, 0], '0.001000_151.250000_2.200000_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -423.0438850271918, 0], '0.001000_166.375000_2.200000_426.883410_0.000000_0.100000': [38.458535002471955, 0, 0, -351.6825739506192, 0, 0, 0, 0], '0.000826_113.636364_2.662000_625.000000_0.000000_0.100000': [212.24086606324, 0, 0, 0, 0, 0, 0, 0], '0.000826_103.305785_2.662000_625.000000_0.000000_0.100000': [0, 801.0119144209618, 0, 0, 0, 0, 0, 0], '0.000826_85.376682_2.662000_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 561.0882823574362, 0, 0], '0.000826_85.376682_2.420000_568.181818_0.000000_0.100000': [0, 0, 0, 0, 2.0261045547269534, 2536.5769716551135, 0, -55.727134442684616], '0.000751_85.376682_2.420000_568.181818_0.000000_0.100000': [265.45594291347845, 0, 0, 0, 0, 2537.1392337975612, 0, 0], '0.000751_77.615165_2.420000_568.181818_0.000000_0.100000': [233.0847969893559, 0, 0, 0, 0, 0, -24.132358446678367, 0], '0.000751_70.559241_2.420000_568.181818_0.000000_0.100000': [0, 0, 0, 0, 2.0825712123538325, 0, 0, 0], '0.000683_70.559241_2.420000_568.181818_0.000000_0.100000': [7530.695177770762, 0, 0, 0, 0, 0, 0, 0], '0.000683_64.144765_2.420000_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0.18474570063194734], '0.000683_64.144765_2.420000_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, -0.8955387030122961, 0, 0], '0.000683_64.144765_2.200000_625.000000_0.000000_0.100000': [0, 0, 0, 0.08141260936475413, 0, 0, 0, 0], '0.000751_125.000000_2.000000_625.000000_0.000000_0.100000': [0, 1412.2236084409533, -79.16450574314403, -1215.6947963369569, 0, 0, -411.15836887063546, 0], '0.000751_125.000000_2.000000_568.181818_0.000000_0.100000': [411.5010335943438, 4.721742405708824, -4.690603603917771, -1241.3205827872187, 4.263354960405195, 0, -443.7343316431952, -593.8758582902824], '0.000683_113.636364_2.000000_568.181818_0.000000_0.100000': [386.383589746056, 536.7577168567901, 618.8712705659672, 0, 0, 1037.1767678134524, 528.8210500549351, 457.3887735418018], '0.000751_93.914350_1.818182_568.181818_0.000000_0.100000': [4529.979526635508, 0, -2.3908830798955023, 0, 0, 0, 0, 0], '0.000826_93.914350_1.818182_568.181818_0.000000_0.100000': [4121.22776419213, 0, 0, 0, 0, 0, -281.7426518232664, 0], '0.000751_93.914350_1.652893_625.000000_0.000000_0.100000': [0, 0, 0, 0, 279.13237348745287, 0, 0, 0], '0.000683_93.914350_1.652893_625.000000_0.000000_0.100000': [740.0798998256063, 0, 0, -227.26072318334664, 0, 0, 0, 0], '0.000751_85.376682_1.818182_687.500000_0.000000_0.100000': [-1974.0247570527763, 0, 0, 0, 0, 0, 0, 0], '0.000751_77.615165_1.818182_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001000_113.636364_1.818182_687.500000_0.000000_0.100000': [1357.8536089720692, 3354.6986366284136, -1013.5363278802431, 1412.1214840615455, 1009.8312609082427, 1093.3990077847234, 1615.3158635256284, 424.3987330928916], '0.001100_125.000000_1.652893_687.500000_0.000000_0.100000': [1878.228654511421, 0, 0, 0, 0, 54.2282404484351, -128.98853272761153, -507.9974651569948], '0.001100_137.500000_1.652893_831.875000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -499.3590675535024], '0.001100_137.500000_1.652893_915.062500_0.000000_0.100000': [0, 0, -4.068470283300849, 0, 0, 0, 0, 0], '0.001210_137.500000_1.652893_915.062500_0.000000_0.100000': [0, 0, -4.4785494403578925, 0, 0, 0, 0, 0], '0.001331_137.500000_1.652893_915.062500_0.000000_0.100000': [0, 0, 0, -339.1693383539705, 0, 0, 0, 0], '0.001331_137.500000_1.818182_915.062500_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000826_103.305785_2.200000_687.500000_0.000000_0.100000': [0, 0, 42.19641088634114, -634.6074551936265, 0, 0, 0, 0], '0.000826_103.305785_2.420000_687.500000_0.000000_0.100000': [0, 536.0765722363697, 0, 0, 2.6198581144528363, 634.6074551936265, 0, 0], '0.000826_103.305785_2.200000_625.000000_0.000000_0.100000': [331.02670209791034, 916.7932643115822, -4.508181436203995, 0, 0, 1256.0568727980426, 0, 0], '0.000826_85.376682_2.200000_625.000000_0.000000_0.100000': [2251.6498114820038, 0, 0, -827.0209628648694, 4.484307308188545, 0, -31.783944418444214, 0], '0.000826_77.615165_2.200000_625.000000_0.000000_0.100000': [5545.571175644572, -378.9532059833898, 0, 0, 0, 0, 0, -194.67406985296185], '0.000826_77.615165_2.200000_568.181818_0.000000_0.100000': [0, -843.90150112413, 0, 0, 0, -143.68309107339246, 0, 0], '0.000826_77.615165_2.200000_516.528926_0.000000_0.100000': [327.37149078607627, 0, 0, 0, 0, 0, 0, 0], '0.000909_103.305785_2.420000_568.181818_0.000000_0.100000': [299.1316442178896, 0, 0, 0, 0, 0, 134.89094625330318, -570.0520273840638], '0.000909_93.914350_2.420000_568.181818_0.000000_0.100000': [256.35684774815235, 0, 0, 0, 0, 0, -271.93785837989964, -591.7140862658574], '0.000909_85.376682_2.420000_568.181818_0.000000_0.100000': [0, 0, 0, -562.2764477454496, 0, 0, 0, 0], '0.000909_85.376682_2.662000_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -621.9095330343124], '0.000909_85.376682_2.662000_625.000000_0.000000_0.100000': [0, 0, -4.097258663780486, 0, 0, 0, 0, 0], '0.001000_85.376682_2.662000_625.000000_0.000000_0.100000': [0, 622.2256884997732, 0, 0, 0, 0, 0, 0], '0.001000_85.376682_2.662000_568.181818_0.000000_0.100000': [221.9264685745089, 0, 0, 0, 0, 0, 0, 0], '0.001000_77.615165_2.662000_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 582.6934435129832, 0, 0], '0.001000_77.615165_2.420000_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -241.04351284021814, 0], '0.001000_85.376682_2.420000_568.181818_0.000000_0.100000': [0, 516.7598472919783, 0, 0, 0, 0, 0, 0], '0.001000_85.376682_2.420000_516.528926_0.000000_0.100000': [0, 0, 0, 0, 2.2661019026385474, 0, 0, 0], '0.000909_85.376682_2.420000_516.528926_0.000000_0.100000': [0, 0, 0, -482.4708294687134, 2.0453198988851002, 0, 0, 0], '0.000826_85.376682_2.420000_516.528926_0.000000_0.100000': [0, 0, -2.0453198988851002, 0, 0, 0, 0, 0], '0.000909_85.376682_2.662000_516.528926_0.000000_0.100000': [0, 0, -3.3374076433259687, 0, 0, 0, 0, 0], '0.001000_85.376682_2.662000_516.528926_0.000000_0.100000': [0, 579.6323734479431, 0, 0, 0, 0, 0, 0], '0.001000_85.376682_2.662000_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000909_113.636364_2.420000_756.250000_0.000000_0.100000': [0, 0, -200.21198788172146, 0, 2.442599642446294, 0, 0, 0], '0.000826_113.636364_2.420000_756.250000_0.000000_0.100000': [0, 524.5009546602354, 0, 0, 0, 0, 0, 0], '0.000826_113.636364_2.420000_687.500000_0.000000_0.100000': [0, 1038.9787587550366, 0, 0, 0, 0, 0, 0], '0.000909_113.636364_2.420000_568.181818_0.000000_0.100000': [-150.37060194283953, 1128.9117539531085, -256.633932288274, 0, 0, 0, -282.37975725015184, 0], '0.000826_125.000000_2.662000_568.181818_0.000000_0.100000': [263.21431587697884, 0, 0, 0, 0, 0, 0, 0], '0.000826_113.636364_2.662000_568.181818_0.000000_0.100000': [251.9520703565431, 0, -3.677475246503491, 0, 0, 0, 0, 0], '0.000909_103.305785_2.420000_687.500000_0.000000_0.100000': [0, 536.5490739212939, 0, -584.3111219696993, 0, 0, 0, 0], '0.000751_93.914350_2.000000_625.000000_0.000000_0.100000': [0, 0, -4.569995583615127, 0, 4.152926825773284, 0, 0, 0], '0.000683_93.914350_2.000000_568.181818_0.000000_0.100000': [0, 0, 0, 0, 457.22286987879147, 0, 0, 0], '0.000621_93.914350_2.000000_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 299.6779474818143, 0, 0], '0.000621_93.914350_1.818182_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -310.65119823519865, 0], '0.000621_113.636364_1.818182_568.181818_0.000000_0.100000': [0, 446.0213679676368, -572.8007490929567, 0, 1.4371431993295118, 0, 0, 0], '0.000621_113.636364_1.818182_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -446.0213679676368], '0.000683_103.305785_2.000000_568.181818_0.000000_0.100000': [0, 553.5941212558566, 0, 0, 0, 0, 0, 0], '0.000683_103.305785_2.000000_516.528926_0.000000_0.100000': [0, 690.0291091353109, 0, 0, 0, 509.8564819860003, 0, 0], '0.000621_125.000000_1.818182_568.181818_0.000000_0.100000': [-202.04531636654275, 0, 0, 0, 0, 0, 0, 0], '0.000826_113.636364_1.818182_915.062500_0.000000_0.100000': [0, 0, -4.259551727797189, -1014.025096695831, 0, 0, 0, 0], '0.000909_113.636364_1.818182_915.062500_0.000000_0.100000': [0, 0, -4.266711783997562, 0, 3.8723197525428987, 0, 0, 0], '0.000826_113.636364_2.000000_915.062500_0.000000_0.100000': [277.60524702703077, 0, 0, 0, 0, 0, 0, 0], '0.000826_103.305785_2.000000_915.062500_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001100_103.305785_2.420000_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 704.8263712179383, 0, -401.53276252602336], '0.001100_103.305785_2.200000_568.181818_0.000000_0.100000': [0, 1086.4397277264568, 0, 131.82540451212796, 1346.3917645865497, 2595.8395988085135, 783.4874662421452, 0], '0.001100_103.305785_2.200000_516.528926_0.000000_0.100000': [383.07637700930627, 713.9206516989265, 0, 0, 6.534669073825075, 0, 207.06688500325654, 0], '0.001100_93.914350_2.200000_516.528926_0.000000_0.100000': [0, 0, 0, 0, 6.08999870120424, 0, 0, -629.9005492448185], '0.001000_93.914350_2.200000_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 973.0006136976209, -572.7865935704016], '0.001000_93.914350_2.200000_568.181818_0.000000_0.100000': [5277.405478791825, 572.7865935704033, 0, 0, 0, 0, 0, -588.609448612941], '0.001210_77.615165_2.000000_687.500000_0.000000_0.100000': [506.92248195487224, 0, 0, 0, 6.6206607305559375, 0, 0, 0], '0.001100_77.615165_2.000000_687.500000_0.000000_0.100000': [0, 1708.034289415458, 0, 892.2584164877879, 0, 0, 0, -664.495573515148], '0.001100_77.615165_2.000000_625.000000_0.000000_0.100000': [890.0254478754449, 0, 0, 0, 0, 0, -1204.9251755279597, -310.55168898462875], '0.001100_77.615165_2.200000_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -2436.7331349244982, 0], '0.001000_85.376682_2.200000_687.500000_0.000000_0.100000': [2479.6795343523527, 683.8630130825153, 0, 0, 0, 0, 0, -631.4539474716054], '0.001000_77.615165_2.200000_687.500000_0.000000_0.100000': [0, 0, 0, -3054.1006365466624, 178.2984662790697, 0, 0, 0], '0.001000_77.615165_2.420000_687.500000_0.000000_0.100000': [0, 661.2039527026877, 0, 0, 0, 0, -208.86173860765012, 0], '0.001100_137.500000_1.652893_516.528926_0.000000_0.100000': [962.6329807992809, 0, 0, 0, 0, 0, -392.1261505079744, 0], '0.001100_151.250000_1.652893_516.528926_0.000000_0.100000': [35.64783186436129, 0, 0, -332.6421022841676, 0, 0, -396.1405522161629, 0], '0.001000_151.250000_1.818182_516.528926_0.000000_0.100000': [1775.1250865467298, 75.32538203780075, 31.888360568600206, 0, 3.587735135537514, 332.3043965795943, 0, 0], '0.000909_151.250000_1.818182_516.528926_0.000000_0.100000': [547.3674557301349, 0, -3.5877351355391514, -901.5080227064857, 0, 0, 0, -468.1796409104007], '0.001000_151.250000_1.652893_687.500000_0.000000_0.100000': [0, 0, -4.018358584160888, 0, 0, 71.77672598658866, -345.55618627443476, 0], '0.001000_151.250000_1.502630_687.500000_0.000000_0.100000': [288.92724169361765, 0, 0, -65.25156907871697, 0, 0, 0, 0], '0.001331_125.000000_1.818182_469.571751_0.000000_0.100000': [0, 422.2463873068497, 0, 0, 0, 0, 0, 0], '0.001331_125.000000_1.652893_426.883410_0.000000_0.100000': [338.9694050336327, 0, -3.426721417737099, 0, 0, 0, 0, 0], '0.001331_113.636364_1.652893_426.883410_0.000000_0.100000': [0, 0, 0, -217.26501716476787, 0, 0, 0, -426.89963147362016], '0.001331_113.636364_1.652893_469.571751_0.000000_0.100000': [461.2981613780773, 426.89963147362016, 0, -236.78338435959193, 382.64108490486655, 0, 0, -448.6525963419864], '0.001331_113.636364_1.818182_469.571751_0.000000_0.100000': [1028.4056808823077, 0, 0, 0, 0, 0, 0, -451.6382548816606], '0.001331_113.636364_1.818182_516.528926_0.000000_0.100000': [0, 0, 0, 0, 315.92751690917163, 221.11028275059107, 0, 0], '0.001464_113.636364_1.652893_516.528926_0.000000_0.100000': [257.94626755279904, 0, -3.5603042082881986, -221.95335251089136, 0, -105.27349297924928, 0, 0], '0.001611_113.636364_1.652893_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, -98.3312310789348, 0, 0], '0.001611_113.636364_1.502630_516.528926_0.000000_0.100000': [0, 512.2294905574652, 0, 0, 0, 0, 0, 0], '0.001611_113.636364_1.502630_469.571751_0.000000_0.100000': [374.9431412260048, 0, 0, 0, 0, 0, 0, 0], '0.001611_103.305785_1.502630_469.571751_0.000000_0.100000': [0, 0, 0, -73.77558571156497, 0, 0, 0, 0], '0.001611_103.305785_1.652893_469.571751_0.000000_0.100000': [0, 439.42238435949366, 0, 0, 0, 0, 0, 0], '0.001611_103.305785_1.652893_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, 143.67428526959694, 0, 0], '0.001611_103.305785_1.502630_426.883410_0.000000_0.100000': [364.45828818857456, 0, 0, 0, 0, 0, 0, 0], '0.001611_93.914350_1.502630_426.883410_0.000000_0.100000': [0, 0, 0, 0, 7.638260840106159, 0, 0, 0], '0.001464_93.914350_1.502630_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -365.6893748736988, 0], '0.001464_103.305785_1.502630_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, 814.802370049544, -380.5813803413128, 0], '0.001464_103.305785_1.366027_426.883410_0.000000_0.100000': [359.04907652181663, 0, 0, 0, 0, 0, 0, 0], '0.001464_93.914350_1.366027_426.883410_0.000000_0.100000': [0, 0, 0, -808.1620716976619, 0, 0, 0, 0], '0.001210_113.636364_1.818182_756.250000_0.000000_0.100000': [599.5417661051304, 0, 0, 0, 0, 0, -297.54402213708653, -547.3011852901009], '0.001210_113.636364_1.818182_831.875000_0.000000_0.100000': [0, 0, -5.404209259461459, 0, 0, 0, 0, 0], '0.001331_113.636364_1.818182_831.875000_0.000000_0.100000': [263.14470216473273, 0, 0, 0, 0, 0, 0, 0], '0.001331_103.305785_1.818182_831.875000_0.000000_0.100000': [0, 0, 0, -945.4037556855479, 0, 0, 0, 0], '0.001331_103.305785_2.000000_831.875000_0.000000_0.100000': [0, 0, -8.104849034389918, 0, 0, 0, 0, 0], '0.001464_93.914350_2.000000_831.875000_0.000000_0.100000': [0, 0, 0, 0, 8.120566254162986, 0, 0, 0], '0.001331_93.914350_2.000000_831.875000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -647.7956506303589], '0.001331_93.914350_2.000000_915.062500_0.000000_0.100000': [245.85299483648743, 0, 0, 0, 7.282959609777208, 0, 0, 0], '0.001210_93.914350_2.000000_915.062500_0.000000_0.100000': [0, 0, -7.282959609777208, 0, 0, 0, 0, 0], '0.001331_85.376682_2.000000_915.062500_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001000_113.636364_2.200000_756.250000_0.000000_0.100000': [284.581201988853, 1990.185451726216, 0, 0, 0, 0, 0, -582.8401024179143], '0.000909_113.636364_2.200000_831.875000_0.000000_0.100000': [247.8449480498841, 0, 0, 0, 0, 0, 0, 0], '0.000909_93.914350_2.200000_831.875000_0.000000_0.100000': [265.00271355527093, 0, 0, 0, 0, 0, 0, 0], '0.000909_85.376682_2.200000_831.875000_0.000000_0.100000': [0, 0, 0, -711.5877690188747, 0, 0, -24.09115577775189, 0], '0.000909_85.376682_2.420000_831.875000_0.000000_0.100000': [166.7656687021183, 0, 0, 0, 0, 0, 0, 0], '0.000909_77.615165_2.420000_831.875000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 11917.647390507409], '0.000909_77.615165_2.420000_915.062500_0.000000_0.100000': [0, 0, 0, 0, 0, -9082.010701232877, 0, 0], '0.000909_77.615165_2.200000_915.062500_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001000_125.000000_1.652893_687.500000_0.000000_0.100000': [199.13139227597298, 1726.9465919767501, 358.1723598719873, 620.5396748400017, -39.22537567940651, 961.051554259779, -316.26138869688606, -470.9232611022353], '0.001000_113.636364_1.652893_756.250000_0.000000_0.100000': [0, 0, 0, -259.23107134069033, 0, 0, -294.1756526632491, 0], '0.001100_113.636364_2.000000_831.875000_0.000000_0.100000': [0, 0, 0, 0, 6.06924987074508, 1251.7959120258274, 0, 0], '0.000909_93.914350_2.000000_756.250000_0.000000_0.100000': [291.8529466724264, 0, 0, -4220.353970244398, 0, 0, 0, -3631.318732892468], '0.000909_93.914350_2.000000_831.875000_0.000000_0.100000': [0, 3631.318732892468, 0, 0, 0, 0, 0, 0], '0.000909_85.376682_2.000000_756.250000_0.000000_0.100000': [0, 0, 0, 0, 3.504837182840159, 0, -0.004352344515893947, -325.649135131207], '0.000826_85.376682_2.000000_756.250000_0.000000_0.100000': [-2695.631230386944, -2313.7039028464533, -0.0003154384995439408, 0, 0, 0, 0, 0], '0.001100_151.250000_2.200000_625.000000_0.000000_0.100000': [1696.9104917584523, 0, 0, 0, 5.900765338517649, 1100.62428587898, 0, 0], '0.000909_166.375000_2.200000_568.181818_0.000000_0.100000': [0, 496.26494546559405, 0, -179.47159735005243, 0, 0, 0, 0], '0.000909_166.375000_2.200000_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 986.7402242813239, 0, 0], '0.000909_166.375000_2.000000_469.571751_0.000000_0.100000': [-415.17419871266054, 0, 0, 0, 0, 0, -491.77847688977744, 0], '0.000909_183.012500_2.000000_469.571751_0.000000_0.100000': [0, 0, -4.667107241011218, 0, 0, 0, 0, 0], '0.001000_183.012500_2.000000_469.571751_0.000000_0.100000': [490.90097900679706, 0, 0, 0, 0, 0, 0, 0], '0.001000_166.375000_2.000000_469.571751_0.000000_0.100000': [504.87982521580125, 0, 0, 0, 0, 0, 0, 0], '0.001000_151.250000_2.000000_469.571751_0.000000_0.100000': [500.87709937821325, 0, 0, 0, 0, 930.6889108716512, 0, 0], '0.000909_151.250000_2.000000_469.571751_0.000000_0.100000': [0, 0, 0, -1022.2955438315015, 9.18025091948937, 843.800689811532, 0, -459.7863554454003], '0.000909_151.250000_1.818182_469.571751_0.000000_0.100000': [0, 0, -3.604075629301624, 0, 2.9649231877410784, 0, 0, -785.1350923862192], '0.001000_151.250000_1.818182_568.181818_0.000000_0.100000': [823.447181889771, 0, 1201.5580813024458, -1174.0214210847337, 0, 410.9440953127048, -420.1124027144384, 113.58870332099997], '0.001000_137.500000_1.502630_625.000000_0.000000_0.100000': [0, 416.79373143540835, 0, 255.44475402888202, 1.5075254489995131, 1549.4108701839305, 0, -387.8866098642837], '0.001000_137.500000_1.366027_625.000000_0.000000_0.100000': [470.6932413511728, 540.3084819707933, -5.877076590156503, 0, 0, 0, 0, 0], '0.001000_137.500000_1.366027_568.181818_0.000000_0.100000': [1161.198169121644, 0, -5.44948419461216, 0, 0, 0, -434.503582844435, 0], '0.001210_103.305785_1.818182_625.000000_0.000000_0.100000': [274.0002221608717, 767.0693817336858, 0, 0, 450.68961249764055, 0, 0, -502.34680714577047], '0.001210_103.305785_1.818182_687.500000_0.000000_0.100000': [0, 0, 0, 675.5125269044413, 0, 0, -278.8710696202383, 0], '0.001210_113.636364_1.818182_687.500000_0.000000_0.100000': [0, 188.8628177102479, -4.5250619808732155, -869.6652858039548, 0, 0, 460.5074157478365, -525.556694510608], '0.001210_113.636364_2.420000_625.000000_0.000000_0.100000': [0, 918.557172783232, 0, 0, 0, 1214.5460881351526, 0, 0], '0.001210_113.636364_2.200000_625.000000_0.000000_0.100000': [718.1302872295842, 0, 0, 0, 0, 0, 0, -652.373178846761], '0.001000_151.250000_2.200000_469.571751_0.000000_0.100000': [882.8484180305782, 0, 0, 0, 0, 0, 0, 0], '0.001000_137.500000_2.200000_469.571751_0.000000_0.100000': [0, 568.4343660530119, 0, -508.945460019683, 5.41720240206323, 0, -37.978704990572325, 0], '0.001000_137.500000_2.200000_426.883410_0.000000_0.100000': [397.7344163104264, 0, 0, 0, 5.190558096953828, 0, 0, -516.7585145936472], '0.001000_125.000000_2.200000_426.883410_0.000000_0.100000': [388.5223255216659, 528.4359073617485, 691.8416044293949, -707.9706995977533, 133.71281667966906, 0, 0, 0], '0.001000_113.636364_2.200000_426.883410_0.000000_0.100000': [0, 594.7747079247371, 420.23803015099213, 0, 0, 0, -388.5223255216659, 0], '0.001100_93.914350_2.200000_426.883410_0.000000_0.100000': [401.81033860201177, 0, -7.106506641419037, -713.2134095746294, 6.4672808621000515, 1206.9773820528746, -43.10932453453983, -593.4115676564837], '0.001000_85.376682_2.200000_426.883410_0.000000_0.100000': [1342.4972584859772, 0, -6.436736231708529, 0, 0, 0, 0, 0], '0.001100_85.376682_2.200000_426.883410_0.000000_0.100000': [0, 0, -7.0810743152097535, 0, 0, 0, -401.81033860201177, 0], '0.001100_93.914350_2.000000_426.883410_0.000000_0.100000': [4132.064287037183, 0, 0, -1327.675120258162, 0, 0, -390.95587472742574, 0], '0.001100_93.914350_2.420000_426.883410_0.000000_0.100000': [0, 500.741779786103, 0, 0, 2.361733761771575, 1050.0403561262162, 0, 0], '0.001100_93.914350_2.420000_388.075827_0.000000_0.100000': [0, 0, 0, -171.1758376822698, 0, 1070.5104063254255, 0, -540.015644867366], '0.001000_93.914350_2.420000_426.883410_0.000000_0.100000': [0, 491.58520050270585, 0, 0, 0, 1159.5604935491992, 0, 0], '0.001210_93.914350_2.200000_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000909_125.000000_2.200000_469.571751_0.000000_0.100000': [407.86081683158517, 598.9748640130513, -4.978592956657667, -601.6364408224562, 0, 2714.179777925525, -374.23322583606114, -516.8097724089861], '0.000909_137.500000_2.200000_469.571751_0.000000_0.100000': [374.23322583606114, 0, -4.924729456421119, 0, 4.475594343558259, 0, 0, 0], '0.000909_137.500000_2.200000_426.883410_0.000000_0.100000': [0, 509.0515311010751, -5.190558096953828, 0, 0, 1075.1440111346271, -404.1507301508997, 0], '0.000909_137.500000_2.000000_426.883410_0.000000_0.100000': [545.8350179961739, 0, -5.588902476398653, -1075.1440111346271, 0, 0, 0, 0], '0.000909_125.000000_2.420000_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001000_166.375000_2.200000_687.500000_0.000000_0.100000': [0, 464.5716805157234, 0, 0, 0, 92.16402030631268, 0, 0], '0.000909_166.375000_2.420000_568.181818_0.000000_0.100000': [254.156693619213, 424.5767651863893, 0, 0, 0, 0, 0, 0], '0.000909_166.375000_2.420000_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -278.5314915713094, 0], '0.000909_183.012500_2.420000_516.528926_0.000000_0.100000': [0, 450.2844638138851, 0, -327.23548474184975, 1.3189353009103797, 0, 0, 0], '0.000909_183.012500_2.420000_469.571751_0.000000_0.100000': [0, 0, 0, 0, 1.2114198451163247, 0, 0, 0], '0.000826_183.012500_2.420000_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -450.17694835809107], '0.000826_183.012500_2.420000_516.528926_0.000000_0.100000': [0, 0, -1.3189353009103797, -325.2745246665909, 0, 0, 0, -411.64185453527847], '0.000826_183.012500_2.662000_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 325.2745246665909, 0, 0], '0.000826_201.313750_2.420000_568.181818_0.000000_0.100000': [0, -11691.803356989656, 0, 0, 0, 0, 0, 0], '0.000826_201.313750_2.420000_516.528926_0.000000_0.100000': [0, 0, -1.6677442727288507, 0, 0, 0, 0, 0], '0.000909_201.313750_2.420000_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 11693.471101262385], '0.000909_201.313750_2.420000_568.181818_0.000000_0.100000': [-11876.838623389309, 0, 0, 0, 0, 0, 0, 0], '0.000909_183.012500_2.420000_568.181818_0.000000_0.100000': [0, 412.27768416057137, 0, 0, 0, 0, 0, 0], '0.000909_183.012500_2.662000_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000826_125.000000_2.420000_568.181818_0.000000_0.100000': [282.89767993063253, 0, 0, 0, 0, 0, 0, 0], '0.000826_113.636364_2.420000_568.181818_0.000000_0.100000': [315.95498956972426, 0, 0, 0, 0, 0, 0, 0], '0.000826_93.914350_2.420000_568.181818_0.000000_0.100000': [0, 0, -2.319918762272573, 0, 2.097893754851975, 0, 0, 0], '0.000826_125.000000_2.662000_625.000000_0.000000_0.100000': [0, 749.756100603124, 0, 0, 0, 0, 0, 0], '0.001100_125.000000_2.662000_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -216.3598004866537, 0], '0.001100_137.500000_2.662000_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 12011.693122544606, 0], '0.001100_151.250000_2.662000_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, -11817.239742751353, 0, 0], '0.000909_151.250000_2.420000_568.181818_0.000000_0.100000': [0, 0, 0, -403.2673444576489, 0, 282.7817529982125, 0, 0], '0.000826_151.250000_2.200000_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -329.4270649894208, 0], '0.000751_166.375000_2.000000_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -507.900006653757, 0], '0.000751_183.012500_2.000000_568.181818_0.000000_0.100000': [46.17272787761425, 0, 0, 0, 0, 1004.5203368710539, -420.94097722893133, 0], '0.000751_201.313750_2.000000_568.181818_0.000000_0.100000': [420.94097722893133, 0, 0, 0, 0, 0, 0, -437.6646983945244], '0.000751_201.313750_2.000000_625.000000_0.000000_0.100000': [0, 437.6646983945244, 0, 0, 0, 0, 0, 0], '0.000751_183.012500_1.818182_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 410.8737829926382, 0, 0], '0.000751_183.012500_1.652893_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001000_125.000000_1.818182_756.250000_0.000000_0.100000': [620.0900423448394, -8970.03172303355, -4.418258017196786, -1529.137174166362, -223.24145734986467, 288.1485257511851, -86.82570128288273, -534.1594406381249], '0.000909_113.636364_1.652893_426.883410_0.000000_0.100000': [290.120701002607, 0, -76.03394272772971, -215.3468016043774, 339.0510633451646, 377.3653892569494, 774.197722060259, 3.2260187784733034], '0.000909_103.305785_1.652893_426.883410_0.000000_0.100000': [0, 0, 0, -146.48708870819664, 0, 1384.236923587741, -78.90096608824592, 141.60913278019353], '0.000683_137.500000_1.652893_687.500000_0.000000_0.100000': [286.90220125909894, 0, -0.2387794067306639, 0, 0, 0, 0, 0], '0.000683_125.000000_1.652893_687.500000_0.000000_0.100000': [0, 499.97336498435544, 0, 0, 2.249887369896715, 0, 0, 0], '0.000683_125.000000_1.502630_625.000000_0.000000_0.100000': [320.62665421883975, 0, -1.2101119780286171, 0, 0, 0, -305.12661912309215, 0], '0.000683_113.636364_1.502630_625.000000_0.000000_0.100000': [0, 0, 0, 0, 1.5502859936879758, 0, 0, 0], '0.000564_113.636364_1.502630_625.000000_0.000000_0.100000': [0, 0, -1.4016195053294724, 0, 0, 0, 0, 0], '0.000621_125.000000_1.502630_625.000000_0.000000_0.100000': [0, 0, -2.07792270846252, 0, 0, 0, 0, 0], '0.000683_137.500000_1.502630_625.000000_0.000000_0.100000': [0, 0, -2.182592434691924, 57.322904826182274, 0, 0, 0, 0], '0.000683_137.500000_1.652893_625.000000_0.000000_0.100000': [1576.5090524027805, 0, 0, 0, 0, 0, -357.7901108904878, 0], '0.000683_151.250000_1.652893_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -355.6825786336482, 0], '0.000826_113.636364_2.000000_516.528926_0.000000_0.100000': [539.0701749868862, 0, 692.5041445588436, 0, 4.593530764830939, 1212.6005031749266, 0, -86.62176263718197], '0.000826_113.636364_1.818182_516.528926_0.000000_0.100000': [429.40518217483964, 611.0685538141543, 0, 0, 2.2119145290325832, 217.77741687798198, 439.6880385098641, -1022.0163567977004], '0.000826_125.000000_1.818182_516.528926_0.000000_0.100000': [1654.2010697875967, 1533.2938581439269, 693.5442367784591, -930.5292581082053, 0, 176.27638300116178, -355.3422007400081, 1252.3058535114412], '0.001000_103.305785_1.652893_426.883410_0.000000_0.100000': [0, 4699.768963030641, -1.4795159243676608, 116.47568779556235, 1.3157032943098785, 230.6312174303245, 0, 0], '0.001000_103.305785_1.502630_426.883410_0.000000_0.100000': [369.3051311647247, 867.4903542712441, 1604.6749350178666, -160.9193468022232, 0, 0, -379.14377382245397, -506.5549554271799], '0.001000_103.305785_1.818182_426.883410_0.000000_0.100000': [437.42084080755205, 0, 2035.1316317387063, 0, 696.0323840181021, 157.56859425639294, 0, 148.16397086407395], '0.001000_93.914350_1.502630_426.883410_0.000000_0.100000': [0, 0, -5.145596254762404, 0, 0, 2187.6617922270175, -369.3051311647247, -525.95007442157], '0.001100_93.914350_1.502630_426.883410_0.000000_0.100000': [341.89438598720443, 503.9420745572401, 0, 0, 0, 2325.528625051728, 0, 0], '0.001100_93.914350_1.502630_388.075827_0.000000_0.100000': [0, 0, 0, -73.98557494123543, 973.8945577600724, 0, 0, 168.23675627143217], '0.001000_93.914350_1.502630_388.075827_0.000000_0.100000': [1652.5548104513477, 0, -0.4587648999201151, -412.37415240268183, 0, 0, -372.3779253808034, 0], '0.001000_93.914350_1.652893_388.075827_0.000000_0.100000': [0, 1599.5034483559675, 0, 0, 0, 0, 1365.738492254488, 0], '0.001000_113.636364_1.502630_625.000000_0.000000_0.100000': [0, 1994.4863303771162, 0, -209.72680067824433, 0, 0, 0, 0], '0.001000_113.636364_1.502630_568.181818_0.000000_0.100000': [0, 3395.357081853538, 0, 162.68523324861337, 1327.6550952588448, 0, 352.8638280460449, 0], '0.001000_125.000000_1.502630_568.181818_0.000000_0.100000': [0, 540.3067618977847, 0, 562.513450658518, 0, 2121.858198784255, 0, 0], '0.001000_125.000000_1.502630_516.528926_0.000000_0.100000': [4175.310374414869, 465.8463258758584, -2.6512871844077384, 623.5242483909071, 2.409567071608035, 1708.8686020284908, -326.80839569910825, 0], '0.000909_125.000000_1.502630_516.528926_0.000000_0.100000': [379.21936395685196, 0, -2.409567071608035, 0, 0, 2208.3802869425217, -32.707607271495064, 289.25408732891276], '0.001000_125.000000_2.000000_426.883410_0.000000_0.100000': [463.9786818752835, 0, 1247.071750721392, -1290.633568416435, 0, 1250.9251523210726, -545.4181917043276, 0], '0.001000_113.636364_2.000000_426.883410_0.000000_0.100000': [428.4445571949061, 458.4187722913157, 794.2256033692466, 0, 0, 1189.3219068314627, 0, 0], '0.001100_125.000000_2.000000_426.883410_0.000000_0.100000': [554.4549239605476, 491.56422216157847, 3.4684006131830216, 0, 6.067117971693188, 2057.1470500222395, 0, -472.4055230595066], '0.001210_125.000000_2.000000_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -872.2735244179539], '0.001210_125.000000_1.652893_469.571751_0.000000_0.100000': [372.3431518133844, 1233.2837338789523, -2.947004191454835, 0, 0, 0, -333.67952232594, 0], '0.001210_137.500000_1.652893_469.571751_0.000000_0.100000': [0, 1100.5109355938148, 0, -304.28924128735724, 0, 0, 0, 0], '0.001000_113.636364_2.662000_568.181818_0.000000_0.100000': [0, 584.2661107214971, 0, 0, 4.053204063904741, 0, 0, 376.48968010574356], '0.000826_103.305785_2.662000_516.528926_0.000000_0.100000': [0, 0, 0, 0, 2.9507485539135816, 0, 0, 0], '0.000751_103.305785_2.662000_516.528926_0.000000_0.100000': [229.60091240141247, 0, 0, 0, 0, 0, 0, 0], '0.000751_93.914350_2.662000_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 447.4328542249248, 0, 0], '0.000751_93.914350_2.420000_516.528926_0.000000_0.100000': [304.4329821515011, 0, 0, 0, 0, 0, 0, 0], '0.000751_85.376682_2.420000_516.528926_0.000000_0.100000': [264.7895125006868, 0, 0, 0, 0, 0, 0, -516.169885314292], '0.000751_85.376682_2.200000_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -1987.8868018587052, 0], '0.000751_93.914350_2.200000_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -362.2744184636591, 0], '0.000751_103.305785_2.200000_568.181818_0.000000_0.100000': [0, 606.8115493972662, 0, 0, 0, 0, 0, 0], '0.000751_103.305785_2.200000_516.528926_0.000000_0.100000': [0, 535.1291591718591, -0.4420895472925622, -783.8186284024476, 0, 0, -873.7062319716287, -55.16468630884238], '0.000751_103.305785_2.420000_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 783.8186284024476, 0, 0], '0.000683_113.636364_2.000000_516.528926_0.000000_0.100000': [0, 0, -4.173926727372055, 0, 0, 0, 0, 0], '0.000826_137.500000_2.000000_568.181818_0.000000_0.100000': [0, 1277.1825512166733, 190.4001802445457, -1201.5727070294897, 0, 0, 0, 0], '0.000909_137.500000_2.200000_516.528926_0.000000_0.100000': [758.6774737484986, 0, 0, -420.6008221826981, 0, 0, -443.71900167601353, 0], '0.000909_151.250000_2.200000_469.571751_0.000000_0.100000': [0, 492.94521628662005, 0, 0, 0, 1022.2955438315015, 0, 0], '0.000826_151.250000_2.000000_516.528926_0.000000_0.100000': [0, 0, -5.169445457560687, 0, 0, 0, 0, 0], '0.000909_137.500000_1.818182_516.528926_0.000000_0.100000': [1039.6189933122007, 0, 1265.498481373658, 0, 255.48884036780436, 337.64172781268195, -438.6691597793805, -1091.2798639930713], '0.000909_166.375000_2.000000_516.528926_0.000000_0.100000': [0, 0, 0, 0, 5.095479643579165, 0, 0, 0], '0.000826_113.636364_1.502630_625.000000_0.000000_0.100000': [0, 0, 0, 0, 1.8991465665569194, 979.4432069700431, 16.29565561524089, -454.4636816222852], '0.000826_113.636364_1.502630_687.500000_0.000000_0.100000': [0, 0, -1.4302551240472894, 934.0423850578679, 0, 0, 313.2491546971828, 0], '0.000909_113.636364_1.366027_756.250000_0.000000_0.100000': [0, 0, -4.927825774242956, 0, 0, 618.0756217105945, 0, 0], '0.001000_113.636364_1.366027_756.250000_0.000000_0.100000': [375.8412084191001, 0, 0, 0, 0, 0, 0, -517.3612811453003], '0.001000_113.636364_1.366027_831.875000_0.000000_0.100000': [0, 0, 0, 0, 5.1084090532571285, 679.5739076126587, 0, 0], '0.001000_113.636364_1.241843_831.875000_0.000000_0.100000': [0, 0, 0, -679.5739076126587, 0, 0, 0, 0], '0.000909_113.636364_1.366027_831.875000_0.000000_0.100000': [0, 0, 0, 0, 0, 744.8968380270261, 0, 0], '0.000909_113.636364_1.241843_831.875000_0.000000_0.100000': [249.15303747278904, 503.88511350754254, 0, 0, 0, 0, 0, 0], '0.000909_103.305785_1.241843_831.875000_0.000000_0.100000': [204.74148397583485, 0, 0, 0, 0, 0, 0, 0], '0.000909_93.914350_1.241843_831.875000_0.000000_0.100000': [0, 0, 0, 0, 0, -409.01052588374716, 0, 0], '0.000909_93.914350_1.128948_831.875000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 963.840901481191], '0.000909_93.914350_1.128948_915.062500_0.000000_0.100000': [-1247.7637032140635, 0, 0, 0, 0, 0, 0, 0], '0.000909_85.376682_1.128948_915.062500_0.000000_0.100000': [0, 464.3633573475372, 0, 0, 0, 0, 0, 0], '0.000909_85.376682_1.128948_831.875000_0.000000_0.100000': [0, 0, -0.5406416939065821, 0, 0, 0, 695.7784457318726, 0], '0.001000_85.376682_1.128948_831.875000_0.000000_0.100000': [0, 0, 0, 0, 0.5406416939065821, 0, 0, 0], '0.000909_125.000000_1.502630_568.181818_0.000000_0.100000': [0, 794.0275260941937, 864.9296398945102, 0, 0.1638227957046001, 1747.8542595338013, -296.4096456893753, 351.4129795823202], '0.000909_113.636364_1.502630_516.528926_0.000000_0.100000': [0, 0, 0, 0, 647.0958437589672, 0, -338.75945305357493, 0], '0.000826_113.636364_1.502630_516.528926_0.000000_0.100000': [1336.8952735496516, 0, 0, 0, 3.219448861876117, 0, 0, 0], '0.000751_113.636364_1.502630_516.528926_0.000000_0.100000': [0, 517.6997458219299, 0, 0, 0, 0, 0, 0], '0.000751_113.636364_1.502630_469.571751_0.000000_0.100000': [0, 0, 0, 0, 3.478206397230315, 0, 0, 0], '0.000683_113.636364_1.502630_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -518.2592426357535], '0.000621_113.636364_1.652893_516.528926_0.000000_0.100000': [0, 0, -196.3898422488501, 0, 0, 0, 0, 0], '0.000683_103.305785_2.000000_469.571751_0.000000_0.100000': [0, 1415.1186092752737, 0, -1268.1887876821918, 3.603994064904782, 0, 0, 0], '0.000683_103.305785_2.000000_426.883410_0.000000_0.100000': [389.6859788179598, 1297.4596925194342, 0, 0, 0, 0, 0, -452.9562038625582], '0.000683_93.914350_2.000000_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -503.10750539754366], '0.000683_93.914350_2.000000_469.571751_0.000000_0.100000': [955.6445192708483, 0, 0, 0, 0, 0, 0, 0], '0.000683_85.376682_2.000000_469.571751_0.000000_0.100000': [0, 0, 0, 0, 2.6758988687301097, 0, 0, 0], '0.000621_85.376682_2.000000_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001100_93.914350_1.366027_756.250000_0.000000_0.100000': [317.84004458887847, 579.9631891801578, -5.896311203273763, 0, 0, 0, -821.5010303146119, -539.0461152697828], '0.001100_93.914350_1.366027_831.875000_0.000000_0.100000': [0, 539.0461152697828, 0, 0, 0, 0, 0, 0], '0.001100_85.376682_1.366027_756.250000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -317.8400445888785, 0], '0.001100_93.914350_1.366027_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 227.58222987980352, -392.75011116240887, -527.2392628910525], '0.001210_93.914350_1.366027_756.250000_0.000000_0.100000': [0, 527.2018242531678, 0, 0, 0, 0, 0, -538.9915498014528], '0.001210_93.914350_1.366027_831.875000_0.000000_0.100000': [0, 0, -6.438398867508113, 0, 0, 0, 0, 0], '0.001331_93.914350_1.366027_831.875000_0.000000_0.100000': [0, 538.9900930096527, 0, 0, 0, 0, 0, 0], '0.001331_93.914350_1.366027_756.250000_0.000000_0.100000': [0, 0, 0, 0, 0, 350.1011642772318, 0, 0], '0.001331_93.914350_1.241843_756.250000_0.000000_0.100000': [810.4646096673428, 0, 0, 0, 0, 0, 0, 0], '0.001331_85.376682_1.241843_756.250000_0.000000_0.100000': [0, 484.672611626447, 0, 0, 0, 0, 0, 0], '0.001331_85.376682_1.241843_687.500000_0.000000_0.100000': [0, 447.3644588792172, 0, 0, 0, 0, 0, -484.672611626447], '0.001331_85.376682_1.241843_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -447.3644588792172], '0.001000_93.914350_2.000000_625.000000_0.000000_0.100000': [320.1607606710324, 2275.4543876302087, 313.7926243456381, 0, 5.5345638762419185, 0, 0, -675.4696792508205], '0.001000_93.914350_2.000000_687.500000_0.000000_0.100000': [2631.9544644284956, 0, -6.143094906231454, 0, 279.0510452898764, 0, -417.8664894499552, 0], '0.001000_85.376682_2.000000_687.500000_0.000000_0.100000': [0, 0, -6.08870837320701, -1262.408756320705, 0, 0, 0, 2572.757478648233], '0.001000_85.376682_2.200000_756.250000_0.000000_0.100000': [0, 631.4539474716054, 0, 0, 0, 0, 0, 0], '0.001000_85.376682_2.200000_625.000000_0.000000_0.100000': [0, 0, -5.966769835519881, 0, 5.425312878183286, 1271.0356734922373, -317.01067438441095, 0], '0.001100_85.376682_2.200000_625.000000_0.000000_0.100000': [0, 0, -6.562543383125922, 0, 0, 0, 0, 0], '0.001210_93.914350_2.000000_625.000000_0.000000_0.100000': [1501.0779740339174, 0, -7.378477376498814, -982.2106831540467, 0, 0, 0, 0], '0.001210_103.305785_2.200000_687.500000_0.000000_0.100000': [0, 574.3525470157559, 0, -689.6593724224431, 0, 2642.8919750637847, 0, 0], '0.001210_103.305785_2.420000_687.500000_0.000000_0.100000': [865.2617875873035, 538.1942022743699, 0, -582.0876245965158, 0, 626.9630658385846, 0, 0], '0.001210_103.305785_2.420000_625.000000_0.000000_0.100000': [0, 0, 0, 0, 3.2665238238623715, 842.2352350784741, 0, -592.0136225018068], '0.001100_113.636364_2.000000_568.181818_0.000000_0.100000': [529.5561145557876, 95.90639857134398, 0, 0, -394.39400537763504, 1972.7819839156625, 2918.9153644454445, 915.9943832667647], '0.001100_137.500000_1.818182_469.571751_0.000000_0.100000': [0, 1315.4030258492908, -3.276647950177903, 0, 0, 303.8062468818551, -40.27155553236562, -40.851827274568684], '0.001100_137.500000_1.818182_426.883410_0.000000_0.100000': [0, 0, 289.9550880704958, -689.2862820298035, 0, 1425.999535735163, -412.5320013802631, 0], '0.001100_137.500000_1.652893_426.883410_0.000000_0.100000': [341.1472307475589, 1640.9266590808616, -262.35763287543944, -329.1670871487303, 2.5729635679768763, -124.45047586830788, -377.6853366756686, 0], '0.001100_125.000000_1.652893_426.883410_0.000000_0.100000': [865.3180341157922, 0, -2.757412068742724, 0, 2.477177257750373, -42.61483715231734, 45.06661974759347, 0], '0.001210_125.000000_1.652893_426.883410_0.000000_0.100000': [378.75943939341124, 1227.8988579490763, 0, -293.1111567896056, 2.757412068742724, -45.32283678599597, -219.24891485263726, 0], '0.001210_125.000000_1.818182_426.883410_0.000000_0.100000': [701.0387228302326, 0, 0, 0, 0, 266.4646879905514, 158.21407080785553, -56.100765622723934], '0.001210_137.500000_1.818182_426.883410_0.000000_0.100000': [0, 0, 0, 0, 296.9500948102254, 1033.5429618902479, 0, 0], '0.001100_137.500000_1.502630_426.883410_0.000000_0.100000': [0, 0, -4.811876791209233, 0, 83.30205778854773, 0, 0, -431.2669363163839], '0.001100_137.500000_1.502630_469.571751_0.000000_0.100000': [0, 0, -2.785598872091032, 0, 0, 0, 0, 0], '0.001210_137.500000_1.502630_469.571751_0.000000_0.100000': [0, 0, -3.056847046794246, 0, 2.785598872091032, 0, 0, 0], '0.001331_137.500000_1.502630_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -344.9497905163013, 0], '0.001331_151.250000_1.502630_469.571751_0.000000_0.100000': [0, 384.5583418915016, 0, 0, 0, 0, 0, 0], '0.001331_151.250000_1.502630_426.883410_0.000000_0.100000': [0, 384.11197497526376, 0, 0, 0, 0, 0, 0], '0.001331_151.250000_1.502630_388.075827_0.000000_0.100000': [0, 0, -4.759455945851914, 0, 0, 0, 0, 0], '0.001464_151.250000_1.502630_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001000_103.305785_2.000000_687.500000_0.000000_0.100000': [-830.8921021107076, -1197.249834042856, -569.3067723912297, -784.0807516480436, -201.5906984833132, 1487.3812052584915, -2977.3265877249078, -1045.6201188976281], '0.000909_85.376682_1.818182_687.500000_0.000000_0.100000': [1418.105823946441, 0, 0, 0, -820.4113466310846, 0, 0, 0], '0.000826_85.376682_1.652893_687.500000_0.000000_0.100000': [0, 0, -2.1672580419482985, 2591.9314273047457, 0, 0, 0, -511.2237392261346], '0.000909_85.376682_1.652893_687.500000_0.000000_0.100000': [1423.997543635774, 0, 0, 0, 0, 0, 0, 0], '0.000909_77.615165_1.652893_687.500000_0.000000_0.100000': [0, 0, 0, 1130.7515001691575, 0, 0, 0, -1581.4521878197975], '0.000909_77.615165_1.818182_687.500000_0.000000_0.100000': [460.8639146709564, 0, 0, -2357.3754900879676, 0, 0, 0, 0], '0.000909_70.559241_1.818182_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 3614.9964756485924, 0, 0], '0.000909_70.559241_1.652893_687.500000_0.000000_0.100000': [0, 0, 0, 0, -9.387439772873893, 0, 0, 0], '0.000826_70.559241_1.652893_687.500000_0.000000_0.100000': [0, 0, 0, 0, 7.231806946110715, 0, -4876.114486833877, 0], '0.000751_70.559241_1.652893_687.500000_0.000000_0.100000': [5.079932995924856, 0, 0, 0, 0, 0, 0, 0], '0.001000_166.375000_1.818182_469.571751_0.000000_0.100000': [496.4063091808111, 0, 0, 0, 3.791501585352671, 0, -480.97595021074284, 0], '0.001000_151.250000_1.818182_426.883410_0.000000_0.100000': [0, 735.3601410053083, 0, 0, 2.933602865305147, 0, -451.6754875710791, 0], '0.001000_166.375000_1.818182_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -369.7618301840968], '0.000909_166.375000_1.818182_469.571751_0.000000_0.100000': [0, 0, 0, 0, 3.435683180816159, 0, 0, 0], '0.000826_166.375000_1.818182_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 388.085611874638, 0, 0], '0.000826_166.375000_1.652893_469.571751_0.000000_0.100000': [401.52398191324136, 0, 0, 0, 0, 0, -442.7696777408988, 0], '0.000826_183.012500_1.652893_469.571751_0.000000_0.100000': [0, 0, -3.4112807831290413, 0, 0, 0, 0, 0], '0.000909_183.012500_1.652893_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -462.1214513250547, 0], '0.000909_201.313750_1.652893_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -517.3798000910382, 0], '0.000909_221.445125_1.652893_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000826_125.000000_2.000000_568.181818_0.000000_0.100000': [2004.6824543484931, 0, 681.6658297808901, -1241.8137557720374, -386.323722626, 839.3704842413194, 0, -1750.4786273479185], '0.000826_113.636364_1.652893_568.181818_0.000000_0.100000': [2105.21860393571, 986.7675572830183, -1.7488585715709632, 0, 1.7489831738396697, 941.1078926066297, 1107.2461478926343, 0], '0.000751_113.636364_1.652893_568.181818_0.000000_0.100000': [286.23485675180024, 466.0289084970382, 1893.835201910943, 0, 2.580891896679496, -83.50270105903074, 0, 603.6421162513802], '0.000751_113.636364_1.502630_568.181818_0.000000_0.100000': [365.24759239278285, 0, -2.006140263410998, 0, 0, 0, 0, 0], '0.000826_113.636364_1.502630_568.181818_0.000000_0.100000': [0, 0, 0, 0, 2.006140263410998, 1062.1409023674412, 0, 0], '0.000826_113.636364_1.366027_568.181818_0.000000_0.100000': [0, 448.46949561430705, 0, 0, 0, 403.90140519003154, -456.6781172217271, -473.98318545042], '0.000826_113.636364_1.366027_625.000000_0.000000_0.100000': [452.4831569889619, 0, -4.7122489647570545, -89.04029154273114, 0, 0, -435.1276906918379, 0], '0.000909_113.636364_1.366027_625.000000_0.000000_0.100000': [1193.0768042363811, 0, 0, 0, 0, 0, -439.17368593703503, 0], '0.000826_125.000000_1.502630_625.000000_0.000000_0.100000': [351.2706413950978, 452.11832763195366, -1.3639672424420497, 0, 0, 193.62770872759677, 0, -392.2562690028694], '0.001000_77.615165_1.818182_568.181818_0.000000_0.100000': [4974.344101278355, 0, -4.537893997854826, 0, -33.72641957894348, 460.7248224294561, 2954.4123608208974, 0], '0.000909_77.615165_1.818182_568.181818_0.000000_0.100000': [5008.425480230499, 0, 0, 0, 3.8613893622536124, 0, 0, 4291.0272621948525], '0.000826_77.615165_1.818182_568.181818_0.000000_0.100000': [0, -172.87552105941015, 0, 0, 4.99209916670975, 0, 0, 0], '0.000826_77.615165_1.818182_516.528926_0.000000_0.100000': [0, 0, 0, 362.8887745838922, 0, 0, 0, 0], '0.000826_77.615165_2.000000_516.528926_0.000000_0.100000': [-337.97659358666726, 490.95642513830245, 0, -1509.2646034815182, 0, 0, -932.3894420693222, 0], '0.000826_85.376682_2.000000_516.528926_0.000000_0.100000': [1025.6283862762534, 0, 0, 0, 0, 1625.9716631522858, 0, 0], '0.000751_85.376682_1.652893_516.528926_0.000000_0.100000': [-862.7953759204449, -598.1429224709368, -1.5310232716948031, -34.37278681970529, -536.7129629903825, -759.1802035308768, -1267.5752715689844, 0], '0.000751_85.376682_1.652893_469.571751_0.000000_0.100000': [0, 400.9288402250972, 593.0626859469609, 0, 0, 0, 0, 669.3287395815546], '0.000751_85.376682_1.652893_426.883410_0.000000_0.100000': [0, 0, 0, 17.773549540162822, 0, 0, 0, 0], '0.000751_85.376682_1.818182_426.883410_0.000000_0.100000': [0, 0, 0, 0, 2.315245090440385, 0, 0, -461.7342805665427], '0.000683_85.376682_1.818182_426.883410_0.000000_0.100000': [0, 0, 0, 0, 2.0956296987485987, 0, 0, 0], '0.000621_85.376682_1.818182_426.883410_0.000000_0.100000': [264.55047581601394, 0, 0, 0, 0, 0, 0, 0], '0.000621_77.615165_1.818182_426.883410_0.000000_0.100000': [1917.280438705552, 0, 0, 0, 1.5268077012830872, -69.93030068918634, -2.3833376199642147, -459.3203040695854], '0.000621_77.615165_1.818182_469.571751_0.000000_0.100000': [0, 459.3203040695854, 0, 0, 0, 0, 0, 0], '0.000621_70.559241_1.818182_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001100_125.000000_2.420000_516.528926_0.000000_0.100000': [0, 600.3773223051578, 0, 0, 0, 134.34281409023527, 0, 0], '0.001100_125.000000_2.420000_469.571751_0.000000_0.100000': [285.1130724946092, 429.1491008559089, 0, 0, 1.328216676048396, 1529.1798764151383, -285.15382805784975, 0], '0.001100_137.500000_2.420000_469.571751_0.000000_0.100000': [285.15382805784975, 0, 0, 0, 0, 0, 0, 0], '0.001100_137.500000_2.420000_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, 607.240478713883, 0, 0], '0.001100_137.500000_2.200000_426.883410_0.000000_0.100000': [542.4263325257572, 0, 0, 0, 0, 0, 0, -516.4772793133201], '0.000826_137.500000_2.200000_426.883410_0.000000_0.100000': [0, 508.8526976879031, 0, -618.2120777348069, 0, 0, 0, 0], '0.000826_137.500000_2.200000_388.075827_0.000000_0.100000': [0, 0, -4.52832661636221, -751.3236177671616, 0, 1000.1543090104591, -408.6006446097019, -508.8526976879031], '0.000826_137.500000_2.000000_388.075827_0.000000_0.100000': [0, 0, 0, -1000.1543090104591, 0, 0, 0, 0], '0.000826_137.500000_2.420000_388.075827_0.000000_0.100000': [297.4127120237823, 0, 0, 0, 1.4054132419716552, 0, -248.13113223450748, 0], '0.000826_125.000000_2.420000_388.075827_0.000000_0.100000': [0, 0, -2.397022639751776, 0, 0, 0, 0, 0], '0.000909_125.000000_2.420000_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001000_85.376682_2.000000_568.181818_0.000000_0.100000': [453.8346389702974, -189.1975685540216, -4.0617571331706674, 0, 5.829815218229747, 1116.192855941235, -1.1727385964496306, -1432.433541081234], '0.001000_85.376682_1.818182_516.528926_0.000000_0.100000': [-165.34050146454555, -311.8214070899578, -454.42440611158173, -1188.0809073232638, 4426.671988885864, -449.98047561223933, 202.76128824130183, -150.2594174941852], '0.000909_85.376682_1.818182_568.181818_0.000000_0.100000': [-3299.8138604053, -2328.2766123878237, -2342.7367510402614, -3499.9893944067103, -21.844094188290626, -3826.1154795891157, -4173.551320488446, -440.50219222782516], '0.000909_85.376682_1.818182_625.000000_0.000000_0.100000': [0, 1390.1308176798025, 0, -1272.351036577956, 5.805582224339678, -503.1263853835084, -1086.665284930785, 0], '0.000909_77.615165_1.818182_625.000000_0.000000_0.100000': [0, 0, 0, 0, 4.155379772252309, 5422.544380135314, 0, 0], '0.000826_77.615165_1.818182_625.000000_0.000000_0.100000': [0, 0, 0, 0, 4.484195325999827, 0, 0, 0], '0.000751_77.615165_1.818182_625.000000_0.000000_0.100000': [4952.352499418525, 0, 0, 500.2420494644739, 0, 0, -199.25015478834703, 0], '0.000751_85.376682_1.652893_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -484.43758451672346], '0.000751_85.376682_1.652893_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 1290.461111998104, 0, 0], '0.000751_85.376682_1.502630_687.500000_0.000000_0.100000': [0, 0, -2.448962309271883, -117.31464654528209, 2.222772107312085, 0, 0, 0], '0.000826_85.376682_1.502630_687.500000_0.000000_0.100000': [0, -521.0018432184943, 0, 0, 0, 0, 0, 0], '0.000826_85.376682_1.502630_625.000000_0.000000_0.100000': [0, 540.8915315410754, 0, 0, 0, 0, 0, 0], '0.000826_85.376682_1.502630_568.181818_0.000000_0.100000': [320.2684511022189, 0, 0, 0, 0, 0, -345.88942239364314, 0], '0.000826_93.914350_1.502630_568.181818_0.000000_0.100000': [0, 527.3418255674196, 0, 0, 0, 0, 0, -482.7575308999185], '0.000826_93.914350_1.502630_516.528926_0.000000_0.100000': [0, 0, 435.29923337743, 0, 0, 986.3670879535764, -421.7105138288951, -0.9493102170431529], '0.001100_103.305785_1.818182_516.528926_0.000000_0.100000': [2626.862191023436, 2420.7686126926055, 446.76606895551544, -578.5581639576139, 424.5106585899093, 162.49472932875432, -344.89499799025754, 15.644159297001352], '0.001000_137.500000_1.502630_516.528926_0.000000_0.100000': [0, 0, -2.2848242304487942, 0, 2.356079061342098, 0, -315.8909101833906, 0], '0.001000_113.636364_1.502630_516.528926_0.000000_0.100000': [653.7579199879665, 2748.6539023089817, 0, 0, 0, 2628.847125946615, -139.96264001513714, 0], '0.001000_103.305785_1.502630_516.528926_0.000000_0.100000': [383.15915815735474, 783.0798508597532, 0, 0, 387.61224231768745, 0, 0, 653.0741553858184], '0.001000_103.305785_1.502630_469.571751_0.000000_0.100000': [0, 4176.751052076283, 0, -97.3257468001544, 228.31892637328224, 0, 0, 175.48350857715218], '0.000826_103.305785_1.652893_469.571751_0.000000_0.100000': [311.43042714244274, 2432.1284315886446, 23.133119666388296, 0, 1.6978668557646415, 0, 329.55612623040116, 260.7427768776943], '0.000909_103.305785_1.502630_426.883410_0.000000_0.100000': [1051.6565524878852, 634.1637550796903, -4.21740441169577, 0, 0, 170.95716872080595, 0, 0], '0.000909_103.305785_1.366027_426.883410_0.000000_0.100000': [1124.4427939646189, 0, 0, -444.89082018937967, 0, 0, 0, 0], '0.000909_103.305785_1.502630_388.075827_0.000000_0.100000': [963.548011210151, 0, 143.04939532186586, -269.62450996557664, 0, 1093.2662717864223, 1.0443932845095447, 563.3161209342143], '0.000909_103.305785_1.366027_388.075827_0.000000_0.100000': [0, 629.2330203605621, 0, 0, 0, 206.97357651507198, 0, 0], '0.000909_103.305785_1.241843_388.075827_0.000000_0.100000': [231.78428222575928, 0, -1.50625089614432, 0, 0, 0, -292.0831060112575, 0], '0.000909_113.636364_1.241843_388.075827_0.000000_0.100000': [0, 0, 0, -101.10566304465884, 0, 0, 0, 0], '0.000909_113.636364_1.366027_388.075827_0.000000_0.100000': [0, 1081.5366302644197, -3.445170875675649, -278.43997059438163, 0, 0, 0, -407.45214529828246], '0.001000_113.636364_1.366027_388.075827_0.000000_0.100000': [0, 0, -3.807686149495203, -248.24540450962664, 1722.6409660460322, 102.41842653401746, 0, 0], '0.001100_113.636364_1.366027_388.075827_0.000000_0.100000': [760.6488519972547, 0, -4.210639309937597, -738.3051763348884, 0, 0, -418.078013767106, -408.8602151572354], '0.001210_113.636364_1.366027_388.075827_0.000000_0.100000': [0, 0, 0, 0, 345.0130391188785, 0, 0, 0], '0.001100_125.000000_1.366027_388.075827_0.000000_0.100000': [418.078013767106, 0, 0, 0, 4.048281773785584, 0, 0, -41.504775932829766], '0.001100_113.636364_1.366027_426.883410_0.000000_0.100000': [397.329646048719, 0, 0, 0, 0, 0, 0, 0], '0.000909_151.250000_1.652893_568.181818_0.000000_0.100000': [0, 0, 0, 0, 742.0738097641788, 0, 0, 0], '0.000826_151.250000_1.652893_568.181818_0.000000_0.100000': [361.04045616045755, 0, 0, -314.0667869361156, 0, -63.15059211023199, -407.24299888434393, 0], '0.000826_151.250000_1.818182_568.181818_0.000000_0.100000': [0, 0, -406.3409551154622, 0, 0, 0, -461.3692626766069, 0], '0.000826_151.250000_1.818182_625.000000_0.000000_0.100000': [0, 69.89823500597993, 0, 0, 0, 0, 0, 0], '0.000909_166.375000_1.818182_568.181818_0.000000_0.100000': [419.75412638461825, 1168.1320417308693, 0, -994.1200832445904, 0, 0, -89.88204697190287, 0], '0.000909_166.375000_1.818182_516.528926_0.000000_0.100000': [903.5255052154379, 0, -4.0250796396301665, 0, 0, 0, 0, -41.26425257849706], '0.001000_166.375000_1.818182_516.528926_0.000000_0.100000': [0, 391.8090091374359, 0, -953.1343180101429, 0, 0, -466.31590828152514, 0], '0.001000_183.012500_1.818182_516.528926_0.000000_0.100000': [466.31590828152514, 0, 0, 0, 0, 0, 0, 0], '0.001000_183.012500_1.818182_469.571751_0.000000_0.100000': [0, 357.22858589451937, 0, 0, 0, 0, 0, 0], '0.001000_183.012500_1.818182_426.883410_0.000000_0.100000': [0, 0, 0, 0, 4.03554287744646, 0, 0, 0], '0.000909_183.012500_1.818182_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001210_137.500000_2.000000_687.500000_0.000000_0.100000': [1593.3361130422225, 0, 0, -1107.617366593229, 0, 979.4344838620427, -444.648468852365, -537.4878997667344], '0.001331_137.500000_1.818182_687.500000_0.000000_0.100000': [963.482861804285, 0, -5.9727067329458805, -634.0440276210869, 0, 0, 0, -502.66358902302744], '0.001464_137.500000_1.818182_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -502.9809431238411], '0.001464_137.500000_1.818182_756.250000_0.000000_0.100000': [0, 0, 0, 0, 0, 273.3410416881716, 0, 0], '0.001464_137.500000_1.652893_756.250000_0.000000_0.100000': [316.05829851275575, 0, 0, 0, 0, 0, -310.6239243844075, 0], '0.001464_151.250000_1.652893_756.250000_0.000000_0.100000': [310.6239243844075, 0, 0, 0, 0, 0, 0, 0], '0.001464_125.000000_1.652893_756.250000_0.000000_0.100000': [0, 472.89196179023514, 0, 0, 0, 124.03165992837039, 0, 0], '0.001464_125.000000_1.502630_756.250000_0.000000_0.100000': [0, 0, 0, -124.03165992837039, 0, 0, 0, 0], '0.001464_125.000000_1.652893_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 59.06963657150118, 0, 0], '0.001464_125.000000_1.502630_687.500000_0.000000_0.100000': [0, 0, 0, 0, 2.323149812204065, 0, 0, 0], '0.001331_125.000000_1.502630_687.500000_0.000000_0.100000': [0, 392.0967142233403, 0, 0, 0, 0, 0, 0], '0.000826_137.500000_1.818182_756.250000_0.000000_0.100000': [0, 0, 0, 0, 6.752759957055145, 0, 0, 0], '0.000751_137.500000_1.818182_756.250000_0.000000_0.100000': [353.1171662935379, 0, 0, 0, 3.4989303422473856, 0, 0, 0], '0.000751_125.000000_1.818182_756.250000_0.000000_0.100000': [296.12119138851125, 0, 0, 0, 2.9902289690553516, 0, 0, 0], '0.000683_125.000000_1.818182_756.250000_0.000000_0.100000': [0, 516.1722444076433, 0, 0, 0, 258.8140093700553, 0, 0], '0.000683_125.000000_1.652893_756.250000_0.000000_0.100000': [0, 0, 0, 0, 2.0917211391662933, 0, 0, 0], '0.000621_125.000000_1.652893_756.250000_0.000000_0.100000': [295.6573951185732, 0, 0, 0, 0, 0, 0, 0], '0.000621_113.636364_1.652893_756.250000_0.000000_0.100000': [0, 0, 0, 0, 2.119999355034088, 0, 0, 0], '0.000564_113.636364_1.652893_756.250000_0.000000_0.100000': [1465.7521778316163, 0, 0, 0, 0, 0, 0, 0], '0.000564_103.305785_1.652893_756.250000_0.000000_0.100000': [0, 0, 0, -1453.4825532678822, 2.28310512249991, 0, 0, 0], '0.000513_103.305785_1.652893_756.250000_0.000000_0.100000': [0, 0, 0, 0, 0, -1099.931900794694, 0, 0], '0.000513_103.305785_1.502630_756.250000_0.000000_0.100000': [0, 0, -0.9405597637472966, 0, 0, 0, 0, 0], '0.000564_103.305785_1.502630_756.250000_0.000000_0.100000': [0, 0, 0, 1098.5893554359416, 0, 0, 0, 0], '0.000564_103.305785_1.818182_756.250000_0.000000_0.100000': [0, 0, -2.084581738867746, 0, 0, 0, 0, 0], '0.000621_103.305785_1.818182_756.250000_0.000000_0.100000': [0, 528.9730236505424, 0, 0, 0, 0, 0, 0], '0.000621_103.305785_1.818182_687.500000_0.000000_0.100000': [0, 1118.8543033528513, 0, 0, 0, 0, 0, 0], '0.000621_103.305785_1.818182_625.000000_0.000000_0.100000': [250.65344590961558, 563.6919259587299, 0, 0, 0, 0, 0, 0], '0.000621_93.914350_1.818182_625.000000_0.000000_0.100000': [0, 0, 0, -595.366542840657, 0, 0, 0, 0], '0.000621_93.914350_2.000000_625.000000_0.000000_0.100000': [0, 0, 0, 0, 3.4298243607019687, 0, -343.73391762428747, 0], '0.000621_103.305785_2.000000_625.000000_0.000000_0.100000': [343.73391762428747, 0, 0, 0, 0, 0, 0, 0], '0.000564_93.914350_2.000000_625.000000_0.000000_0.100000': [319.7395283349721, 0, 0, 0, 0, 0, 0, 0], '0.000564_85.376682_2.000000_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000909_113.636364_2.200000_469.571751_0.000000_0.100000': [431.07619653857586, 521.5383692415443, -0.5284639789615171, 0, 0, 2235.7086369767003, 78.68449479229736, 0], '0.000909_113.636364_2.200000_426.883410_0.000000_0.100000': [0, 544.54574111525, 42.56731175998266, 0, 0, 0, -388.7932842688155, 0], '0.000909_125.000000_2.200000_426.883410_0.000000_0.100000': [0, 0, 0, 0, 543.0135575916737, 0, -397.8147844061952, 0], '0.000909_137.500000_2.200000_388.075827_0.000000_0.100000': [0, 0, 0, 0, 4.52832661636221, 0, 0, -509.0515311010751], '0.000909_151.250000_2.200000_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -384.9516269924361, 0], '0.000909_113.636364_2.420000_687.500000_0.000000_0.100000': [237.4205393887436, 0, 0, 0, 0, 0, 0, 0], '0.000909_103.305785_2.662000_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 584.3111219696993, 0, 0], '0.001000_103.305785_2.420000_625.000000_0.000000_0.100000': [0, 0, 0, 0, 783.0168905696942, 0, 313.332893816578, 0], '0.000909_103.305785_2.200000_516.528926_0.000000_0.100000': [383.5954357754819, 634.638946563244, 0, 0, 0, 2976.2954264033465, 0, 0], '0.001100_103.305785_2.000000_516.528926_0.000000_0.100000': [374.4500125502409, 527.7475017389452, 40.25713844069327, 0, 2741.872551655204, 0, -1553.4848656098616, 393.8044158999386], '0.001100_93.914350_2.000000_516.528926_0.000000_0.100000': [344.1348075098843, 0, 0, -1267.5962899008468, 563.7500192970497, 0, 0, 0], '0.001100_93.914350_2.200000_568.181818_0.000000_0.100000': [2402.968813579743, 0, 0, -794.5739838543923, 0, 0, 0, 0], '0.001100_93.914350_2.420000_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -539.1359459604239], '0.001100_93.914350_2.420000_625.000000_0.000000_0.100000': [0, 0, -3.8351864669053612, 0, 0, 0, 0, 0], '0.001210_93.914350_2.420000_625.000000_0.000000_0.100000': [0, 539.8247203070325, 0, 0, 0, 0, 0, -558.1420729442676], '0.001210_93.914350_2.420000_516.528926_0.000000_0.100000': [0, 511.68200269729647, 0, 0, 0, 0, 0, 0], '0.001210_93.914350_2.420000_469.571751_0.000000_0.100000': [300.1004721447762, 456.24071765363425, 0, 0, 0, 0, -306.010372212188, 0], '0.001210_77.615165_2.420000_469.571751_0.000000_0.100000': [272.0206504566704, 0, 0, -429.2162964483332, 0, 0, 0, 0], '0.001210_77.615165_2.662000_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 429.2162964483332, 0, 0], '0.001100_137.500000_1.652893_568.181818_0.000000_0.100000': [390.03852150613994, 0, -3.854731555091712, -715.5792128277348, 0, -107.6728168951173, -351.12094428849326, -73.77791434003655], '0.001000_125.000000_1.502630_469.571751_0.000000_0.100000': [0, 536.041470217806, -4.599146436125912, 0, 0, 2746.6825723079132, -373.7858454042471, -465.8463258758584], '0.001000_125.000000_1.502630_426.883410_0.000000_0.100000': [0, 832.6338759275989, -4.975899595234114, 0, 4.512039492127497, 778.6780299312545, -27.283644022719535, -4.829202434394666], '0.000909_125.000000_1.366027_426.883410_0.000000_0.100000': [0, 0, 0, -77.83736594423952, 0, 286.81535205998847, -445.61658648643845, 0], '0.000909_125.000000_1.241843_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, -20.818117961356894, 0, 0], '0.000909_125.000000_1.128948_426.883410_0.000000_0.100000': [0, 0, 0, 0, 1.1096904814822666, 0, 0, 0], '0.000826_125.000000_1.128948_426.883410_0.000000_0.100000': [255.44523008750977, 0, 0, 0, 0.988602725263172, 0, 0, 0], '0.000751_125.000000_1.128948_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0.8819526634445538, 0, 0, 0], '0.000683_125.000000_1.128948_426.883410_0.000000_0.100000': [0, 345.95341667220373, 0, 0, 0, 0, 0, 0], '0.000683_125.000000_1.128948_388.075827_0.000000_0.100000': [0, 0, -0.6673780898800942, 0, 0, 0, 0, 0], '0.000751_125.000000_1.128948_388.075827_0.000000_0.100000': [0, 332.1488971581601, 0, 0, 0, 0, 0, 0], '0.000751_125.000000_1.128948_352.796206_0.000000_0.100000': [247.56683843354332, 0, 0, 0, 0, 0, 0, 0], '0.000751_113.636364_1.128948_352.796206_0.000000_0.100000': [0, 0, 0, 0, -0.1355659995626411, 0, 0, 0], '0.000683_113.636364_1.128948_352.796206_0.000000_0.100000': [0, 291.60319744997497, 0, 0, 0, 0, 0, 0], '0.000683_113.636364_1.128948_320.723824_0.000000_0.100000': [173.93688066155593, 0, 0, 0, 0, 0, 0, 0], '0.000683_103.305785_1.128948_320.723824_0.000000_0.100000': [129.80976316947755, 0, 0, 0, 0, 0, 0, -288.6411780105509], '0.000683_103.305785_1.128948_352.796206_0.000000_0.100000': [137.2797870751624, 0, 0, 0, 0, 0, 0, 0], '0.001000_103.305785_1.502630_687.500000_0.000000_0.100000': [0, 442.6119360999543, 0, 0, 0, 1320.3296339393755, 0, 0], '0.001000_103.305785_1.502630_625.000000_0.000000_0.100000': [339.4739740944966, 1106.1787445699656, 0, 0, 0, 1415.4610650965901, 0, 0], '0.001000_93.914350_1.502630_568.181818_0.000000_0.100000': [380.7960474339085, 165.3088438320467, 0, 0, 4.292728192620053, 996.0669962963999, 0, 0], '0.001000_93.914350_1.502630_516.528926_0.000000_0.100000': [4374.979093985742, 378.0368380637912, -4.850482777702382, -165.19244805288355, 4.4071728057259865, 0, 86.24026385208042, -495.46846384224517], '0.000909_93.914350_1.502630_516.528926_0.000000_0.100000': [0, 536.9185366341408, 0, -168.08344501050087, -0.27414483528152217, 2734.8141138737683, -3.526125148749038, 0], '0.000909_93.914350_1.502630_469.571751_0.000000_0.100000': [365.3280457871014, 0, 4775.368717950822, -199.10006931039152, 0, 1995.4598209445512, 0, 399.5928888905977], '0.001000_93.914350_1.502630_469.571751_0.000000_0.100000': [4911.01383850621, 1624.000594281427, 0, -200.19533942757135, 4.209166025420927, 1380.357790679638, 0, 0], '0.000909_85.376682_1.502630_469.571751_0.000000_0.100000': [1269.593922217208, 0, -4.8511179721761435, 0, 4.406881498875191, 0, 0, 0], '0.001000_85.376682_1.502630_469.571751_0.000000_0.100000': [3922.0153415248787, 507.356770564669, -5.336738429286015, 0, 1048.5667922753407, 1282.3239813027592, 0, 0], '0.001100_85.376682_1.502630_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -363.916334524501, 0], '0.001100_93.914350_1.502630_469.571751_0.000000_0.100000': [0, 0, 0, 0, 4.566979113440811, 0, 0, 0], '0.000909_125.000000_2.000000_516.528926_0.000000_0.100000': [803.1179392019399, 1550.5504396387682, -200.39180607311823, -746.6188888953085, 5.706954256242341, -136.78459148180355, -457.49910610892624, 0], '0.000826_125.000000_2.000000_516.528926_0.000000_0.100000': [974.3992013433424, 0, -5.141400230849103, 0, 0, 0, 1105.8919311684149, 1327.6016798125365], '0.000909_151.250000_2.420000_516.528926_0.000000_0.100000': [0, 0, -1.6350539396578825, 0, 0, 0, 0, 0], '0.001000_151.250000_2.420000_469.571751_0.000000_0.100000': [0, 0, -1.4724351221113465, 0, 0, 0, 0, 0], '0.001100_151.250000_2.420000_469.571751_0.000000_0.100000': [0, 0, -1.6515881384515525, 0, 0, 0, 0, 0], '0.001210_151.250000_2.662000_469.571751_0.000000_0.100000': [300.01931522433296, 0, 0, 0, 4.252818481928261, 0, 0, 0], '0.001100_151.250000_2.662000_469.571751_0.000000_0.100000': [0, 0, -4.6781003301210875, 0, 0, 0, 0, 0], '0.001210_137.500000_2.662000_469.571751_0.000000_0.100000': [0, 0, 0, 0, 4.112666165712471, 0, 0, 0], '0.001100_137.500000_2.662000_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -303.70669954185263, 0], '0.000751_103.305785_2.000000_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 585.4402328083594, 0, 0], '0.000751_125.000000_1.818182_516.528926_0.000000_0.100000': [0, 0, 1740.59007110803, -709.8016490977436, 0, 0, 0, 0], '0.000751_125.000000_2.000000_516.528926_0.000000_0.100000': [-119.4195006202574, 0, -4.668934877681932, 0, 4.243094795428078, 0, 0, -516.9926763817465], '0.000826_113.636364_1.652893_516.528926_0.000000_0.100000': [1202.078635705701, 0, 0, 0, 0, 0, -320.74834832120735, 0], '0.000826_125.000000_1.652893_516.528926_0.000000_0.100000': [291.58940756473396, 491.658001900411, 0, 1818.4777211759788, 0, -131.49271767738065, -38.0436171288419, 0], '0.000826_137.500000_1.818182_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 333.67481427461775, 0, 102.57064438495846], '0.000826_137.500000_1.652893_469.571751_0.000000_0.100000': [1277.7137601749353, 0, 0, -30.334074024965258, 2.2260999651049134, 0, 0, -419.6554785574499], '0.001100_103.305785_1.502630_568.181818_0.000000_0.100000': [0, 469.7311534081832, -4.587740394354878, 0, 1522.6304956553938, 1923.449919204692, 0, -441.9551296101603], '0.001100_103.305785_1.366027_568.181818_0.000000_0.100000': [0, 1629.5327315626394, 408.15490812220935, -10.033314241189101, 5.239676598958067, -103.19166072787229, 98.95576371317179, 0], '0.001100_113.636364_1.366027_568.181818_0.000000_0.100000': [705.3605693758228, 0, -6.128777166449709, 0, 0, 0, -457.3908303408659, 0], '0.001210_113.636364_1.366027_568.181818_0.000000_0.100000': [507.5737808668266, 0, 0, -94.5641748589, 6.128777166449709, 0, 0, -869.4886046848602], '0.001100_125.000000_1.366027_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 585.7018203224817, 0, 0], '0.001100_125.000000_1.241843_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -402.9582069953373], '0.001100_125.000000_1.241843_625.000000_0.000000_0.100000': [0, 0, 0, -638.7126079433319, 0, 0, 0, 0], '0.001100_125.000000_1.366027_625.000000_0.000000_0.100000': [439.05925684007525, 0, 0, 0, 0, 0, 0, 0], '0.001100_113.636364_1.366027_625.000000_0.000000_0.100000': [1016.3828568178723, 0, 0, 0, 0, 0, 0, -535.783845159866], '0.001100_113.636364_1.366027_687.500000_0.000000_0.100000': [0, 969.2000901591358, 0, 0, 5.590239989769725, 0, 0, 0], '0.001100_103.305785_1.366027_625.000000_0.000000_0.100000': [367.2018102121928, 0, 0, 0, 1023.4037564234594, 428.2920539734778, 0, -502.5064984726952], '0.001100_103.305785_1.366027_687.500000_0.000000_0.100000': [357.0455556021899, 0, -5.93126257939457, -903.1304203296331, 0, 566.8646614692086, 0, 0], '0.001210_103.305785_1.366027_687.500000_0.000000_0.100000': [357.04306834042603, 0, 0, 0, 0, 0, -392.80266219864444, 0], '0.001210_113.636364_1.366027_687.500000_0.000000_0.100000': [392.80266219864444, 0, 0, -801.6152934847787, 0, 0, 0, 0], '0.001210_113.636364_1.502630_687.500000_0.000000_0.100000': [294.198370808616, 0, 0, -37.082077435138494, 0, 0, 0, 0], '0.000909_125.000000_1.652893_687.500000_0.000000_0.100000': [0, 0, -2.8491719151143116, 0, -255.53475669576432, 1203.0395362778536, 0, -799.8625713836248], '0.001100_125.000000_1.502630_625.000000_0.000000_0.100000': [1628.8418456433437, 0, -1.8834829859792082, 1788.923572863139, -272.87858183139866, 0, 44.91702639222129, -177.0118956703135], '0.001100_125.000000_1.502630_687.500000_0.000000_0.100000': [330.88660408135615, -291.3864881898965, 0, 0, 0, 0, 0, -450.5215365739111], '0.001210_113.636364_1.502630_756.250000_0.000000_0.100000': [0, 0, 0, 0, 2.1738905344095656, 0, 0, 0], '0.001100_113.636364_1.652893_687.500000_0.000000_0.100000': [1629.1267632763352, 330.2045309551163, 0, 0, 114.23351314038587, 442.1982914166909, 0, 0], '0.001000_103.305785_1.366027_687.500000_0.000000_0.100000': [1188.5418488460164, 0, -5.381946050500301, 0, 0, 0, 0, -256.21682609753867], '0.001100_103.305785_1.502630_687.500000_0.000000_0.100000': [1000.8557791922075, 0, -3.020427124568596, 0, 0, 0, 1420.8650404523546, 0], '0.001100_93.914350_1.502630_687.500000_0.000000_0.100000': [0, -705.4271948035008, 0, 0, -1117.6558424623756, 0, 0, 0], '0.001100_93.914350_1.502630_625.000000_0.000000_0.100000': [379.0395480259026, 0, 0, 0, 0, 0, -338.1913253075436, 0], '0.000751_93.914350_2.000000_687.500000_0.000000_0.100000': [0, 613.9280555362595, 0, 0, 0, 0, 1079.8620661301036, 2358.2160836576995], '0.000751_103.305785_2.000000_687.500000_0.000000_0.100000': [-1079.8620661301036, -809.7675298841878, -0.3738879671865107, 0, 3.7376434234687257, 0, -1804.1823785843155, -60.448868031601194], '0.000909_93.914350_2.000000_625.000000_0.000000_0.100000': [2436.0140132784773, 0, -5.5345638762419185, 0, 0, 0, 0, -614.0148296051435], '0.000826_93.914350_2.000000_687.500000_0.000000_0.100000': [334.1876827619045, 0, 0, 0, 0, 0, 0, 0], '0.001210_125.000000_2.200000_687.500000_0.000000_0.100000': [0, 566.7601311426317, 0, -457.0587919331207, 0, 0, 0, -493.5300181577137], '0.001331_125.000000_2.420000_625.000000_0.000000_0.100000': [741.7855057227182, 0, -3.374212002508102, 0, 0, 0, -253.74974498318826, 0], '0.001331_137.500000_2.420000_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -225.84679952985172, 0], '0.001331_151.250000_2.420000_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 232.3845926556056, 0, 0], '0.001331_151.250000_2.200000_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 1202.6395022721067, 0, 0], '0.001331_151.250000_2.000000_625.000000_0.000000_0.100000': [0, 0, 0, -109.33086384291867, 7.521228310124025, 0, 0, 0], '0.001210_151.250000_2.000000_625.000000_0.000000_0.100000': [2174.863234769547, 0, 0, -1097.1846956403024, 0, 0, 0, 0], '0.001210_137.500000_2.000000_625.000000_0.000000_0.100000': [0, 511.53566227521117, 0, -1171.6590179526033, 0, 1118.886127373898, -430.8583568534614, -942.2987881940894], '0.001210_125.000000_2.420000_687.500000_0.000000_0.100000': [0, 732.5156199866456, -3.7191231696166143, 0, 0, 0, 0, 0], '0.001331_125.000000_2.420000_687.500000_0.000000_0.100000': [0, 0, -4.076844009621346, 0, 0, 455.63121228804374, 0, 0], '0.001464_125.000000_2.420000_687.500000_0.000000_0.100000': [0, 0, -4.408341248855322, 11836.671235773978, 0, 0, 0, 0], '0.001464_125.000000_2.662000_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001000_137.500000_2.662000_625.000000_0.000000_0.100000': [216.21432096394747, 0, 0, 0, 0, 0, 0, 0], '0.001000_125.000000_2.662000_625.000000_0.000000_0.100000': [0, 746.5407766938516, 0, 0, 0, 962.0450444032401, 0, 0], '0.001000_113.636364_2.662000_516.528926_0.000000_0.100000': [0, 0, 0, -270.7866008957732, 0, 339.72118693222404, 0, 0], '0.001000_113.636364_2.928200_516.528926_0.000000_0.100000': [0, 489.46145930683105, 0, 0, 0, 0, 0, 0], '0.001000_113.636364_2.928200_469.571751_0.000000_0.100000': [241.5776584719966, 0, 0, 0, 0, 0, 0, -44.900642497720185], '0.001000_103.305785_2.928200_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 398.60033849342364, 0, 0], '0.001000_103.305785_2.662000_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -572.0014838695754], '0.001000_103.305785_2.662000_516.528926_0.000000_0.100000': [0, 57.200148386957494, 0, 0, 0, 0, 0, 0], '0.001000_103.305785_2.000000_426.883410_0.000000_0.100000': [941.8551637686761, 0, 0, -1293.9628588239752, 0, 0, 0, 0], '0.001000_103.305785_2.200000_426.883410_0.000000_0.100000': [0, 1612.4235790719072, -6.470179729512893, -915.8734720447464, 11.284472601997622, 124.62652973508068, 0, 53.771167784452096], '0.001000_103.305785_2.200000_388.075827_0.000000_0.100000': [472.9180243248594, 1430.5676188932248, 0, -953.0839707182033, 5.600560015930569, 0, 0, -512.5245447510987], '0.001100_103.305785_2.200000_469.571751_0.000000_0.100000': [1839.551102003004, 0, 0, -859.884864670882, 0, 0, 0, 0], '0.001100_103.305785_2.420000_469.571751_0.000000_0.100000': [0, 0, 0, -270.71566054314917, 0, 0, 0, 0], '0.001100_103.305785_2.662000_469.571751_0.000000_0.100000': [0, 0, 0, -397.42855739709086, 0, 0, 0, 0], '0.001100_103.305785_2.928200_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -240.6037085155953, 0], '0.001100_113.636364_2.928200_469.571751_0.000000_0.100000': [0, 486.5177412878664, 0, 0, 0, 0, 0, 0], '0.001100_113.636364_2.928200_426.883410_0.000000_0.100000': [0, 0, -2.5529710332357354, 0, 0, 0, 0, 0], '0.001210_113.636364_2.928200_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -278.380258473777, 0], '0.001210_125.000000_2.928200_426.883410_0.000000_0.100000': [0, 494.85940270386476, 0, 0, 0, 0, 0, 0], '0.001210_125.000000_2.928200_388.075827_0.000000_0.100000': [0, 0, 0, 153.80247593850254, 0, 0, -230.2067977008979, 0], '0.001210_137.500000_2.928200_388.075827_0.000000_0.100000': [230.2067977008979, 0, 0, 0, 2.9490463367126725, 0, 0, 0], '0.001100_137.500000_2.928200_388.075827_0.000000_0.100000': [0, 0, -2.9490463367126725, 0, 0, 0, 0, 0], '0.001210_125.000000_3.221020_388.075827_0.000000_0.100000': [0, 0, 0, -309.3698480802888, 0, 0, 0, -467.7146045163836], '0.001210_125.000000_3.543122_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 309.3698480802888, 0, 0], '0.001210_125.000000_3.221020_426.883410_0.000000_0.100000': [0, 0, -1.4802879290376951, 0, 0, 0, 0, 0], '0.001331_125.000000_3.221020_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001000_125.000000_2.200000_469.571751_0.000000_0.100000': [407.5547699986277, 535.0904554985524, 1714.4350108256258, 1122.4708284574492, 0, 0, 0, 0], '0.001000_113.636364_2.200000_388.075827_0.000000_0.100000': [0, 508.834939767774, 0, 0, 5.160683317779513, 0, 0, 0], '0.000909_113.636364_2.200000_388.075827_0.000000_0.100000': [0, 0, 0, 0, 404.7981462066261, 0, 0, 0], '0.000826_113.636364_2.200000_388.075827_0.000000_0.100000': [452.6597113976235, 0, 0, 0, 0, 0, -401.1613716574631, 0], '0.000826_125.000000_2.200000_388.075827_0.000000_0.100000': [804.5839759945463, 0, 0, 0, 0, 0, -1073.2892653163897, -527.8597584513473], '0.000826_151.250000_2.200000_388.075827_0.000000_0.100000': [408.6006446097019, 0, 0, 0, 0, 0, 0, 0], '0.000826_137.500000_2.420000_426.883410_0.000000_0.100000': [0, 444.0433047252904, 0, 0, 0, 0, 0, 0], '0.000826_151.250000_2.420000_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -214.573898128156, 0], '0.000826_166.375000_2.420000_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 413.3193414966454, 0, 0], '0.000826_166.375000_2.200000_388.075827_0.000000_0.100000': [0, 0, 0, 0, 3.8784093993101125, 0, 0, 0], '0.000751_166.375000_2.200000_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 717.3514226151868, 0, 0], '0.000751_166.375000_2.000000_388.075827_0.000000_0.100000': [0, 0, 0, 0, 3.7453133351393264, 0, 0, 0], '0.000683_166.375000_2.000000_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001000_125.000000_1.366027_426.883410_0.000000_0.100000': [424.17042189528496, 0, 0, -778.6780299312545, 0, 0, 0, 0], '0.001100_125.000000_1.502630_426.883410_0.000000_0.100000': [0, 463.2785491034034, 0, 0, 0, 0, -422.9828694635495, 0], '0.001210_137.500000_1.502630_426.883410_0.000000_0.100000': [0, 0, 0, 125.96296996454475, 0, 0, 0, -429.6781017419211], '0.001210_137.500000_1.652893_426.883410_0.000000_0.100000': [1076.954455004945, 0, -3.1861610932810436, 0, 385.5963857869545, 0, -75.77191699248732, -409.8586619374241], '0.001331_137.500000_1.652893_426.883410_0.000000_0.100000': [0, 0, 0, 0, 3.1861610932810436, 0, 0, 0], '0.001210_137.500000_1.818182_469.571751_0.000000_0.100000': [0, 0, 0, -759.2916750973203, 0, 0, 0, -409.00620367043797], '0.001331_137.500000_1.818182_516.528926_0.000000_0.100000': [0, 0, -4.618020409705605, -829.0648104704396, 0, 0, 0, 0], '0.001464_137.500000_1.818182_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001100_125.000000_1.818182_388.075827_0.000000_0.100000': [746.1288628183283, 387.91727527639705, -3.3529814340216038, 0, 3.026239559012083, 0, 0, 24.631229648800634], '0.001100_113.636364_1.818182_352.796206_0.000000_0.100000': [710.2161925815113, 0, -3.8108034487677287, 0, -355.52741585835855, 371.6125960940824, 0, 0], '0.001000_113.636364_1.818182_352.796206_0.000000_0.100000': [354.36036727637287, 0, -0.34409254357782637, 0, 0, 0, -70.9511201983974, 0], '0.001000_125.000000_1.818182_352.796206_0.000000_0.100000': [0, 1858.2800897919237, 0, 0, 3.7495839965467894, 0, 0, 0], '0.000909_125.000000_1.818182_352.796206_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -390.0470843060252], '0.000909_125.000000_1.818182_388.075827_0.000000_0.100000': [0, 0, 349.73367674258316, -536.3553975825038, 2.48073103220504, 0, -156.4001256029888, -393.2977985646401], '0.000909_137.500000_1.818182_388.075827_0.000000_0.100000': [403.78701150046703, 364.6579903444511, 0, 0, 0, 670.1513954483121, 0, 0], '0.000909_137.500000_1.652893_388.075827_0.000000_0.100000': [383.280574190355, 0, 0, -319.73954116392014, 0, -67.16732753979159, 0, 0], '0.000826_125.000000_1.818182_388.075827_0.000000_0.100000': [0, 0, -2.48073103220504, 0, 0, 0, 0, 0], '0.000909_125.000000_1.818182_426.883410_0.000000_0.100000': [1198.1864555290308, 421.66888906126326, 464.73489646615207, 0, 0, 0, -401.95229465476825, 840.7570197685664], '0.000909_137.500000_1.818182_426.883410_0.000000_0.100000': [0, 0, 0, 0, 46.82267248925467, 0, -60.680286287107116, 0], '0.000826_137.500000_1.818182_426.883410_0.000000_0.100000': [449.0962404209639, 0, 0, 0, 2.6440489396041813, 852.5423774801164, 0, 0], '0.001000_151.250000_1.652893_469.571751_0.000000_0.100000': [0, 808.5925463661184, 0, 0, 0, 0, -402.71483449049686, 274.2593759333497], '0.001000_151.250000_1.652893_426.883410_0.000000_0.100000': [418.8837342974183, 377.98641102167494, -2.6769479746939395, 0, 2.4325182214575762, 0, 0, 0], '0.001100_151.250000_1.652893_426.883410_0.000000_0.100000': [0, 380.37533453095045, 0, -396.7692364459871, 2.6769479746939395, -139.69332361099404, 0, 0], '0.001100_151.250000_1.502630_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, 676.6603278769138, 0, 0], '0.001100_151.250000_1.366027_426.883410_0.000000_0.100000': [0, 398.8987517013958, 0, 0, 0, 0, 0, 0], '0.001100_151.250000_1.366027_388.075827_0.000000_0.100000': [0, 0, 0, -687.5499653803162, 0, 0, 0, 0], '0.001100_151.250000_1.502630_388.075827_0.000000_0.100000': [444.975182190909, 0, 0, 0, 0, 0, 0, 0], '0.001100_137.500000_1.502630_388.075827_0.000000_0.100000': [0, 0, -5.157923935206964, 0, 4.698605738782499, 0, 0, -484.4554221777923], '0.001000_137.500000_1.502630_388.075827_0.000000_0.100000': [446.05804136359524, 420.87366762844425, -4.698605738782499, 0, 0, 0, 0, -484.87551940592766], '0.001000_137.500000_1.502630_426.883410_0.000000_0.100000': [0, 88.1591853465323, 0, 0, 0, 0, 0, -432.706110761597], '0.001000_137.500000_1.502630_352.796206_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -473.5400346372985, 0], '0.001000_151.250000_1.502630_352.796206_0.000000_0.100000': [0, 0, -4.111956864353397, 0, 0, 0, 0, 0], '0.001100_151.250000_1.502630_352.796206_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -458.90793477219114, 0], '0.001100_166.375000_1.502630_352.796206_0.000000_0.100000': [0, 0, 0, 114.37977750525306, 0, 0, 0, 0], '0.001100_166.375000_1.652893_352.796206_0.000000_0.100000': [0, 0, 0, -425.38422421327334, 0, 0, 0, 0], '0.001100_166.375000_1.818182_352.796206_0.000000_0.100000': [0, 0, 0, -553.5938562485211, 0, 0, 0, 0], '0.001000_125.000000_2.200000_756.250000_0.000000_0.100000': [0, 214.27303326006546, 0, 0, 3.358426250705088, 2266.8146270613565, -291.81958994315903, 0], '0.000909_125.000000_2.200000_756.250000_0.000000_0.100000': [0, 0, -3.358426250705088, 0, 0, 0, 0, 0], '0.001000_137.500000_2.200000_756.250000_0.000000_0.100000': [0, 0, -2.8665157473773433, 0, 0, 0, 134.7475500435581, 0], '0.001100_137.500000_2.200000_756.250000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001000_151.250000_2.000000_516.528926_0.000000_0.100000': [1483.3248939855964, 0, 0, 0, 5.685870963583875, 903.6061585345402, 0, 0], '0.000909_151.250000_1.652893_516.528926_0.000000_0.100000': [637.0309792347612, 0, -0.32763545030957175, -331.9930159471525, 0, 0, 0, 0], '0.000909_137.500000_1.652893_756.250000_0.000000_0.100000': [0, 518.2926840824935, 0, -6.62250722744551, 3.2060389192805814, 0, 0, 0], '0.001000_137.500000_1.818182_756.250000_0.000000_0.100000': [0, 0, 0, 0, 458.68602873536184, 299.41620118711245, 0, -570.8027198277472], '0.001000_137.500000_1.652893_756.250000_0.000000_0.100000': [0, 497.324136142112, -3.9070966032772048, 0, -238.5857385460713, 0, 0, 0], '0.001000_137.500000_1.652893_687.500000_0.000000_0.100000': [737.3171888954753, 47.79189709302861, 729.3717415461119, 0, 519.8367822498999, 310.2526462637359, -316.1174998967341, 0], '0.001100_137.500000_1.652893_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -358.40980208572495, 0], '0.001100_151.250000_1.652893_469.571751_0.000000_0.100000': [0, 0, 0, -348.1120001197864, 0, 0, 0, 0], '0.001100_151.250000_2.000000_469.571751_0.000000_0.100000': [501.24485900820235, 0, -6.728517434204696, 0, 0, 1753.4961585533624, -504.8573277313368, 0], '0.001100_166.375000_2.000000_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -487.68932788256495], '0.001100_166.375000_2.200000_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 933.2947702081552, 0, -495.78300718326557], '0.001210_166.375000_2.000000_516.528926_0.000000_0.100000': [0, 437.89493331219273, 0, 0, 0, 0, 0, -455.22394845398], '0.001210_166.375000_2.000000_469.571751_0.000000_0.100000': [504.82096253180174, 0, 0, 0, -387.3321267720802, 0, 0, 0], '0.001210_166.375000_2.000000_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 998.6779928333734, 0, 0], '0.001331_166.375000_1.818182_568.181818_0.000000_0.100000': [0, 0, 0, 0, 10.284463584936404, 0, 0, 0], '0.001100_166.375000_1.818182_516.528926_0.000000_0.100000': [0, 431.2639732177639, 0, 0, 0, 0, 0, 0], '0.001100_166.375000_1.818182_469.571751_0.000000_0.100000': [444.31139588915795, 0, 0, 0, 0, 0, 0, 0], '0.000751_137.500000_2.000000_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 1184.1398723132218, 0, 0], '0.000621_125.000000_1.652893_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 46.03868512734243, 0, 0], '0.000621_125.000000_1.502630_687.500000_0.000000_0.100000': [0, 391.46791158453163, 0, 0, 0, 0, 0, 0], '0.000751_125.000000_1.241843_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -347.0554018459092, 0], '0.000751_137.500000_1.241843_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -401.58539863314417, 0], '0.000751_151.250000_1.241843_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 34.69702336197173, 0, 0], '0.000751_151.250000_1.128948_625.000000_0.000000_0.100000': [0, 0, 0, -3.1542748510883376, 0, 0, -368.16731094317873, 0], '0.000751_166.375000_1.128948_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000826_103.305785_1.652893_516.528926_0.000000_0.100000': [264.9383847296383, 3364.7330721905782, 0, 0, 0, 0, -520.9015840182068, -464.2929871601994], '0.000751_103.305785_1.652893_568.181818_0.000000_0.100000': [280.8210084976972, 0, 0, -179.2335200024066, 0, 0, -260.21350613800024, 0], '0.000826_77.615165_1.818182_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -429.9798234442509, 0], '0.000826_85.376682_1.652893_756.250000_0.000000_0.100000': [224.57499145277325, 0, -2.1479706311607516, 0, 0, 0, 0, 0], '0.000909_85.376682_1.652893_756.250000_0.000000_0.100000': [0, 0, 0, 0, 2.1479706311607516, 0, 0, 0], '0.000826_77.615165_1.652893_756.250000_0.000000_0.100000': [6420.810789363722, 0, 0, 0, 0, 0, -224.57499145277325, 0], '0.001000_125.000000_1.652893_756.250000_0.000000_0.100000': [0, 0, -3.439813284165757, -261.9532052283501, 0, 114.03106994227073, 0, -493.6678802483097], '0.001210_125.000000_1.652893_756.250000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -314.88960964666893, 0], '0.001210_137.500000_1.652893_756.250000_0.000000_0.100000': [0, 0, 0, -271.38499004187616, 0, 0, 0, 0], '0.001210_137.500000_1.818182_756.250000_0.000000_0.100000': [837.548741259199, -323.35594263642815, 0, 0, 0, 0, 0, 0], '0.001210_137.500000_1.502630_687.500000_0.000000_0.100000': [0, 386.09515323969083, 0, 0, 0.9248351088037453, 0, 0, -388.88264356419023], '0.001100_137.500000_1.502630_687.500000_0.000000_0.100000': [627.9089486171738, 744.4210394899338, -1.0173186196841197, 0, 0, 0, -291.12195323021115, 0], '0.001210_137.500000_1.502630_756.250000_0.000000_0.100000': [850.6055464902374, 0, 0, 0, 0, 0, 0, 0], '0.001210_125.000000_1.502630_756.250000_0.000000_0.100000': [0, 0, 0, 0, 0, 665.0233745260622, 0, 0], '0.001210_125.000000_1.366027_756.250000_0.000000_0.100000': [405.965206905018, 0, 0, 0, 0, 732.6635168438523, 0, -490.00216315926394], '0.001210_113.636364_1.366027_756.250000_0.000000_0.100000': [376.25637912323145, 498.3171707182002, 0, 0, 0, 0, 0, 0], '0.001210_103.305785_1.366027_756.250000_0.000000_0.100000': [0, 0, 0, 0, 5.760668613416056, 0, 0, 0], '0.001100_103.305785_1.366027_756.250000_0.000000_0.100000': [344.84034047072913, 0, 0, 0, 0, 0, -376.03838299798156, -105.92459774308497], '0.001100_103.305785_1.366027_831.875000_0.000000_0.100000': [0, 529.6229887154252, 0, 0, 0, 0, 0, 0], '0.001000_113.636364_2.420000_426.883410_0.000000_0.100000': [564.7124495763765, 0, 0, -146.08728885179553, 0, 0, 0, 0], '0.001000_113.636364_2.662000_426.883410_0.000000_0.100000': [0, 0, 0, -444.9077680433267, 0, 0, 0, 0], '0.001000_113.636364_2.928200_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, 444.9077680433267, 0, 0], '0.000751_137.500000_2.420000_568.181818_0.000000_0.100000': [0, 464.0918890305406, 0, 0, 0, 0, 126.85504135825434, 0], '0.000751_151.250000_2.420000_568.181818_0.000000_0.100000': [0, 483.6292185028041, 0, 0, 0, 0, 0, 0], '0.000751_151.250000_2.420000_516.528926_0.000000_0.100000': [26.861473110461255, 0, -1.4630242639520057, 0, 0, 0, 0, 0], '0.000826_151.250000_2.420000_516.528926_0.000000_0.100000': [0, 424.64346618656964, 0, -326.90922543305606, 0, 0, 0, -441.03645417404294], '0.000826_151.250000_2.662000_568.181818_0.000000_0.100000': [251.64098892077746, 0, 0, 0, 0, 0, 0, 0], '0.000826_137.500000_2.662000_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 398.0125618433785, 0, 0], '0.000751_137.500000_2.420000_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -296.6624401047649, 0], '0.000826_151.250000_2.420000_469.571751_0.000000_0.100000': [279.0938415660052, 0, 0, 0, 0, 0, 0, 0], '0.000826_137.500000_2.420000_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001000_103.305785_1.818182_831.875000_0.000000_0.100000': [0, 0, -4.178964566907417, 0, 0, 0, 0, 0], '0.001100_103.305785_1.818182_831.875000_0.000000_0.100000': [240.53487379437465, 0, 0, 0, 0, 0, 0, 0], '0.001100_93.914350_1.818182_831.875000_0.000000_0.100000': [0, 0, 0, -897.9902574557218, 0, 0, 0, 0], '0.001100_93.914350_2.000000_831.875000_0.000000_0.100000': [0, 3122.6295234444124, 0, 0, 0, 0, 0, 0], '0.001100_93.914350_2.000000_756.250000_0.000000_0.100000': [0, 0, 0, -3710.5502737621996, 0, 0, -2786.114096712587, 0], '0.001100_103.305785_2.000000_756.250000_0.000000_0.100000': [2786.114096712587, 1144.4316331525174, 0, -1205.1911149665457, 0, 0, 0, -620.0951889209975], '0.001100_93.914350_2.420000_756.250000_0.000000_0.100000': [195.52328544336726, 0, 0, 0, 0, 0, 0, 0], '0.001100_85.376682_2.420000_756.250000_0.000000_0.100000': [0, 596.2086337923938, 0, 0, 0, 745.9150642593362, 0, 0], '0.001100_85.376682_2.200000_756.250000_0.000000_0.100000': [0, 0, -6.135948445453141, 0, 0, 0, 0, 0], '0.001210_85.376682_2.200000_756.250000_0.000000_0.100000': [1755.3404217295285, 0, 0, 0, 0, 0, 0, -638.055968370951], '0.001210_85.376682_2.200000_831.875000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -624.1100128722416], '0.001210_85.376682_2.200000_915.062500_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000909_93.914350_2.000000_568.181818_0.000000_0.100000': [0, 624.2734772142479, 0, -1272.4837019314807, 0, 0, 0, 0], '0.000751_93.914350_2.420000_568.181818_0.000000_0.100000': [0, 744.7007840017702, 0, 0, 0, 0, 0, 0], '0.000751_77.615165_2.420000_516.528926_0.000000_0.100000': [0, 0, -2.206580967271839, 0, 0, 0, 0, 0], '0.000826_77.615165_2.420000_516.528926_0.000000_0.100000': [0, 0, -2.432217327620856, 0, 0, 0, 0, 0], '0.000909_77.615165_2.420000_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 2270.8027156329877, 0, 0], '0.000909_77.615165_2.200000_516.528926_0.000000_0.100000': [0, 0, -2.9146200419714936, 0, 0, 0, 0, -448.7250558589997], '0.001000_77.615165_2.200000_516.528926_0.000000_0.100000': [-900.8104824296495, -625.6920201741333, 0, 0, 2.914620041971493, 0, 0, 0], '0.001000_70.559241_2.200000_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 5701.205968404299], '0.001000_70.559241_2.200000_568.181818_0.000000_0.100000': [161.98827107171834, 0, 0, 0, 0, 0, -5251.428925903164, 0], '0.001000_77.615165_2.200000_568.181818_0.000000_0.100000': [0, 0, -1.9723133907540886, 0, 0, 0, 0, 0], '0.001100_77.615165_2.200000_568.181818_0.000000_0.100000': [5248.5074315045385, 0, 0, 0, 0, 0, 0, 0], '0.001100_70.559241_2.200000_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -3430.1316735143887], '0.001100_70.559241_2.200000_625.000000_0.000000_0.100000': [0, 0, 0, -4532.004416255388, 0, 0, 0, 0], '0.001100_70.559241_2.420000_625.000000_0.000000_0.100000': [0, 0, 0, 0, 3.322514445705019, 0, 0, 0], '0.001000_70.559241_2.420000_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 7984.157627734517, 0, 0], '0.001000_70.559241_2.200000_625.000000_0.000000_0.100000': [141.42131970343272, 0, 0, 0, -3453.031636848893, 0, 0, 0], '0.000909_70.559241_2.200000_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 1268.4613286356978, 0, 0], '0.000909_70.559241_2.000000_687.500000_0.000000_0.100000': [6553.6519505284405, 0, 0, 0, 0, 0, 0, 0], '0.000909_64.144765_2.000000_687.500000_0.000000_0.100000': [0, 0, 0, 0.16615543834529944, 0, 0, 0, 0], '0.000909_64.144765_2.200000_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001331_125.000000_1.652893_568.181818_0.000000_0.100000': [308.25252253867126, 450.85985346401196, 0, 0, -233.4184163349592, 0, 0, 0], '0.001331_113.636364_1.652893_625.000000_0.000000_0.100000': [0, 83.41138163345232, 0, 0, 0, 964.6169939780608, -263.39874159032877, 0], '0.001331_125.000000_1.652893_625.000000_0.000000_0.100000': [614.1103918546451, 530.0334880663092, -5.63873064946442, 0, 0, -49.005839383617605, -308.9954174075792, 0], '0.000826_125.000000_2.000000_756.250000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -572.5950650110598], '0.000826_125.000000_2.000000_831.875000_0.000000_0.100000': [0, 572.5950650110598, 0, 0, 0, 0, 0, 0], '0.000909_125.000000_1.652893_756.250000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -156.37765572695605, 0], '0.000751_125.000000_1.366027_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 680.9418131938448, 0, 0], '0.000751_125.000000_1.241843_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -440.57358406937493], '0.000751_125.000000_1.241843_756.250000_0.000000_0.100000': [0, 0, -2.6204285197585704, 0, 0, 0, 0, 0], '0.000826_125.000000_1.241843_756.250000_0.000000_0.100000': [296.65986943744196, 0, 0, 0, 0, 0, 0, -499.1929913537066], '0.000826_125.000000_1.241843_831.875000_0.000000_0.100000': [0, 45.38118103215512, -2.95352099326974, 0, 0, 0, 0, 0], '0.000909_125.000000_1.241843_831.875000_0.000000_0.100000': [0, 453.8763892074289, 0, 0, 0, 0, 0, 0], '0.000909_125.000000_1.241843_756.250000_0.000000_0.100000': [0, 0, 0, -726.3480497628025, 0, 0, 0, 0], '0.000909_125.000000_1.366027_756.250000_0.000000_0.100000': [0, 0, 0, 0, 4.606939347045227, 0, -416.0437537528258, 0], '0.000909_137.500000_1.366027_756.250000_0.000000_0.100000': [0, 454.2268678410561, 0, 0, 0, 801.6258967789502, 0, 0], '0.000909_137.500000_1.241843_756.250000_0.000000_0.100000': [0, 434.14837523819153, 0, 0, 0, 0, 0, 0], '0.000909_137.500000_1.241843_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -395.317349115729, 0], '0.001100_103.305785_2.200000_687.500000_0.000000_0.100000': [0, 0, 0, 0, 5.493162054716595, 4110.855979724203, 2114.8270472248, 0], '0.001000_93.914350_1.652893_687.500000_0.000000_0.100000': [0, 0, -2.382170589402631, -252.19810975654238, 0, 0, 927.2488896343245, 0], '0.001100_93.914350_1.652893_687.500000_0.000000_0.100000': [249.59071921398427, 0, -413.310012717413, 0, 0, 0, 0, 0], '0.001331_103.305785_1.652893_687.500000_0.000000_0.100000': [0, 0, 0, 0, -568.6946403287776, 0, 0, 0], '0.001210_93.914350_1.652893_756.250000_0.000000_0.100000': [0, 0, -3.753163533835141, -289.61777270624225, 0, 107.89838493065551, 0, 0], '0.001210_93.914350_1.502630_756.250000_0.000000_0.100000': [270.06185113249995, 0, 0, 0, 0, 0, 0, 0], '0.001100_113.636364_1.652893_469.571751_0.000000_0.100000': [1284.4482575397851, 85.36964876764819, 256.3902685840559, 216.7013628412535, 0, 3007.4618635787147, -573.7664273943749, 0], '0.001100_103.305785_1.652893_469.571751_0.000000_0.100000': [244.5936151840383, 4153.7873822771835, 434.4012764462351, 240.48382821719275, 581.0801948258304, 0, -294.5750783910944, 0], '0.001100_93.914350_1.818182_426.883410_0.000000_0.100000': [1008.8525360157141, 0, -0.03549275029469308, -1408.4749040422435, 0, 72.23008545055191, -31.172170072665303, 0], '0.001100_85.376682_1.818182_426.883410_0.000000_0.100000': [243.97875284175095, 0, 0, 1099.5208781065771, 0, -9.14864992555904, 0, 0], '0.001100_85.376682_1.652893_426.883410_0.000000_0.100000': [190.39385992127617, 0, 0, 9.14864992555904, 0.7163910193476113, 1668.8923145659362, 0, 0], '0.001000_85.376682_1.652893_426.883410_0.000000_0.100000': [0, 0, 0, 11.89609469135803, 0, 0, -190.04340514412516, 0], '0.001000_93.914350_1.652893_426.883410_0.000000_0.100000': [0, 723.7193851269351, 0, 0, 0, 297.655138841771, 0, 0], '0.000909_103.305785_1.502630_469.571751_0.000000_0.100000': [4359.1951298643235, 0, 0, 0, 86.14543347518031, 908.8958425789981, 0, 1399.0230400811333], '0.000909_93.914350_1.366027_469.571751_0.000000_0.100000': [0, 0, 0, 0, 666.5787185638034, 0, 0, 928.9874862144745], '0.000826_93.914350_1.366027_469.571751_0.000000_0.100000': [982.6141793948921, 0, 0, 0, 0, 0, 0, 0], '0.001000_125.000000_2.200000_388.075827_0.000000_0.100000': [0, 0, 0, 0, 449.9602793524148, 0, 0, 0], '0.000909_125.000000_2.200000_388.075827_0.000000_0.100000': [0, 543.2693606642229, -0.4970150983308305, 0, 0, 109.2037131975189, 0, 0], '0.000909_125.000000_2.200000_352.796206_0.000000_0.100000': [0, 0, -4.5513904176250435, 0, 0, 0, 0, 0], '0.001000_137.500000_1.502630_568.181818_0.000000_0.100000': [0, 456.667316288206, 0, 0, 0.3063173966392423, 0, 0, 175.754530633539], '0.001100_137.500000_1.502630_516.528926_0.000000_0.100000': [0, 0, -2.362824053861914, 0, 0, 698.008750436802, 0, 0], '0.001100_137.500000_1.366027_516.528926_0.000000_0.100000': [0, 0, 0, -698.008750436802, 0, 0, 0, 0], '0.001210_137.500000_1.502630_516.528926_0.000000_0.100000': [0, 0, -2.4008771307086136, 0, 0, 694.4143848662403, 0, 0], '0.001210_137.500000_1.366027_516.528926_0.000000_0.100000': [0, 0, 0, -694.4143848662403, 0, 0, 0, 0], '0.000826_166.375000_1.818182_756.250000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -389.96260241327, 0], '0.000826_183.012500_1.818182_756.250000_0.000000_0.100000': [0, 0, 0, 0, 4.349374542688093, 0, 0, 0], '0.000751_183.012500_1.818182_756.250000_0.000000_0.100000': [389.8430328893146, 445.48135587708657, -0.39539768569891726, 0, 0, 0, 0, 0], '0.000751_183.012500_1.818182_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -445.48135587708657], '0.000751_166.375000_1.818182_756.250000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -480.6540651432375], '0.000751_166.375000_1.818182_831.875000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 11281.839297964696], '0.000751_166.375000_1.818182_915.062500_0.000000_0.100000': [0, -11281.839297964696, 0, 0, 0, 0, 0, 0], '0.001100_93.914350_1.818182_625.000000_0.000000_0.100000': [0, 0, -3.1901170582703346, -281.3533008077285, 2.929142757847694, 3913.8917779273143, 0, -501.3714314305962], '0.001000_93.914350_1.818182_687.500000_0.000000_0.100000': [1075.8847859577045, 0, -3.069941676599192, -419.1116898278782, 0, 335.1359880667748, 0, -533.3993435362205], '0.001100_93.914350_1.818182_687.500000_0.000000_0.100000': [0, 814.968433700883, 224.665836321811, 311.6929148145207, 0, 0, -694.662842628377, 0], '0.001000_93.914350_1.818182_756.250000_0.000000_0.100000': [0, 1467.0086482188754, -3.395060857133467, 0, 0, 0, 0, 0], '0.001000_85.376682_1.818182_687.500000_0.000000_0.100000': [1947.0403198000984, 0, 0, 691.8698447112655, 533.5854557545908, 0, 0, 2095.4011218035753], '0.001000_85.376682_1.818182_756.250000_0.000000_0.100000': [112.77804073733934, -135.42250742082388, -1619.4160426853887, 0, -2282.010863502956, 0, -2865.4620406351187, 0], '0.001100_85.376682_1.818182_756.250000_0.000000_0.100000': [0, -206.78235932847284, 0, 0, 0, 0, 0, 0], '0.001100_85.376682_1.818182_687.500000_0.000000_0.100000': [0, 0, -23.61737693757059, 0, 2086.120579170728, 0, 0, 0], '0.001210_85.376682_1.818182_687.500000_0.000000_0.100000': [0, 0, 0, -1136.771662272528, 0, -465.85700886905653, 0, 0], '0.001210_70.559241_2.000000_687.500000_0.000000_0.100000': [264.83160641832666, 0, 208.88311535270026, 0, 0, 0, 0, 0], '0.001331_70.559241_2.000000_687.500000_0.000000_0.100000': [5969.1696930621265, 0, 0, 0, 0, 0, 0, 0], '0.001331_64.144765_2.000000_687.500000_0.000000_0.100000': [6338.734362476979, 0, 0, 7035.190526697706, 0, 0, 0, 0], '0.001331_64.144765_2.200000_687.500000_0.000000_0.100000': [0, 0, -0.05505450287989788, 0, 0, -6338.009483511447, 0, 0], '0.001100_85.376682_2.000000_687.500000_0.000000_0.100000': [282.5815859296332, 0, 0, 0, 0, 1408.0850670685656, 0, 0], '0.001000_77.615165_1.818182_687.500000_0.000000_0.100000': [0, 0, 0, -3058.6819423553034, -1950.7778764405932, -1129.978368265336, 0, 0], '0.001000_77.615165_2.000000_687.500000_0.000000_0.100000': [0, 0, 0, 934.7626680157681, 0, 3058.6819423553006, 0, -664.4290293484075], '0.000909_77.615165_2.200000_687.500000_0.000000_0.100000': [0, 214.60692284517816, -0.04884655767921886, 0, 0, -931.7396547905417, 0, -1356.4493434711887], '0.000909_77.615165_2.000000_687.500000_0.000000_0.100000': [0, 0, -5.4653411091872846, 1034.0112073079447, 0, 0, 0, 0], '0.001000_77.615165_2.000000_756.250000_0.000000_0.100000': [0, 0, 0, 0, 604.0110224557876, 0, 0, 0], '0.000909_77.615165_2.000000_756.250000_0.000000_0.100000': [0, 664.3639252378255, 0, 0, 0, 0, 0, 0], '0.000909_77.615165_2.200000_625.000000_0.000000_0.100000': [2093.822189614709, 279.9895012093381, 0, 0, 254.5423551947802, 0, 0, 0], '0.000826_77.615165_2.200000_687.500000_0.000000_0.100000': [0, 0, 0, 0, 2.0000233653838224, 0, 0, 0], '0.000751_77.615165_2.200000_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001000_85.376682_2.000000_516.528926_0.000000_0.100000': [1038.1986199652806, 0, -127.31486341728807, 0, 844.4093584290176, 393.2637323896171, 0, 0], '0.001000_77.615165_2.000000_516.528926_0.000000_0.100000': [3818.4391185983277, 1800.7906104800657, 0, -608.7668560423066, 0.31932504639370574, 0, 0, -258.72283203262253], '0.001000_77.615165_2.200000_469.571751_0.000000_0.100000': [0, 0, -6.441594136475032, 0, 0, 0, -396.14736012389704, 0], '0.001100_77.615165_2.200000_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 5943.861964200016, 0, 0], '0.001100_77.615165_2.000000_469.571751_0.000000_0.100000': [2605.3565328928885, 0, 0, 0, 0, -1878.5334687124089, 0, 0], '0.001100_77.615165_1.818182_469.571751_0.000000_0.100000': [2542.041920990386, 0, 412.65973122855235, 4691.483963179114, 0, 12.415696280866724, 2652.178490272875, -46.19989271464171], '0.001210_77.615165_1.818182_469.571751_0.000000_0.100000': [0, 509.5623493516917, 0, 0, 0, 15.564957472582591, 0, 0], '0.001210_77.615165_1.652893_469.571751_0.000000_0.100000': [4044.1117620997093, 0, -0.8011389048382626, 0, 0, 0, 0, -388.7887173354591], '0.001331_77.615165_1.652893_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0.8812527953220888, 0, 0, -389.33952072180045], '0.001210_77.615165_1.652893_516.528926_0.000000_0.100000': [0, 0, 0, 0, 1.3067687593633581, 0, 0, 0], '0.001100_77.615165_1.652893_516.528926_0.000000_0.100000': [1330.9567922573317, 0, 0, -86.12987920637143, 0, 0, -228.35654808363452, 0], '0.001100_85.376682_1.652893_516.528926_0.000000_0.100000': [0, 0, -2.200142532957516, 0, 0, 262.93245699791913, 0, 0], '0.001210_85.376682_1.652893_516.528926_0.000000_0.100000': [230.23788049910908, 0, 0, 0, 0, 0, -256.52789046346834, 0], '0.001210_93.914350_1.652893_516.528926_0.000000_0.100000': [0, 1058.1124622830207, 0, 0, 0, 158.5681471022229, 0, -427.3542198551854], '0.001210_93.914350_1.652893_469.571751_0.000000_0.100000': [0, 425.51681869745715, 59.19935801821123, 0, 0, 189.9946220369041, 0, 0], '0.001210_93.914350_1.502630_469.571751_0.000000_0.100000': [0, 0, 0, -189.9946220369041, 0, 0, 0, 0], '0.001210_93.914350_1.652893_426.883410_0.000000_0.100000': [0, 0, 0, -74.53615532873482, 62.32353899911709, 0, 0, 0], '0.001210_93.914350_1.818182_426.883410_0.000000_0.100000': [0, 0, 0, 0, 273.02129535280585, 0, 0, 0], '0.001100_93.914350_1.652893_426.883410_0.000000_0.100000': [256.0207599403181, 0, 0, -72.23008545055272, 1.0938465294254456, 0, 0, 0], '0.001100_85.376682_1.502630_426.883410_0.000000_0.100000': [3650.2283877160507, 0, 0, -44.50769144494426, 0, 0, 0, 0], '0.001210_125.000000_1.818182_687.500000_0.000000_0.100000': [341.2983321948589, 588.9335640870897, 620.0173037779001, 0, 0, 0, 0, -247.83512177399336], '0.001000_125.000000_1.502630_756.250000_0.000000_0.100000': [0, 409.71383772998615, 0, 0, 0, 674.570748740678, 0, 0], '0.001000_125.000000_1.366027_756.250000_0.000000_0.100000': [0, 0, 0, -613.2461352187983, 5.073420395834001, 0, 0, 0], '0.001000_125.000000_1.502630_687.500000_0.000000_0.100000': [0, 441.47762832440947, -1.6523678298261075, 0, 0, 0, -300.79353667091164, 142.20768396693228], '0.001000_103.305785_1.366027_625.000000_0.000000_0.100000': [366.8929213416997, 168.4545999653006, 1017.7678233473998, 0, 5.027239545577323, 0, 0, -502.6637985799013], '0.001000_93.914350_1.366027_625.000000_0.000000_0.100000': [0, 560.5516962190992, 0, 0, 0, 0, 0, 0], '0.001000_93.914350_1.366027_568.181818_0.000000_0.100000': [951.5712982860947, 1181.7001218910782, -5.2397889234763175, 0, 425.1073978215179, 62.62427383780314, -166.8750526303573, -9.705778283057477], '0.001100_93.914350_1.366027_568.181818_0.000000_0.100000': [829.4755267225327, 0, -5.753588568675878, -995.5609829857555, 0, 0, -370.67765452426914, 0], '0.000751_103.305785_2.000000_516.528926_0.000000_0.100000': [0, 0, 0, 0, 1617.3778388863227, 0, 0, 0], '0.000683_103.305785_2.200000_469.571751_0.000000_0.100000': [0, 0, 0, 0, 4.001187612942886, 0, 0, 0], '0.000621_103.305785_2.200000_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 1267.7915941341537, 0, 0], '0.000621_103.305785_2.000000_469.571751_0.000000_0.100000': [383.639793852154, 0, 0, 0, 0, 0, -417.0544470347808, 0], '0.000621_113.636364_2.000000_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -448.59287710949593, 0], '0.000621_125.000000_2.000000_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -477.7454840459097, 0], '0.000621_137.500000_2.000000_469.571751_0.000000_0.100000': [0, 453.7889894537495, 0, 0, 0, 0, 0, 0], '0.000621_137.500000_2.000000_426.883410_0.000000_0.100000': [495.89829519032554, 0, 0, 0, 0, 0, 0, 0], '0.000621_125.000000_2.000000_426.883410_0.000000_0.100000': [0, 0, -3.754080377772334, 0, 0, 0, 0, 0], '0.000683_125.000000_2.000000_426.883410_0.000000_0.100000': [0, 446.33104002785706, 0, 0, 0, 0, 0, 0], '0.000683_125.000000_2.000000_388.075827_0.000000_0.100000': [522.3025481769296, 0, 0, 0, 0, 0, 0, 0], '0.000683_113.636364_2.000000_388.075827_0.000000_0.100000': [434.35878469305743, 0, 0, 0, 0, 444.60914762736746, 0, 0], '0.000683_113.636364_1.818182_388.075827_0.000000_0.100000': [0, 399.8650785947235, 0, 0, 0, 0, 0, 0], '0.000683_113.636364_1.818182_352.796206_0.000000_0.100000': [0, 0, 0, 0, 2.0788557164803936, 160.36140293652215, 0, 0], '0.000621_113.636364_1.818182_352.796206_0.000000_0.100000': [0, 0, -2.0788557164803936, 0, 0, 0, 0, 0], '0.000683_113.636364_1.652893_352.796206_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -371.5867914868083], '0.000683_113.636364_1.652893_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001100_113.636364_2.420000_756.250000_0.000000_0.100000': [0, 0, 0, 0, 0, 524.5987224943713, 0, 0], '0.001331_125.000000_2.200000_469.571751_0.000000_0.100000': [0, 0, 0, -584.1464844588626, 0, 0, -373.9597065589078, 0], '0.001331_125.000000_2.420000_469.571751_0.000000_0.100000': [0, 0, 0, -232.39985912980174, 0, 0, 0, 0], '0.001331_125.000000_2.662000_469.571751_0.000000_0.100000': [0, 0, 0, 0, 4.398508873732135, 0, 0, 0], '0.001210_125.000000_2.662000_469.571751_0.000000_0.100000': [0, 0, 0, -344.6073582016576, 3.978265118705167, 0, 0, 0], '0.001100_125.000000_2.662000_469.571751_0.000000_0.100000': [0, 0, -3.978265118705167, 0, 0, 0, 0, 0], '0.001210_125.000000_2.928200_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -229.33318213685342, 0], '0.001210_137.500000_2.928200_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0.3953928786993856, 0, 0, 0], '0.001100_137.500000_2.928200_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0.329317460870152, 0, 0, 0], '0.001000_137.500000_2.928200_469.571751_0.000000_0.100000': [0, 430.55044709302655, 0, 179.17993762255483, 0, 0, 0, 0], '0.001000_137.500000_3.221020_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, -179.17993762255483, 0, 0], '0.001000_137.500000_2.928200_426.883410_0.000000_0.100000': [0, 0, 0, 194.23289984304265, 1.324827221754458, 0, 0, 0], '0.000909_137.500000_2.928200_426.883410_0.000000_0.100000': [0, 0, -1.324827221754458, 0, 0, 0, -203.17709064014034, 0], '0.000909_151.250000_2.928200_426.883410_0.000000_0.100000': [203.17709064014034, 0, 0, 0, 0, 0, 0, 0], '0.001000_137.500000_3.221020_426.883410_0.000000_0.100000': [0, 480.544625622168, 0, 0, 0, 0, 0, 0], '0.001000_137.500000_3.221020_388.075827_0.000000_0.100000': [0, 0, -2.5687637823153637, 0, 0, 0, 0, 0], '0.001100_137.500000_3.221020_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001100_151.250000_2.200000_469.571751_0.000000_0.100000': [379.4676722242217, 0, 0, 0, 0, 2541.5731968968144, -378.94103995548215, 0], '0.000909_151.250000_1.652893_426.883410_0.000000_0.100000': [0, 0, -2.4325182214575762, -359.62792992130875, 0, 0, 0, 0], '0.000909_151.250000_1.818182_426.883410_0.000000_0.100000': [0, 394.74000269505984, 0, -781.2316337032522, 0, 359.62792992130875, 0, 0], '0.000909_151.250000_1.818182_388.075827_0.000000_0.100000': [809.9880126943888, 0, 0, 0, 0, 382.15586148332005, 0, -355.62162404960344], '0.000909_137.500000_1.818182_352.796206_0.000000_0.100000': [0, 0, 0, 0, 0, 351.9193242218463, -457.07593880008716, 0], '0.000909_151.250000_1.818182_352.796206_0.000000_0.100000': [457.0759388000888, 0, 0, 0, 0, 0, 0, 0], '0.000826_103.305785_2.200000_469.571751_0.000000_0.100000': [413.44437044887, 521.8427160862274, -5.8555758694382165, 0, 0, 0, 0, 0], '0.000909_103.305785_2.200000_469.571751_0.000000_0.100000': [0, 185.77947022539627, -5.8544878558379425, -86.84546731460038, 5.323250790398379, 1686.8782496712502, 0, -534.2503317331981], '0.001000_103.305785_2.200000_469.571751_0.000000_0.100000': [0, 1670.5181669387243, 406.53402626624944, 0, 0, 0, 0, -533.7467833438108], '0.000909_103.305785_2.200000_426.883410_0.000000_0.100000': [0, 0, -5.882653618450786, 0, 5.883232204302149, 0, -41.5081710880267, 0], '0.001000_93.914350_2.200000_388.075827_0.000000_0.100000': [412.66339092293094, 0, 0, 0, 0, 0, 0, -511.3596749320761], '0.000909_85.376682_2.200000_426.883410_0.000000_0.100000': [0, 2184.970094182597, 0, 0, 5.317455105910631, 0, 0, -553.9634626406686], '0.000909_85.376682_2.200000_469.571751_0.000000_0.100000': [0, 553.9634626406686, 0, 0, 0, 0, 0, 0], '0.000909_85.376682_2.200000_388.075827_0.000000_0.100000': [0, 0, 0, -1194.9708706449685, 0, 2174.7593630985484, 0, 0], '0.000909_85.376682_2.000000_388.075827_0.000000_0.100000': [0, 0, 0, 0, 23.881741667780233, 0, 0, 0], '0.001100_113.636364_2.000000_516.528926_0.000000_0.100000': [1336.9566965945423, 664.4041521889363, 316.32684350592683, -1769.2674530887969, 6.1240166544303065, 842.2863061902308, -389.3974895968361, -742.3042522795731], '0.001000_125.000000_2.420000_469.571751_0.000000_0.100000': [285.492003910436, 0, 2533.1518839324413, 0, 0, 0, 0, 0], '0.000826_137.500000_2.000000_687.500000_0.000000_0.100000': [0, 0, -0.5108399885532522, 0, 0, 112.94449456704297, 0, 0], '0.000826_166.375000_1.818182_687.500000_0.000000_0.100000': [0, 0, 0, -1049.6303947247293, 0, 0, 0, 0], '0.000826_166.375000_2.000000_687.500000_0.000000_0.100000': [421.0607816098027, 0, 0, -928.3880817576027, 0, 0, 0, 0], '0.000826_151.250000_2.000000_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -421.0607816098027, 0], '0.000826_166.375000_2.200000_687.500000_0.000000_0.100000': [0, 0, 0, 0, 1.6295312472015213, 0, 0, 0], '0.000751_166.375000_2.200000_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 931.3274802859693, 0, 0], '0.000751_166.375000_2.000000_687.500000_0.000000_0.100000': [0, 0, 0, -931.3274802859693, 4.154460922576436, 0, 0, 0], '0.000683_166.375000_2.000000_687.500000_0.000000_0.100000': [0, 0, -4.154460922576436, 0, 0, 0, 0, 0], '0.001100_85.376682_1.818182_625.000000_0.000000_0.100000': [-252.9395099158444, 3699.679494133886, -5.524840776605925, 0, 0, 0, -452.8456346139771, 1494.430539902692], '0.000826_85.376682_1.818182_568.181818_0.000000_0.100000': [-2944.1479090414537, -2513.916610323034, -3707.971016581849, -692.5552489965921, -0.6626690432260326, 0, 0, 0], '0.000751_77.615165_1.818182_568.181818_0.000000_0.100000': [4917.534424860047, 0, 0, 0, 0, 457.53907851730554, 0, -491.0178374695701], '0.000751_77.615165_2.000000_625.000000_0.000000_0.100000': [888.9994581592941, 0, 0, 0, 0, 0, 0, 0], '0.000751_70.559241_2.000000_625.000000_0.000000_0.100000': [4373.785834256402, 0, 0, 0, 0, 0, 0, 0], '0.000751_64.144765_2.000000_625.000000_0.000000_0.100000': [0.06491081898202537, 0, 0, 0, -0.6982900657898199, 0, 0, 0], '0.000683_64.144765_2.000000_625.000000_0.000000_0.100000': [0.6792154892710255, 0, 0, 0, 0, 0, 0, 0], '0.000683_58.313423_2.000000_625.000000_0.000000_0.100000': [0, 0, 0.8816801618778982, 0, 0, 0, 0, 0], '0.000751_58.313423_2.000000_625.000000_0.000000_0.100000': [-0.7920500023376008, -0.7787863883903355, 0, -0.7468220869131755, 0, 0, 0, 0], '0.000751_53.012202_2.000000_625.000000_0.000000_0.100000': [0, -0.16786130497262094, 0, 0, 0, 0, 0, 0], '0.000751_53.012202_2.000000_568.181818_0.000000_0.100000': [0, 0.24698174947593543, 0, 0, 0, 0.17850111956088313, 0, 0], '0.000751_53.012202_2.000000_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -0.24698174947593543], '0.000751_53.012202_1.818182_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001100_113.636364_1.652893_426.883410_0.000000_0.100000': [277.60464706632496, 405.61309558642233, 0, 0, 1035.552540802135, 0, 0, 116.17194521324308], '0.001210_113.636364_1.652893_469.571751_0.000000_0.100000': [-11.163724621965706, 0, -168.19083582657203, 0, 0, -24.44995500842465, -298.55259967524364, -190.31007585273554], '0.001464_113.636364_1.502630_516.528926_0.000000_0.100000': [0, 513.0575749095409, 0, 95.70317543568117, 0, 0, 0, 0], '0.001464_113.636364_1.502630_469.571751_0.000000_0.100000': [0, 503.07357475358197, 0, 0, 0, 0, 0, 0], '0.001464_113.636364_1.366027_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -439.9406821913635], '0.001464_113.636364_1.366027_469.571751_0.000000_0.100000': [0, 0, 0, 0, 6.990852784647723, 0, 0, 0], '0.001331_113.636364_1.366027_469.571751_0.000000_0.100000': [0, 0, 0, -860.851532287386, 0, 0, 0, 0], '0.001331_113.636364_1.502630_469.571751_0.000000_0.100000': [0, 502.9314766319042, 0, 0, 0, 0, 0, 0], '0.001331_113.636364_1.502630_426.883410_0.000000_0.100000': [0, 0, -6.277315307484787, 0, 0, 0, 0, 0], '0.001000_85.376682_2.200000_388.075827_0.000000_0.100000': [377.85326403049623, 0, 0, 0, 0, 0, 0, 0], '0.001000_77.615165_2.200000_388.075827_0.000000_0.100000': [0, 0, 0, -1248.6257574511046, 0, 1782.277742509999, 0, 0], '0.001000_77.615165_2.420000_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -488.0727696134558], '0.001000_77.615165_2.420000_426.883410_0.000000_0.100000': [0, 0, 0, -355.66433211837705, 0, 0, 0, 0], '0.001000_77.615165_2.662000_426.883410_0.000000_0.100000': [2090.929866717, 0, 0, 0, 0, 0, 0, 0], '0.001000_70.559241_2.662000_426.883410_0.000000_0.100000': [0, 0, 4475.954147083077, 0, 0, 0, 0, 0], '0.001100_70.559241_2.662000_426.883410_0.000000_0.100000': [0, 0, -2964.638584713935, 0, 0, 0, 0, 0], '0.001210_70.559241_2.662000_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -3608.7646814951668, 0], '0.001210_77.615165_2.662000_426.883410_0.000000_0.100000': [0, 0, 0, 1359.387510279107, 0, 0, 0, 0], '0.001210_77.615165_2.928200_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -428.05439953426907], '0.001210_77.615165_2.928200_469.571751_0.000000_0.100000': [0, 0, 0, -1830.2902035744792, 0, 0, 0, 0], '0.001210_77.615165_3.221020_469.571751_0.000000_0.100000': [0, 0, 0, 0, 1.162708712597123, 0, 0, 0], '0.001100_77.615165_3.221020_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -184.01490386392908, 0], '0.001100_85.376682_3.221020_469.571751_0.000000_0.100000': [184.01490386392908, 0, 0, 0, 0, 0, 0, 0], '0.001331_113.636364_2.420000_625.000000_0.000000_0.100000': [0, 544.124755711174, 0, 0, 0, 0, 0, 0], '0.001331_113.636364_2.420000_568.181818_0.000000_0.100000': [0, 0, 0, 0, 3.959890966120511, 0, 0, 0], '0.001210_103.305785_2.420000_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -518.289391911645], '0.001100_103.305785_2.420000_625.000000_0.000000_0.100000': [0, 0, 173.6968448090313, 0, 5.356391357301946, 0, 0, 0], '0.001210_125.000000_2.200000_756.250000_0.000000_0.100000': [0, 0, -4.508205940151129, 0, 0, 0, 0, 0], '0.001331_125.000000_2.200000_756.250000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -288.83122235124483, 0], '0.001331_137.500000_2.200000_756.250000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001210_103.305785_1.502630_687.500000_0.000000_0.100000': [0, 0, 0, 230.7473772338297, 0, 0, 0, -446.97738655628876], '0.001331_93.914350_1.652893_756.250000_0.000000_0.100000': [0, 0, 0, 0, 92.03366029528055, 0, 0, 0], '0.001100_137.500000_1.652893_687.500000_0.000000_0.100000': [2021.1993060107516, 0, 281.78649051550445, -2.660970852666401, 0, 0, -316.57722401367937, -214.7319377303593], '0.001210_166.375000_1.652893_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 128.2764634394185, 0, 0], '0.001210_166.375000_1.502630_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 214.10541962499937, 0, 0], '0.001210_166.375000_1.366027_687.500000_0.000000_0.100000': [0, 469.4199974216585, 0, 0, 0, 0, 0, 0], '0.001210_166.375000_1.366027_625.000000_0.000000_0.100000': [0, 0, -5.991530918738682, 0, 0, 0, 0, 0], '0.001331_166.375000_1.366027_625.000000_0.000000_0.100000': [413.798229170941, 0, 0, 0, 0, 0, 0, 0], '0.001331_151.250000_1.366027_625.000000_0.000000_0.100000': [0, 0, 0, -403.4385048690339, 0, 0, 0, 0], '0.001331_151.250000_1.502630_625.000000_0.000000_0.100000': [276.71213599438164, 0, -1.081284369520654, 0, 0, 0, 0, 0], '0.001464_151.250000_1.502630_625.000000_0.000000_0.100000': [0, 0, 0, 0, 1.081284369520654, 0, 0, 0], '0.001331_137.500000_1.652893_625.000000_0.000000_0.100000': [308.9954174075792, 0, 0, 0, 0, 0, 0, -478.7181138838807], '0.001331_137.500000_1.652893_687.500000_0.000000_0.100000': [0, 0, 0, 0, 4.355813303928153, 0, 0, 0], '0.001210_137.500000_1.652893_625.000000_0.000000_0.100000': [309.3179696686082, 969.4158415531833, 0, 0, -426.1462212433407, -46.849862352141464, -325.1443916611617, 0], '0.001331_125.000000_1.818182_625.000000_0.000000_0.100000': [0, 0, -5.019647624717618, -1466.0953120963472, 1080.2495838600944, 810.7906467066325, -346.6876220488763, 0], '0.001464_125.000000_1.652893_625.000000_0.000000_0.100000': [285.48224484778933, 0, -5.684966302369685, 0, 0, 0, 0, 0], '0.001611_125.000000_1.652893_625.000000_0.000000_0.100000': [286.4287519996904, 0, 0, 0, 0, -43.58828233430577, 0, 0], '0.001611_125.000000_1.502630_625.000000_0.000000_0.100000': [0, 0, -2.0948345234122825, 0, 0, 0, 0, 0], '0.001772_125.000000_1.502630_625.000000_0.000000_0.100000': [0, 0, 0, 39.94847765563664, 0, 0, 0, 0], '0.001772_113.636364_1.652893_625.000000_0.000000_0.100000': [0, 525.0172295138212, 0, 0, 0, -16.186248241494106, 0, 0], '0.001772_113.636364_1.652893_568.181818_0.000000_0.100000': [258.9023778078537, 0, 0, 0, 3.7699321488138593, 0, 0, 0], '0.001772_103.305785_1.652893_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -442.6912256478103], '0.001772_103.305785_1.652893_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 13.114183075410073, -293.49954262715363, 0], '0.001772_103.305785_1.502630_625.000000_0.000000_0.100000': [0, 0, -5.124338015479589, -13.114183075410073, 0, 0, 0, 0], '0.001949_103.305785_1.502630_625.000000_0.000000_0.100000': [0, 0, 0, 0, 5.124338015479589, 0, 0, 0], '0.001772_113.636364_1.502630_625.000000_0.000000_0.100000': [0, 408.071450680379, 0, 0, 0, 0, 0, 0], '0.001772_113.636364_1.502630_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 926.67720992785, 0, 0], '0.001772_113.636364_1.366027_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000909_93.914350_2.200000_388.075827_0.000000_0.100000': [2422.886953763372, 0, 0, -1077.0328000131694, 0, 0, 0, 0], '0.000909_93.914350_2.420000_388.075827_0.000000_0.100000': [294.66377932590257, 0, -2.7361664542644575, 0, 0, 0, 0, 0], '0.001000_93.914350_2.420000_388.075827_0.000000_0.100000': [0, 0, -3.023620748690155, 0, 0, 0, 0, 0], '0.000826_137.500000_2.000000_756.250000_0.000000_0.100000': [0, 0, 0, 0, 0, 1012.332489576305, 0, 0], '0.000683_137.500000_1.818182_756.250000_0.000000_0.100000': [0, 0, 285.73319938357884, 0, 2.8874712284139608, 0, 0, 0], '0.000621_137.500000_1.818182_756.250000_0.000000_0.100000': [0, 0, 0, 0, 2.621499051973842, 0, 0, 0], '0.000564_137.500000_1.818182_756.250000_0.000000_0.100000': [0, 0, 0, 0, 0, 266.2700769421126, 0, 0], '0.000564_137.500000_1.652893_756.250000_0.000000_0.100000': [0, 0, 0, 0, 0, 139.06997981900113, 0, 0], '0.000564_137.500000_1.502630_756.250000_0.000000_0.100000': [0, 0, 0, 0, 0.7183305287757321, 0, 0, 0], '0.000513_137.500000_1.502630_756.250000_0.000000_0.100000': [0, 0, 0, 0, 0.6465884898905643, 0, 0, 0], '0.000467_137.500000_1.502630_756.250000_0.000000_0.100000': [278.7344061559747, 0, 0, 0, 0, 0, 0, 0], '0.000467_125.000000_1.502630_756.250000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -415.42995652374157], '0.000467_125.000000_1.502630_831.875000_0.000000_0.100000': [256.06443407685623, 0, 0, 0, 0, 0, 0, 0], '0.000467_113.636364_1.502630_831.875000_0.000000_0.100000': [0, 0, 0, 0, 0.735393376355205, 0, 0, 0], '0.000424_113.636364_1.502630_831.875000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 1170.677299683604], '0.000424_113.636364_1.502630_915.062500_0.000000_0.100000': [-1383.874413111652, 0, 0, 0, 0, 0, -1865.1745292956132, 0], '0.000424_103.305785_1.502630_915.062500_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 1383.874413111652, 0], '0.000424_125.000000_1.502630_915.062500_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -272.23209285093634, 0], '0.000424_137.500000_1.502630_915.062500_0.000000_0.100000': [0, 0, -1.1161356735314258, 0, 0, 0, 0, 0], '0.000467_137.500000_1.502630_915.062500_0.000000_0.100000': [0, 436.45121467077405, 0, 0, 0, 0, 0, 0], '0.000467_137.500000_1.502630_831.875000_0.000000_0.100000': [0, 0, 0, -210.85155867359555, 0, 0, 0, 0], '0.000467_137.500000_1.652893_831.875000_0.000000_0.100000': [0, 0, 0, -297.8490558313948, 0, 0, 0, 0], '0.000467_137.500000_1.818182_831.875000_0.000000_0.100000': [0, 0, 0, -1033.079372131408, 0, 0, 0, 0], '0.000467_137.500000_2.000000_831.875000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000909_85.376682_2.000000_625.000000_0.000000_0.100000': [1317.3935077932047, 1715.3578613942182, 0, 0, 4.953564787314463, 0, 0, 0], '0.000909_77.615165_2.000000_625.000000_0.000000_0.100000': [891.7598009885152, 0, 0, -418.5890537401096, 0, 0, 0, 0], '0.000826_77.615165_2.000000_568.181818_0.000000_0.100000': [5405.084439448007, 0, 0, 0, 4.249677929609425, 0, 527.8557283750001, 0], '0.000826_70.559241_2.000000_568.181818_0.000000_0.100000': [0, -1799.385497767336, 0, 3023.8603063999994, 0, 0, 0, 0], '0.000826_70.559241_2.200000_516.528926_0.000000_0.100000': [5851.987447093887, 607.479695475958, 0, 0, 0, 0, -29.52762465913629, 0], '0.000826_64.144765_2.200000_516.528926_0.000000_0.100000': [0, 0, 0.02321959851472002, 0, 0, 0, 0, 0], '0.000909_64.144765_2.200000_516.528926_0.000000_0.100000': [0.033332752736669136, 0, 0, 0, 0, 0, 0, 0], '0.000909_58.313423_2.200000_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000909_93.914350_1.652893_687.500000_0.000000_0.100000': [1415.1962525443712, 0, 0, 0, 0, 0, 0, 0], '0.000909_77.615165_1.652893_756.250000_0.000000_0.100000': [0, 0, -2.6868386347789057, 0, 0, 0, 0, 0], '0.001000_77.615165_1.652893_756.250000_0.000000_0.100000': [0, 0, 0, 2915.674310821638, 0, 0, 0, -541.9830038724962], '0.001000_77.615165_1.818182_756.250000_0.000000_0.100000': [-2011.8931047814356, 443.0748473677582, -2.247198187111617, 0, 0, -265.0613009837854, 0, 0], '0.001000_70.559241_1.818182_756.250000_0.000000_0.100000': [0, 0, -5.231904305581338, 0, 0, 0, 0, 0], '0.001100_70.559241_1.818182_756.250000_0.000000_0.100000': [6481.483429365043, 0, 0, 0, 0, 0, 0, 0], '0.001100_64.144765_1.818182_756.250000_0.000000_0.100000': [0, 0.7731195402881053, 0.825453103081702, 0, 0, 0, 0, 0], '0.001100_64.144765_1.818182_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0.7436778538765998, 0, 0, 0], '0.000909_77.615165_1.652893_568.181818_0.000000_0.100000': [4489.557097722585, 0, 0, 0, 0, -675.9677509812018, -1200.5404745398953, 0], '0.000909_70.559241_1.652893_568.181818_0.000000_0.100000': [0, 0, 0.42732186974124037, 0, 31.873316994379266, 2.002290863513844, 0, 0], '0.000909_70.559241_1.502630_568.181818_0.000000_0.100000': [0, -173.56610517085323, 1.1741065550904273, 0, 0, 0, 0, 0], '0.001000_70.559241_1.502630_568.181818_0.000000_0.100000': [27.158366022046987, 0, 0, -64.52432604071475, 0, 0, 0, 0], '0.001000_70.559241_1.652893_568.181818_0.000000_0.100000': [0, 0, 70.57564717376727, 0, 62.90311529233181, 0, -3957.9556654075727, 0], '0.001100_70.559241_1.652893_568.181818_0.000000_0.100000': [0, -337.1037225565379, 0.4895949586248207, 0, 0, 0, 0, 0], '0.000826_103.305785_2.000000_687.500000_0.000000_0.100000': [-1080.7348785038298, -2162.500595721981, -1074.9921194506644, -2179.8739674161466, -358.53931287416464, -614.958813241481, 0, 0], '0.000909_85.376682_2.000000_568.181818_0.000000_0.100000': [0, 0, -5.913017668378453, 0, 2.4286029079406943, 0, 0, 0], '0.000826_85.376682_2.000000_568.181818_0.000000_0.100000': [-527.8557283750001, 1034.5266948174874, 0, -450.4583300719928, 0, 3816.1074885067933, -1285.6420005921325, 0], '0.000826_85.376682_2.200000_568.181818_0.000000_0.100000': [0, -1046.4861682490841, 0, 0, 0, 264.4401959975113, 0, 0], '0.000826_85.376682_2.200000_516.528926_0.000000_0.100000': [0, 0, -5.3260964321711075, 0, 0, 2079.357254981592, 0, 0], '0.000909_85.376682_2.200000_516.528926_0.000000_0.100000': [0, 0, -5.857702992007581, 0, 0, 0, 0, 0], '0.001000_85.376682_2.200000_516.528926_0.000000_0.100000': [1567.3566533081566, 0, 0, 0, 0, 0, 0, 0], '0.000909_77.615165_2.200000_568.181818_0.000000_0.100000': [5245.7295231979215, 0, 0, 0, 0, 0, 0, 0], '0.000751_151.250000_2.200000_687.500000_0.000000_0.100000': [0, 0, 0, 0, 2.2700014873291368, 0, 0, 0], '0.000683_151.250000_2.200000_687.500000_0.000000_0.100000': [0, 0, 0, -273.83268170425566, 0, 0, 0, 0], '0.000683_151.250000_2.420000_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000909_151.250000_1.502630_756.250000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -311.8246679790871, -411.34352928596763], '0.000909_151.250000_1.502630_831.875000_0.000000_0.100000': [0, 411.34352928596763, 0, 0, 0, 0, 0, 0], '0.000909_166.375000_1.502630_756.250000_0.000000_0.100000': [0, 0, 0, -167.97297031969285, 0, 0, 0, 0], '0.000909_166.375000_1.652893_756.250000_0.000000_0.100000': [0, 0, 0, -343.7234370118467, 0, 0, 0, 0], '0.000909_166.375000_2.000000_756.250000_0.000000_0.100000': [0, 861.4281253089189, 0, 0, 0, 0, 0, 0], '0.001210_93.914350_2.420000_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 2085.900624244834, 0, 0], '0.001000_93.914350_2.000000_831.875000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000909_125.000000_1.502630_756.250000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -278.4449167901443, 0], '0.000909_137.500000_1.366027_687.500000_0.000000_0.100000': [0, 1167.9559511309749, 0, 0, 0, 0, 0, 0], '0.000909_137.500000_1.366027_625.000000_0.000000_0.100000': [470.8245674912636, 0, 0, 0, 0, 839.0689493271123, 0, 0], '0.000909_137.500000_1.241843_625.000000_0.000000_0.100000': [0, 0, -3.1085562837335603, 0, 2.811379922027663, 0, 0, -417.6703645551221], '0.001000_137.500000_1.241843_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -418.017490447409], '0.001000_137.500000_1.241843_687.500000_0.000000_0.100000': [348.4617896022684, 0, 0, 0, 0, 0, 0, 0], '0.001000_125.000000_1.241843_687.500000_0.000000_0.100000': [294.23900547115267, 0, 0, 0, 0, 0, 0, 0], '0.001000_113.636364_1.241843_687.500000_0.000000_0.100000': [235.8658739627933, 0, -2.329476308377798, 0, 0, 0, 0, 0], '0.001100_113.636364_1.241843_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -295.04111535812746, 0], '0.001100_125.000000_1.241843_687.500000_0.000000_0.100000': [0, 0, -3.469403947457431, 0, 0, 0, 0, -441.92959745344854], '0.001100_125.000000_1.241843_756.250000_0.000000_0.100000': [0, 0, 0, -730.3366401743768, 0, 0, 0, 0], '0.001100_125.000000_1.366027_756.250000_0.000000_0.100000': [0, 0, -6.155475458523688, 0, 0, 0, -415.15410765229717, 0], '0.001100_137.500000_1.366027_756.250000_0.000000_0.100000': [0, 0, 0, -472.472885379019, 0, 0, 0, 0], '0.001210_125.000000_1.652893_625.000000_0.000000_0.100000': [212.76788352490848, 294.256309980162, -4.205624547411389, -538.4062202703009, 58.57432919486911, -51.13989403166543, 0, -99.79074648663044], '0.001210_125.000000_1.502630_568.181818_0.000000_0.100000': [315.71961555642247, 0, 0, 0, 0, 0, 0, 0], '0.001210_113.636364_1.502630_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 1917.5648145107355, 0, 0], '0.001210_103.305785_1.366027_568.181818_0.000000_0.100000': [0, 903.9294312931461, 0, 0, 6.354918843408796, 233.93736739810285, 0, 0], '0.001100_103.305785_1.241843_568.181818_0.000000_0.100000': [0, 440.5809583045815, 0, 0, 1.7917533621343864, 140.3020401917921, 0, -438.0145444768423], '0.001100_103.305785_1.241843_625.000000_0.000000_0.100000': [189.05515078765913, 287.5339968406299, -1.2413705179873433, -308.02012755443815, 0, -247.71030010448334, 0, -449.91532780602876], '0.001100_103.305785_1.241843_687.500000_0.000000_0.100000': [195.39436414063252, 0, 0, 0, 0, -290.5967526211886, 0, -428.2357912206807], '0.001100_103.305785_1.128948_687.500000_0.000000_0.100000': [0, 450.9249967867807, 0, 0, 0, 0, 0, 0], '0.001100_103.305785_1.128948_625.000000_0.000000_0.100000': [0, 0, 0, 0, 1.1921434543842406, 0, 0, 0], '0.001000_103.305785_1.128948_625.000000_0.000000_0.100000': [136.74107988675243, 0, 0, 0, 0, -520.7601798378286, 0, -451.09852610141843], '0.001000_103.305785_1.026316_625.000000_0.000000_0.100000': [0, 0, 0, 520.7601798378286, 0, 0, 0, 0], '0.001000_103.305785_1.128948_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, -603.8678159922347, 0, 0], '0.001000_103.305785_1.026316_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 512.8039841696184, 0, 0], '0.001000_103.305785_0.933015_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000683_85.376682_1.652893_625.000000_0.000000_0.100000': [0, 0, 0, 736.6123776986115, 0, 170.1479856200028, 0, 0], '0.000683_85.376682_1.502630_625.000000_0.000000_0.100000': [0, 0, 0, 0, 3.217982618600372, 0, 0, 0], '0.000621_85.376682_1.502630_625.000000_0.000000_0.100000': [307.2181984117193, 0, -0.2925438744182155, 0, 0, 0, 0, 0], '0.000621_77.615165_1.502630_625.000000_0.000000_0.100000': [0, 506.13173745063335, 0, 0, 0, 0, 0, 0], '0.000621_77.615165_1.502630_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001000_85.376682_1.652893_516.528926_0.000000_0.100000': [-980.5860958471395, -596.1872809459635, -1112.8137890491528, -221.9535658184509, -346.217293696737, -844.6094612560245, -1202.5817266380564, -1580.1290531464479], '0.001000_85.376682_1.502630_516.528926_0.000000_0.100000': [3104.658299593507, 0, 0, 0, 0, 0, -357.9557461372274, 0], '0.001000_77.615165_1.502630_516.528926_0.000000_0.100000': [2631.712191878545, 0, 0, 0, 0, 0, -333.4946382804217, 0], '0.001000_70.559241_1.502630_516.528926_0.000000_0.100000': [0, 0, 0, 0, 174.10069946647263, 1845.4989712677893, 0, 0], '0.000909_70.559241_1.502630_516.528926_0.000000_0.100000': [2211.652250348561, 0, -0.01291536371675459, -1254.0986305094564, 1918.5089173060135, 0, 0, 0], '0.000826_70.559241_1.502630_516.528926_0.000000_0.100000': [92.06548175419262, 0, 0, 0, 0, 0, 0, 0], '0.000826_64.144765_1.502630_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0.7605969413894377, 0, 0, 0], '0.000751_64.144765_1.502630_516.528926_0.000000_0.100000': [-0.6338548910409145, -0.7055124337092366, 0, 0, 0, 0, -85.48538633306087, 0], '0.000751_64.144765_1.502630_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0.7055124337092366], '0.000751_70.559241_1.502630_516.528926_0.000000_0.100000': [0, 0, 0, -709.2574191441281, 0, 0, 0, 0], '0.000751_70.559241_1.652893_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 3612.337542241665], '0.000751_70.559241_1.652893_568.181818_0.000000_0.100000': [162.8983990144421, 0, 0, 94.6670957719604, 0, 175.23616253932906, 0, 0], '0.000751_70.559241_1.502630_568.181818_0.000000_0.100000': [30.502822619622012, -57.282528339945294, 0, 0, 0, -2111.683614345331, 0, 0], '0.000751_70.559241_1.366027_568.181818_0.000000_0.100000': [0, 2025.067438254862, 0, 0, 0, 0, 0, 0], '0.000751_70.559241_1.366027_516.528926_0.000000_0.100000': [0, 0, 0, -2845.584712783256, 0, 0, -2889.2738753466388, 0], '0.000751_70.559241_1.818182_568.181818_0.000000_0.100000': [68.28997287783135, 0, 0, -2814.900585027745, -2727.6693112129415, 0, 0, 45.79021614703832], '0.000683_70.559241_1.818182_568.181818_0.000000_0.100000': [3076.1296965864326, 0, 0, 0, 0, 0, 0, 0], '0.001210_93.914350_1.818182_568.181818_0.000000_0.100000': [0, 544.6342432316478, 0, 0, -54.13353287248901, 280.31192110249873, 0, 0], '0.001210_93.914350_1.818182_516.528926_0.000000_0.100000': [1175.0041393322126, 471.7816836919055, 0, -471.72519552302776, 1674.136941783027, 0, 0, 0], '0.001210_93.914350_1.818182_469.571751_0.000000_0.100000': [601.7906033745944, 0, -4.6726884122092995, 0, 0, 0, 0, 3234.2220014595546], '0.001331_85.376682_1.818182_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -278.23588559383217, 0], '0.001331_93.914350_1.818182_426.883410_0.000000_0.100000': [0, 0, -4.369426364724859, -327.03668219192895, 7.1306119209808, 492.4532442938343, 0, 0], '0.001210_85.376682_2.200000_426.883410_0.000000_0.100000': [0, 0, 0, -1138.334194254618, 0, 0, 0, 0], '0.001210_85.376682_2.420000_426.883410_0.000000_0.100000': [308.40636356967224, 0, 0, 0, 0, 0, -309.05960119653645, 0], '0.001210_93.914350_2.420000_426.883410_0.000000_0.100000': [309.05960119653645, 0, -2.9471343448834157, 0, 0, 0, -293.098965277722, 0], '0.001210_77.615165_2.420000_426.883410_0.000000_0.100000': [296.1181721365396, 0, 0, 0, 0, 0, 0, 0], '0.001210_70.559241_2.420000_426.883410_0.000000_0.100000': [5941.59390076637, 0, 0, 0, 0, 0, 0, 0], '0.001210_64.144765_2.420000_426.883410_0.000000_0.100000': [0, 0, 0.01997114770309054, 0, 0, 0, 0, 0], '0.001331_64.144765_2.420000_426.883410_0.000000_0.100000': [0, 0, 0, 0, -0.0018155588820991404, 0, 0, 0], '0.001331_113.636364_1.502630_568.181818_0.000000_0.100000': [0, 458.07119139209556, 0, 85.93615573123262, 0, 0, 0, 0], '0.001331_113.636364_1.502630_516.528926_0.000000_0.100000': [0, 0, 0, 325.3787185334941, 0, 0, 0, 0], '0.001331_103.305785_1.818182_687.500000_0.000000_0.100000': [1214.7923377180753, 0, -4.879560873701975, 666.9570864079932, 0, 0, -1092.7124284855886, 0], '0.001331_125.000000_2.000000_756.250000_0.000000_0.100000': [342.48820667538115, 0, -8.155947337871476, 0, 0, 0, 0, 0], '0.001464_125.000000_2.000000_756.250000_0.000000_0.100000': [0, 0, 0, -1095.8031460220861, 0, 0, 0, -571.9652318594981], '0.001464_125.000000_2.000000_831.875000_0.000000_0.100000': [0, 571.9652318594981, 0, 0, 0, 0, 0, 0], '0.001611_125.000000_2.420000_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000909_113.636364_2.200000_516.528926_0.000000_0.100000': [0, 508.2385899098712, 0, -739.0287374341259, 1160.2991829514117, 3760.5956304716315, -451.3669704960471, 0], '0.001000_103.305785_2.200000_352.796206_0.000000_0.100000': [0, 969.5134894596283, -5.653444003258573, 0, 0, 0, -403.81219106705015, 0], '0.001000_103.305785_2.200000_320.723824_0.000000_0.100000': [1517.7143456501317, 0, -5.643004147549027, 0, 0, 0, 0, 0], '0.001100_103.305785_2.200000_320.723824_0.000000_0.100000': [0, 0, 0, 0, 0, 1030.4109118119547, 0, -479.2847096219045], '0.001100_103.305785_2.000000_320.723824_0.000000_0.100000': [0, 0, 0, 0, 0, 1331.6885254858635, 0, 0], '0.001100_103.305785_1.818182_320.723824_0.000000_0.100000': [0, 1688.939654098548, 0, -32.0751103584112, 0, 0, 0, 0], '0.001100_103.305785_1.818182_291.567113_0.000000_0.100000': [1580.6663342626964, 0, 0, -294.04814266358807, 0, 0, 0, 0], '0.001100_93.914350_1.818182_291.567113_0.000000_0.100000': [0, 1297.6492207408985, 0, -168.714145249716, 0, 0, -285.75824275771157, 0], '0.001100_93.914350_1.818182_265.061011_0.000000_0.100000': [0, 0, 0, 0, 0, 1262.7682633054799, 0, 0], '0.001100_93.914350_1.652893_265.061011_0.000000_0.100000': [0, 0, 0, 0, 0, 1320.1057973855911, 0, 0], '0.001100_93.914350_1.502630_265.061011_0.000000_0.100000': [0, 882.3070218519522, 0, 0, 0, 0, 0, -307.05956759625093], '0.001100_93.914350_1.502630_240.964556_0.000000_0.100000': [670.2161566890601, 0, 0, 0, 0, 0, 0, -250.4691402572557], '0.001100_93.914350_1.502630_291.567113_0.000000_0.100000': [0, 0, 0, 0, 0, 422.7137276912202, 0, 0], '0.001100_93.914350_1.366027_291.567113_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000683_113.636364_1.818182_756.250000_0.000000_0.100000': [270.66333307036604, 0, 0, 0, 0, 0, 0, 0], '0.000683_103.305785_1.818182_831.875000_0.000000_0.100000': [239.32002301277498, 0, -2.824306696661188, 0, 0, 0, 0, 0], '0.000751_103.305785_1.818182_915.062500_0.000000_0.100000': [0, 0, -3.3647623930930424, 0, 0, 0, 0, 0], '0.000826_103.305785_1.818182_915.062500_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -253.33528095680546, 0], '0.001100_113.636364_1.652893_831.875000_0.000000_0.100000': [264.3741986444802, 0, 0, 0, 0, 459.2106132017645, 0, 0], '0.001100_103.305785_1.652893_831.875000_0.000000_0.100000': [0, 0, 0, 0, 0, 157.76108491109463, -264.3741986444802, 0], '0.001100_103.305785_1.502630_831.875000_0.000000_0.100000': [274.4152081936427, 538.9685350853158, 0, 0, 0, 0, -256.9099354683911, 0], '0.001100_93.914350_1.502630_831.875000_0.000000_0.100000': [240.46216200195087, 0, 0, 0, 0, 0, -24.94683710851296, 0], '0.001331_93.914350_2.000000_469.571751_0.000000_0.100000': [0, 0, 0, 0, 6.798324515926561, 1267.7525159860961, 0, 0], '0.001331_93.914350_1.652893_469.571751_0.000000_0.100000': [236.5461059791188, 1184.9580726858485, -2.2112619196595005, 332.4776333026089, 0, 0, 0, 0], '0.001464_77.615165_1.818182_469.571751_0.000000_0.100000': [0, 514.497563693943, 0, 0, 0, 0, 0, 0], '0.001464_77.615165_1.818182_426.883410_0.000000_0.100000': [0, 0, 0, 0, 4.258774262408833, 0, 0, 0], '0.001331_77.615165_1.818182_426.883410_0.000000_0.100000': [1922.6046449385756, 0, 0, 0, 0, -55.435882131749985, 0, 0], '0.001331_77.615165_1.652893_426.883410_0.000000_0.100000': [0, 407.583738295553, 0, 0, 0, 0, 0, 0], '0.001331_77.615165_1.652893_388.075827_0.000000_0.100000': [1007.9343741236071, 0, 0, 0, 0, 0, 0, 0], '0.001331_70.559241_1.652893_388.075827_0.000000_0.100000': [0, 1407.6782713965442, 0, 0, 0, 0, 0, 0], '0.001331_70.559241_1.652893_352.796206_0.000000_0.100000': [0, 0, -22.367775882051728, 0, 0, 0, 0, 0], '0.001464_70.559241_1.652893_352.796206_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -1389.9351807252062], '0.001464_70.559241_1.652893_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 2282.263086120799, 0, 0], '0.001464_70.559241_1.502630_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001100_137.500000_2.420000_625.000000_0.000000_0.100000': [0, 477.5580306924232, 0, 0, 0, 0, 0, -475.669265883305], '0.001100_137.500000_2.420000_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 505.7039096907711, 0, 0], '0.001000_125.000000_1.818182_831.875000_0.000000_0.100000': [0, 1426.5644351583096, 0, 0, 0, 0, 0, 0], '0.001100_93.914350_2.200000_687.500000_0.000000_0.100000': [0, 0, 256.34325033635577, 0, 0, 0, 0, 0], '0.001331_93.914350_2.000000_625.000000_0.000000_0.100000': [0, 0, 0, 0, 7.378477376498814, 0, -344.2572748448994, 0], '0.000909_125.000000_1.652893_516.528926_0.000000_0.100000': [321.0344266880802, 439.4020698530241, 1277.8745886575903, 1445.4862683801011, 0, 1888.34464815177, -347.57962652427267, 0], '0.000909_113.636364_1.652893_516.528926_0.000000_0.100000': [213.00149252379856, 1845.114318225228, 2912.1394666435444, 0, 0, 0, 91.57720432485218, 0], '0.000909_103.305785_1.652893_516.528926_0.000000_0.100000': [0, 1902.3981845776457, 441.0880455606764, -309.7515021855547, 2.1190839669295203, 26.224316676211583, -276.5896185666645, 0], '0.001000_103.305785_1.818182_469.571751_0.000000_0.100000': [361.77930316045035, 1744.5039078765553, 0, -865.0585613445716, 1321.513216812099, 0, -81.26240508522918, 41.353633359281886], '0.000909_113.636364_2.000000_426.883410_0.000000_0.100000': [779.6328352811121, 0, 0, -1248.5481260264944, 0, 1246.553844881031, 0, -487.0062704944663], '0.001331_125.000000_2.000000_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -472.6195584014613], '0.001464_125.000000_2.000000_516.528926_0.000000_0.100000': [0, 496.1433009549661, 0, 0, 8.292052731200966, 0, 0, 0], '0.001464_125.000000_2.000000_469.571751_0.000000_0.100000': [0, 472.73969178521895, 0, 0, 0, 0, 0, 0], '0.001464_125.000000_2.000000_426.883410_0.000000_0.100000': [464.8771742169961, 0, 0, 0, 0, 0, 0, 0], '0.001464_113.636364_2.000000_426.883410_0.000000_0.100000': [0, 0, 0, 0, 7.843171146743134, 0, 0, 0], '0.001331_103.305785_2.200000_516.528926_0.000000_0.100000': [382.86660283749956, 0, 0, 0, 0, 0, -404.93323966796305, 0], '0.001331_113.636364_2.200000_516.528926_0.000000_0.100000': [0, 508.41317539868777, 0, 0, 0, 0, 0, 0], '0.001331_113.636364_2.200000_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001210_103.305785_1.502630_568.181818_0.000000_0.100000': [0, 0, -4.601686359471933, 0, 0, 0, 0, 0], '0.001331_103.305785_1.502630_568.181818_0.000000_0.100000': [541.5332862513096, 0, -5.069476883460084, 0, 0, 0, 0, -440.1051508714625], '0.001464_103.305785_1.502630_568.181818_0.000000_0.100000': [0, 466.2670057000535, 0, 0, 5.069476883460084, 997.9970978252288, 0, 0], '0.001331_93.914350_1.502630_568.181818_0.000000_0.100000': [346.98985081286963, 1509.371691494509, 0, -109.0122600687183, 10.92916539671023, 0, 0, 0], '0.001331_85.376682_1.502630_568.181818_0.000000_0.100000': [0, 0, 0, 0, 5.444140346064978, 0, 0, 0], '0.001210_85.376682_1.502630_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000909_103.305785_1.502630_516.528926_0.000000_0.100000': [2945.955202627502, 0, 0, 0, 0, 0, 0, -471.5288866165287], '0.000909_93.914350_1.652893_516.528926_0.000000_0.100000': [0, 457.091136276601, 0, 0, 1029.2122514556856, 0, 0, 0], '0.000909_93.914350_1.652893_469.571751_0.000000_0.100000': [213.8799473339209, 466.56493286181257, -1.3790373302954322, 0, 0, 195.31181988751268, 0, 0], '0.001000_93.914350_1.652893_469.571751_0.000000_0.100000': [526.1794463737502, 0, 0, 0, 1.3790373302954322, 180.56834536604475, 148.41311769006953, -293.68775122205955], '0.001100_103.305785_1.652893_426.883410_0.000000_0.100000': [0, 367.2789652856952, 0, 1538.2709291392293, 1.4795159243676608, 894.3286371180006, 0, -436.6573650944801], '0.000909_103.305785_1.818182_426.883410_0.000000_0.100000': [0, 4999.302079574411, 880.4379325195072, -398.0112548352517, 223.48036017759185, 376.08493780076674, 0, -81.25208125887222], '0.000826_103.305785_1.818182_426.883410_0.000000_0.100000': [129.78916617855418, 0, 0, 0, 281.67748562358366, 250.52588028801665, 0, 334.8879464586259], '0.000826_93.914350_1.818182_426.883410_0.000000_0.100000': [696.3759644608235, 545.8600996430916, 273.31886758494034, 867.8903391755659, 128.66888477257726, 66.43339499937393, 0, 0], '0.000826_93.914350_1.652893_426.883410_0.000000_0.100000': [0, 500.53038281318095, 169.9721370697681, 707.6395668089083, 0, 2144.846446066096, 122.69098902430716, -235.35166819666614], '0.000826_93.914350_1.652893_469.571751_0.000000_0.100000': [831.4362358080525, 0, 0, -107.0826613773681, 0, 201.7892212742826, 0, 0], '0.000826_85.376682_2.000000_687.500000_0.000000_0.100000': [0, 0, 0, 828.5466786128648, 4.5691467096177805, 0, -30.38069843290041, 0], '0.000751_85.376682_2.000000_687.500000_0.000000_0.100000': [282.3223392596854, 0, 0, -1263.6851822387205, 0, 0, 0, 0], '0.000751_77.615165_2.000000_687.500000_0.000000_0.100000': [0, 0, 0, 0, 4.1006677913163, 0, 0, 0], '0.000683_77.615165_2.000000_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -282.2706167732999, 0], '0.000683_85.376682_2.000000_687.500000_0.000000_0.100000': [0, 630.0201154298633, 0, 0, 0, 1714.947566945777, -303.72519910928133, 0], '0.000683_85.376682_1.818182_687.500000_0.000000_0.100000': [0, 0, 0, -1714.947566945777, 0, 0, 0, 0], '0.000683_85.376682_2.000000_625.000000_0.000000_0.100000': [0, 1437.4789725978833, 0, 0, 0, 0, 0, 0], '0.000683_85.376682_2.000000_568.181818_0.000000_0.100000': [-525.1168304497124, 0, 0, 0, 0, 0, 0, 0], '0.000683_77.615165_2.000000_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 1046.0291604525046, 0, 282.07819276203736], '0.000683_77.615165_2.000000_625.000000_0.000000_0.100000': [0, -282.07819276203736, 0, 0, 0, 0, 0, 0], '0.000683_77.615165_1.818182_568.181818_0.000000_0.100000': [4215.765829001011, 0, 0, 0, 0, 0, -921.0515675049212, 0], '0.000683_85.376682_1.818182_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 382.42732977316876, 0, 0], '0.000683_85.376682_1.652893_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 521.6168837827981, 0, 0], '0.000683_85.376682_1.502630_568.181818_0.000000_0.100000': [4567.281204484721, 0, -3.1795901095892076, 0, 0, 0, 0, 0], '0.000751_85.376682_1.502630_568.181818_0.000000_0.100000': [0, 838.244343714148, 0, 0, 0, 0, 0, 0], '0.000751_85.376682_1.502630_516.528926_0.000000_0.100000': [3552.644671277731, 0, 0, 0, 0, 0, 0, 0], '0.000751_77.615165_1.502630_516.528926_0.000000_0.100000': [0, 0, -3.7393180172708527, 4452.702470855174, 0, 0, 0, -499.9397931519282], '0.000751_77.615165_1.652893_516.528926_0.000000_0.100000': [5061.322666150045, 0, 0, 0, 1183.4599354577417, 0, 0, 0], '0.000683_77.615165_1.652893_516.528926_0.000000_0.100000': [4369.897359956767, 386.3435417984043, 0, 0, 0.5689718230564722, 0, 0, 0], '0.000683_77.615165_1.652893_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000826_137.500000_1.818182_516.528926_0.000000_0.100000': [0, 0, 37.83780778945777, 0, 0, 0, 0, -433.98191556327765], '0.000826_137.500000_1.818182_568.181818_0.000000_0.100000': [0, 0, -355.0407812177899, -427.3598013233882, 0, 0, 0, -244.9164904063006], '0.000751_137.500000_1.818182_568.181818_0.000000_0.100000': [707.32232096413, 0, 0, 0, 0, 276.7049649242161, 0, 0], '0.000826_125.000000_1.502630_516.528926_0.000000_0.100000': [0, 468.47804157609073, 0, 0, 0, 0, 0, 0], '0.000826_113.636364_1.502630_469.571751_0.000000_0.100000': [0, 501.4127119789746, 0, 0, 0, 0, 0, 0], '0.000826_103.305785_1.502630_426.883410_0.000000_0.100000': [599.0310712792384, 1336.8137935742684, 416.27558234640605, -166.63351287226897, 0, 0, 29.946209533905687, -506.46070761657876], '0.000826_93.914350_1.652893_388.075827_0.000000_0.100000': [197.7555175039531, 447.17836390267576, 0, -22.02110067554413, 336.01348865342084, 1174.340973548739, 0, 0], '0.000826_93.914350_1.818182_352.796206_0.000000_0.100000': [271.01105714306163, 0, -2.2363934784752018, 0, 0, -4.427145253101935, 0, 0], '0.000826_93.914350_1.652893_352.796206_0.000000_0.100000': [479.695889036452, 0, -1.1396700277382479, 0, 0, 0, 0, 98.76587591899715], '0.000909_93.914350_1.652893_352.796206_0.000000_0.100000': [245.41375970332535, 0, 0, 0, 0, 0, 0, 0], '0.000826_113.636364_1.366027_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -466.3974468287985, 0], '0.000826_125.000000_1.366027_426.883410_0.000000_0.100000': [0, 0, -3.8193767321371523, 0, 3.4681064931243784, 0, 0, 0], '0.000751_125.000000_1.366027_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, 283.02834683678185, 0, -840.9079949242503], '0.000751_125.000000_1.241843_426.883410_0.000000_0.100000': [0, 0, -1.7231671251737681, 0, 0, 0, 0, 0], '0.000826_125.000000_1.241843_426.883410_0.000000_0.100000': [0, 0, 0, -284.77328620473247, 0, 0, 0, 0], '0.001331_125.000000_1.818182_568.181818_0.000000_0.100000': [569.7540309540428, 0, -4.4377290880278455, 0, 0, 0, 0, -468.26699642790663], '0.001210_85.376682_1.818182_625.000000_0.000000_0.100000': [0, 3471.4551875818424, 0, -1276.3119303112906, 0, -497.05401172276027, 0, 0], '0.001210_85.376682_1.818182_568.181818_0.000000_0.100000': [-245.9513016488065, -844.3969580546259, -114.00213348488023, -766.5809988420913, 1831.9965445034632, 0, 0, -93.78678351382172], '0.001210_85.376682_1.818182_516.528926_0.000000_0.100000': [-824.600800147097, -602.5274731150992, -3.8934158781666155, -749.8607070094008, 2532.1304586402134, 0, 0, -145.10338631442846], '0.001331_85.376682_1.818182_516.528926_0.000000_0.100000': [0, 0, -4.682350917498115, 0, 0, -424.43725994932424, 0, 0], '0.001464_85.376682_1.818182_516.528926_0.000000_0.100000': [0, 0, 0, -1442.1173900631745, 0, 0, -928.2415468456085, 0], '0.001464_93.914350_2.000000_426.883410_0.000000_0.100000': [0, 0, 0, -1211.0573785535098, 7.842319244316204, 0, 0, 0], '0.001464_93.914350_2.200000_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, 1211.0573785535098, 0, 0], '0.001331_93.914350_2.000000_426.883410_0.000000_0.100000': [920.6660401587068, 467.83490987693807, 0, 0, 0, 0, -26.686120819913942, 0], '0.000751_125.000000_2.000000_687.500000_0.000000_0.100000': [0, 0, 0, 0, 4.235614565998367, 0, 0, 0], '0.000683_125.000000_2.000000_687.500000_0.000000_0.100000': [0, 0, 0, 0, 4.2356888834084385, 0, 0, 0], '0.000621_125.000000_2.000000_687.500000_0.000000_0.100000': [366.6856542683696, 0, -0.38506262576440337, -1182.0653020957473, 0, 0, -386.2025655200652, 0], '0.000621_137.500000_2.000000_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -538.0132425363487], '0.000621_137.500000_2.000000_756.250000_0.000000_0.100000': [360.5386100871125, 0, 0, 0, 0, 0, 0, 0], '0.000621_125.000000_2.000000_756.250000_0.000000_0.100000': [0, 620.0449177662316, 0, -223.2545449965338, 0, 0, 0, 0], '0.000621_125.000000_2.200000_756.250000_0.000000_0.100000': [0, 0, 0, 0, 0, 1735.2094756533083, 0, 0], '0.000621_125.000000_2.200000_687.500000_0.000000_0.100000': [0, 0, 0, -465.2585018312349, 0, 0, 0, 0], '0.000621_125.000000_2.420000_687.500000_0.000000_0.100000': [246.138645992666, 0, 0, 0, 0, 0, 0, 0], '0.000621_113.636364_2.420000_687.500000_0.000000_0.100000': [0, 0, 0, 0, 1.6515783087415912, 0, 0, 0], '0.000564_113.636364_2.420000_687.500000_0.000000_0.100000': [239.25692670315257, 0, 0, 0, 0, 0, 0, 0], '0.000564_103.305785_2.420000_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -548.6635907604635], '0.000564_103.305785_2.420000_756.250000_0.000000_0.100000': [205.1254980983018, 0, 0, 0, 0, 0, 0, 0], '0.000564_93.914350_2.420000_756.250000_0.000000_0.100000': [197.95092827890022, 0, 0, 0, 1.7468722140931277, 0, 0, 0], '0.000513_93.914350_2.420000_756.250000_0.000000_0.100000': [0, 0, -1.7468722140931277, 0, 0, 0, 0, 0], '0.000564_85.376682_2.420000_756.250000_0.000000_0.100000': [0, 0, 0, 0, 0, 754.9591386993078, 0, 0], '0.000564_85.376682_2.200000_756.250000_0.000000_0.100000': [0, 0, -3.147955389067829, 0, 0, 0, 0, -639.913983534267], '0.000564_85.376682_2.200000_831.875000_0.000000_0.100000': [0, 639.913983534267, 0, 0, 0, 0, 0, 0], '0.000621_85.376682_2.200000_756.250000_0.000000_0.100000': [1748.8994666561898, 0, 0, 0, 0, 0, 0, 0], '0.000621_77.615165_2.200000_756.250000_0.000000_0.100000': [0, 0, 0, -2313.6410919331447, 0, 0, 0, 0], '0.000621_77.615165_2.420000_756.250000_0.000000_0.100000': [0, 0, -2.4020059050561033, 0, 0, 0, 0, 0], '0.000683_77.615165_2.420000_756.250000_0.000000_0.100000': [177.52818529620146, 0, 0, 0, 0, 0, 0, 0], '0.000683_70.559241_2.420000_756.250000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000826_125.000000_1.818182_426.883410_0.000000_0.100000': [0, 0, 0, 0, 2.8844809341961084, 0, -402.00193212310705, 0], '0.000751_125.000000_1.818182_426.883410_0.000000_0.100000': [547.6679301644385, 0, 0, 0, 0, 261.7011285581096, -442.1495932188755, -423.70309830955404], '0.000751_137.500000_1.818182_426.883410_0.000000_0.100000': [40.19541756535232, 431.0438396934834, -2.644048939605818, 0, 0, 0, 0, 0], '0.000751_137.500000_2.200000_516.528926_0.000000_0.100000': [0, 0, -3.7897558483757168, 0, 0, 0, 0, 0], '0.000826_137.500000_2.200000_516.528926_0.000000_0.100000': [383.97744213231766, 0, 0, 0, 0, 1190.317072321094, 0, 0], '0.000826_125.000000_2.420000_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 533.8199148855025, 0, 0], '0.000826_137.500000_2.000000_516.528926_0.000000_0.100000': [0, 0, 780.653786110275, 0, 0, 1261.5847715806067, 0, -494.6202671212881], '0.001210_151.250000_2.200000_625.000000_0.000000_0.100000': [690.424845379848, 0, 0, 0, 0, 0, 0, 0], '0.001100_103.305785_1.652893_516.528926_0.000000_0.100000': [0, 0, 216.22414273877206, 1490.062592274108, 2.3820108687187713, 0, 0, 0], '0.001210_93.914350_1.652893_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 934.5336620581464, -280.1137796118259, 0], '0.001000_93.914350_2.420000_756.250000_0.000000_0.100000': [0, 0, 0, 0, 0, 673.296407385075, 0, 0], '0.001000_93.914350_2.200000_756.250000_0.000000_0.100000': [0, 1464.5420779882081, 0, 0, 0, 0, -28.121272182599878, 0], '0.001000_85.376682_2.000000_625.000000_0.000000_0.100000': [1200.8453583636033, 2761.082822763534, 1256.4899301653593, 0, 1083.3157506508626, 0, 0, 0], '0.001000_77.615165_2.000000_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 1165.1781754322574, 0, 287.44324897172555], '0.000909_70.559241_1.818182_568.181818_0.000000_0.100000': [0, 0, 0.03568751190489366, 0, 0, 0, 0, 4.37951156927311], '0.000909_70.559241_1.818182_625.000000_0.000000_0.100000': [23.213453698478844, 0, 0, 0, 0, 0, 0, 0], '0.000909_64.144765_1.818182_625.000000_0.000000_0.100000': [0, -0.744509557174888, 0, 0, 0, 0, 0, 0.014269183696503195], '0.000909_64.144765_1.818182_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, -0.0005222409757379865, 0, 0], '0.000909_64.144765_1.652893_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0.7331982548105385], '0.000909_64.144765_1.652893_625.000000_0.000000_0.100000': [0.05087843187509407, -0.06665438680095798, 0, 0, 0, 0, 0, 0], '0.000909_58.313423_1.652893_625.000000_0.000000_0.100000': [0, 0, -0.7924529064981698, 0, 0, 0, 0, 0], '0.001000_58.313423_1.652893_625.000000_0.000000_0.100000': [0, 0, 0, 0.13197404542455388, 0, 0, 0, 0], '0.001000_58.313423_1.818182_625.000000_0.000000_0.100000': [0.729703890897622, 0, -0.03673687499547214, 0, 0, 0, 0, 0], '0.001100_58.313423_1.818182_625.000000_0.000000_0.100000': [0, 0, -0.016750197114697586, 0, 0, 0, 0, 0], '0.001210_58.313423_1.818182_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, -0.06780275844739662, 0, 0], '0.001210_58.313423_1.652893_625.000000_0.000000_0.100000': [0, 0, 0.021883133659162012, 0, 0, 0, 0, 0], '0.000909_125.000000_2.000000_388.075827_0.000000_0.100000': [0, 419.43747267008547, 0, -1201.2408451727088, 0, 0, 0, 0], '0.001000_125.000000_2.200000_352.796206_0.000000_0.100000': [0, 0, 0, -795.4674066598649, 0, 0, 0, 0], '0.001000_125.000000_2.420000_352.796206_0.000000_0.100000': [314.3075899117343, 0, 0, 0, 0, 0, 0, 0], '0.001000_113.636364_2.420000_352.796206_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -482.1620904530997], '0.001000_113.636364_2.420000_388.075827_0.000000_0.100000': [0, 0, 0, 0, -408.54558044075935, 0, 0, 0], '0.000909_113.636364_2.420000_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -503.20512009171875], '0.000909_113.636364_2.420000_426.883410_0.000000_0.100000': [0, 0, 0, -145.1354754373915, 0, 0, 0, -455.84763535152905], '0.000909_113.636364_2.662000_426.883410_0.000000_0.100000': [273.11406295753477, 0, 0, 0, 0, 0, -289.5995038813997, 0], '0.000909_125.000000_2.662000_426.883410_0.000000_0.100000': [289.5995038813997, 0, 0, 0, 0, 0, 0, 0], '0.000909_103.305785_2.662000_426.883410_0.000000_0.100000': [0, 0, 0, 0, 2.866660657340617, 0, 0, 0], '0.000826_103.305785_2.662000_426.883410_0.000000_0.100000': [0, 0, 0, 0, 2.595081487037169, 172.2778129926239, 0, 0], '0.000826_103.305785_2.420000_426.883410_0.000000_0.100000': [292.5701913969222, 0, 0, -172.2778129926239, 0, 0, 0, 0], '0.000751_103.305785_2.662000_426.883410_0.000000_0.100000': [0, 0, 0, 0, 2.349806010647626, 0, 0, 0], '0.000683_103.305785_2.662000_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, 170.83912583774418, 0, 0], '0.000683_103.305785_2.420000_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001464_125.000000_2.420000_625.000000_0.000000_0.100000': [0, 0, 0, 0, 248.20968826673507, 0, 0, 0], '0.001210_113.636364_2.420000_516.528926_0.000000_0.100000': [994.7185028590915, 0, 0, 0, 0, 0, -303.70430827467806, -4.743866215442654], '0.001210_125.000000_2.420000_516.528926_0.000000_0.100000': [276.0948257042528, 0, 0, 0, 0, 0, -274.59728818803177, 0], '0.001210_137.500000_2.420000_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -266.90482225620417, 0], '0.001210_166.375000_2.200000_469.571751_0.000000_0.100000': [0, 0, 0, 0, 5.207243235666647, 0, 0, 0], '0.001100_166.375000_2.200000_469.571751_0.000000_0.100000': [378.94103995548215, 0, 0, 0, 0, 0, 0, -476.6519634012493], '0.001100_166.375000_2.200000_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -461.32102862891935], '0.000909_137.500000_1.818182_469.571751_0.000000_0.100000': [0, 860.8491060397962, 0, 0, 0, 0, -401.440091085637, 0], '0.000826_151.250000_1.818182_469.571751_0.000000_0.100000': [0, 377.0329833572925, 0, 0, 0, 0, 0, 0], '0.000826_151.250000_1.818182_426.883410_0.000000_0.100000': [0, 0, 0, -778.9246017636744, 0, 0, 0, 0], '0.000826_151.250000_2.000000_426.883410_0.000000_0.100000': [0, 0, -4.958566889482609, 0, 0, 0, 0, 0], '0.000909_151.250000_2.000000_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, 1101.2910953478954, 0, -439.91542770340857], '0.000751_151.250000_2.000000_516.528926_0.000000_0.100000': [0, 1215.3856733740406, 0, 0, 0, 0, 0, 0], '0.000826_85.376682_1.652893_568.181818_0.000000_0.100000': [1334.7468953226285, 0, 0, 0, 0, 0, 1149.4476467071472, 0], '0.000826_77.615165_1.652893_568.181818_0.000000_0.100000': [4486.518146346131, 0, -3.4140849083449214, -457.2027883236336, 0, 0, 0, 0], '0.000751_70.559241_2.000000_568.181818_0.000000_0.100000': [2883.168052432248, 0, 0, 0, 0, 0, 0, 0], '0.000751_64.144765_2.000000_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0.013515480927278532, 0, 0, 0], '0.000683_64.144765_2.000000_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 0.7084944049901498, 0, -0.05397030048648731], '0.000751_58.313423_2.200000_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -0.028944298184546826], '0.000751_58.313423_2.200000_687.500000_0.000000_0.100000': [0, 0, 0, 0.018055636415215303, 0, 0, 0, 0], '0.000751_58.313423_2.420000_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001100_151.250000_1.652893_756.250000_0.000000_0.100000': [0, 0, -4.793442168808906, 0, 0, 152.71743075265894, 0, 0], '0.001210_151.250000_1.652893_756.250000_0.000000_0.100000': [0, 0, 0, -308.14192866099603, 0, 0, 0, 0], '0.001210_151.250000_1.818182_756.250000_0.000000_0.100000': [0, 0, -5.982545092451619, 0, 0, 0, 0, 0], '0.001331_151.250000_1.818182_756.250000_0.000000_0.100000': [0, 0, 0, 0, 0, 308.82668906618454, 0, 0], '0.001331_151.250000_1.652893_756.250000_0.000000_0.100000': [0, 490.88437949994153, 0, 0, 0, 0, 0, 0], '0.001331_151.250000_1.652893_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -346.8765886701922, 0], '0.001331_166.375000_1.652893_687.500000_0.000000_0.100000': [0, 0, -5.945606310384938, 0, 0, 0, 0, 0], '0.001464_166.375000_1.652893_687.500000_0.000000_0.100000': [0, 0, -6.56714256431751, 0, 0, 0, 0, 0], '0.001611_166.375000_1.652893_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -473.52899161309233], '0.001611_166.375000_1.652893_756.250000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -455.16233671028806], '0.001611_166.375000_1.652893_831.875000_0.000000_0.100000': [0, 0, 0, 0, 0, 222.98134492492974, 0, 0], '0.001611_166.375000_1.502630_831.875000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -283.9586149348594, 0], '0.001611_183.012500_1.502630_831.875000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -276.7151555555915, 0], '0.001611_201.313750_1.502630_831.875000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001210_125.000000_1.818182_756.250000_0.000000_0.100000': [847.3684232331946, 0, 0, 0, 0, 0, 0, 0], '0.001210_103.305785_1.818182_756.250000_0.000000_0.100000': [0, 0, 0, 0, 0, 305.34404594304493, 0, 0], '0.001210_103.305785_1.652893_756.250000_0.000000_0.100000': [0, 0, 0, 0, 3.825557272901642, 91.56092378784979, 0, -527.2452337309193], '0.000909_85.376682_1.818182_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -461.3929396952773], '0.000909_85.376682_1.818182_516.528926_0.000000_0.100000': [-826.1537179458824, -603.9717423355096, -951.9680815058111, -1430.6566892837109, 4.703918476041236, 2175.640210237986, 0, 3462.075242463028], '0.000826_85.376682_1.818182_469.571751_0.000000_0.100000': [4935.063757356953, 0, 595.2812486887867, 327.54883977523417, 0, 0, 0, 0], '0.000826_77.615165_1.818182_469.571751_0.000000_0.100000': [4680.671083271745, 506.79490272376825, 0, 0, 0, 4.903503322751785, 0, 0], '0.000826_77.615165_1.818182_426.883410_0.000000_0.100000': [0, 0, 0, 4040.372947259845, 0, -6.585141966203948, 0, -460.72263883978934], '0.000826_77.615165_2.000000_426.883410_0.000000_0.100000': [3583.7910023171407, 0, 0, 0, 0, 0, 0, 0], '0.000826_70.559241_2.000000_426.883410_0.000000_0.100000': [25.760925697581524, 0, 0, 0, -2018.083644438383, 0, 0, 0], '0.000751_70.559241_2.000000_426.883410_0.000000_0.100000': [0, 0, 0, 0, 6.982482413467188, 0, 0, 0], '0.000683_70.559241_2.000000_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, 208.99387836214325, 0, 0], '0.000683_70.559241_1.818182_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -2112.687452088836, 0], '0.000683_77.615165_1.818182_426.883410_0.000000_0.100000': [0, 0, 0, 520.6337400942294, 0, -68.74026351019329, 0, -459.7488614237442], '0.000683_77.615165_1.818182_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0.23436123598949052, 0, 0], '0.000909_125.000000_1.502630_687.500000_0.000000_0.100000': [0, 0, -1.4785139221032295, 0, 0, 1282.7254718663953, 0, 0], '0.001000_137.500000_1.502630_687.500000_0.000000_0.100000': [330.8728903380028, 0, -0.973056871794859, 0, 0, 0, 0, 0], '0.000826_113.636364_1.241843_756.250000_0.000000_0.100000': [0, 0, 0, -615.94722112023, 0, 0, 0, 0], '0.000826_113.636364_1.366027_756.250000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -517.0126818744319], '0.000826_113.636364_1.366027_831.875000_0.000000_0.100000': [0, 0, 604.8218743307684, 0, 0, 0, 0, 0], '0.000909_113.636364_1.241843_756.250000_0.000000_0.100000': [245.36871081464923, 0, -2.5944259936501113, 0, 0, 0, -297.2060531629821, -45.80773759159475], '0.001000_113.636364_1.241843_756.250000_0.000000_0.100000': [0, 0, 0, 0, 2.5944259936501113, 0, 0, 0], '0.001331_137.500000_1.818182_568.181818_0.000000_0.100000': [0, 0, -5.135546303967203, 0, 4.64316049858935, 0, 0, -460.5725398805666], '0.001000_103.305785_1.818182_388.075827_0.000000_0.100000': [0, 0, 1062.9329066709508, -366.8042486417714, 3.244751500517268, 2371.060589918226, 0, 0], '0.000909_103.305785_1.818182_388.075827_0.000000_0.100000': [311.8293325682915, 2087.0900953730434, 0, -364.893612124982, 8.9578438597238, 0, 310.6790507756359, 11.13730565107357], '0.000826_103.305785_1.818182_388.075827_0.000000_0.100000': [0, 0, 0, 0, 5.386587705777959, 0, 0, 0], '0.000751_103.305785_1.818182_426.883410_0.000000_0.100000': [347.75855456285376, 0, -2.8421546919083993, 0, 0, 0, 0, 0], '0.000751_93.914350_1.818182_426.883410_0.000000_0.100000': [0, 877.6403003707812, -2.3600885914181164, 0, 2.3473539445943787, 496.189328089531, 34.81459703809568, -463.95160152637385], '0.000751_85.376682_1.818182_388.075827_0.000000_0.100000': [0, 0, 437.3660521016205, 2365.3732786546107, 442.38849871381655, 355.73929031716045, -268.8981506173251, -429.81346002461606], '0.000751_85.376682_1.652893_388.075827_0.000000_0.100000': [0, 0, 0, 0, 46.26024591293924, 0, 0, 0], '0.000683_85.376682_1.652893_388.075827_0.000000_0.100000': [205.36070655064614, 0, 0, 752.5223340255342, 204.98563781183958, 0, 0, 0], '0.000683_85.376682_1.818182_388.075827_0.000000_0.100000': [0, 0, -0.18007596697958425, 723.6692594418917, 0, 0, 0, 0], '0.000683_85.376682_2.000000_388.075827_0.000000_0.100000': [228.85763885122657, 0, 0, 0, 0, 0, 0, 0], '0.000683_77.615165_2.000000_388.075827_0.000000_0.100000': [0, 0, 0, -1611.700874078844, 0, -479.7082241483272, -2.059924742135269, 0], '0.000683_77.615165_1.818182_388.075827_0.000000_0.100000': [3241.0149817258284, 0, 0, 0, 0, 0, 0, -406.18471664846584], '0.000683_77.615165_2.000000_426.883410_0.000000_0.100000': [1190.997337988017, 0, 0, 0, 0, 0, 0, 0], '0.000909_137.500000_1.502630_625.000000_0.000000_0.100000': [0, 458.6233344148795, -1.6582779938994645, 0, 0, 0, 0, 0], '0.000909_137.500000_1.502630_568.181818_0.000000_0.100000': [0, 0, -376.79910397338335, 111.06403446310343, 2.6247937956565153, 0, 0, 0], '0.000826_137.500000_1.502630_568.181818_0.000000_0.100000': [0, 0, 0, 515.7424732688363, 1.3716458900584567, 0, 0, 0], '0.000751_137.500000_1.502630_568.181818_0.000000_0.100000': [0, 0, 0, 113.87505017638196, 0, 0, -300.4595127194969, -446.39766658045875], '0.000751_151.250000_1.502630_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 504.14515111782623, 0, 0], '0.000751_151.250000_1.366027_568.181818_0.000000_0.100000': [0, 472.252624693008, 0, 0, 0, 0, 0, 0], '0.000751_151.250000_1.366027_516.528926_0.000000_0.100000': [0, 499.95640937964765, 0, 0, 0, 0, 0, 0], '0.000751_151.250000_1.366027_469.571751_0.000000_0.100000': [505.9149588493885, 0, 0, 0, 0, 0, 0, -45.450582670877026], '0.000751_137.500000_1.366027_469.571751_0.000000_0.100000': [437.30583820623514, 0, 0, 0, 3.330471330207365, 0, -45.99226898630805, 0], '0.000751_125.000000_1.366027_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -437.30583820623514, 0], '0.000683_137.500000_1.366027_469.571751_0.000000_0.100000': [0, 0, 0, 0, 3.323570362044575, 0, 0, 0], '0.000621_137.500000_1.366027_469.571751_0.000000_0.100000': [0, 0, -3.0516418778772914, -759.2240508801577, 0, 0, 0, 0], '0.000621_137.500000_1.502630_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 758.9521223959905, 0, 0], '0.001000_85.376682_2.200000_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 820.2066544562672, 0, 0], '0.000909_85.376682_1.652893_625.000000_0.000000_0.100000': [0, 0, 0, 3895.2743939014567, 0, 0, 0, 0], '0.001000_113.636364_1.818182_426.883410_0.000000_0.100000': [445.0333934989659, 1902.863468933664, -291.4863663136748, -419.84627146755656, 9.091822781860294, -72.77251514304771, -937.117885551476, 141.8358493129932], '0.001000_125.000000_1.818182_388.075827_0.000000_0.100000': [0, 875.2467568705559, 0, 0, 237.92007796005817, 1443.7669537401578, 0, 0], '0.001000_125.000000_1.652893_388.075827_0.000000_0.100000': [316.39540294602176, 376.22990957279325, 378.2410850098792, -0.2667188145553439, 0, 2230.1414623000774, -350.46731799155936, 0], '0.001000_113.636364_1.652893_388.075827_0.000000_0.100000': [3985.514149361658, 0, -18.847695865879416, 167.65900668366862, 0, 158.44260035382757, 205.8122599699978, 0], '0.001100_93.914350_1.652893_388.075827_0.000000_0.100000': [0, 2015.686341749459, 0, 0, 48.310530906421036, 0, 50.26848338596897, 0], '0.001000_93.914350_1.652893_352.796206_0.000000_0.100000': [588.4646969093042, 3821.979191045102, 452.8042693700138, 2.039726304251144, 201.98334600233014, 0, 0, 0], '0.001000_93.914350_1.652893_320.723824_0.000000_0.100000': [3410.8612424281364, 0, -1.4477442069102109, 0, 314.4010123606119, 223.09942716688988, 0, 454.7457986134366], '0.001000_85.376682_1.652893_320.723824_0.000000_0.100000': [256.87609714052377, 0, 0, 0, 0, 3150.679049623983, -234.18358707346633, 0], '0.001000_85.376682_1.502630_320.723824_0.000000_0.100000': [1925.5090410019795, 0, 0, 0, 0, 0, 0, -274.35248034524096], '0.001000_85.376682_1.502630_352.796206_0.000000_0.100000': [543.3205367785382, 0, -2.1468406102965676, 0, 0, 494.2024846677708, 0, 0], '0.001000_85.376682_1.366027_352.796206_0.000000_0.100000': [0, 0, 0, 104.97483668190442, 709.2753782685727, -627.9046839230855, 0, 0], '0.000909_85.376682_1.366027_352.796206_0.000000_0.100000': [173.0372725244829, 0, 0, 0, 1736.463471759882, 0, 0, -291.6158142533593], '0.000909_85.376682_1.366027_388.075827_0.000000_0.100000': [2089.476535232427, 0, 0, 0, 0, 0, 0, 0], '0.000909_113.636364_2.420000_516.528926_0.000000_0.100000': [299.0323095274975, 456.5439446472425, 0, -338.1356731396545, 1.8754655965713027, 750.3426804975945, 0, 0], '0.000826_113.636364_2.420000_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 1595.0014015150941, 0, 0], '0.000826_113.636364_2.200000_516.528926_0.000000_0.100000': [405.0886200436298, 1072.5424074474086, 0, -67.48082945947418, 0, 661.0262374651508, 0, 0], '0.000826_113.636364_2.200000_426.883410_0.000000_0.100000': [1595.1257835121387, 0, 0, 0, 0, 0, 99.77068545847649, 0], '0.000826_103.305785_2.200000_426.883410_0.000000_0.100000': [431.083529262067, 0, -5.348392913001954, -922.8104189144897, 0, 1373.0314945973857, 0, 0], '0.000826_103.305785_2.000000_426.883410_0.000000_0.100000': [0, 0, 389.00157954601946, 0, 355.57538782997176, 0, 0, 0], '0.000751_103.305785_2.000000_426.883410_0.000000_0.100000': [712.1691542205212, 0, -0.04206157916028071, -1244.112618547133, 0, 0, 0, -497.3557558098919], '0.000751_103.305785_2.000000_469.571751_0.000000_0.100000': [0, 497.3557558098919, 0, 0, 496.20305082837143, 0, 0, 0], '0.000683_103.305785_2.000000_388.075827_0.000000_0.100000': [390.8316895513217, 511.82886865733786, -3.635016563841782, -1196.4213964230091, 0, 0, 0, -4.644470251007999], '0.000683_103.305785_2.200000_388.075827_0.000000_0.100000': [473.8672914891689, 0, 0, 0, 0, 0, 0, 0], '0.000683_93.914350_2.200000_388.075827_0.000000_0.100000': [412.4459848590891, 0, -4.392488288052573, 0, 0, 0, 0, 0], '0.000751_93.914350_2.200000_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 1295.3335638850544, 0, 0], '0.000751_93.914350_2.000000_388.075827_0.000000_0.100000': [2463.8856040667706, 0, 0, 83.11359592043861, 0, 551.4519645902835, 0, 0], '0.001100_93.914350_1.818182_516.528926_0.000000_0.100000': [1190.3559394518388, 473.0381311671211, -94.00458254395926, 0, 1777.8637534935642, 0, 552.95968065807, -481.1800117058846], '0.001100_93.914350_1.652893_568.181818_0.000000_0.100000': [620.7055670951422, 0, -2.7265483901495826, 3770.2601805686973, -157.8403390767863, 0, 0, -159.07021096528115], '0.001210_93.914350_1.502630_568.181818_0.000000_0.100000': [0, 0, 0, 0, 1081.0066621907752, 0, 0, 0], '0.001100_93.914350_1.502630_568.181818_0.000000_0.100000': [0, 0, -0.522124263380683, 0, 0, 1105.0726911141885, 0, 0], '0.001210_151.250000_2.000000_469.571751_0.000000_0.100000': [495.44882884076486, 0, 0, 0, 0, 0, -504.82096253180174, 0], '0.001000_103.305785_1.652893_388.075827_0.000000_0.100000': [0, 4429.240183191585, 0, -114.04884548687214, 0, 614.0404885137968, -221.48057192479976, 265.8535687178554], '0.001000_103.305785_1.652893_352.796206_0.000000_0.100000': [4075.035629323008, 384.5949542242039, -1.3856181269358785, -41.27651069610687, 326.7203248717569, 440.6576359349889, -261.7279026894552, 0], '0.001000_103.305785_1.818182_352.796206_0.000000_0.100000': [0, 0, 0, -340.3375935501508, 377.3192205668434, 4066.2622259767836, 0, 0], '0.001000_85.376682_1.652893_352.796206_0.000000_0.100000': [0, 1723.7986364829624, 0, 0, 0, 0, 0, 0], '0.001000_77.615165_1.652893_320.723824_0.000000_0.100000': [0, 0, -3.6693754341566094, 0, 0, 0, 0, 0], '0.001100_77.615165_1.652893_320.723824_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -255.74764322718684, 0], '0.001100_85.376682_1.652893_320.723824_0.000000_0.100000': [0, 0, 0, 0, 995.1860598324978, 0, 0, 0], '0.001100_93.914350_1.652893_320.723824_0.000000_0.100000': [235.24562601857517, 0, 0, -17.914535629303465, 0, 0, 0, 1035.192179757078], '0.001100_93.914350_1.818182_320.723824_0.000000_0.100000': [0, 0, 0, 0, 1.9803241634829647, 0, 0, 0], '0.001000_93.914350_1.818182_320.723824_0.000000_0.100000': [0, 327.99788245181725, 0, 0, 0, 0, 0, 0], '0.001000_93.914350_1.818182_291.567113_0.000000_0.100000': [895.0218480531219, 0, -2.6465425923924157, -215.40462099452708, 0, 0, 0, 0], '0.000826_103.305785_1.502630_568.181818_0.000000_0.100000': [461.81215321284145, 519.5032989906917, 0, 0, 0, 1007.2632133756247, 0, 0], '0.000826_85.376682_1.502630_469.571751_0.000000_0.100000': [0, 502.19547558102107, 0, 0, 4.442379439221981, 0, -365.8162533004158, 0], '0.000751_85.376682_1.502630_469.571751_0.000000_0.100000': [1155.7688949308322, 0, 0, 144.64294411612065, 3.634159632586943, 0, -366.1522246849607, 0], '0.000751_93.914350_1.502630_469.571751_0.000000_0.100000': [80.66097670168489, 0, 0, -204.3573005305043, 0, 0, 0, -488.06787088375404], '0.000751_93.914350_1.502630_516.528926_0.000000_0.100000': [358.9925054544874, 488.0678708837549, 0, 0, 0, 0, 0, 0], '0.000909_125.000000_2.420000_516.528926_0.000000_0.100000': [1010.1274628978044, 0, 0, -318.1020367387297, 0, 0, -213.21277039192765, 0], '0.000909_113.636364_2.662000_516.528926_0.000000_0.100000': [241.8774512828726, 0, 0, 0, 0, 0, 0, -583.8778042896191], '0.001000_166.375000_1.818182_831.875000_0.000000_0.100000': [0, 0, 0, 0, 0, 365.4306624012814, 0, 0], '0.001000_166.375000_1.652893_831.875000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001000_103.305785_2.420000_687.500000_0.000000_0.100000': [230.71803991897687, 0, 0, 0, 0, 0, 0, 0], '0.001000_93.914350_2.420000_687.500000_0.000000_0.100000': [239.17252584387847, 0, 0, 0, 0, 0, 0, 0], '0.001000_77.615165_2.420000_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 2649.902912818983, 0, -601.0945024569888], '0.001000_77.615165_2.200000_625.000000_0.000000_0.100000': [0, 273.824395507135, 0, 0, 0, 0, 0, 0], '0.000826_103.305785_1.652893_687.500000_0.000000_0.100000': [0, 0, 467.9817708932061, 0, 627.6357754874405, 0, -289.4378493775661, 0], '0.000751_103.305785_1.652893_756.250000_0.000000_0.100000': [0, 0, 0, 0, -469.66618093589415, 0, 0, 0], '0.000683_103.305785_1.652893_756.250000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -577.1571374401987], '0.000683_103.305785_1.652893_831.875000_0.000000_0.100000': [0, 0, 0, -303.52211390512605, 2.4131736176213963, 0, 0, 0], '0.000683_93.914350_1.818182_831.875000_0.000000_0.100000': [0, 0, 0, 0, 2.3812757064544714, 0, 0, 0], '0.000621_93.914350_1.818182_831.875000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -584.4396902065745], '0.000621_93.914350_1.818182_915.062500_0.000000_0.100000': [0, 0, 0, 0, 0, 348.58949175946617, 0, 0], '0.000621_93.914350_1.652893_915.062500_0.000000_0.100000': [228.98540166795337, 0, 0, 0, 0, 0, 0, -563.8589739129014], '0.000621_85.376682_1.652893_915.062500_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -228.98540166795337, 0], '0.000621_93.914350_1.652893_1006.568750_0.000000_0.100000': [206.80907703110606, 0, 0, 0, 0, 0, 0, 0], '0.000621_85.376682_1.652893_1006.568750_0.000000_0.100000': [0, 0, -2.384064535721882, 0, 0, 0, 0, 0], '0.000683_85.376682_1.652893_1006.568750_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -206.78461754970667, 0], '0.000683_93.914350_1.652893_1006.568750_0.000000_0.100000': [0, 0, -2.6020921828801877, 0, 0, 0, 0, 0], '0.000751_93.914350_1.652893_1006.568750_0.000000_0.100000': [0, 563.5332412572803, 0, 0, 0, 0, 0, 0], '0.000751_93.914350_1.652893_915.062500_0.000000_0.100000': [229.30860385584765, 0, 0, 0, 0, 0, 0, 0], '0.000751_85.376682_1.652893_915.062500_0.000000_0.100000': [0, 561.4473237412889, 0, 0, 0, 0, 0, 0], '0.000751_85.376682_1.652893_831.875000_0.000000_0.100000': [211.84879326404027, 0, 0, 0, 0, 0, 0, 0], '0.000751_77.615165_1.652893_831.875000_0.000000_0.100000': [0, 0, -2.1860221717404786, 0, 0, 0, 0, 0], '0.000826_77.615165_1.652893_831.875000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -212.11249366867307, 0], '0.000826_85.376682_1.652893_831.875000_0.000000_0.100000': [0, 0, 0, 0, 0, 154.69524968416863, 0, 0], '0.000826_85.376682_1.502630_831.875000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001000_103.305785_2.200000_568.181818_0.000000_0.100000': [0, 759.6789055484044, 43.29320480791711, -562.8388605046015, 0, 0, 0, -974.4343123573103], '0.001000_103.305785_2.420000_568.181818_0.000000_0.100000': [0, 0, 0, 0, 1.532834899233194, 0, -311.0101060789527, 0], '0.000909_103.305785_2.662000_516.528926_0.000000_0.100000': [0, 572.042288889237, 0, 0, 0, 0, 0, 0], '0.000909_103.305785_2.662000_469.571751_0.000000_0.100000': [0, 0, 0, -399.76499422861036, 0, 0, 0, 0], '0.000909_103.305785_2.928200_469.571751_0.000000_0.100000': [0, 507.80943828855834, 0, 0, 0, 0, 0, 0], '0.000909_103.305785_2.928200_426.883410_0.000000_0.100000': [0, 0, 0, 115.78313532607754, 0, 0, 0, 0], '0.000909_103.305785_3.221020_426.883410_0.000000_0.100000': [0, 438.2546257782742, 0, 0, 0, 0, 0, 0], '0.000909_103.305785_3.221020_388.075827_0.000000_0.100000': [0, 0, -0.8712415473561123, 0, 0, 0, 0, 0], '0.001000_103.305785_3.221020_388.075827_0.000000_0.100000': [0, 0, -0.9858950007142994, 0, 0, 0, 0, 0], '0.001100_103.305785_3.221020_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -229.09981947232208, 0], '0.001100_113.636364_3.221020_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, -126.65912669022255, 0, 0], '0.001100_113.636364_2.928200_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 494.12158130511074, 0, 0], '0.001100_113.636364_2.662000_388.075827_0.000000_0.100000': [282.506816964702, 0, 0, 0, 2.903781000813797, 0, 0, 0], '0.001100_103.305785_2.662000_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -282.506816964702, 0], '0.001000_113.636364_2.662000_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 92.2872555960622, 0, 0], '0.001210_137.500000_2.000000_568.181818_0.000000_0.100000': [1302.329862916449, 2281.1941210745754, -7.595375359622994, -1315.9437536757057, 0, 0, 0, 0], '0.001331_137.500000_2.000000_568.181818_0.000000_0.100000': [0, 0, 0, 0, 7.595375359622994, 0, 0, 0], '0.001000_125.000000_1.502630_625.000000_0.000000_0.100000': [762.2760977713845, 2174.8479022700194, 42.674962625333336, 0, 1.5260423326795718, 0, 763.8149925458049, 51.6060648684635], '0.000751_125.000000_1.502630_516.528926_0.000000_0.100000': [0, 0, 0, 0, 1.7811944291306645, 0, 0, 0], '0.000683_125.000000_1.502630_516.528926_0.000000_0.100000': [0, 0, 0, 0, 1.6088806746189221, 0, 0, 0], '0.000621_125.000000_1.502630_516.528926_0.000000_0.100000': [377.91138454852535, 0, 0, 0, 0, 0, 0, 0], '0.000621_113.636364_1.502630_516.528926_0.000000_0.100000': [370.54825192566994, 0, 0, 0, 0, 0, -34.355580413502295, 0], '0.000909_137.500000_1.652893_516.528926_0.000000_0.100000': [1950.6009563056712, 0, -2.6704489060000927, 0, 0, -145.49826360024983, 0, -133.84029584991174], '0.000909_137.500000_1.502630_516.528926_0.000000_0.100000': [1509.037242712172, 419.0010656442717, 0, 0, 1.9670810915557013, 0, 0, 0], '0.000826_137.500000_1.502630_516.528926_0.000000_0.100000': [0, 0, 0, 0, 1.7924913085460503, 0, 0, -413.3785395513479], '0.000751_137.500000_1.502630_625.000000_0.000000_0.100000': [0, 0, 0.13598836284688431, 0, 0, 0, 0, 0], '0.000826_137.500000_1.502630_625.000000_0.000000_0.100000': [0, 0, 0, 55.76509616159921, 0, 0, 0, -90.1731347841773], '0.000909_93.914350_2.200000_756.250000_0.000000_0.100000': [0, 0, 0, 0, 3805.7335634561864, 4454.225747421854, 0, 0], '0.000826_77.615165_2.000000_756.250000_0.000000_0.100000': [247.42084577378628, 0, 0, 0, 0, 0, 0, 0], '0.000826_70.559241_2.000000_756.250000_0.000000_0.100000': [7227.147269420348, 0, -4.981569731912351, 0, 0, 0, 0, 0], '0.000909_70.559241_2.000000_756.250000_0.000000_0.100000': [0, 0, 0, 0, 0, 1403.3485601347309, 0, 0], '0.000909_70.559241_1.818182_756.250000_0.000000_0.100000': [0, 5496.292249418462, 0, 0, 0, 0, 0, 0], '0.000826_77.615165_1.652893_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -1579.334428694861], '0.000826_70.559241_1.652893_756.250000_0.000000_0.100000': [0, 0, 0, -5777.4022782763495, 0, 0, 0, 0], '0.000826_70.559241_1.818182_756.250000_0.000000_0.100000': [0, 0, 0, -1403.4190817854828, 0, 0, 0, 0], '0.000826_64.144765_2.000000_756.250000_0.000000_0.100000': [0, 0, 0.04169805161013756, 0, 0, 0, 0, 0], '0.000909_64.144765_2.000000_756.250000_0.000000_0.100000': [0.4611563414626289, 0, 0, 0, 0, 0, 0, 0], '0.000909_58.313423_2.000000_756.250000_0.000000_0.100000': [0, 0, 0, 0, 0, 0.06589027116638135, 0, 0], '0.000909_58.313423_1.818182_756.250000_0.000000_0.100000': [-0.03745279110553384, 0, 0, 0, 0, 0, 0, 0], '0.001000_77.615165_2.000000_469.571751_0.000000_0.100000': [2836.310542704253, -483.20711229090125, 0, -3662.8950027402534, 0, 0, -1579.4094739670045, 0], '0.001100_113.636364_2.420000_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 2283.2919195022178, 0, 0], '0.001100_93.914350_2.200000_469.571751_0.000000_0.100000': [0, 13.38938024981934, 0, -966.9401953087034, 6.460326311060271, 1537.6482908475748, -4.133860447527332, -1078.8780845560595], '0.001000_137.500000_2.000000_426.883410_0.000000_0.100000': [0, 433.8419443944187, 0, 0, 50.862910493944014, 0, 0, 1741.4373309951895], '0.000909_125.000000_2.000000_426.883410_0.000000_0.100000': [0, 0, 0, -1168.8036702210898, 5.005911466824364, 0, 0, 0], '0.000826_125.000000_2.000000_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, 587.550758500178, 0, 0], '0.000751_137.500000_1.818182_388.075827_0.000000_0.100000': [402.6672601455408, 364.00226091866443, 0, 0, 0, 0, 0, 0], '0.000751_137.500000_1.818182_352.796206_0.000000_0.100000': [0, 0, 0, -594.2477075651017, 0, 0, 0, 0], '0.000751_137.500000_2.000000_352.796206_0.000000_0.100000': [0, 0, -4.341902476482938, 0, 0, 0, 0, 0], '0.000826_137.500000_2.000000_352.796206_0.000000_0.100000': [520.3592136917425, 0, 0, 0, 0, 0, 0, 0], '0.000826_125.000000_2.000000_352.796206_0.000000_0.100000': [930.3950237006663, 0, 0, 0, 0, 0, 0, 0], '0.000826_113.636364_2.000000_352.796206_0.000000_0.100000': [480.96718723462345, 395.13413340385125, 0, 0, 0, 0, -48.32370980767246, 0], '0.000826_103.305785_2.000000_352.796206_0.000000_0.100000': [387.36018766785594, 0, 0, 0, 0, 0, 0, -426.14868333037504], '0.001000_113.636364_2.200000_469.571751_0.000000_0.100000': [430.50634847235307, 0, 0, 0, 5.813103768576694, 2254.082560750062, 0, 0], '0.001100_93.914350_2.200000_388.075827_0.000000_0.100000': [412.72966282899296, 0, 0, 0, 0, 0, 0, 0], '0.001100_85.376682_2.200000_388.075827_0.000000_0.100000': [0, 0, 0, 0, 346.43525780056433, 0, 0, 0], '0.000909_103.305785_2.000000_426.883410_0.000000_0.100000': [1610.8966785948267, 0, 0, 0, 4.6324337650647065, 0, 0, 0], '0.001000_93.914350_2.000000_426.883410_0.000000_0.100000': [0, 1832.2779223039518, -5.319210939445748, 0, 0, 0, 0, 566.7739902319486], '0.001000_93.914350_2.000000_388.075827_0.000000_0.100000': [2293.0038341403015, 0, -4.974341865836687, 0, 0, 286.40723761869293, 0, -466.70164477500606], '0.001100_85.376682_2.000000_426.883410_0.000000_0.100000': [808.3882251875589, 3993.9652995522556, -8.318464360100824, 0, 0, -32.212928503686925, 0, 0], '0.001100_77.615165_2.000000_426.883410_0.000000_0.100000': [0, 2944.3254290353702, 0, 0, 0, -516.4332065784176, 0, 0], '0.001331_137.500000_2.200000_469.571751_0.000000_0.100000': [0, 0, 0, 0, 11.921911352615243, 0, 0, 0], '0.001000_113.636364_2.420000_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000751_125.000000_2.200000_568.181818_0.000000_0.100000': [0, 0, -3.981159339402075, 0, 0, 1753.1433324051475, 0, 0], '0.000751_137.500000_2.000000_568.181818_0.000000_0.100000': [400.2309657958231, 0, 0, -1206.246984391553, 0, 0, -456.65942153034706, 0], '0.000751_151.250000_2.000000_568.181818_0.000000_0.100000': [456.65942153034706, 0, -4.7143785497832145, 0, 0, 0, 0, 0], '0.000826_151.250000_2.000000_568.181818_0.000000_0.100000': [0, 0, 0, 0, 4.7143785497832145, 0, 0, 0], '0.000683_125.000000_2.000000_568.181818_0.000000_0.100000': [1449.3936691541426, 0, 0, 0, 0, 0, 0, 0], '0.000621_103.305785_1.652893_831.875000_0.000000_0.100000': [0, 0, 0, 0, 0, 148.38004523484452, 0, 0], '0.000621_103.305785_1.502630_831.875000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -257.99528748787435, 0], '0.000621_113.636364_1.502630_831.875000_0.000000_0.100000': [0, 0, 0, 0, 0.9575331360099881, 0, 0, 0], '0.000564_113.636364_1.502630_831.875000_0.000000_0.100000': [0, 0, 0, 0, 0, 678.9051895240824, 0, 0], '0.000564_113.636364_1.366027_831.875000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001210_137.500000_2.200000_568.181818_0.000000_0.100000': [0, 0, 0, 0, 6.404059488396597, 0, 0, 0], '0.001210_103.305785_2.662000_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000909_70.559241_2.000000_625.000000_0.000000_0.100000': [4377.889629965577, 0, 0, 0, 2.5103346672146474, 0, 0, 0], '0.000909_64.144765_2.000000_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0.04227202249235234, 0, 0, 0], '0.000826_64.144765_2.000000_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0.6560517020537873, 0, -4375.421567320855, 0], '0.000751_58.313423_2.000000_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0.03383324241544642, 0], '0.000683_64.144765_1.818182_568.181818_0.000000_0.100000': [0, 0, 0, -0.00047991636564802276, 0, 0, 0, 0.7383399189346429], '0.000683_64.144765_1.818182_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0.8465763184850124, 0, 0, 0], '0.000621_64.144765_1.818182_625.000000_0.000000_0.100000': [-0.7818774619954638, 0.029491835366891338, 0, 0, 0, -0.7036236807083242, 0, 0], '0.000621_64.144765_1.652893_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0.0016430827402018622, 0, 0, 0], '0.000564_64.144765_1.652893_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -27.207087525053794, 0], '0.000564_70.559241_1.652893_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, -3615.736714339318, 0, 0], '0.000564_70.559241_1.502630_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 826.1244422363415, 0, 0], '0.000564_70.559241_1.366027_625.000000_0.000000_0.100000': [0, 0, -1.2013109711085235, 0, 0, 0, 0, 0], '0.000621_70.559241_1.366027_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -382.6820570204338], '0.000621_70.559241_1.366027_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000909_103.305785_1.818182_469.571751_0.000000_0.100000': [392.6498012049485, 5102.969475064022, 121.80517490899501, 0, 778.233707965411, 0, 151.9212670461639, -453.097509437224], '0.000826_103.305785_1.818182_469.571751_0.000000_0.100000': [300.5185466295064, 1452.6074763518131, 0, -439.07947658898456, 0, 383.3550368909256, -212.18787650132776, -452.5852161916856], '0.000751_103.305785_2.200000_426.883410_0.000000_0.100000': [431.0798951504625, 0, 0, 0, 4.42088165374571, 0, 0, -521.8652122752277], '0.000751_103.305785_2.200000_469.571751_0.000000_0.100000': [0, 521.8652122752277, 0, 0, 0, 1268.622075012469, 0, 0], '0.000751_93.914350_2.000000_426.883410_0.000000_0.100000': [0, 1145.666124414949, 0, 0, 0, 349.5748764924195, 0, 0], '0.000683_93.914350_1.818182_426.883410_0.000000_0.100000': [0, 0, 0, 0, 1.9303245394865371, 63.35935606935391, 0, -463.5225096722109], '0.000683_93.914350_1.818182_469.571751_0.000000_0.100000': [0, 463.5225096722109, 0, 0, 0, 0, 0, 0], '0.000683_93.914350_1.652893_426.883410_0.000000_0.100000': [766.1899323988134, 0, 0, 0, 0, 0, 0, -423.1117726550154], '0.000683_93.914350_1.652893_469.571751_0.000000_0.100000': [630.6810438704508, 0, 0, 0, 0, 0, 0, 0], '0.000683_85.376682_1.652893_469.571751_0.000000_0.100000': [0, 831.8160398298875, 0, 0, 0, 0, 0, -108.19735019063926], '0.000683_85.376682_1.652893_426.883410_0.000000_0.100000': [0, 0, 0, 0, 927.1349212387522, 0, -19.0328895128358, 0], '0.000826_103.305785_1.818182_516.528926_0.000000_0.100000': [841.9777953227319, 0, -485.19486476490493, 0, 0, 3941.5488086613436, -603.9294313924783, 3.131940936148351], '0.001464_113.636364_1.818182_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -318.5874633876016, 0], '0.001464_125.000000_1.652893_568.181818_0.000000_0.100000': [0, 0, 0, 0, 278.6291135552621, 0, 0, 0], '0.000909_151.250000_1.652893_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -402.092522365378, 0], '0.000909_166.375000_1.652893_469.571751_0.000000_0.100000': [0, 0, -3.4781380623830045, 0, 0, 0, 0, 0], '0.001000_166.375000_1.652893_469.571751_0.000000_0.100000': [0, 0, 0, 358.15141859420277, 0, -72.66369948751908, 0, 0], '0.001000_137.500000_1.652893_426.883410_0.000000_0.100000': [341.05144443733246, 0, -2.830259924774564, -325.0836420635069, 370.78745231179755, 0, 0, -409.85449133532296], '0.001100_137.500000_1.652893_388.075827_0.000000_0.100000': [1210.7082573151733, 0, 0, -321.7515036909288, 0, -470.58860670906023, -386.50267327958267, 0], '0.001100_137.500000_1.818182_388.075827_0.000000_0.100000': [405.180658195602, 0, 0, 0, 0, 0, 0, 0], '0.001100_125.000000_1.818182_352.796206_0.000000_0.100000': [0, 0, 0, 0, 0, 256.2693714343774, 0, 0], '0.001100_125.000000_1.652893_352.796206_0.000000_0.100000': [528.2323162931451, 0, 0, 0, 0, 0, 0, 0], '0.001100_113.636364_1.652893_352.796206_0.000000_0.100000': [0, 0, 0, 0, 0, 873.842994165193, 0, -372.63700506099195], '0.001100_113.636364_1.502630_352.796206_0.000000_0.100000': [1048.3168076678016, 418.5667183390025, 0, 0, 0, 2498.9286921606968, 0, 0], '0.001100_113.636364_1.366027_352.796206_0.000000_0.100000': [1897.8901681469006, 0, 0, -647.6819851720252, 3.1091278586634874, 0, 0, 0], '0.001100_103.305785_1.502630_352.796206_0.000000_0.100000': [0, 425.35084146690645, 0, 0, 0, 1217.2580500681797, 0, 0], '0.000826_77.615165_1.502630_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -504.6052139302246], '0.000826_77.615165_1.502630_625.000000_0.000000_0.100000': [5193.078897940299, 0, 0, 0, 0, 0, 0, 0], '0.000826_70.559241_1.502630_625.000000_0.000000_0.100000': [0, 0, -3630.7590408196947, 0, 0, 0, 0, 0], '0.000909_70.559241_1.502630_625.000000_0.000000_0.100000': [0, 0, -2.484863478178704, 0, 0, 0, 0, 0], '0.001000_70.559241_1.502630_625.000000_0.000000_0.100000': [4068.899229657279, 0, 0, 0, 0, 0, 0, 0], '0.001000_64.144765_1.502630_625.000000_0.000000_0.100000': [-0.0033027699186095553, 0.8189093678977846, -0.06192686894428334, 0, 0, 0, -0.3795208375696348, 0], '0.001100_64.144765_1.502630_625.000000_0.000000_0.100000': [0, 0, 0.016989331352899626, 0, 0, 0, 0, 0], '0.001210_64.144765_1.502630_625.000000_0.000000_0.100000': [0, 0, 0, -0.006872039118388784, 0, 0, 0, 0], '0.001210_64.144765_1.652893_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0.8265482283749862], '0.001210_64.144765_1.652893_687.500000_0.000000_0.100000': [0, 0, -0.6982604403488267, 0, 0, 0, 0, 0], '0.001331_64.144765_1.652893_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, -0.06131631843431933, 0, 0], '0.001331_64.144765_1.502630_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001100_93.914350_1.818182_756.250000_0.000000_0.100000': [0, 0, -3.755440127384645, 0, 0, 382.79129534740366, 0, 0], '0.001210_93.914350_1.818182_756.250000_0.000000_0.100000': [3422.0249298096096, 0, 0, 2185.4223657953767, 0, 0, 0, 0], '0.001210_93.914350_2.200000_756.250000_0.000000_0.100000': [270.1183750410581, 0, 0, 0, 0, 0, 0, -607.1296474773875], '0.001210_77.615165_2.200000_756.250000_0.000000_0.100000': [261.3179429763224, 0, 0, 0, 0, 0, 0, 0], '0.001210_70.559241_2.200000_756.250000_0.000000_0.100000': [0, -848.4083010311051, 0, 0, 0, 0, 0, 0], '0.001210_70.559241_2.200000_687.500000_0.000000_0.100000': [7840.375925715104, 0, 0, 0, 0, 0, 0, 0], '0.001210_64.144765_2.200000_687.500000_0.000000_0.100000': [0.08146115659349107, 0, 0, 0, 0, 0, 0, 0], '0.001210_58.313423_2.200000_687.500000_0.000000_0.100000': [0.5797134718761754, 0, 0, 0.7388710810894822, 0, 0, 0, 0], '0.001210_58.313423_2.420000_687.500000_0.000000_0.100000': [-0.7581052213507974, -0.026236209364833486, 0, 0, -0.017309438406641675, -0.6656496226027685, 0, 0], '0.001210_53.012202_2.200000_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0.08121677334719696, 0], '0.001210_53.012202_2.420000_687.500000_0.000000_0.100000': [0.6842344208791928, 0, 0, 0, 0, 0, 0, 0], '0.001210_48.192911_2.420000_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -0.6848962725269075], '0.001210_48.192911_2.420000_756.250000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001100_85.376682_1.502630_831.875000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -505.26864359825686], '0.001100_85.376682_1.502630_915.062500_0.000000_0.100000': [210.56298070142057, 0, 0, 0, 0, 0, 0, 0], '0.001100_77.615165_1.502630_915.062500_0.000000_0.100000': [0, 0, 0, 0, 0, 913.4696176364289, 0, 0], '0.001100_77.615165_1.366027_915.062500_0.000000_0.100000': [6350.53568971145, 0, -5.728535450710843, 0, 0, 0, 0, 0], '0.001210_77.615165_1.366027_915.062500_0.000000_0.100000': [0, 0, 0, 0, 5.728535450710843, 0, 0, 0], '0.001100_70.559241_1.366027_915.062500_0.000000_0.100000': [0, 0, 0, 0, 0, -86.31780582078814, 0, 0], '0.001100_70.559241_1.241843_915.062500_0.000000_0.100000': [0, 0, 2.2882859511139486, 0, 0, 0, 0, 0], '0.001210_70.559241_1.241843_915.062500_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -5383.3008519485165, 0], '0.001210_77.615165_1.241843_915.062500_0.000000_0.100000': [0, 0, 0, 0, 0, 1888.6737251301292, 0, 0], '0.001210_77.615165_1.128948_915.062500_0.000000_0.100000': [3674.5895841975207, 0, 0, 0, 0, 0, 0, 0], '0.001210_70.559241_1.128948_915.062500_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001000_103.305785_2.200000_516.528926_0.000000_0.100000': [0, 1959.00828192892, -0.5887089255698262, -774.4767644818475, 5.350939466450654, 352.4222308742743, 0, 0], '0.001100_103.305785_1.652893_352.796206_0.000000_0.100000': [231.45071629227832, 1536.9132149274003, 0, 316.13200530801964, 0, 0, 0, 0], '0.001100_103.305785_1.652893_320.723824_0.000000_0.100000': [0, 0, 0, 504.19250687772296, 0, 0, 0, 0], '0.001000_103.305785_1.366027_756.250000_0.000000_0.100000': [662.5270338257143, 0, -5.2262828095193985, 0, 0, 0, 0, 0], '0.001100_125.000000_2.000000_388.075827_0.000000_0.100000': [1541.8665645929643, 0, 0, 0, 0, 0, -101.84465934666895, -3.9312773351168744], '0.001100_137.500000_2.000000_388.075827_0.000000_0.100000': [926.9530394319193, 0, 0, 0, 0, 0, 0, -434.02171710854094], '0.001100_137.500000_2.000000_426.883410_0.000000_0.100000': [0, 434.02171710854094, 0, 228.85918974447424, 413.6096447998671, 1840.3619394512066, -134.85339843860714, 0], '0.001210_125.000000_1.818182_388.075827_0.000000_0.100000': [384.89183523569557, 0, 0, 0, 0, 267.8810235447875, 0, 0], '0.001210_113.636364_1.818182_388.075827_0.000000_0.100000': [633.6039777451099, 0, 0, 0, 0, -72.55720343247083, 0, 0], '0.001210_113.636364_1.652893_388.075827_0.000000_0.100000': [2442.2120004386197, 0, 70.21131643914998, -138.62868197911848, 73.63084664264625, 0, 0, 139.70828881302367], '0.001000_151.250000_1.652893_568.181818_0.000000_0.100000': [54.01111359796511, 1556.1350524747081, 312.0051234153868, 386.42107967265054, 6.906752673950674, -13.987963945812027, -369.8086973963418, -455.1316005826251], '0.001000_125.000000_1.652893_426.883410_0.000000_0.100000': [1409.473393975931, 0, 0, 0, 0, -44.12772629631696, 0, 0], '0.001000_137.500000_1.502630_469.571751_0.000000_0.100000': [1234.849090197299, 0, 0, 0, 2.2740795293177145, 0, 0, 0], '0.000909_137.500000_1.502630_469.571751_0.000000_0.100000': [0, 434.0173724469488, 440.18669366358074, 0, 0, 0, 0, 0], '0.000909_137.500000_1.502630_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -396.3979389422872, 0], '0.000909_151.250000_1.502630_426.883410_0.000000_0.100000': [0, 390.97136402344523, 0, 0, 0, 0, 0, 0], '0.000909_151.250000_1.502630_388.075827_0.000000_0.100000': [446.91010763427204, 0, 0, 0, 0, 0, 0, 0], '0.000909_137.500000_1.502630_388.075827_0.000000_0.100000': [0, 0, -4.2729471974379525, 0, 3.883530570055336, 0, 0, 0], '0.001100_125.000000_1.502630_568.181818_0.000000_0.100000': [0, 0, 0, 0, 401.502098895642, 0, 0, -407.71249773839537], '0.001000_125.000000_1.366027_469.571751_0.000000_0.100000': [1549.020335779041, 0, -5.233968976052256, 0, 4.767139994850822, 0, -436.44605194334855, -46.25776928102891], '0.001000_137.500000_1.366027_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -460.81500605676223, 0], '0.001000_151.250000_1.366027_469.571751_0.000000_0.100000': [0, 470.939760320741, 0, -63.64536509622868, 4.7857267538688575, 0, 0, 0], '0.000909_151.250000_1.366027_469.571751_0.000000_0.100000': [0, 0, -4.7857267538688575, 0, 0, 0, 0, 0], '0.001000_151.250000_1.366027_426.883410_0.000000_0.100000': [0, 398.88732288486625, 0, 0, 0, 444.69004066306087, 0, 0], '0.001000_151.250000_1.366027_388.075827_0.000000_0.100000': [0, 0, 0, -688.7366756864546, 0, 0, 0, 0], '0.001000_151.250000_1.502630_388.075827_0.000000_0.100000': [445.98589774385493, 0, 0, 0, 0, 0, 0, 0], '0.001000_125.000000_1.502630_388.075827_0.000000_0.100000': [110.76928036155607, 1233.33838142479, 0, 0, 4.121869913244154, 0, 0, 0], '0.001000_125.000000_1.502630_352.796206_0.000000_0.100000': [2845.1792691985597, 0, 0, 0, 0, 0, 0, -427.309131844223], '0.001000_113.636364_1.502630_352.796206_0.000000_0.100000': [2009.7199070542877, 0, 1980.8480808937393, 0, 0, 0, 0, 0], '0.001000_103.305785_1.502630_352.796206_0.000000_0.100000': [2726.0654594663, 0, 0, 0, 4.484437280489519, 923.8854349271983, 0, -459.6419334796414], '0.001000_103.305785_1.502630_388.075827_0.000000_0.100000': [529.7383893776812, 2319.2286593377294, 99.74256150904762, 486.92837699378623, 143.8247558271878, 0, 591.9654190289712, 679.2425269514421], '0.001100_103.305785_1.502630_388.075827_0.000000_0.100000': [347.2593348499504, 2808.826893507157, -5.567124563876587, -502.63671777321423, 2665.751158758464, 484.6658448833531, 2132.2026240698283, 0], '0.001100_93.914350_1.652893_625.000000_0.000000_0.100000': [456.7275922427632, 2024.1531795556095, 0, 0, 232.88740344200826, 0, 201.55599729412404, -478.21358512524495], '0.000751_113.636364_1.502630_625.000000_0.000000_0.100000': [0, 0, -326.6757054064796, 0, 0, 0, 0, 0], '0.000909_113.636364_1.502630_625.000000_0.000000_0.100000': [0, 0, 0, 636.8347585376675, 0, 0, -318.59759493315215, 0], '0.001000_85.376682_1.652893_469.571751_0.000000_0.100000': [0, 0, 0, 309.81151702225276, 486.53187344082215, 719.0926951025258, 0, 0], '0.000826_85.376682_1.502630_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 1578.2102470680204, 0, 0], '0.000826_85.376682_1.366027_516.528926_0.000000_0.100000': [122.39485915791275, 0, 0, -14.218110333946015, 0, 0, -794.1218431554381, 0], '0.000751_64.144765_1.502630_568.181818_0.000000_0.100000': [0.65720549765324, 0, 0, 0, 0.03795869354204115, 0, 0, 0], '0.000751_58.313423_1.502630_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 0.10913954173824095, 0, 0], '0.000751_58.313423_1.366027_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 0.055619881381392136, 0, 0], '0.000751_58.313423_1.241843_568.181818_0.000000_0.100000': [0, 0.017934752071200818, 0, 0, 0, 0, 0, 0], '0.000751_58.313423_1.241843_516.528926_0.000000_0.100000': [-0.01342207707327816, 0, 0, 0, 0, 0, 0, 0], '0.000683_166.375000_1.652893_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 62.70631923303063, 0, 0], '0.000683_166.375000_1.502630_625.000000_0.000000_0.100000': [278.0943186096083, 0, 0, 0, 0, 0, 0, 0], '0.000683_151.250000_1.502630_625.000000_0.000000_0.100000': [282.8227731379977, 0, 0, 0, 0, 0, 0, 0], '0.000826_166.375000_1.652893_568.181818_0.000000_0.100000': [37.02209080766764, 0, -3.101360345890498, 0, 0, 0, 0, 0], '0.000909_166.375000_1.652893_568.181818_0.000000_0.100000': [372.99219762409905, 464.2655026351736, 0, 0, 0, 0, 0, 0], '0.000909_166.375000_1.652893_516.528926_0.000000_0.100000': [395.107205044997, 0, -3.781628602938872, 0, 0, -32.927229378890296, 0, 0], '0.000909_166.375000_1.502630_516.528926_0.000000_0.100000': [0, 368.4892006883034, 0, 0, 0, 0, 0, 0], '0.000909_166.375000_1.502630_469.571751_0.000000_0.100000': [340.61369744550774, 0, 0, 0, 0, 0, 0, 0], '0.000909_151.250000_1.502630_469.571751_0.000000_0.100000': [0, 0, 0, 0, 1.6961222883783194, 0, 0, 0], '0.000826_151.250000_1.502630_469.571751_0.000000_0.100000': [347.9998832244426, 0, 0, 0, 0, 0, 0, 0], '0.000826_137.500000_1.502630_469.571751_0.000000_0.100000': [0, 0, 0, 0, 1.8527332000077876, 0, 0, 0], '0.000751_137.500000_1.502630_469.571751_0.000000_0.100000': [377.9452456481817, 0, 0, 0, 0, 0, 0, 0], '0.000751_125.000000_1.502630_469.571751_0.000000_0.100000': [0, 0, -3.1173529157788837, 0, 0, 0, 0, 0], '0.000909_137.500000_1.502630_687.500000_0.000000_0.100000': [0, 764.9335927943936, 0, 0, 0, 0, 0, 0], '0.001210_137.500000_1.502630_625.000000_0.000000_0.100000': [0, 0, 0, 0, 1.8490161379970231, 590.3408628552763, 0, 0], '0.001100_137.500000_1.502630_625.000000_0.000000_0.100000': [419.11442990096504, 0, 0, 0, 0, 0, 0, -387.0193342688841], '0.001100_137.500000_1.366027_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0.5342796900142276, 0, 0, 0], '0.001100_77.615165_2.000000_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 1050.8247908669089, 0, 0], '0.001100_77.615165_1.818182_568.181818_0.000000_0.100000': [1708.6292844641416, 2004.0009318453497, -2.917855020089519, 0, 492.8902109978044, 3003.557756594713, 710.6280799940996, 0], '0.001210_77.615165_1.818182_568.181818_0.000000_0.100000': [1709.694362413067, 0, 0, 0, 0, 0, 0, -492.5303626115373], '0.001210_77.615165_1.818182_625.000000_0.000000_0.100000': [0, 0, 0, 0, 3.872663183085297, 0, 0, 0], '0.001100_77.615165_1.818182_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -550.8231705780211], '0.001100_77.615165_1.818182_687.500000_0.000000_0.100000': [0, 0, -5.716737006046424, 0, 0, 0, 1753.2643564671043, 0], '0.001210_77.615165_1.818182_687.500000_0.000000_0.100000': [0, 502.59241071207117, 0, -1062.095010417323, 0, 613.0377939574612, 0, 0], '0.001210_113.636364_1.818182_352.796206_0.000000_0.100000': [0, 0, -4.223329490372271, -428.12890742898384, 0, 0, 0, 0], '0.001210_113.636364_2.000000_352.796206_0.000000_0.100000': [0, 0, 0, -1084.8926328020996, 0, 428.12890742898384, 0, 0], '0.001331_113.636364_1.818182_352.796206_0.000000_0.100000': [0, 0, 0, 0, 0, 174.2020471601285, 0, 0], '0.001331_113.636364_1.652893_352.796206_0.000000_0.100000': [0, 666.7896646627307, -1.9011317184698784, 0, 0, 196.30203448253116, -314.13031499188963, 0], '0.001464_113.636364_1.652893_352.796206_0.000000_0.100000': [0, 0, 0, 0, 2.091244890316866, 0, 0, 0], '0.001331_113.636364_1.502630_352.796206_0.000000_0.100000': [413.5827851962481, 0, -6.451911173980716, 0, 0, 0, 0, 0], '0.001464_113.636364_1.502630_352.796206_0.000000_0.100000': [0, 0, 0, -190.04023648039743, 0, 0, 0, 0], '0.001331_113.636364_1.652893_320.723824_0.000000_0.100000': [527.3157378026107, 0, -1.544179281729339, 0, 0, 0, 0, 0], '0.001464_113.636364_1.652893_320.723824_0.000000_0.100000': [0, 320.1256091411571, 0, 0, 0, 0, 0, 0], '0.001464_113.636364_1.652893_291.567113_0.000000_0.100000': [0, 0, 0, 0, 0, 334.6008085173834, 0, 0], '0.001464_113.636364_1.502630_291.567113_0.000000_0.100000': [422.0220749187853, 0, 0, 0, 0, 0, 0, 0], '0.001464_103.305785_1.502630_291.567113_0.000000_0.100000': [0, 0, 0, 0, 0, 355.05030284450044, 0, 0], '0.001464_103.305785_1.366027_291.567113_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001210_103.305785_1.502630_516.528926_0.000000_0.100000': [0, 0, -5.749184529212653, 0, 0, 0, 0, 0], '0.001331_103.305785_1.502630_516.528926_0.000000_0.100000': [0, 521.2892508062101, 0, 0, 0, 0, 0, -467.542103751174], '0.001331_103.305785_1.502630_625.000000_0.000000_0.100000': [335.30403297697194, 893.8400211161243, -3.8827159131804367, 0, 0, 0, -3.2549586809622157, 0], '0.001464_103.305785_1.502630_625.000000_0.000000_0.100000': [333.7687240737334, 0, 0, 0, 0, 0, 0, 0], '0.001464_93.914350_1.502630_625.000000_0.000000_0.100000': [0, 0, 0, -74.77945304167079, 10.504075179457923, 0, 0, 0], '0.001331_93.914350_1.502630_625.000000_0.000000_0.100000': [0, 0, -5.418024816418984, 0, 5.554080743929717, 0, 0, 0], '0.001464_93.914350_1.652893_625.000000_0.000000_0.100000': [0, 0, 0, -229.9862967795476, 0, 0, 0, -477.98113656042227], '0.001464_93.914350_1.818182_625.000000_0.000000_0.100000': [0, 0, 0, 0, 3.0587982544058834, 0, 0, 0], '0.001331_93.914350_1.818182_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -551.5157214027184], '0.001331_93.914350_1.818182_687.500000_0.000000_0.100000': [1080.9187005319354, 0, 0, -730.9954359009125, 0, 0, -250.72971291931754, 0], '0.001331_85.376682_1.818182_687.500000_0.000000_0.100000': [0, 0, 0, -140.07011352510074, 0, -471.3150474076785, -983.5386374209563, 2136.567615824642], '0.001331_85.376682_1.652893_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 131.18141686075725, 0, 0], '0.001331_85.376682_1.502630_687.500000_0.000000_0.100000': [301.49995942397453, 0, -5.800714289367352, 0, 0, 0, 0, 0], '0.001464_85.376682_1.502630_687.500000_0.000000_0.100000': [301.10926275053697, 0, 0, 0, 0, 0, 819.9195342960522, 0], '0.001210_113.636364_1.502630_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 1341.823748869098, -385.49632141135424, 0], '0.001210_113.636364_1.366027_469.571751_0.000000_0.100000': [0, 486.447817066672, 0, 0, 0, 0, 0, 0], '0.001210_113.636364_1.366027_426.883410_0.000000_0.100000': [0, 455.17332652133996, 0, 0, 5.010275122345229, 0, -424.36859195116995, 0], '0.001100_103.305785_1.366027_426.883410_0.000000_0.100000': [0, 2897.457875351542, 0, -814.6263301359365, 0, 0, 0, -412.7470546318529], '0.001100_103.305785_1.366027_469.571751_0.000000_0.100000': [2074.2098065546033, 0, -5.376676103245245, -908.754084915206, 0, 0, -42.208851169364955, 0], '0.001100_93.914350_1.366027_469.571751_0.000000_0.100000': [0, 2117.437360933648, 0, -928.9462886821683, 0, 0, 123.26968497342818, 0], '0.001100_93.914350_1.366027_426.883410_0.000000_0.100000': [0, 0, 0, 0, 1855.1595330834436, 0, -356.8742570996622, -399.28410002518694], '0.001000_93.914350_1.366027_469.571751_0.000000_0.100000': [0, 438.4484197913433, 0, 0, 0, 0, 0, 0], '0.001000_93.914350_1.366027_426.883410_0.000000_0.100000': [2621.7268467915287, 396.9014766308314, 0, -801.391457566754, 0, 0, -356.27255243551093, 0], '0.001000_85.376682_1.366027_426.883410_0.000000_0.100000': [213.0434792409465, 1800.2655708261125, 8.927030603460935, -1133.1093201682909, 0, 0, 0, 0], '0.001000_77.615165_1.366027_426.883410_0.000000_0.100000': [0, 0, 0, 2442.8246413136117, 6.024990733279383, 0, 0, -358.1528435575748], '0.000909_77.615165_1.366027_426.883410_0.000000_0.100000': [0, 340.491566364808, 0, 0, 0, 0, -194.88655095018157, 0], '0.000909_77.615165_1.366027_388.075827_0.000000_0.100000': [2097.853878600936, 0, 1517.1291986275962, -205.88305183563133, 0, 0, 0, 0], '0.000909_77.615165_1.502630_388.075827_0.000000_0.100000': [2202.4969594025556, 0, 0, 0, 0, 0, 0, 0], '0.001100_103.305785_1.241843_516.528926_0.000000_0.100000': [0, 0, 0, 0, 1097.469464873177, 0, 0, -400.20103381387116], '0.000683_103.305785_2.000000_687.500000_0.000000_0.100000': [-1079.0682936263513, -808.9604805992922, 0, 0, 0, 0, 0, -2014.1552929063732], '0.000683_113.636364_2.000000_756.250000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -597.3414154158635], '0.000683_113.636364_2.000000_831.875000_0.000000_0.100000': [0, 0, 0, 0, 3.7715289852149, 0, 0, 0], '0.000621_113.636364_2.000000_831.875000_0.000000_0.100000': [0, 0, 0, 0, 3.429035568170184, 0, 0, 0], '0.000564_113.636364_2.000000_831.875000_0.000000_0.100000': [0, 0, 0, 0, 3.117731596336489, 0, 0, 0], '0.000513_113.636364_2.000000_831.875000_0.000000_0.100000': [0, 0, 0, 0, 0, 969.6442643433498, 0, -603.3733114548073], '0.000513_113.636364_2.000000_915.062500_0.000000_0.100000': [0, 603.3733114548073, 0, 0, 0, 0, 0, 0], '0.000513_113.636364_1.818182_831.875000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000826_137.500000_1.366027_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -462.37627479657294, -443.8764350057383], '0.000826_137.500000_1.366027_687.500000_0.000000_0.100000': [536.2698588545016, 0, 0, 0, 0, 0, -454.0472856341083, 0], '0.000826_103.305785_1.366027_625.000000_0.000000_0.100000': [1239.1189878739078, 0, 0, 0, 4.142608178081082, 0, 0, -181.40260946144932], '0.000826_93.914350_1.366027_625.000000_0.000000_0.100000': [0, 577.200120671884, 0, 0, 0, 0, 47.42461859716172, 0], '0.000826_93.914350_1.366027_568.181818_0.000000_0.100000': [0, 1178.7556641299466, 0, 0, 93.85147648526203, 0, -370.70679003298, 0], '0.000751_93.914350_1.366027_568.181818_0.000000_0.100000': [0, 510.1451536742631, 0, 0, 0, 58.351650432736115, 0, 0], '0.000751_93.914350_1.241843_568.181818_0.000000_0.100000': [1157.9854643189628, 407.0245595522247, -1.0227524222703324, 0, 0, 0, 0, 0], '0.000751_93.914350_1.241843_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, -253.91834920315725, 0, 0], '0.000751_93.914350_1.128948_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0.13093437747538703, 0, 0, -426.88865616782635], '0.000751_93.914350_1.128948_568.181818_0.000000_0.100000': [0, 426.88865616782635, 0, 0, 0, 0, 0, 0], '0.000683_93.914350_1.128948_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -426.3740937382851], '0.000683_93.914350_1.128948_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001000_151.250000_1.502630_625.000000_0.000000_0.100000': [1320.3456202932762, 0, 0, 0, 0, 0, -279.5598373324216, 0], '0.001000_166.375000_1.502630_625.000000_0.000000_0.100000': [0, 0, 0, 0, 1.0080071155211046, 0, 0, 0], '0.000909_166.375000_1.502630_625.000000_0.000000_0.100000': [0, 378.75103371469794, 0, 0, 0, 0, 0, 0], '0.000909_166.375000_1.502630_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -291.14205049938465, 0], '0.000909_183.012500_1.502630_568.181818_0.000000_0.100000': [0, 0, 0, -121.4685592378779, 0, 0, 0, 0], '0.000909_183.012500_1.652893_568.181818_0.000000_0.100000': [777.8741823572482, 0, -3.8916866668263537, 0, 0, 0, 0, 0], '0.001000_183.012500_1.652893_568.181818_0.000000_0.100000': [0, 0, 0, 0, 3.8916866668263537, 0, 0, 0], '0.000909_113.636364_2.420000_469.571751_0.000000_0.100000': [286.1604754134973, 455.84763535152905, 0, 0, 0, 0, 0, 0], '0.000909_103.305785_2.420000_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 955.3001404606039, 0, 0], '0.001000_113.636364_2.200000_352.796206_0.000000_0.100000': [0, 0, 0, 0, 5.2090476255606974, 0, 0, 0], '0.000909_113.636364_2.200000_352.796206_0.000000_0.100000': [408.479089523699, 0, 0, -897.3222245763643, 0, 0, 0, 0], '0.000909_103.305785_2.200000_352.796206_0.000000_0.100000': [407.8678767315073, 0, 0, 0, 0, 0, 0, 0], '0.000909_93.914350_2.200000_352.796206_0.000000_0.100000': [407.9140482021786, 0, 0, 0, 4.940001961894268, 0, 0, 0], '0.000826_93.914350_2.200000_352.796206_0.000000_0.100000': [0, 0, 0, 0, 0, 1093.3986650333236, 0, 0], '0.000826_93.914350_2.000000_352.796206_0.000000_0.100000': [0, 0, -3.751440804205868, 0, 3.400966528063236, 0, 0, 0], '0.000751_93.914350_2.000000_352.796206_0.000000_0.100000': [0, 0, 0, 0, 0, 255.49498298500905, 0, 0], '0.000751_93.914350_1.818182_352.796206_0.000000_0.100000': [0, 0, 0, 0, 1.8207874621695737, 0, 0, 0], '0.000683_93.914350_1.818182_352.796206_0.000000_0.100000': [245.10103788329835, 0, 0, 0, 0, 0, 0, 0], '0.000683_85.376682_1.818182_352.796206_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -24.51010378832983, 0], '0.000909_85.376682_1.366027_516.528926_0.000000_0.100000': [0, 486.39110503397563, -3.5326706804007517, 0, 0, 0, 0, 0], '0.000909_85.376682_1.366027_469.571751_0.000000_0.100000': [0, 0, 2505.741489550149, -1280.0276749019338, 22.996195160269053, 0, 0, 0], '0.000826_85.376682_1.366027_469.571751_0.000000_0.100000': [185.4274128913696, 0, 0, -1298.6169885633276, 0, 0, -736.2459218942248, 0], '0.000826_93.914350_1.502630_469.571751_0.000000_0.100000': [369.41818257546197, 0, 0, 0, 333.20317447085773, 0, 0, 0], '0.000751_77.615165_1.502630_469.571751_0.000000_0.100000': [2747.316525500756, 0, -1.2335047741590643, 0, 0, 0, 0, 0], '0.000826_77.615165_1.502630_469.571751_0.000000_0.100000': [2689.3449932938797, 0, 0, -1312.8778095837029, 0, 0, 0, -1333.8830773715786], '0.000826_77.615165_1.652893_469.571751_0.000000_0.100000': [1212.2693460492642, 390.4134888843391, -0.43044966398920226, 0, 0, 0, 0, 0], '0.000909_77.615165_1.652893_469.571751_0.000000_0.100000': [0, 390.2539578433781, 0, 0, 0, 0, 0, 0], '0.000909_77.615165_1.652893_426.883410_0.000000_0.100000': [1119.2652397353859, 0, 0, 0, 0, 0, 0, 0], '0.000909_70.559241_1.652893_426.883410_0.000000_0.100000': [3000.1440696018567, 0, 0, 866.4957519830788, 0, 0, 0, 0], '0.000909_64.144765_1.652893_426.883410_0.000000_0.100000': [-0.032936897997933555, 0, -0.01887473378769755, 0, 0, 0, 0, 0], '0.001210_93.914350_1.818182_625.000000_0.000000_0.100000': [0, 0, -454.5739618532283, 0, 3.1901170582703346, 229.93971968054993, 0, 0], '0.001464_103.305785_1.818182_687.500000_0.000000_0.100000': [0, 0, 0, 604.1187805167538, 4.43596443063816, 0, 0, -533.1989439522446], '0.001331_103.305785_2.000000_687.500000_0.000000_0.100000': [-1086.5903469888615, -297.4740518850074, -6.6418079211509395, -2643.5056824965645, -970.6337052086541, -121.66416160021066, -660.2818597183756, -2021.7681191559473], '0.001464_103.305785_1.818182_756.250000_0.000000_0.100000': [0, 533.1989439522446, 0, 0, 0, 0, 0, 0], '0.001464_103.305785_2.000000_687.500000_0.000000_0.100000': [-1088.1331409602913, 0, 0, -2644.1771971213143, 0, 0, 0, 0], '0.001464_103.305785_2.200000_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001100_113.636364_2.662000_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 525.9776515075617, 0, 0], '0.000909_85.376682_2.420000_687.500000_0.000000_0.100000': [0, 0, 184.84227431804024, 0, 0, 0, 0, 0], '0.001000_85.376682_2.420000_756.250000_0.000000_0.100000': [0, 0, -3.891703561613031, 0, 0, 0, 0, 0], '0.000751_113.636364_1.818182_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 230.50662231494957, 0, 0], '0.000751_113.636364_1.652893_469.571751_0.000000_0.100000': [0, 425.2548471427098, 1609.132675280714, 0, 0, 0, 0, -213.5126876061351], '0.000751_103.305785_1.652893_469.571751_0.000000_0.100000': [241.86193730192235, 0, 0, 0, 0, 0, -267.65204702938445, 0], '0.001000_103.305785_2.420000_426.883410_0.000000_0.100000': [1270.8310197296964, 0, 0, -174.01831235869486, 0, 0, 0, -470.2789395808779], '0.001000_103.305785_2.662000_426.883410_0.000000_0.100000': [256.6991745581061, 0, 0, 0, 0, 174.01831235869486, 0, 0], '0.001100_93.914350_2.662000_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000909_93.914350_1.502630_426.883410_0.000000_0.100000': [0, 0, 1405.8167920243907, 0, 0, 0, 0, 0], '0.000909_85.376682_1.366027_426.883410_0.000000_0.100000': [0, 0, 0, 0, 361.193522465303, 0, 0, 0], '0.000826_85.376682_1.366027_426.883410_0.000000_0.100000': [31.91666191485723, 0, 0, -1129.8235241288503, 0, 0, 0, 0], '0.000826_85.376682_1.502630_426.883410_0.000000_0.100000': [1287.9370655031532, 0, 0, 0, 0, 0, 1240.0912465463512, -498.2333533784702], '0.000683_85.376682_1.502630_469.571751_0.000000_0.100000': [0, 501.57290758080245, 0, 0, 0, 0, 0, -195.90535706606804], '0.000683_85.376682_1.502630_426.883410_0.000000_0.100000': [0, 0, 0, 0, 682.3716578265534, 0, 0, 0], '0.000621_85.376682_1.502630_426.883410_0.000000_0.100000': [0, 0, 0, -1142.8412538958119, 0, 0, 0, 0], '0.000621_85.376682_1.652893_426.883410_0.000000_0.100000': [0, 1090.334071399102, 39.56014796781336, 0, 0, 0, 0, 0], '0.000621_85.376682_1.652893_388.075827_0.000000_0.100000': [679.6604728344416, 0, 0, 0, 0, 0, 0, 0], '0.001331_85.376682_1.652893_469.571751_0.000000_0.100000': [224.10818171922583, 0, 0, 0, 0, 0, 0, 0], '0.001210_151.250000_1.652893_568.181818_0.000000_0.100000': [0, 0, 0, 68.79139581898251, 0, 0, -369.40016576288184, -872.2427515357211], '0.001210_166.375000_1.652893_568.181818_0.000000_0.100000': [0, 0, 0, -366.61273717111726, 0, 0, 0, 0], '0.001331_151.250000_1.818182_568.181818_0.000000_0.100000': [389.10492366499983, 0, 0, 0, 0, 0, 0, 0], '0.001331_137.500000_1.818182_625.000000_0.000000_0.100000': [-446.93055675235246, 460.5725398805666, 0, -938.5830862630121, 0, 0, 0, -482.91183271084486], '0.001331_137.500000_2.000000_625.000000_0.000000_0.100000': [0, 0, -8.305066846918999, 0, 0, 0, 0, 0], '0.001464_137.500000_2.000000_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -525.7416079628304], '0.001464_137.500000_2.000000_687.500000_0.000000_0.100000': [386.02831772156986, 0, 0, 0, 0, 0, 0, 0], '0.001464_125.000000_2.000000_687.500000_0.000000_0.100000': [0, 550.8796515693933, 0, 0, 0, 0, 0, 0], '0.001464_125.000000_2.000000_625.000000_0.000000_0.100000': [0, 0, -9.139550467657319, 0, 0, 0, 0, 0], '0.001611_125.000000_2.000000_625.000000_0.000000_0.100000': [0, 0, -10.054558926840764, 0, 0, 880.9734292725203, 0, 0], '0.001772_125.000000_2.000000_625.000000_0.000000_0.100000': [0, 0, 0, 0, 10.054558926840764, 0, 0, 0], '0.001611_113.636364_1.818182_625.000000_0.000000_0.100000': [0, 0, 0, -807.0586618189042, 0, 0, 0, 0], '0.001611_113.636364_2.000000_625.000000_0.000000_0.100000': [0, 557.4578548860376, 0, 0, 0, 0, 0, 0], '0.001611_113.636364_2.000000_568.181818_0.000000_0.100000': [0, 0, 0, -1254.9725725533212, 9.07736369589602, 0, 0, 0], '0.001611_113.636364_2.200000_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 1254.9725725533212, 0, 0], '0.001464_113.636364_2.000000_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000909_85.376682_1.652893_516.528926_0.000000_0.100000': [-784.237942952951, -597.963096239544, -596.0459424005478, -66.52670752954296, 174.24559190634136, -741.5749591697773, -1267.991542196995, 0], '0.001000_93.914350_1.366027_388.075827_0.000000_0.100000': [662.2579011720503, 321.3698617113062, 0, 0, 0, -100.261357629746, 0, 0], '0.001000_93.914350_1.241843_388.075827_0.000000_0.100000': [215.85940127784633, 0, 0, 0, 0, -248.21066224398393, 0, 0], '0.001000_93.914350_1.128948_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001000_151.250000_2.000000_388.075827_0.000000_0.100000': [0, 0, 0, 0, 5.326305705193954, 0, 0, 0], '0.000909_151.250000_2.000000_388.075827_0.000000_0.100000': [0, 428.19915987188637, 0, 0, 0, 0, 0, 0], '0.000909_151.250000_2.000000_352.796206_0.000000_0.100000': [0, 0, -0.497080405340039, 0, 0, 1124.7562900548203, 0, 0], '0.000909_137.500000_1.652893_352.796206_0.000000_0.100000': [361.01224850175333, 0, 0, -31.99266583835115, 0, 0, 0, 0], '0.000909_125.000000_1.652893_352.796206_0.000000_0.100000': [0, 0, -1.5973035052749764, 0, 0, 0, 0, 0], '0.001000_125.000000_1.652893_352.796206_0.000000_0.100000': [0, 347.00616007826096, 279.9389920190181, 0, 0, 0, 0, 0], '0.001000_125.000000_1.652893_320.723824_0.000000_0.100000': [0, 0, 0, 0, 1.4684192828744926, 0, 0, 0], '0.000909_125.000000_1.652893_320.723824_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -360.2073606795329, 0], '0.000909_137.500000_1.652893_320.723824_0.000000_0.100000': [0, 0, 0, 0, 0, 28.955170127723797, 0, 0], '0.000909_137.500000_1.502630_320.723824_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -483.5923641062059, 0], '0.000909_151.250000_1.502630_320.723824_0.000000_0.100000': [0, 0, 0, 47.75924638360311, 0, 0, 0, 0], '0.000909_151.250000_1.652893_320.723824_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000683_113.636364_1.652893_426.883410_0.000000_0.100000': [273.93427103856374, 0, 0, -213.92494701585602, 0, 0, 0, 0], '0.000683_113.636364_1.818182_426.883410_0.000000_0.100000': [346.7773661442716, 0, 0, 0, 0, 0, 0, 0], '0.001000_125.000000_1.366027_568.181818_0.000000_0.100000': [0, 0, 0, -811.9374761128198, 5.2838110687072914, 1358.1747191503046, 466.9724408174633, 0], '0.001464_93.914350_1.818182_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -310.86188826206416, 0], '0.001464_103.305785_1.818182_388.075827_0.000000_0.100000': [0, 402.61783742741545, 0, 0, 0, 0, 0, 0], '0.001464_103.305785_1.818182_352.796206_0.000000_0.100000': [0, 0, 0, 0, 0, 84.90004961393097, 0, -40.26178374274155], '0.000826_93.914350_1.652893_625.000000_0.000000_0.100000': [0, 0, 0, 0, 211.72626953429375, 0, 0, 0], '0.000683_85.376682_1.818182_625.000000_0.000000_0.100000': [220.57851428932184, 0, 0, 0, 0, 0, 0, 0], '0.000683_77.615165_1.818182_625.000000_0.000000_0.100000': [4965.231236609493, 490.7164997241204, 0, 0, 0, 0, 0, 0], '0.001100_93.914350_1.241843_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, -300.89019812379, 0, -420.4378450289284], '0.001100_93.914350_1.128948_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -450.73450169417185], '0.001100_93.914350_1.128948_687.500000_0.000000_0.100000': [0, 0, -1.0248858487841062, 0, 0, 0, 0, 0], '0.001210_93.914350_1.128948_687.500000_0.000000_0.100000': [100.03459139981715, 0, 0, 0, 0, 0, 0, 0], '0.001210_85.376682_1.128948_687.500000_0.000000_0.100000': [1574.478826564408, 0, 0, 0, 0, 0, 0, 0], '0.001210_77.615165_1.128948_687.500000_0.000000_0.100000': [0, 0, 0, 1008.5427046930195, 0, 0, 0, 0], '0.001210_77.615165_1.241843_687.500000_0.000000_0.100000': [0, -710.134198511349, 0, 0, 0, 0, 0, 0], '0.001210_77.615165_1.241843_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001100_85.376682_1.818182_516.528926_0.000000_0.100000': [2975.7691307051846, 3989.4961788330957, 19.480883657148297, -1434.4843093438687, -62.714253779128285, -969.3982643303644, -1353.2248209143381, -478.19225073655934], '0.000909_77.615165_1.818182_516.528926_0.000000_0.100000': [0, 462.5072132335203, 0, 0, 0, 0, 0, 0], '0.000909_77.615165_1.818182_469.571751_0.000000_0.100000': [0, 0, 0, 0, 2.849534330199185, 0, 0, 0], '0.000826_70.559241_1.652893_469.571751_0.000000_0.100000': [0, 296.2118008977645, 0, 0, 0, 0, 0, 0], '0.000826_70.559241_1.652893_426.883410_0.000000_0.100000': [0, 299.04173605170894, 0, 0, 0, 0, 0, 0], '0.000826_70.559241_1.652893_388.075827_0.000000_0.100000': [2401.9591904080103, 0, 0, 0, 0, 0, 0, 0], '0.000826_64.144765_1.652893_388.075827_0.000000_0.100000': [-0.01848909697732779, 0, 0, -0.6067085981635046, 0, 0, 0, 0], '0.000826_58.313423_1.652893_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0.01848909697732779, 0], '0.000826_64.144765_1.818182_388.075827_0.000000_0.100000': [0, 0.595972229218387, 0, 0, -0.1057586830300579, 0, 0, 0], '0.000751_64.144765_1.818182_388.075827_0.000000_0.100000': [0.7285686071304054, 0, 0, 0, 0, 0, -10.109238151012137, 0], '0.000751_58.313423_1.818182_388.075827_0.000000_0.100000': [0, 0, 0, 0.7210309590254292, 0, 0, 0, 0], '0.000751_58.313423_2.000000_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0.00499553109460976, 0, 0, 0], '0.000683_58.313423_2.000000_388.075827_0.000000_0.100000': [-0.7188608078443622, 0, 0, 0, 0, 0, -0.8763879249930142, 0], '0.000683_64.144765_2.000000_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001331_137.500000_2.000000_687.500000_0.000000_0.100000': [424.6645453867003, 0, 0, 0, 0, 0, 0, 0], '0.001331_125.000000_1.818182_687.500000_0.000000_0.100000': [0, 553.0911493802171, 0, 0, 0, 0, 0, 0], '0.001331_125.000000_1.652893_516.528926_0.000000_0.100000': [320.77552388469445, 0, 0, 0, 0, 0, 0, 0], '0.001464_103.305785_1.652893_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 13.309167010462534, 0, 0], '0.001464_103.305785_1.502630_516.528926_0.000000_0.100000': [0, 0, 0, 0, 6.344574934580578, 0, 0, -466.2670057000535], '0.001331_103.305785_1.502630_469.571751_0.000000_0.100000': [0, 0, 0, -81.7995853254406, 0, 0, 0, 0], '0.001000_70.559241_2.000000_516.528926_0.000000_0.100000': [0, 5159.275846731602, 0, 0, 0, 0, 0, 0], '0.001000_70.559241_2.000000_469.571751_0.000000_0.100000': [40.35676198483449, 0, -4.375810670552233, 0, 0, 0, -2517.296086843554, 0], '0.001100_70.559241_2.000000_469.571751_0.000000_0.100000': [45.193083321909654, -1924.3194874188068, 0, 0, 0, 0, 0, -4658.033875868337], '0.001100_70.559241_2.000000_426.883410_0.000000_0.100000': [0, 0, 0, 0, 1818.2174195601197, 0, 0, 0], '0.001000_70.559241_2.000000_426.883410_0.000000_0.100000': [0, 0, 0, -1150.5901309325295, 0, 0, 0, 0], '0.001000_70.559241_2.200000_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -2755.1300832961856, 0], '0.001000_77.615165_2.200000_426.883410_0.000000_0.100000': [0, -442.40072257209414, 0, 0, 0, 1492.0316120005402, 0, 0], '0.001000_77.615165_2.000000_388.075827_0.000000_0.100000': [3030.1806061337484, 0, 0, -162.02524931909088, 0, 0, 0, -365.9891560905036], '0.001000_77.615165_2.000000_426.883410_0.000000_0.100000': [0, 0, -3.7624478379334505, 0, 0, -515.0848261812108, 0, -1741.8671263809883], '0.001000_64.144765_2.000000_469.571751_0.000000_0.100000': [0, 0, 0, 0.6844200352342125, 0, -0.06488382480256405, 0, 0], '0.001000_64.144765_2.200000_469.571751_0.000000_0.100000': [-0.759166086184473, -0.000619371217430853, -0.7844346645070573, -0.7450813453741149, -0.6221592673705573, 0, 0, 0], '0.001100_64.144765_2.200000_469.571751_0.000000_0.100000': [0, 0.7762593160036204, 0, 0, 0, 0.06394535913489108, 0, 0], '0.001100_64.144765_2.200000_426.883410_0.000000_0.100000': [0, -0.0009907460030490257, 0, 0, 0, 0.023649996462847458, 0, 0], '0.001100_64.144765_2.000000_426.883410_0.000000_0.100000': [-0.06860095561123672, -0.0008758305804510586, -0.6957220041518667, -0.0002128519051198062, -0.6015127146881468, 0, 0, 0], '0.001210_64.144765_2.000000_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -36.9332895710108, 0], '0.001210_70.559241_2.000000_426.883410_0.000000_0.100000': [0, 0, 11.595158075560084, -3466.723040026309, 0, 0, -3210.9004425447583, 0], '0.001210_70.559241_2.200000_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, 3466.723040026309, 0, 0], '0.001210_77.615165_2.000000_426.883410_0.000000_0.100000': [3210.9004425447574, 0, 0, 0, 0, -515.4751710775357, 0, 0], '0.001210_77.615165_1.818182_426.883410_0.000000_0.100000': [0, 0, 0, 515.4751710775373, 0, 0, 0, 0], '0.001000_77.615165_1.502630_469.571751_0.000000_0.100000': [0, 2965.547049575859, 0, 0, 6.731742886412303, 311.5754378260305, 0, -1319.3449344243295], '0.000909_77.615165_1.502630_469.571751_0.000000_0.100000': [0, 0, 2880.7211364872724, 0, 0, 0, -1159.6840825590718, 0], '0.001000_77.615165_1.502630_426.883410_0.000000_0.100000': [2610.6656754866426, 0, 0, 0, 0, 0, 0, 0], '0.000826_103.305785_2.000000_469.571751_0.000000_0.100000': [0, 0, 0, -1269.0943079592105, 0, 0, 0, 0], '0.000826_93.914350_2.200000_426.883410_0.000000_0.100000': [0, 0, 356.1779944761284, -1061.3237567796346, 0, 0, 0, 0], '0.000826_93.914350_2.420000_426.883410_0.000000_0.100000': [0, 544.066404438354, 0, -229.2473965609987, 0, 0, 0, -45.629247260971816], '0.000826_93.914350_2.662000_426.883410_0.000000_0.100000': [239.99109092181152, 0, 0, 0, 0, 0, 0, 0], '0.000826_85.376682_2.662000_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -579.3289254266505], '0.000826_85.376682_2.662000_469.571751_0.000000_0.100000': [2339.3797914222105, 0, 0, 0, 0, 0, -236.13732192839726, 0], '0.000826_93.914350_2.662000_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 327.58940796988026, 0, 0], '0.000826_93.914350_2.420000_469.571751_0.000000_0.100000': [0, 314.355668138964, -1.9716495602999202, 0, 0, 0, 0, 0], '0.001100_151.250000_1.652893_388.075827_0.000000_0.100000': [0, 0, 0, -383.56591590794636, 3.168269119602828, 0, 0, 0], '0.001000_151.250000_1.652893_388.075827_0.000000_0.100000': [1428.3716285664082, 353.6490105199894, -2.880244654184389, 0, 2.595662811400689, 0, 0, 0], '0.001000_151.250000_1.652893_352.796206_0.000000_0.100000': [0, 0, 0, -380.7258973011997, 0, 0, -434.35627029696036, 0], '0.001000_166.375000_1.652893_352.796206_0.000000_0.100000': [434.35627029696036, 0, 0, 0, 0, -114.73076966810514, 0, 0], '0.001000_166.375000_1.502630_352.796206_0.000000_0.100000': [0, 0, 0, 114.73076966810514, 0, 0, 0, 0], '0.001000_151.250000_1.818182_352.796206_0.000000_0.100000': [0, 0, 0, -649.8194082112312, 0, 0, 0, 0], '0.001000_151.250000_2.000000_352.796206_0.000000_0.100000': [0, 0, 0, 0, 589.1489304780762, 0, 0, 0], '0.001210_93.914350_1.502630_388.075827_0.000000_0.100000': [944.8665557415737, 0, 0, -405.4528729573057, 0, 0, 0, 0], '0.001210_93.914350_1.652893_388.075827_0.000000_0.100000': [0, 0, 0, 0, 1.4392486178063337, 0, 0, 0], '0.001100_93.914350_1.652893_352.796206_0.000000_0.100000': [247.70570449674787, 370.00448930421055, 0, 0, 0, 1927.098873273912, -231.45071629227832, 0], '0.001100_85.376682_1.652893_352.796206_0.000000_0.100000': [231.43937672565232, 0, 0, 0, 0, 0, -2.231582923394137, -397.43912798850664], '0.001100_77.615165_1.652893_352.796206_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -21.039943338695657, 0], '0.000621_93.914350_1.652893_568.181818_0.000000_0.100000': [0, 1612.5505436268222, 0, 0, 0, 0, 0, -449.0563702676057], '0.000621_93.914350_1.818182_516.528926_0.000000_0.100000': [0, 0, 0, 0, 1.4438933597952202, 0, 0, 0], '0.000564_93.914350_1.818182_516.528926_0.000000_0.100000': [1358.0319280591445, 0, 0, 0, 0, 0, 0, 0], '0.000564_85.376682_1.818182_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -396.30955444278715], '0.000564_85.376682_1.818182_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000751_103.305785_1.366027_625.000000_0.000000_0.100000': [366.1174120232261, 0, 0, 0, 0, 0, 0, 0], '0.000751_93.914350_1.366027_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 124.63877534718067, 0, 0], '0.000751_93.914350_1.241843_625.000000_0.000000_0.100000': [0, 413.7684268269248, 0, 0, 0, 0, 0, 0], '0.000751_85.376682_1.241843_568.181818_0.000000_0.100000': [-731.5809234956594, -458.0819160658346, 0, 0, 0, 0, -105.27140584717846, 0], '0.000751_77.615165_1.241843_568.181818_0.000000_0.100000': [0, 0, 0, 217.5724604901903, 0, 0, 0, 0], '0.000751_77.615165_1.366027_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 667.825360046106, 0], '0.000751_85.376682_1.366027_568.181818_0.000000_0.100000': [0, 335.91431722556433, 0, 0, 0, 0, 0, 0], '0.000751_85.376682_1.366027_516.528926_0.000000_0.100000': [138.01535178069292, 0, 0, 0, 0, -606.7104443792326, 0, 0], '0.000751_77.615165_1.366027_516.528926_0.000000_0.100000': [2889.2738753466388, 418.03239426079216, -1.9865279856203415, 0, 0, -619.0287638445541, 0, -465.3289251793479], '0.000751_77.615165_1.241843_516.528926_0.000000_0.100000': [3594.9588599198337, 0, -0.6429884474016034, 0, 0, 0, 0, 0], '0.000826_77.615165_1.241843_516.528926_0.000000_0.100000': [0, 0, 0, 3213.69828444515, 0, 0, 0, 0], '0.001100_151.250000_1.502630_687.500000_0.000000_0.100000': [291.12195323021115, 0, -1.7126586363165188, 0, 1.5264879813717926, 0, 0, 0], '0.001000_166.375000_1.652893_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -373.4148616374159, 0], '0.001000_183.012500_1.652893_687.500000_0.000000_0.100000': [0, 0, -4.743383591772909, 0, 0, 0, 0, 0], '0.001100_183.012500_1.652893_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -434.72667020475274], '0.001100_183.012500_1.652893_756.250000_0.000000_0.100000': [332.7652130754095, 0, 0, 0, 0, 0, 0, 0], '0.001100_166.375000_1.652893_756.250000_0.000000_0.100000': [331.52041677378423, 0, 0, 0, 0, 0, 0, 0], '0.001210_151.250000_1.502630_756.250000_0.000000_0.100000': [559.658022124513, 0, 0, 0, 0, 0, 0, 0], '0.001210_125.000000_1.366027_831.875000_0.000000_0.100000': [0, 0, -6.263627739299227, 0, 0, 0, 0, 0], '0.001331_125.000000_1.366027_831.875000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001210_103.305785_1.818182_388.075827_0.000000_0.100000': [109.702338689069, 0, 0, 0, 0, 0, 300.2065247420355, 0], '0.001210_93.914350_1.818182_388.075827_0.000000_0.100000': [299.4271829557585, 0, 0, 623.251518149446, 0, 0, 0, 0], '0.001210_93.914350_2.000000_388.075827_0.000000_0.100000': [1151.1181511203326, 0, 0, -1164.2226643715733, 0, 0, 0, 0], '0.001210_93.914350_2.200000_388.075827_0.000000_0.100000': [1378.721031744223, 0, 0, 0, 0, 0, -429.3509045685091, 0], '0.001210_103.305785_2.200000_388.075827_0.000000_0.100000': [429.3509045685091, 0, 0, 0, 0, 0, -401.82084258423896, 0], '0.001210_113.636364_2.200000_388.075827_0.000000_0.100000': [0, 560.7731707255185, 0, 0, 0, 0, 0, 0], '0.001210_113.636364_2.200000_352.796206_0.000000_0.100000': [407.16754123023173, 471.64914080123907, 0, 0, 0, 0, -414.90979507101537, 0], '0.001210_125.000000_2.200000_352.796206_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -495.2613848803698], '0.001210_125.000000_2.200000_388.075827_0.000000_0.100000': [0, 0, 0, 0, 6.0341223967910995, 0, 0, 0], '0.001100_125.000000_2.200000_388.075827_0.000000_0.100000': [400.58959143701253, 0, 0, 0, 0, 0, 0, 0], '0.001100_113.636364_2.200000_388.075827_0.000000_0.100000': [0, 0, 452.56808696071624, 0, 0, 0, 0, 0], '0.001210_103.305785_2.200000_352.796206_0.000000_0.100000': [406.8798246356714, 0, 0, 0, 0, 0, 0, 0], '0.001210_93.914350_2.200000_352.796206_0.000000_0.100000': [0, 0, -7.243063182101014, 0, 0, 0, 0, 0], '0.001331_93.914350_2.200000_352.796206_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -406.47581124440467, 0], '0.001331_103.305785_2.200000_352.796206_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001000_113.636364_1.366027_469.571751_0.000000_0.100000': [0, 2172.3789599728575, -5.210551812841823, 0, 4.726194581714572, 0, -436.2174992991092, 0], '0.001000_113.636364_1.366027_426.883410_0.000000_0.100000': [819.5076807304041, 579.6886235833081, 0, 0, 4.118749691988614, 0, -424.17042189528496, -156.06409176367094], '0.001100_113.636364_1.366027_469.571751_0.000000_0.100000': [464.29736286301437, 0, 0, 0, 0, 271.9458670181013, 0, 0], '0.001100_103.305785_1.502630_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 1723.0138030699625, 0, 0], '0.001000_85.376682_1.366027_388.075827_0.000000_0.100000': [211.60474044422745, 569.1893728560825, 0, 0, 1.8702003338956275, 0, 0, 11.759927579996486], '0.000909_70.559241_1.366027_388.075827_0.000000_0.100000': [19.579753544658917, -822.7631059524427, 0, 0, 0, 0, 0, 0], '0.000909_70.559241_1.366027_352.796206_0.000000_0.100000': [840.7578156716572, 0, 0, 0, -31.507494649752516, 0, 0, 0], '0.000826_70.559241_1.366027_352.796206_0.000000_0.100000': [969.1806923627935, 0, 0, 0, 0, 0, 0, 0], '0.001100_85.376682_1.818182_469.571751_0.000000_0.100000': [4934.045892933599, 0, 0, 548.9342384794984, 0, 0, 0, -51.50016897140438], '0.001100_85.376682_2.200000_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 1822.3949980005582, 0, 0], '0.001000_85.376682_2.000000_426.883410_0.000000_0.100000': [1058.3508196286032, 0, 0, 0, 0, 0, 0, 0], '0.001000_85.376682_2.200000_469.571751_0.000000_0.100000': [0, 553.9841860451246, 0, 0, 0, 1997.571374126348, 0, 0], '0.000909_64.144765_2.200000_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001100_151.250000_2.200000_568.181818_0.000000_0.100000': [369.3616356977139, 0, 0, 0, 0, 0, 0, 0], '0.001000_93.914350_1.652893_516.528926_0.000000_0.100000': [1529.9264804923578, 0, 0, 0, 0, -5.742434033747486, 2067.3054831535915, 0], '0.001000_137.500000_1.652893_388.075827_0.000000_0.100000': [869.580962903575, 0, 0, 0, 0, 0, 0, 0], '0.000826_113.636364_1.652893_756.250000_0.000000_0.100000': [0, 0, 0, 212.86323043100177, 0, 0, 0, 0], '0.001331_113.636364_1.502630_625.000000_0.000000_0.100000': [431.710738292986, 0, 0, 0, 0, 0, 0, 0], '0.001210_93.914350_1.366027_568.181818_0.000000_0.100000': [2080.120132199525, 0, 0, 0, 0, 0, 0, 0], '0.001210_85.376682_1.366027_568.181818_0.000000_0.100000': [0, -1091.8180724940325, 0, 0, 0, 0, 0, 0], '0.001210_85.376682_1.366027_516.528926_0.000000_0.100000': [0, 0, 0, -1152.0629938423606, 0, 0, 0, 0], '0.001210_85.376682_1.502630_516.528926_0.000000_0.100000': [0, 0, -6.392543199516422, 0, 0, 0, 0, 0], '0.001331_85.376682_1.502630_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 541.520788273083, 0], '0.000826_125.000000_2.200000_426.883410_0.000000_0.100000': [389.0430524294612, 947.1222325425092, 0, 0, 4.348012360556459, 0, 0, 0], '0.000751_137.500000_2.420000_388.075827_0.000000_0.100000': [0, 0, 0, 0, 1.2906562363552438, 0, 0, 0], '0.000683_137.500000_2.420000_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -247.58643277975008, 0], '0.000683_151.250000_2.420000_388.075827_0.000000_0.100000': [0, 0, 0, -16.990429480573948, 0, 0, 0, 0], '0.000683_151.250000_2.662000_388.075827_0.000000_0.100000': [234.88073309626216, 0, 0, 0, 0, 0, 0, 0], '0.000683_137.500000_2.662000_388.075827_0.000000_0.100000': [0, 0, 0, -404.2150300201529, 0, 0, 0, 0], '0.000683_137.500000_2.928200_388.075827_0.000000_0.100000': [0, 0, 0, 0, 1.5954845738180665, 0, 0, -515.4067941178895], '0.000621_137.500000_2.928200_388.075827_0.000000_0.100000': [0, 0, -1.5954845738180665, 0, 0, 0, 0, 0], '0.000683_137.500000_2.928200_426.883410_0.000000_0.100000': [0, 0, -1.0393710588135947, 0, 0, 0, 0, 0], '0.000751_137.500000_2.928200_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, 418.58186153126996, 0, 0], '0.000751_137.500000_2.662000_426.883410_0.000000_0.100000': [305.1322144312073, 0, 0, 0, 0, 0, 0, -510.6150681215822], '0.000751_125.000000_2.662000_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -530.1730066971501], '0.000751_125.000000_2.662000_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -285.5742758556395, 0], '0.000751_137.500000_2.662000_469.571751_0.000000_0.100000': [0, 0, -2.7676507552461773, 0, 0, 219.18879660060048, -299.4108275549517, 0], '0.000751_151.250000_2.662000_469.571751_0.000000_0.100000': [299.4108275549517, 0, 0, 0, 0, 0, 0, 0], '0.000751_137.500000_2.420000_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -278.9003633677445, 0], '0.000751_151.250000_2.420000_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -283.87843423271534, 0], '0.000751_166.375000_2.420000_469.571751_0.000000_0.100000': [0, 0, 0, 0, 1.139583179136389, 0, 0, 0], '0.000683_166.375000_2.420000_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000826_103.305785_1.502630_516.528926_0.000000_0.100000': [76.6746388779809, 0, 341.0479959794809, 0, 0, 1094.8801774364565, -369.6187561712442, -47.22757263551744], '0.000909_93.914350_1.366027_516.528926_0.000000_0.100000': [1836.1249947281199, 0, 0, -986.3340549634325, 0, 0, 0, 0], '0.000826_93.914350_1.366027_516.528926_0.000000_0.100000': [873.5340274709837, 0, 0, 0, 816.2036999127763, 0, -374.39466649215683, 0], '0.000683_125.000000_1.502630_469.571751_0.000000_0.100000': [0, 0, 0, 0, 2.5844342226933805, 0, 0, 0], '0.000621_125.000000_1.502630_469.571751_0.000000_0.100000': [0, 0, 0, 0, 2.3626419744742635, 0, 0, 0], '0.000564_125.000000_1.502630_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -472.56237797067433], '0.000564_125.000000_1.502630_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -446.1710786476362], '0.000564_125.000000_1.502630_568.181818_0.000000_0.100000': [0, 0, 0, 129.7764645569401, 0, 0, 0, 0], '0.000564_125.000000_1.652893_568.181818_0.000000_0.100000': [0, 0, 0, 0, 1.5430039946141552, 0, 0, 0], '0.000513_125.000000_1.652893_568.181818_0.000000_0.100000': [271.84592644405603, 0, -0.14027309041946862, 0, 0, 0, 0, 0], '0.000513_113.636364_1.652893_568.181818_0.000000_0.100000': [0, 0, -1.0662264840435456, 0, 0, 0, 0, 0], '0.000564_113.636364_1.652893_568.181818_0.000000_0.100000': [0, 0, -1.1698808143310089, 0, 0, 0, 0, 0], '0.000621_93.914350_1.652893_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -275.8519041873498, 0], '0.000621_103.305785_1.652893_625.000000_0.000000_0.100000': [0, 0, 0, 0, 1.2479769866537027, 0, 0, 0], '0.000564_103.305785_1.652893_625.000000_0.000000_0.100000': [275.9100239015183, 0, 0, 0, 0, 0, 0, 0], '0.000564_93.914350_1.652893_625.000000_0.000000_0.100000': [257.0887177087267, 0, 0, 0, 0, 0, 0, 0], '0.000564_85.376682_1.652893_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000826_151.250000_2.662000_516.528926_0.000000_0.100000': [0, 561.038780128488, 0, 0, 0, 0, 0, 0], '0.000826_151.250000_2.662000_469.571751_0.000000_0.100000': [0, 0, 0, -219.66899013774892, 0, 0, 0, -51.00352546622619], '0.000826_151.250000_2.928200_469.571751_0.000000_0.100000': [218.3442271070957, 0, 0, 0, 0, 0, 0, 0], '0.000826_137.500000_2.928200_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0.18566074933369237, 0, 0, 0], '0.000751_137.500000_2.928200_469.571751_0.000000_0.100000': [0, 775.9808724693228, 0, 0, 0, 0, 0, 0], '0.000826_137.500000_2.662000_469.571751_0.000000_0.100000': [0, 0, 0, -300.66961617027044, 0, 0, 0, 0], '0.001331_77.615165_2.000000_756.250000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 2708.210297154119, 0], '0.001331_85.376682_2.000000_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 3278.366175387523, 0, 0], '0.001331_93.914350_2.000000_687.500000_0.000000_0.100000': [1611.9112045383602, 0, -8.184601892577303, 0, 0, 0, 0, 0], '0.001464_93.914350_2.000000_687.500000_0.000000_0.100000': [902.2349408010034, 0, -9.006481914628058, -1247.3578219323506, 0, 0, 0, 0], '0.001000_166.375000_1.502630_469.571751_0.000000_0.100000': [0, 0, 0, 84.66998945301957, 0, 0, 0, 0], '0.001000_137.500000_1.818182_426.883410_0.000000_0.100000': [1730.260260780353, 0, 926.3853131840318, 0, 0, 388.6692655678364, -412.62672477060096, 0], '0.001100_113.636364_1.502630_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -316.4340749233203, 0], '0.001210_137.500000_1.366027_625.000000_0.000000_0.100000': [517.1751052729398, 878.0190092983905, 0, 0, 0, 0, 0, 0], '0.001210_125.000000_1.366027_625.000000_0.000000_0.100000': [403.92067700105247, 7.4897135351080015, 0, -755.4721114779555, 0, 0, -474.8607784778811, -468.6109850568173], '0.001210_125.000000_1.502630_687.500000_0.000000_0.100000': [297.9573268414915, 0, 0, 0, 0, 0, 0, 0], '0.001100_103.305785_1.502630_625.000000_0.000000_0.100000': [641.6681645720349, 0, 0, 438.8109112878441, 0, 1887.8449382654892, 0, 0], '0.001000_77.615165_1.502630_625.000000_0.000000_0.100000': [5751.861856646522, 0, -4.621743422148938, 0, 0, 2943.953103336953, 0, 0], '0.001000_77.615165_1.366027_625.000000_0.000000_0.100000': [-277.96825298122485, -1248.02463247702, -91.9875533222601, -2676.321003033594, -87.85236366879035, -1103.5074660619468, -1489.4355911912216, -0.00023068622843152299], '0.001000_77.615165_1.366027_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 706.7214796440624, 0, 0], '0.001000_77.615165_1.241843_568.181818_0.000000_0.100000': [3432.2365764402152, 0, 0, 0, 0, 0, 277.61574550081195, -562.204313228989], '0.001000_70.559241_1.241843_568.181818_0.000000_0.100000': [0, 32.733410574796096, 0, 0, -32.03024527741882, 0, 0, 0], '0.001000_70.559241_1.241843_516.528926_0.000000_0.100000': [4.801634540350385, 0, 0, 0, 0, 0, 0, -26.153393988621538], '0.001000_64.144765_1.241843_516.528926_0.000000_0.100000': [0, 0.8617929307863907, 0, 0, 0, 0, 0, 0], '0.001000_64.144765_1.241843_469.571751_0.000000_0.100000': [-0.0017751766607511854, 0, 0, -0.013272143344102006, 0, 0, 0, 0], '0.001000_64.144765_1.366027_469.571751_0.000000_0.100000': [0, 0, 0, 0, -0.6501050834853687, 0, 0, 0], '0.000909_64.144765_1.366027_469.571751_0.000000_0.100000': [0, 0, 0, -0.12834916868241636, 0, 0, 0, 0], '0.000909_64.144765_1.502630_469.571751_0.000000_0.100000': [-0.0064419726350479324, 0.014296412948965553, 0, 0, 0, 0, 0, 0], '0.000909_58.313423_1.502630_469.571751_0.000000_0.100000': [0, 0.7898399548110092, 0, 0, 0, 0, 0, 0], '0.000909_58.313423_1.502630_426.883410_0.000000_0.100000': [-0.002234172152150223, -0.6420968779959594, -0.7326949330418756, 0, 0, 0, 0, 0], '0.001000_58.313423_1.502630_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -0.0374831510021977, 0], '0.001000_64.144765_1.502630_426.883410_0.000000_0.100000': [0, 0.8368789131144212, 0, 0.7538931876426886, 0, 0, 0.004745819957332742, 0.07389102136263409], '0.001000_64.144765_1.502630_469.571751_0.000000_0.100000': [0.7390104410073243, -0.006656848771408472, 0, 0, -0.0797883858459727, 0, 0, 0], '0.000751_70.559241_1.818182_625.000000_0.000000_0.100000': [0, 0, 4.184629275488283, 0, 17.926025046957548, 0, 0, 4.634142575338456], '0.000826_70.559241_1.818182_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, -281.0256718725633, 0, 0], '0.000826_70.559241_1.652893_625.000000_0.000000_0.100000': [0, 0, 0, 0, 227.66248534096255, 0, 0, 0], '0.000751_70.559241_1.652893_625.000000_0.000000_0.100000': [0, 0, 0, 54.09641298172396, 0, 0, 0, 0], '0.000751_70.559241_1.818182_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 28.199363334017175, 0, 0], '0.000751_64.144765_1.652893_687.500000_0.000000_0.100000': [0, -0.8222026756068772, 0, 0, 0, 0, 0, 0], '0.000751_64.144765_1.652893_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -42.00909195528487, 0], '0.000683_70.559241_1.818182_625.000000_0.000000_0.100000': [20.949870941557414, -2775.089166000613, 0, 0, 0, 0, -4965.231236609493, 0], '0.000621_58.313423_1.818182_625.000000_0.000000_0.100000': [0, 0, 0, 0.7279546614407264, 0, 0, 0, 0], '0.000621_58.313423_2.000000_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0.7630688126824497, 0, 0, 0], '0.000564_58.313423_2.000000_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -0.030625087703811005, 0], '0.000564_64.144765_2.000000_625.000000_0.000000_0.100000': [0, 0, 0, 0, -0.9047805439829972, 0, 0, 0], '0.000513_64.144765_2.000000_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001210_125.000000_1.502630_469.571751_0.000000_0.100000': [1160.6471758718164, 0, 0, 0, 0, 0, 0, 0], '0.001100_103.305785_1.366027_388.075827_0.000000_0.100000': [0, 1132.2561457259278, -3.886044414060052, 34.88546503000102, 0, 54.69302533968215, 0, -77.937053707225], '0.001210_103.305785_1.366027_388.075827_0.000000_0.100000': [0, 350.71408291283217, 0, 0, 0, 0, 0, 0], '0.001210_103.305785_1.366027_352.796206_0.000000_0.100000': [302.02355100101124, 0, 0, 0, 0, -17.37475185964204, 0, 0], '0.001210_93.914350_1.366027_352.796206_0.000000_0.100000': [620.0955190089927, 0, 0, 0, 0, 0, -302.02355100101124, 0], '0.001210_103.305785_1.241843_352.796206_0.000000_0.100000': [0, 315.00595221305844, 0, 0, 0, 0, 0, 0], '0.001210_103.305785_1.241843_320.723824_0.000000_0.100000': [296.73527738078656, 0, 0, 0, 0, 0, 0, 0], '0.000564_113.636364_1.818182_568.181818_0.000000_0.100000': [0, 446.37459199498534, 0, 0, 0, 0, 0, 0], '0.000564_113.636364_1.818182_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -323.2238337179437, 0], '0.000564_125.000000_1.818182_516.528926_0.000000_0.100000': [0, 0, 0, -703.1710956144051, 0, 0, 0, 0], '0.000564_125.000000_2.000000_516.528926_0.000000_0.100000': [0, 0, -3.5047589386565963, -1266.561902654751, 0, 0, 0, 0], '0.000621_125.000000_2.000000_516.528926_0.000000_0.100000': [0, 0, -0.3856276818524928, 0, 3.5047589386565963, 0, 0, -516.953612120691], '0.000564_125.000000_2.200000_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -384.28325523278, 0], '0.000564_137.500000_2.200000_516.528926_0.000000_0.100000': [0, 0, 0, 0, 2.5783816803883384, 0, 0, -509.25380992132017], '0.000564_137.500000_2.200000_568.181818_0.000000_0.100000': [0, 509.25380992132017, 0, 0, 0, 0, 0, 0], '0.000513_137.500000_2.200000_516.528926_0.000000_0.100000': [0, 0, 0, -434.12703908228633, 0, 0, 0, 0], '0.000513_137.500000_2.420000_516.528926_0.000000_0.100000': [0, 432.2957815043301, 0, 0, 0, 0, 0, 0], '0.000513_137.500000_2.420000_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -278.3059060188913, 0], '0.000513_151.250000_2.420000_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000826_85.376682_1.652893_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 456.23754317226906, 0, 0], '0.000826_85.376682_1.502630_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -1176.1919927809117, 0], '0.000826_93.914350_1.502630_388.075827_0.000000_0.100000': [0, 468.15094765752673, -4.177150817347896, -44.858693212446816, 0, 1544.8722897268294, 368.2664117512113, -502.4067482792232], '0.000826_93.914350_1.502630_426.883410_0.000000_0.100000': [341.29453908886774, 902.7980414365686, 0, 0, 0, 798.8344327011254, 0, 0], '0.000909_93.914350_1.502630_388.075827_0.000000_0.100000': [0, 0, -4.154377424976793, 0, 653.1478264204893, 1824.2084920269474, 0, 0], '0.000909_93.914350_1.366027_388.075827_0.000000_0.100000': [0, 2079.3970408213454, -2.646921669484436, 0, 0, 0, 0, 0], '0.000909_93.914350_1.366027_352.796206_0.000000_0.100000': [0, 1248.5367861021937, 0, 0, 0, 0, 0, 0], '0.000909_93.914350_1.366027_320.723824_0.000000_0.100000': [0, 0, 0, 0, 1809.8270308853298, 0, 0, 0], '0.000826_93.914350_1.366027_320.723824_0.000000_0.100000': [1127.3673841369337, 0, 0, -360.92325902358925, 0, -121.07825323553799, 0, 0], '0.001210_151.250000_2.000000_687.500000_0.000000_0.100000': [751.7041254251145, 0, 0, 0, 0, 0, -420.82431311721285, 0], '0.001210_166.375000_2.000000_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 1053.7744599160671, 0, 0], '0.001210_166.375000_1.818182_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -467.103629193644], '0.001210_166.375000_1.818182_756.250000_0.000000_0.100000': [0, 0, 0, 0, 5.64108429117623, 0, 0, 0], '0.001100_166.375000_1.818182_756.250000_0.000000_0.100000': [680.370768750565, 0, 0, 0, 0, 0, 0, 0], '0.001210_103.305785_1.652893_831.875000_0.000000_0.100000': [257.8835514721419, 0, 0, 0, 0, 0, 0, 0], '0.001210_93.914350_1.652893_831.875000_0.000000_0.100000': [0, 0, 0, 0, 0, 151.203532494271, 0, 0], '0.001210_93.914350_1.502630_516.528926_0.000000_0.100000': [0, 0, 0, 318.1124173538406, 5.357971814860593, 0, 0, 0], '0.001100_93.914350_1.502630_516.528926_0.000000_0.100000': [0, 0, -5.357971814860593, 0, 2455.2991898966716, 0, 0, 0], '0.001331_93.914350_1.502630_469.571751_0.000000_0.100000': [398.3311441935932, 0, 0, 0, 0, 0, 0, 0], '0.001331_85.376682_1.502630_469.571751_0.000000_0.100000': [1163.2459067547222, 0, -7.0167560123138175, 0, 0, 0, 0, 0], '0.001464_85.376682_1.502630_469.571751_0.000000_0.100000': [1165.0043031051966, 0, 0, 0, 0, 0, 0, -494.8792214300579], '0.001464_85.376682_1.502630_516.528926_0.000000_0.100000': [334.6526708086138, 494.8792214300579, 0, 0, 0, 0, 0, 0], '0.001464_77.615165_1.502630_516.528926_0.000000_0.100000': [4138.302743004174, 0, 0, -354.0936140864963, 0, 0, 0, 0], '0.001464_77.615165_1.652893_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 354.0936140864963, 0, 0], '0.001464_70.559241_1.502630_516.528926_0.000000_0.100000': [0, 0, -1918.127714134474, 0, 0, 0, 0, 0], '0.001611_70.559241_1.502630_516.528926_0.000000_0.100000': [2028.2696979710972, 0, 0, 0, 0, 0, 0, 0], '0.001611_64.144765_1.502630_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0.8058601847076944, 0, 0, 0], '0.001464_64.144765_1.502630_516.528926_0.000000_0.100000': [-0.7793625741493315, 0, 0, 0, 0, 0, 0, 0], '0.001464_58.313423_1.502630_516.528926_0.000000_0.100000': [0, 0.7634322675919748, 0, 0, 0, 0, 0, 0], '0.001464_58.313423_1.502630_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001100_113.636364_2.000000_388.075827_0.000000_0.100000': [0, 2605.539756622088, 0, 0, 397.53298457844636, 0, -934.2087104708504, 0], '0.001100_113.636364_2.000000_352.796206_0.000000_0.100000': [657.7836283269332, 0, -6.006224685837333, 0, 0, 0, 0, 0], '0.001210_113.636364_2.200000_320.723824_0.000000_0.100000': [0, 0, -5.9873948922779165, 0, 0, 0, 0, 0], '0.001331_113.636364_2.200000_320.723824_0.000000_0.100000': [0, 0, -6.596293015427546, 0, 0, 0, 0, 0], '0.001464_113.636364_2.200000_320.723824_0.000000_0.100000': [0, 0, -7.26090606196849, 0, 0, 0, -410.4947185923138, 0], '0.001611_113.636364_2.200000_320.723824_0.000000_0.100000': [0, 0, 0, 0, 7.26090606196849, 0, 0, 0], '0.001464_125.000000_2.200000_320.723824_0.000000_0.100000': [0, 542.2222122235024, -6.9442979981011375, 0, 0, 929.2318249286328, 0, 0], '0.001464_125.000000_2.000000_320.723824_0.000000_0.100000': [0, 0, 0, -929.2318249286328, 0, 0, 0, 0], '0.001611_125.000000_2.200000_320.723824_0.000000_0.100000': [0, 0, 0, 0, 6.9442979981011375, 0, 0, 0], '0.001464_125.000000_2.200000_291.567113_0.000000_0.100000': [0, 0, 0, 0, 6.162650172795293, 0, 0, 0], '0.001331_125.000000_2.200000_291.567113_0.000000_0.100000': [0, 560.9926136724533, 0, 0, 0, 0, 0, 0], '0.001331_125.000000_2.200000_265.061011_0.000000_0.100000': [0, 0, 0, 0, 5.693365684500714, 0, 0, 0], '0.001210_125.000000_2.200000_265.061011_0.000000_0.100000': [0, 0, 0, 0, 5.195347935985319, 0, 0, 0], '0.001100_125.000000_2.200000_265.061011_0.000000_0.100000': [307.96744084667637, 0, 0, 0, 0, 0, 0, 0], '0.001100_113.636364_2.200000_265.061011_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001000_85.376682_1.818182_625.000000_0.000000_0.100000': [0, 1125.4448478187028, 0, -1273.362651029379, -884.6530578474565, 0, 0, -376.6518767548464], '0.000909_64.144765_1.818182_687.500000_0.000000_0.100000': [0, -1.962104907579478e-05, 0, 0, 0, -0.7495127827347687, 0, 0.02247303744000783], '0.000909_64.144765_1.652893_687.500000_0.000000_0.100000': [0, 0, 0.8190804006380086, 0, 0, 0, 0, 0], '0.001000_64.144765_1.652893_687.500000_0.000000_0.100000': [-0.007388966606390568, 0.002277647239657199, 0, 0, 0, 0, 0, 0], '0.001000_58.313423_1.652893_687.500000_0.000000_0.100000': [0, 0, 0, -0.7584146673577152, 0, 0, 0, 0], '0.001000_58.313423_1.818182_687.500000_0.000000_0.100000': [-0.10812395767397902, 0, 0, 0, 0, 0, 0.752680553240657, 0], '0.001000_64.144765_1.818182_687.500000_0.000000_0.100000': [-0.7526805532408617, 0.01755426492384004, 0, 0, 0, 0, -3575.053412305404, 0], '0.001000_70.559241_1.818182_687.500000_0.000000_0.100000': [0, 0, 0, -2984.912769398264, 0, 0, 0, 0], '0.001000_70.559241_2.000000_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -261.2985533025787, 0], '0.001000_77.615165_1.652893_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -1580.2611984574653], '0.001000_77.615165_1.652893_831.875000_0.000000_0.100000': [0, 0, 0, 0, 0, 182.52217745672007, 0, 0], '0.001331_113.636364_1.652893_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 42.85308861605888, 0, 0], '0.001331_113.636364_1.502630_687.500000_0.000000_0.100000': [293.0691696397948, 0, 0, 0, 0, 0, 0, -426.4379349373492], '0.001331_113.636364_1.502630_756.250000_0.000000_0.100000': [0, 0, 0, 0, 0, 725.3682034268495, 0, 0], '0.001331_113.636364_1.366027_756.250000_0.000000_0.100000': [0, 0, 0, 0, 6.590852816381994, 0, 0, 0], '0.001210_93.914350_1.366027_687.500000_0.000000_0.100000': [883.2948212529902, 0, 0, 0, 0, 0, 0, -527.2018242531678], '0.001210_85.376682_1.366027_687.500000_0.000000_0.100000': [0, 0, 0, 0, 4.179150555659726, 0, 0, 0], '0.001100_85.376682_1.366027_687.500000_0.000000_0.100000': [0, 471.64279121798893, 0, 0, 3.678996685859329, 659.063114752836, 0, 0], '0.001100_85.376682_1.241843_687.500000_0.000000_0.100000': [0, 655.152229650296, 0, -236.1879711579724, 123.76813593264274, 0, 0, 0], '0.001100_85.376682_1.241843_625.000000_0.000000_0.100000': [195.37457555071836, 0, 0, 0, 0, 0, 0, -446.2164902700074], '0.001000_85.376682_1.366027_687.500000_0.000000_0.100000': [2175.1723955656407, 0, -3.678996685859329, -153.44265499452578, 1228.570262712965, 234.3537514643952, 0, 0], '0.000826_125.000000_1.366027_568.181818_0.000000_0.100000': [456.6781172217271, 0, -4.7978642761167976, 0, 0, 0, 0, 0], '0.000909_125.000000_1.366027_568.181818_0.000000_0.100000': [544.9528737217671, 0, -5.2838110687072914, -815.4782973815962, 0, 0, -479.77383497526097, 0], '0.000909_113.636364_1.366027_568.181818_0.000000_0.100000': [456.39699854135887, 0, -5.046963562398832, 0, 0, 0, -456.89586058851285, -474.115313505846], '0.000909_137.500000_1.366027_568.181818_0.000000_0.100000': [0, 485.7893131214816, 0, 0, 0, 0, 0, 0], '0.000909_137.500000_1.366027_516.528926_0.000000_0.100000': [51.03956478336062, 0, 0, 0, 0, 0, 0, 0], '0.000909_125.000000_1.366027_516.528926_0.000000_0.100000': [0, 0, 0, -866.8349618075576, 0, 1379.7080503311436, 0, 0], '0.000909_125.000000_1.241843_516.528926_0.000000_0.100000': [640.4232375224803, 0, 0, 0, 0, 0, 193.68351129571084, 0], '0.000909_113.636364_1.241843_516.528926_0.000000_0.100000': [650.7201850703904, 0, 0, 0, 1.4572813512660105, -136.4609165045815, 0, 0], '0.000909_103.305785_1.241843_516.528926_0.000000_0.100000': [235.40803677356405, 410.4032461450222, 0, 0, 0, 0, -25.54497282422241, -399.43716967249793], '0.000909_103.305785_1.241843_568.181818_0.000000_0.100000': [226.88127008687664, 0, -1.8826709962234418, 0, 0, 0, -231.21275546377956, 0], '0.000909_93.914350_1.241843_568.181818_0.000000_0.100000': [1050.3262191957042, 0, 0, 1301.2497421774185, 0, 0, 0, 0], '0.001210_93.914350_1.652893_625.000000_0.000000_0.100000': [255.5811778414016, 0, -3.0021624572465955, 0, 0, 0, 0, 0], '0.001331_93.914350_1.652893_625.000000_0.000000_0.100000': [255.30325536147575, 0, 0, 0, 0, 0, 0, 0], '0.001331_85.376682_1.652893_625.000000_0.000000_0.100000': [0, 0, -3.647094182053661, 0, 0, 0, 0, 0], '0.001464_85.376682_1.652893_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -255.00004603767576, 0], '0.001464_93.914350_1.652893_687.500000_0.000000_0.100000': [389.48973295456824, 0, 0, 0, 0, 0, 0, 0], '0.001464_85.376682_1.652893_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 141.93554310344504, 0, 0], '0.001464_77.615165_1.502630_687.500000_0.000000_0.100000': [0, 0, -6.835180881326778, 0, 0, 0, 0, 0], '0.001611_77.615165_1.502630_687.500000_0.000000_0.100000': [5746.029375906947, 0, 0, -198.00412794550394, 0, 0, 0, 0], '0.001611_77.615165_1.652893_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 198.00412794550394, 0, 0], '0.001611_70.559241_1.502630_687.500000_0.000000_0.100000': [28.046444782888695, 0, 0, 0, 0, 0, -574.6029375906946, 0], '0.001611_64.144765_1.502630_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -28.046444782888695, 0], '0.000683_93.914350_1.818182_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000909_103.305785_1.502630_568.181818_0.000000_0.100000': [0, 0, 0, 398.88178612339584, 0, 1006.1023233910367, 0, 0], '0.000909_103.305785_1.366027_568.181818_0.000000_0.100000': [2423.5171404034413, 0, -4.753397610831598, 0, 0, 223.57673362809192, 0, 0], '0.001000_103.305785_1.241843_568.181818_0.000000_0.100000': [1148.4265109008, 0, 0, 1422.8463629526962, 187.7010681669267, 0, -230.68610217024653, 0], '0.001000_113.636364_1.241843_568.181818_0.000000_0.100000': [0, 1842.7117775879994, 0, 0, 0, -165.81965897925238, 0, 0], '0.001000_113.636364_1.241843_516.528926_0.000000_0.100000': [401.4279700327661, 0, 0, 0, 0, -135.41589136655858, 0, 0], '0.001000_113.636364_1.128948_516.528926_0.000000_0.100000': [193.59892445924334, 0, -0.6140142584239584, 0, 0, 0, 0, 0], '0.001100_113.636364_1.128948_516.528926_0.000000_0.100000': [0, 396.0414479930147, 0, 0, 0, 0, 0, 0], '0.001100_113.636364_1.128948_469.571751_0.000000_0.100000': [0, 380.08324109972864, 0, 0, 0, 0, 0, 0], '0.001100_113.636364_1.128948_426.883410_0.000000_0.100000': [203.08855271483853, 0, 0, 0, 0, 0, 0, 0], '0.001100_103.305785_1.128948_426.883410_0.000000_0.100000': [0, 0, 0, 154.97702359271972, 0, 0, 0, 0], '0.001100_103.305785_1.241843_426.883410_0.000000_0.100000': [0, 344.0695761186398, 0, 0, 0, 0, 0, 0], '0.001100_103.305785_1.241843_388.075827_0.000000_0.100000': [342.20610356440756, 224.13500192672774, 0, 0, 312.1486464407989, 0, 0, -344.0695761186398], '0.001100_103.305785_1.241843_352.796206_0.000000_0.100000': [232.04400009415085, 0, 256.2677942512584, 0, 0, -177.08184292558855, 0, -153.48400424335335], '0.000826_125.000000_1.502630_568.181818_0.000000_0.100000': [0, 0, 1175.5907284521545, 0, 0, 818.637933700667, 0, -4.084069966781539], '0.000826_137.500000_1.241843_625.000000_0.000000_0.100000': [0, 396.97816827658716, 0, 0, 0, 0, 0, 0], '0.000826_137.500000_1.241843_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, -27.408062855352092, 0, 0], '0.000826_137.500000_1.128948_568.181818_0.000000_0.100000': [0, 0, -2.11190792469597, 0, 0, 0, 0, 0], '0.000909_137.500000_1.128948_568.181818_0.000000_0.100000': [0, 402.633792106259, 0, 0, 0, 0, 0, 0], '0.000909_137.500000_1.128948_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, -416.91665771815843, 0, 0], '0.000909_137.500000_1.026316_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0.5530871336569362, 0, 0, 0], '0.000826_137.500000_1.026316_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0.5728205988732044, 0, 0, 0], '0.000751_137.500000_1.026316_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 303.9121905809338, 0, 0], '0.000751_137.500000_0.933015_516.528926_0.000000_0.100000': [0, 419.3034326687991, 0, 0, 0, 0, 0, 0], '0.000751_137.500000_0.933015_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001210_85.376682_1.818182_469.571751_0.000000_0.100000': [0, 0, 0, 538.2615164173619, 3.5999455951546224, 0, 0, 0], '0.001100_93.914350_1.818182_469.571751_0.000000_0.100000': [0, 0, 0, 0, 3.8185780394219364, 0, 0, -196.20739302454285], '0.001000_85.376682_2.000000_388.075827_0.000000_0.100000': [2779.484000773149, 0, 0, 0, 0, 0, -1043.775653095657, 0], '0.001100_93.914350_2.000000_388.075827_0.000000_0.100000': [1044.9091490735511, 0, 0, 0, 0, 0, 0, 0], '0.001100_85.376682_2.000000_388.075827_0.000000_0.100000': [3423.3081138240286, 0, 0, 0, 0, 0, 0, 0], '0.001100_77.615165_2.000000_388.075827_0.000000_0.100000': [3204.0356167479645, 0, 0, 0, -320.4314377260291, 0, 0, 0], '0.000909_113.636364_1.502630_568.181818_0.000000_0.100000': [0, 0, 0, 2318.797269010039, 2.2110252641221453, 0, 0, 0], '0.000826_125.000000_1.652893_388.075827_0.000000_0.100000': [588.3506538945057, 0, 23.60898216053296, 0, 0, 0, 0, 0], '0.000826_113.636364_1.652893_388.075827_0.000000_0.100000': [298.59401568006615, 0, -1.2388436277615256, -192.11998448130944, 0, 0, 0, 0], '0.000909_113.636364_1.652893_388.075827_0.000000_0.100000': [0, 0, 0, 0, 267.68869797930677, 0, 0, 0], '0.000826_103.305785_1.652893_388.075827_0.000000_0.100000': [0, 0, -0.8737817424543209, 0, 0.7733277172807903, 0, 0, -396.2931901295203], '0.000909_103.305785_1.652893_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 976.8774504988364, 0, 0], '0.001331_103.305785_1.652893_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 385.1802845804869, 0, 0], '0.001331_93.914350_1.502630_516.528926_0.000000_0.100000': [-541.520788273083, -82.85168559834689, 0, 0, -664.3302448121457, 0, 0, 0], '0.001464_93.914350_1.652893_469.571751_0.000000_0.100000': [0, 0, 0, 346.37831450293925, 0, 0, 0, 0], '0.001331_85.376682_2.000000_426.883410_0.000000_0.100000': [0, 595.1423052721352, 0, 0, 0, 0, 0, 0], '0.001331_85.376682_2.000000_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, -483.46896459858374, 0, 0], '0.001331_85.376682_1.818182_388.075827_0.000000_0.100000': [0, 380.0614045671365, 0, 0, 0, 0, 0, 0], '0.001331_85.376682_1.818182_352.796206_0.000000_0.100000': [207.47415535863502, 0, 0, 0, 0, 0, 0, 0], '0.001331_77.615165_1.818182_352.796206_0.000000_0.100000': [0, 0, 0, 380.0058947789834, 0, 0, 0, 0], '0.001331_77.615165_2.000000_352.796206_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001000_151.250000_1.502630_756.250000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -411.8286403008695], '0.001000_151.250000_1.502630_831.875000_0.000000_0.100000': [304.5912790720337, 0, -2.697305639437218, 0, 0, 0, 0, 0], '0.001100_151.250000_1.502630_831.875000_0.000000_0.100000': [0, 0, 0, 0, 2.697305639437218, 0, 0, 0], '0.001000_137.500000_1.502630_831.875000_0.000000_0.100000': [0, 0, 0, 0, 0, 417.7357598032577, 0, 0], '0.001000_137.500000_1.366027_831.875000_0.000000_0.100000': [0, 0, -5.313544804681806, 0, 0, 0, 0, 0], '0.001100_137.500000_1.366027_831.875000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -481.6227427904887], '0.001100_137.500000_1.366027_915.062500_0.000000_0.100000': [373.4638944673363, 0, 0, -373.07019110511646, 0, 0, 0, 0], '0.001100_125.000000_1.366027_915.062500_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -373.4638944673363, 0], '0.001100_137.500000_1.502630_915.062500_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -287.2939479111448, 0], '0.001100_151.250000_1.502630_915.062500_0.000000_0.100000': [0, 0, -2.7371106053715266, 0, 0, 0, 0, 0], '0.001210_151.250000_1.502630_915.062500_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -301.02827737707247, 0], '0.001210_166.375000_1.502630_915.062500_0.000000_0.100000': [0, 0, 0, 0, 2.983975335057585, 170.6209459416361, 0, 0], '0.001100_166.375000_1.502630_915.062500_0.000000_0.100000': [0, 0, -2.983975335057585, 0, 0, 0, 0, 0], '0.001210_166.375000_1.366027_915.062500_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -437.90729818519213], '0.001210_166.375000_1.366027_1006.568750_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 11395.523647044767, 0], '0.001210_183.012500_1.366027_1006.568750_0.000000_0.100000': [0, -11350.506201655555, 0, 0, 0, 0, 0, 0], '0.001210_183.012500_1.366027_915.062500_0.000000_0.100000': [0, 0, -4.618812243172215, 0, 0, 0, 0, 0], '0.001331_183.012500_1.366027_915.062500_0.000000_0.100000': [0, 399.82782569522027, 0, 0, 0, 0, 0, 0], '0.001331_183.012500_1.366027_831.875000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000826_93.914350_2.200000_469.571751_0.000000_0.100000': [0, 0, 0, -978.1343544899286, 0, 0, 0, 0], '0.000909_93.914350_2.420000_469.571751_0.000000_0.100000': [0, 0, 0, -328.9345584384255, 0, 0, 0, 0], '0.000909_93.914350_2.662000_469.571751_0.000000_0.100000': [236.2826691170776, 0, 0, 0, 0, 0, 0, 0], '0.000909_85.376682_2.662000_469.571751_0.000000_0.100000': [0, 0, 0, 0, 3.171452840164784, 0, 0, 0], '0.000826_77.615165_2.662000_469.571751_0.000000_0.100000': [0, 0, 0, 0, -305.27222659278686, 0, 0, 0], '0.000751_77.615165_2.662000_469.571751_0.000000_0.100000': [0, 0, 0, 0, 2.6887212712275868, 0, 0, 0], '0.000683_77.615165_2.662000_469.571751_0.000000_0.100000': [0, 0, 0, 0, 2.436652021311602, 0, 0, 0], '0.000621_77.615165_2.662000_469.571751_0.000000_0.100000': [0, -1220.316004245285, 0, 0, 0, 0, 0, 0], '0.000621_77.615165_2.662000_426.883410_0.000000_0.100000': [2091.5061798341676, 0, 0, 0, 0, 0, 0, 0], '0.000621_70.559241_2.662000_426.883410_0.000000_0.100000': [0, 4472.532986159994, 0, 0, 0, 0, 0, 0], '0.000621_70.559241_2.662000_388.075827_0.000000_0.100000': [10.950891851647066, 0, 0, 0, 0, 0, 0, 0], '0.000621_64.144765_2.662000_388.075827_0.000000_0.100000': [0, 0, 0, 0.7086557982922613, 0, 0, 0, 0], '0.000621_64.144765_2.928200_388.075827_0.000000_0.100000': [0, 0, 0, -0.7677869202788543, 0, 0, 0, 0], '0.000621_64.144765_3.221020_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0.02356252731447057, 0, 0, 0], '0.000564_64.144765_3.221020_388.075827_0.000000_0.100000': [0, 0.7406034030866522, 0, 0, 0, 0, 0, 0], '0.000564_64.144765_3.221020_352.796206_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000909_77.615165_1.652893_625.000000_0.000000_0.100000': [0, 85.31783395028597, 0, -488.5175117239008, 0, -5112.1470904610715, 0, 0], '0.000909_77.615165_1.502630_625.000000_0.000000_0.100000': [0, 0, 2404.4981985323475, 0, 0, 0, 0, 0], '0.001000_70.559241_1.366027_625.000000_0.000000_0.100000': [0, 686.9652117443585, 0, 0, 0, 0, 0, 0], '0.001000_70.559241_1.366027_568.181818_0.000000_0.100000': [0, 0, 0, 0, 4.919634621803107, 0, 0, 0], '0.000909_70.559241_1.366027_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -1657.979826156974, 0], '0.000909_77.615165_1.366027_568.181818_0.000000_0.100000': [0, 0, 0, 0, 3.589564220887587, 705.6726605048165, 1478.5066294104033, 0], '0.000826_77.615165_1.366027_568.181818_0.000000_0.100000': [0, 866.9432375452532, 0, 0, 0, 0, 0, 0], '0.000826_77.615165_1.366027_516.528926_0.000000_0.100000': [3172.9762944415957, 0, 0, -1211.1776339795458, 0, -617.626770811117, 0, -78.81302159502303], '0.000909_93.914350_1.652893_426.883410_0.000000_0.100000': [565.9299260701785, 388.85742634304444, 0, -68.19358481744267, 0, 0, 0, 0], '0.000909_93.914350_1.818182_426.883410_0.000000_0.100000': [0, 0, 0, 0, 112.54375713195702, 0, 1566.130550242151, 0], '0.001000_77.615165_2.000000_625.000000_0.000000_0.100000': [891.198945518817, -287.44324897172555, 0, 0, 0, 0, 0, 0], '0.001000_70.559241_2.000000_625.000000_0.000000_0.100000': [0, 0, 0, 0, 2.7942939199760075, 0, 0, 0], '0.000826_70.559241_2.000000_625.000000_0.000000_0.100000': [0, 1490.8018547399245, 0, 0, 0, 0, -888.945368442214, 0], '0.000826_70.559241_2.000000_516.528926_0.000000_0.100000': [0, 0, -4.221764574583176, -1168.078485475378, 0, 0, 0, 0], '0.000909_70.559241_2.000000_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 2222.8178365204108, 0, 0], '0.000909_70.559241_1.818182_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 2462.1733961475516], '0.001000_70.559241_1.818182_568.181818_0.000000_0.100000': [3.172147255277332, 0, 0, 0, 0, -30.576720447884874, 0, -33.67498560813286], '0.001000_70.559241_1.818182_625.000000_0.000000_0.100000': [0, 30.889493613794254, 7.484305927740183, 0, 0, 0, 0, 0], '0.001100_70.559241_1.818182_625.000000_0.000000_0.100000': [27.244669910370046, 0, 0, -410.4827649913154, 0, -212.3998039414224, 0, 0], '0.001100_70.559241_1.652893_625.000000_0.000000_0.100000': [0, 0, 0, 0, 67.16598059283693, 0, 0, 0], '0.001000_70.559241_1.652893_625.000000_0.000000_0.100000': [0, 81.51918432627113, 0, 0, 0, 0, 0, 143.89742459706196], '0.001000_70.559241_1.652893_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, -4054.2320358297925, 0, 0], '0.001000_70.559241_1.502630_687.500000_0.000000_0.100000': [0, 4417.843120477193, 0, 0, 0, 0, 0, 0], '0.001000_58.313423_1.502630_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0.01816523455235256, 0], '0.000621_93.914350_1.818182_426.883410_0.000000_0.100000': [487.56251143980876, 0, 0, 0, 0, 0, 0, 0], '0.000621_77.615165_1.652893_426.883410_0.000000_0.100000': [0, 0, 0, 69.9303006891847, 0, 0, 0, 0], '0.000564_77.615165_1.818182_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -458.93038772613994], '0.000564_77.615165_1.818182_469.571751_0.000000_0.100000': [0, 0, 0, 0, 1.7353634859451632, 0, 0, 0], '0.000513_77.615165_1.818182_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001000_85.376682_1.502630_388.075827_0.000000_0.100000': [0, 0, 0, 174.10272998305004, 0, 569.8526445116415, 0, -1236.0322255539738], '0.001000_77.615165_1.366027_388.075827_0.000000_0.100000': [0, 455.616570569289, 0, -209.33071472202064, 2022.1137889947945, 0, 0, 1623.5078931782723], '0.001000_77.615165_1.502630_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 209.33071472202064, 0, 0], '0.001000_77.615165_1.366027_352.796206_0.000000_0.100000': [0, 1667.2723272602223, 43.41263265627463, -155.63324608845753, 0, 0, 0, 0], '0.001100_77.615165_1.366027_352.796206_0.000000_0.100000': [738.6444264323692, 0, 0, 0, -43.41263265627463, 0, 0, 0], '0.001000_77.615165_1.502630_352.796206_0.000000_0.100000': [0, 0, 0, 0, 0, 1323.6764562481242, 0, 0], '0.001000_77.615165_1.366027_320.723824_0.000000_0.100000': [1593.1458122930865, 0, -1.4321740150511688, 0, 0, 0, 0, 0], '0.001100_77.615165_1.366027_320.723824_0.000000_0.100000': [1602.1504274415317, 0, -1.4100089890194658, -72.53565165383652, 701.7522055619867, 0, 0, 0], '0.001210_77.615165_1.366027_320.723824_0.000000_0.100000': [1442.2063652529503, 0, 0, 0, 0, 0, -116.23546666092766, 0], '0.001210_85.376682_1.366027_320.723824_0.000000_0.100000': [771.2274083925216, 0, 0, -94.06077458913315, 0, -587.7163921512499, 0, 0], '0.001210_85.376682_1.241843_320.723824_0.000000_0.100000': [91.56443791858875, 0, 0, 0, 0, 0, 0, 0], '0.001000_93.914350_1.502630_687.500000_0.000000_0.100000': [306.9817086955924, 0, 0, 0, 0, 0, 0, 0], '0.001000_85.376682_1.502630_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 1542.3128381692138, 0, 0], '0.001000_77.615165_1.366027_687.500000_0.000000_0.100000': [-433.290197772171, 253.75487436569566, -18.554324028300503, 0, 42.6295601823134, 0, 0, -2800.49247395985], '0.001000_77.615165_1.366027_756.250000_0.000000_0.100000': [5205.064613263843, 0, 0, 0, 0, 0, 0, -553.6226574602964], '0.001000_77.615165_1.366027_831.875000_0.000000_0.100000': [0, 0, 0, -950.6841082505256, 0, 0, 0, 0], '0.001000_77.615165_1.502630_831.875000_0.000000_0.100000': [1886.6012442630158, 0, 0, -182.52217745672007, 0, 0, 0, 0], '0.001000_70.559241_1.502630_831.875000_0.000000_0.100000': [0, 0, 0, 0, 0, 4851.6958365178425, 0, 0], '0.001000_70.559241_1.366027_831.875000_0.000000_0.100000': [0, 0, -28.726363461840858, 0, 0, 0, 0, 0], '0.001100_70.559241_1.366027_831.875000_0.000000_0.100000': [0, -0.20667009932803923, 5.805417059295223, 0, 0, 0, 0, 0], '0.001210_70.559241_1.366027_831.875000_0.000000_0.100000': [0, 0, 0, 0, -5.805417059295223, 0, 0, 0], '0.001100_70.559241_1.366027_756.250000_0.000000_0.100000': [0, 0, 0, 0, 0, -40.068287321214896, -5209.968536427858, 0], '0.001100_70.559241_1.241843_756.250000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000909_85.376682_1.652893_426.883410_0.000000_0.100000': [0, 436.37157927618784, 0, 0, 0, 0, 0, 0], '0.000909_85.376682_1.652893_388.075827_0.000000_0.100000': [0, 0, -1.0677917282215277, 0, 0, 1292.3535470789461, 0, 0], '0.001000_85.376682_1.652893_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 2054.628210386453, 0, 0], '0.001000_85.376682_1.502630_426.883410_0.000000_0.100000': [0, 1505.179430642108, 0, 0, 1117.4781371680078, 0, 0, 0], '0.000909_85.376682_1.502630_426.883410_0.000000_0.100000': [0, 1561.4213475005545, -0.44659712855529493, 0, 0, 0, 0, 0], '0.000909_85.376682_1.502630_388.075827_0.000000_0.100000': [2014.9696641076516, 0, 0, 0, 0, 0, -483.14582950109633, 0], '0.001100_70.559241_1.652893_516.528926_0.000000_0.100000': [0, 3548.1818453479978, 0, 0, 0, 0, 0, 0], '0.001100_70.559241_1.652893_469.571751_0.000000_0.100000': [41.95600913185961, 0, 0, 0, 0, 0, 0, 0], '0.000909_103.305785_1.502630_687.500000_0.000000_0.100000': [335.38125406542014, 0, 0, 0, 0, 908.2173273220045, 0, 0], '0.000909_93.914350_1.502630_687.500000_0.000000_0.100000': [0, 0, 0, 0, 788.3703702959905, 0, 0, 0], '0.000826_93.914350_1.502630_687.500000_0.000000_0.100000': [0, 482.3439176184658, 0, 0, 89.97547364961804, 0, 0, 0], '0.000751_93.914350_1.502630_625.000000_0.000000_0.100000': [336.94288090428546, 0, 0, 0, 0, 0, 0, 0], '0.000751_85.376682_1.502630_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 519.987808421889], '0.000683_85.376682_1.502630_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 505.6997857481183, 0, 0], '0.000683_85.376682_1.366027_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 232.64413575798628, 0, 0], '0.000683_85.376682_1.241843_687.500000_0.000000_0.100000': [1458.9366474262833, 0, 0, 0, 0, -1133.5009752036199, 0, 0], '0.000683_85.376682_1.128948_687.500000_0.000000_0.100000': [1585.8345710772157, 0, 0, 0, 0, 0, 0, 0], '0.000683_77.615165_1.128948_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 845.310025971161], '0.000683_77.615165_1.128948_756.250000_0.000000_0.100000': [186.14159596275778, 0, 0, 0, 0, 0, 0, 0], '0.000683_70.559241_1.128948_756.250000_0.000000_0.100000': [0, 0, 0, 0, -2.163163967251785, 0, 0, 0], '0.000621_70.559241_1.128948_756.250000_0.000000_0.100000': [2783.7918392223437, 0, 0, 0, 0, 0, 0, 0], '0.000621_64.144765_1.128948_756.250000_0.000000_0.100000': [0, 0, 0, 0, 0, -0.6000643818771778, 0, 0], '0.000621_64.144765_1.026316_756.250000_0.000000_0.100000': [0, 0, 0.61004886184096, 0, 0, 0, 0, 0], '0.000683_64.144765_1.026316_756.250000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000826_137.500000_1.652893_516.528926_0.000000_0.100000': [0, 419.6554785574499, 0, 0, 44.255141435244354, -145.94136825825044, -354.65670958941627, -448.4559518304059], '0.000826_151.250000_1.652893_516.528926_0.000000_0.100000': [0, 416.9350995405401, 0, 0, 0, 0, 0, 0], '0.000826_151.250000_1.652893_469.571751_0.000000_0.100000': [357.37708860632614, 0, 0, 0, 0, 0, -401.52398191324136, 0], '0.000909_70.559241_1.502630_388.075827_0.000000_0.100000': [0, 127.52905753032445, 0, 0, 0, 0, 0, 0], '0.000909_70.559241_1.502630_352.796206_0.000000_0.100000': [85.48401013892457, 0, 0, 0, 0, 0, 0, 0], '0.001210_151.250000_1.818182_516.528926_0.000000_0.100000': [466.26803610912225, 0, 0, 0, 7.942381603100675, 0, 0, 0], '0.001331_137.500000_1.502630_516.528926_0.000000_0.100000': [325.27037475141805, 0, 0, 0, 0, 0, 0, 0], '0.001331_125.000000_1.502630_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 806.4821032995804, 0, 0], '0.001331_125.000000_1.366027_516.528926_0.000000_0.100000': [0, 0, 0, 0, 6.624981013261549, 0, 0, 0], '0.001210_125.000000_1.366027_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 519.7078862981489, 0, -448.82140988986976], '0.001210_125.000000_1.366027_568.181818_0.000000_0.100000': [0, 493.70355087885673, 0, 0, 0, 0, 0, -822.034726772253], '0.000909_85.376682_1.502630_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 1493.3528988742278, 0, 0], '0.000909_85.376682_1.366027_625.000000_0.000000_0.100000': [1398.2224083143694, 1017.4496657162643, 0, 0, 0, 0, 0, 0], '0.001000_85.376682_1.366027_516.528926_0.000000_0.100000': [134.91289755174625, 0, -3.515116459140427, 0, 0, 0, -796.7053591476507, 0], '0.001000_93.914350_1.366027_516.528926_0.000000_0.100000': [986.759174877481, 0, -4.90321617008467, 0, 0, 0, 0, 0], '0.001100_93.914350_1.366027_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 4.559803887169164, 0, 0], '0.001100_93.914350_1.241843_516.528926_0.000000_0.100000': [884.4362335428344, 0, 0, 0, 0, 0, 0, 0], '0.001100_85.376682_1.241843_516.528926_0.000000_0.100000': [0, 0, 0, 0, 2.768096318122198, 0, 0, 0], '0.001000_85.376682_1.241843_516.528926_0.000000_0.100000': [-580.3203691063354, 0, 0, 0, 2327.3810518443975, -1045.3133630358757, -884.4016120937357, 0], '0.001000_77.615165_1.241843_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 1072.1285835691344, 0, 0], '0.001000_77.615165_1.128948_516.528926_0.000000_0.100000': [999.5308941517577, 0, 0, 0, 0, 0, -1537.1215774986747, 0], '0.001000_70.559241_1.128948_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001210_77.615165_1.818182_516.528926_0.000000_0.100000': [0, 0, 0, 0, 483.09654356144364, 0, 0, 0], '0.001100_77.615165_1.818182_516.528926_0.000000_0.100000': [0, 5109.378411850417, -0.3557116536477769, 0, 3.215472102541662, 0, 0, 0], '0.001000_77.615165_1.818182_516.528926_0.000000_0.100000': [0, 508.4866096550357, 0, 0, 0, 0, 0, 0], '0.001000_77.615165_1.818182_469.571751_0.000000_0.100000': [0, 464.172136820129, -3.477099187975091, 0, 0, 0, 0, 0], '0.001100_70.559241_1.818182_469.571751_0.000000_0.100000': [0, 0, -5.749195781620529, 0, 0, 0, -2119.8397722244295, 2305.099186604891], '0.001100_70.559241_1.818182_516.528926_0.000000_0.100000': [26.880164137863858, -20.766659338782755, 0, 0, -2441.1993730227227, 0, 0, 0], '0.001000_70.559241_1.818182_516.528926_0.000000_0.100000': [0, 366.4662445826901, 0, 0, 0, 0, 0, 2464.1615612962146], '0.000909_85.376682_1.502630_516.528926_0.000000_0.100000': [333.25074220962654, 0, 0, 0, 1283.9724408942507, 0, 0, 0], '0.000909_77.615165_1.502630_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -333.25074220962654, 0], '0.000751_93.914350_1.366027_516.528926_0.000000_0.100000': [1003.8476805868437, 0, 0, -986.3958740327456, 0, 0, -374.06473567991833, 0], '0.000751_93.914350_1.652893_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 204.3573005305043, 0, -456.81890216233035], '0.000683_85.376682_1.652893_516.528926_0.000000_0.100000': [-866.4603637530696, -299.94886378502633, -3.448266058207173, -37.42671470543255, 1.31901865349123, -735.3445956062397, 0, 0], '0.000683_70.559241_1.652893_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 3574.15358284731, 0, 0], '0.000683_70.559241_1.502630_516.528926_0.000000_0.100000': [31.714263439870958, -116.49658156387196, 0, 0, 0, 0, 0, 0], '0.000683_70.559241_1.502630_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001100_77.615165_1.502630_426.883410_0.000000_0.100000': [2518.764579789528, 277.5323169441076, 0, 0, 0, 0, -118.62746086996094, 0], '0.001100_70.559241_1.502630_426.883410_0.000000_0.100000': [0, 0, 0, 0, 13.549790737990975, 0, 0, 0], '0.001000_70.559241_1.502630_426.883410_0.000000_0.100000': [145.6568210435666, 0, 0, -396.77093840713815, -0.05019603246180395, 0, -2240.4385670741485, -1582.3372785020938], '0.001000_58.313423_1.502630_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0.023333476022189416, 0, 0], '0.001000_58.313423_1.366027_469.571751_0.000000_0.100000': [0.021112025125272795, 0, -0.7164363707788425, 0, 0, 0, 0, 0], '0.001100_58.313423_1.366027_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0.7401245559910195, 0, 0], '0.001100_58.313423_1.241843_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0.024216500041961807, 0], '0.001100_64.144765_1.241843_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000909_85.376682_2.000000_516.528926_0.000000_0.100000': [1025.8170694661007, 0, 0, 0, 0, 0, 0, 0], '0.000909_77.615165_2.000000_516.528926_0.000000_0.100000': [0, 4063.6881086409294, -308.7876659785151, 0, 0, 0, 0, 0], '0.001000_64.144765_1.818182_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 0.8004242860416025, 0, 0], '0.001000_64.144765_1.652893_568.181818_0.000000_0.100000': [-0.759792409361603, -0.7386527168228441, 0, 0, 0, 0, 0, 0], '0.001000_58.313423_1.652893_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0.025240053770289705, 0, 0, 0], '0.000909_58.313423_1.652893_568.181818_0.000000_0.100000': [0, 0.7605466471443606, 0, 0, 0, 0, 0, 0], '0.000909_58.313423_1.652893_516.528926_0.000000_0.100000': [-0.04600843608127434, 0, 0, -0.7552946440229334, 0, 0, 0, 0], '0.000909_58.313423_1.818182_516.528926_0.000000_0.100000': [0, 0, 0, -0.1590114405849363, 0, 0, 0, 0], '0.000909_58.313423_2.000000_516.528926_0.000000_0.100000': [0, 0.2208841089264297, 0, 0, 0, 0, 0, 0], '0.000909_58.313423_2.000000_469.571751_0.000000_0.100000': [0, 0, 0, 0, -0.045319215910649295, 0, 0, 0], '0.000826_58.313423_2.000000_469.571751_0.000000_0.100000': [0, 0, 0, 0.729598376103413, 0, 0, 0, -0.032650183641817425], '0.000826_58.313423_2.200000_469.571751_0.000000_0.100000': [0.04320425927924134, 0, 0, -0.006412015222804257, 0, 0, 0, 0], '0.000826_58.313423_2.420000_469.571751_0.000000_0.100000': [-0.6894226944361208, 0, 0, 0, 0, 0, 0, 0], '0.000826_53.012202_2.420000_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001000_103.305785_1.241843_516.528926_0.000000_0.100000': [333.0736018038739, 0, -0.17476362697738285, 0, 0, 0, 0, 0], '0.001000_93.914350_1.241843_516.528926_0.000000_0.100000': [1259.0048566349192, 0, 0, -2.2076603644760326, 0, 0, 0, -404.24868518486556], '0.001000_77.615165_1.366027_516.528926_0.000000_0.100000': [1173.4876080181125, 0, 0, 978.0740524767056, 0, 0, 0, 0], '0.000909_64.144765_1.502630_516.528926_0.000000_0.100000': [0.150007612244558, 0, 0, 0, 0, 0, -2010.4813789523946, 0], '0.000909_70.559241_1.652893_516.528926_0.000000_0.100000': [0, 295.6645500522893, 0, 0, 0, 0, 0, 0], '0.000909_70.559241_1.652893_469.571751_0.000000_0.100000': [2996.5787252654854, 0, 0, 0, 0, 0, 0, 0], '0.000826_93.914350_2.000000_516.528926_0.000000_0.100000': [1762.9285152841474, 0, 0, 0, 0, 0, 0, -567.3847255161389], '0.000826_93.914350_2.000000_568.181818_0.000000_0.100000': [117.93809451390135, 0, 0, 0, 4.487758146821579, 0, -360.36199826550654, -592.9833618892432], '0.000751_93.914350_2.000000_568.181818_0.000000_0.100000': [1167.2870652496124, 0, 0, 0, 0, 0, 0, 0], '0.000751_85.376682_2.000000_568.181818_0.000000_0.100000': [0, 0, -4.070917318202828, 0, 0, 0, 0, 0], '0.001000_93.914350_1.241843_568.181818_0.000000_0.100000': [4173.134225862843, 0, 0, -62.62427383780314, 1.2837906313117855, -272.6975191193496, 0, 0], '0.000909_93.914350_1.818182_516.528926_0.000000_0.100000': [1554.8446979642843, 799.384290863662, 0, 0, 478.5568534046234, 0, 0, 0], '0.000826_85.376682_2.000000_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000683_125.000000_2.000000_516.528926_0.000000_0.100000': [0, 0, 0, 0, 4.241904500377423, 0, 0, 0], '0.000621_125.000000_2.000000_568.181818_0.000000_0.100000': [0, 0, 0, -1242.550717126443, 0, 0, 0, 0], '0.000621_125.000000_2.200000_568.181818_0.000000_0.100000': [0, 0, 0, -524.66907473625, 0, 0, 0, 0], '0.000621_125.000000_2.420000_568.181818_0.000000_0.100000': [0, 478.4447348439024, 0, 0, 0, 0, 0, 0], '0.000621_125.000000_2.420000_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -478.4447348439024], '0.001100_85.376682_2.000000_516.528926_0.000000_0.100000': [1031.5082963729174, 0, -6.193680380950082, 0, 0, 0, -357.65619282723526, 0], '0.001210_85.376682_2.000000_516.528926_0.000000_0.100000': [1047.5923751828468, 0, -6.8238370814679, 0, 0, 14.363501945002008, 0, 262.20718020760944], '0.001331_85.376682_2.000000_516.528926_0.000000_0.100000': [940.4641830558589, 0, -7.51912527715158, 0, 0, 0, 0, 0], '0.001464_85.376682_2.000000_516.528926_0.000000_0.100000': [944.438103779329, 0, -8.286519748137199, 0, 0, 0, 0, 0], '0.001611_85.376682_2.000000_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 1312.1457291884203, 0, 0], '0.001611_85.376682_1.818182_516.528926_0.000000_0.100000': [-571.6485588399813, 0, 0, 0, 0, -706.7021116173906, 0, 0], '0.001611_85.376682_1.652893_516.528926_0.000000_0.100000': [232.06908336304613, 0, 0, 0, 0, 0, 0, 0], '0.001611_77.615165_1.652893_516.528926_0.000000_0.100000': [0, 390.5947438310844, 0, 0, 0, 0, 0, 0], '0.001611_77.615165_1.652893_469.571751_0.000000_0.100000': [0, 428.0295735990603, 0, 0, 0, 0, 0, 0], '0.001611_77.615165_1.652893_426.883410_0.000000_0.100000': [1135.8262630936858, 0, -1.5509465954856752, 0, 0, 0, 0, 0], '0.001772_77.615165_1.652893_426.883410_0.000000_0.100000': [0, 404.92669858787315, 0, 0, 0, 0, 0, 0], '0.001772_77.615165_1.652893_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001210_85.376682_2.000000_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -452.94747705205464], '0.001210_85.376682_2.000000_469.571751_0.000000_0.100000': [0, 0, 0, 0, 2164.169253067067, 0, 0, 0], '0.001210_113.636364_1.652893_426.883410_0.000000_0.100000': [278.5551479300088, 1001.0296000717931, 0, -216.7285690810214, 0, 53.62130810430945, -307.98985165338576, 0], '0.001100_77.615165_1.652893_426.883410_0.000000_0.100000': [0, 409.46334234215186, 0, 0, 0, 0, 0, 0], '0.001100_77.615165_1.652893_388.075827_0.000000_0.100000': [0, 0, -1.7938937432220883, 0, 0, 0, 0, 0], '0.001210_77.615165_1.652893_388.075827_0.000000_0.100000': [0, 402.48348584188125, 0, 0, 0, 0, 0, 0], '0.001210_77.615165_1.652893_352.796206_0.000000_0.100000': [2200.853599618852, 0, 0, 0, 0, 0, 0, 0], '0.001210_70.559241_1.652893_352.796206_0.000000_0.100000': [1233.0235084246503, 0, 0, 0, 0, 1032.0297679084, 0, 0], '0.000909_113.636364_1.241843_568.181818_0.000000_0.100000': [0, 0, 0, 0, 1.2111856247240211, 0, 0, 0], '0.000826_113.636364_1.241843_568.181818_0.000000_0.100000': [614.123370379451, 191.55033522708206, 0, 0, 0, -166.09270645860016, 0, 0], '0.000826_113.636364_1.128948_568.181818_0.000000_0.100000': [0, 404.21030432106517, -1.0801930823518888, 0, 0, 0, 0, 0], '0.000909_113.636364_1.128948_568.181818_0.000000_0.100000': [0, 0, 0, 0, 1.0801930823518888, 0, 0, 0], '0.000826_113.636364_1.128948_516.528926_0.000000_0.100000': [0, 396.79925175055035, 0, 0, 0, 0, 0, 0], '0.000826_113.636364_1.128948_469.571751_0.000000_0.100000': [0, 380.6987024778988, 0, 0, 0, 0, 0, 0], '0.000826_113.636364_1.128948_426.883410_0.000000_0.100000': [0, 355.0034357283281, 0, 0, 0.8449395612435183, 0, -255.44523008750977, 0], '0.000826_113.636364_1.128948_388.075827_0.000000_0.100000': [0, 0, 0, 57.01192359557581, 0, 0, 0, 0], '0.000826_113.636364_1.241843_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -331.21858343716997], '0.000826_113.636364_1.241843_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, -80.79677588673394, 0, 0], '0.000751_113.636364_1.128948_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000751_125.000000_2.200000_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -397.9217956765868, 0], '0.000751_137.500000_2.200000_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, 1075.825482267813, 0, 0], '0.000751_137.500000_2.000000_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, 1032.3751197289785, 0, 0], '0.000751_125.000000_1.818182_388.075827_0.000000_0.100000': [0, 0, 0, 0, 2.2755673392176323, 0, 0, 0], '0.000683_125.000000_1.818182_388.075827_0.000000_0.100000': [0, 0, 0, -102.4378135131715, 0, 0, 0, 0], '0.000683_93.914350_2.000000_388.075827_0.000000_0.100000': [1245.0154632947374, 0, 0, 0, 0, 0, 0, 0], '0.000683_77.615165_2.200000_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -528.5795511185846], '0.000683_77.615165_2.200000_426.883410_0.000000_0.100000': [1525.3122496766698, 0, 0, 0, 0, 0, 0, 0], '0.000683_70.559241_2.200000_426.883410_0.000000_0.100000': [3478.3423715946956, 0, 0, 0, 0, 0, 0, 0], '0.000683_64.144765_2.200000_426.883410_0.000000_0.100000': [0, 0, 0, 0.0020174717547888576, 0, 0, 0, 0], '0.000683_64.144765_2.420000_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -5919.593526587121, 0], '0.000683_70.559241_2.420000_426.883410_0.000000_0.100000': [0, 0, 0, 1438.4873528750427, 0, 0, 0, 0], '0.000683_70.559241_2.662000_426.883410_0.000000_0.100000': [0, 0, -7.916397964683438, 0, 0, 0, 0, 0], '0.000751_70.559241_2.662000_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001100_85.376682_1.502630_568.181818_0.000000_0.100000': [349.86304141684195, 0, 0, 0, 0, 0, 0, 0], '0.001100_77.615165_1.502630_568.181818_0.000000_0.100000': [5245.518842643799, 0, 0, 644.9355596188806, 0, 0, 0, 0], '0.001100_70.559241_1.502630_568.181818_0.000000_0.100000': [33.929810761428, 0, 0, 0, 0, 0, 0, -3629.6361380007406], '0.001100_70.559241_1.502630_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001000_93.914350_1.502630_320.723824_0.000000_0.100000': [0, 0, 0, 596.2502973063862, 0, 471.60498833000065, 0, 0], '0.001000_93.914350_1.366027_320.723824_0.000000_0.100000': [669.1685941223622, 0, 1091.3039186997876, 0, 0, 0, 0, 0], '0.000909_151.250000_1.652893_388.075827_0.000000_0.100000': [386.1708314461509, 0, 0, 21.572575105675373, 0, 0, 0, 0], '0.000909_125.000000_1.652893_388.075827_0.000000_0.100000': [0, 0, -4.692639537846972, 0, 0, 28.472409032542238, 0, 0], '0.000909_125.000000_1.502630_388.075827_0.000000_0.100000': [411.3211192051237, 427.3702206023243, 0, 0, 0, 0, 0, 0], '0.000909_113.636364_1.502630_388.075827_0.000000_0.100000': [0, 513.8480139903915, 0, 0, 0, 0, 0, 0], '0.000909_113.636364_1.502630_352.796206_0.000000_0.100000': [0, 417.7994347855999, 0, 0, 4.450917810415877, 0, 0, 0], '0.000826_113.636364_1.502630_352.796206_0.000000_0.100000': [0, 0, 0, 0, 3.6477606164731697, 932.8194897018094, 0, -455.27690109325584], '0.000751_113.636364_1.502630_352.796206_0.000000_0.100000': [0, 458.84519603026763, 0, 0, 0, 0, 0, 0], '0.000751_113.636364_1.502630_320.723824_0.000000_0.100000': [420.2507443062445, 0, 0, 0, 0, 0, 0, 0], '0.000826_151.250000_1.652893_625.000000_0.000000_0.100000': [0, 450.4887927022608, 0, 0, 0, 0, 0, 0], '0.000826_151.250000_1.502630_568.181818_0.000000_0.100000': [300.2693168372645, 0, 0, 0, 0, 0, 0, 0], '0.000751_125.000000_1.652893_469.571751_0.000000_0.100000': [0, 0, 0, 0, 423.79966527789185, 0, 0, -13.73419814768505], '0.000683_137.500000_1.652893_426.883410_0.000000_0.100000': [0, 387.1594360821188, 0, 0, 0, 0, 0, 0], '0.000683_137.500000_1.652893_388.075827_0.000000_0.100000': [0, 0, 0, -317.3204480699161, 0, 0, 0, 0], '0.000683_137.500000_1.818182_388.075827_0.000000_0.100000': [0, 363.71595870609934, 0, 0, 0, 0, 0, 0], '0.000683_137.500000_1.818182_352.796206_0.000000_0.100000': [0, 351.60508491111534, 0, 0, 0, 0, 0, 0], '0.000683_137.500000_1.818182_320.723824_0.000000_0.100000': [0, 347.6516609602715, 0, 0, 0, 0, 0, 0], '0.000683_137.500000_1.818182_291.567113_0.000000_0.100000': [0, 395.8719128646106, 0, 0, 0, 0, 0, 0], '0.000683_137.500000_1.818182_265.061011_0.000000_0.100000': [0, 0, 0, 0, 0, 197.9914961872495, 0, 0], '0.000683_137.500000_1.652893_265.061011_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001210_151.250000_1.652893_426.883410_0.000000_0.100000': [0, 763.3235418978858, 0, 0, 0, 0, 0, 0], '0.001210_151.250000_1.652893_388.075827_0.000000_0.100000': [755.1794488073715, 0, 0, 0, 0, 0, 0, 0], '0.001210_137.500000_1.652893_388.075827_0.000000_0.100000': [0, 729.0508441599206, 0, 0, 0, 0, 0, 0], '0.001210_137.500000_1.652893_352.796206_0.000000_0.100000': [398.37090117405586, 0, 0, 0, 2.374105615481858, 0, 0, 0], '0.001100_137.500000_1.652893_352.796206_0.000000_0.100000': [0, 0, -2.374105615481858, 0, 0, 0, 0, 0], '0.001210_125.000000_1.652893_352.796206_0.000000_0.100000': [313.5553284782018, 0, 0, 0, 0, 0, 0, 0], '0.000683_103.305785_1.502630_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -366.2697207906891, 0], '0.000683_113.636364_1.502630_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 960.9934495655617, 0, 0], '0.000683_113.636364_1.366027_568.181818_0.000000_0.100000': [414.1730257039097, 0, 0, 0, 0, 0, -456.29813103698683, 0], '0.000683_125.000000_1.366027_568.181818_0.000000_0.100000': [456.29813103698683, 0, 0, 0, 0, 0, 0, 0], '0.000683_103.305785_1.366027_568.181818_0.000000_0.100000': [0, 0, 0, 0, 3.225884840145227, 0, 0, 0], '0.000621_103.305785_1.366027_568.181818_0.000000_0.100000': [0, 0, 0, 0, 2.929492455503896, 0, 0, 0], '0.000564_103.305785_1.366027_568.181818_0.000000_0.100000': [0, 0, 0, 0, 2.660664786849793, 0, 0, 0], '0.000513_103.305785_1.366027_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001210_103.305785_1.502630_469.571751_0.000000_0.100000': [0, 507.34379364833643, 0, 0, 0, 0, 0, 0], '0.001210_103.305785_1.502630_426.883410_0.000000_0.100000': [0, 0, 0, 0, 4.8316183672021, 0, 0, 0], '0.001100_103.305785_1.502630_426.883410_0.000000_0.100000': [1466.3528757906263, 909.4404525743994, 0, 0, 4.482120898529502, 0, 0, 0], '0.001100_85.376682_1.652893_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 44.53507194359414], '0.001331_137.500000_1.818182_756.250000_0.000000_0.100000': [0, 502.66358902302744, 0, 0, 0, 0, 0, 0], '0.001464_125.000000_1.818182_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -347.2557050420619, 0], '0.001464_137.500000_1.818182_625.000000_0.000000_0.100000': [0, 0, 0, 0, 870.4483202546969, 0, 0, 0], '0.001331_77.615165_1.502630_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001100_125.000000_1.652893_469.571751_0.000000_0.100000': [0, 417.5445648447662, 0, 577.0366724007009, -305.8009358801472, 0, 0, 0], '0.000909_113.636364_1.502630_426.883410_0.000000_0.100000': [0, 0, 0, 0, 822.8981617094873, 0, 0, 0], '0.000909_93.914350_1.366027_426.883410_0.000000_0.100000': [1575.7210333527978, 0, 0, 0, 0, 0, 0, 0], '0.000826_85.376682_2.200000_687.500000_0.000000_0.100000': [0, 709.2668526781179, 0, 0, 0, 0, 0, 0], '0.000826_64.144765_2.200000_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 0.0412904111315811, 0, 0], '0.000826_77.615165_2.000000_625.000000_0.000000_0.100000': [0, -285.1691102409824, 0, 0, 0, 0, 0, 0], '0.000751_77.615165_2.000000_568.181818_0.000000_0.100000': [0, 0, 2394.1750221511993, 0, 0, 0, 0, 0], '0.000909_70.559241_2.200000_568.181818_0.000000_0.100000': [0, 0, 5.557818397277874, 0, 0, 0, 0, 0], '0.001210_137.500000_2.000000_516.528926_0.000000_0.100000': [2536.2904086424323, 0, 0, -1185.627869323241, 0, 0, -480.3003544099829, 0], '0.001210_137.500000_2.200000_516.528926_0.000000_0.100000': [383.5869441670346, 0, 0, 0, 0, 0, 0, 0], '0.001210_103.305785_2.420000_469.571751_0.000000_0.100000': [306.010372212188, 0, 0, 0, 0, 0, 0, 0], '0.001331_93.914350_2.420000_426.883410_0.000000_0.100000': [0, 0, 0, 0, 2.947134344883416, 0, 0, 0], '0.001210_103.305785_2.420000_426.883410_0.000000_0.100000': [0, 0, -3.0703893318253903, 0, 0, 0, 0, 0], '0.001331_103.305785_2.420000_426.883410_0.000000_0.100000': [0, 0, 0, 0, 3.0703893318253903, 0, 0, 0], '0.001210_125.000000_1.366027_687.500000_0.000000_0.100000': [0, 0, 0, 0, 5.926367582079002, 0, 0, 0], '0.001100_125.000000_1.366027_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -483.43876487115176], '0.001210_125.000000_1.241843_756.250000_0.000000_0.100000': [0, 442.2887922950393, 0, 0, 0, 0, 0, 0], '0.001210_125.000000_1.241843_687.500000_0.000000_0.100000': [295.9186326233291, 0, 0, 0, 3.469403947457431, 0, 0, 0], '0.001210_113.636364_1.241843_687.500000_0.000000_0.100000': [0, 0, -2.885757674580964, 0, 0, 0, 0, 0], '0.001331_113.636364_1.241843_687.500000_0.000000_0.100000': [0, 0, 0, -570.4477966516735, 0, 0, 0, 0], '0.001331_113.636364_1.366027_687.500000_0.000000_0.100000': [0, 0, -7.374302932503269, 0, 0, 0, 0, 0], '0.001464_113.636364_1.366027_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -420.2279078384896, 0], '0.001464_125.000000_1.366027_687.500000_0.000000_0.100000': [0, 0, -7.939390142592674, 0, 0, 0, 0, 0], '0.001611_125.000000_1.366027_687.500000_0.000000_0.100000': [0, 0, -8.755563159402845, 0, 0, 0, 0, 0], '0.001772_125.000000_1.366027_687.500000_0.000000_0.100000': [0, 514.1404530581087, 0, 0, 0, 0, 0, 0], '0.001772_125.000000_1.366027_625.000000_0.000000_0.100000': [0, 0, -10.031322083230043, 0, 0, 0, 0, -46.740041187100786], '0.001949_125.000000_1.366027_625.000000_0.000000_0.100000': [0, 0, 0, 0, 10.031322083230043, 0, 0, 0], '0.001000_93.914350_1.818182_426.883410_0.000000_0.100000': [488.77752395003523, 0, 0, -219.49841248977282, 0, 0, 0, 0], '0.001000_70.559241_2.000000_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 1831.1526837682, 0, 0], '0.001000_70.559241_1.818182_388.075827_0.000000_0.100000': [110.21391311864886, 0, 0, 0, 0, 0, -3248.797258804313, -41.7703028773019], '0.001000_64.144765_1.818182_388.075827_0.000000_0.100000': [-0.6709037234739071, 0, 0, 0, 0, 0, -10.019446647149898, 0], '0.001000_58.313423_1.818182_388.075827_0.000000_0.100000': [0.7059702492364978, 0, 0, 0, 0, 0, 0, 0], '0.001000_53.012202_1.818182_388.075827_0.000000_0.100000': [-0.6593173645755769, -0.00021942323173789193, 0, 0, -0.007172027592514497, 0, 0, 0], '0.000909_53.012202_1.818182_388.075827_0.000000_0.100000': [0, 0, 0.03944615175882973, 0, 0, 0, 0, 0], '0.001000_48.192911_1.818182_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0.6213637390894065], '0.001000_48.192911_1.818182_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -0.6845776632042772, 0], '0.001000_53.012202_1.818182_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000826_103.305785_2.200000_388.075827_0.000000_0.100000': [0, 0, 0, -958.5323774825853, 1330.245912576393, 0, 0, 0], '0.000751_103.305785_2.200000_388.075827_0.000000_0.100000': [2566.462285176105, 0, 0, 0, 0, 0, -404.0359450483449, 0], '0.000751_113.636364_2.200000_388.075827_0.000000_0.100000': [0, 0, 0, -876.0910604292775, 0, 0, 0, 0], '0.000751_113.636364_2.420000_388.075827_0.000000_0.100000': [0, 0, 0, 0, 2.345242453240935, 0, 0, 0], '0.000683_113.636364_2.420000_388.075827_0.000000_0.100000': [0, 0, -0.2132038593855394, 0, 0, 877.8378148138221, 0, 0], '0.000683_113.636364_2.200000_388.075827_0.000000_0.100000': [792.0739609097166, 0, 0, 0, 0, 0, 0, 0], '0.000683_85.376682_2.200000_388.075827_0.000000_0.100000': [0, 0, 0, 0, 3.9493001851502414, 0, 0, 0], '0.000621_85.376682_2.200000_388.075827_0.000000_0.100000': [0, 0, 0, -1201.1445339683344, 0, 0, 0, 0], '0.000621_85.376682_2.420000_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000826_93.914350_2.000000_469.571751_0.000000_0.100000': [0, 0, 615.7514579642165, 0, 0, 0, 0, 0], '0.000826_85.376682_2.200000_426.883410_0.000000_0.100000': [0, 522.1206380928564, 0, 0, 0, 0, 0, 0], '0.000826_85.376682_2.200000_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 1787.4153949492832, 0, 0], '0.000826_77.615165_2.000000_388.075827_0.000000_0.100000': [1232.3256678028506, 275.57730779564645, 0, 0, 0, -477.6401660059126, 0, 0], '0.000826_77.615165_1.818182_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, -61.420481067178024, 0, 0], '0.000826_77.615165_1.652893_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 955.6231879469863, 0], '0.001100_125.000000_1.366027_469.571751_0.000000_0.100000': [0, 449.4812788009349, 0, 0, 0, 0, 0, 0], '0.001100_125.000000_1.366027_426.883410_0.000000_0.100000': [0, 456.5525352611274, 0, 0, 0, 0, 0, 0], '0.001000_125.000000_1.366027_388.075827_0.000000_0.100000': [0, 0, 0, -730.2827677841775, 0, 0, 0, 0], '0.000909_125.000000_1.502630_352.796206_0.000000_0.100000': [442.97557540994603, 0, 0, 0, 0, 0, 0, 0], '0.000909_113.636364_1.502630_320.723824_0.000000_0.100000': [0, 0, 0, -290.0868640051232, 0, 604.2878699751976, 0, 0], '0.000909_113.636364_1.366027_320.723824_0.000000_0.100000': [337.59145572160344, 0, 0, 0, 32.65112760818921, 31.495295431199064, 0, 0], '0.000909_103.305785_1.366027_320.723824_0.000000_0.100000': [0, 0, 0, -466.94516350353564, 0, 0, 0, 0], '0.000909_103.305785_1.502630_320.723824_0.000000_0.100000': [0, 0, 0, 0, 4.179326001887757, 0, 0, 0], '0.000826_103.305785_1.502630_320.723824_0.000000_0.100000': [378.0906083268188, 0, 0, 0, 0, 0, 0, 0], '0.001000_113.636364_1.652893_352.796206_0.000000_0.100000': [0, 0, 0, -167.19216191299265, 0, 0, 0, 0], '0.000909_103.305785_1.818182_352.796206_0.000000_0.100000': [0, 1030.0702049775832, 3315.6209381996487, -338.95151680181436, 0, 0, 0, 0], '0.000909_103.305785_1.818182_320.723824_0.000000_0.100000': [0, 327.7799496211455, 1087.290447327083, -317.3565926286727, 1390.2539699745178, 0, 0, 0], '0.000909_103.305785_2.000000_320.723824_0.000000_0.100000': [0, 0, -4.2466058846561285, 0, 0, 0, 0, 0], '0.001000_103.305785_2.000000_320.723824_0.000000_0.100000': [378.5683885691191, 0, 0, 0, 0, 0, 0, -393.4701182349698], '0.001000_103.305785_2.000000_352.796206_0.000000_0.100000': [388.1996655641063, 0, 0, 0, 0, 0, 0, 0], '0.001000_93.914350_1.366027_687.500000_0.000000_0.100000': [2697.2097586957666, 0, 0, 0, 4.959590759275398, 222.9156364623046, 0, 0], '0.001000_93.914350_1.241843_687.500000_0.000000_0.100000': [2303.9331703433068, 421.8462422903693, 0, 0, 0, 0, 0, 0], '0.001000_85.376682_1.241843_687.500000_0.000000_0.100000': [1845.1244450640322, 0, 437.5710536110849, -101.94648813017511, 0, 0, 0, 0], '0.001000_70.559241_1.366027_687.500000_0.000000_0.100000': [0, 0, 0, 2476.8118539131065, 0, 0, 0, 0], '0.001000_64.144765_1.502630_568.181818_0.000000_0.100000': [0, 0.7428953156731335, 0.04932060524327812, 0, 0, 0, 0, 0], '0.001100_64.144765_1.502630_568.181818_0.000000_0.100000': [0.7219002375116127, 0, 0, 0, -0.04487730747361343, 0, 0, 0], '0.001210_103.305785_1.652893_388.075827_0.000000_0.100000': [0, 2980.121882382157, 0, 0, 1.2736696811482033, 0, -297.75811003067025, 0], '0.001210_113.636364_1.502630_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001611_113.636364_1.652893_625.000000_0.000000_0.100000': [0, 0, 424.8617226224026, 0, 0, 0, 0, 0], '0.001611_113.636364_1.652893_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, -85.7024013254646, 0, 0], '0.001611_113.636364_1.502630_568.181818_0.000000_0.100000': [353.61338236045486, 0, 0, 0, 0, 0, 0, 0], '0.001611_103.305785_1.502630_568.181818_0.000000_0.100000': [0, 0, 0, 0, 5.540985388370973, 0, 0, 0], '0.001464_103.305785_1.366027_568.181818_0.000000_0.100000': [370.916736736044, 0, 0, 0, 0, 0, 0, 0], '0.001464_93.914350_1.366027_568.181818_0.000000_0.100000': [0, 0, 0, 0, 6.880881142022554, 0, 0, 0], '0.001331_93.914350_1.366027_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -370.77009184880944, 0], '0.001331_103.305785_1.366027_568.181818_0.000000_0.100000': [0, 0, 0, 487.69273377410417, 0, 0, 0, 0], '0.001331_93.914350_1.652893_568.181818_0.000000_0.100000': [0, 428.26552939774604, 0, 0, 0, 0, 0, 0], '0.001331_93.914350_1.652893_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000909_103.305785_2.420000_516.528926_0.000000_0.100000': [0, 0, 0, 0, 1.7577303140904406, 0, 0, 0], '0.000826_103.305785_2.420000_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 780.982340393355, 0, 0], '0.000826_103.305785_2.200000_516.528926_0.000000_0.100000': [0, 0, 0, 0, 4.862985020218185, 0, -405.0886200436298, 0], '0.000683_103.305785_2.200000_426.883410_0.000000_0.100000': [0, 0, -4.42088165374571, 0, 0, 0, 0, 0], '0.000751_93.914350_2.200000_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, 1516.2691035323091, 0, 0], '0.000751_93.914350_1.818182_469.571751_0.000000_0.100000': [876.6139715322763, 0, 0, 0, 0, 0, 0, -477.0587300929546], '0.000751_93.914350_1.818182_516.528926_0.000000_0.100000': [1358.7936110300866, 0, 0, 0, 0, 0, 0, 0], '0.000751_85.376682_1.818182_516.528926_0.000000_0.100000': [-829.03769588891, 30.15230316989805, 0, 0, 0, 0, 0, 0], '0.000826_103.305785_1.818182_756.250000_0.000000_0.100000': [0, 806.5282133807291, 0, 0, 0, 0, 0, 0], '0.000826_85.376682_2.420000_625.000000_0.000000_0.100000': [0, 607.4809408058986, 0, 0, 0, 0, 0, 0], '0.000683_103.305785_1.652893_388.075827_0.000000_0.100000': [246.3746143959073, 0, 0, -495.80546992221423, 0, 0, 0, 0], '0.000683_93.914350_1.652893_388.075827_0.000000_0.100000': [197.97096632520234, 0, 0, -20.985137953578754, 0, 0, 0, 0], '0.000751_85.376682_1.818182_469.571751_0.000000_0.100000': [0, 0, -2.3961763693904685, 0, 2.167069975710183, 113.80459409266177, 0, 0], '0.000683_85.376682_1.818182_469.571751_0.000000_0.100000': [0, 0, -2.167069975710183, 0, 0, 0, 0, 0], '0.001100_77.615165_2.000000_516.528926_0.000000_0.100000': [-341.9632033493399, 4786.947560788099, 0, -612.5140245647931, 0, 0, -93.77348148844705, 0], '0.001100_70.559241_2.000000_516.528926_0.000000_0.100000': [0, 4658.033875868337, -5.653903200395234, 0, 0, 0, 0, 0], '0.001100_64.144765_2.000000_469.571751_0.000000_0.100000': [0.7446194730182669, 0, -0.01747005945219371, 0, 0, 0, -43.98078963928162, 0], '0.001000_113.636364_1.366027_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 559.7738305086674, 0, 0], '0.001000_103.305785_1.241843_687.500000_0.000000_0.100000': [0, 0, 0, -81.87332782155714, 0, 0, 0, 0], '0.000909_93.914350_1.366027_687.500000_0.000000_0.100000': [877.805623088458, 0, 0, 0, 0, 0, -357.17084433828296, 0], '0.000909_103.305785_1.366027_687.500000_0.000000_0.100000': [357.17084433828296, 0, 0, 0, 4.433080330241319, 0, -392.18407888168, 0], '0.000826_103.305785_1.366027_687.500000_0.000000_0.100000': [392.98558519041217, 0, -4.433080330241319, 0, 0, 0, 0, 0], '0.000826_93.914350_1.366027_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 214.6907063192779, -357.2596229003747, 0], '0.000826_93.914350_1.241843_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0.5133355146367649, 0, 0, 0], '0.000751_93.914350_1.241843_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, -330.7344971946413, 0, 0], '0.000751_93.914350_1.128948_687.500000_0.000000_0.100000': [0, 450.9984922611794, 0, 0, 0, 0, 0, 0], '0.000751_93.914350_1.128948_625.000000_0.000000_0.100000': [114.66043050515246, 0, 0, 0, 0, 0, 0, 0], '0.000751_85.376682_1.128948_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, -293.075397716074, 0, 0], '0.000751_85.376682_1.026316_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001331_113.636364_1.818182_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 204.13438749234055, 0, 0], '0.001331_113.636364_1.652893_388.075827_0.000000_0.100000': [0, 1004.975738042332, 0, 0, 0, 0, -319.3086133820394, 0], '0.001331_103.305785_1.652893_320.723824_0.000000_0.100000': [280.99087814722105, 0, 0, 0, 1.469452373060358, 0, 0, -346.0078611410387], '0.001331_103.305785_1.652893_352.796206_0.000000_0.100000': [0, 0, -1.683400572137998, -89.95841989891814, 0, 0, 0, 0], '0.001464_103.305785_1.652893_352.796206_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001000_125.000000_1.366027_625.000000_0.000000_0.100000': [0, 456.19203005149757, 0, 0, 0, 0, 0, 0], '0.001000_125.000000_1.241843_568.181818_0.000000_0.100000': [1748.8425848502188, 0, 0, -582.4281690280258, 0, 0, -354.48695456132657, 0], '0.001000_137.500000_1.241843_568.181818_0.000000_0.100000': [0, 0, 0, 0, 3.058595549746042, 0, 0, 0], '0.000909_137.500000_1.241843_568.181818_0.000000_0.100000': [0, 381.7062290291455, 0, 0, 0, 0, 0, 0], '0.000909_137.500000_1.241843_516.528926_0.000000_0.100000': [689.492918224645, 0, 0, 0, 0, 0, 0, 0], '0.000826_113.636364_1.241843_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -204.3288415483482], '0.000826_103.305785_1.241843_568.181818_0.000000_0.100000': [0, 438.8901474952411, 0, 0, 0, 0, 0, 0], '0.000826_103.305785_1.241843_516.528926_0.000000_0.100000': [0, 0, 0, -159.74175437509967, 0, 0, -593.2931698424399, 0], '0.000826_103.305785_1.366027_516.528926_0.000000_0.100000': [374.39466649215683, 0, 0, -995.3456158513242, 0, 0, 0, 0], '0.001000_103.305785_2.420000_516.528926_0.000000_0.100000': [0, 0, -2.1677598935866627, 0, 0, 0, 0, 0], '0.001100_103.305785_2.420000_516.528926_0.000000_0.100000': [0, 0, -2.4100937143719423, 0, 0, 0, 0, 0], '0.001210_103.305785_2.420000_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 1223.0613748706683, 0, 0], '0.000909_103.305785_2.200000_388.075827_0.000000_0.100000': [430.1735054545514, 0, 0, 0, 0, 1198.2909586438598, 0, 0], '0.000909_85.376682_2.420000_388.075827_0.000000_0.100000': [322.23077349478865, 0, 0, 0, 0, 0, -294.66377932590257, 0], '0.000909_77.615165_2.420000_388.075827_0.000000_0.100000': [1506.694162826534, 0, 0, 0, 0, 0, 0, 0], '0.000909_70.559241_2.420000_388.075827_0.000000_0.100000': [4232.313189816386, 0, 0, 0, 0, 0, 0, 0], '0.000909_64.144765_2.420000_388.075827_0.000000_0.100000': [0, 0, 0, -0.04186487977442539, 0, 0, 0, 0], '0.000909_64.144765_2.662000_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001210_64.144765_2.000000_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 6331.6816067196305, -247.2223312078236, 6330.986912269377], '0.001210_64.144765_1.818182_687.500000_0.000000_0.100000': [0, 0, 0, -6331.6816067196305, -0.673373526679302, 0, 0, 0], '0.001464_64.144765_2.200000_687.500000_0.000000_0.100000': [0.05808604714725334, 0, 0, 0, 0, 0, 0, 0], '0.001464_58.313423_2.200000_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 0.032859436129206186, 0, 0], '0.001464_58.313423_2.000000_687.500000_0.000000_0.100000': [0, 0, 0, -0.00328594361292062, 0, 0, 0, 0], '0.000909_103.305785_2.000000_388.075827_0.000000_0.100000': [395.80383197386254, 0, 0, -1198.2909586438598, 0, 0, 0, -13.654831086373406], '0.001000_137.500000_2.000000_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 708.135859872256, 0, 0], '0.001000_137.500000_1.818182_388.075827_0.000000_0.100000': [644.4963116480227, 0, 0, -64.3759872611142, 0, 0, 0, 0], '0.001100_125.000000_1.652893_388.075827_0.000000_0.100000': [1205.670099389773, 0, 0, 0, 2.479829518396309, 0, 79.09759084932993, -396.1475993000094], '0.001210_137.500000_1.502630_388.075827_0.000000_0.100000': [897.3744097701501, 0, 0, 0, 0, 0, 0, 0], '0.001210_113.636364_1.502630_388.075827_0.000000_0.100000': [0, 458.95892772216723, -5.3132756858075485, 0, 5.075987555213487, 0, 0, 0], '0.000909_103.305785_1.366027_625.000000_0.000000_0.100000': [0, 0, 0, 0, 924.5419767021197, 0, 0, 0], '0.000909_85.376682_1.366027_687.500000_0.000000_0.100000': [0, 2244.6400278287842, 0, 0, 0, 0, 0, 0], '0.000909_77.615165_1.241843_568.181818_0.000000_0.100000': [3332.8709074497756, 0, 0, 0, 0, 0, 0, 0], '0.000909_70.559241_1.241843_568.181818_0.000000_0.100000': [62.19698090565867, 0, 56.11358502218679, 0, -44.94572983306232, 0, 0, 0], '0.000826_70.559241_1.241843_568.181818_0.000000_0.100000': [106.3979472326771, 0, 0, 0, 0, -1689.460578718006, 0, 0], '0.000826_70.559241_1.128948_568.181818_0.000000_0.100000': [0, 0, 0, 1689.460578718006, 0, 0, 0, 0], '0.000826_64.144765_1.241843_568.181818_0.000000_0.100000': [0, 0, 0.744763506043887, 0, 0, 0, 0, 0], '0.000909_64.144765_1.241843_568.181818_0.000000_0.100000': [0, 0, 0.03640011852485259, 0.04219901063425823, 0, 0, 0, 0], '0.000909_64.144765_1.366027_568.181818_0.000000_0.100000': [-0.8172659878244986, -0.7022554014096158, 0, 0, 0, 0, 0, 0], '0.000909_64.144765_1.366027_516.528926_0.000000_0.100000': [0.07799617017924447, 0, 0, -0.07364616791487606, 0, 0, 0, 0], '0.000909_58.313423_1.502630_516.528926_0.000000_0.100000': [0, 0, 0, 0, -0.1214645521949933, 0, 0, 0], '0.001100_151.250000_1.652893_568.181818_0.000000_0.100000': [167.97919428528053, 0, 0, 0, 0, -56.1158662501266, 0, 0], '0.001100_151.250000_1.502630_568.181818_0.000000_0.100000': [0, 0, -1.615730645211079, 0, 0, 0, 0, -396.0015387720323], '0.001210_151.250000_1.502630_568.181818_0.000000_0.100000': [0, 0, 0, 0, 1.615730645211079, 0, 0, 0], '0.001100_151.250000_1.502630_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -399.3383293762128], '0.001210_151.250000_1.502630_687.500000_0.000000_0.100000': [0, 0, 0, 0, 1.7126586363165188, 0, 0, 0], '0.001000_113.636364_1.502630_687.500000_0.000000_0.100000': [0, 1190.8729002929997, 0, 0, 0, 0, 0, 0], '0.001210_85.376682_2.200000_516.528926_0.000000_0.100000': [1743.8452120050138, 0, 0, 0, 0, 0, 0, 0], '0.001210_77.615165_2.200000_516.528926_0.000000_0.100000': [-903.0692564922966, -627.9531152643711, -5.7109021133141145, -2265.450535652071, 2074.6010375363617, 0, -15.696176525697865, 0], '0.001331_77.615165_2.200000_516.528926_0.000000_0.100000': [-905.1510552244474, 0, 0, 0, 0, 0, 0, -448.38378379343584], '0.001331_77.615165_2.200000_568.181818_0.000000_0.100000': [0, 0, 0, 0, 5.372922529396601, 0, 0, 0], '0.001210_77.615165_2.200000_568.181818_0.000000_0.100000': [5255.423774640915, 0, 0, 0, 0, 0, 0, -277.644916089225], '0.001210_77.615165_2.200000_625.000000_0.000000_0.100000': [0, 0, 0, -2645.8469791929356, 0, 0, 0, 0], '0.001210_77.615165_2.420000_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 2645.8469791929356, 0, 0], '0.000826_113.636364_2.000000_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 84.03718220636067], '0.001210_151.250000_2.000000_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -470.94071131357634], '0.001210_151.250000_2.000000_568.181818_0.000000_0.100000': [0, 0, 0, -1109.328398723057, 1237.4581915623858, 0, 0, 0], '0.001100_166.375000_1.652893_516.528926_0.000000_0.100000': [0, 0, 0, -17.28941800316228, 0, 0, 0, 0], '0.001100_137.500000_1.502630_568.181818_0.000000_0.100000': [0, 0, -1.9051439579608995, 0, 3.08578985212529, 0, 0, 0], '0.001100_137.500000_1.366027_568.181818_0.000000_0.100000': [0, 0, 0, -624.9559868343346, 0, 0, 0, 0], '0.001210_137.500000_1.502630_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 620.8571413011462, 0, 0], '0.001210_137.500000_1.366027_568.181818_0.000000_0.100000': [930.4680609206778, 0, 0, 0, 0, 0, 0, 0], '0.001210_125.000000_1.241843_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -379.24739275980573], '0.001210_125.000000_1.241843_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -356.1824880507121, 0], '0.001210_137.500000_1.241843_568.181818_0.000000_0.100000': [0, 382.777516565752, 0, 0, 0, 0, 0, 0], '0.001210_137.500000_1.241843_516.528926_0.000000_0.100000': [0, 0, 0, 0, 3.3358626055926837, 0, 0, 0], '0.001100_137.500000_1.241843_516.528926_0.000000_0.100000': [0, 0, 0, -605.9963921018634, 0, 0, 0, 0], '0.001100_103.305785_1.818182_352.796206_0.000000_0.100000': [0, 0, 0, 0, 47.18018889712221, 680.1867610567678, 0, 0], '0.001000_103.305785_1.652893_320.723824_0.000000_0.100000': [0, 0, 0, 0, 1.2503417850768073, 0, 0, -34.556031289111786], '0.000909_103.305785_1.652893_320.723824_0.000000_0.100000': [255.855778744305, 0, 0, 0, 0, 0, 0, 0], '0.001210_70.559241_2.200000_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 903.0692564922966, 0], '0.001210_77.615165_2.200000_469.571751_0.000000_0.100000': [0, 0, 0, -1118.9217265006885, 0, 0, 0, 0], '0.001210_70.559241_2.420000_469.571751_0.000000_0.100000': [0, 0, -3.681820824235183, 0, 0, 0, 0, 0], '0.001331_70.559241_2.420000_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -537.055459768899], '0.001331_70.559241_2.420000_516.528926_0.000000_0.100000': [6989.254502839046, 0, 0, 0, 0, 0, 0, 0], '0.001331_64.144765_2.420000_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0.04613491049831282], '0.001331_64.144765_2.420000_568.181818_0.000000_0.100000': [0, -0.04613491049831282, 0, 0, 0, 0, 0, 0], '0.001000_113.636364_2.000000_388.075827_0.000000_0.100000': [-183.79680624173733, 432.02238919077496, 0, 0, 0, 0, 0, 0], '0.001000_113.636364_2.000000_352.796206_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -432.022389190775], '0.001000_103.305785_2.000000_388.075827_0.000000_0.100000': [0, 0, 0, -331.9302838911978, 291.8050554957698, 0, 0, 0], '0.001100_103.305785_2.200000_352.796206_0.000000_0.100000': [0, 0, 0, 0, 5.653444003258573, 3111.737328125915, 0, 0], '0.001100_103.305785_2.000000_352.796206_0.000000_0.100000': [2107.2672488450794, 0, 0, -111.58639314789343, 0, 0, 0, 0], '0.001100_93.914350_2.000000_352.796206_0.000000_0.100000': [0, 2915.581338667258, 0, 0, 0, 0, 0, 0], '0.000826_70.559241_1.652893_568.181818_0.000000_0.100000': [6.052585187876134, -3273.2045528914564, 0, 0, 0, 0, -40.37050684589167, 0], '0.000826_70.559241_1.652893_516.528926_0.000000_0.100000': [3278.652468020506, 0, 0, 0, 0, 0, 0, 0], '0.001100_70.559241_2.000000_625.000000_0.000000_0.100000': [4822.9449839944855, 0, 0, 0, 0, 0, 0, -3541.282513508799], '0.001100_70.559241_2.000000_687.500000_0.000000_0.100000': [0, 436.27976001292404, 0, -1395.4646853127635, 0, 0, 0, 0], '0.001100_64.144765_2.000000_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -6325.246485673107], '0.001100_64.144765_2.000000_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -240.64775438650142, 0], '0.001100_70.559241_2.200000_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 3231.8631794333987, 0, 846.7764337345224], '0.001100_70.559241_2.200000_756.250000_0.000000_0.100000': [0, 0, 0, 0, 0, -256.49164561052714, 0, 0], '0.001100_70.559241_2.000000_756.250000_0.000000_0.100000': [0, 0, 0, 0, 0, 6659.589224098722, 0, 0], '0.001210_64.144765_1.818182_756.250000_0.000000_0.100000': [0, 0.03446085227260483, 0, 0, -0.6500625981554572, 0, 0, 0], '0.001000_53.012202_1.818182_687.500000_0.000000_0.100000': [0, 0, 0, 0.9231257745895277, 0, 0, 0.10812395767397902, 0], '0.001000_53.012202_2.000000_687.500000_0.000000_0.100000': [-0.7897300711774505, 0, 0, 0, 0, -0.8392052496268434, 0, 0], '0.001210_77.615165_2.200000_687.500000_0.000000_0.100000': [0, 0, -6.84239466329218, 0, 0, 0, 0, 0], '0.001331_77.615165_2.200000_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 1269.1820658874888, 0, 0], '0.001331_77.615165_2.000000_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 1463.8525708665, 0, 0], '0.001331_77.615165_1.818182_687.500000_0.000000_0.100000': [6045.572004281299, 0, 0, -133.0775064424091, 0, 0, 0, 0], '0.001331_70.559241_1.818182_687.500000_0.000000_0.100000': [13.634093112802885, 0, 0, 0, 0, -13.538431894392557, 0, 0], '0.001331_70.559241_1.652893_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001331_125.000000_1.652893_352.796206_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -378.39206362616113], '0.001331_125.000000_1.652893_388.075827_0.000000_0.100000': [0, 0, -3.1836192655100604, 0, 0, 0, 0, -396.25092107702056], '0.001464_125.000000_1.652893_388.075827_0.000000_0.100000': [0, 0, 0, -269.5583286630129, 0, 0, 0, 0], '0.001464_125.000000_1.818182_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000826_93.914350_2.200000_756.250000_0.000000_0.100000': [0, 0, 0, 0, 4.151885806260362, 4646.307271227804, 0, 0], '0.000826_93.914350_2.000000_756.250000_0.000000_0.100000': [0, 0, 0, 0, 236.1536371583736, 0, 0, 0], '0.000751_93.914350_2.000000_756.250000_0.000000_0.100000': [287.1470918197135, 0, 0, -4219.964659522769, 0, 0, 0, 0], '0.000751_93.914350_2.200000_756.250000_0.000000_0.100000': [0, 0, -4.151885806260362, 0, 0, 0, -282.34818963348806, -610.7225533861329], '0.000751_93.914350_2.200000_831.875000_0.000000_0.100000': [0, 610.7225533861329, 0, 0, 0, 0, 0, 0], '0.000683_103.305785_2.200000_756.250000_0.000000_0.100000': [0, 0, 0, -602.4134568078042, 0, 0, 0, 0], '0.000683_103.305785_2.420000_756.250000_0.000000_0.100000': [0, 0, -2.355504877697058, 0, 0, 0, 0, 0], '0.000751_103.305785_2.420000_756.250000_0.000000_0.100000': [0, 548.6057710662301, 0, 0, 0, 0, 0, 0], '0.000751_93.914350_2.420000_687.500000_0.000000_0.100000': [217.66161238108617, 0, 0, 0, 0, 0, 0, 0], '0.000751_85.376682_2.420000_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 788.5742093916082, 0, 0], '0.000751_85.376682_2.200000_687.500000_0.000000_0.100000': [0, 0, 777.7774080484011, 0, 0, 1263.6851822387205, 0, 0], '0.000826_166.375000_2.000000_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 950.1035845670666, -477.2635657581086, 0], '0.000826_183.012500_2.000000_516.528926_0.000000_0.100000': [477.2635657581086, 0, 0, 0, 0, 0, 0, 0], '0.000826_166.375000_1.818182_516.528926_0.000000_0.100000': [0, 0, -3.6449428156944124, 0, 0, 0, -465.63926167874615, 0], '0.000826_183.012500_1.818182_516.528926_0.000000_0.100000': [465.63926167874615, 0, 0, 0, 0, 0, 0, 0], '0.001000_113.636364_1.502630_426.883410_0.000000_0.100000': [0, 1804.2837125956198, 1469.8320025822147, 0, 0, 796.5005142032772, 0, 0], '0.001000_113.636364_1.502630_388.075827_0.000000_0.100000': [3391.97904283649, 0, 0, -67.99174019556017, 0, 0, 0, 340.2121075540925], '0.001100_103.305785_1.366027_352.796206_0.000000_0.100000': [0, 2076.7706287885135, 0, 0, 0, -18.920692289105137, -335.98109970452475, -289.97451703606026], '0.001100_93.914350_1.241843_388.075827_0.000000_0.100000': [0, 0, 0, 109.18232010281075, 0, 0, 0, 0], '0.001100_93.914350_1.366027_388.075827_0.000000_0.100000': [0, 0, 0, -1114.0263640871062, 0, -91.15039427820862, -328.75062867926016, 0], '0.001100_93.914350_1.652893_469.571751_0.000000_0.100000': [0, 0, 0, 0, 1.5477287649213396, 0, 0, 0], '0.000909_93.914350_1.652893_388.075827_0.000000_0.100000': [0, 372.3163809410445, 0, 0, 0, 0, 0, 0], '0.000909_85.376682_1.652893_352.796206_0.000000_0.100000': [0, 0, 0, 341.8621702613243, 0.9910510214524948, 0, 0, 0], '0.000826_85.376682_1.652893_352.796206_0.000000_0.100000': [259.63637777202547, 0, 0, 0, 0, 0, 0, 0], '0.000826_77.615165_1.652893_352.796206_0.000000_0.100000': [0, 0, 0, 0, 4.5579098919555285, 0, 0, 0], '0.000751_77.615165_1.652893_352.796206_0.000000_0.100000': [0, 0, 0, 24.594899295176894, 0, 1218.544272471603, 0, 0], '0.000751_77.615165_1.818182_352.796206_0.000000_0.100000': [0, 686.9092508906576, 0, 0, 1.6150672087593194, 1074.3308458651927, 0, 0], '0.000683_77.615165_1.818182_352.796206_0.000000_0.100000': [0, 0, 0, 385.8899077850833, 0, 0, 0, 0], '0.000683_77.615165_2.000000_352.796206_0.000000_0.100000': [0, 310.6978079610339, 0, 0, 0, 0, 0, 0], '0.000683_77.615165_2.000000_320.723824_0.000000_0.100000': [1034.525882507188, 0, 0, 0, 0, 0, 0, 0], '0.000683_70.559241_2.000000_320.723824_0.000000_0.100000': [0, 0, 0, 0, 0, 1117.4577880865847, 0, 0], '0.000683_70.559241_1.818182_320.723824_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001000_166.375000_1.652893_568.181818_0.000000_0.100000': [0, 0, 0, -360.99656868233194, 45.176271348872206, 0, 0, 0], '0.001000_166.375000_1.652893_516.528926_0.000000_0.100000': [0, 0, 0, -369.8587179115697, 0, 0, 0, 0], '0.001000_166.375000_2.000000_516.528926_0.000000_0.100000': [1905.1642434655819, 0, -6.165863093369262, 0, 0, 0, 0, -455.2128603407131], '0.001100_151.250000_2.000000_516.528926_0.000000_0.100000': [-587.9289504384185, 0, 0, 0, 0, 0, 0, 0], '0.001100_113.636364_1.502630_320.723824_0.000000_0.100000': [0, 0, -4.891611254723375, 0, 0, 0, 0, 0], '0.001210_113.636364_1.502630_320.723824_0.000000_0.100000': [0, 0, -5.375637867774913, 0, 0, 0, 0, 0], '0.001331_113.636364_1.502630_320.723824_0.000000_0.100000': [419.1495352625264, 0, 0, 0, 0, 0, 0, 0], '0.001331_103.305785_1.502630_320.723824_0.000000_0.100000': [0, 0, -6.253334815504149, -427.2900011436704, 0, 0, 0, 0], '0.001331_93.914350_1.652893_320.723824_0.000000_0.100000': [230.201305336451, 0, 0, -19.434603132815756, 0, 0, 0, 0], '0.001331_93.914350_1.818182_320.723824_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001000_113.636364_1.128948_568.181818_0.000000_0.100000': [0, 405.33338541184867, 0, 0, 0, 0, 0, 0], '0.001000_103.305785_1.128948_516.528926_0.000000_0.100000': [0, 0, -1.3932969156971013, 1169.3624231124572, 0, 0, 0, 0], '0.001100_103.305785_1.128948_516.528926_0.000000_0.100000': [0, 0, 0, 0, 437.21106778238436, 0, 0, 0], '0.001100_103.305785_1.128948_352.796206_0.000000_0.100000': [0, 0, 0.31083525994281447, 0, 0, 0, 0, -316.47957696645335], '0.001210_103.305785_1.128948_352.796206_0.000000_0.100000': [0, 291.0042674661537, 0, 0, 0, 0, 0, 0], '0.001210_103.305785_1.128948_320.723824_0.000000_0.100000': [0, 0, 0.8240927170822943, 0, 0, 0, 0, 0], '0.001331_103.305785_1.128948_320.723824_0.000000_0.100000': [135.38785070824534, 0, 0, 0, 0, 0, 0, 0], '0.001331_93.914350_1.128948_320.723824_0.000000_0.100000': [0, 253.4489055408156, 0, 0, 0, 0, 0, 0], '0.001331_93.914350_1.128948_291.567113_0.000000_0.100000': [0, 0, 0, 0, -0.635400685253444, 0, 0, 0], '0.001210_93.914350_1.128948_291.567113_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -254.6609652184573], '0.001210_93.914350_1.128948_320.723824_0.000000_0.100000': [0, 0, 0, 213.4438030511823, 0, 0, 0, 0], '0.001210_93.914350_1.241843_320.723824_0.000000_0.100000': [201.416457428651, 0, 0, 0, 0, 0, 0, 0], '0.001210_77.615165_1.241843_320.723824_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -83.24039810780714, -275.65315324278754], '0.001210_103.305785_1.366027_516.528926_0.000000_0.100000': [0, 584.0524295758703, -6.488177853563957, 0, 0, 0, 0, 0], '0.001210_103.305785_1.366027_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 454.4113274430957, 11.401396797667577, 0], '0.001210_103.305785_1.241843_469.571751_0.000000_0.100000': [0, 413.83071651249287, 0, -10.434464809677442, 0, 0, 0, 0], '0.001210_103.305785_1.241843_426.883410_0.000000_0.100000': [226.56745339228428, 0, -2.5480890061532593, 0, 0, 0, 0, 0], '0.001331_103.305785_1.241843_426.883410_0.000000_0.100000': [227.4396435204507, 0, 0, 0, 0, 0, 0, 0], '0.001331_93.914350_1.241843_426.883410_0.000000_0.100000': [157.03047964720528, 0, 0, 0, 0, 0, 0, 0], '0.001331_85.376682_1.241843_426.883410_0.000000_0.100000': [0, 0, 0, 815.4198150705132, 0, 0, 0, 0], '0.001331_85.376682_1.366027_426.883410_0.000000_0.100000': [199.96325136061094, 0, 0, 0, 0, -59.25577440934, -688.6070663348731, 0], '0.001331_93.914350_1.366027_426.883410_0.000000_0.100000': [0, 0, 0, 0, 4.502507111821342, 0, 0, 0], '0.001210_93.914350_1.366027_426.883410_0.000000_0.100000': [674.1146400040427, 0, 0, 0, 0, 0, 0, 0], '0.001210_85.376682_1.366027_426.883410_0.000000_0.100000': [0, 0, 9.989919219009062, 0, 0, 0, 0, 0], '0.001331_77.615165_1.366027_426.883410_0.000000_0.100000': [927.2168604601816, 0, 0, 0, 0, -682.5994920554662, 0, 0], '0.001331_77.615165_1.241843_426.883410_0.000000_0.100000': [0, 288.4950688441463, -0.650809829309128, 0, 0, 0, 0, 0], '0.001464_77.615165_1.241843_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0.650809829309128, 0, 0, 0], '0.001331_77.615165_1.241843_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001000_125.000000_1.652893_831.875000_0.000000_0.100000': [315.01961136572413, 0, 0, 0, 0, 0, 0, 0], '0.001000_103.305785_1.502630_831.875000_0.000000_0.100000': [0, 0, 0, 0, 1.7892247097992369, 0, 0, 0], '0.000909_103.305785_1.502630_831.875000_0.000000_0.100000': [0, 703.1200370162329, 0, 0, 0, 0, 0, 0], '0.000751_93.914350_1.366027_756.250000_0.000000_0.100000': [0, 0, 0, 0, 0, 325.91551670489133, 0, 0], '0.000751_93.914350_1.241843_756.250000_0.000000_0.100000': [0, 0, -0.8405203161001736, 0, 0, 0, 0, 0], '0.000826_93.914350_1.241843_756.250000_0.000000_0.100000': [828.8183619067285, 0, 0, 0, 0, 0, 0, -438.9152645985655], '0.000826_93.914350_1.241843_831.875000_0.000000_0.100000': [0, 0, 0, 0, 1.4001343516049019, 0, 0, 0], '0.000751_93.914350_1.241843_831.875000_0.000000_0.100000': [0, 0, 0, -426.9251992093401, 0, 0, 0, 0], '0.000751_93.914350_1.366027_831.875000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -559.2714635650963], '0.000751_93.914350_1.366027_915.062500_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000909_125.000000_1.241843_625.000000_0.000000_0.100000': [0, 0, 0, -633.2025319741825, 0, 0, 0, 0], '0.000909_151.250000_1.241843_687.500000_0.000000_0.100000': [0, 0, 0, 0, 3.3535981078503028, 0, 0, 0], '0.000826_151.250000_1.241843_687.500000_0.000000_0.100000': [395.0923825563254, 0, 0, 0, 0, 0, 0, 0], '0.000826_137.500000_1.241843_687.500000_0.000000_0.100000': [0, 0, 0, 38.362218715472785, 0, 0, 0, 0], '0.000826_151.250000_1.366027_687.500000_0.000000_0.100000': [0, 435.54744584327364, 0, 0, 0, 0, 0, 0], '0.000826_151.250000_1.366027_625.000000_0.000000_0.100000': [462.37627479657294, 0, 0, 0, 0, 0, 0, 0], '0.001210_125.000000_1.652893_388.075827_0.000000_0.100000': [959.9176487898136, 0, 0, -267.8810235447875, 0, 22.22608200930472, 0, -396.1678288827238], '0.001100_113.636364_1.502630_426.883410_0.000000_0.100000': [0, 4016.2953862053905, 0, 0, 0, 0, 0, 0], '0.001210_113.636364_1.502630_352.796206_0.000000_0.100000': [0, 0, -5.859766219858011, 0, 0, 0, 0, 0], '0.001331_103.305785_1.502630_352.796206_0.000000_0.100000': [0, 425.0521882136906, -6.081451677037603, 0, 0, 0, 0, 0], '0.001464_103.305785_1.502630_352.796206_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -462.12746962518816], '0.001464_103.305785_1.502630_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001331_103.305785_1.366027_516.528926_0.000000_0.100000': [376.5752663335141, 0, 0, 0, 0, 0, 0, 0], '0.001331_93.914350_1.366027_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 9.946366159989703, 0, 0], '0.001331_93.914350_1.241843_516.528926_0.000000_0.100000': [0, 401.5517559754736, 0, 0, 0, 0, 0, 0], '0.001331_93.914350_1.241843_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -242.2001400019769, 0], '0.001331_103.305785_1.241843_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, -151.89429374993088, 0, 0], '0.001331_103.305785_1.128948_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, -191.49053365212367, 0, 0], '0.001331_103.305785_1.026316_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -296.7435207538642, 0], '0.001331_113.636364_1.026316_469.571751_0.000000_0.100000': [0, 460.3470656845626, 0, 0, 0, 0, 0, 0], '0.001331_113.636364_1.026316_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -338.49991587631087, 0], '0.001331_125.000000_1.026316_426.883410_0.000000_0.100000': [0, 0, -3.5316467501246733, 0, 0, 0, 0, 0], '0.001464_125.000000_1.026316_426.883410_0.000000_0.100000': [0, 422.79606347776627, 0, 0, 0, 0, 0, 0], '0.001464_125.000000_1.026316_388.075827_0.000000_0.100000': [0, 0, 0, 230.3812366710922, 0, 0, 0, 0], '0.001464_125.000000_1.128948_388.075827_0.000000_0.100000': [267.2771813518995, 0, 0, 0, 0, -20.943748788281106, 0, 0], '0.001464_113.636364_1.128948_388.075827_0.000000_0.100000': [183.6831282238216, 0, 0, 0, 0, 0, 0, 0], '0.001464_103.305785_1.128948_388.075827_0.000000_0.100000': [0, 0, 0, 156.77856461867768, 0, 0, 0, 0], '0.001464_103.305785_1.241843_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001331_93.914350_2.000000_388.075827_0.000000_0.100000': [0, 0, 0, 0, 6.052354926378757, 0, 0, 0], '0.001210_85.376682_2.000000_388.075827_0.000000_0.100000': [206.67232716395893, 0, 0, -1797.8917257250484, 0, 0, 0, 0], '0.001210_85.376682_2.200000_388.075827_0.000000_0.100000': [0, 0, 1083.0001431852354, -1187.1747325608155, 0, 0, 0, 0], '0.001331_85.376682_2.200000_388.075827_0.000000_0.100000': [-605.2132786216528, -496.275420400795, 0, -2166.7907525110286, -975.6758046713833, 0, 0, 0], '0.001331_85.376682_2.420000_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 2166.7907525110286, 0, 0], '0.001210_85.376682_2.420000_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 2065.28295676506, 0, 0], '0.001331_77.615165_2.200000_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 1628.8866917202965, 0, 0], '0.001331_77.615165_2.000000_388.075827_0.000000_0.100000': [0, 0, -4.044993847825572, 0, 0, 0, 0, 0], '0.001464_77.615165_2.000000_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001331_77.615165_1.818182_756.250000_0.000000_0.100000': [0, 3253.8095018721183, 0, 0, 0, 0, 0, 0], '0.001331_64.144765_1.818182_687.500000_0.000000_0.100000': [0, 0, 0, -63.29864678115137, 0, 0, 0, 0], '0.001331_58.313423_2.000000_687.500000_0.000000_0.100000': [0, 0, 0, -0.7602353231511643, 0, 0, 0, 0], '0.001331_58.313423_2.200000_687.500000_0.000000_0.100000': [0, 0, 0, 0.02077815594752792, 0, 0, 0, 0], '0.001331_58.313423_2.420000_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0.7610383968021097, 0, 0, 0], '0.001100_58.313423_2.420000_687.500000_0.000000_0.100000': [0, 0, 0, -0.011538358675397832, 0, 0, 0, 0], '0.001100_58.313423_2.662000_687.500000_0.000000_0.100000': [-0.6820140765213865, 0, 0, 0, 0, 0, 0, 0], '0.001100_53.012202_2.662000_687.500000_0.000000_0.100000': [0, 0, 0, 0, -0.13695008658905863, 0, 0, 0], '0.001000_53.012202_2.662000_687.500000_0.000000_0.100000': [0, 0.8517763172733794, 0, 0, 0, 0, 0, 0], '0.001000_53.012202_2.662000_625.000000_0.000000_0.100000': [-0.8686159742934934, -0.0027643156495742007, 0, -0.04538909693137612, 0, 0, 0, 0], '0.001000_53.012202_2.928200_625.000000_0.000000_0.100000': [0.010961238152276565, 0, 0, 0, 0, 0, 0, 0], '0.001000_48.192911_2.928200_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000826_85.376682_1.818182_352.796206_0.000000_0.100000': [210.37434050172814, 0, 0, 0, 0, 0, 0, -375.80227779898024], '0.000909_85.376682_1.818182_388.075827_0.000000_0.100000': [0, 376.50743673873797, 0, 0, 487.73271038736493, 0, 0, 0], '0.000909_85.376682_1.818182_352.796206_0.000000_0.100000': [0, 764.463276785506, 0, 0, 0, 0, 0, 0], '0.000909_85.376682_1.818182_320.723824_0.000000_0.100000': [0, 521.0824747824171, 0, 0, 0, 0, 0, 0], '0.000909_85.376682_1.818182_291.567113_0.000000_0.100000': [0, 0, 0, 0, 0, 73.70211115445701, 0, 0], '0.000909_85.376682_1.652893_291.567113_0.000000_0.100000': [1271.0144964341173, 361.984577632105, 0, 0, 2.499195996479466, 0, -271.4420048418183, 0], '0.000909_77.615165_1.652893_291.567113_0.000000_0.100000': [0, 0, 0, 0, 0, 1895.8179005843135, 0, 0], '0.000909_77.615165_1.502630_291.567113_0.000000_0.100000': [1363.4708775769232, 0, 0, 0, 0, 0, -547.4873769862702, 0], '0.000909_85.376682_1.502630_291.567113_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000826_77.615165_1.502630_426.883410_0.000000_0.100000': [0, 0, 1003.6663978400011, 0, 0, 0, -1011.1819624080458, 0], '0.000909_77.615165_1.502630_426.883410_0.000000_0.100000': [2339.5957931545113, 0, 0, 0, 0, 0, 0, -531.0780487833857], '0.000909_70.559241_1.502630_426.883410_0.000000_0.100000': [0, 0, 1477.9126632957368, 1073.839966289887, 0, 0, 0, 0], '0.001000_151.250000_1.818182_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 768.8139365994548, 0, 0], '0.001100_151.250000_1.818182_388.075827_0.000000_0.100000': [0, 0, -3.966683946496596, 0, 0, 0, 0, 0], '0.001210_151.250000_1.818182_388.075827_0.000000_0.100000': [0, 0, 0, -727.3579908896554, 0, 0, 0, 0], '0.001210_151.250000_2.000000_388.075827_0.000000_0.100000': [0, 0, 0, -878.8183209948842, 0, 0, 0, 0], '0.001210_151.250000_2.200000_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -503.9048309116477], '0.001210_151.250000_2.200000_426.883410_0.000000_0.100000': [401.8177229863637, 0, 0, 0, 0, 0, 0, 0], '0.001210_137.500000_2.200000_426.883410_0.000000_0.100000': [0, 0, 0, 0, 6.257515421613789, 0, 0, 0], '0.001210_103.305785_1.366027_625.000000_0.000000_0.100000': [367.53931539711857, 0, 0, 0, 0, 0, -40.80519424527764, 0], '0.001210_93.914350_1.366027_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 138.7042403047074, 0, 0], '0.001210_93.914350_1.241843_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -187.876800235256, 0], '0.001210_103.305785_1.241843_625.000000_0.000000_0.100000': [0, 397.701195921506, 0, 0, 1.2413705179873433, 0, 0, 0], '0.001210_103.305785_1.241843_568.181818_0.000000_0.100000': [207.31992218991155, 0, 0, 0, 0, -216.4054976347821, 0, 0], '0.001210_103.305785_1.128948_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -179.47998384777074, 0], '0.001210_113.636364_1.128948_568.181818_0.000000_0.100000': [0, 406.9403810383113, 0, 0, 0, 0, 0, 0], '0.001210_113.636364_1.128948_516.528926_0.000000_0.100000': [0, 395.63117584053265, 0, 0, 0, 0, 0, 0], '0.001210_113.636364_1.128948_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -239.48748972470713, 0], '0.001210_125.000000_1.128948_469.571751_0.000000_0.100000': [0, 0, -1.321557599474272, 0, 0, 0, 0, 0], '0.001331_125.000000_1.128948_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000826_113.636364_1.366027_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -457.7372195270345, 0], '0.000826_125.000000_1.366027_516.528926_0.000000_0.100000': [0, 0, -4.490040415950898, 0, 0, 0, 0, 0], '0.001000_151.250000_1.502630_568.181818_0.000000_0.100000': [0, 0, 0, 1107.5596819372636, 0, 0, 0, 0], '0.001000_151.250000_1.366027_568.181818_0.000000_0.100000': [0, 472.9543397707066, 0, 0, 0, 0, 0, 0], '0.001000_151.250000_1.366027_516.528926_0.000000_0.100000': [447.38033996908126, 0, 0, 0, 0, 0, 0, 0], '0.001000_137.500000_1.366027_516.528926_0.000000_0.100000': [0, 0, 0, -701.1295561406718, 0, 0, 0, 0], '0.000909_113.636364_1.128948_516.528926_0.000000_0.100000': [194.26640861349287, 0, 0, 0, 0, 0, -228.01970601126132, 0], '0.000909_103.305785_1.128948_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -194.26640861349287, 0], '0.000909_125.000000_1.128948_516.528926_0.000000_0.100000': [0, 385.36474827900673, 0, 0, 0, 0, 0, 0], '0.000909_125.000000_1.128948_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000826_113.636364_2.000000_756.250000_0.000000_0.100000': [0, 0, 0, 0, 586.4098945245184, 0, 0, 0], '0.001000_137.500000_1.818182_831.875000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 123.02075752621658, -532.8262903904961], '0.001000_137.500000_1.818182_915.062500_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 11151.087097294325, 0], '0.001000_151.250000_1.818182_915.062500_0.000000_0.100000': [0, -10946.185499132176, 0, 0, 0, 0, 0, 0], '0.001000_151.250000_1.818182_831.875000_0.000000_0.100000': [0, 1180.8623688294465, 0, 0, 0, 0, 0, 0], '0.001100_137.500000_1.818182_831.875000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -532.9826008801883], '0.001100_137.500000_1.818182_915.062500_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 11156.118704980176, 0], '0.001100_151.250000_1.818182_915.062500_0.000000_0.100000': [0, -10951.248963708973, 0, 0, 0, 0, 0, 0], '0.001100_151.250000_1.818182_831.875000_0.000000_0.100000': [0, 0, 0, 0, 0, 310.8460497886943, 0, 0], '0.001100_151.250000_1.652893_831.875000_0.000000_0.100000': [0, 0, -4.093157668755521, 0, 0, 0, 0, 0], '0.001210_151.250000_1.652893_831.875000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -464.6360331412785], '0.001210_151.250000_1.652893_915.062500_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000751_125.000000_1.818182_687.500000_0.000000_0.100000': [0, 0, 0, 0, 3.0410786041020947, 0, 0, 0], '0.000683_125.000000_1.818182_687.500000_0.000000_0.100000': [0, 0, 0, 0, 2.7579781389700475, 0, 0, -513.9157168393951], '0.000621_125.000000_1.818182_687.500000_0.000000_0.100000': [335.6512756122108, 0, 0, 0, 0, 0, 0, -513.7096491091024], '0.000621_113.636364_1.818182_687.500000_0.000000_0.100000': [842.6357654231799, 0, 0, 0, 0, 0, -30.51375232838279, 0], '0.000621_103.305785_2.000000_568.181818_0.000000_0.100000': [0, 0, 0, 0, 3.4346449267566643, 0, 0, 0], '0.000564_103.305785_2.000000_568.181818_0.000000_0.100000': [360.048038043972, 0, 0, 0, 0, 0, -386.27450746354134, 0], '0.000564_113.636364_2.000000_568.181818_0.000000_0.100000': [0, 0, 0, 0, 3.1690577649564147, 0, 0, 0], '0.000513_113.636364_2.000000_568.181818_0.000000_0.100000': [0, 0, 0, -1264.2344512035786, 0, 0, 0, 0], '0.000513_113.636364_2.200000_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001100_151.250000_1.652893_687.500000_0.000000_0.100000': [0, 0, 0, -311.7561410556247, 0, 0, -345.95792801162224, 0], '0.001210_151.250000_1.818182_687.500000_0.000000_0.100000': [796.8690854703594, 0, 0, 0, 0, 0, 0, 0], '0.001100_93.914350_1.241843_687.500000_0.000000_0.100000': [887.8074684817201, 0, 0, -548.9232299217745, 0.7660219667368438, 0, -177.63124012784775, 0], '0.001100_103.305785_1.241843_756.250000_0.000000_0.100000': [191.68328845157603, 0, 0, 0, 0, 0, -246.87550186883692, 0], '0.001100_93.914350_1.241843_756.250000_0.000000_0.100000': [0, 0, 0, 0, 0, -383.4459701943827, 0, 0], '0.001100_93.914350_1.128948_756.250000_0.000000_0.100000': [0, 0, 0, 0, 0.726789858974007, 0, 0, 0], '0.001000_93.914350_1.128948_756.250000_0.000000_0.100000': [0, 0, 0, 0, 0, -637.1011172386303, 0, -464.9236968121797], '0.001000_93.914350_1.026316_756.250000_0.000000_0.100000': [0, 0, 0, 637.1011172386303, 0, 0, 0, 0], '0.001000_93.914350_1.128948_831.875000_0.000000_0.100000': [0, 0, 0, 0, 0, -689.6385141705923, 0, 0], '0.001000_93.914350_1.026316_831.875000_0.000000_0.100000': [115.41060447285072, 0, 0, 0, 0, 0, 0, 0], '0.001000_85.376682_1.026316_831.875000_0.000000_0.100000': [0, 0, 0, 0, 1.4473043297617552, 0, 0, 0], '0.000909_85.376682_1.026316_831.875000_0.000000_0.100000': [0, 0, -0.14473043297617544, 0, 0, 0, 0, 0], '0.000751_103.305785_1.652893_426.883410_0.000000_0.100000': [255.3659668938347, 0, -1.045021521882427, 0, 0, 0, 0, -433.683385755776], '0.000909_93.914350_1.241843_388.075827_0.000000_0.100000': [148.1675282287658, 0, -1.574632957031463, 0, 0, 0, 0, 0], '0.001000_85.376682_1.241843_388.075827_0.000000_0.100000': [55.78584527477724, 0, -0.5913933299069868, 0, 0, 0, 0, 0], '0.001000_77.615165_1.241843_388.075827_0.000000_0.100000': [0, 0, -0.5191602352698282, 0, 0, 0, 0, -288.69266877326345], '0.001100_77.615165_1.241843_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000751_103.305785_1.366027_516.528926_0.000000_0.100000': [0, 0, 0, -995.7858096726558, 0, 0, 0, 0], '0.001000_103.305785_1.241843_388.075827_0.000000_0.100000': [392.8841176101263, 0, 0, 0, 0, 0, 0, 0], '0.001000_93.914350_1.502630_352.796206_0.000000_0.100000': [2345.7328766592823, 423.95485760128275, 0, 0, 4.2874130668275034, 0, 0, 0], '0.001100_85.376682_1.502630_352.796206_0.000000_0.100000': [1044.8793978156489, 0, 0, 0, 0, 0, 0, 0], '0.001100_103.305785_1.502630_516.528926_0.000000_0.100000': [1808.4208422820232, 0, 0, -592.3117655421614, 0, 0, 0, 0], '0.000826_70.559241_1.818182_469.571751_0.000000_0.100000': [167.5416935146514, 0, 0, 0, -1.1731141158446918, 0, 0, 0], '0.000751_70.559241_1.818182_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -2292.348773746015], '0.000751_70.559241_1.818182_516.528926_0.000000_0.100000': [0, 0, 0, 0, 1.3237875637114485, 0, 0, 0], '0.000826_93.914350_2.200000_516.528926_0.000000_0.100000': [369.8354703074125, 0, 0, 0, 0, 0, 0, 0], '0.000826_70.559241_2.200000_469.571751_0.000000_0.100000': [0, 0, 0, 0, 4.808053839640979, 0, 0, 0], '0.000751_70.559241_2.200000_469.571751_0.000000_0.100000': [0, 0, 0, -1194.0920767824502, 0, 0, 0, 0], '0.000751_70.559241_2.420000_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001210_125.000000_1.502630_426.883410_0.000000_0.100000': [0, 833.1756482470763, 0, 0, 0, 0, 0, 0], '0.001331_113.636364_1.502630_388.075827_0.000000_0.100000': [410.5735789842448, 0, 0, 0, 0, 0, 0, 0], '0.001331_103.305785_1.502630_388.075827_0.000000_0.100000': [0, 461.42164340012005, 0, 0, 0, 0, 0, 0], '0.001464_103.305785_1.502630_320.723824_0.000000_0.100000': [0, 0, -6.916563542897257, 0, 0, 0, 0, 0], '0.001611_103.305785_1.502630_320.723824_0.000000_0.100000': [0, 0, -7.654275456076266, 0, 0, 0, 0, 0], '0.001772_103.305785_1.502630_320.723824_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -424.48459998771705], '0.001772_103.305785_1.502630_352.796206_0.000000_0.100000': [416.7564063688345, 0, 0, 0, 0, 0, 0, 0], '0.001772_93.914350_1.502630_352.796206_0.000000_0.100000': [826.4605064471588, 0, 0, 0, 0, 0, -37.886946033530386, 0], '0.001772_85.376682_1.502630_352.796206_0.000000_0.100000': [334.9541697684503, 0, 0, 0, 0, 0, 0, 0], '0.001772_77.615165_1.502630_352.796206_0.000000_0.100000': [0, 0, 0, 0, 4.436738102924664, 0, 0, 0], '0.001611_77.615165_1.502630_352.796206_0.000000_0.100000': [0, 322.36087008386977, 0, 0, 0, 0, 0, 0], '0.001611_77.615165_1.502630_320.723824_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000826_93.914350_1.818182_469.571751_0.000000_0.100000': [914.2068407875558, 1717.8456557580344, 0, 0, 250.55197066405134, 0, -300.5185466295064, 0], '0.000621_77.615165_1.652893_516.528926_0.000000_0.100000': [0, 0, 0, -75.65012373344761, 0.5568236534542302, 0, 0, 0], '0.000621_77.615165_1.818182_516.528926_0.000000_0.100000': [0, 0, 0, 0, 1.75737139098419, 0, 0, 0], '0.000564_77.615165_1.818182_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000909_113.636364_1.818182_388.075827_0.000000_0.100000': [0, 0, 460.7257523480762, 0, 0, 214.8613402567225, 0, 0], '0.000826_113.636364_1.818182_388.075827_0.000000_0.100000': [0, 0, 0, -447.1457720836374, 0, 0, 0, 0], '0.000826_113.636364_2.000000_388.075827_0.000000_0.100000': [0, 0, -4.681840764262415, 0, 0, 0, 0, 0], '0.000909_113.636364_2.000000_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -475.4141051541051, 0], '0.000909_125.000000_2.000000_352.796206_0.000000_0.100000': [0, 0, 0, 0, 483.1665571878335, 0, 0, 0], '0.000826_113.636364_2.000000_320.723824_0.000000_0.100000': [0, 361.9712843174556, 0, 0, 0, 0, 0, 0], '0.000826_113.636364_2.000000_291.567113_0.000000_0.100000': [0, 0, 0, 0, 3.632889978075855, 0, 0, 0], '0.000751_113.636364_2.000000_291.567113_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001100_85.376682_1.366027_625.000000_0.000000_0.100000': [0, 521.6582240499997, 0, 0, 0, 0, 0, -306.2378962976126], '0.001100_85.376682_1.366027_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 284.1773275676852, 0, -898.7132612325535], '0.001100_77.615165_1.241843_625.000000_0.000000_0.100000': [1313.6346440972582, 0, 0, 1013.9766927552525, 0, 0, 0, 0], '0.001100_70.559241_1.241843_625.000000_0.000000_0.100000': [0, 0, 0, 0, 4.041880316701463, 0, 0, 0], '0.001000_70.559241_1.241843_625.000000_0.000000_0.100000': [0, 0, 0, 0, 2545.481949419327, 0, 0, 0], '0.000909_70.559241_1.241843_625.000000_0.000000_0.100000': [0, 0, 0, 0, 25.263413171002583, 0, 0, 0], '0.000826_70.559241_1.241843_625.000000_0.000000_0.100000': [29.793357033323893, 0, 0, 0, 0, -36.33460633381674, 0, 0], '0.000826_70.559241_1.128948_625.000000_0.000000_0.100000': [0, 0, 0, 36.33460633381674, 0, 0, 0, 0], '0.000826_64.144765_1.241843_625.000000_0.000000_0.100000': [0, 0, 0.022806929653256702, 0, 0, 0, 0, 0], '0.000909_64.144765_1.241843_625.000000_0.000000_0.100000': [0, 0.6983912320430045, 0, 0, 0, 0, 0, 0], '0.000909_58.313423_1.366027_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001464_113.636364_1.652893_625.000000_0.000000_0.100000': [0, 0, 0, -196.67190413211083, 0, 0, 0, 0], '0.001464_113.636364_1.818182_625.000000_0.000000_0.100000': [0, 0, 0, 0, 180.85965540640527, 0, 0, 0], '0.001210_93.914350_1.502630_625.000000_0.000000_0.100000': [337.0987463093417, 0, 0, 0, 0, 0, 0, 0], '0.001100_77.615165_1.502630_625.000000_0.000000_0.100000': [0, 4733.64403804827, 0, 0, 0, 0, 0, 0], '0.001000_64.144765_1.502630_516.528926_0.000000_0.100000': [-0.6558298300727756, -0.04885546352752038, 0.034653318084315286, -0.7280076526203857, 0, 0, 0, -0.6246840034512768], '0.001100_58.313423_1.502630_568.181818_0.000000_0.100000': [-0.019186139242719946, -0.7818464071567632, -0.030046043372044553, -0.7666222748580594, -0.0037646352962610766, 0, 0, 0], '0.001100_53.012202_1.502630_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, -0.6700843164787103, 0, 0], '0.001100_53.012202_1.366027_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0.6240448793412497], '0.001100_53.012202_1.366027_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001000_85.376682_1.366027_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 440.061201429005, 0, 0], '0.001000_85.376682_1.241843_568.181818_0.000000_0.100000': [3333.999659139872, 0, 0, 0, 0, -1206.111172788348, -1150.8101372251115, 0], '0.001000_58.313423_1.241843_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0.010896436039713537, 0, 0], '0.001000_58.313423_1.128948_469.571751_0.000000_0.100000': [0, 0.008987648043109857, 0, 0, 0, 0, 0, 0], '0.001000_58.313423_1.128948_426.883410_0.000000_0.100000': [-0.7553387384799407, -0.7810376046299098, 0, 0, 0, 0, 0, 0], '0.001000_58.313423_1.128948_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0.7810376046299098], '0.000909_125.000000_1.818182_831.875000_0.000000_0.100000': [0, 0, 0, 9074.813453100409, 0, 0, 0, 0], '0.000909_125.000000_2.000000_831.875000_0.000000_0.100000': [0, 0, 0, 11317.524154201847, 0, 100.99620778920487, 0, 0], '0.000909_125.000000_2.200000_831.875000_0.000000_0.100000': [0, 0, 0, 0, 0, -11317.524154201847, 0, 0], '0.001100_93.914350_1.366027_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 135.3573218260338, 0, 0], '0.001000_70.559241_1.366027_756.250000_0.000000_0.100000': [0, 0, 0, 0, 94.41623482779735, 0, 0, 0], '0.000909_70.559241_1.366027_756.250000_0.000000_0.100000': [27.74607905651094, 0, 0, 0, 0, 0, 0, 0], '0.000909_64.144765_1.366027_756.250000_0.000000_0.100000': [0, 0, 0.020764901599022778, 0, 0, 0, 0, 0], '0.001000_64.144765_1.366027_756.250000_0.000000_0.100000': [0, 0, -0.03872036751690757, 0, 0, 0, 0, 0], '0.001100_64.144765_1.366027_756.250000_0.000000_0.100000': [0, 0, 0.033526624368710144, 0, 0, 0, 0, 0], '0.001210_64.144765_1.366027_756.250000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -27.932504442760592, 0], '0.001210_70.559241_1.366027_756.250000_0.000000_0.100000': [0, 0, 0, 0, -94.25271235497347, 0, 0, 0], '0.001100_77.615165_1.366027_756.250000_0.000000_0.100000': [0, 2365.270263690395, 0, 0, 0, 0, 0, 0], '0.001100_77.615165_1.366027_687.500000_0.000000_0.100000': [0, 0, 0, 0, 434.10421166860215, 0, 0, 0], '0.001100_77.615165_1.366027_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -138.24747362958544], '0.000826_93.914350_1.241843_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, -273.4707788485851, 0, 0], '0.000826_93.914350_1.128948_568.181818_0.000000_0.100000': [111.33893749337976, 427.43688227805205, 0, 0, 0, 0, 0, 0], '0.000826_85.376682_1.128948_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -111.33893749337976, 0], '0.000826_93.914350_1.128948_516.528926_0.000000_0.100000': [0, 0, -0.20236541322665288, 0, 0, 0, 0, 0], '0.000909_93.914350_1.128948_516.528926_0.000000_0.100000': [86.54209947241326, 0, 0, 0, 0, 0, 0, 0], '0.001000_85.376682_1.366027_469.571751_0.000000_0.100000': [187.0549141681071, 1312.1135829413133, 0, 1278.3284040072435, 0, 0, 0, 0], '0.001000_77.615165_1.366027_469.571751_0.000000_0.100000': [1365.0042939535908, 0, -2.5775900575686137, -311.5754378260305, 0, 0, 0, 0], '0.001000_70.559241_1.366027_469.571751_0.000000_0.100000': [0, 249.35523741848937, 0, 0, 0, 0, 0, 0], '0.001000_70.559241_1.366027_426.883410_0.000000_0.100000': [0, 0, 0, 0, 2.4862314253719613, 0, 0, 0], '0.000909_70.559241_1.366027_426.883410_0.000000_0.100000': [1310.2214335768751, 0, 0, -32.905374780005424, 0, 0, 0, 0], '0.001000_85.376682_2.000000_756.250000_0.000000_0.100000': [-2694.7430489484454, -2317.799530313724, -0.5047638661190831, 0, 0, 0, 0, 0], '0.001100_77.615165_2.000000_756.250000_0.000000_0.100000': [0, 0, 0, 0, 0, 3667.52328400213, 0, 0], '0.001100_77.615165_1.818182_756.250000_0.000000_0.100000': [3765.09805876833, -527.4059549508318, 0, 0, 0, -2651.545688128603, 0, 0], '0.001100_77.615165_1.652893_756.250000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -223.63982897633704, 0], '0.001100_85.376682_1.652893_756.250000_0.000000_0.100000': [0, 0, 0, 0, 2.631427314510802, 0, 0, 0], '0.001000_85.376682_1.652893_756.250000_0.000000_0.100000': [0, 0, 0, 2821.2282624562386, 0, 0, 0, 0], '0.000909_85.376682_1.818182_756.250000_0.000000_0.100000': [0, 0, 0, 1374.926332584861, 0, 0, 0, 0], '0.000909_85.376682_2.000000_831.875000_0.000000_0.100000': [0, 328.8003657546177, 0, 0, 0, 0, 0, 0], '0.001331_137.500000_2.000000_516.528926_0.000000_0.100000': [0, 0, 0, 0, 7.594509953182751, 0, 0, 0], '0.001331_125.000000_1.652893_469.571751_0.000000_0.100000': [0, 694.6441336884028, 0, 0, 0, 0, 0, 0], '0.001100_85.376682_1.652893_687.500000_0.000000_0.100000': [0, 646.6187873717998, 0, 0, 0, 0, 0, 0], '0.000826_93.914350_1.818182_516.528926_0.000000_0.100000': [41.09982523792314, 1578.9294942838224, 0, -416.7282706561418, 0, 247.86418207584023, 0, -474.8606093032169], '0.001000_85.376682_1.502630_568.181818_0.000000_0.100000': [0, 0, 281.7415447608112, 0, 2454.112587539742, 0, 0, 0], '0.001100_77.615165_1.652893_568.181818_0.000000_0.100000': [0, 0, 0, 3429.8685916436393, 0, 0, 0, 0], '0.001100_64.144765_1.818182_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0.6570396912509295], '0.001000_64.144765_1.818182_625.000000_0.000000_0.100000': [0, -0.07075311599316586, 0, 0, 0, -0.0005576946949304329, 0, 0], '0.001000_64.144765_1.652893_516.528926_0.000000_0.100000': [0, 0, 0.7623698922529911, 0, 0, 0.7280076526203857, 0, 0], '0.001100_64.144765_1.652893_516.528926_0.000000_0.100000': [0.0007358769265238152, 0, 0, 0, 0, 0, 0, 0], '0.001000_70.559241_1.366027_516.528926_0.000000_0.100000': [2021.2754263705951, 0, -6.260410587973957, 0, 0, 0, 0, 0], '0.001100_70.559241_1.366027_516.528926_0.000000_0.100000': [1844.495282457903, 0, 0, 0, 0, 202.4799406279028, 0, 0], '0.001100_70.559241_1.241843_516.528926_0.000000_0.100000': [0, 0, 0, -202.4799406279028, 0, 0, 0, 0], '0.001100_64.144765_1.366027_516.528926_0.000000_0.100000': [0, 0, 0, -0.0030098103724412796, 0, 0, 0, 0], '0.001100_64.144765_1.502630_516.528926_0.000000_0.100000': [-0.07863756383072769, 0, 0, 0, 0, 0, 0, -0.07356847485731802], '0.001100_58.313423_1.502630_516.528926_0.000000_0.100000': [0, 0, 0, 0.8699059853609583, 0, 0, 0, 0], '0.001100_58.313423_1.652893_516.528926_0.000000_0.100000': [-0.8282151455328404, 0.031026950891555317, 0, 0, -0.807617651435362, 0, 0, 0], '0.001000_58.313423_1.652893_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0.8091588906317384, 0, 0, 0], '0.000909_53.012202_1.652893_516.528926_0.000000_0.100000': [0, -0.6268875070722516, 0, 0, 0, 0, 0, 0], '0.000909_53.012202_1.652893_469.571751_0.000000_0.100000': [0, 0.7631020954988074, 0, 0, 0, 0, 0, 0], '0.000909_53.012202_1.652893_426.883410_0.000000_0.100000': [0, 0, 0.018034788379077873, 0, 0, -0.012434130489396011, 0, 0], '0.001000_53.012202_1.652893_426.883410_0.000000_0.100000': [-0.0468294575071468, -0.07625515945867843, -0.02491530654694998, -0.6988274036473058, -0.007067663149973669, 0.012445417868225377, 0, 0], '0.000909_93.914350_1.818182_388.075827_0.000000_0.100000': [270.0447323401648, 0, 0, 0, 0, 0, 0, 0], '0.000909_151.250000_2.662000_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000826_137.500000_1.502630_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -333.55060359503256], '0.001100_64.144765_1.652893_469.571751_0.000000_0.100000': [0, 0, 0, 0.030116148338970562, 0.7072261853919827, 0, 0, 0], '0.001000_64.144765_1.652893_469.571751_0.000000_0.100000': [0, 0.03130726343611059, 0, 0, -0.0012202429622902855, 0, 0, 0], '0.000909_64.144765_1.652893_469.571751_0.000000_0.100000': [0.03708369664271935, 0, 0, 0, 0, -0.7425535975318553, 0, 0], '0.000909_64.144765_1.502630_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -135.4264140717105, 0], '0.001000_70.559241_1.652893_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, 2971.3233872946284, 0, 0], '0.000683_93.914350_2.000000_687.500000_0.000000_0.100000': [303.72519910928133, 0, -4.191630161451212, -715.4938790218956, 0, 0, 0, 0], '0.000751_85.376682_2.000000_756.250000_0.000000_0.100000': [-2697.4249574608943, -2315.494455409863, 0, 0, 0, 0, 0, 0], '0.000751_85.376682_2.200000_625.000000_0.000000_0.100000': [2045.3040888013688, 0, 0, 0, 0, 0, 0, 0], '0.000751_77.615165_2.200000_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 1221.6303383746665, 0, 0], '0.000826_103.305785_2.420000_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 962.6975913186228, 0, 0], '0.000751_77.615165_2.000000_388.075827_0.000000_0.100000': [3019.3764044905206, 0, 0, 0, 0, 0, 0, 0], '0.000751_70.559241_2.000000_388.075827_0.000000_0.100000': [0, 272.09667380922747, 0, -2392.2890300404943, 0.405322250747804, 1925.0542838964614, 0, 0], '0.000751_70.559241_1.818182_388.075827_0.000000_0.100000': [112.81490450141328, 0, 0, 0, 0, 0, 0, -1727.7923407597796], '0.000751_70.559241_1.818182_426.883410_0.000000_0.100000': [0, 0, 0, 0, -1727.6180563241464, 0, 0, 0], '0.000683_77.615165_1.652893_426.883410_0.000000_0.100000': [1121.5261601868826, 0, 0, 0, 0, 0, 0, 0], '0.000683_70.559241_1.652893_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000826_85.376682_2.000000_426.883410_0.000000_0.100000': [0, 0, 28.645563535099964, 0, 0, 0, 0, 0], '0.000909_85.376682_2.000000_426.883410_0.000000_0.100000': [432.5660237804895, 0, 0, 0, -2.6041421395545403, 0, 0, 0], '0.000909_77.615165_2.000000_426.883410_0.000000_0.100000': [0, 0, -3.274444066495198, 0, 0, 0, 0, 0], '0.001100_70.559241_2.000000_388.075827_0.000000_0.100000': [59.0634018522556, 0, 0, 0, 0, 0, 0, 0], '0.000909_93.914350_1.652893_320.723824_0.000000_0.100000': [0, 856.092371933997, 0, 0, 0, 0, 0, 0], '0.000909_93.914350_1.652893_291.567113_0.000000_0.100000': [1389.3856135916117, 0, 0, 0, 0, 0, -262.41724124093525, 0], '0.000909_103.305785_1.652893_291.567113_0.000000_0.100000': [0, 0, 0, 0, 1.2125842364770507, 0, 0, 0], '0.000826_103.305785_1.652893_291.567113_0.000000_0.100000': [0, 0, 0, 0, 1.0826856423585014, 0, -290.79272789726, 0], '0.000826_113.636364_1.652893_291.567113_0.000000_0.100000': [0, 310.42414846060603, 0, 0, 0, 0, 0, 0], '0.000826_113.636364_1.652893_265.061011_0.000000_0.100000': [0, 0, 0, -104.64571396504071, 0, 0, 0, 0], '0.000826_113.636364_1.818182_265.061011_0.000000_0.100000': [0, 0, 0, 0, 1.9012079086313862, 0, 0, 0], '0.000751_113.636364_1.818182_265.061011_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000909_85.376682_1.241843_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, -292.2831271797486, 0, 0], '0.000909_85.376682_1.128948_388.075827_0.000000_0.100000': [111.0662310246862, 0, 0, 0, 0, 0, 0, 0], '0.000909_77.615165_1.128948_388.075827_0.000000_0.100000': [0, 0, 0.0515149636874412, 0, 0, 0, 0, 0], '0.001000_77.615165_1.128948_388.075827_0.000000_0.100000': [0, 0, 0, 236.4452279204872, 0, 0, 0, 0], '0.001000_77.615165_1.241843_426.883410_0.000000_0.100000': [0, 0, 0, 687.9887951587174, 0, 0, 0, 0], '0.001000_70.559241_1.502630_469.571751_0.000000_0.100000': [0, 0, 0, -1282.3004088972866, 0, 0, 0, 0], '0.001000_70.559241_1.652893_469.571751_0.000000_0.100000': [3295.3513732177034, 0, 0, 0, 0, 0, 0, 0], '0.001210_103.305785_1.652893_352.796206_0.000000_0.100000': [919.3341711472315, 0, -1.6399900254556996, 0, 0, 0, 0, 0], '0.001210_93.914350_1.652893_352.796206_0.000000_0.100000': [2361.231420161367, 0, 0, 0, 0, 0, -231.647684550903, 0], '0.001331_103.305785_1.818182_352.796206_0.000000_0.100000': [0, 0, -4.803068459778934, 0, 0, 8.178038172628916, 0, 0], '0.001100_103.305785_2.000000_426.883410_0.000000_0.100000': [1399.2682479644973, 0, 0, 0, 0, -5.161398786388185, 0, 36.267319618810795], '0.000683_125.000000_1.502630_426.883410_0.000000_0.100000': [0, 0, 0, 0, 3.0894392001011557, 0, -427.2219265717658, 0], '0.000621_125.000000_1.502630_426.883410_0.000000_0.100000': [0, 0, -3.0894392001011557, 0, 0, 0, 0, 0], '0.000683_137.500000_1.502630_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, 835.7642305034634, -400.3525184235784, 0], '0.000683_151.250000_1.502630_426.883410_0.000000_0.100000': [400.3525184235784, 0, 0, 0, 0, 0, 0, 0], '0.000683_137.500000_1.366027_426.883410_0.000000_0.100000': [445.0664216345014, 0, 0, -75.97856640940577, 0, 0, 0, 0], '0.000683_125.000000_1.366027_426.883410_0.000000_0.100000': [0, 0, -3.1500700429490283, 0, 0, 0, 0, 0], '0.001210_85.376682_1.502630_756.250000_0.000000_0.100000': [270.39290643585645, 0, 0, 0, 0, 0, 0, 0], '0.001210_77.615165_1.502630_756.250000_0.000000_0.100000': [0, 543.8431803059124, 0, 0, 0, 0, 0, 0], '0.001210_77.615165_1.502630_687.500000_0.000000_0.100000': [0, 0, 0, 0, 5.084317269705025, 0, 0, 0], '0.001100_77.615165_1.502630_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -544.0491356796236], '0.001100_77.615165_1.502630_756.250000_0.000000_0.100000': [1827.3433927006697, 0, 0, 0, 0, 0, 0, 0], '0.001100_70.559241_1.502630_756.250000_0.000000_0.100000': [0, 420.4851810756212, 0, 0, 0, 0, 0, 0], '0.001100_70.559241_1.502630_687.500000_0.000000_0.100000': [4494.181065208501, 0, 0, 0, 0, 0, 0, -38.22592555232921], '0.001100_64.144765_1.502630_687.500000_0.000000_0.100000': [-0.6962902901257166, 0, 0, 0, 0, 0, -408.56191501895444, 0], '0.001100_58.313423_1.502630_687.500000_0.000000_0.100000': [0, 0, 0, -0.039688752935353476, 0, 0, 0, 0], '0.001100_58.313423_1.652893_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001331_77.615165_1.652893_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -230.05395179125517, 0], '0.001331_85.376682_1.652893_516.528926_0.000000_0.100000': [0, 677.9201576099196, 0, 0, 0, 0, 0, 0], '0.001210_70.559241_1.652893_469.571751_0.000000_0.100000': [3299.7282527459647, 0, 0, 0, 0, 0, -121.11645117708507, 0], '0.001210_64.144765_1.652893_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0.07945436283430354, 0, 0, 0], '0.001100_64.144765_1.818182_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000909_113.636364_1.366027_426.883410_0.000000_0.100000': [0, 0, 402.71630559075817, 0, 0, 0, 0, 0], '0.001000_113.636364_1.366027_568.181818_0.000000_0.100000': [817.308969879502, 448.69780665384144, 0, 0, 5.046963562398832, 0, 0, 0], '0.001000_113.636364_1.366027_516.528926_0.000000_0.100000': [0, 2068.76476164587, 0, 0, 0, 0, 0, 0], '0.000909_113.636364_1.366027_352.796206_0.000000_0.100000': [1507.4510760958663, 0, 0, 0, 2.530723729959027, 0, -410.18312681405286, 0], '0.000909_103.305785_1.366027_352.796206_0.000000_0.100000': [0, 0, 0, 0, 1289.3533962138472, 0, -358.6778182083869, -162.05941312473638], '0.000909_125.000000_1.366027_352.796206_0.000000_0.100000': [0, 0, 0, 0, 2.8186228692000443, 0, 0, 0], '0.000826_125.000000_1.366027_352.796206_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000909_103.305785_1.502630_352.796206_0.000000_0.100000': [0, 429.0177013836455, 0, 0, 0, 1141.883015878214, 0, 0], '0.000826_93.914350_1.502630_320.723824_0.000000_0.100000': [772.7023880393509, 0, 0, -566.6451822415188, 0, 0, 0, 0], '0.000826_93.914350_1.652893_320.723824_0.000000_0.100000': [236.40790578578108, 0, 0, -16.089165544365006, 0, 0, 0, 0], '0.000826_93.914350_1.818182_320.723824_0.000000_0.100000': [3105.168563730351, 0, -1.5909363991806458, 0, 1.428220017811873, 0, 0, 0], '0.000909_93.914350_1.818182_320.723824_0.000000_0.100000': [0, 0, 0, 0, 1.5909363991806458, 0, 0, 0], '0.000826_85.376682_1.818182_320.723824_0.000000_0.100000': [2975.1566250531887, 0, 296.9769908199412, 0, 1.8116581150379716, 0, 0, 0], '0.001000_93.914350_1.502630_756.250000_0.000000_0.100000': [272.05369003760205, 0, 0, 0, 0, 0, 0, 0], '0.001000_85.376682_1.502630_756.250000_0.000000_0.100000': [0, 3576.147184566406, 0, 0, 0, 0, 0, 0], '0.001000_58.313423_1.502630_516.528926_0.000000_0.100000': [0, 0.6774560817442102, 0, 0, 0, 0, 0, 0], '0.001000_53.012202_1.366027_469.571751_0.000000_0.100000': [-0.7929260543539386, -0.0007915387496041148, -0.008653414391778825, 0, -0.852893823350405, 0, 0, 0], '0.001210_103.305785_1.652893_320.723824_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -268.04557696458517, 0], '0.001210_113.636364_1.652893_320.723824_0.000000_0.100000': [0, 0, 0, -138.2217473603091, 0, 0, 0, 0], '0.001210_113.636364_1.818182_320.723824_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -400.6963780689219, 0], '0.001210_125.000000_1.818182_320.723824_0.000000_0.100000': [0, 0, 0, 0, 4.105936813225617, 0, 0, 0], '0.001100_125.000000_1.818182_320.723824_0.000000_0.100000': [0, 0, 0, 0, 0, 254.11702209862486, 0, 0], '0.001100_125.000000_1.652893_320.723824_0.000000_0.100000': [305.0184285846609, 0, 0, 0, 0, 0, 0, 0], '0.001000_85.376682_1.366027_320.723824_0.000000_0.100000': [0, 0, 0, 0, 262.9011736532889, -605.1257358607717, 0, 0], '0.000909_85.376682_1.366027_320.723824_0.000000_0.100000': [531.1365239257881, 0, -0.6792423189399415, 0, 1175.0722378375979, 0, 0, 0], '0.001000_85.376682_1.241843_320.723824_0.000000_0.100000': [0, 0, -1.2134685858694867, 0, 0, 0, 0, 0], '0.001100_85.376682_1.241843_320.723824_0.000000_0.100000': [83.09785668579353, 0, 0, 0, 0, -38.439399214237895, 0, 0], '0.001100_85.376682_1.128948_320.723824_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000909_113.636364_2.420000_352.796206_0.000000_0.100000': [0, 0, 0, -88.97088815111293, 0, 0, 0, 0], '0.000909_113.636364_2.662000_352.796206_0.000000_0.100000': [0, 0, 0, -466.2196816388023, 0, 0, 0, 0], '0.000909_113.636364_2.928200_352.796206_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001611_137.500000_1.818182_568.181818_0.000000_0.100000': [0, 0, -6.29242704204371, 0, 0, 0, 0, 0], '0.001772_137.500000_1.818182_568.181818_0.000000_0.100000': [0, 0, -6.970152648000476, 0, 0, 0, -430.19363301500533, 0], '0.001772_151.250000_1.818182_568.181818_0.000000_0.100000': [391.08512092273213, 0, 0, 0, 0, 0, 0, 0], '0.001949_137.500000_1.818182_568.181818_0.000000_0.100000': [0, 0, 0, 0, -345.00645618245846, 0, 0, 0], '0.000909_77.615165_1.652893_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 4166.416180536958], '0.001210_70.559241_1.652893_568.181818_0.000000_0.100000': [27.636575453058324, 0, 0, 0, 0, 0, 0, 0], '0.001210_64.144765_1.652893_568.181818_0.000000_0.100000': [-0.824855072946184, -0.7194419887793174, -0.6709410053707779, 0, -0.793431543522388, 0, 0, 0], '0.001331_64.144765_1.652893_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0.7438693755197755, 0, 0, 0], '0.001210_58.313423_1.652893_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0.824855072946184, -0.006247374494387259], '0.001331_58.313423_1.652893_625.000000_0.000000_0.100000': [0, 0, 0.049319919675644995, 0, 0, 0, 0, 0], '0.001464_58.313423_1.652893_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001000_151.250000_1.502630_516.528926_0.000000_0.100000': [0, 387.54457310677736, 0, 0, 0, 0, 0, 0], '0.001000_151.250000_1.502630_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 1085.4133654118498, 0, 0], '0.001000_151.250000_1.241843_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, 108.32179282689559, 0, 0], '0.001000_151.250000_1.128948_426.883410_0.000000_0.100000': [374.3498989629098, 0, -2.015427008768893, 0, 0, 0, 0, 0], '0.001100_151.250000_1.128948_426.883410_0.000000_0.100000': [340.79352797061966, 0, 0, 0, 0, 0, 0, 0], '0.001100_137.500000_1.128948_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -354.57294060117766], '0.001100_137.500000_1.128948_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -339.2485419981705, 0], '0.001100_151.250000_1.128948_469.571751_0.000000_0.100000': [0, 0, 0, 0, 2.5475369844697524, 0, 0, 0], '0.001000_151.250000_1.128948_469.571751_0.000000_0.100000': [0, 352.49584465302763, 0, 0, 0, 0, 0, 0], '0.001000_137.500000_1.128948_426.883410_0.000000_0.100000': [0, 0, 0, -35.57813198427239, 0, 0, -34.031808996628165, 0], '0.001000_137.500000_1.241843_426.883410_0.000000_0.100000': [349.50164584382276, 0, 0, 0, 0, 0, 0, 0], '0.001000_125.000000_1.241843_426.883410_0.000000_0.100000': [0, 0, 0, 0, 1.7882099704430403, 0, 0, 0], '0.000683_64.144765_1.502630_516.528926_0.000000_0.100000': [0, 0, 0, -0.019520658559645198, 0, 0, 0, 0], '0.000683_64.144765_1.652893_516.528926_0.000000_0.100000': [0, 0, 0, 0, -0.038530531666060595, 0, 0, 0], '0.000621_64.144765_1.652893_516.528926_0.000000_0.100000': [0, 0, 0, -0.6885399242499034, 0, 0.025067701364355346, 0, 0], '0.000621_64.144765_1.818182_516.528926_0.000000_0.100000': [0, -0.0697550271514956, 0, 0, 0, 0, 0, 0], '0.000621_64.144765_1.818182_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0.820664315944373, 0, 0], '0.000621_64.144765_1.652893_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -0.062369364542973926], '0.000621_64.144765_1.502630_516.528926_0.000000_0.100000': [0.005433542828700411, 0, 0, 0, 0, 0, 0, 0], '0.000621_58.313423_1.502630_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001000_125.000000_1.818182_320.723824_0.000000_0.100000': [1940.1109047655696, 0, 204.2035998891472, -489.176983096981, 3.3541579852016183, 0, 0, 32.42169026500809], '0.001000_125.000000_2.000000_320.723824_0.000000_0.100000': [0, 0, 0, 0, 5.067385234414451, 0, 0, 0], '0.000909_125.000000_2.000000_320.723824_0.000000_0.100000': [0, 0, 0, -933.7031380171868, 0, 0, 0, 0], '0.000909_125.000000_2.200000_320.723824_0.000000_0.100000': [0, 0, 0, 0, 3.9840058257334023, 0, 0, 0], '0.000826_125.000000_2.200000_320.723824_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -474.82248438511823], '0.000826_125.000000_2.200000_352.796206_0.000000_0.100000': [0, 0, 0, 0, 0, 1500.9812005765339, 0, 0], '0.000909_93.914350_2.000000_352.796206_0.000000_0.100000': [945.8238569988218, 0, 0, -1094.587226191012, 0, 0, 0, 0], '0.000909_85.376682_2.200000_352.796206_0.000000_0.100000': [379.3269743312676, 0, -5.679718462622441, 0, 0, 0, 0, 0], '0.001000_85.376682_2.200000_352.796206_0.000000_0.100000': [379.46305889189244, 0, -6.250781537286457, 0, 0, 0, 0, 0], '0.001100_85.376682_2.200000_352.796206_0.000000_0.100000': [0, 0, -6.879218064272391, 0, 6.250781537286457, 0, 0, 0], '0.001000_77.615165_2.200000_352.796206_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000683_93.914350_1.502630_687.500000_0.000000_0.100000': [0, 0, 953.9178260827075, 0, 0, 0, 0, 0], '0.000683_77.615165_1.241843_687.500000_0.000000_0.100000': [0, 0, 0, 0, 1.3396819537461853, 0, 0, 0], '0.000621_77.615165_1.241843_687.500000_0.000000_0.100000': [0, -842.8658893107194, 0, 0, 0, 0, 0, 0], '0.000621_77.615165_1.241843_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, -1215.1463634310878, 0, 0], '0.000621_77.615165_1.128948_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000826_77.615165_1.652893_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 4187.332867882089], '0.000826_64.144765_1.652893_568.181818_0.000000_0.100000': [-0.04342041578106546, 0.001737346863843868, 0, 0, 0, 0, 0, 0], '0.000826_58.313423_1.652893_568.181818_0.000000_0.100000': [0, 0, 0, 0, -0.8026962617173922, 0, 0, 0], '0.000751_58.313423_1.652893_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 0.8052932332733531, 0, 0], '0.000751_53.012202_1.241843_516.528926_0.000000_0.100000': [0, 0, 0, -0.04768737436390893, 0, 0, 0, 0], '0.000751_53.012202_1.366027_516.528926_0.000000_0.100000': [0, -0.7104517362713295, 0, 0, 0, 0, 0, 0], '0.000751_53.012202_1.366027_469.571751_0.000000_0.100000': [0.7240353810827401, 0, 0, 0, 0, 0, 0, 0], '0.000751_48.192911_1.366027_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000564_103.305785_1.818182_568.181818_0.000000_0.100000': [0, 0, 0, -579.3607486388792, 0, 0, 0, 0], '0.000564_93.914350_2.000000_568.181818_0.000000_0.100000': [0, 1365.6524222898227, 0, 0, 0, 0, 0, 0], '0.000564_93.914350_2.000000_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -1172.2447617262135, 0], '0.000564_103.305785_2.000000_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000826_103.305785_1.366027_352.796206_0.000000_0.100000': [0, 1449.4652947733603, 0, 0, 0, -22.857123359781507, -358.3740135330757, 0], '0.000826_103.305785_1.366027_320.723824_0.000000_0.100000': [1839.11078045684, 0, 0, 0, 1.413983677826036, -37.28505395565244, -337.2629570113158, 0], '0.000826_113.636364_1.366027_320.723824_0.000000_0.100000': [0, 0, 0, -547.5821940441267, 0, -283.92603171313584, 0, 0], '0.000826_113.636364_1.502630_320.723824_0.000000_0.100000': [0, 0, -3.664873415341572, 0, 0, 0, 0, 0], '0.000909_113.636364_1.652893_320.723824_0.000000_0.100000': [0, 319.37736064473887, 0, 0, 0, 0, 0, 0], '0.000909_113.636364_1.652893_291.567113_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000826_85.376682_1.818182_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -228.44922165654157, 0], '0.000909_93.914350_1.818182_352.796206_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 27.187048478242144, 0], '0.000826_103.305785_1.818182_320.723824_0.000000_0.100000': [2550.1922560202015, 327.2087619443806, 0, 0, 0, 0, -351.6281087721233, -371.02781627171333], '0.000826_77.615165_1.818182_320.723824_0.000000_0.100000': [0, 0, 0, 614.2310839899428, 0, 0, 0, 0], '0.000826_77.615165_2.000000_320.723824_0.000000_0.100000': [0, 2501.088023040729, 0, 0, 12.528488789052972, 0, 0, 0], '0.000826_77.615165_2.000000_291.567113_0.000000_0.100000': [2307.631676692057, 0, 0, -351.85143842303535, 0, 0, 0, 0], '0.000826_77.615165_2.200000_291.567113_0.000000_0.100000': [0, 0, 0, 0, 4.295924088408447, 0, 0, 0], '0.000751_77.615165_2.200000_291.567113_0.000000_0.100000': [0, -319.6263841678944, 0, 0, 0, 0, 0, 0], '0.000751_77.615165_2.200000_265.061011_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -320.9750660941933, 0], '0.000751_85.376682_2.200000_265.061011_0.000000_0.100000': [0, 0, 0, -409.0667024826271, 0, 0, 0, 0], '0.000751_85.376682_2.420000_265.061011_0.000000_0.100000': [300.5374345668727, 0, 0, 0, 0, 0, 0, 0], '0.000751_77.615165_2.420000_265.061011_0.000000_0.100000': [0, 0, 0, -1022.6880166435429, 0, 0, 0, 0], '0.000751_77.615165_2.662000_265.061011_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001331_85.376682_1.652893_320.723824_0.000000_0.100000': [0, 382.36847590700756, 0, 0, 0, 0, 0, 0], '0.001331_85.376682_1.652893_291.567113_0.000000_0.100000': [0, 0, 0, -57.45210831521481, 0, 0, 0, 0], '0.001331_85.376682_1.818182_291.567113_0.000000_0.100000': [209.20979463534786, 0, 0, 0, 0, 0, 0, 0], '0.001331_77.615165_1.818182_291.567113_0.000000_0.100000': [0, 0, 0, 216.8308360529214, 0, 0, 0, -326.7931954257948], '0.001331_77.615165_1.818182_320.723824_0.000000_0.100000': [0, 326.7931954257948, 0, 0, 0, 0, 0, 0], '0.001331_77.615165_2.000000_291.567113_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001000_64.144765_1.366027_516.528926_0.000000_0.100000': [0.6807390615953431, 0, 0, 0, 0, 0, 0, 0], '0.001000_58.313423_1.366027_516.528926_0.000000_0.100000': [0, 0.7781398346271462, 0, 0, 0, 0, 0, 0], '0.001000_48.192911_1.366027_469.571751_0.000000_0.100000': [0, 0.046618806783862966, 0, 0, 0, 0, 0, 0], '0.001000_48.192911_1.366027_426.883410_0.000000_0.100000': [0, 0.773820031273499, 0, 0, 0, 0, 0, 0], '0.001000_48.192911_1.366027_388.075827_0.000000_0.100000': [0, 0, 0, 0, -0.011422127638161328, 0, 0, 0], '0.000909_48.192911_1.366027_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001000_93.914350_1.818182_388.075827_0.000000_0.100000': [0, 400.0784626489168, 0, 0, 0, 0, 0, 0], '0.001000_93.914350_1.818182_352.796206_0.000000_0.100000': [247.90243606803452, 0, -2.7543740790388256, 0, 0, 0, 0, 0], '0.001100_93.914350_1.818182_352.796206_0.000000_0.100000': [0, 0, 0, -261.8635767547181, 0, 0, 0, 0], '0.001100_93.914350_2.000000_320.723824_0.000000_0.100000': [0, 1640.7910052192146, 0, 0, 0, 0, 0, 0], '0.001100_93.914350_2.000000_291.567113_0.000000_0.100000': [3104.5380609433764, 0, 261.7024009072832, 0, 0, 0, 0, -34.01159200980222], '0.001210_93.914350_2.000000_291.567113_0.000000_0.100000': [0, 920.2278838666141, 0, 0, 0, 0, 0, 0], '0.001210_93.914350_2.000000_265.061011_0.000000_0.100000': [667.1865734558107, 0, 0, 0, 0, 0, 0, 0], '0.001210_85.376682_2.000000_265.061011_0.000000_0.100000': [330.5065088880361, 0, 0, 0, 0, -124.52727539556209, -60.10689850953249, 0], '0.001210_85.376682_1.818182_265.061011_0.000000_0.100000': [0, 0, 0, 0, 0, 113.37263296599394, -247.58213155611466, 0], '0.001210_85.376682_1.652893_265.061011_0.000000_0.100000': [0, 300.64779456012457, 0, 0, 0, 0, 0, 0], '0.001210_85.376682_1.652893_240.964556_0.000000_0.100000': [0, 0, 0, -120.52341358704261, 0, 0, 0, 0], '0.001210_85.376682_1.818182_240.964556_0.000000_0.100000': [1430.8069137216776, 0, 0, 0, 0, 0, 0, 0], '0.001210_77.615165_1.818182_240.964556_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000826_103.305785_1.818182_352.796206_0.000000_0.100000': [0, 0, 0, 0, 0, 77.92345672112116, 0, 0], '0.000826_103.305785_1.652893_352.796206_0.000000_0.100000': [231.28818416121666, 0, 0, -7.083950611011007, 0, 0, 0, 0], '0.000751_77.615165_1.818182_320.723824_0.000000_0.100000': [0, 554.7269384287326, 0, 0, 0, 0, 0, 0], '0.000751_77.615165_1.818182_291.567113_0.000000_0.100000': [0, 0, 0, 234.41869075677425, 0, 0, 0, 0], '0.000751_77.615165_2.000000_291.567113_0.000000_0.100000': [2002.1087464133457, 0, 0, 0, 0, 0, 0, 0], '0.001100_103.305785_1.128948_568.181818_0.000000_0.100000': [0, 777.051061173587, 0, 0, 0, 0, 0, 0], '0.001100_125.000000_1.502630_469.571751_0.000000_0.100000': [3052.855159681837, 0, 0, 241.27901333050227, 0, 0, 0, 0], '0.000751_113.636364_1.818182_516.528926_0.000000_0.100000': [0, 0, 280.2725907871398, 0, 0, 0, 0, 0], '0.000826_77.615165_1.502630_516.528926_0.000000_0.100000': [0, 1333.8830773715786, 0, 0, 0, 0, 0, 0], '0.000826_77.615165_2.000000_469.571751_0.000000_0.100000': [0, 0, 1304.0425784979598, 0, 0, 0, 0, 0], '0.000909_77.615165_2.000000_469.571751_0.000000_0.100000': [0, 0, 2084.2062812389663, 0, 0, 0, 0, 0], '0.001000_64.144765_1.818182_469.571751_0.000000_0.100000': [0, 0, 0, 0.6747081744397437, -0.1658120218200793, 0, 0, 0], '0.001000_58.313423_2.200000_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001100_103.305785_1.366027_320.723824_0.000000_0.100000': [455.85142060767475, 461.3047269250456, 0, 0, 870.1625831669385, 0, -338.3935958902948, 0], '0.001331_103.305785_2.000000_756.250000_0.000000_0.100000': [3314.6332953061587, 0, 0, -1202.6017392472054, 0, 0, 0, 0], '0.001331_103.305785_2.200000_756.250000_0.000000_0.100000': [0, 0, 0, 0, 6.080649513909157, 0, 0, 0], '0.001210_93.914350_2.200000_915.062500_0.000000_0.100000': [0, -10552.85982488909, 0, 0, 0, 0, 0, 0], '0.001210_93.914350_1.818182_831.875000_0.000000_0.100000': [0, 0, -4.750086297502821, 0, 0, 0, 0, 0], '0.001331_93.914350_1.818182_831.875000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 889.6151473370077], '0.001331_93.914350_1.818182_915.062500_0.000000_0.100000': [564.7527092364944, 0, 0, 0, 0, 0, 0, 0], '0.001331_85.376682_1.818182_915.062500_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001000_125.000000_1.366027_516.528926_0.000000_0.100000': [0, 1373.7983914420233, 0, 0, 0, 0, 0, 0], '0.000751_103.305785_1.652893_388.075827_0.000000_0.100000': [223.8445200994115, 0, 0, 0, 0, 0, 0, 0], '0.000751_93.914350_1.652893_388.075827_0.000000_0.100000': [0, 0, 456.3755718209347, 0, 0, 0, 0, 0], '0.000826_93.914350_1.366027_388.075827_0.000000_0.100000': [194.84401181498754, 0, 1091.427960909342, -655.9440681676178, 2.155523992763574, 0, 0, -359.5163837457157], '0.000826_93.914350_1.366027_426.883410_0.000000_0.100000': [672.2836305165935, 0, 0, 0, 0, -72.83428445845902, 0, 0], '0.000826_93.914350_1.241843_426.883410_0.000000_0.100000': [0, 330.04307784645124, 0, 0, 0, 0, 0, 0], '0.000826_93.914350_1.241843_388.075827_0.000000_0.100000': [0, 0, 0, 1750.220319236253, 0, 0, 0, 0], '0.000826_85.376682_1.366027_388.075827_0.000000_0.100000': [207.9371146978546, 444.0946839965477, 163.6347570828571, 0, 0, -679.3395206345907, -6.627663096200831, 0], '0.000826_85.376682_1.241843_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0.6988893453233322, 0, 0, 0], '0.000751_85.376682_1.241843_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0.35301650189685463, 0, 0, 0], '0.000683_85.376682_1.241843_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000683_85.376682_1.502630_516.528926_0.000000_0.100000': [362.85159979876505, 0, 0, 0, 0, 0, 0, 0], '0.000683_77.615165_1.502630_516.528926_0.000000_0.100000': [0, 0, -3.3946628355659416, 0, 0, 0, 0, 0], '0.000751_77.615165_1.502630_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 118.32178868420888, 0], '0.000683_64.144765_1.502630_568.181818_0.000000_0.100000': [-0.16980246911223276, 0, 0, 0, 0, 0, 0, -0.021752241676904305], '0.000683_64.144765_1.502630_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000751_85.376682_1.818182_568.181818_0.000000_0.100000': [-2783.7561089055557, -2284.9411540823035, -3114.9757922063063, -3458.0316525848048, -3699.6038340915898, -3683.330326535067, 0, 0], '0.000751_64.144765_1.818182_568.181818_0.000000_0.100000': [0, 0, -0.07398765847888171, 0, 0, 0, 0, 0], '0.000826_64.144765_1.818182_568.181818_0.000000_0.100000': [0.01673443222186961, 0, 0, 0, 0, 0, 0, 0], '0.000826_58.313423_1.818182_568.181818_0.000000_0.100000': [0, 0.04265239513069901, 0, 0, 0, 0, 0, 0], '0.000826_58.313423_1.818182_516.528926_0.000000_0.100000': [0, 0, 0, -0.01850202603411617, 0, 0, 0, 0], '0.000826_58.313423_2.000000_516.528926_0.000000_0.100000': [0, 0.6328593279089592, 0, 0, 0, 0, -0.19924958743908974, 0], '0.000826_64.144765_2.000000_516.528926_0.000000_0.100000': [0.22863475271672543, 0, 0, 0, 0, 0, 0, 0], '0.000826_53.012202_2.200000_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, -0.7368922031145758, 0, 0], '0.000826_53.012202_2.000000_469.571751_0.000000_0.100000': [0, 0, 0.03344634662212229, 0, 0, 0, 0, 0], '0.000909_53.012202_2.000000_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0.7424722468067103, 0, 0], '0.000909_53.012202_1.818182_469.571751_0.000000_0.100000': [0, 0, 0, 0, -0.7563734589187562, 0, 0, 0], '0.000826_53.012202_1.818182_469.571751_0.000000_0.100000': [-0.037607091500331084, 0, 0, 0, 0, 0, 0, 0], '0.000826_48.192911_1.818182_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0.0338421346060386, 0, 0, 0], '0.000751_48.192911_1.818182_469.571751_0.000000_0.100000': [0.7273155742909694, 0, 0, 0, 0, 0, 0, 0], '0.000751_43.811737_1.818182_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000909_166.375000_2.200000_426.883410_0.000000_0.100000': [0, 0, -4.141915563188923, 0, 0, 0, 0, 0], '0.001000_166.375000_2.420000_426.883410_0.000000_0.100000': [0, 508.6703596193715, 0, 0, 0, 0, 0, 0], '0.001000_166.375000_2.420000_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -202.7618550381545, 0], '0.001000_183.012500_2.420000_388.075827_0.000000_0.100000': [202.7618550381545, 0, 0, 0, 0, 0, 0, 0], '0.001000_70.559241_1.366027_320.723824_0.000000_0.100000': [816.0442623617668, 142.01738307094115, 0, 0, 0, 703.6661302609713, 0, 0], '0.001000_70.559241_1.366027_291.567113_0.000000_0.100000': [0, 0, 0, 0, 0, 536.9220183776977, 0, 0], '0.001000_70.559241_1.241843_291.567113_0.000000_0.100000': [0, 0, 0, -14.970545669843077, 0, 0, 0, 485.0101051860651], '0.001000_70.559241_1.241843_320.723824_0.000000_0.100000': [33.9203977938099, 0, 0, 0, 0, -1382.6709162226837, 0, 0], '0.001000_77.615165_1.652893_516.528926_0.000000_0.100000': [1328.3021801203315, 387.82318700965027, -1.0463438059321561, 0, 0, 0, 0, 0], '0.001210_70.559241_1.818182_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001210_77.615165_2.000000_516.528926_0.000000_0.100000': [0, 0, 0, 0, 3878.6957623341928, 0, 0, 0], '0.001100_77.615165_2.200000_516.528926_0.000000_0.100000': [0, 2821.857857733169, 0, 0, 0, 0, 0, 0], '0.001210_70.559241_2.000000_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001100_70.559241_1.818182_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 2778.3037363930566], '0.001464_125.000000_1.652893_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001100_103.305785_1.366027_516.528926_0.000000_0.100000': [0, 2163.400283982225, 0, -993.695324988599, 0, 0, 0, 0], '0.001210_125.000000_1.366027_426.883410_0.000000_0.100000': [796.4730756648983, 0, 0, 0, 0, 0, 0, 0], '0.001000_93.914350_1.366027_756.250000_0.000000_0.100000': [2871.57838569351, 0, 0, 0, 0, 0, 0, -544.5131943258577], '0.001000_93.914350_1.366027_831.875000_0.000000_0.100000': [0, 544.5131943258577, 0, 0, 0, 0, 0, 0], '0.001000_85.376682_1.366027_756.250000_0.000000_0.100000': [0, 1372.8737612381105, 0, 0, 0, 844.973958047029, -3.144855417774585, 0], '0.000909_77.615165_1.366027_687.500000_0.000000_0.100000': [0, 0, 0, 0, 2.506711217751945, -63.288304682748915, 0, 0], '0.000826_77.615165_1.366027_687.500000_0.000000_0.100000': [0, 0, 0, -2997.769045038904, 0, 0, 0, 0], '0.000826_77.615165_1.502630_687.500000_0.000000_0.100000': [0, 0, 0, 0, 3.438165303927417, 0, 0, 0], '0.000751_77.615165_1.502630_687.500000_0.000000_0.100000': [0, 516.5674064993926, 0, 0, 0, 0, 0, 0], '0.000751_77.615165_1.502630_625.000000_0.000000_0.100000': [0, 0, 0, 5105.642832008487, 0, 0, 0, 0], '0.000751_77.615165_1.652893_625.000000_0.000000_0.100000': [0, 0, 0, -5122.617295750962, 0, 0, 0, 0], '0.000826_103.305785_1.241843_320.723824_0.000000_0.100000': [190.50893216061124, 0, 0, 0, 0, 0, 0, 0], '0.000826_93.914350_1.241843_320.723824_0.000000_0.100000': [129.76486337278712, 0, 0, 0, 0, 0, 0, -183.84381762698547], '0.000826_93.914350_1.241843_352.796206_0.000000_0.100000': [125.77195524856569, 26.91080043096184, 0, 0, 0, 0, 0, 0], '0.000826_85.376682_1.241843_352.796206_0.000000_0.100000': [79.42506433932668, 0, 0, 0, 0, -239.67139039514132, -11.330806779150059, 0], '0.000826_77.615165_1.241843_352.796206_0.000000_0.100000': [0, 0, 0, 756.2685564525385, 0, 0, 0, 0], '0.000826_77.615165_1.366027_352.796206_0.000000_0.100000': [1756.4482803012218, 0, 0, 0, 0, 0, 0, 0], '0.000826_64.144765_1.366027_352.796206_0.000000_0.100000': [0, 0, 0, 0, 0, 0.019840582313488293, 0, 0], '0.000826_64.144765_1.241843_352.796206_0.000000_0.100000': [-0.044639230055736334, -0.0009061354872073934, -0.08972999155896334, -1.7856541938837678e-07, -0.8039606042008245, 0, 0, 0], '0.001210_151.250000_2.200000_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 2461.8767775260017, 0, 0], '0.001210_103.305785_2.000000_756.250000_0.000000_0.100000': [3536.638104803195, 0, 0, 0, 0, 0, 0, -364.8120913816243], '0.001210_77.615165_1.818182_756.250000_0.000000_0.100000': [-2024.8011922146975, -1696.5054399526036, 0, 0, 0, 0, 0, 0], '0.001210_77.615165_1.652893_687.500000_0.000000_0.100000': [0, 0, 0, -613.0377939574587, 0, 0, 0, 0], '0.001210_64.144765_2.000000_756.250000_0.000000_0.100000': [-0.14819463086623727, 0, 0, 0, 0, 0, 0, 0], '0.001210_58.313423_2.000000_756.250000_0.000000_0.100000': [0.8331925992703874, 0, 0, 0, 0.7563224817063656, 0, 0, 0], '0.001100_58.313423_2.000000_756.250000_0.000000_0.100000': [0, 0, -0.7563224817063656, 0, 0, 0, 0, 0], '0.001210_53.012202_2.000000_756.250000_0.000000_0.100000': [0, 0, 0, -0.04840661483090116, 0, 0, 0, 0], '0.001210_53.012202_2.200000_756.250000_0.000000_0.100000': [0, -0.0030938915497703795, 0, 0, 0, 0, 0, 0], '0.001210_58.313423_2.420000_625.000000_0.000000_0.100000': [0, -0.04080260410273695, 0, 0, 0, 0, 0, 0], '0.001210_58.313423_2.420000_568.181818_0.000000_0.100000': [-0.6927525185976492, 0, -0.7346655483081577, 0, 0, 0, 0, 0], '0.001331_58.313423_2.420000_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000826_103.305785_1.502630_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 1039.0657220317078, -74.52910169443491, 0], '0.000826_103.305785_1.366027_388.075827_0.000000_0.100000': [0, 0, 0, 0, 2.598065200474412, 351.01073324864296, 0, 0], '0.000751_103.305785_1.366027_388.075827_0.000000_0.100000': [0, 347.1080732702453, 0, 0, 0, 0, 0, 0], '0.000751_103.305785_1.366027_352.796206_0.000000_0.100000': [0, 0, -2.0147433912841737, 0, 0, -23.960786766606997, 0, 0], '0.000826_113.636364_1.366027_352.796206_0.000000_0.100000': [0, 382.34955183737645, 0, 0, 2.285579481604054, 0, 0, 0], '0.000826_113.636364_1.241843_320.723824_0.000000_0.100000': [0, 263.8014965813324, 0, 0, 0, 0, -349.3988076942225, 0], '0.000826_113.636364_1.241843_291.567113_0.000000_0.100000': [265.3449943815029, 0, 0, 0, 0, 0, 0, 0], '0.000826_103.305785_1.241843_291.567113_0.000000_0.100000': [175.2554743572683, 0, 0, 0, 0, 0, 0, 0], '0.000826_93.914350_1.241843_291.567113_0.000000_0.100000': [0, 221.7646395314685, 0, 0, 0, 0, 0, 0], '0.000826_93.914350_1.241843_265.061011_0.000000_0.100000': [0, 0, 0, 0, 0.5017746799260749, 0, 0, 0], '0.000751_93.914350_1.241843_265.061011_0.000000_0.100000': [0, 0, 0, 0, 0, -137.3252747732932, 0, 0], '0.000751_93.914350_1.128948_265.061011_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -292.8585502738594], '0.000751_93.914350_1.128948_291.567113_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -131.59722305885825, 0], '0.000751_103.305785_1.128948_291.567113_0.000000_0.100000': [0, 0, 0, 0, -0.8427284527883785, 0, 0, 0], '0.000683_103.305785_1.128948_291.567113_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -259.2106386116012], '0.000683_93.914350_1.128948_320.723824_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001100_85.376682_1.241843_568.181818_0.000000_0.100000': [0, 0, 0, -28.417732756768498, 0, 0, -1047.230034810502, 0], '0.001100_93.914350_1.241843_568.181818_0.000000_0.100000': [1047.230034810502, 0, 0, 0, 0, -272.22857123504883, 0, 0], '0.001100_93.914350_1.128948_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, -284.9020844622669, 0, 0], '0.001100_93.914350_1.026316_568.181818_0.000000_0.100000': [227.74095591582628, 0, 0, 0, 0, 0, 0, 0], '0.001100_85.376682_1.026316_568.181818_0.000000_0.100000': [0, 0, -2.628611743455804, 0, 0, 0, 0, 0], '0.001210_85.376682_1.026316_568.181818_0.000000_0.100000': [0, 566.6914376424204, 0, 0, 0, 0, 0, 0], '0.001210_85.376682_1.026316_516.528926_0.000000_0.100000': [0, 533.8225607418866, 0, 0, 0, 0, 0, 0], '0.001210_85.376682_1.026316_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001000_103.305785_2.420000_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -475.3140460776467], '0.001000_103.305785_2.420000_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -286.05987429301825, 0], '0.001210_85.376682_1.502630_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001100_64.144765_1.818182_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 0.005547008191297209, 0, 0], '0.001100_53.012202_1.652893_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001000_77.615165_1.241843_687.500000_0.000000_0.100000': [0, 0, 0, 86.72634543863938, 0, 0, 0, 0], '0.000909_77.615165_1.366027_625.000000_0.000000_0.100000': [-191.89604736633137, -1156.210869103247, 0, 0, 0, 0, 0, 0], '0.001000_77.615165_1.241843_625.000000_0.000000_0.100000': [3397.281754490829, 0, 0, 0, 0, 0, -177.70288375953243, 0], '0.001000_85.376682_1.241843_625.000000_0.000000_0.100000': [0, 0, 0, 0, 3.424974103524164, 0, 0, 0], '0.000909_85.376682_1.241843_625.000000_0.000000_0.100000': [0, 0, 0, -208.28939626520477, 0, 0, 0, 0], '0.000909_70.559241_1.366027_625.000000_0.000000_0.100000': [2833.4818451039655, 0, 0, 0, 0, 0, 0, 0], '0.000909_64.144765_1.366027_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001100_77.615165_1.502630_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 1842.719771647515], '0.000751_137.500000_1.502630_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -790.3341785639369], '0.001100_93.914350_1.241843_352.796206_0.000000_0.100000': [115.72532742747781, 269.76968701964563, 0, 0, 0, 0, -209.04864873346924, -221.123306713835], '0.001100_93.914350_1.366027_320.723824_0.000000_0.100000': [1710.7349308900962, 0, 0, 0, 0, 0, 0, -277.61727886698736], '0.001100_85.376682_1.366027_320.723824_0.000000_0.100000': [126.30625463600293, 1308.7966333414656, -18.15600853873384, 0, 0, 0, 0, 0], '0.001100_70.559241_1.366027_320.723824_0.000000_0.100000': [811.2983760743366, 0, 0, 0, 0, 0, 0, 0], '0.001100_64.144765_1.366027_320.723824_0.000000_0.100000': [0, 0, 0, 0, 0, -0.7621274706951681, 0, 0.017549813913102074], '0.001100_64.144765_1.366027_352.796206_0.000000_0.100000': [0.01758196298170333, 0, 0, 0, 0, 0, 0, 0], '0.001100_58.313423_1.366027_352.796206_0.000000_0.100000': [-0.015291625019517597, 0, 0, 0, 0, 0, -0.001598360271063937, 0], '0.000683_77.615165_1.502630_568.181818_0.000000_0.100000': [5150.313384539768, 0, 0, 0, 0, 0, 0, 0], '0.000683_70.559241_1.502630_568.181818_0.000000_0.100000': [0, 0, 26.69372323191068, 0, 0, 0, 0, -8.97725393677058], '0.000683_58.313423_1.502630_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001100_77.615165_1.818182_426.883410_0.000000_0.100000': [0, 0, 0, 0, 1562.1517415711203, 0, 0, -462.5384077729631], '0.001000_77.615165_1.818182_426.883410_0.000000_0.100000': [0, 0, -2.821886319934856, 0, 3541.7571221735134, 0, 0, 0], '0.000909_77.615165_1.818182_426.883410_0.000000_0.100000': [4106.316180832204, 0, 0, 0, 469.3204382558386, 0, 0, 0], '0.000909_70.559241_1.818182_426.883410_0.000000_0.100000': [0, 1734.6144669632376, 0, 1688.6325135818402, 0, -665.3063973544272, 0, 0], '0.000909_58.313423_1.652893_426.883410_0.000000_0.100000': [0, 0, 0.005208810703925338, 0, 0, 0, 0, 0], '0.001000_58.313423_1.652893_426.883410_0.000000_0.100000': [0, -0.7714121588532635, -0.014257763078512653, 0, 0, 0, 0, 0], '0.001000_58.313423_1.652893_388.075827_0.000000_0.100000': [0.023190868136362043, 0, 0, 0, 0, 0, 0, 0], '0.001000_53.012202_1.652893_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0.8388067540454631], '0.001000_48.192911_1.652893_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0.0468294575071468, 0], '0.001210_64.144765_2.000000_469.571751_0.000000_0.100000': [-0.03560968997755936, 0, 0, -0.011861265314860248, 0, 0, 0, 0], '0.001210_64.144765_2.200000_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0.011861265314860248, 0, 0], '0.001210_58.313423_2.000000_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0.049945179112319237, 0, 0], '0.001210_58.313423_1.818182_469.571751_0.000000_0.100000': [0.7014642590382891, 0, -0.04650757693091236, 0, 0, 0, 0, 0], '0.001331_58.313423_1.818182_469.571751_0.000000_0.100000': [0, 0.786849328715698, 0, 0, 0, 0, 0, 0], '0.001331_58.313423_1.818182_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0.006545389520624667, 0, 0, 0], '0.001210_58.313423_1.818182_426.883410_0.000000_0.100000': [0, -0.03803133352337227, 0, 0, 0, 0, 0, 0], '0.001210_58.313423_1.818182_388.075827_0.000000_0.100000': [-0.009871063103673805, 0, 0, 0, 0, 0, 0, 0], '0.001210_53.012202_1.818182_388.075827_0.000000_0.100000': [0, 0, 0, 0, -0.7711125646396341, 0, 0, 0], '0.001100_53.012202_1.818182_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0.7980496226467949, 0, 0, 0], '0.001000_53.012202_1.818182_352.796206_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001000_64.144765_1.502630_388.075827_0.000000_0.100000': [-0.596096760705973, -0.7618026422737785, -0.007664075127672732, 0.03039958803394029, -0.005411152838269118, -0.6697713986767738, -36.89482454466679, 0], '0.001000_64.144765_1.366027_388.075827_0.000000_0.100000': [0, 0, -0.14506641760668815, 0, 0, 0, 0, 0], '0.001100_64.144765_1.366027_388.075827_0.000000_0.100000': [0.8597236031072157, 0, 0.00012050573786837049, 0, 0, 0, 0, 0], '0.001100_58.313423_1.366027_388.075827_0.000000_0.100000': [0, 0, 0, 0.01775614215813448, 0, 0, 0, 0], '0.001100_58.313423_1.502630_388.075827_0.000000_0.100000': [-0.007555821560640652, -0.8136384907333877, -0.8213609437215837, -0.7966659140964566, 0, 0, 0, 0], '0.001100_58.313423_1.502630_352.796206_0.000000_0.100000': [0, 0, 0, 0.8657951916767889, 0, 0, 0, 0], '0.001100_58.313423_1.652893_352.796206_0.000000_0.100000': [0, 0, 0, 0, 0, -0.07870865378879895, 0, -0.7701139612510588], '0.001100_58.313423_1.652893_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001331_70.559241_2.000000_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -13.173036600562545], '0.001331_70.559241_2.000000_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -4671.431460496307], '0.001331_70.559241_2.000000_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 4682.2335077824955, 0, 0], '0.001331_70.559241_1.818182_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001100_85.376682_1.241843_352.796206_0.000000_0.100000': [0, 0, 0, 615.8046850622803, 0, 0, 0, 0], '0.001100_85.376682_1.366027_352.796206_0.000000_0.100000': [214.3946080193475, 0, 0, 0, 0, 0, 0, 0], '0.001100_70.559241_1.366027_352.796206_0.000000_0.100000': [0, 0, -2.6375318165708452, 0, 0, 0, 0, 0], '0.001210_70.559241_1.366027_352.796206_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000826_113.636364_1.818182_320.723824_0.000000_0.100000': [351.6281087721233, 0, 0, 0, 0, 0, 0, 0], '0.000826_103.305785_1.818182_291.567113_0.000000_0.100000': [0, 0, 0, 0, 0, 59.6332074180943, 0, 0], '0.000751_103.305785_1.652893_291.567113_0.000000_0.100000': [0, 311.3878308284281, 0, 0, 0, 0, 0, 0], '0.000751_103.305785_1.652893_265.061011_0.000000_0.100000': [0, 0, 0, -64.18526859980848, 0, 0, 0, 0], '0.000751_103.305785_1.818182_265.061011_0.000000_0.100000': [0, 0, 0, 0, 1.9531281644703087, 0, 0, 0], '0.000683_103.305785_1.818182_265.061011_0.000000_0.100000': [0, 285.2957149290811, 0, 0, 0, 0, 0, 0], '0.000683_103.305785_1.818182_240.964556_0.000000_0.100000': [0, 0, -2.0463613360099773, 0, 0, 0, 0, 0], '0.000751_103.305785_1.818182_240.964556_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000621_113.636364_1.652893_625.000000_0.000000_0.100000': [0, -197.57507281474133, 0, 0, 0, 0, 0, 0], '0.000621_85.376682_1.652893_516.528926_0.000000_0.100000': [-788.3213758192713, -601.1424870942552, 0, 0, 57.682010725086826, 0, 0, 0], '0.000621_85.376682_1.652893_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 363.18237259665193, -212.4004561899349, 0], '0.000621_85.376682_1.502630_469.571751_0.000000_0.100000': [0, 0, 448.11567283670445, 0, 0, 0, 0, 0], '0.000751_125.000000_1.818182_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -419.88458205483806], '0.000826_85.376682_1.652893_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, 1455.9425964343784, 0, 0], '0.000826_70.559241_2.000000_388.075827_0.000000_0.100000': [1747.0814333671715, 279.4916935258613, 0, 0, 0, 1562.2036855624806, 0, 0], '0.000826_70.559241_1.818182_388.075827_0.000000_0.100000': [0, 0, 0, -1562.2036855624806, 0, 0, 0, 0], '0.000826_70.559241_2.000000_352.796206_0.000000_0.100000': [1468.1908982972411, 0, 0, 0, 0, 0, 0, 0], '0.000826_64.144765_2.000000_352.796206_0.000000_0.100000': [0, 0, 0, -0.6772840100815074, 0, 0, 0, 0], '0.000826_64.144765_2.200000_352.796206_0.000000_0.100000': [0, 0, 0.8049165678440204, 0, 0, 0, 0, 0], '0.000909_64.144765_2.200000_352.796206_0.000000_0.100000': [0, 0.009897555936254321, 0, 0, 0, 0, 0, 0], '0.000909_64.144765_2.200000_320.723824_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000826_103.305785_1.241843_388.075827_0.000000_0.100000': [785.901352232002, 0, 0, 0, 0, -172.20380768540574, 0, 0], '0.000826_103.305785_1.128948_388.075827_0.000000_0.100000': [131.2685631004666, 317.5185061521752, 0, 0, 0, 0, 0, 0], '0.000826_93.914350_1.128948_388.075827_0.000000_0.100000': [0, 323.79195816361863, 0, 0, 0, 0, 0, 0], '0.000826_93.914350_1.128948_352.796206_0.000000_0.100000': [0, 0, 0, 348.1105998414906, 0, 0, 0, 0], '0.000751_85.376682_1.818182_320.723824_0.000000_0.100000': [0, 0, 0, 337.49702082868066, 0, 0, 0, 0], '0.000751_85.376682_2.000000_320.723824_0.000000_0.100000': [258.35559105514415, 0, 0, 0, 0, -30.681547348061866, 0, 0], '0.000751_77.615165_2.000000_320.723824_0.000000_0.100000': [0, 0, 0, -1277.8550903448022, 0, 0, -25.835559105514392, 0], '0.000751_77.615165_2.200000_320.723824_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001100_113.636364_1.502630_469.571751_0.000000_0.100000': [1492.755901186135, 2255.090986126, 0, 0, 0, 861.5696565018803, 0, 0], '0.001100_113.636364_1.241843_469.571751_0.000000_0.100000': [0, 0, 0, 0, 2.130248460474013, 0, 0, 0], '0.001000_113.636364_1.241843_469.571751_0.000000_0.100000': [0, 0, 0, 0, 1.9114382421246774, 0, 0, 0], '0.000909_113.636364_1.241843_469.571751_0.000000_0.100000': [0, 0, 0, 0, 1.716778270589657, 0, 0, 0], '0.000826_113.636364_1.241843_469.571751_0.000000_0.100000': [0, 0, 0, 0, 1.5433625318293707, 0, 0, 0], '0.000751_113.636364_1.241843_469.571751_0.000000_0.100000': [0, 0, 0, 0, 1.388663227932375, 0, 0, 0], '0.000683_113.636364_1.241843_469.571751_0.000000_0.100000': [0, 0, 0, -258.99081439980813, 0, 0, 0, 0], '0.000683_113.636364_1.366027_469.571751_0.000000_0.100000': [420.68847633165797, 0, 0, 0, 0, 0, 0, 0], '0.000683_103.305785_1.366027_469.571751_0.000000_0.100000': [0, 410.22806407534983, 0, 0, 0, 0, 0, 0], '0.000683_103.305785_1.366027_426.883410_0.000000_0.100000': [0, 386.46602004551227, 0, 0, 0, 0, 0, 0], '0.000683_103.305785_1.366027_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 11.410505038398787, 0, 0], '0.000683_103.305785_1.241843_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000909_125.000000_1.366027_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 41.87368079789968], '0.000909_93.914350_1.241843_516.528926_0.000000_0.100000': [0, 0, 0, 1113.3818372540595, 2.067832872449344, -250.3519508173991, 0, 0], '0.000909_85.376682_1.128948_516.528926_0.000000_0.100000': [4225.869502219034, 0, 0, 0, 0, 0, 0, 0], '0.000909_77.615165_1.128948_516.528926_0.000000_0.100000': [2768.9897122241955, 0, 0, 0, 0, 0, 0, 0], '0.000909_70.559241_1.128948_516.528926_0.000000_0.100000': [32.915555244817575, -2835.550604564122, 0, 0, 0, 0, 0, 0], '0.000909_70.559241_1.128948_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 2771.536888775066, 0, 0], '0.000909_70.559241_1.026316_469.571751_0.000000_0.100000': [0, 0, -88.24925176448653, 0, 0, 0, 0, 0], '0.001000_70.559241_1.026316_469.571751_0.000000_0.100000': [0, 0, 1.5818871810538895, 0, 0, 0, 0, 0], '0.001100_70.559241_1.026316_469.571751_0.000000_0.100000': [0, -546.5107185927501, 0, 0, 0, 0, -2367.4312936049537, 0], '0.001100_77.615165_1.026316_469.571751_0.000000_0.100000': [2367.4312936049537, 0, 0, 0, 0, 0, 0, 0], '0.001100_70.559241_1.026316_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001100_93.914350_1.502630_352.796206_0.000000_0.100000': [2005.9929150573378, 0, 0, 0, 0, 0, 0, 0], '0.000826_85.376682_1.366027_320.723824_0.000000_0.100000': [0, 1140.954398088707, -0.851481623590189, -188.90867046556073, -78.39143286817989, -665.0919048111028, 0, 0], '0.000826_85.376682_1.366027_291.567113_0.000000_0.100000': [0, 0, 12.759653292171242, 0, -267.3736081206148, -491.07887116515604, -299.823309213639, 2.8046826650582206], '0.000751_85.376682_1.366027_291.567113_0.000000_0.100000': [1188.145490984963, 397.82873738451997, -1.4041325696342883, 14.954776523585224, 0, 0, -319.65602248070826, -220.00126006287164], '0.000751_77.615165_1.366027_291.567113_0.000000_0.100000': [1202.0062588083551, 0, 0, 0, 0, 70.10128327406483, 676.570461682045, 0], '0.000751_85.376682_1.502630_291.567113_0.000000_0.100000': [1324.2479319328813, 0, 0, 0, 0, 0, 0, 0], '0.000909_77.615165_1.502630_568.181818_0.000000_0.100000': [0, 0, 0, 4652.497866328837, 0, 0, 0, 0], '0.000826_58.313423_1.502630_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0.7004206918297109, 0], '0.000751_58.313423_1.502630_516.528926_0.000000_0.100000': [0, 0.6256196321734478, 0, 0, 0, 0, 0, 0], '0.000751_58.313423_1.502630_469.571751_0.000000_0.100000': [0, 0, 0, -0.7095575276978252, 0, 0, 0, 0], '0.000751_58.313423_1.652893_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0.707987206601706, 0, 0, 0], '0.000683_58.313423_1.652893_469.571751_0.000000_0.100000': [0, 0, -0.07079872066017062, 0, 0, 0, 0, 0], '0.000826_93.914350_1.502630_352.796206_0.000000_0.100000': [0, 0, 0, 0, 0, 558.2782968290943, 0, 0], '0.000826_93.914350_1.366027_352.796206_0.000000_0.100000': [632.8775020984233, 0, 273.1690938970419, 0, 0, -114.02822922786855, 0, 0], '0.000826_85.376682_1.128948_352.796206_0.000000_0.100000': [0, 0, 0, 0, 0, 1125.6943383518549, 0, 0], '0.000826_85.376682_1.026316_352.796206_0.000000_0.100000': [-225.69587784659583, 0, 0, 0, 0, 0, 0, 0], '0.000826_77.615165_1.026316_352.796206_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 159.4888270088141], '0.000826_77.615165_1.026316_388.075827_0.000000_0.100000': [838.6621780997981, 0, 0, 0, 0, 0, 0, 0], '0.000826_70.559241_1.026316_388.075827_0.000000_0.100000': [833.3320786742113, 0, 0, 0, 0, 0, 0, 0], '0.000826_64.144765_1.026316_388.075827_0.000000_0.100000': [0.8332929635499113, 0, 0, 0, 0, 0, -75.75746169765557, 0], '0.000826_58.313423_1.026316_388.075827_0.000000_0.100000': [0, 0, 0, -0.7706801342250856, 0, 0, 0, 0], '0.000826_58.313423_1.128948_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0.005171454484457172, 0], '0.000826_64.144765_1.128948_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000826_125.000000_1.241843_320.723824_0.000000_0.100000': [0, 0, 0, 0, 0, 48.60768723471932, 0, 0], '0.000826_125.000000_1.128948_320.723824_0.000000_0.100000': [241.08063623503887, 0, 0, -4.4188806577017585, 0, 0, 0, 0], '0.000909_93.914350_1.502630_352.796206_0.000000_0.100000': [0, 0, 0, 0, 460.65739802364783, 0, 0, 0], '0.000826_85.376682_1.366027_352.796206_0.000000_0.100000': [1964.2272483445677, -225.51936074027083, 0, 0, 0, 0, -632.8775020984233, 0], '0.000826_85.376682_1.241843_320.723824_0.000000_0.100000': [0, 0, 0, 0, 0.9395021452141519, 0, -11.028122187805433, 0], '0.000751_85.376682_1.241843_320.723824_0.000000_0.100000': [82.63632902673089, 0, -0.08463983290217586, 0, 0, 0, -117.45620539495404, 0], '0.000751_77.615165_1.502630_352.796206_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -315.6200224942483], '0.000751_77.615165_1.502630_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000751_113.636364_1.366027_352.796206_0.000000_0.100000': [0, 0, 0, 0, 2.2720147427596475, 0, 0, 0], '0.000683_113.636364_1.366027_352.796206_0.000000_0.100000': [357.8606288325782, 0, -0.20654679479633137, 0, 0, 0, 0, 0], '0.000683_103.305785_1.366027_352.796206_0.000000_0.100000': [0, 0, 0, 0, 0, -24.97202268808169, 0, 0], '0.000683_103.305785_1.241843_352.796206_0.000000_0.100000': [0, 0, 0, 0, 0.7244939051894107, 0, 0, 0], '0.000621_103.305785_1.241843_352.796206_0.000000_0.100000': [0, 0, 0, 0, 0.6473461782380582, 0, 0, 0], '0.000564_103.305785_1.241843_352.796206_0.000000_0.100000': [207.06094139836478, 0, 0, 0, 0, 0, 0, 0], '0.000564_93.914350_1.241843_352.796206_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000826_85.376682_2.000000_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, -327.54883977523417, 0, 0], '0.000909_137.500000_1.366027_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -439.4552063597684], '0.000909_137.500000_1.366027_469.571751_0.000000_0.100000': [0, 0, 0, 0, 4.049216722665551, 0, 0, 0], '0.000826_137.500000_1.366027_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -466.95004939167956], '0.000826_137.500000_1.366027_516.528926_0.000000_0.100000': [0, 0, 393.07978467092596, 0, 0, 0, 0, 0], '0.000909_103.305785_1.652893_352.796206_0.000000_0.100000': [0, 0, 0, 0, 0, 1194.7624906393596, 0, 0], '0.000751_70.559241_1.366027_291.567113_0.000000_0.100000': [0, 0, -101.92311546146462, 238.62705568463082, -103.74142682268094, 0, 15.869556353196833, 0], '0.000751_70.559241_1.502630_291.567113_0.000000_0.100000': [339.12823976664106, 0, 0, 0, 0, -18.980757312753195, -976.5444087895613, 283.6911164574491], '0.000751_93.914350_1.652893_426.883410_0.000000_0.100000': [0, 0, 0, 0, 624.5315555507025, 0, 0, 0], '0.000909_64.144765_1.366027_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0.8433925059630627, 0, 0, 0], '0.000826_64.144765_1.366027_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 0.05105271731352627, 0, 0], '0.000826_64.144765_1.241843_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0.0030559255269190545, -0.7600851567388418, 0, 0], '0.000751_64.144765_1.241843_388.075827_0.000000_0.100000': [0.008386748841746793, 0, 0, 0, 0, -0.7571938359751812, 0, 0], '0.000751_64.144765_1.128948_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0.759492151833706, 0, 0, 0], '0.000683_64.144765_1.128948_388.075827_0.000000_0.100000': [0, 0, 0, -0.0024875198775816897, 0, 0, 0, 0], '0.000683_64.144765_1.241843_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -0.7706201282692974], '0.000683_64.144765_1.241843_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000826_64.144765_2.000000_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0.7805392661893166, 0, 0, 0], '0.000751_64.144765_2.000000_426.883410_0.000000_0.100000': [0, 0, 0, -0.7370380438464509, 0, 0, 0, 0], '0.000751_64.144765_2.200000_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0.7304698436338413, 0, 0, 0], '0.000909_77.615165_1.366027_352.796206_0.000000_0.100000': [787.1383980544151, 0, 0, 0, 0, 0, 0, -263.02491980679093], '0.001100_93.914350_1.241843_320.723824_0.000000_0.100000': [115.69395764867814, 0, 0, 0, 0, 0, 0, -269.76968701964563], '0.001210_77.615165_1.241843_352.796206_0.000000_0.100000': [0, 0, -1.2082842728520518, 0, 0, 0, 0, 0], '0.001331_77.615165_1.241843_352.796206_0.000000_0.100000': [0, 0, -1.3819801831645238, 0, 0, 0, 0, 0], '0.001464_77.615165_1.241843_352.796206_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000826_93.914350_2.000000_426.883410_0.000000_0.100000': [0, 0, 0, 0, 1238.6474373023013, 0, 0, 0], '0.000751_77.615165_1.502630_291.567113_0.000000_0.100000': [0, 0, 0, 0, 0, 1279.6739670178836, 0, -214.7779765401405], '0.000751_77.615165_1.502630_320.723824_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001100_77.615165_1.502630_352.796206_0.000000_0.100000': [0, 0, 0, 0, 1590.896190562033, 0, 0, 0], '0.001100_85.376682_1.502630_516.528926_0.000000_0.100000': [333.7609608017794, 0, 0, 0, 0, 0, 0, 0], '0.001100_77.615165_1.502630_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 4152.993067047439], '0.001210_58.313423_1.502630_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0.01260762013071144], '0.001210_58.313423_1.502630_625.000000_0.000000_0.100000': [0, 0, 0, 0, -0.014277110077318867, 0, 0, 0], '0.001100_58.313423_1.502630_625.000000_0.000000_0.100000': [0, 0.03171553331865198, 0, 0, 0, 0, 0, 0], '0.001100_58.313423_1.652893_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 0.7666222748580594, 0, 0], '0.001000_58.313423_1.502630_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001210_113.636364_2.000000_388.075827_0.000000_0.100000': [436.0593040754625, 0, 0, 0, 0, 0, 0, 0], '0.001210_103.305785_2.000000_388.075827_0.000000_0.100000': [0, 0, 0, -1200.6977740020943, 0, 0, 0, 0], '0.001331_85.376682_2.200000_352.796206_0.000000_0.100000': [0, 0, 0, -1114.7167692844196, 0, 0, 0, 0], '0.001331_85.376682_2.420000_352.796206_0.000000_0.100000': [0, 0, 0, -247.83312405395864, 0, 0, 0, 0], '0.001331_85.376682_2.662000_352.796206_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -520.7013560987247], '0.001331_85.376682_2.662000_388.075827_0.000000_0.100000': [235.0702855813348, 0, 0, 0, 0, 0, 0, 0], '0.001331_77.615165_2.662000_388.075827_0.000000_0.100000': [6036.403308667286, 0, 0, 0, 0, 0, 0, 0], '0.001331_70.559241_2.662000_388.075827_0.000000_0.100000': [0, -3747.8492981096297, 0, 0, 0, 0, 0, 0], '0.001331_70.559241_2.662000_352.796206_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001000_64.144765_1.241843_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, -0.006754385233034554, 0, 0], '0.001000_64.144765_1.128948_568.181818_0.000000_0.100000': [0, 0.010929146333000971, 0, 0, 0, 0, 0, 0], '0.001000_93.914350_2.200000_320.723824_0.000000_0.100000': [1305.3250976519807, 0, 0, -955.1197039661652, 0, 0, 0, 0], '0.001000_93.914350_2.420000_320.723824_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -350.6915933268102, -551.6263305353617], '0.001000_103.305785_2.420000_320.723824_0.000000_0.100000': [350.6915933268102, 0, -4.9415255998134855, 0, 0, 0, 0, 0], '0.001100_103.305785_2.420000_320.723824_0.000000_0.100000': [0, 0, 0, 0, 4.9415255998134855, 0, 0, 0], '0.001000_93.914350_2.420000_352.796206_0.000000_0.100000': [320.2205666857709, 0, 0, 0, 0, 0, 0, 0], '0.001000_85.376682_2.420000_352.796206_0.000000_0.100000': [0, 0, -5.295435459462351, 0, 0, 0, 0, 0], '0.001100_85.376682_2.420000_352.796206_0.000000_0.100000': [0, 0, 0, 0, 0, 1116.8973773895843, 0, 0], '0.001210_85.376682_2.200000_352.796206_0.000000_0.100000': [0, 1254.401120734118, 0, 0, 0, 0, 0, 0], '0.001210_85.376682_2.200000_320.723824_0.000000_0.100000': [273.1542041764896, 0, 0, 0, 0, 0, 0, 0], '0.001210_77.615165_2.200000_320.723824_0.000000_0.100000': [0, 0, 0, -1780.9093244585163, 0, 0, 0, 0], '0.001210_77.615165_2.420000_320.723824_0.000000_0.100000': [0, 0, 0, -372.0796036224988, 0, 0, 0, 0], '0.001210_77.615165_2.662000_320.723824_0.000000_0.100000': [0, 0, -2.8739560275834264, 0, 0, 0, 0, 0], '0.001331_77.615165_2.662000_320.723824_0.000000_0.100000': [0, 0, -3.2066279274182308, 0, 0, 0, 0, 0], '0.001464_77.615165_2.662000_320.723824_0.000000_0.100000': [0, 0, 0, -441.73694292194887, 0, 0, 0, 0], '0.001464_77.615165_2.928200_320.723824_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001000_70.559241_1.652893_516.528926_0.000000_0.100000': [0, 0, 0, 801.8268570937688, 0, 0, 0, 0], '0.001000_70.559241_1.818182_469.571751_0.000000_0.100000': [2100.9433094104565, 0, 0, 0, 0, 0, 0, 0], '0.000909_64.144765_1.818182_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0.9118039196973359, 0, 0], '0.000909_58.313423_1.652893_469.571751_0.000000_0.100000': [0.028867679581257022, 0, 0, 0, -0.6816457765211226, 0, 0, 0], '0.000826_58.313423_1.652893_469.571751_0.000000_0.100000': [0.6366453094838107, 0, 0, 0, 0, 0, 0, 0], '0.000826_53.012202_1.652893_469.571751_0.000000_0.100000': [0, 0, -0.003489342435523213, 0, 0, 0, 0, 0], '0.000909_53.012202_1.502630_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0.024575893673652444, 0], '0.000909_58.313423_1.502630_388.075827_0.000000_0.100000': [0, 0, 0.029012986587122213, 0, 0.03661199368089001, 0, 0, 0], '0.001000_58.313423_1.502630_388.075827_0.000000_0.100000': [-0.1275519579498905, 0, 0.6346118709667735, 0, 0, 0, 0, 0], '0.001000_53.012202_1.502630_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0.7597476123734168, 0, 0, 0], '0.000909_53.012202_1.502630_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000751_103.305785_1.241843_352.796206_0.000000_0.100000': [0, 0, 0, 0, 0, -166.6800346592153, 0, 0], '0.000751_103.305785_1.128948_352.796206_0.000000_0.100000': [0, 0, 0, 0, 0, 93.42943586693218, 0, 0], '0.000751_103.305785_1.026316_352.796206_0.000000_0.100000': [0, 0, 0, 0, 0.9996230202436891, 0, 0, 0], '0.000683_103.305785_1.026316_352.796206_0.000000_0.100000': [0, 0, 0, -94.74509861145752, 0, 0, 0, 0], '0.000683_93.914350_1.128948_352.796206_0.000000_0.100000': [0, 0, 0, 0, -0.4185397517349884, 0, 0, 0], '0.000621_93.914350_1.128948_352.796206_0.000000_0.100000': [0, 0, 0, 0, 0, 262.9603338294097, 0, 0], '0.000621_93.914350_1.026316_352.796206_0.000000_0.100000': [0, 0, 0, 0, 0, -62.29314106445527, 0, 0], '0.000621_93.914350_0.933015_352.796206_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001000_93.914350_1.241843_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, -302.0904135558964, 0, 0], '0.001000_93.914350_1.128948_625.000000_0.000000_0.100000': [114.92395686172277, 0, 0, 0, 0, 0, 0, -450.81105006064917], '0.001000_93.914350_1.128948_687.500000_0.000000_0.100000': [0, 0, 0, 1133.1574677702615, 0, 0, 0, 0], '0.000909_77.615165_1.241843_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 2382.5377907578886], '0.000909_77.615165_1.241843_756.250000_0.000000_0.100000': [0, 0, 0, -4899.437964731197, 0, 0, 0, 0], '0.000909_77.615165_1.366027_756.250000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000909_113.636364_1.366027_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -498.8039861206675], '0.000909_103.305785_1.241843_756.250000_0.000000_0.100000': [0, 0, 0, 0, 1.6859585723696, 0, 0, 0], '0.000826_103.305785_1.241843_756.250000_0.000000_0.100000': [0, 0, 0, 0, 0, -319.00897416974203, 0, 0], '0.000826_103.305785_1.128948_756.250000_0.000000_0.100000': [0, 0, 0, 0, 1.1979615574337004, 0, 0, 0], '0.000751_103.305785_1.128948_756.250000_0.000000_0.100000': [123.07166499739105, 0, 0, 0, 0, 0, 0, 0], '0.000751_93.914350_1.128948_756.250000_0.000000_0.100000': [0, 0, -0.6524826296143147, 0, 0, 0, 0, 0], '0.000826_93.914350_1.128948_756.250000_0.000000_0.100000': [0, 0, 0, 384.62711403928, 0, 0, 0, 0], '0.000826_85.376682_1.241843_756.250000_0.000000_0.100000': [0, 481.37646551053825, 0, 0, 0, 0, 0, 0], '0.000826_85.376682_1.241843_687.500000_0.000000_0.100000': [0, 0, 0, 0, 3.833268632598356, 0, 0, 0], '0.000751_85.376682_1.241843_687.500000_0.000000_0.100000': [0, 440.8672544720531, 0, 0, 0, 0, 0, 0], '0.000751_85.376682_1.241843_625.000000_0.000000_0.100000': [0, 0, 0, 0, 2.5245658653972898, 0, 0, 0], '0.000683_85.376682_1.241843_625.000000_0.000000_0.100000': [0, 551.0758706157719, 0, 0, 0, 0, 0, 0], '0.000683_85.376682_1.241843_568.181818_0.000000_0.100000': [0, 0, -3.1226656438764624, 0, 0, 0, 0, 0], '0.000751_85.376682_1.241843_516.528926_0.000000_0.100000': [111.90791976122482, 0, 0, 0, 0, 0, 0, 0], '0.001000_77.615165_1.652893_568.181818_0.000000_0.100000': [3957.9556654075727, 0, 0, 0, 0, 0, 0, -1415.1930457146264], '0.001000_77.615165_1.652893_625.000000_0.000000_0.100000': [5291.629526795928, 0, 0, 0, 0, 0, 0, 0], '0.001000_103.305785_1.366027_426.883410_0.000000_0.100000': [0, 0, 0, 0, 944.8214131726058, 0, 0, 0], '0.001210_85.376682_1.818182_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001000_64.144765_2.200000_426.883410_0.000000_0.100000': [0, -0.7923829866184405, 0, 0, 0, 0, 0, 0], '0.001000_64.144765_2.200000_388.075827_0.000000_0.100000': [0, 0, 0.8794608939770283, 0, 0, 0, 0, 0], '0.001100_64.144765_2.200000_388.075827_0.000000_0.100000': [0, 0.01881874397804495, 0, -0.6868329875087283, 0, 0, 0, 0.034648086122831774], '0.001100_64.144765_2.420000_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 0.6868329875087283, 0, 0], '0.000683_85.376682_1.818182_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 37.42671470543255, 0, 0], '0.000564_77.615165_1.652893_516.528926_0.000000_0.100000': [0, 0, -0.05062033213220273, 0, 0.4960999183837885, 0, 0, 0], '0.000513_77.615165_1.652893_516.528926_0.000000_0.100000': [1320.769898838085, 0, -0.04509999258034441, 0, 0, 0, 0, 0], '0.000513_70.559241_1.652893_516.528926_0.000000_0.100000': [3272.06265333426, 0, 0, 0, 0, 0, 0, 0], '0.000513_64.144765_1.652893_516.528926_0.000000_0.100000': [0, 0.7359960693046788, 0, 0, 0, 0, 0, 0], '0.000513_64.144765_1.652893_469.571751_0.000000_0.100000': [-0.5670717480117901, 0, 0, 0, 0, 0, 0, 0], '0.000513_58.313423_1.652893_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001100_166.375000_1.652893_687.500000_0.000000_0.100000': [0, 457.95958756467644, 0, 0, 0, 0, 0, 0], '0.001000_166.375000_1.652893_625.000000_0.000000_0.100000': [0, 813.016865302958, 0, 0, 0, 0, 0, 0], '0.000909_64.144765_1.502630_352.796206_0.000000_0.100000': [0, 0, 0, 0, 0, 0.7953989725186011, 0, 0], '0.000909_64.144765_1.366027_352.796206_0.000000_0.100000': [0, 0, 0.048775253167136726, 0, 0, -0.7298957192283524, 0, 0], '0.000909_64.144765_1.241843_352.796206_0.000000_0.100000': [0, 0, 0, 0, 0, 0.845309538045476, 0, 0], '0.000909_64.144765_1.128948_352.796206_0.000000_0.100000': [-0.7283492956462169, 0.008216927288651732, 0, 0, 0, 0, 0, 0], '0.000909_58.313423_1.128948_352.796206_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0.7283492956462169, 0], '0.000909_64.144765_1.128948_320.723824_0.000000_0.100000': [0, -0.00689107806301763, 0, 0, -0.02992269824176219, 0, 0, 0], '0.000909_64.144765_1.128948_291.567113_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001000_64.144765_2.420000_469.571751_0.000000_0.100000': [0, 0, -0.03484238568544242, 0, 0, 0, 0, 0], '0.001100_64.144765_2.420000_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, -0.004510933447500066, 0, 0], '0.001100_58.313423_2.000000_469.571751_0.000000_0.100000': [0, 0.6973269603206359, 0, 0, 0, 0, 0, 0], '0.001100_58.313423_2.000000_426.883410_0.000000_0.100000': [-0.7150045978983371, 0.030561748593656398, 0, 0, -0.7256659386240927, 0, 0, -0.62732082617174], '0.001100_58.313423_2.000000_388.075827_0.000000_0.100000': [-0.624371166558808, -0.7818848877453093, 0.0202649851010392, 0, 0, 0, 0, 0], '0.001100_58.313423_2.000000_352.796206_0.000000_0.100000': [0, 0, 0, 0, 0.80852501000943, 0, 0, 0], '0.001000_58.313423_2.000000_352.796206_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000826_64.144765_1.818182_469.571751_0.000000_0.100000': [0, 0.9935437368886958, 0.7566781529743116, 0, 0, 0, 0, 0], '0.000826_64.144765_1.818182_426.883410_0.000000_0.100000': [-0.04395146307874711, -0.6173297412879037, -0.03598806169047748, 0, 0, 0, 0, 0], '0.000826_58.313423_1.818182_426.883410_0.000000_0.100000': [0, 0, 0, -0.001505552173784963, 0, 0, 0, 0], '0.000826_58.313423_2.000000_426.883410_0.000000_0.100000': [0, -0.7818885038378994, 0, 0, 0, 0, 0, 0], '0.000826_58.313423_2.000000_388.075827_0.000000_0.100000': [0, 0, 0, 0.013077532045122098, 0, 0, 0, 0], '0.000826_58.313423_2.200000_388.075827_0.000000_0.100000': [0.01275135412279269, 0, 0, 0, 0, 0, 0, 0], '0.000826_53.012202_2.200000_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 0.7835962355242145, 0, 0], '0.000826_53.012202_2.000000_388.075827_0.000000_0.100000': [-0.022441162588211227, -0.005508760530233303, -0.023999115133960913, 0, 0, 0, 0, -0.8236016363354111], '0.000826_53.012202_2.000000_426.883410_0.000000_0.100000': [0, 0.9059617999689522, 0, 0, 0, 0, 0, 0], '0.000826_48.192911_2.000000_388.075827_0.000000_0.100000': [0.007824095093974392, 0, 0, 0, 0, 0, 0, 0], '0.000826_43.811737_2.000000_388.075827_0.000000_0.100000': [0, 0, -0.02253437921146997, 0, 0, 0, 0, 0], '0.000909_43.811737_2.000000_388.075827_0.000000_0.100000': [0, 0, 0.04183721987274112, 0, 0, 0, 0, 0], '0.001000_43.811737_2.000000_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, -0.7850660721904433, 0, 0], '0.001000_43.811737_1.818182_388.075827_0.000000_0.100000': [0, 0, 0, 0.7850660721904433, 0, 0, 0, 0], '0.000826_103.305785_1.366027_568.181818_0.000000_0.100000': [370.70679003298, 0, 0, -1007.2632133756247, 0, 0, 0, 0], '0.000826_70.559241_1.366027_516.528926_0.000000_0.100000': [121.56997164831648, -2.5887694783441932, 0, 0, 0, 0, 0, 0], '0.000826_70.559241_1.366027_469.571751_0.000000_0.100000': [124.59182638400218, 0, 0, 0, 0, -1292.5676886338788, 0, 112.00174396182904], '0.000826_70.559241_1.241843_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -1836.4168843776579, 0], '0.000826_77.615165_1.241843_469.571751_0.000000_0.100000': [1836.4168843776579, 0, 0, 0, 0, 0, 0, 0], '0.000564_85.376682_1.652893_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, -782.7838198388348, 0, 0], '0.000564_85.376682_1.502630_516.528926_0.000000_0.100000': [332.3268970932336, 0, 0, 0, 0, 0, 0, 0], '0.000564_77.615165_1.502630_516.528926_0.000000_0.100000': [0, 1320.7340279202756, 0, 0, 0, 0, 0, 0], '0.000564_77.615165_1.502630_469.571751_0.000000_0.100000': [0, 0, -2.6269106909498077, 0, 0, 0, 0, 0], '0.000621_77.615165_1.502630_469.571751_0.000000_0.100000': [0, 0, -2.4431949883324706, 0, 0, 0, 0, 0], '0.000683_77.615165_1.502630_469.571751_0.000000_0.100000': [0, 0, -6.770514249027019, 0, 0, 0, 0, 0], '0.000751_70.559241_1.502630_469.571751_0.000000_0.100000': [0, 0, 0, -2848.338901326626, 0, 0, 0, 0], '0.000751_70.559241_1.652893_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000909_113.636364_1.366027_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -440.8369074994965], '0.000909_113.636364_1.366027_516.528926_0.000000_0.100000': [0, 0, 745.8471961324891, 0, 0, 0, 0, 0], '0.000826_85.376682_1.502630_320.723824_0.000000_0.100000': [0, -104.70751965939621, -2.343005421755197, 0, 0, 0, 0, 0], '0.000826_85.376682_1.502630_291.567113_0.000000_0.100000': [0, 0, 0, 0, 2.0348209632127694, 0, -303.2154159535734, 0], '0.000826_93.914350_1.502630_291.567113_0.000000_0.100000': [303.2154159535734, 0, 0, 0, 0, 0, 0, 0], '0.001000_113.636364_1.241843_388.075827_0.000000_0.100000': [292.7092625014302, 0, 0, 0, 0, 0, 0, 0], '0.001100_85.376682_1.241843_388.075827_0.000000_0.100000': [0, 0, -0.6921949605072769, 0, 0, 0, 0, 0], '0.001210_85.376682_1.241843_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -151.64218220325264, 0], '0.001210_93.914350_1.241843_388.075827_0.000000_0.100000': [0, 0, 0, 97.75603177593875, 0, 0, 0, 0], '0.001210_93.914350_1.366027_388.075827_0.000000_0.100000': [0, 323.3598880393328, 0, 0, 0, 0, 0, 0], '0.001210_85.376682_1.366027_352.796206_0.000000_0.100000': [0, 371.72151454968576, 0, 0, 0, 0, 0, 0], '0.001210_85.376682_1.502630_320.723824_0.000000_0.100000': [136.67545336043992, 0, 0, 0, 0, 0, 0, 0], '0.001210_77.615165_1.502630_320.723824_0.000000_0.100000': [0, 0, 0, -1068.1675103804566, 0, 0, 0, 0], '0.001210_77.615165_1.652893_320.723824_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001000_77.615165_1.652893_469.571751_0.000000_0.100000': [3910.792958601026, 0, 0, 0, 0, 0, 0, 0], '0.001000_64.144765_1.652893_426.883410_0.000000_0.100000': [-0.009738688856941735, -0.011594461815298019, 0, 0, 0, 0, 0, 0], '0.001000_64.144765_1.652893_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0.03455000217033444, 0, 0, 0], '0.000909_64.144765_1.652893_388.075827_0.000000_0.100000': [-0.01771405689580661, -0.775900931608794, 0, 0, 0, -0.0336130715615127, 0, 0], '0.000909_64.144765_1.502630_388.075827_0.000000_0.100000': [0, 0, 0, 0, -0.06853486239301675, 0, -86.65431881323227, 0], '0.000826_64.144765_1.502630_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 0.09369918990745875, 0, 0], '0.000683_113.636364_1.502630_388.075827_0.000000_0.100000': [0, 458.28036576012386, 0, 0, 0, 0, 0, 0], '0.000683_113.636364_1.502630_352.796206_0.000000_0.100000': [0, 0, 372.1009235474325, 0, 0, 0, 0, 0], '0.000751_103.305785_1.502630_320.723824_0.000000_0.100000': [0, 0, 0, -446.92069422846646, 0, 0, 0, 0], '0.000751_103.305785_1.652893_320.723824_0.000000_0.100000': [0, 0, -0.9386484512215247, 0, 0, 0, 0, 0], '0.000826_103.305785_1.652893_320.723824_0.000000_0.100000': [255.86215493235105, 0, 0, 0, 0, 0, 0, 0], '0.000826_85.376682_1.652893_320.723824_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -23.640790578578102, 0], '0.000621_77.615165_1.652893_388.075827_0.000000_0.100000': [0, 0, 0, 0, 1957.3640478876277, 0, 0, 0], '0.000564_77.615165_1.652893_388.075827_0.000000_0.100000': [0, 3429.726340584535, 0, 0, 0, 0, 0, 0], '0.000564_77.615165_1.652893_352.796206_0.000000_0.100000': [3287.573592868836, 0, -1.777334622061426, 0, 0, 0, 0, 0], '0.000621_77.615165_1.652893_352.796206_0.000000_0.100000': [0, 428.7908046012514, 0, 0, 0, 0, 0, 0], '0.000621_77.615165_1.652893_320.723824_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -261.6292792387873, 0], '0.000621_85.376682_1.652893_320.723824_0.000000_0.100000': [0, 385.5517213895699, 0, 0, 0, 0, 0, 0], '0.000621_85.376682_1.652893_291.567113_0.000000_0.100000': [283.66372862546353, 0, -1.9982101997185964, 0, 0, 0, 0, 0], '0.000683_85.376682_1.652893_291.567113_0.000000_0.100000': [0, 0, 0, 0, 1.9982101997185964, 0, 0, 0], '0.000621_77.615165_1.652893_291.567113_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000826_77.615165_1.818182_352.796206_0.000000_0.100000': [0, 0, 0, 0, 33.87671234642744, 0, 0, 0], '0.000751_70.559241_2.000000_291.567113_0.000000_0.100000': [0, -4.47637542307482, 0, 0, 0, 0, 0, 0], '0.000751_70.559241_2.000000_265.061011_0.000000_0.100000': [79.36430289008047, 0, 0, 0, 0, 0, 0, 0], '0.000751_64.144765_2.000000_265.061011_0.000000_0.100000': [0, -0.695961563581568, 0, 0, 0, 0, 0, 0.015826739772819565], '0.000751_64.144765_2.000000_240.964556_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -285.0060848331793, 0], '0.000751_70.559241_2.000000_240.964556_0.000000_0.100000': [0, 0, 0, 0, -0.8234386005750821, 0, 0, 0], '0.000683_70.559241_2.000000_240.964556_0.000000_0.100000': [0, 0, 0, 0, 0.17734529500839927, 0, 0, 0], '0.000621_70.559241_2.000000_240.964556_0.000000_0.100000': [0, 0, 0, 210.64700815148237, 0, 0, 0, 0], '0.000621_70.559241_2.200000_240.964556_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000751_93.914350_1.818182_320.723824_0.000000_0.100000': [0, 0, 0, 0, 1.283567813706395, 0, 0, 0], '0.000683_93.914350_1.818182_320.723824_0.000000_0.100000': [247.6621075711175, 0, 0, 0, 0, 0, 0, 0], '0.000683_85.376682_1.818182_320.723824_0.000000_0.100000': [0, 333.3260278726355, 0, 0, 0, 0, 0, 0], '0.000683_85.376682_1.818182_291.567113_0.000000_0.100000': [0, 0, 0, 0, 1.7837312821250635, 0, 0, 0], '0.000621_85.376682_1.818182_291.567113_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -250.97147118741552, 0], '0.000621_93.914350_1.818182_291.567113_0.000000_0.100000': [0, 0, 0, 0, 1.47261362412155, 0, 0, 0], '0.000564_93.914350_1.818182_291.567113_0.000000_0.100000': [0, 310.6337633485194, 0, 0, 0, 0, 0, 0], '0.000564_93.914350_1.818182_265.061011_0.000000_0.100000': [0, 0, 0, -230.28564703820368, 0, 0, 0, 0], '0.000564_93.914350_2.000000_265.061011_0.000000_0.100000': [0, 247.1006550390094, 0, 0, 0, 0, 0, 0], '0.000564_93.914350_2.000000_240.964556_0.000000_0.100000': [0, 0, 0, -546.7482579711132, 0, 0, 0, 0], '0.000564_93.914350_2.200000_240.964556_0.000000_0.100000': [0, 464.52772066239277, 0, 0, 0, 0, 0, 0], '0.000564_93.914350_2.200000_219.058687_0.000000_0.100000': [0, 0, 0, 0, 2.0677208051311027, 0, 0, 0], '0.000513_93.914350_2.200000_219.058687_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001331_85.376682_1.502630_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, 1152.4106382013626, 0, 0], '0.001331_70.559241_1.366027_426.883410_0.000000_0.100000': [0, 1133.1668123996374, 0, 0, 0, 0, 0, 0], '0.001331_70.559241_1.366027_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000909_77.615165_2.200000_756.250000_0.000000_0.100000': [0, 0, -4.0767872591306515, 0, 0, 0, 0, 0], '0.001000_77.615165_2.200000_756.250000_0.000000_0.100000': [261.37845955656815, 0, 0, 0, 0, 0, 0, 0], '0.001000_70.559241_2.200000_756.250000_0.000000_0.100000': [0, -845.2903817756651, 0, 0, 0, 0, 0, 0], '0.001000_70.559241_2.200000_687.500000_0.000000_0.100000': [0, 7686.389081999543, 0, 0, 0, 0, 0, 0], '0.001000_64.144765_2.200000_625.000000_0.000000_0.100000': [0, 0, 0, 0.7530800562113018, 0, 0, 0, 0], '0.001000_64.144765_2.420000_625.000000_0.000000_0.100000': [-0.7334957469982328, 0, 0, 0, 0, -0.0684618232919365, 0, 0], '0.001000_58.313423_2.420000_625.000000_0.000000_0.100000': [0.7239348960976714, 0, 0, 0, 0, 0, 0, 0], '0.001000_53.012202_2.420000_625.000000_0.000000_0.100000': [0, 0, 0, 0.03993516065620497, 0, 0, 0, 0], '0.001000_48.192911_2.662000_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0.8686159742934934, 0], '0.001000_53.012202_2.662000_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, -0.7698377064799615, 0, 0], '0.001000_53.012202_2.420000_568.181818_0.000000_0.100000': [0, 0, 0.7200632124566937, 0, 0, 0, 0, 0], '0.001100_53.012202_2.420000_568.181818_0.000000_0.100000': [0, 0, 0, -0.7368943667612712, 0, 0, 0, 0], '0.001100_53.012202_2.662000_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001100_85.376682_1.366027_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 93.16738464987222, 0, 0], '0.000909_85.376682_1.241843_516.528926_0.000000_0.100000': [0, 0, -0.2937317306352905, 0, 0, 2987.5160192686294, 0, 0], '0.000621_64.144765_1.818182_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001100_85.376682_2.000000_756.250000_0.000000_0.100000': [0, 0, 900.5188718300742, 0, 0, 0, 0, -321.04374069351036], '0.001210_70.559241_1.818182_756.250000_0.000000_0.100000': [0, 0, 0, -1409.6160643379415, 0, 0, 0, 0], '0.001210_70.559241_2.000000_756.250000_0.000000_0.100000': [0, 0, -7.305848481223438, 0, 0, 0, 0, 0], '0.001331_70.559241_2.000000_756.250000_0.000000_0.100000': [0, 6050.805365933688, 0, 0, 0, 0, 0, 0], '0.000826_77.615165_1.366027_469.571751_0.000000_0.100000': [0, 359.7937775874043, 0, 0, 0, 0, -16.705172332555822, 0], '0.001100_77.615165_1.366027_469.571751_0.000000_0.100000': [0, 0, -3.129868737322431, 0, 0, -658.2860426133557, 0, 0], '0.001210_77.615165_1.366027_469.571751_0.000000_0.100000': [0, 0, 0, 0, 3.129868737322431, 0, 0, 0], '0.001100_77.615165_1.241843_469.571751_0.000000_0.100000': [0, 0, 0, 0, 1.02750049093047, 0, 0, 0], '0.001000_77.615165_1.241843_469.571751_0.000000_0.100000': [0, 0, 0, 1611.0958033179973, 0, 0, 0, 0], '0.000909_64.144765_1.366027_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0.7458489383219103, 0, 0, 0], '0.000826_64.144765_1.366027_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, 0.02630567267087827, 0, 0], '0.000826_64.144765_1.241843_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000826_93.914350_1.241843_516.528926_0.000000_0.100000': [0, 0, 0, 5.19502021234137, 0, 0, 0, 0], '0.000751_77.615165_1.366027_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -168.28053812412736, 0], '0.000751_85.376682_1.366027_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -249.87608237589134], '0.000826_70.559241_1.502630_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -2689.3449932938797, 0], '0.000826_103.305785_1.502630_469.571751_0.000000_0.100000': [0, 0, 1184.6029862178045, 0, 0, 1083.6823287866794, 0, 0], '0.000826_103.305785_1.366027_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 320.0567613735154, 0, 0], '0.000826_103.305785_1.241843_469.571751_0.000000_0.100000': [268.49400438591664, 0, 0, 0, 0, -158.11223262699522, 0, 0], '0.000826_103.305785_1.128948_469.571751_0.000000_0.100000': [0, 373.95590338819375, 0, 0, 0, 0, 0, 0], '0.000826_103.305785_1.128948_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, -80.58727658649559, 0, 0], '0.000826_103.305785_1.026316_426.883410_0.000000_0.100000': [0, 438.2613772094479, 0, 0, 0, 0, 0, 0], '0.000826_103.305785_1.026316_388.075827_0.000000_0.100000': [0, 383.4307589273315, 0, 0, 0, 0, 0, 0], '0.000826_103.305785_1.026316_352.796206_0.000000_0.100000': [0, 0, 0, -92.01192874564859, 0, 0, 0, 0], '0.000826_103.305785_1.128948_352.796206_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -317.5185061521752], '0.000826_77.615165_1.652893_426.883410_0.000000_0.100000': [0, 0, 0, 3184.4459134379495, 0, 0, 0, 0], '0.001210_85.376682_2.000000_568.181818_0.000000_0.100000': [4314.368070465193, -262.20718020760944, -3.0930794749604504, -439.0488320531746, 0, 0, 0, 0], '0.000751_125.000000_1.652893_388.075827_0.000000_0.100000': [314.0594477206849, 0, 0, -263.4517489452565, 0, 0, 0, 0], '0.001331_85.376682_1.652893_756.250000_0.000000_0.100000': [0, 0, 0, 0, 0, 130.4627460003294, 0, 0], '0.001331_85.376682_1.502630_756.250000_0.000000_0.100000': [0, 511.78041935964177, 0, 0, 0, 0, 0, 0], '0.001331_77.615165_1.502630_687.500000_0.000000_0.100000': [0, 516.494801300928, 0, 0, 0, 3164.6925799506234, 0, 0], '0.001331_77.615165_1.502630_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -516.494801300928], '0.001331_77.615165_1.366027_687.500000_0.000000_0.100000': [0, 0, 17.156535370176336, 0, 0, 0, 0, -2459.384074108251], '0.001331_77.615165_1.366027_756.250000_0.000000_0.100000': [0, 0, -6.562871782670664, 0, 0, 0, 0, 0], '0.001464_77.615165_1.366027_756.250000_0.000000_0.100000': [0, 0, 0, -960.340627037909, 0, 0, 0, 0], '0.001464_77.615165_1.502630_756.250000_0.000000_0.100000': [0, 0, 0, 0, 5.758900081471984, 0, 0, 0], '0.001331_77.615165_1.502630_756.250000_0.000000_0.100000': [0, 3132.8293473973017, 0, 0, 0, 0, 0, 0], '0.001464_77.615165_1.366027_687.500000_0.000000_0.100000': [0, 0, -109.59943092712346, 0, 0, 0, 0, 0], '0.001611_77.615165_1.366027_687.500000_0.000000_0.100000': [201.14421128687454, 0, 0, 0, 0, 0, 0, 0], '0.001611_70.559241_1.366027_687.500000_0.000000_0.100000': [0, -35.87560972167785, 0, 0, 0, 0, 0, 0], '0.001611_70.559241_1.366027_625.000000_0.000000_0.100000': [2812.083259282359, 0, 0, 0, 0, 0, 0, 0], '0.001611_64.144765_1.366027_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000826_64.144765_2.000000_388.075827_0.000000_0.100000': [0, 0, 0.7941540325840356, 0, 0, 0, 0, 0], '0.000909_64.144765_2.000000_388.075827_0.000000_0.100000': [-0.015808422270015398, -0.977181677889439, 0, 0, 0, -0.05091519942363903, 0, 0], '0.000909_64.144765_1.818182_388.075827_0.000000_0.100000': [0, 0, 0, 0.05091519942363903, 0, 0, 0, 0], '0.000909_58.313423_2.000000_388.075827_0.000000_0.100000': [0, 0, 0, -0.09035288050417876, 0, 0, 0, 0], '0.000909_58.313423_2.200000_388.075827_0.000000_0.100000': [0, 0, 0, 0.08908563659038009, -0.6927616781401548, 0, 0, 0], '0.000909_58.313423_2.420000_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, -0.08908563659038009, 0, 0], '0.000751_85.376682_1.652893_568.181818_0.000000_0.100000': [0, 0, 0, 5079.9490043109345, 0, 0, 0, 0], '0.000751_77.615165_1.652893_568.181818_0.000000_0.100000': [3878.0050192783397, 0, 0, 0, 0, 0, 0, 0], '0.000751_64.144765_1.652893_568.181818_0.000000_0.100000': [0, 0.025543149243208062, 0, 0, 0, 0, 0, 0], '0.000751_64.144765_1.652893_516.528926_0.000000_0.100000': [0, 0, 0, 0.026127507846877052, 0, 0, 0, 0], '0.000751_64.144765_1.818182_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0.7196961352672361, 0, 0, 0], '0.000683_64.144765_1.818182_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -0.7181166658349867], '0.001331_93.914350_1.818182_568.181818_0.000000_0.100000': [4168.133717911034, 0, 0, 0, 0, 0, 0, 0], '0.001331_85.376682_1.818182_568.181818_0.000000_0.100000': [-2830.308785026353, -2339.1572154895903, -288.6608769464797, -3863.4715055641523, -2557.1871959309597, 0, 0, 0], '0.001464_85.376682_1.818182_568.181818_0.000000_0.100000': [-2864.1025262119356, -2370.995587421464, -2807.7516115615226, -3896.862851779094, -22.752297095827263, -2587.3600552338876, -4199.691241808417, 0], '0.001464_85.376682_1.652893_568.181818_0.000000_0.100000': [0, -745.072824807852, 0, 0, 0, 0, 0, 0], '0.001464_85.376682_1.652893_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 250.45998731526188, 0, 0], '0.001464_77.615165_1.502630_469.571751_0.000000_0.100000': [0, 0, 0, -1289.3845453259198, 0, 0, 0, 0], '0.001464_77.615165_1.652893_469.571751_0.000000_0.100000': [0, 0, 349.1019004316171, 0, 0, 0, 0, 0], '0.001611_70.559241_1.652893_426.883410_0.000000_0.100000': [0, 0, 0, 841.4673812045585, 0, 0, 0, 0], '0.001611_70.559241_1.818182_426.883410_0.000000_0.100000': [0, 0, 0, 1730.2453303018285, 0, 0, 0, 0], '0.001611_70.559241_2.000000_426.883410_0.000000_0.100000': [0, 123.58722713273566, 0, 0, 0, 0, 0, 0], '0.001611_70.559241_2.000000_388.075827_0.000000_0.100000': [33.42175053706045, 0, 0, 0, 0, 0, 0, 0], '0.001611_64.144765_2.000000_388.075827_0.000000_0.100000': [-0.017674327896656905, -0.6380095344889014, 0, 0, 0, 0, 0, 0], '0.001611_64.144765_2.000000_352.796206_0.000000_0.100000': [0, 0, 0.6083781681443953, 0, 0, 0, 0, 0], '0.001772_64.144765_2.000000_352.796206_0.000000_0.100000': [0, 0, 0, 0, -0.06083781681443946, 0, 0, 0], '0.000751_77.615165_1.818182_516.528926_0.000000_0.100000': [2229.9072378583164, 0, 0, 0, 0, 0, 0, 0], '0.000683_70.559241_1.818182_516.528926_0.000000_0.100000': [0, 0, 0, 2365.6090920689903, 0, 0, 0, 0], '0.000683_70.559241_2.000000_516.528926_0.000000_0.100000': [0, 39.64860220748621, 0, 0, 0, 0, 0, 0], '0.000683_70.559241_2.000000_469.571751_0.000000_0.100000': [0, 0, -5.450372066801651, 0, 0, 0, 0, 0], '0.000751_70.559241_2.000000_469.571751_0.000000_0.100000': [42.46990923138706, 0, -2202.559962161687, 0, 0, 0, 0, 0], '0.000826_70.559241_2.000000_469.571751_0.000000_0.100000': [0, 0, 0, 0, 2202.559962161687, 0, 0, 0], '0.000751_64.144765_2.000000_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, -0.06099347674007732, 0, 0], '0.000751_64.144765_1.818182_469.571751_0.000000_0.100000': [0, 0, 0.7772223748140528, 0, 0, 0, 0, 0], '0.001000_85.376682_1.128948_516.528926_0.000000_0.100000': [1537.1215774986747, 0, 0.63159286808459, 0, 0, 0, 0, 0], '0.001100_85.376682_1.128948_516.528926_0.000000_0.100000': [1534.2469582915498, 0, 0, 0, 0, 0, 0, 0], '0.001100_77.615165_1.128948_516.528926_0.000000_0.100000': [2512.863540370292, 0, 0, 0, 0, 0, 0, 0], '0.001100_70.559241_1.128948_516.528926_0.000000_0.100000': [42.046160251383256, 0, 0, 0, 0, 0, 0, 0], '0.001100_64.144765_1.128948_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, -0.01657922968361163, 0, 0], '0.001100_64.144765_1.026316_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -122.59563589892139, 0], '0.001100_70.559241_1.026316_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001000_85.376682_1.818182_426.883410_0.000000_0.100000': [267.39465236418687, 0, 0, 0, 0, -11.89609469135803, 0, 0], '0.001100_77.615165_1.652893_469.571751_0.000000_0.100000': [0, 0, 1198.3688681682672, 0, 0, 0, 0, 0], '0.000751_137.500000_1.652893_469.571751_0.000000_0.100000': [739.7042248890355, 0, 0, 0, 0, 0, 0, 0], '0.000683_113.636364_1.652893_469.571751_0.000000_0.100000': [267.630909792577, 0, 0, 0, 0, 0, 0, 0], '0.000683_103.305785_1.652893_469.571751_0.000000_0.100000': [432.56187084210734, 0, 0, 0, 0, 0, 0, 0], '0.000564_70.559241_1.652893_352.796206_0.000000_0.100000': [1265.0140983756867, 0, 0, 0, 0, 0, 0, 0], '0.000826_64.144765_1.652893_516.528926_0.000000_0.100000': [-0.8406500609068417, -0.0058832095752904935, -0.7431421582375378, 0, 0, 0, -3278.652468020506, 0], '0.000826_58.313423_1.652893_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001100_85.376682_2.000000_831.875000_0.000000_0.100000': [0, 0, 0, 0, 0.13348298332884953, 0, 0, 0], '0.001000_85.376682_2.000000_831.875000_0.000000_0.100000': [0, 0, 0, 0, 0, -1419.5143617571462, 0, 0], '0.001000_85.376682_1.818182_831.875000_0.000000_0.100000': [0, 1000.6840884046264, 0, 0, 0, 0, 0, 0], '0.001000_64.144765_1.652893_625.000000_0.000000_0.100000': [-0.7114291557728711, 0, -0.752882919858098, 0, 0, 0, 0, 0], '0.001100_64.144765_1.652893_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, -0.05362497935552937, 0, 0], '0.000909_64.144765_1.818182_756.250000_0.000000_0.100000': [-0.03153863708096196, -0.0010920568484834804, 0, 0, 0, 0.012638127154107224, 0, 0], '0.001000_85.376682_1.128948_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -465.6877252012174], '0.001000_85.376682_1.128948_687.500000_0.000000_0.100000': [0, 0, -0.9036421428641915, 0, 0, 0, 0, 0], '0.001100_85.376682_1.128948_687.500000_0.000000_0.100000': [0, 0, 0, 2195.216805850653, 0, 0, 0, 0], '0.001611_93.914350_2.000000_687.500000_0.000000_0.100000': [0, 0, 0, 0, 9.006481914628058, 0, 0, 0], '0.001464_93.914350_2.200000_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 1247.3578219323506, 0, 0], '0.001464_85.376682_2.000000_687.500000_0.000000_0.100000': [0, 693.7975302749776, 0, 0, 0, 0, 0, 0], '0.001464_85.376682_2.000000_625.000000_0.000000_0.100000': [1225.2747737485681, 0, 0, 0, 0, 0, -320.597148407299, 0], '0.001464_93.914350_2.000000_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -344.3502158037272, 0], '0.001464_103.305785_2.000000_625.000000_0.000000_0.100000': [0, 0, 0, -1251.5576077006128, 0, 0, 0, 0], '0.001464_103.305785_2.200000_625.000000_0.000000_0.100000': [0, 556.8673678809308, 0, 0, 0, 0, 0, 0], '0.001464_103.305785_2.200000_568.181818_0.000000_0.100000': [0, 0, 0, 0, 8.021039268987497, 0, 0, 0], '0.001331_93.914350_2.200000_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -1670.8780841631435, 0], '0.001331_93.914350_2.200000_516.528926_0.000000_0.100000': [0, 562.3979158110354, 0, 0, 0, 0, 0, 0], '0.001331_93.914350_2.200000_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 2122.995348700859, 0, 0], '0.000751_93.914350_1.366027_388.075827_0.000000_0.100000': [0, 0, 653.9831225311179, 0, 0, 0, 0, 0], '0.000826_77.615165_1.366027_388.075827_0.000000_0.100000': [1841.354821654684, 1711.282517310704, 0, 0, 0, 0, 0, -345.58931371719916], '0.000826_70.559241_1.366027_388.075827_0.000000_0.100000': [0, 0, 0, 0, 20.362358288996006, 0, 0, -126.86117154760555], '0.000751_70.559241_1.366027_388.075827_0.000000_0.100000': [26.648621720476694, 0, 0, 0, -44.11665556053483, 0, 0, 0], '0.000683_70.559241_1.366027_388.075827_0.000000_0.100000': [0, -805.8481017557554, 0, 0, 0, 0, 0, 0], '0.000683_70.559241_1.366027_352.796206_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000826_77.615165_1.366027_426.883410_0.000000_0.100000': [1010.1656618854106, 345.58931371719916, 0, -161.06924099809424, 0, 0, 0, 0], '0.001000_125.000000_2.420000_426.883410_0.000000_0.100000': [310.4578598469416, 0, 0, 0, 0, 0, 0, 0], '0.000826_93.914350_2.420000_388.075827_0.000000_0.100000': [0, 0, 0, 0, 2.485701373406373, 0, 0, 0], '0.000751_93.914350_2.420000_388.075827_0.000000_0.100000': [296.52751801023663, 0, 0, 0, 0, 0, 0, -493.28640950849604], '0.000751_93.914350_2.420000_426.883410_0.000000_0.100000': [0, 0, 441.75098564456266, 0, 0, 0, 0, 0], '0.000751_85.376682_2.420000_388.075827_0.000000_0.100000': [0, 0, 0, 0, 2.9050848381813923, 0, 0, 0], '0.000683_85.376682_2.420000_388.075827_0.000000_0.100000': [0, 0, 0, 0, 2.6503468015931504, 0, 0, 0], '0.001100_85.376682_1.366027_426.883410_0.000000_0.100000': [0, 0, 0, 38.12929464085228, 0, 0, 0, 0], '0.001210_64.144765_1.366027_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 0.034436330465018726, 0, 0], '0.001210_64.144765_1.241843_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0.732134646055738, 0, 0, 0], '0.001100_64.144765_1.241843_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000909_103.305785_1.241843_469.571751_0.000000_0.100000': [0, 0, 0, 0, 242.3746384700557, 0, 0, -36.827952070649644], '0.000826_93.914350_1.241843_469.571751_0.000000_0.100000': [0, 0, 0, 762.9300957416592, 2.4455184961875243, 0, 0, 0], '0.000751_93.914350_1.241843_469.571751_0.000000_0.100000': [0, 0, 0, 0, 2.1949269509971234, 0, 0, 0], '0.000683_93.914350_1.241843_469.571751_0.000000_0.100000': [0, 0, 0, 0, 1.1362855136776306, 0, 0, 0], '0.000621_93.914350_1.241843_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0.9285177040447706, 0, -240.2335969517696, 0], '0.000621_103.305785_1.241843_469.571751_0.000000_0.100000': [0, 0, 0, 0, 1.589474692641752, 0, 0, 0], '0.000564_103.305785_1.241843_469.571751_0.000000_0.100000': [239.7171376625037, 0, -0.14449769933106824, -86.93976813853423, 0, 0, 0, 0], '0.000564_103.305785_1.366027_469.571751_0.000000_0.100000': [0, 0, -2.720818183202664, 0, 0, 0, 0, 0], '0.000621_103.305785_1.366027_469.571751_0.000000_0.100000': [367.9518290827509, 0, 0, 0, 0, 0, 0, 0], '0.000621_93.914350_1.366027_469.571751_0.000000_0.100000': [731.6567204633835, 0, 0, 0, 0, 0, 0, 0], '0.000621_85.376682_1.366027_469.571751_0.000000_0.100000': [0, 0, -0.7454400432451621, 0, 0, 0, 0, 0], '0.000683_85.376682_1.366027_469.571751_0.000000_0.100000': [0, 0, 0, -1294.5673493981421, 0, 0, 0, 0], '0.000683_103.305785_1.652893_426.883410_0.000000_0.100000': [0, 598.2938238979125, 0, 0, 0, 0, 0, 0], '0.000751_70.559241_2.000000_352.796206_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -272.09667380922747], '0.000683_70.559241_2.000000_388.075827_0.000000_0.100000': [0, 0, 0, -2388.506468444871, 0, 0, 0, 0], '0.000683_70.559241_2.200000_388.075827_0.000000_0.100000': [0, 0, -4.1878838463708234, 0, 0, 0, 0, 0], '0.000751_70.559241_2.200000_388.075827_0.000000_0.100000': [4137.049772530972, 0, -4.608430339756615, 0, 0, 0, 0, 0], '0.000826_70.559241_2.200000_388.075827_0.000000_0.100000': [0, 0, -5.071735944398552, 0, 0, 0, 0, 0], '0.000909_70.559241_2.200000_388.075827_0.000000_0.100000': [4147.45212078587, 0, 0, 0, 0, 0, 0, 0], '0.000909_64.144765_2.200000_388.075827_0.000000_0.100000': [0, 0, -0.11587677407148245, 0, 0, 0, 0, 0], '0.001100_58.313423_1.652893_426.883410_0.000000_0.100000': [0.003672760401445885, 0, 0, 0, 0, 0, -0.7244279800332606, 0], '0.001100_64.144765_1.652893_426.883410_0.000000_0.100000': [0, 0, 0, -0.06190286311630189, 0, 0, 0, 0], '0.001100_64.144765_1.818182_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, 0.06190286311630189, 0, 0], '0.000909_93.914350_1.502630_568.181818_0.000000_0.100000': [2993.1601303455654, 0, 0, 0, 0, 0, 0, 0], '0.000826_85.376682_1.366027_568.181818_0.000000_0.100000': [0, 0, 0, 0, 278.0613438578392, 0, 0, 0], '0.000751_70.559241_1.241843_516.528926_0.000000_0.100000': [0, -1396.0550825585356, 0, 0, 0, 0, -3595.4849413767984, 0], '0.000751_70.559241_1.241843_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 1396.0550825585356], '0.000826_64.144765_1.366027_516.528926_0.000000_0.100000': [0, 0, -0.7252639750831122, 0, 0, 0, 0, 0], '0.000909_58.313423_1.366027_516.528926_0.000000_0.100000': [0, -0.15584706784536592, 0, 0, 0, 0, 0, 0], '0.000909_58.313423_1.366027_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001000_64.144765_2.200000_568.181818_0.000000_0.100000': [0.761961973223697, 0, 0, 0, 0, 0, 0, 0], '0.001000_58.313423_2.200000_568.181818_0.000000_0.100000': [0, -0.7516024177840336, 0, 0, 0, 0, 0, 0], '0.001000_58.313423_2.200000_516.528926_0.000000_0.100000': [0, 0.027485348463960693, 0, 0, 0, 0, 0, 0], '0.000683_77.615165_1.652893_388.075827_0.000000_0.100000': [0, 0, 0, 61.77821499966967, 0, 0, 0, 0], '0.000683_70.559241_1.818182_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0.4608153668194973, 0, 0, 0], '0.000621_70.559241_1.818182_388.075827_0.000000_0.100000': [0, 0, 0, 0, -0.575963890514231, 0, 0, 0], '0.000564_70.559241_1.818182_388.075827_0.000000_0.100000': [0, 0, 0, 0, -2.92349620476316, 0, 0, 0], '0.000513_70.559241_1.818182_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, -2286.2305392467665, 0, 0], '0.000513_70.559241_1.652893_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001000_113.636364_1.818182_320.723824_0.000000_0.100000': [1220.071151615916, 0, 0, 0, 0, 0, 0, 0], '0.001000_103.305785_1.818182_320.723824_0.000000_0.100000': [0, 1326.2085242477142, 0, -318.97808232819983, 0, 0, 0, 0], '0.001000_103.305785_1.818182_291.567113_0.000000_0.100000': [682.7328799505492, 0, 0, 0, 0, 0, 0, -32.84053472936583], '0.001000_85.376682_1.818182_291.567113_0.000000_0.100000': [0, 0, 403.781465634383, 0, 0, 311.3434549805054, 0, 0], '0.001100_85.376682_1.818182_291.567113_0.000000_0.100000': [0, 1128.2335426998861, 0, 0, 0, 0, 0, 0], '0.001100_85.376682_1.818182_265.061011_0.000000_0.100000': [1672.1347745525668, 0, -3.737220274410538, 130.9592205985352, 0, 0, 0, 0], '0.001210_93.914350_1.818182_265.061011_0.000000_0.100000': [0, 287.1281602165236, 0, 0, 0, 0, 0, 0], '0.001210_93.914350_1.818182_240.964556_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001331_77.615165_1.818182_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 268.35694014016, 0, 0], '0.001331_77.615165_1.652893_568.181818_0.000000_0.100000': [3802.1418326929297, 0, 0, 0, 0, 0, 0, 0], '0.001331_70.559241_1.652893_568.181818_0.000000_0.100000': [288.80403849952796, 0, 0, 0, 0, 0, 0, 0], '0.001100_64.144765_1.652893_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0.7856208648061056, 0, 0, 0], '0.000909_53.012202_1.366027_469.571751_0.000000_0.100000': [0, 0, 0, 0.7815204006189788, 0, 0, 0, 0], '0.000909_53.012202_1.502630_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001210_77.615165_2.000000_568.181818_0.000000_0.100000': [6044.87354769548, 0, 0, 0, 0, 0, 0, 0], '0.001210_70.559241_2.000000_568.181818_0.000000_0.100000': [125.43055698574913, 0, 0, -41.55441056985624, 0, 0, -54.458320249508965, 0], '0.001210_70.559241_2.200000_568.181818_0.000000_0.100000': [166.9446340553717, 0, 0, 0, 0, 0, -5255.423774640915, 0], '0.001210_64.144765_2.200000_568.181818_0.000000_0.100000': [0, 0, -0.039082068878633434, 0, 0, 0, 0, 0], '0.001331_64.144765_2.200000_568.181818_0.000000_0.100000': [0, 0, 0.8040649342362258, 0, 0, 0, 0, 0], '0.001464_64.144765_2.200000_568.181818_0.000000_0.100000': [0, -0.7509235392943537, 0, 0, 0, 0, 0, 0], '0.001464_64.144765_2.200000_516.528926_0.000000_0.100000': [0.8379064711664023, 0, 0, 0, 0, 0, 0, 0], '0.001464_58.313423_2.200000_516.528926_0.000000_0.100000': [-0.761030701674963, 0, -0.012959340939528375, 0, 0, 0, 0, 0], '0.001611_58.313423_2.200000_516.528926_0.000000_0.100000': [0.0432032009493696, 0, 0, 0, 0, -0.009123061272498489, 0, 0], '0.001611_58.313423_2.000000_516.528926_0.000000_0.100000': [0, 0, 0.01179013242599467, 0, 0, 0, 0, 0], '0.001772_58.313423_2.000000_516.528926_0.000000_0.100000': [0, 0, 0, -0.7505902825691237, 0, 0, 0, -0.6999766282268184], '0.001772_58.313423_2.200000_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0.7479232114156276, 0, 0, 0], '0.001611_53.012202_2.200000_516.528926_0.000000_0.100000': [0, 0, -0.0004895625312883568, 0, 0, 0, 0, 0], '0.001772_53.012202_2.200000_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, -0.7720035280570073, 0, 0], '0.001772_53.012202_2.000000_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0.7319569607924222, 0], '0.001772_58.313423_2.000000_568.181818_0.000000_0.100000': [0, 0, 0, -0.09503337033520438, 0, 0, 0, 0], '0.001772_58.313423_2.200000_568.181818_0.000000_0.100000': [0.07867235800147228, 0, 0, 0, 0, 0, 0, 0], '0.001772_53.012202_2.200000_568.181818_0.000000_0.100000': [-0.03346950643408491, 0, 0, 0, 0, 0, 0, 0], '0.001772_48.192911_2.200000_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001100_53.012202_1.366027_352.796206_0.000000_0.100000': [0, 0, 0, 0, -0.7876929330698659, 0, 0, 0], '0.001000_53.012202_1.366027_352.796206_0.000000_0.100000': [0, 0, 0, 0.794434563577829, 0, 0, 0, 0.7346531851472946], '0.001000_53.012202_1.366027_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0.028493397814577295, 0], '0.001000_58.313423_1.366027_388.075827_0.000000_0.100000': [0, 0.008991514168178583, 0, 0, 0, 0, 0, 0], '0.001000_58.313423_1.366027_352.796206_0.000000_0.100000': [-0.7721380971298458, 0, 0, 0, 0, 0, 0, 0], '0.001331_85.376682_2.000000_568.181818_0.000000_0.100000': [-537.3204308745497, -265.9379378141169, 0, 0, -476.77277329712706, 0, 0, 0], '0.000683_103.305785_2.000000_352.796206_0.000000_0.100000': [0, 431.54020727339383, 0, 0, 0, 0, -436.6547563788926, 0], '0.000683_103.305785_2.000000_320.723824_0.000000_0.100000': [376.6861787970188, 0, -3.166537594912461, 0, 0, 0, 0, 0], '0.000751_103.305785_2.000000_320.723824_0.000000_0.100000': [0, 354.35687822671906, 0, 0, 0, 0, 0, 0], '0.000751_103.305785_2.000000_291.567113_0.000000_0.100000': [0, 0, -3.1791319950801156, 0, 0, 0, 0, 0], '0.000826_103.305785_2.000000_291.567113_0.000000_0.100000': [361.53684875974506, 0, -3.5083787973840117, 0, 0, 0, 0, 0], '0.000909_103.305785_2.000000_291.567113_0.000000_0.100000': [0, 0, 0, 0, 3.5083787973840117, 0, 0, 0], '0.000826_93.914350_2.000000_291.567113_0.000000_0.100000': [0, 0, -2.91890072637716, 0, 0, 0, 0, 0], '0.000909_93.914350_2.000000_291.567113_0.000000_0.100000': [0, 0, 0, 0, 0, 214.56429045964333, 0, 0], '0.000909_93.914350_1.818182_291.567113_0.000000_0.100000': [301.7715952831725, 0, 0, 0, 0, 0, 0, 0], '0.001100_103.305785_1.502630_320.723824_0.000000_0.100000': [0, 0, 0, 0, 0, 472.12181552346823, 0, 0], '0.001000_103.305785_1.366027_320.723824_0.000000_0.100000': [1468.1317138853979, 0, 0, 0, 0, -36.28286106638016, 0, 0], '0.001000_103.305785_1.241843_320.723824_0.000000_0.100000': [0, 0, 0, 0, 0, -159.0484056598807, 0, 0], '0.001000_103.305785_1.128948_320.723824_0.000000_0.100000': [0, 0, 0, 0, 0, 164.10692326421812, 0, 0], '0.001000_103.305785_1.026316_320.723824_0.000000_0.100000': [325.2199067234061, 0, 0, 0, 0, 0, 0, 0], '0.001000_93.914350_1.026316_320.723824_0.000000_0.100000': [822.4377097346566, 0, 0, 0, 0, 0, 0, 0], '0.001000_85.376682_1.026316_320.723824_0.000000_0.100000': [0, 0, 0, 0, 0.2748843837652203, 0, 0, 0], '0.000909_85.376682_1.026316_320.723824_0.000000_0.100000': [0, 0, 0, -883.1672447840017, 0, 0, 0, 0], '0.000909_85.376682_1.128948_320.723824_0.000000_0.100000': [0, 0, 0, 39.166417701982574, 0, 0, 0, 0], '0.000909_85.376682_1.241843_320.723824_0.000000_0.100000': [82.84655163370762, 0, 0, 0, 0, 0, 0, 0], '0.000909_77.615165_1.241843_320.723824_0.000000_0.100000': [0, 0, 0, 621.4351299176806, 0, 0, 0, 0], '0.000909_77.615165_1.366027_320.723824_0.000000_0.100000': [0, 1271.3370972367616, 0, 0, 0, 0, 0, 0], '0.000909_77.615165_1.366027_291.567113_0.000000_0.100000': [0, 1301.794630857658, 0, 0, 0, 0, 0, 0], '0.000909_77.615165_1.366027_265.061011_0.000000_0.100000': [1094.0709192837303, 0, 0, 0, 0, 0, 0, 0], '0.000909_70.559241_1.366027_265.061011_0.000000_0.100000': [31.679385542225763, 0, 0, 0, 0, 0, 0, 0], '0.001100_103.305785_1.128948_388.075827_0.000000_0.100000': [130.00729826323777, 0, 0, 0, 0, 0, 0, 0], '0.001100_93.914350_1.128948_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 169.2209900650515, 0, 0], '0.001100_93.914350_1.026316_388.075827_0.000000_0.100000': [0, 404.1908528852459, -1.1465185396639754, 0, 0, 0, 0, 0], '0.001100_93.914350_1.026316_352.796206_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -299.6991538459769, -404.1908528852459], '0.001100_103.305785_1.026316_352.796206_0.000000_0.100000': [299.6991538459769, 0, 0, 0, 0, 0, 0, -384.7343352273405], '0.001100_103.305785_1.026316_388.075827_0.000000_0.100000': [0, 384.7343352273405, 0, 0, 0, 0, 0, 0], '0.001210_93.914350_1.026316_388.075827_0.000000_0.100000': [0, 0, -1.300751145018785, 0, 0, 0, 0, 0], '0.001331_93.914350_1.026316_388.075827_0.000000_0.100000': [0, 401.2042408540252, 0, 0, 0, 0, 0, 0], '0.001331_93.914350_1.026316_352.796206_0.000000_0.100000': [0, 388.1064874053324, 0, 0, 0, 0, 0, 0], '0.001331_93.914350_1.026316_320.723824_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000909_70.559241_2.000000_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, -1688.6325135818402, 0, 0], '0.000909_70.559241_1.818182_388.075827_0.000000_0.100000': [0, 1.3944847727378373, 0, 0, 0, 0, 0, 0], '0.000909_70.559241_1.818182_352.796206_0.000000_0.100000': [97.1092975038509, 0, 0, 0, 0, 0, 0, 0], '0.000909_64.144765_1.818182_352.796206_0.000000_0.100000': [0, 0, 0, -0.19110505311673479, 0, 0, 0, 0], '0.000909_64.144765_2.000000_352.796206_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001464_77.615165_1.818182_568.181818_0.000000_0.100000': [0, 0, -5.900481497587134, 0, 0, 0, 0, 0], '0.001611_77.615165_1.818182_568.181818_0.000000_0.100000': [0, 0, 0, -1222.9175322997942, 0, 0, 0, 0], '0.001611_77.615165_2.000000_568.181818_0.000000_0.100000': [5470.315799652497, 0, 0, 0, 0, 0, 0, 0], '0.001611_70.559241_2.000000_568.181818_0.000000_0.100000': [123.72481665371117, -4600.547470443095, 0, 0, 0, 0, 0, 0], '0.001611_70.559241_2.000000_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 4600.547470443095], '0.001611_64.144765_2.000000_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 0.0565128400525964, 0, 0], '0.001611_64.144765_1.818182_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000564_93.914350_1.241843_469.571751_0.000000_0.100000': [0, 0, 0, 41.01931211364454, 0, 0, -239.7171376625037, 0], '0.000564_93.914350_1.366027_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001611_85.376682_1.818182_568.181818_0.000000_0.100000': [0, 0, 0, -741.5034683504831, 0, 0, 0, 0], '0.001611_85.376682_2.000000_568.181818_0.000000_0.100000': [0, 801.4293611061198, 0, 0, 0, 0, 0, 0], '0.001611_77.615165_1.818182_516.528926_0.000000_0.100000': [0, 0, -5.387906262284105, 0, 0, 0, 0, 0], '0.001772_77.615165_1.818182_516.528926_0.000000_0.100000': [0, 0, 0, 321.68371112098185, 0, 0, 0, 0], '0.001772_77.615165_2.000000_516.528926_0.000000_0.100000': [-347.6287126955811, 0, 0, 0, 0, 0, 0, 0], '0.001772_70.559241_2.000000_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 4694.141498056146, 0, 0], '0.001772_70.559241_1.818182_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, -3229.1379695897226, 0, 0], '0.001772_70.559241_1.652893_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 3240.4049975267026], '0.001772_70.559241_1.652893_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -4077.886297642954, 0], '0.001772_77.615165_1.652893_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000826_70.559241_2.000000_291.567113_0.000000_0.100000': [194.76781088575265, 0, 100.70283210535725, 0, 0, 0, -19.034964453254045, 0], '0.000826_64.144765_2.000000_291.567113_0.000000_0.100000': [-0.006660594894697169, 0.028414185822055175, -0.002981040229647078, -0.004472267316441503, 0, 0, 0, -0.09403368312790757], '0.000909_64.144765_2.000000_291.567113_0.000000_0.100000': [0, 0, 0, 0, 0.01654477327231075, 0, 0, 0], '0.000826_58.313423_2.000000_291.567113_0.000000_0.100000': [0, 0, 0, 0, 0.02346226839317751, 0, 0, 0], '0.000751_58.313423_2.000000_291.567113_0.000000_0.100000': [0, 0, 0, -0.8335435150688227, 0, 0, 0, 0], '0.000751_58.313423_2.200000_291.567113_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0.8079298416869961, 0], '0.000751_64.144765_2.200000_291.567113_0.000000_0.100000': [0, 0, 0, 0, -0.7736267611129961, 0, 0, 0], '0.000683_64.144765_2.200000_291.567113_0.000000_0.100000': [0, 0, 0, 0, 0, 0.7818954697206436, 0, 0], '0.000683_64.144765_2.000000_291.567113_0.000000_0.100000': [-0.7748544849946711, 0, 0, 0, 0, 0, 0, 0], '0.000683_58.313423_2.000000_291.567113_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001000_103.305785_1.366027_352.796206_0.000000_0.100000': [0, 1116.8583649049576, 0, 0, 0, 0, 0, 0], '0.000909_64.144765_1.366027_265.061011_0.000000_0.100000': [-0.7698759333524322, 0, 0, 0, -0.012124711347337325, 0, 0, 0], '0.000826_64.144765_1.366027_265.061011_0.000000_0.100000': [0, 0.03796691794818798, 0, 0, 0, 0, 0, 0], '0.000826_64.144765_1.366027_240.964556_0.000000_0.100000': [0, 0, 0.010178293129388293, 0, 0, 0, 0, 0], '0.000909_64.144765_1.366027_240.964556_0.000000_0.100000': [0, 0, -0.016358186273373578, 0, 0, 0, 0, 0], '0.001000_64.144765_1.366027_240.964556_0.000000_0.100000': [0, 0, 0.0021899044937754297, 0, 0, 0, 0, 0], '0.001100_64.144765_1.366027_240.964556_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001000_85.376682_1.241843_352.796206_0.000000_0.100000': [0, 0, 0, 811.1148036527151, 0, 0, 0, 0], '0.000621_125.000000_1.818182_756.250000_0.000000_0.100000': [0, 0, 0, -1970.974381416202, 0, 0, 0, 0], '0.000621_113.636364_2.000000_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 1507.511125540547, 0, 0], '0.001000_93.914350_2.000000_352.796206_0.000000_0.100000': [953.9423280299383, 0, 0, 0, 0, 0, 0, 0], '0.001000_85.376682_2.000000_352.796206_0.000000_0.100000': [1155.3364754581119, 0, 0, 0, 0, 0, 0, 0], '0.001000_77.615165_2.000000_352.796206_0.000000_0.100000': [0, -702.0679357631664, 0, 0, 0, 0, 0, 0], '0.001000_77.615165_2.000000_320.723824_0.000000_0.100000': [2514.092212351894, 0, 0, -573.1864562165766, 0, 0, 0, 0], '0.001000_77.615165_2.200000_320.723824_0.000000_0.100000': [0, 0, 0, 0, -692.6580892643523, 0, 0, 0], '0.000909_77.615165_2.200000_320.723824_0.000000_0.100000': [1922.29410058037, 1122.1167302037595, 0, 0, 0, 0, 0, 0], '0.000909_77.615165_2.200000_291.567113_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -1122.1167302037595], '0.000909_70.559241_2.200000_320.723824_0.000000_0.100000': [0, -575.1973643850907, 0, 0, 0, 0, 0, 0], '0.000909_70.559241_2.200000_291.567113_0.000000_0.100000': [0, 0, 0, -1614.7756652811474, 0, 0, 0, 0], '0.000909_70.559241_2.420000_291.567113_0.000000_0.100000': [0, 0, 0, 0, 4.387813959041523, 0, 0, 0], '0.000826_70.559241_2.420000_291.567113_0.000000_0.100000': [0, 0, -0.3988921780946839, 0, 0, 0, 0, 0], '0.001100_85.376682_2.000000_291.567113_0.000000_0.100000': [0, 0, 0, 0, 566.7031234049193, 0, 0, 0], '0.001000_85.376682_2.000000_291.567113_0.000000_0.100000': [2234.944332707811, 0, -0.2622014835665411, -557.8613569343721, 4.0037070743542245, 0, 0, 0], '0.001000_77.615165_2.000000_291.567113_0.000000_0.100000': [2300.7283987175606, 285.2533472258654, -2.2917561020517496, 0, 0, 0, 1967.3148660251238, 0], '0.001100_77.615165_2.000000_291.567113_0.000000_0.100000': [0, 0, 0, 0, 2.5209317122569246, 0, 0, 0], '0.001000_70.559241_2.000000_291.567113_0.000000_0.100000': [127.22092731596727, -10.077460735998875, 0.5042641701596949, 0, 0, -34.557844097519364, 0, 0], '0.001000_70.559241_1.818182_291.567113_0.000000_0.100000': [0, 0, -15.120354382041342, 0, 0, 0, 0, 0], '0.001100_70.559241_1.818182_291.567113_0.000000_0.100000': [165.4012868840607, 0, 0, 0, 0, 0, 0, 0], '0.001100_64.144765_2.000000_388.075827_0.000000_0.100000': [0, 0, 0.021857462466787457, -0.027445032058021526, 0, 0, 0, 0], '0.001100_64.144765_2.200000_352.796206_0.000000_0.100000': [0.004868054008488798, 0, 0, 0, 0, 0, 0, 0], '0.001100_58.313423_2.200000_352.796206_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001100_103.305785_2.000000_388.075827_0.000000_0.100000': [0, 0, 0, 0, 282.68802080013825, 0, 0, 0], '0.000751_64.144765_2.200000_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 0.72439958421337, 0, 0], '0.000751_64.144765_2.000000_388.075827_0.000000_0.100000': [0, 0, -0.07007425993549307, 0, 0, 0, 0, 0], '0.000909_70.559241_1.502630_291.567113_0.000000_0.100000': [291.572240135737, 0, -44.197928243276785, 0, 0, 0, 0, 0], '0.001000_70.559241_1.502630_291.567113_0.000000_0.100000': [339.49484464060447, 0, 0, 0, 0, 0, -979.9264208146764, 0], '0.001000_77.615165_1.502630_291.567113_0.000000_0.100000': [1356.239421617011, 0, 0, 0, 0, 0, 0, 0], '0.001000_64.144765_1.502630_291.567113_0.000000_0.100000': [-0.6909854040859955, -0.011510404425052911, 0.034971263826891995, 0, -0.007122163582618649, 0, 0, 0], '0.000909_64.144765_1.502630_291.567113_0.000000_0.100000': [-0.7898606600636641, -0.16338645617150238, -0.01190307317954114, -0.7867533822653741, -0.762266370729094, -0.07997620046711057, -265.06567285067, -0.7868202085180656], '0.000909_58.313423_1.502630_291.567113_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000826_64.144765_2.200000_291.567113_0.000000_0.100000': [0, 0, -0.6363504712432814, 0, 0, 0, 0, 0], '0.000909_64.144765_2.200000_291.567113_0.000000_0.100000': [0.6283932450302245, 0, 0, 0, 0, 0, 0, 0], '0.000909_58.313423_2.200000_291.567113_0.000000_0.100000': [0, 0, 0, 0, 0, -0.7419934758979935, 0, 0], '0.000909_58.313423_2.000000_291.567113_0.000000_0.100000': [0.7555585192872286, 0, 0, 0, 0, 0, 0, 0], '0.000909_53.012202_2.000000_291.567113_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000683_70.559241_1.366027_291.567113_0.000000_0.100000': [0, 0, 0, 0, 0, 87.86737750321504, 0, 0], '0.000683_70.559241_1.241843_291.567113_0.000000_0.100000': [0, 164.17219276587022, 0, 0, 0, 0, 0, 0], '0.000683_70.559241_1.241843_265.061011_0.000000_0.100000': [349.1083287793949, 0, 0, 0, 0, 0, 0, 0], '0.000683_64.144765_1.241843_265.061011_0.000000_0.100000': [0, 0.7035753190305514, 0, 0, 0, 0, 0, 0], '0.000683_64.144765_1.241843_240.964556_0.000000_0.100000': [0, 0, 0, 0.01381802482699186, 0, 0, 0, 0], '0.000683_64.144765_1.366027_240.964556_0.000000_0.100000': [0, 0, 0.0019068518839958416, 0, 0, 0, 0, 0], '0.000751_64.144765_1.366027_240.964556_0.000000_0.100000': [0, 0, 0.0010424767067888751, 0, 0, 0, 0, 0], '0.000826_64.144765_1.818182_352.796206_0.000000_0.100000': [0, 0, 0, 0, -0.011743417989259796, 0, 0, 0], '0.000751_64.144765_1.818182_352.796206_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -0.6675110816016194], '0.001100_77.615165_1.818182_265.061011_0.000000_0.100000': [0, 0, 0, 0, 0, 944.3761504082108, 0, 0], '0.001100_77.615165_1.652893_265.061011_0.000000_0.100000': [0, 0, 0, 0, 0, 1543.5777255640223, 0, 0], '0.001100_77.615165_1.502630_265.061011_0.000000_0.100000': [1326.4586990107784, 0, -2.317726900950447, 0, 0, 0, 0, -28.295335462294315], '0.001210_77.615165_1.502630_265.061011_0.000000_0.100000': [0, 0, 0, 0, 2.317726900950447, 0, 0, 0], '0.001100_70.559241_1.502630_265.061011_0.000000_0.100000': [396.60724239665, 0, 0, -258.343139211729, 0, 0, 0, 99.88615387923146], '0.001100_70.559241_1.652893_265.061011_0.000000_0.100000': [0, 454.5500050279412, 0, 0, 0, 0, 0, 0], '0.001100_70.559241_1.652893_240.964556_0.000000_0.100000': [180.1126132058717, 0, 0, 0, 0, 0, 0, 0], '0.001100_64.144765_1.652893_240.964556_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -16.373873927806518, 0], '0.001210_77.615165_2.000000_388.075827_0.000000_0.100000': [0, 0, 0, 0, 2800.1549246306768, 0, 0, 0], '0.001210_64.144765_2.000000_388.075827_0.000000_0.100000': [-0.03705718754088139, 0, -0.015937036685977547, -0.0022130469884518787, 0, 0, 0, 0], '0.001210_64.144765_2.200000_388.075827_0.000000_0.100000': [-0.8471788011329409, 0, 0, 0, -0.014293978448687097, 0, 0, 0], '0.001000_64.144765_2.000000_426.883410_0.000000_0.100000': [-0.19275417954738713, 0, 0, 0, 0, 0, 0, 0], '0.001000_58.313423_2.000000_426.883410_0.000000_0.100000': [0.7091291269360227, 0, 0, 0, 0, 0, 0, 0], '0.001100_85.376682_1.502630_240.964556_0.000000_0.100000': [0, 0, 0, 0, 319.93024003949085, 0, 0, 0], '0.001000_85.376682_1.502630_240.964556_0.000000_0.100000': [0, 0, 0, 0, 0, 457.737780271329, 0, -287.36472918626043], '0.001000_85.376682_1.366027_240.964556_0.000000_0.100000': [828.463907902368, 0, 0, -198.97095852410834, 0, 0, 0, 0], '0.001000_77.615165_1.366027_240.964556_0.000000_0.100000': [0, 821.0886027360118, 0, 0, 0, 0, 0, 0], '0.001000_77.615165_1.366027_219.058687_0.000000_0.100000': [750.6754897508647, 0, 0, 0, 0, 0, -117.16091759454126, 0], '0.001000_85.376682_1.366027_219.058687_0.000000_0.100000': [117.16091759454126, 0, 0, 0, 0, 0, 0, 0], '0.001000_70.559241_1.366027_219.058687_0.000000_0.100000': [39.527675791464006, 0, -4.7007401379758225, 0, 0, 0, 0, 0], '0.001100_70.559241_1.366027_219.058687_0.000000_0.100000': [0, 2.524640137002143, 0, 0, 0, 0, 0, 0], '0.001100_70.559241_1.366027_199.144261_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -586.0789814056576, 0], '0.001100_77.615165_1.366027_199.144261_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001100_93.914350_1.818182_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 90.97174630958307, 0, 0], '0.001000_77.615165_1.502630_320.723824_0.000000_0.100000': [1672.2934769924318, 1097.7965738046323, 0, 0, 0, 0, -22.789525808102145, 0], '0.001000_70.559241_1.502630_320.723824_0.000000_0.100000': [115.86477247434613, 0, 0.18046113647806125, 0, 0, 0, -140.07868358156207, 0], '0.001000_64.144765_1.502630_320.723824_0.000000_0.100000': [-0.005520115047124819, 0.8655360064579324, 0, 0, 0, 0, -94.17703119990327, 0], '0.001000_58.313423_1.502630_320.723824_0.000000_0.100000': [0, 0.852845115389198, 0, 0, 0, 0, 0, 0], '0.001000_58.313423_1.502630_291.567113_0.000000_0.100000': [0.8129952328648241, 0, 0, 0, 0, 0, 0.7161749091738635, -0.006209870908026005], '0.001000_53.012202_1.502630_291.567113_0.000000_0.100000': [0.007228689060336819, 0, 0, 0, 0, 0, 0, 0], '0.001100_53.012202_1.502630_388.075827_0.000000_0.100000': [0, 0, -0.002967249516609627, 0, 0, 0, 0, 0], '0.001210_53.012202_1.502630_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, -0.7550813181624563, 0, 0], '0.001210_53.012202_1.366027_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0.00981591658121488, 0], '0.001210_58.313423_1.366027_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 0.7454194339640708, 0, 0], '0.001210_58.313423_1.241843_388.075827_0.000000_0.100000': [-0.013080628840134523, 0, 0, 0, 0, 0, 0, 0], '0.001210_53.012202_1.241843_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, -0.0048995843818602225, 0, 0], '0.001210_53.012202_1.128948_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0.019327295110360864, 0, 0, 0], '0.001100_53.012202_1.128948_388.075827_0.000000_0.100000': [-0.7254023852128058, 0, 0, 0, 0, 0, 0, 0], '0.001100_48.192911_1.128948_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000909_103.305785_2.000000_352.796206_0.000000_0.100000': [387.7610352887759, 0, 0, 0, 0, 0, 0, 0], '0.000909_85.376682_2.000000_352.796206_0.000000_0.100000': [0, 0, 0, -1632.497034987652, 0, 0, 0, 0], '0.000909_77.615165_2.200000_352.796206_0.000000_0.100000': [0, 0, 0, 0, 1951.4776824089088, 0, 0, 0], '0.000826_77.615165_2.200000_352.796206_0.000000_0.100000': [2720.289072132071, 0, 0, 0, 0, 0, 0, 0], '0.000826_70.559241_2.200000_352.796206_0.000000_0.100000': [0, 1726.4768794483412, 0, 0, 0, 0, 0, 0], '0.000826_70.559241_2.200000_320.723824_0.000000_0.100000': [1599.3663915349857, 0, 0, 0, 0, 0, 0, 0], '0.000826_64.144765_2.200000_320.723824_0.000000_0.100000': [-0.0073259587612028556, 0, 0, 0, 0, -0.8217097135058111, 0, 0], '0.000826_64.144765_2.000000_320.723824_0.000000_0.100000': [0, 0.9177371772335011, 0, 0, 0, 0, 0, 0], '0.000826_64.144765_2.000000_265.061011_0.000000_0.100000': [0.006983519979650737, 0, 0, 0, -0.01758028644849219, 0, 0, 0], '0.000751_64.144765_2.000000_291.567113_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001000_85.376682_2.200000_291.567113_0.000000_0.100000': [340.99088267458745, 0, 0, 0, 0, 0, 0, 0], '0.001000_77.615165_2.200000_291.567113_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000909_70.559241_2.000000_291.567113_0.000000_0.100000': [74.58176084656466, 0, 0, 0, 0, 0, 0, 0], '0.000826_58.313423_2.000000_265.061011_0.000000_0.100000': [-0.6524794250308105, -0.0033816696371786747, 0, 0, 0, -0.006503366409515365, 0, 0], '0.000826_58.313423_1.818182_265.061011_0.000000_0.100000': [0, 0, 0, 0, -0.6562528565583989, 0, 0, 0], '0.000751_58.313423_1.818182_265.061011_0.000000_0.100000': [0, 0, 0.6562528565583989, 0, 0, 0, 0, 0], '0.000826_70.559241_1.366027_426.883410_0.000000_0.100000': [0, 1412.1329983510211, 0, 0, 0, 0, 0, 1187.1790787332252], '0.000826_64.144765_1.366027_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -124.59182638400218, 0], '0.001000_70.559241_1.818182_426.883410_0.000000_0.100000': [141.1575251306696, 0, 0, 0, 0, 0, 0, 0], '0.001000_64.144765_1.818182_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0.8802866641241076, 0, 0, 0], '0.000909_64.144765_1.818182_426.883410_0.000000_0.100000': [-0.8012608446233799, 0.02702775432787121, -0.08002606037491886, 0, 0, 0, 0, 0], '0.000909_58.313423_1.818182_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0.04725582029514044], '0.000909_58.313423_1.818182_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0.7457977086801976, 0, 0], '0.000909_85.376682_1.502630_320.723824_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -273.61902633830215], '0.000909_85.376682_1.502630_352.796206_0.000000_0.100000': [0, 0, 0, 0, 0, 646.2227323445078, 0, 0], '0.000909_58.313423_1.366027_265.061011_0.000000_0.100000': [0, 0, 0, 0, 0.7848515069215182, 0, 0, 0], '0.000826_58.313423_1.366027_265.061011_0.000000_0.100000': [0, 0, 0, 0.002617226035613385, 0, 0, 0, 0], '0.000826_58.313423_1.502630_265.061011_0.000000_0.100000': [0, 0, 0, 0, -0.8154528104710608, 0, 0, 0], '0.000751_58.313423_1.502630_265.061011_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0.013021012670265009], '0.000751_58.313423_1.502630_291.567113_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001000_64.144765_1.502630_352.796206_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -83.7784421118022, 0], '0.001000_70.559241_1.502630_352.796206_0.000000_0.100000': [0, 0, 0, 0, 0, -767.746930292274, 0, 0], '0.001000_70.559241_1.366027_352.796206_0.000000_0.100000': [852.3346831659439, 0, 0, 0, 0, 0, 0, 0], '0.001000_64.144765_1.366027_352.796206_0.000000_0.100000': [-0.024469837221727176, -0.006722272905388946, 0, 0, 0, 0, 0, -0.7859579712724459], '0.001210_58.313423_1.502630_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0.8213609437215837, 0, 0, 0], '0.000909_85.376682_1.366027_291.567113_0.000000_0.100000': [0, 258.94323968077975, 0, 0, 0, 0, 0, 0], '0.000909_85.376682_1.366027_265.061011_0.000000_0.100000': [0, 0, 0, -315.97829167152366, 2.8157190665979215, 0, 0, 0], '0.000909_85.376682_1.502630_265.061011_0.000000_0.100000': [0, 0, 0, 0, 0, 315.97829167152366, 0, 0], '0.000826_85.376682_1.366027_265.061011_0.000000_0.100000': [0, 0, 0, 0, 2.646333644706101, 0, 0, 0], '0.000751_85.376682_1.366027_265.061011_0.000000_0.100000': [164.63958792918734, 0, 0, 0, 0, 0, 0, 0], '0.000751_77.615165_1.366027_265.061011_0.000000_0.100000': [960.7656519485458, 0, 0, 0, 0, 0, 0, -391.8338517706209], '0.001000_85.376682_1.241843_756.250000_0.000000_0.100000': [0, 0, 0, -844.973958047029, 0, 0, 0, 0], '0.001000_85.376682_1.128948_568.181818_0.000000_0.100000': [0, 0, 0, 264.53924233143863, 0, 0, 0, 0], '0.000909_85.376682_1.241843_568.181818_0.000000_0.100000': [2916.2711835774135, 0, 0, 0, 0, 0, 0, 0], '0.001210_113.636364_1.652893_352.796206_0.000000_0.100000': [0, 0, 0, 0, 1.4799797949105595, 0, 0, 0], '0.001000_64.144765_2.000000_291.567113_0.000000_0.100000': [0, 0, 0, 0.1595460483074438, 0, 0, 0, 0], '0.001000_64.144765_2.200000_291.567113_0.000000_0.100000': [0.015385895488248025, 0, 0, -0.8046316960467675, 0, 0, 0, 0], '0.001000_64.144765_2.420000_291.567113_0.000000_0.100000': [0, 0.020559707585091472, 0, 0, 0, 0, 0, 0], '0.001000_64.144765_2.420000_265.061011_0.000000_0.100000': [0, 0, 0, 0, 0.7489187377801273, -0.03943879561888935, 0, 0], '0.000909_64.144765_2.420000_265.061011_0.000000_0.100000': [0, 0, 0, 0, -0.791699978284646, 0, 0, 0], '0.000826_64.144765_2.420000_265.061011_0.000000_0.100000': [0, 0, 0, 0.0715815571596977, 0, 0, 0, 0], '0.000826_64.144765_2.662000_265.061011_0.000000_0.100000': [0.6983176724246732, 0, 0, 0, 0, 0, 0, 0], '0.000826_58.313423_2.662000_265.061011_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000909_85.376682_2.000000_291.567113_0.000000_0.100000': [0, 0, 0, 0, 0, 659.799133030708, 0, 0], '0.000826_85.376682_1.652893_291.567113_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000826_70.559241_1.366027_291.567113_0.000000_0.100000': [0, 0, -2.964104237323045, 0, 0, 0, 0, 0], '0.000909_70.559241_1.366027_291.567113_0.000000_0.100000': [0, 0, 158.70280742358761, 0, 0, 0, 0, 0], '0.001000_70.559241_1.128948_320.723824_0.000000_0.100000': [0, 0, 0, 1382.6709162226837, 0, 0, 0, 0], '0.001100_64.144765_1.502630_265.061011_0.000000_0.100000': [0.020171980987674944, 0, -0.13006710270858535, 0, 0, 0, 0, 0], '0.001210_64.144765_1.502630_265.061011_0.000000_0.100000': [0, 0.8415964500451598, 0, 0, 0, 0, 0, 0], '0.001210_64.144765_1.502630_240.964556_0.000000_0.100000': [-0.7048313264491981, 0, 0, 0, 0, 0, 0, -0.07643927793325699], '0.001100_85.376682_1.366027_291.567113_0.000000_0.100000': [1235.913276353604, 0, 0, -437.40719658558737, 0, 0, 0, 0], '0.001100_77.615165_1.366027_291.567113_0.000000_0.100000': [1010.928291139978, 0, 0, 0, 0, -295.76091643578604, 0, 0], '0.001100_77.615165_1.241843_291.567113_0.000000_0.100000': [0, 0, -0.6861334056459782, 295.76091643578604, 0, 0, 0, 0], '0.001210_77.615165_1.241843_291.567113_0.000000_0.100000': [0, 0, 0, 0, 0.6861334056459782, 0, 0, 0], '0.001100_70.559241_1.366027_291.567113_0.000000_0.100000': [0, 0, 0, 504.7762732800003, 0, 0, 19.02834029473329, 0], '0.001100_70.559241_1.502630_291.567113_0.000000_0.100000': [286.7509007460139, 0, -0.5917884970092019, 0, 0, 0, -1030.0373528182229, 0], '0.001100_77.615165_1.502630_291.567113_0.000000_0.100000': [0, 921.1326403042243, -2.03765584357011, 0, 0, 0, 0, 0], '0.001210_77.615165_1.502630_291.567113_0.000000_0.100000': [1031.4832201647837, 0, 0, 0, 0, 0, 0, 0], '0.000826_93.914350_2.200000_388.075827_0.000000_0.100000': [0, 0, 1997.6266588232368, 0, 0, 0, 0, 0], '0.000826_77.615165_2.000000_352.796206_0.000000_0.100000': [0, 0, 0, 0, 0, -95.05343603739932, 0, 0], '0.000826_53.012202_2.000000_265.061011_0.000000_0.100000': [0, 0, 0, 0, 0, 0.6230494377495915, 0, 0], '0.000826_53.012202_1.818182_265.061011_0.000000_0.100000': [0.01616839557389085, -0.05870362612859026, 0, 0, 0, 0, 0, 0], '0.000826_53.012202_1.818182_240.964556_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0.05870362612859026], '0.000826_48.192911_1.818182_265.061011_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001100_64.144765_1.241843_320.723824_0.000000_0.100000': [0, 0.7651811862378736, 0, 0, 0, 0, 0, 0], '0.001100_64.144765_1.241843_291.567113_0.000000_0.100000': [0, 0, 0, 0, -0.011335442137783503, 0, -521.9275490162514, 0], '0.001100_70.559241_1.241843_291.567113_0.000000_0.100000': [0, 165.04935686606296, 0, 0, 0, 0, 0, 0], '0.001100_70.559241_1.241843_265.061011_0.000000_0.100000': [0, 0, 0, 267.62151243661725, 0, 0, 0, 0], '0.001100_70.559241_1.366027_265.061011_0.000000_0.100000': [0, 0, 0.763906380510116, 0, 0, 0, 0, 0], '0.001210_70.559241_1.366027_265.061011_0.000000_0.100000': [0, -47.64934189750259, 0, 0, 0, 0, 0, 0], '0.001210_70.559241_1.366027_240.964556_0.000000_0.100000': [136.15258440186048, 0, 0, 0, 0, 0, 0, 0], '0.001210_64.144765_1.366027_240.964556_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -0.010522519268079122], '0.001210_64.144765_1.366027_265.061011_0.000000_0.100000': [0, 0, 0, 0, -0.0017827526773089631, 0, 0, 0], '0.001100_64.144765_1.366027_265.061011_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -88.56732787046352, 0], '0.000564_64.144765_1.652893_352.796206_0.000000_0.100000': [0, 0.8772358922834476, 0, 0, 0, 0, 0, 0], '0.000564_64.144765_1.652893_320.723824_0.000000_0.100000': [0.028723483537871745, 0, 0, 0, 0, 0, 0, 0], '0.001000_93.914350_1.366027_352.796206_0.000000_0.100000': [1354.7352154586085, 0, 0, 0, 0, -112.16133778205531, 0, 0], '0.001000_93.914350_1.241843_352.796206_0.000000_0.100000': [0, 269.4769664272589, 0, 0, 0, 0, 0, 0], '0.001000_93.914350_1.241843_320.723824_0.000000_0.100000': [0, 0, -0.6361546635414016, 0, 0, 0, 0, 0], '0.001100_77.615165_1.241843_320.723824_0.000000_0.100000': [0, 0, 0, 0, 1.0823503913961758, 0, 0, 0], '0.001000_77.615165_1.241843_320.723824_0.000000_0.100000': [0, 0, 0, 1257.767918226131, 0, 0, 0, 0], '0.001000_64.144765_1.366027_320.723824_0.000000_0.100000': [0.014432707113306333, 0, 0, 0, 0, 0, 0, 0], '0.001464_77.615165_2.000000_516.528926_0.000000_0.100000': [-346.89423200613857, 488.250415916339, 0, -610.9699903265882, 0, 0, 0, 0], '0.001464_77.615165_2.200000_516.528926_0.000000_0.100000': [-912.3036225037265, 0, 0, 0, 0, 0, 0, 0], '0.001464_70.559241_2.200000_516.528926_0.000000_0.100000': [5893.779562864628, 0, 0, -1100.5225578708557, 0, 0, 0, 0], '0.001464_70.559241_2.420000_516.528926_0.000000_0.100000': [0, 0, 0, 1704.5170336484084, 0, 0, 0, 0], '0.001464_70.559241_2.662000_516.528926_0.000000_0.100000': [5289.775605548363, 0, 0, 0, 0, 0, 0, 0], '0.001464_64.144765_2.662000_516.528926_0.000000_0.100000': [0, 0, -0.6516311269404014, 0, 0, 0, 0, 0], '0.001611_64.144765_2.662000_516.528926_0.000000_0.100000': [0.7489438825980415, 0, 0, 0, 0, 0, 0, 0], '0.001611_58.313423_2.662000_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001100_70.559241_2.000000_291.567113_0.000000_0.100000': [114.46312208895979, 0, -13.846963842620061, 0, 0, 0, 0, 0], '0.001210_70.559241_2.000000_291.567113_0.000000_0.100000': [0, 0, 0, 0, 13.846963842620061, 0, 0, 0], '0.001100_64.144765_2.000000_291.567113_0.000000_0.100000': [0, 0, 0, 0, 0.6705582558686388, 0, 0, 0], '0.001000_58.313423_2.200000_291.567113_0.000000_0.100000': [0.010358227148458354, 0, 0, 0, 0, 0, 0, 0], '0.001000_53.012202_2.200000_291.567113_0.000000_0.100000': [0, -0.07330799626631689, 0, 0, 0, 0, 0, 0], '0.001000_53.012202_2.200000_265.061011_0.000000_0.100000': [0, 0.10229911370886384, 0, 0, 0, 0, 0, 0], '0.001000_53.012202_2.200000_240.964556_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000909_103.305785_1.366027_469.571751_0.000000_0.100000': [0, 0, 0, 1132.3213249629955, 0, 0, 0, 0], '0.001100_64.144765_1.502630_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0.014550607482931355], '0.001100_64.144765_1.502630_426.883410_0.000000_0.100000': [-0.009350864881434972, -0.001322782498448305, 0, 0, 0, 0, 0, 0], '0.001100_58.313423_1.502630_426.883410_0.000000_0.100000': [0, 0, 0, -0.010653749103039446, 0, 0, 0, 0], '0.001100_53.012202_1.652893_426.883410_0.000000_0.100000': [0.01324174998592298, 0, 0, 0, 0, 0, 0, 0], '0.001210_58.313423_2.200000_388.075827_0.000000_0.100000': [0, 0.07724282813319405, 0, 0, 0, 0, 0, 0], '0.001210_58.313423_2.200000_352.796206_0.000000_0.100000': [0, 0, -0.001955396229550388, 0, 0, 0, 0, 0], '0.001331_58.313423_2.200000_352.796206_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0.7548504773868672, 0], '0.001331_64.144765_2.200000_352.796206_0.000000_0.100000': [0, 0, 0, -0.7485731695597906, 0, 0, 0, 0], '0.001331_64.144765_2.420000_352.796206_0.000000_0.100000': [0.7479477728747724, 0, 0, 0, 0, 0, 0, 0], '0.001331_58.313423_2.420000_352.796206_0.000000_0.100000': [0, 0, 0, 0, -0.7615005156831786, 0, 0, 0], '0.001210_58.313423_2.420000_352.796206_0.000000_0.100000': [0, 0, 0, 0.03938086426251175, 0, 0, 0, 0], '0.001210_58.313423_2.662000_352.796206_0.000000_0.100000': [0, 0, -0.023347401470982732, 0, 0, 0, 0, 0], '0.001331_58.313423_2.662000_352.796206_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001464_85.376682_2.000000_568.181818_0.000000_0.100000': [552.3389857986148, 0, 0, -457.3291038091481, 0, 0, 0, 0], '0.001464_85.376682_2.200000_568.181818_0.000000_0.100000': [68.37640174580757, 0, 0, 0, 0, 0, 0, 0], '0.001464_77.615165_2.200000_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, -153.00690934076403, 0, 0], '0.001464_77.615165_2.000000_568.181818_0.000000_0.100000': [0, 1337.4760632477703, 0, 0, 0, 0, 0, 0], '0.001464_70.559241_2.000000_516.528926_0.000000_0.100000': [0, 4681.509378003758, 0, 0, 0, 0, 0, 0], '0.001464_70.559241_2.000000_469.571751_0.000000_0.100000': [0, 9.430129031655156, 0, 0, 0, -116.36826690858136, 0, 0], '0.001464_70.559241_2.000000_426.883410_0.000000_0.100000': [0, 0, -12.54807615462588, 0, 0, 0, 0, 0], '0.001611_58.313423_2.000000_388.075827_0.000000_0.100000': [0, 0, 0, -0.7571278194605838, 0, 0, 0, 0], '0.001611_58.313423_2.200000_388.075827_0.000000_0.100000': [0, -0.00479143592156106, 0, 0, 0, 0, 0, 0], '0.001611_58.313423_2.200000_352.796206_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001100_113.636364_2.420000_469.571751_0.000000_0.100000': [0, 1158.552980825191, 0, 0, 0, 0, 0, 0], '0.001210_77.615165_2.000000_265.061011_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000826_58.313423_2.000000_240.964556_0.000000_0.100000': [0, 0, 0.00023011636142200587, 0, 0, 0, 0, 0], '0.000909_58.313423_2.000000_240.964556_0.000000_0.100000': [-0.7953976735809193, 0, -0.0127394028743538, 0, 0, 0, 0, 0], '0.001000_58.313423_2.000000_240.964556_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0.013370246102044804, 0], '0.001000_64.144765_2.000000_240.964556_0.000000_0.100000': [0, 0, 0, 0, 0, -0.006717727834279686, 0, 0], '0.001000_64.144765_1.818182_240.964556_0.000000_0.100000': [0, 0, 0, 0, -0.6280399138124494, 0, 0, 0], '0.000909_64.144765_1.818182_240.964556_0.000000_0.100000': [0, 0, 0, 0.6407959979211683, 0, 0, 0, 0], '0.000909_64.144765_2.000000_240.964556_0.000000_0.100000': [-0.006669199502130141, 0, 0, 0, 0, 0, 0, 0], '0.000909_64.144765_1.502630_265.061011_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0.8556801708711794], '0.000909_64.144765_1.652893_291.567113_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000909_103.305785_1.818182_291.567113_0.000000_0.100000': [0, 0, 524.5424248334082, 0, 0, 0, 0, 0], '0.001210_70.559241_1.502630_291.567113_0.000000_0.100000': [261.2789330752507, 0, 0, 0, 0, 0, -1031.4832201647837, 0], '0.001210_64.144765_1.502630_291.567113_0.000000_0.100000': [-0.018202488496763182, 0, 0, 0, 0, 0, 0, 0], '0.000826_64.144765_1.502630_291.567113_0.000000_0.100000': [0, 0, 0, 0, 0, 0.7774306652601354, 0, 0], '0.000826_64.144765_1.366027_291.567113_0.000000_0.100000': [-0.038052191424890226, 0, -0.8149262992021477, 0, 0, 0, 0, 0], '0.000909_64.144765_1.366027_291.567113_0.000000_0.100000': [0, 0, 0, 0.17951053583177778, 0, 0, 0, 0], '0.000683_93.914350_2.000000_320.723824_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000751_58.313423_1.241843_388.075827_0.000000_0.100000': [0, 0, 0, 0.006849169073095709, 0, 0, 0, 0], '0.000751_58.313423_1.366027_388.075827_0.000000_0.100000': [-0.012848376978513444, 0.010950748860000204, 0, -0.018628744545844712, 0, 0, 0, 0], '0.000751_58.313423_1.502630_388.075827_0.000000_0.100000': [0, 0, -0.6055308004547214, 0, 0, 0, 0, 0], '0.000826_58.313423_1.502630_388.075827_0.000000_0.100000': [0, 0, -0.03661199368089001, 0, 0, 0, 0, 0], '0.000909_64.144765_1.128948_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -0.6058127143467346], '0.000909_64.144765_1.128948_568.181818_0.000000_0.100000': [0, 0, 0.6231775219051997, 0, 0, 0, 0, 0], '0.001000_64.144765_1.128948_516.528926_0.000000_0.100000': [-0.03933770773719516, 0, 0, 0, 0, 0, 0, -0.0009935587575455429], '0.001000_58.313423_1.128948_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000751_93.914350_1.241843_320.723824_0.000000_0.100000': [0, 0, 0, 0, 0, -224.08126578614193, 0, 0], '0.000751_93.914350_1.128948_320.723824_0.000000_0.100000': [0, 260.6190864205323, 0, 0, 0, 0, 0, 0], '0.001331_103.305785_2.200000_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -317.09102414977144, 0], '0.001210_77.615165_2.420000_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 2265.450535652071, 0, 0], '0.001464_77.615165_2.000000_469.571751_0.000000_0.100000': [3846.3647300812804, 0, 0, 0, 0, 0, 0, 0], '0.001464_70.559241_1.818182_469.571751_0.000000_0.100000': [152.32319916954356, 0, 0, 0, 0, 0, 0, 0], '0.001464_64.144765_1.818182_469.571751_0.000000_0.100000': [0, -0.7158128196673488, 0, 0, 0, 0, 0, 0], '0.001464_64.144765_1.818182_426.883410_0.000000_0.100000': [0.7541784437105434, 0, 0, 0, 0, 0, 0, 0], '0.001464_58.313423_1.818182_426.883410_0.000000_0.100000': [-0.7499783518793492, 0, 0, 0, 0, 0, 0, 0], '0.001464_53.012202_1.818182_426.883410_0.000000_0.100000': [0, 0, 0.014646396586863375, 0, 0, 0, 0, 0], '0.001611_53.012202_1.818182_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, 0.7362238365734811, 0, 0], '0.001611_53.012202_1.652893_426.883410_0.000000_0.100000': [0, 0.017914099938570873, 0, 0, 0, 0, 0, 0], '0.001611_53.012202_1.652893_388.075827_0.000000_0.100000': [0, 0, -0.0053037706531540605, 0, 0, 0, 0, 0], '0.001772_53.012202_1.652893_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000909_64.144765_1.652893_352.796206_0.000000_0.100000': [0, 0, 0, 0, 0, 0.6785569957349389, 0, 0], '0.001000_53.012202_1.502630_352.796206_0.000000_0.100000': [0, -0.773775084352792, 0, 0, 0, 0, 0, 0], '0.001000_53.012202_1.502630_320.723824_0.000000_0.100000': [0, 0, 0, 0.7583920012083354, 0, 0, 0, 0], '0.001000_53.012202_1.652893_320.723824_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001210_85.376682_1.652893_352.796206_0.000000_0.100000': [3238.562023854858, 0, 0, 0, 0, 1205.9664897620291, 0, 0], '0.001000_70.559241_2.000000_265.061011_0.000000_0.100000': [0, 0, 0, 0, 90.35685901487734, 0, -167.28908056188442, 0], '0.000909_70.559241_2.000000_265.061011_0.000000_0.100000': [44.43028000391403, 0, -8.140257568907867, 0, 0, 0, 0, 0], '0.001100_53.012202_2.000000_388.075827_0.000000_0.100000': [0, 0, -0.08643750327521503, 0, 0, 0, 0, 0], '0.001210_53.012202_2.000000_388.075827_0.000000_0.100000': [0.747260617646566, 0, 0, 0, 0, 0, 0, 0], '0.001210_93.914350_1.241843_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -360.3380961519576], '0.001210_93.914350_1.241843_469.571751_0.000000_0.100000': [0, 0, 0, 0, 2.002033322261468, 0, 0, 0], '0.001100_93.914350_1.241843_469.571751_0.000000_0.100000': [0, 395.8123803711976, 0, 0, 0, 0, 0, 0], '0.001100_93.914350_1.241843_426.883410_0.000000_0.100000': [156.7792882378155, 0, 0, 0, 0, 0, 0, -35.98294367010888], '0.001100_85.376682_1.241843_426.883410_0.000000_0.100000': [0, 0, 0, 0, 1.247497725953508, 0, 0, 0], '0.001000_85.376682_1.241843_426.883410_0.000000_0.100000': [0, 0, 0, 0, 1.247679429303771, 0, 0, 0], '0.000909_85.376682_1.241843_426.883410_0.000000_0.100000': [0, 0, -0.11342540266397923, 0, 0, 0, 0, 0], '0.000751_85.376682_1.366027_320.723824_0.000000_0.100000': [180.52474863655354, 0, 0, -108.47995637426202, 0, 0, 0, 0], '0.000751_85.376682_1.502630_320.723824_0.000000_0.100000': [0, -104.70997991930226, 0, 0, 0, 0, 0, 0], '0.000751_64.144765_1.366027_388.075827_0.000000_0.100000': [0.012412141521667764, 0, 0, 0, 0, 0, -2.422601974588789, 0], '0.000751_53.012202_1.366027_388.075827_0.000000_0.100000': [0, 0, -0.0026029138332205547, 0, 0, 0, 0, 0], '0.000826_53.012202_1.366027_388.075827_0.000000_0.100000': [0, 0.0018199651950226326, 0, 0, 0, 0, 0, 0], '0.000826_53.012202_1.366027_352.796206_0.000000_0.100000': [0, 0, 0.029516203935872908, 0, 0, 0, 0, 0], '0.000909_53.012202_1.366027_352.796206_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001210_64.144765_2.000000_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0.05943577042971811, 0, 0, 0], '0.001100_64.144765_2.000000_568.181818_0.000000_0.100000': [0, 0, 0, 0.6285617332312541, 0, 0, 0, 0], '0.001100_64.144765_2.200000_568.181818_0.000000_0.100000': [0, 0, 0, 0.05636208270116185, 0, 0, 0, 0], '0.001100_64.144765_2.420000_568.181818_0.000000_0.100000': [-0.8122066838521506, 0, 0, 0, 0, 0, 0, 0], '0.001100_58.313423_2.420000_568.181818_0.000000_0.100000': [0, 0, 0.7378072860012254, 0, 0, 0, 0, 0], '0.001210_53.012202_2.420000_568.181818_0.000000_0.100000': [0, 0, 0.7245723182744087, 0, 0, 0, 0, 0], '0.001331_53.012202_2.420000_568.181818_0.000000_0.100000': [0.012931293026122148, 0, 0, 0, 0, 0, 0, 0], '0.001331_48.192911_2.420000_568.181818_0.000000_0.100000': [-0.7444334536590532, 0, 0, 0, 0, 0, 0, 0], '0.001331_43.811737_2.420000_568.181818_0.000000_0.100000': [0, 0, 0.7278369171257282, 0, 0, 0, 0, 0], '0.001464_43.811737_2.420000_568.181818_0.000000_0.100000': [-0.7351401067547386, 0, 0, 0, 0, 0, 0, 0], '0.001464_39.828852_2.420000_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, -0.014339305774615242, 0, 0], '0.001464_39.828852_2.200000_568.181818_0.000000_0.100000': [0, 0, 0.01435369180037469, 0, 0, 0, 0, 0], '0.001611_39.828852_2.200000_568.181818_0.000000_0.100000': [0, 0, 0.008820757472176411, 0, 0, 0, 0, 0], '0.001772_39.828852_2.200000_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001100_77.615165_1.502630_320.723824_0.000000_0.100000': [0, 0, 0, 0, 0, 1508.4316305619236, 0, 0], '0.001100_85.376682_2.000000_265.061011_0.000000_0.100000': [299.2141734399915, 0, 0, 0, 0, 0, 0, 0], '0.001100_77.615165_2.000000_265.061011_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -308.29285115743977], '0.001000_93.914350_2.000000_291.567113_0.000000_0.100000': [2858.590426039562, 0, 0, 0, 0, 0, 0, 0], '0.001000_77.615165_2.000000_265.061011_0.000000_0.100000': [1920.7684361129159, 0, 0, 0, 0, 0, 0, 0], '0.000909_64.144765_2.000000_265.061011_0.000000_0.100000': [-0.0006672417232493899, 0.010626169073157143, 0, -0.027824319951946563, 0, 0, 0, 0], '0.000909_64.144765_2.200000_265.061011_0.000000_0.100000': [0, 0, 0, 0, 0, 0.027824319951946563, 0, 0], '0.000909_58.313423_2.000000_265.061011_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000751_64.144765_1.502630_291.567113_0.000000_0.100000': [-0.798166916315563, -0.004714177605796977, 0, 0, 0, -0.0008240679018399532, 0, 0], '0.000751_64.144765_1.366027_291.567113_0.000000_0.100000': [0, -0.006698612575041807, 0, 0, 0, 0, 0, 0], '0.000751_64.144765_1.366027_265.061011_0.000000_0.100000': [0, 0, 0, 0, 0, -0.6779393181572914, 0, 0], '0.000751_64.144765_1.241843_265.061011_0.000000_0.100000': [0, 0, -0.07414339476142687, 0, 0, 0, 0, 0], '0.000826_64.144765_1.241843_265.061011_0.000000_0.100000': [0, 0.03949349688978145, 0, 0, 0, 0, 0, 0], '0.000826_64.144765_1.241843_240.964556_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000909_64.144765_1.502630_320.723824_0.000000_0.100000': [0, 0, 0, 0, 0.8753609640549553, 0, 0, 0], '0.000826_64.144765_1.502630_320.723824_0.000000_0.100000': [-0.024357959150120224, 0, -0.07957826945954133, 0, 0, 0, 0, 0], '0.000826_58.313423_1.502630_320.723824_0.000000_0.100000': [0, 0, 0, 0, -0.7391868813419251, 0, 0, 0], '0.000751_58.313423_1.502630_320.723824_0.000000_0.100000': [0.012899632398057293, 0, 0, 0, 0, 0, 0, 0], '0.000751_53.012202_1.502630_320.723824_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001000_77.615165_1.818182_388.075827_0.000000_0.100000': [0, 368.4752988658871, 0, 0, 0, 0, 0, 0], '0.001000_77.615165_1.818182_352.796206_0.000000_0.100000': [0, 691.5905506588379, 0, 0, 0, 0, 0, 0], '0.001000_77.615165_1.818182_320.723824_0.000000_0.100000': [0, 0, 0, 2406.4165848423954, 0, 0, 0, 0], '0.001000_70.559241_2.000000_320.723824_0.000000_0.100000': [30.862469851502063, 0, 0, 0, 0, 0, 0, 0], '0.001000_64.144765_2.000000_320.723824_0.000000_0.100000': [-0.0065022624128005194, -0.020197457299290157, -0.8174666131275444, 0.03248398540476956, 0, 0, 0, 0], '0.001100_64.144765_2.000000_320.723824_0.000000_0.100000': [0, 0, 0, 0.8351629546264576, 0, 0, 0, 0], '0.001100_64.144765_2.200000_320.723824_0.000000_0.100000': [0, 0, 0, -0.015647533449055118, 0, 0, 0, 0], '0.001100_64.144765_2.420000_320.723824_0.000000_0.100000': [-0.014160403921437137, -0.0032949274539760156, 0, 0, 0, 0, 0, 0], '0.001100_64.144765_2.420000_291.567113_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0.0032949274539760156], '0.001100_58.313423_2.420000_320.723824_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001000_58.313423_1.366027_320.723824_0.000000_0.100000': [-0.013340682628404465, -0.03375155061701207, -0.0060603191113386854, -0.12509049583892487, -0.0018050153878448327, -0.021293598453485173, 0, 0], '0.001000_58.313423_1.366027_291.567113_0.000000_0.100000': [0.02473291917428924, 0, 0, 0, 0, 0, 0, 0], '0.001000_53.012202_1.366027_291.567113_0.000000_0.100000': [0, 0, 0, 0, 0, -0.747253566810491, 0, 0], '0.001000_53.012202_1.241843_291.567113_0.000000_0.100000': [0, 0, 0.7294480656509904, 0, 0, 0, 0, 0], '0.001100_53.012202_1.241843_291.567113_0.000000_0.100000': [0, 0.02261844454643324, 0, 0, 0, 0, 0, 0], '0.001100_53.012202_1.241843_265.061011_0.000000_0.100000': [0, 0, 0, 0.0025034655044191825, 0, 0, 0, 0], '0.001100_53.012202_1.366027_265.061011_0.000000_0.100000': [-0.007699550968459335, 0, -0.7281301963077113, 0, 0, 0, 0, 0], '0.001210_53.012202_1.366027_265.061011_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001210_103.305785_1.502630_388.075827_0.000000_0.100000': [0, 0, 0, 292.66630412888213, 0, 0, 0, 0], '0.001210_64.144765_1.652893_352.796206_0.000000_0.100000': [-0.8050348456081793, 0, 0, 0, 0, -0.008784039189004033, 0, 0], '0.001210_64.144765_1.502630_352.796206_0.000000_0.100000': [-0.019288843781123433, 0, 0, 0, 0, 0, -88.89191025663884, 0], '0.001210_70.559241_1.502630_352.796206_0.000000_0.100000': [88.89191025663884, -14.231522027029996, 0, 0, 0, 0, 0, 0], '0.001210_70.559241_1.502630_320.723824_0.000000_0.100000': [114.49847485937595, 0, 0, 0, 0, 0, 0, 0], '0.001210_64.144765_1.502630_320.723824_0.000000_0.100000': [-0.762470810756804, 0.00171843194327721, 0, -0.012929279501645397, 0, -0.0401493508112253, 0, 0], '0.001210_64.144765_1.366027_320.723824_0.000000_0.100000': [0, 0, 0, 0.0401493508112253, 0, 0, 0, 0], '0.001210_58.313423_1.502630_320.723824_0.000000_0.100000': [0, 0, 0, -0.05973312444466501, 0, 0, 0, 0], '0.001210_58.313423_1.652893_320.723824_0.000000_0.100000': [0.7883058337985631, 0, 0, 0, 0, 0, 0, 0], '0.001210_53.012202_1.652893_320.723824_0.000000_0.100000': [0, 0, 0, 0, -0.877272572983793, 0, 0, 0], '0.001100_53.012202_1.652893_320.723824_0.000000_0.100000': [0, 0, 0.877272572983793, 0, 0, 0, 0, 0], '0.001000_113.636364_1.366027_352.796206_0.000000_0.100000': [0, 0, 0, -237.54189597003483, 0, 0, 0, 0], '0.001331_103.305785_1.502630_687.500000_0.000000_0.100000': [1420.5264373411367, 0, 0, 0, 0, 0, 0, 0], '0.001331_93.914350_1.502630_687.500000_0.000000_0.100000': [0, 0, 0, -1198.5423850702787, 0, 0, 0, 0], '0.001331_93.914350_1.652893_687.500000_0.000000_0.100000': [0, 0, 220.40433115536317, 0, 0, 0, 0, 0], '0.001464_93.914350_1.502630_687.500000_0.000000_0.100000': [0, -639.8247181201027, 0, 0, 0, 0, 0, 0], '0.001210_85.376682_1.502630_625.000000_0.000000_0.100000': [0, 0, 0, 0, 1351.093705701056, 0, 0, 0], '0.001331_93.914350_2.000000_756.250000_0.000000_0.100000': [250.94079217596547, 0, 0, 0, 0, 0, 0, 0], '0.001464_85.376682_2.000000_756.250000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -363.3310232925464, 0], '0.001464_93.914350_2.000000_756.250000_0.000000_0.100000': [0, -1999.6664500528946, 0, 0, 0, 0, 0, 0], '0.001464_77.615165_2.000000_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 252.27515650015212, 0, 0], '0.001464_77.615165_1.818182_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -195.5341985962964, 0], '0.001464_85.376682_1.818182_625.000000_0.000000_0.100000': [0, 3717.4097810305825, 0, 0, 0, 0, 0, 0], '0.001464_93.914350_1.818182_568.181818_0.000000_0.100000': [0, 1359.9347757383252, 0, 0, 0, 0, 0, 0], '0.001331_70.559241_1.818182_426.883410_0.000000_0.100000': [0, 0, 2.668991628387539, 0, 0, 0, 0, 0], '0.001464_70.559241_1.818182_426.883410_0.000000_0.100000': [0, 1741.8340592457637, 0, 0, 0, 0, 0, 0], '0.001464_70.559241_1.818182_388.075827_0.000000_0.100000': [0, -82.28449973095881, 0, 0, 0, 0, 0, 0], '0.001464_70.559241_1.818182_352.796206_0.000000_0.100000': [182.5524925132445, 0, 0, 0, 0, 0, 0, 0], '0.001464_64.144765_1.818182_352.796206_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000909_53.012202_1.818182_756.250000_0.000000_0.100000': [0, 0, 0, 0, 0, -0.6915189898858216, 0, 0], '0.000909_53.012202_1.652893_756.250000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001000_64.144765_1.502630_265.061011_0.000000_0.100000': [0, 0, 0, -0.7225997212319498, 0, 0, 0, 0], '0.001000_64.144765_1.652893_265.061011_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000751_77.615165_1.241843_291.567113_0.000000_0.100000': [1157.7893639570966, 0, 0, 0, 0, 0, 0, 0], '0.000751_70.559241_1.241843_291.567113_0.000000_0.100000': [488.7391270975845, 0, 0, 0, 0, 0, 0, 0], '0.000751_64.144765_1.241843_291.567113_0.000000_0.100000': [0.629455450291, 0, 0, 0, 0, 0, -44.029943671955685, 0], '0.000751_58.313423_1.241843_291.567113_0.000000_0.100000': [0, 0, 0, 0, 0.690225101849569, 0, 0, 0], '0.000683_58.313423_1.241843_291.567113_0.000000_0.100000': [0, 0, 0, 0, 0.005934419330446872, 0, 0, 0], '0.000621_58.313423_1.241843_291.567113_0.000000_0.100000': [0, -0.7575727431323003, 0, 0, 0, 0, 0, 0], '0.000621_58.313423_1.241843_265.061011_0.000000_0.100000': [0.7499885109794832, 0, 0, 0, 0, 0, 0, 0], '0.000621_53.012202_1.241843_265.061011_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001100_70.559241_1.502630_320.723824_0.000000_0.100000': [0, 0, 93.77802414339872, 0, 0, 0, 0, 0], '0.001210_58.313423_1.502630_291.567113_0.000000_0.100000': [0, 0, 0, -0.10048126645726825, 0.016748616584482078, 0, 0, 0], '0.001100_58.313423_1.502630_291.567113_0.000000_0.100000': [-0.8112125914806712, -0.010801887037107463, -0.016748616584482078, -0.7320267151676262, -0.716349263255985, 0.00530435900011935, 0, 0], '0.001000_53.012202_1.366027_426.883410_0.000000_0.100000': [0, 0, 0.028655097305545496, 0, 0, 0, 0, 0], '0.001100_53.012202_1.366027_426.883410_0.000000_0.100000': [-0.9996481296194134, 0, 0, 0, 0, -0.8380813207460506, 0, 0], '0.001100_53.012202_1.241843_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0.8292229148521756, 0, 0, 0], '0.001000_53.012202_1.241843_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -0.024368922940584525], '0.001000_53.012202_1.241843_469.571751_0.000000_0.100000': [0, 0, 0, 0.005363770278518132, 0, 0, 0, 0], '0.001100_53.012202_1.366027_469.571751_0.000000_0.100000': [0, 0.03651697294772021, 0, 0, 0, 0, 0, 0], '0.001100_48.192911_1.366027_426.883410_0.000000_0.100000': [0, 0, 0.21134832109628407, 0, 0, 0, 0, 0], '0.001210_48.192911_1.366027_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001210_93.914350_2.000000_516.528926_0.000000_0.100000': [1277.834232428249, 0, 0, 0, 0, 0, -106.75648626525529, 0], '0.001331_64.144765_2.000000_388.075827_0.000000_0.100000': [0, 0, 0, -0.7859628968492416, 0, 0, 0, 0], '0.001331_64.144765_2.200000_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000751_70.559241_1.366027_265.061011_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -87.34233199532234, 0], '0.001331_77.615165_2.000000_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 537.3204308745497, 0], '0.001331_77.615165_2.000000_516.528926_0.000000_0.100000': [0, 0, 0, -613.0964211177242, 0, 0, 0, 0], '0.001331_70.559241_2.200000_516.528926_0.000000_0.100000': [0, 0, -8.5713410418246, 0, 0, 0, 0, 0], '0.001464_53.012202_2.200000_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0.02511873237972395, 0, 0, 0], '0.001331_53.012202_2.200000_516.528926_0.000000_0.100000': [0, 0, 0, 0, -0.019716854528837757, 0, 0, 0], '0.001210_53.012202_2.200000_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001100_85.376682_1.502630_291.567113_0.000000_0.100000': [0, 0, 0, -614.3650810037215, 0, 0, 0, 0], '0.001100_85.376682_1.652893_291.567113_0.000000_0.100000': [0, 0, 0, 0, 2.9784411214862305, 0, 0, 0], '0.001000_85.376682_1.652893_291.567113_0.000000_0.100000': [310.4151945539619, 0, 0, 0, 0, 0, 0, 0], '0.001000_77.615165_1.652893_291.567113_0.000000_0.100000': [783.4062521158721, 0, 0, -129.1657481068096, 0, 0, 0, 0], '0.001000_77.615165_1.818182_291.567113_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000826_85.376682_1.241843_291.567113_0.000000_0.100000': [0, 0, 0, 0, 0.14050728422615977, 0, 0, 0], '0.000751_85.376682_1.241843_291.567113_0.000000_0.100000': [1396.0863860773222, 0, -0.012773389475105437, 0, 0, 0, 0, 0], '0.000751_77.615165_1.241843_320.723824_0.000000_0.100000': [2028.253360678303, 0, 0, 0, 0, 0, 0, 0], '0.000751_70.559241_1.241843_320.723824_0.000000_0.100000': [31.40934420984372, 0, 0, 0, -5.987125310531474, 0, 0, 0], '0.000683_70.559241_1.241843_320.723824_0.000000_0.100000': [0, 0, 5.987125310531474, 0, 0, 0, 0, 0], '0.000751_64.144765_1.241843_320.723824_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001000_64.144765_1.241843_320.723824_0.000000_0.100000': [-0.010499194421640823, -0.00064404908312099, 0, 0, 0, 0, 0, 0], '0.001000_58.313423_1.241843_320.723824_0.000000_0.100000': [-0.7354833584402968, -0.69844954868895, 0, 0, 0, 0, 0, 0], '0.001000_53.012202_1.241843_320.723824_0.000000_0.100000': [0.7239628582443175, 0, 0, 0, 0, 0, 0, 0], '0.001000_48.192911_1.241843_320.723824_0.000000_0.100000': [0, 0, 0, 0, 0, 0.010720239380157181, 0, 0], '0.001000_48.192911_1.128948_320.723824_0.000000_0.100000': [0, 0, 0, 0, 0.019804586267741797, 0, 0, 0], '0.000909_48.192911_1.128948_320.723824_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001000_93.914350_2.000000_320.723824_0.000000_0.100000': [0, 0, 0, 0, 3.7099887818875685, 0, 0, 0], '0.000909_93.914350_2.000000_320.723824_0.000000_0.100000': [778.4144762704553, 0, 0, 0, 0, 0, 0, 0], '0.000909_85.376682_2.000000_320.723824_0.000000_0.100000': [2302.332915431869, 0, 0, 0, 0, 0, 0, 0], '0.000909_77.615165_2.000000_320.723824_0.000000_0.100000': [2488.204294818659, 0, 0, 0, 0, -334.9005078891308, 0, 0], '0.000909_77.615165_1.818182_320.723824_0.000000_0.100000': [0, 0, 0, 334.9005078891308, 0, 0, 0, 0], '0.000909_70.559241_2.000000_320.723824_0.000000_0.100000': [26.93525296812609, 0, 0, 0, -1206.375909176099, 0, 0, 0], '0.000826_70.559241_2.000000_320.723824_0.000000_0.100000': [0, 0, 1206.375909176099, 0, 0, 0, 0, 0], '0.001100_113.636364_1.652893_320.723824_0.000000_0.100000': [0, 0, 0, 0, 1.1157436223624246, 0, 0, 0], '0.001000_113.636364_1.652893_320.723824_0.000000_0.100000': [0, 0, 0, 0, 0, 287.09254206348925, 0, 0], '0.001000_113.636364_1.502630_320.723824_0.000000_0.100000': [0, 0, 0, 0, 498.4551035185696, 0, 0, 0], '0.000909_113.636364_1.241843_320.723824_0.000000_0.100000': [0, 0, 0, -31.495295431199064, 0, 0, 0, 0], '0.000826_113.636364_1.128948_320.723824_0.000000_0.100000': [0, 265.59872659665183, 0, 0, 0, 0, 0, 0], '0.000826_113.636364_1.128948_291.567113_0.000000_0.100000': [0, 0, 0, 0, 0, 92.85812110767311, 0, 0], '0.000826_113.636364_1.026316_291.567113_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000909_125.000000_1.818182_320.723824_0.000000_0.100000': [400.740146743127, 0, 0, 0, 0, 0, 0, 0], '0.000909_113.636364_1.818182_320.723824_0.000000_0.100000': [0, 0, 423.54989664890746, 0, 0, 0, 0, 0], '0.001000_70.559241_1.652893_291.567113_0.000000_0.100000': [0, 0, 2.7809882081050548, 0, 0, 0, 0, 0], '0.001100_70.559241_1.652893_291.567113_0.000000_0.100000': [0, 0, 0, 1218.0538156508246, 0, 0, 0, 0], '0.001100_64.144765_1.818182_291.567113_0.000000_0.100000': [0, 0, 0.019176404689787888, 0, 0, 0, 0, 0], '0.001210_64.144765_1.818182_291.567113_0.000000_0.100000': [0, 0.016079042222600037, 0, 0, 0, 0, 0, 0], '0.001210_64.144765_1.818182_265.061011_0.000000_0.100000': [0, 0, 0, 0, -0.7427006985823255, 0, 0, 0], '0.001100_64.144765_1.818182_265.061011_0.000000_0.100000': [0, 0.09607415962143477, 0, 0, 0, 0, 0, 0], '0.001100_64.144765_1.818182_240.964556_0.000000_0.100000': [0, 0, 0, 0.6500007166105206, 0, 0, 0, 0], '0.001100_64.144765_2.000000_240.964556_0.000000_0.100000': [0.013627970768970954, 0, 0, 0, 0, 0, 0, 0], '0.001100_58.313423_2.000000_240.964556_0.000000_0.100000': [0, 0, 0, 0, 0, -0.010069158609803707, 0, 0], '0.001100_58.313423_1.818182_240.964556_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000909_113.636364_2.000000_831.875000_0.000000_0.100000': [0, 0, 0, 0, 275.8749511744864, 0, 0, 0], '0.000826_103.305785_2.200000_831.875000_0.000000_0.100000': [0, 0, 507.49034648940335, 0, 0, 0, 0, 0], '0.000751_77.615165_2.000000_756.250000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001210_70.559241_1.366027_320.723824_0.000000_0.100000': [0, 0, 0, 801.6154858903294, 0, 0, 0, 0], '0.001100_58.313423_1.502630_265.061011_0.000000_0.100000': [0, 0, 0, 0, 0, -0.004590758277111036, 0, 0], '0.001100_58.313423_1.366027_265.061011_0.000000_0.100000': [0, 0, 0, 0, 0.011172526730050777, 0, 0, 0], '0.001000_58.313423_1.366027_265.061011_0.000000_0.100000': [-0.6781447874980586, 0, 0, -0.7450144835648417, 0, -0.0012011520656187713, 0, 0], '0.001000_58.313423_1.502630_265.061011_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001000_93.914350_1.128948_568.181818_0.000000_0.100000': [0, 0, 0, 1702.8203506665893, 0, 0, 0, 0], '0.001000_53.012202_1.128948_426.883410_0.000000_0.100000': [0, 0.01091022877433261, 0, 0, 0, 0, 0, 0], '0.001000_53.012202_1.128948_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001100_113.636364_1.366027_320.723824_0.000000_0.100000': [0, 278.2443696334398, 0, 0, 0, 0, 0, 0], '0.001100_113.636364_1.366027_291.567113_0.000000_0.100000': [318.18225211119926, 0, 0, 0, 0, 0, 0, 0], '0.001100_103.305785_1.366027_291.567113_0.000000_0.100000': [0, 249.88589346699916, 0, 0, 0, 0, 0, 0], '0.001100_103.305785_1.366027_265.061011_0.000000_0.100000': [0, 0, 0, 0, 1.7633039960252563, 0, 0, 0], '0.001000_103.305785_1.366027_265.061011_0.000000_0.100000': [335.99482756208084, 0, 0, -238.90406946381947, 0, 0, 0, 0], '0.001000_103.305785_1.502630_265.061011_0.000000_0.100000': [0, 276.7577763414304, 0, 0, 0, 0, 0, 0], '0.001000_103.305785_1.502630_240.964556_0.000000_0.100000': [0, 0, 0, 0, 2.190864192209483, 0, 0, 0], '0.000909_103.305785_1.502630_240.964556_0.000000_0.100000': [0, 0, 0, -472.4886494353326, 0, 0, 0, 0], '0.000909_103.305785_1.652893_240.964556_0.000000_0.100000': [0, 0, 0, 0, 1.982775875220068, 0, 0, 0], '0.000826_103.305785_1.652893_240.964556_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -317.46632150326985, 0], '0.000826_113.636364_1.652893_240.964556_0.000000_0.100000': [0, 0, 0, -94.29287347497066, 0, 0, 0, 0], '0.000826_113.636364_1.818182_240.964556_0.000000_0.100000': [0, 0, 0, 0, 2.331713449592553, 0, 0, 0], '0.000751_113.636364_1.818182_240.964556_0.000000_0.100000': [0, 335.3540917957972, 0, 0, 0, 0, 0, 0], '0.000751_113.636364_1.818182_219.058687_0.000000_0.100000': [0, 0, -2.296548878770045, 0, 0, 0, 0, 0], '0.000826_113.636364_1.818182_219.058687_0.000000_0.100000': [0, 321.1391203745302, 0, 0, 0, 0, 0, 0], '0.000826_113.636364_1.818182_199.144261_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001000_85.376682_2.200000_320.723824_0.000000_0.100000': [0, 0, 0, 0, 4.244470729274144, 595.833454760787, 0, 0], '0.000909_85.376682_2.200000_320.723824_0.000000_0.100000': [0, 0, 0, -1814.0512415028918, 0, 0, 0, 0], '0.000909_85.376682_2.420000_320.723824_0.000000_0.100000': [301.9675491068054, 0, 0, -324.4630352356142, 0, 0, 0, 0], '0.000909_85.376682_2.662000_320.723824_0.000000_0.100000': [0, 0, 0, 0, 0, 324.4630352356142, 0, 0], '0.000909_77.615165_2.420000_320.723824_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -560.7090360835834], '0.000909_77.615165_2.420000_352.796206_0.000000_0.100000': [0, 0, 0, 0, 0, 1202.7564028558033, 0, 0], '0.000826_58.313423_2.200000_320.723824_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001210_58.313423_2.000000_388.075827_0.000000_0.100000': [-0.7310736549350623, 0, -0.038413803099024334, 0, 0, 0, 0, 0], '0.001331_58.313423_2.000000_388.075827_0.000000_0.100000': [0, -0.6919325102002859, 0, 0, 0, 0, 0, 0], '0.001331_58.313423_2.000000_352.796206_0.000000_0.100000': [-0.002369287286683175, -0.034523982151426935, 0, 0, 0, 0, 0, 0], '0.001331_53.012202_2.000000_352.796206_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0.002369287286683175, 0], '0.001331_58.313423_2.000000_320.723824_0.000000_0.100000': [0, 0.08421267801315935, 0, 0, 0, 0, 0, 0], '0.001331_58.313423_2.000000_291.567113_0.000000_0.100000': [0, 0, 0, 0.7540659916264804, 0, 0, 0, 0], '0.001331_58.313423_2.200000_291.567113_0.000000_0.100000': [0.041502841749752405, 0, 0, 0, 0, -0.06855145378422545, 0, 0], '0.001331_53.012202_2.200000_291.567113_0.000000_0.100000': [0, -0.8060330942857036, 0, 0, 0, 0, 0, 0], '0.001331_53.012202_2.200000_265.061011_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001210_48.192911_2.000000_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, -0.6443093854656354, 0, 0], '0.001210_48.192911_1.818182_388.075827_0.000000_0.100000': [-0.11263670238097348, 0, 0, 0, 0, 0, 0, 0], '0.001210_43.811737_1.818182_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -0.09639359240486556], '0.001210_43.811737_1.818182_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0.04578083873918786, 0, 0, 0], '0.001100_43.811737_1.818182_426.883410_0.000000_0.100000': [0, 0, 0, 0.019831642593817377, 0, 0, 0, 0], '0.001100_43.811737_2.000000_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001210_70.559241_1.818182_568.181818_0.000000_0.100000': [0, 0, 3019.2277136038756, 0, 0, 0, 0, 0], '0.001331_70.559241_1.818182_568.181818_0.000000_0.100000': [63.382168252302975, 0, 0, 0, -274.4752466912612, 0, 0, 0], '0.001331_64.144765_1.818182_568.181818_0.000000_0.100000': [-0.6793607793160163, 0, -0.7249817600305051, 0, 0, 0, 0, 0], '0.001464_64.144765_1.818182_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0.7249817600305051, 0, 0, 0], '0.001331_58.313423_1.818182_568.181818_0.000000_0.100000': [0, 0, 0, 0, 0, -0.06231449871543191, 0, 0], '0.001331_58.313423_1.652893_568.181818_0.000000_0.100000': [0, 0, 0, 0, -0.068105985730449, 0, 0, 0], '0.001210_64.144765_1.652893_516.528926_0.000000_0.100000': [0, -0.0004013431382509175, 0, 0, 0, 0, 0, 0], '0.001210_58.313423_1.652893_352.796206_0.000000_0.100000': [0, 0, 0.04680853877687241, 0, 0, 0, 0, 0], '0.001331_58.313423_1.652893_352.796206_0.000000_0.100000': [0, 0, 0, 0.724552924903378, 0, 0, 0, 0], '0.001331_58.313423_1.818182_352.796206_0.000000_0.100000': [0, 0, 0, 0, -0.7998644024903797, 0, 0, 0], '0.001210_58.313423_1.818182_352.796206_0.000000_0.100000': [0, 0, 0.7998644024903797, 0, 0, 0, 0, 0], '0.001210_93.914350_1.241843_568.181818_0.000000_0.100000': [0, 0, 0, 0, 1.6203833645240593, 0, 0, 0], '0.001000_53.012202_2.000000_426.883410_0.000000_0.100000': [0, 0, 0, 0, -0.7521437014253024, 0, 0, 0], '0.000909_53.012202_2.000000_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0.752401038392884, 0, 0, 0], '0.000826_53.012202_2.000000_352.796206_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0.005508760530233303], '0.000909_53.012202_2.000000_388.075827_0.000000_0.100000': [0, -0.7513428803564012, 0, 0, 0, 0, 0, 0], '0.000909_53.012202_2.000000_352.796206_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001000_85.376682_2.000000_320.723824_0.000000_0.100000': [0, 0, 0, 0, 221.0818963567178, 0, 0, 0], '0.000909_64.144765_2.000000_320.723824_0.000000_0.100000': [0, 0, 0.6485577359999752, 0, 0, 0, 0, 0], '0.001000_58.313423_2.000000_320.723824_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0.0065022624128005194, 0], '0.001100_53.012202_2.000000_426.883410_0.000000_0.100000': [0, 0, -0.0047538774017084505, 0, 0, 0, 0, 0], '0.001210_53.012202_2.000000_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, 0.0488636301636916, 0, 0], '0.001210_53.012202_1.818182_426.883410_0.000000_0.100000': [0.7024709494552553, 0, 0, 0, 0, 0, 0, 0], '0.001210_48.192911_1.818182_426.883410_0.000000_0.100000': [0, 0, 0, -0.7271099379303451, 0, 0, 0, 0], '0.001210_48.192911_2.000000_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0.7469592747326033, 0, 0, 0], '0.001100_48.192911_2.000000_426.883410_0.000000_0.100000': [0, 0.02247986664247037, 0, 0, 0, 0, 0, 0], '0.001100_48.192911_2.000000_388.075827_0.000000_0.100000': [0, -0.8171484305581771, 0, 0, 0, 0, 0, 0], '0.001100_48.192911_2.000000_352.796206_0.000000_0.100000': [0, 0, 0, 0, 0, 0.8088953599385604, 0, 0], '0.001100_48.192911_1.818182_352.796206_0.000000_0.100000': [-0.824359746983896, 0, 0, 0, 0, 0, 0, 0], '0.001100_43.811737_1.818182_352.796206_0.000000_0.100000': [0, 0.8466112817244493, 0, 0, 0, 0, 0, 0], '0.001100_43.811737_1.818182_320.723824_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001000_53.012202_1.366027_320.723824_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0.014674750891244911, 0], '0.001100_58.313423_1.366027_320.723824_0.000000_0.100000': [-0.6909416431198679, 0, 0, -0.7590622844041945, 0, 0, 0, 0], '0.001100_58.313423_1.502630_320.723824_0.000000_0.100000': [0, 0.7704064991226688, 0, 0, 0, 0, 0, 0], '0.001100_53.012202_1.502630_291.567113_0.000000_0.100000': [0, 0.03704744038182071, 0, 0, 0, 0, 0, 0], '0.001100_53.012202_1.502630_265.061011_0.000000_0.100000': [0, 0, 0.7697531783697287, 0, 0, 0, 0, 0], '0.001210_53.012202_1.502630_265.061011_0.000000_0.100000': [0.002419266842025536, 0, 0, 0, 0, 0, 0, 0], '0.001210_48.192911_1.502630_265.061011_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001100_48.192911_1.652893_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -0.01324174998592298, 0], '0.001000_48.192911_1.502630_291.567113_0.000000_0.100000': [-0.7664815878238642, -0.02679900134944546, 0, 0, 0, 0, 0, 0], '0.001000_43.811737_1.502630_291.567113_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001100_64.144765_1.502630_291.567113_0.000000_0.100000': [0.02396549179971013, -0.677929184055406, 0, -0.7748942391669517, 0, 0, 0, -0.00025301085258888634], '0.001100_64.144765_1.652893_291.567113_0.000000_0.100000': [0, 0, 0, 0, 0.7731975161177616, 0, 0, 0], '0.001000_64.144765_1.652893_291.567113_0.000000_0.100000': [0, 0, 0, -0.7557199426978286, 0, 0, 0, 0], '0.001000_64.144765_1.818182_291.567113_0.000000_0.100000': [0, 0, 0.7465569921833457, 0, 0, 0, 0, 0], '0.000564_58.313423_1.652893_320.723824_0.000000_0.100000': [-0.042973002972030376, -0.030593379528500008, 0, 0, 0, 0, 0, 0], '0.000564_53.012202_1.652893_320.723824_0.000000_0.100000': [0, 0, 0, 0, -0.7294238228472069, 0, 0, 0], '0.000513_53.012202_1.652893_320.723824_0.000000_0.100000': [0, 0.010786850060776487, 0, 0, 0, 0, 0, 0], '0.000513_53.012202_1.652893_291.567113_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000909_64.144765_1.652893_756.250000_0.000000_0.100000': [-0.7991259194664508, -0.10176586629223947, 0, 0, 0, 0, 0, 0], '0.000909_58.313423_1.652893_756.250000_0.000000_0.100000': [0, 0, 0, 0, 0.03127860909262381, 0, 0, 0], '0.000826_58.313423_1.652893_756.250000_0.000000_0.100000': [-0.029867912718623303, 0, 0, 0, 0, 0, 0, 0], '0.000826_53.012202_1.652893_756.250000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0.18085071563555175], '0.000826_53.012202_1.652893_831.875000_0.000000_0.100000': [0, 0, 0, 0.5578238833465321, 0, 0, 0, 0], '0.000826_53.012202_1.818182_831.875000_0.000000_0.100000': [0, 0, 0.020924492097151415, 0, 0, 0, 0, 0], '0.000909_53.012202_1.818182_831.875000_0.000000_0.100000': [0, 0, -0.00964164608540159, 0, 0, 0, 0, 0], '0.001000_53.012202_1.818182_831.875000_0.000000_0.100000': [0, -0.7851812652568697, 0, 0, 0, 0, 0, 0], '0.001000_53.012202_1.818182_756.250000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000683_53.012202_2.000000_388.075827_0.000000_0.100000': [0, -0.04017251719815249, 0, 0, 0, 0, 0, 0], '0.000683_53.012202_2.000000_352.796206_0.000000_0.100000': [0, 0, 0.7900409163552468, 0, 0, 0, 0, 0], '0.000751_53.012202_2.000000_352.796206_0.000000_0.100000': [0, 0.019508513894902536, 0, 0, 0, 0, 0, 0], '0.000751_53.012202_2.000000_320.723824_0.000000_0.100000': [0, 0, 0, 0, 0, -0.7076275478897287, 0, 0], '0.000751_53.012202_1.818182_320.723824_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001210_85.376682_1.502630_352.796206_0.000000_0.100000': [0, 0, 0, 0, 0, 428.27649263700215, 0, 0], '0.001210_64.144765_1.652893_320.723824_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000751_93.914350_1.366027_291.567113_0.000000_0.100000': [0, 0, 0, 0, 0, -298.1298781108457, 0, 0], '0.000751_93.914350_1.241843_291.567113_0.000000_0.100000': [0, 0, 0, 0, 0.5779761998409413, 0, 0, 0], '0.000683_93.914350_1.241843_291.567113_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -174.68438486462665, 0], '0.000683_103.305785_1.241843_291.567113_0.000000_0.100000': [0, 0, 0, 0, 0.7826004613013539, 0, 0, 0], '0.000621_103.305785_1.241843_291.567113_0.000000_0.100000': [191.87932809455341, 0, 0, 0, 0, 0, 0, 0], '0.001000_93.914350_1.366027_265.061011_0.000000_0.100000': [458.4691264027246, 0, 0, 0, 0, 0, 0, 0], '0.001000_85.376682_1.366027_265.061011_0.000000_0.100000': [0, 0, 11.082638743020153, 0, 0, 0, 0, 0], '0.001100_85.376682_1.366027_265.061011_0.000000_0.100000': [0, 0, 0, -330.2251300682377, 0, 0, 0, 0], '0.001100_85.376682_1.502630_265.061011_0.000000_0.100000': [1390.1129527125768, 0, 0, -634.3073732160198, 0, 0, 0, 0], '0.001100_85.376682_1.652893_265.061011_0.000000_0.100000': [0, 0, 0, 0, 0, 634.3073732160198, 0, 0], '0.001000_85.376682_1.502630_265.061011_0.000000_0.100000': [0, 28.451953384778275, 0, 0, 0, 0, 0, 0], '0.001000_53.012202_1.366027_265.061011_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001000_53.012202_1.818182_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -0.15119288983167964], '0.001000_48.192911_2.000000_687.500000_0.000000_0.100000': [-0.11521547014669978, 0, 0, 0, 0, 0, 0, 0], '0.001000_43.811737_2.000000_687.500000_0.000000_0.100000': [0.22907494923779267, 0, 0, 0, 0, 0, 0, 0], '0.001000_39.828852_2.000000_687.500000_0.000000_0.100000': [0.653629437097984, 0, 0, 0, 0, 0, 0, 0], '0.001000_36.208047_2.000000_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -0.059420857917998525, 0], '0.000826_93.914350_1.366027_291.567113_0.000000_0.100000': [316.3610122889778, 0, 0, 0, 0, 0, 0, 0], '0.001000_85.376682_1.366027_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 1466.363039882053], '0.001100_125.000000_1.502630_516.528926_0.000000_0.100000': [0, 812.0153560044849, 0, 0, 0, 0, 0, 0], '0.001210_58.313423_1.652893_291.567113_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000826_58.313423_1.241843_352.796206_0.000000_0.100000': [0, 0, -0.7220547545431373, 0, 0, 0, 0, 0], '0.000909_58.313423_1.241843_352.796206_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0.7598382303290009], '0.000909_58.313423_1.241843_388.075827_0.000000_0.100000': [-0.7763563166057565, 0, 0, 0, 0, 0, 0, 0], '0.000909_53.012202_1.241843_388.075827_0.000000_0.100000': [0, 0, 0.7606907584829287, 0, 0, 0, 0, 0], '0.001000_53.012202_1.241843_388.075827_0.000000_0.100000': [0, 0, 0, -0.015398818941500848, 0, 0, 0, 0], '0.000909_85.376682_1.652893_265.061011_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -291.02763069540407, 0], '0.000909_93.914350_1.652893_265.061011_0.000000_0.100000': [0, 319.6897775093516, 0, 0, 0, 0, 0, 0], '0.000909_93.914350_1.652893_240.964556_0.000000_0.100000': [876.7720629735378, 0, 0, 0, 0, 0, 0, 0], '0.000909_85.376682_1.652893_240.964556_0.000000_0.100000': [0, 0, 0, 0, 0, 10.36679528742816, 0, 0], '0.000909_85.376682_1.502630_240.964556_0.000000_0.100000': [115.34822287997868, 0, 0, 0, 0, 0, 0, 0], '0.000909_77.615165_1.502630_240.964556_0.000000_0.100000': [758.6082400116948, 0, 0, 0, 0, 0, 0, 0], '0.000909_70.559241_1.502630_240.964556_0.000000_0.100000': [0, 21.36533921986893, 0, 0, 0, 0, 0, 0], '0.000909_70.559241_1.502630_219.058687_0.000000_0.100000': [0, 45.609730503034505, 0, 0, 0, 0, 0, 0], '0.000909_70.559241_1.502630_199.144261_0.000000_0.100000': [0, 0, 0, 0, -0.8256218899892247, 0, 0, 0], '0.000826_70.559241_1.502630_199.144261_0.000000_0.100000': [0, 0, 0.8256218899892247, 0, 0, 0, 0, 0], '0.000826_64.144765_1.241843_320.723824_0.000000_0.100000': [0, -0.7395031410544107, 0, 0, 0, 0, 0, 0], '0.000826_64.144765_1.241843_291.567113_0.000000_0.100000': [0, 0, 0, 0.7445921536063679, 0, 0, 0, 0], '0.000826_58.313423_1.366027_291.567113_0.000000_0.100000': [0.00203702577043714, 0, 0, 0, 0, 0, 0, 0], '0.000826_53.012202_1.366027_291.567113_0.000000_0.100000': [0, 0, 0, 0.007508476844486723, 0, 0, 0, 0], '0.000826_53.012202_1.502630_291.567113_0.000000_0.100000': [0, 0, 0, 0, 0.01152806819854959, 0, 0, 0], '0.000751_53.012202_1.502630_291.567113_0.000000_0.100000': [0, 0, 0, 0, 0, -0.7609467930573601, 0, 0], '0.000751_53.012202_1.366027_291.567113_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000826_103.305785_1.241843_352.796206_0.000000_0.100000': [0, 0, 0, 684.7850272760464, 0, 0, 0, 0], '0.001000_70.559241_1.502630_388.075827_0.000000_0.100000': [36.91740491999038, 0, 0, 0, 0, 0, 0, 0], '0.000909_58.313423_1.652893_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000683_113.636364_2.000000_352.796206_0.000000_0.100000': [0, 394.7387773224844, 0, 0, 0, 0, 0, 0], '0.000683_113.636364_2.000000_320.723824_0.000000_0.100000': [0, 0, 0, 0, 3.1862122053939856, 0, 0, 0], '0.000621_113.636364_2.000000_320.723824_0.000000_0.100000': [0, 0, 0, 0, 0, 396.13669224945033, 0, 0], '0.000621_113.636364_1.818182_320.723824_0.000000_0.100000': [0, 0, 0, 0, 0, 123.542364649701, 0, 0], '0.000621_113.636364_1.652893_320.723824_0.000000_0.100000': [0, 0, 0, 0, 0, 299.1227426724778, 0, 0], '0.000621_113.636364_1.502630_320.723824_0.000000_0.100000': [0, 0, 0, 0, 0, 543.1306976050561, 0, 0], '0.000621_113.636364_1.366027_320.723824_0.000000_0.100000': [0, 0, 0, 0, 0, 29.992931645543287, 0, 0], '0.000621_113.636364_1.241843_320.723824_0.000000_0.100000': [267.96930364840637, 0, 0, 0, 0, 0, 0, 0], '0.000621_103.305785_1.241843_320.723824_0.000000_0.100000': [0, 260.163257428623, 0, 0, 0, 0, 0, 0], '0.000621_93.914350_1.241843_291.567113_0.000000_0.100000': [111.85220717527086, 0, 0, 0, 0, 0, -17.44357528132303, 0], '0.000621_85.376682_1.241843_291.567113_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -239.14696144109527], '0.000621_85.376682_1.241843_320.723824_0.000000_0.100000': [82.46098398800832, 0, 0, 0, 0, 0, -117.93019787297926, 0], '0.000621_93.914350_1.241843_320.723824_0.000000_0.100000': [117.93019787297926, 0, 0, 0, 0, 0, 0, 0], '0.000621_77.615165_1.241843_320.723824_0.000000_0.100000': [2020.361157412025, 0, 0, 0, 0, 0, 0, 0], '0.000751_103.305785_1.366027_320.723824_0.000000_0.100000': [0, 0, 0, 0, 1.2763050564190963, 0, 0, 0], '0.000683_103.305785_1.366027_320.723824_0.000000_0.100000': [273.6201944221302, 0, 0, 0, 0, 0, 0, 0], '0.000683_93.914350_1.366027_320.723824_0.000000_0.100000': [521.470426658401, 0, 0, -357.2164361079231, 0, 0, 0, 0], '0.000683_93.914350_1.502630_320.723824_0.000000_0.100000': [0, 0, 0, 0, 0, 357.2164361079231, 0, 0], '0.000683_85.376682_1.366027_320.723824_0.000000_0.100000': [0, 0, 0, 0, 1.5316703615178948, 0, 0, 0], '0.000621_85.376682_1.366027_320.723824_0.000000_0.100000': [179.38342074159215, 0, -0.1392427601379905, 0, 0, 0, 0, 0], '0.000621_77.615165_1.366027_320.723824_0.000000_0.100000': [0, 0, 0, 0, 0, -623.4647263008616, 0, 0], '0.000621_70.559241_1.241843_320.723824_0.000000_0.100000': [0, 0, 0, 0, -376.9574064097606, 0, 0, 0], '0.000564_70.559241_1.241843_320.723824_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000751_77.615165_1.366027_320.723824_0.000000_0.100000': [0, 0, 0, 0, 0.25050131136090387, 0, 0, 0], '0.000683_77.615165_1.366027_320.723824_0.000000_0.100000': [705.0041553941096, 0, 0, 0, 0, -622.8688206153939, 0, 0], '0.000683_77.615165_1.241843_320.723824_0.000000_0.100000': [0, 0, 0, 622.8688206153939, 0, 0, 0, 0], '0.000683_70.559241_1.366027_320.723824_0.000000_0.100000': [0, 0, 0, 717.5589247648246, 0, 0, 0, 0], '0.000683_70.559241_1.502630_320.723824_0.000000_0.100000': [0, -292.7694773138533, 0, 0, 0, 0, 0, 0], '0.000683_70.559241_1.502630_291.567113_0.000000_0.100000': [0, 0, 304.0733605103839, 0, 0, 0, 0, 0], '0.001100_93.914350_1.366027_352.796206_0.000000_0.100000': [0, 0, 0, 0, 1.9897855940786031, 0, 0, 0], '0.000826_64.144765_1.128948_320.723824_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001100_58.313423_1.652893_291.567113_0.000000_0.100000': [0, 0, 0, 0, -0.1445559530912533, 0, 0, 0], '0.001000_58.313423_1.652893_291.567113_0.000000_0.100000': [0, 0, 0, 0, 0, 0.8840144848578908, 0, 0], '0.001000_48.192911_1.502630_265.061011_0.000000_0.100000': [0, 0, 0, 0, -0.7602857585615084, 0, 0, 0], '0.000909_48.192911_1.502630_265.061011_0.000000_0.100000': [0, 0.7678865280756554, 0, 0, 0, 0, 0, 0], '0.000909_48.192911_1.502630_240.964556_0.000000_0.100000': [0, 0, 0, 0.012281699202321762, 0, 0, 0, 0], '0.000909_48.192911_1.652893_240.964556_0.000000_0.100000': [0, 0, 0, 0, -0.04368232793447078, 0, 0, 0], '0.000826_48.192911_1.652893_240.964556_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001210_58.313423_1.502630_240.964556_0.000000_0.100000': [0, 0, -0.04835452805773457, 0, 0, 0, 0, 0], '0.001331_58.313423_1.502630_240.964556_0.000000_0.100000': [0.05970525287434612, 0, 0, 0, 0, 0, 0, 0], '0.001331_53.012202_1.502630_240.964556_0.000000_0.100000': [0, 0, 0, 0, 0.620506585991825, 0, 0, 0.6985305776719998], '0.001210_53.012202_1.502630_240.964556_0.000000_0.100000': [0, 0, -0.620506585991825, 0, 0, 0, 0, 0], '0.001331_53.012202_1.502630_265.061011_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000826_64.144765_1.652893_469.571751_0.000000_0.100000': [0, 0, -0.01496186065691268, 0, 0, 0, 0, 0], '0.001000_53.012202_1.502630_426.883410_0.000000_0.100000': [0, 0, -0.022045739737268377, 0, 0, 0, 0, 0], '0.001100_53.012202_1.502630_426.883410_0.000000_0.100000': [0, 0, 0, 0, 0.022045739737268377, 0, 0, 0], '0.000621_93.914350_2.000000_469.571751_0.000000_0.100000': [0, 0, 0, 0, 3.140399361137406, 0, 0, 0], '0.000564_93.914350_2.000000_469.571751_0.000000_0.100000': [0, 502.76147772068646, 0, 0, 0, 0, 0, 0], '0.000564_93.914350_2.000000_426.883410_0.000000_0.100000': [0, 465.1721188455231, 0, 0, 0, 0, 0, 0], '0.000564_93.914350_2.000000_388.075827_0.000000_0.100000': [1038.0010235043433, 0, 0, 0, 0, 0, 0, 0], '0.000564_85.376682_2.000000_388.075827_0.000000_0.100000': [206.149905492133, 0, 0, 0, 0, -490.84897323077496, 0, 0], '0.000564_85.376682_1.818182_388.075827_0.000000_0.100000': [0, 0, 0, 490.84897323077496, 0, 0, 0, 0], '0.000564_77.615165_2.000000_388.075827_0.000000_0.100000': [0, 0, -2.013735091352919, 0, 0, 0, 0, 0], '0.000621_77.615165_2.000000_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, -481.3320004503425, 0, 0], '0.000621_77.615165_1.818182_388.075827_0.000000_0.100000': [0, 0, 0, 0, 1.0110441551722944, 0, 0, 0], '0.000564_77.615165_1.818182_388.075827_0.000000_0.100000': [0, 0, 0, 0, 0, 2149.39862903687, 0, 0], '0.000564_58.313423_1.652893_291.567113_0.000000_0.100000': [0, 0, 0, 0, 0.017234344726580277, 0, 0, 0], '0.000513_58.313423_1.652893_291.567113_0.000000_0.100000': [0, 0, 0, 0, 0, -0.017583956072098773, 0, 0], '0.000513_58.313423_1.502630_291.567113_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000751_70.559241_1.502630_320.723824_0.000000_0.100000': [0, 0, 0, -49.007488029980486, 0, 0, 0, 0], '0.000751_70.559241_1.652893_320.723824_0.000000_0.100000': [73.56871266343943, 0, 0, 0, 0, 0, 0, 0], '0.000751_64.144765_1.652893_320.723824_0.000000_0.100000': [0, 0, 0, 0, 0, -0.7676762770480082, 0, 0], '0.000751_64.144765_1.502630_320.723824_0.000000_0.100000': [0, 0.8137349739954971, 0, 0, 0, 0, 0, 0], '0.000751_64.144765_1.502630_265.061011_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000909_64.144765_1.652893_516.528926_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -3022.7246430267323, 0], '0.001100_64.144765_1.502630_320.723824_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001100_58.313423_1.366027_291.567113_0.000000_0.100000': [-0.002733879141783291, -0.006851994042484875, -0.03458294829365514, -4.7739235775001726e-06, -0.02154974254478702, -0.8277473737540504, -0.009902583943039644, -0.01611813829597395], '0.001100_53.012202_1.366027_291.567113_0.000000_0.100000': [0, 0, 0, 0, 0, -0.017317946163761918, 0, 0], '0.001100_48.192911_1.366027_265.061011_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001000_58.313423_1.241843_265.061011_0.000000_0.100000': [0, 0, 0, 0, 0, -0.011629003068560451, 0, 0], '0.001000_58.313423_1.128948_265.061011_0.000000_0.100000': [0, 0.011877012320223913, 0, 0, 0, 0, 0, 0], '0.001000_58.313423_1.128948_240.964556_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001210_58.313423_1.366027_291.567113_0.000000_0.100000': [0, 0, 0, 0, 0.03458294829365514, 0, 0, 0], '0.001100_103.305785_2.000000_291.567113_0.000000_0.100000': [2823.163940047865, 0, 0, 0, 0, 0, 0, 0], '0.001000_64.144765_2.200000_265.061011_0.000000_0.100000': [0, 0, 0, 0, 0, 0.12575222069579015, 0, 0], '0.001000_64.144765_2.000000_265.061011_0.000000_0.100000': [0, 0, 0, 0, 0.6844912544544514, 0, 0, 0], '0.000909_53.012202_2.000000_240.964556_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000909_58.313423_1.366027_320.723824_0.000000_0.100000': [0, 0, 0, 0, -0.003370661407893749, 0, 0, 0], '0.000826_58.313423_1.366027_320.723824_0.000000_0.100000': [0, 0, 0, 0, 0.0037253317574823088, 0, 0, 0], '0.000751_58.313423_1.366027_320.723824_0.000000_0.100000': [-0.0076035028292380956, 0, -0.0003386665234074826, 0, 0, 0, 0, 0], '0.000751_53.012202_1.366027_320.723824_0.000000_0.100000': [0, 0, 0, -0.734952902759278, 0, 0, 0, 0], '0.001000_85.376682_1.818182_352.796206_0.000000_0.100000': [527.4655906521556, 0, 0, 0, 0, 0, 0, 0], '0.001000_64.144765_2.200000_320.723824_0.000000_0.100000': [0, 0, 0, 0, -0.004722855450518182, 0, 0, 0], '0.001210_58.313423_1.502630_352.796206_0.000000_0.100000': [0, 0, 0, 0, 0, 0.02546216915595778, 0, 0], '0.001210_58.313423_1.366027_352.796206_0.000000_0.100000': [0, -0.7616566458333182, 0, 0, 0, 0, 0, 0], '0.001210_58.313423_1.366027_320.723824_0.000000_0.100000': [0, 0, 0.08323975077416434, 0, 0, 0, 0, 0], '0.001331_58.313423_1.366027_320.723824_0.000000_0.100000': [-0.07605743777749012, 0, 0, 0, -0.00756725007037858, 0, 0, 0], '0.001331_53.012202_1.366027_320.723824_0.000000_0.100000': [0, 0, 0, 0.7779297418715715, 0, 0, 0, 0], '0.001331_53.012202_1.502630_320.723824_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001100_58.313423_1.241843_291.567113_0.000000_0.100000': [0, 0, 0.00582976840464653, 0, 0, 0, 0, 0], '0.001210_58.313423_1.241843_291.567113_0.000000_0.100000': [0.8063782986369006, 0, 0, 0, 0, 0, 0, 0], '0.001210_53.012202_1.241843_291.567113_0.000000_0.100000': [0, -0.6638863958148704, 0, 0, 0, 0, 0, 0], '0.001210_53.012202_1.241843_265.061011_0.000000_0.100000': [0, 0.6677002354073466, 0, 0, 0, 0, 0, 0], '0.001210_53.012202_1.241843_240.964556_0.000000_0.100000': [0, 0, 0, 0.0048533775927580795, 0, 0, 0, 0], '0.001210_53.012202_1.366027_240.964556_0.000000_0.100000': [0, 0, 0, -0.07619833089354415, 0, 0, 0, 0], '0.001000_64.144765_1.366027_219.058687_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001100_64.144765_1.366027_291.567113_0.000000_0.100000': [0, 0, 0, 0, 0, -0.0002582462187774581, 0, 0], '0.001000_64.144765_1.241843_291.567113_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -34.95551412293278, 0], '0.001100_53.012202_1.366027_320.723824_0.000000_0.100000': [0, 0, 0, 0, 0.6956678939683661, 0, 0, 0], '0.001000_58.313423_1.241843_291.567113_0.000000_0.100000': [0.6199742079751127, 0, 0, 0, 0, 0, 0, 0], '0.000751_64.144765_1.241843_352.796206_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0.7840253291081035], '0.000751_58.313423_1.366027_352.796206_0.000000_0.100000': [-0.7180741919639104, 0, 0, 0, 0, 0, 0, 0], '0.000751_53.012202_1.366027_352.796206_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.001100_58.313423_1.652893_469.571751_0.000000_0.100000': [0, 0, -0.02766750081407281, 0, 0, 0, 0, 0], '0.001210_58.313423_1.652893_469.571751_0.000000_0.100000': [0, 0, 0, -0.7223813619690418, 0, 0, 0, 0], '0.001210_53.012202_1.818182_469.571751_0.000000_0.100000': [-0.7264748421195236, 0, 0, 0, 0, 0, 0, 0], '0.001210_48.192911_1.818182_469.571751_0.000000_0.100000': [0, 0, 0, 0.1595431799313019, 0, 0, 0, 0], '0.001210_48.192911_2.000000_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, -0.22672946267662156, 0], '0.001210_53.012202_2.000000_469.571751_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0], '0.000683_70.559241_1.502630_625.000000_0.000000_0.100000': [0, 0, -0.9620276339722977, 0, 0, 0, 0, 0], '0.000751_70.559241_1.502630_625.000000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, -4040.8694408828737], '0.000751_70.559241_1.502630_687.500000_0.000000_0.100000': [4078.516673698424, 0, 0, 0, 0, 0, 0, 0], '0.000751_64.144765_1.502630_687.500000_0.000000_0.100000': [0.00454829067132323, 0, 0, 0, 0, 0, 0, 0], '0.000751_58.313423_1.502630_687.500000_0.000000_0.100000': [-0.0025458958022454682, 0, 0, 0, 0, 0, 0, 0], '0.000751_53.012202_1.502630_687.500000_0.000000_0.100000': [0, 0, 0, -0.9099598118533095, 0, 0, 0, 0], '0.000751_53.012202_1.652893_687.500000_0.000000_0.100000': [0.2862332730657727, 0, 0, 0, 0, 0, 0, 0], '0.000751_48.192911_1.652893_687.500000_0.000000_0.100000': [0, 0, 0.007980758109692943, 0, 0, 0, 0, 0], '0.000826_48.192911_1.652893_687.500000_0.000000_0.100000': [0, 0, 0, -0.34512367604161226, 0, 0, 0, 0], '0.000826_48.192911_1.818182_687.500000_0.000000_0.100000': [0, 0, 0, 0, 0, 0, 0, 0]}


In [7]:
# total_reward=[]
alpha = 0.9
gamma = 1
n_episodes = 1000
for i in range(1,1000):
        epsilon = i/n_episodes
        print(i)
        env.reset()
        total_reward.append(SARSA(env, alpha, gamma, epsilon, Q, 30))
        if i%20 == 0:
            file = open("backup.txt", "w")
            str_dictionary = repr({"i" : i, "total_reward" : total_reward, "Q" : Q})
            file.write("a_dictionary = " + str_dictionary + "\n")
            file.close()

1
Policy: Random
State:  0.001000_125.000000_2.000000_568.181818_0.000000_0.100000
Policy: Random
Action:  7
Reward:  -595.0563919188453
-----
State:  0.001000_125.000000_2.000000_625.000000_0.000000_0.100000
Policy: Random
Action:  7
Reward:  -612.2998031793813
-----
State:  0.001000_125.000000_2.000000_687.500000_0.000000_0.100000
Policy: Random
Action:  5
Reward:  1027.1884745258922
-----
State:  0.001000_125.000000_1.818182_687.500000_0.000000_0.100000
Policy: Random
Action:  5
Reward:  242.14219189544656
-----
State:  0.001000_125.000000_1.652893_687.500000_0.000000_0.100000
Policy: Random
Action:  2
Reward:  -3.398758529954648
-----
State:  0.001100_125.000000_1.652893_687.500000_0.000000_0.100000
Policy: Random
Action:  0
Reward:  352.1727185660029
-----
State:  0.001100_113.636364_1.652893_687.500000_0.000000_0.100000
Policy: Random
Action:  3
Reward:  -254.30429043366712
-----
State:  0.001100_113.636364_1.818182_687.500000_0.000000_0.100000
Policy: Random
Action:  7
Reward:  

State:  0.001000_137.500000_2.000000_625.000000_0.000000_0.100000
Policy: Random
Action:  7
Reward:  -584.4811176110125
-----
State:  0.001000_137.500000_2.000000_687.500000_0.000000_0.100000
Policy: Random
Action:  4
Reward:  6.243159428226136
-----
State:  0.000909_137.500000_2.000000_687.500000_0.000000_0.100000
Policy: Random
Action:  4
Reward:  5.676242588024252
-----
State:  0.000826_137.500000_2.000000_687.500000_0.000000_0.100000
Policy: Random
Action:  4
Reward:  5.160738996555665
-----
State:  0.000751_137.500000_2.000000_687.500000_0.000000_0.100000
Policy: Random
Action:  3
Reward:  803.9087481931183
-----
State:  0.000751_137.500000_2.200000_687.500000_0.000000_0.100000
Policy: Random
Action:  6
Reward:  -299.45724526139566
-----
State:  0.000751_151.250000_2.200000_687.500000_0.000000_0.100000
Policy: Random
Action:  0
Reward:  299.45724526139566
-----
State:  0.000751_137.500000_2.200000_687.500000_0.000000_0.100000
Policy: Random
Action:  1
Reward:  275.64443880448744
-

Reward:  383.6147296543395
-----
State:  0.001000_125.000000_1.818182_625.000000_0.000000_0.100000
Policy: Random
Action:  7
Reward:  -547.1073772128639
-----
State:  0.001000_125.000000_1.818182_687.500000_0.000000_0.100000
Policy: Random
Action:  4
Reward:  4.124748614092823
-----
State:  0.000909_125.000000_1.818182_687.500000_0.000000_0.100000
Policy: Random
Action:  5
Reward:  241.1827869439585
-----
State:  0.000909_125.000000_1.652893_687.500000_0.000000_0.100000
Policy: Random
Action:  1
Reward:  555.5957869879085
-----
State:  0.000909_125.000000_1.652893_625.000000_0.000000_0.100000
Policy: Random
Action:  5
Reward:  -62.63612584240764
-----
State:  0.000909_125.000000_1.502630_625.000000_0.000000_0.100000
Policy: Random
Action:  7
Reward:  -435.79244148541784
-----
State:  0.000909_125.000000_1.502630_687.500000_0.000000_0.100000
Policy: Random
Action:  5
Reward:  766.8489519100449
-----
State:  0.000909_125.000000_1.366027_687.500000_0.000000_0.100000
Policy: Random
Action:

Reward:  6.302465100387053
-----
State:  0.000909_125.000000_2.000000_625.000000_0.000000_0.100000
Policy: Random
Action:  1
Reward:  595.0629327387269
-----
State:  0.000909_125.000000_2.000000_568.181818_0.000000_0.100000
Policy: Random
Action:  1
Reward:  574.4858819489627
-----
State:  0.000909_125.000000_2.000000_516.528926_0.000000_0.100000
Policy: Random
Action:  2
Reward:  -6.28160283914076
-----
State:  0.001000_125.000000_2.000000_516.528926_0.000000_0.100000
Policy: Random
Action:  2
Reward:  -6.912314559293918
-----
State:  0.001100_125.000000_2.000000_516.528926_0.000000_0.100000
Policy: Random
Action:  4
Reward:  6.912314559293918
-----
State:  0.001000_125.000000_2.000000_516.528926_0.000000_0.100000
Policy: Random
Action:  3
Reward:  148.91982694744001
-----
State:  0.001000_125.000000_2.200000_516.528926_0.000000_0.100000
Policy: Random
Action:  0
Reward:  -1089.0913610149628
-----
State:  0.001000_113.636364_2.200000_516.528926_0.000000_0.100000
Policy: Random
Action:

Reward:  -1373.2257564114116
-----
State:  0.001100_113.636364_2.200000_625.000000_0.000000_0.100000
Policy: Random
Action:  0
Reward:  387.9981891931038
-----
State:  0.001100_103.305785_2.200000_625.000000_0.000000_0.100000
Policy: Random
Action:  4
Reward:  6.064737166172563
-----
State:  0.001000_103.305785_2.200000_625.000000_0.000000_0.100000
Policy: Random
Action:  6
Reward:  -388.0414573300477
-----
State:  0.001000_113.636364_2.200000_625.000000_0.000000_0.100000
Policy: Random
Action:  6
Reward:  -407.43836926236327
-----
State:  0.001000_125.000000_2.200000_625.000000_0.000000_0.100000
Policy: Random
Action:  6
Reward:  1398.049149811688
-----
State:  0.001000_137.500000_2.200000_625.000000_0.000000_0.100000
Policy: Random
Action:  3
Reward:  160.11053220502072
-----
State:  0.001000_137.500000_2.420000_625.000000_0.000000_0.100000
Policy: Random
Action:  4
Reward:  2.33823279405442
-----
State:  0.000909_137.500000_2.420000_625.000000_0.000000_0.100000
Policy: Random
Action

...

Reward:  265.9294256928797
-----
State:  0.001210_125.000000_1.652893_568.181818_0.000000_0.100000
Policy: Greedy
Q[s]:  [307.0881802730412, 861.3990954553152, -3.5245692688594317, 0, 251.79738616348274, -137.27922865599834, 58.61575833091274, -529.2840923452243]
Action:  1
Reward:  500.11570967011994
-----
State:  0.001210_125.000000_1.652893_516.528926_0.000000_0.100000
Policy: Greedy
Q[s]:  [540.5037659643518, 0, -2.7687790035269244, -249.8407790133837, 0, -143.09706834674702, -314.9964574753609, 0]
Action:  0
Reward:  324.42824523769923
-----
State:  0.001210_113.636364_1.652893_516.528926_0.000000_0.100000
Policy: Greedy
Q[s]:  [0, 1294.3788827001551, -3.138746677888175, 634.1682768743617, 0, 0, -291.98542071392933, 0]
Action:  1
Reward:  495.41326789922914
-----
State:  0.001210_113.636364_1.652893_469.571751_0.000000_0.100000
Policy: Random
Action:  3
Reward:  -237.87379405582942
-----
State:  0.001210_113.636364_1.818182_469.571751_0.000000_0.100000
Policy: Random
Action:  3
Re

Reward:  1921.6435060921533
-----
State:  0.001000_70.559241_1.502630_352.796206_0.000000_0.100000
Policy: Random
Action:  1
Reward:  -179.71390538183144
-----
State:  0.001000_70.559241_1.502630_320.723824_0.000000_0.100000
Policy: Random
Action:  1
Reward:  -87.9309793270219
-----
State:  0.001000_70.559241_1.502630_291.567113_0.000000_0.100000
Policy: Random
Action:  1
Reward:  17.904285691434097
-----
State:  0.001000_70.559241_1.502630_265.061011_0.000000_0.100000
Policy: Random
Action:  7
Reward:  -17.904285691434097
-----
State:  0.001000_70.559241_1.502630_291.567113_0.000000_0.100000
Policy: Random
Action:  0
Reward:  -748.5455022190898
-----
State:  0.001000_64.144765_1.502630_291.567113_0.000000_0.100000
Policy: Random
Action:  5
Reward:  1075.826693112266
-----
State:  0.001000_64.144765_1.366027_291.567113_0.000000_0.100000
Policy: Random
Action:  2
Reward:  0.006057322629089867
-----
State:  0.001100_64.144765_1.366027_291.567113_0.000000_0.100000
Policy: Random
Action:  

Reward:  883.7017176840836
-----
State:  0.001210_113.636364_1.818182_625.000000_0.000000_0.100000
Policy: Greedy
Q[s]:  [306.4902257574603, -148.20228711376603, 0, -795.3315459156753, 381.5696808684069, 0, 0, 442.52189035683097]
Action:  7
Reward:  -555.4396052528518
-----
State:  0.001210_113.636364_1.818182_687.500000_0.000000_0.100000
Policy: Greedy
Q[s]:  [0, 0, 445.8995347169084, 0, 322.61246406134455, 0, 0, 0]
Action:  2
Reward:  -5.027309568728015
-----
State:  0.001331_113.636364_1.818182_687.500000_0.000000_0.100000
Policy: Greedy
Q[s]:  [0, 809.302648677244, 0, 0, 8.576333867055398, 229.6911176012285, 0, 0]
Action:  1
Reward:  556.0791522577329
-----
State:  0.001331_113.636364_1.818182_625.000000_0.000000_0.100000
Policy: Random
Action:  6
Reward:  -349.7329077416889
-----
State:  0.001331_125.000000_1.818182_625.000000_0.000000_0.100000
Policy: Random
Action:  6
Reward:  -385.2086027252417
-----
State:  0.001331_137.500000_1.818182_625.000000_0.000000_0.100000
Policy: Rand

Reward:  1389.689999625731
-----
State:  0.001331_70.559241_2.000000_352.796206_0.000000_0.100000
Policy: Greedy
Q[s]:  [0, 0, -210.68552132990706, 0, 0, 0, 0, 0]
Action:  0
Reward:  1592.230801958269
-----
State:  0.001331_64.144765_2.000000_352.796206_0.000000_0.100000
Policy: Random
Action:  4
Reward:  -0.0529565824751046
-----
State:  0.001210_64.144765_2.000000_352.796206_0.000000_0.100000
Policy: Random
Action:  4
Reward:  -0.01286866067948722
-----
State:  0.001100_64.144765_2.000000_352.796206_0.000000_0.100000
Policy: Random
Action:  7
Reward:  0.023171158872514752
-----
State:  0.001100_64.144765_2.000000_388.075827_0.000000_0.100000
Policy: Random
Action:  3
Reward:  0.011754524027139723
-----
State:  0.001100_64.144765_2.200000_388.075827_0.000000_0.100000
Policy: Random
Action:  7
Reward:  0.009435012858631353
-----
State:  0.001100_64.144765_2.200000_426.883410_0.000000_0.100000
Policy: Random
Action:  1
Reward:  -0.009435012858631353
-----
State:  0.001100_64.144765_2.20

RuntimeError: <PyCall.jlwrap (in a Julia function called from Python)
JULIA: DimensionMismatch("second dimension of A, 47, does not match length of x, 46")
Stacktrace:
 [1] gemv!(::Array{Float64,1}, ::Char, ::Array{Float64,2}, ::Array{Float64,1}, ::Bool, ::Bool) at /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.3/LinearAlgebra/src/matmul.jl:451
 [2] mul! at /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.3/LinearAlgebra/src/matmul.jl:66 [inlined]
 [3] mul! at /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.3/LinearAlgebra/src/matmul.jl:203 [inlined]
 [4] * at /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.3/LinearAlgebra/src/matmul.jl:47 [inlined]
 [5] Dopt(::LinearAlgebra.Adjoint{Float64,Array{Float64,2}}, ::Array{Float64,2}, ::Array{Float64,2}, ::Float64, ::Float64, ::Array{Float64,1}, ::Array{Float64,1}) at /Users/julianotalvaro/.julia/packages/npod/1P3PM/src/npod.jl

In [ ]:
plt.plot(total_reward)

In [ ]:
# Q['0.000564_93.914350_1.366027_516.528926_0.000000_0.100000']


In [ ]:
# np.argmax(Q['0.000909_103.305785_1.652893_625.000000_0.000000_0.100000'])

In [ ]:
file = open("reward.txt", "w")
file.write(repr(total_reward))
file.close()

In [ ]:
count(0, [0,1])

In [ ]:
201%100

In [ ]:
reward = list(filter(lambda x: x>=-17000 and x<=-1000, total_reward))
plt.plot(reward)

In [ ]:
x= -19000
x>=-17000 and x<=-1000